# Settings

In [ ]:
import numpy as np
import sys
import pandas as pd
import itertools
#from pandas_profiling import ProfileReport

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.svar_model import SVAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import zivot_andrews
from statsmodels.regression.linear_model import OLS

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

import tigramite
import tigramite.data_processing as pp
import tigramite.plotting as tp
from tigramite.plotting import plot_time_series_graph

from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr
from tigramite.independence_tests.robust_parcorr import RobustParCorr

from tigramite.models import LinearMediation, Models
from tigramite.causal_effects import CausalEffects

import os

import pkg_resources

import cdsapi
import random
import zipfile

import netCDF4 as nc
import xarray as xr

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.subplots as sp
import plotly.subplots as sp

import seaborn as sns

from arch.unitroot import PhillipsPerron
from scipy import stats
from scipy.stats import gaussian_kde
from scipy.stats import f

import networkx as nx

from scipy.stats import pearsonr

import dagviz
from collections import Counter

from tqdm import tqdm

import nbformat

import pingouin as pg

import chow_test 
from chow_test import chow_test

# VAR Model

This section will contain the functions to generate a random VAR model based on _equation_ (TILLIE: EQUATIONNUMMER) from the thesis. It will be based on either (1) a given $\Phi$ matrix, or (2) a random $\Phi$ matrix that will be generated via a special function. For the given $\Phi$ we might want to retrieve the information from a val_matrix (given in the output of Tigramite). Therefore, a special function is defined as well. 

$Y_t = \Phi Y_{t-1} + \epsilon_t$

**Here we assume a VAR(1) model to be used all the time. The functions have therefore been simplified to only include VAR(1) models, not VAR(p).**

## Phi matrix

We can generate the phi matrix randomly or based on a certain val_matrix. In both cases we will need to check stability to ensure the process is stationary and Gaussian. This is needed for further inference. Therefore, the companion form matrix is analysed. For a VAR(1) model, this companion form matrix is simply equal to the normal phi matrix.

In [ ]:
def CompanionFormMatrix(phi):
    """
    Construct the Companion Form matrix of an input phi and compute the eigenvalues to check stability.
    Parameters:
        phi = adjacency matrix in a (S)(C)VAR model
    """
    # Check the eigenvalues
    eigenvalues = np.linalg.eigvals(phi)
    abs_eigenvalues = np.abs(eigenvalues)
    score = 0
    for i in range(0,len(abs_eigenvalues)):
        if abs_eigenvalues[i] > 1:
                score += 1
    # Check stability
    if score == 0:
        stability = True
    else:
        stability = False
        
    return stability, abs_eigenvalues

Then define two functions; one for the random phi and one for the non-random phi. The random phi will introduce the parameter 'sparseness' which describes the 

In [ ]:
def Random_phi(n, s, beta=0.5, gamma=0.01, seed=None):
    """
    Generate a random phi based on the approach of Kalisch and Bühlmann.
    Made for a VAR(1) model (p=1)
    Parameters:
        n = dimension of the data Y_t, number of variables
        s = sparseness where 0 < s < 1, which decides the expected number of neighbors
        beta = diversion of 0 for the off-diagonal entries
        gamma = diversion of 1 for the diagonal entries (they are unaffected by s)
    Returns:
        Phi = the adjacency matrix for the VAR(1) model
        eigenvalues = corresponding to phi
    """
    if seed is not None:
        np.random.seed(seed)
    
    stable_system = False
    while not stable_system:
        Phi = np.random.binomial(1, s, (n, n)).astype(float)
        for i in range(n):
            for j in range(n):
                if i == j:
                    Phi[i,j] = np.random.uniform(1-gamma, 1+gamma)
                if (i is not j) and Phi[i,j] == 1: 
                    Phi[i,j] = np.random.uniform(-beta, beta) # Fill the off-diagonal entries with a 1 with a random number between -beta and beta    
                
        stable, eigenvalues = CompanionFormMatrix(Phi)
        if stable:
            stable_system = True
    
    return Phi, eigenvalues

In [ ]:
def Non_random_phi(val_matrix):
    """
    Construct a Phi Matrix for the pcmci option based on an input val_matrix.
    Parameters:
        val_matrix = the output val_matrix from the Tigramite package
    Returns:
        Phi = the adjacency matrix for the VAR(1) model
        eigenvalues = corresponding to phi
    """
    n, _, _ = val_matrix.shape # Or just fill in 4
    Phi = np.zeros((n,n))
    for i in range(0,n): 
        for j in range(0,n): 
            Phi[i,j] = val_matrix[j,i,1]
        
    stable, eigenvalues = CompanionFormMatrix(Phi)
    
    if not stable:
        print("Note that the generated phi is not stable")
              
    return Phi, eigenvalues

A last option for the user is to develop your own phi matrix. Make sure that it is constructed in such a way that it is equal to the dimensions of these functions.

## Functions

Generate the error terms.

In [ ]:
def Generate_Error(phi, T, standard=True, seed=None):
    """
    Generates an error term with the same dimensionality as Y_t (number of variables).
    Parameters:
        phi = the adjacency matrix of the model.
        T = the length of the time series wished to generate
        standard = TRUE returns standard Gaussian errors
    Returns:
        epsilon = the error terms stored in a nxT matrix
        sigma = the covariance matrix for the Normal distribution fo the error terms
    """
    if seed is not None:
        np.random.seed(seed)

    n, _ = phi.shape

    if standard:
        Sigma = np.eye(n)
    else:
        Sigma = np.diag(np.random.uniform(-1, 1, n))
    
    epsilon = np.random.multivariate_normal(np.zeros(n), Sigma, T).T
    
    return epsilon, Sigma

Generate the constant term (which is only applicable if incorporated in the model. In the standard model from equation (TILLIE VUL WEER EQUATION IN VAN HIIERBOVEN) there is no constant term, so the constants will be set to zero).

In [ ]:
def Generate_Constant(phi, T, constant=False, seed=None):
    """
    Generates an error term with the same dimensionality as Y_t (number of variables).
    Parameters:
        phi = the adjacency matrix of the model.
        T = the length of the time series wished to generate
        constant = FALSE returns only zeros, so that the constant does not influence the time series, TRUE returns random constants
    Returns:
        epsilon = the error terms stored in a nxT matrix
        sigma = the covariance matrix for the Normal distribution fo the error terms
    """
    if seed is not None:
        np.random.seed(seed)

    n, _ = phi.shape
    
    if constant:
        C = np.random.randn(n, T)
    else:
        C = np.zeros((n, T))
    
    return C

Generate the time series using the errors, constant and a phi matrix (either randomly generated, based on a val_matrix or chosen entirely).

In [ ]:
def Generate_Time_Series(T, phi, C, epsilon, lower_bound_Y1=30, upper_bound_Y1=40, seed=None):
    """
    Generates a time series raw dataset based on chosen specifics. 
    Parameters:
        phi = the adjacency matrix of the model.
        T = the length of the time series wished to generate
        constant = FALSE returns only zeros, so that the constant does not influence the time series, TRUE returns random constants
    Returns:
        epsilon = the error terms stored in a nxT matrix
        sigma = the covariance matrix for the Normal distribution fo the error terms
    """
    
    if seed is not None:
        np.random.seed(seed)

    n, _ = phi.shape
    Y = np.zeros((n, T))
    
    if isinstance(lower_bound_Y1, list):
        for i in range(n):
            Y[i, :1] = np.random.uniform(lower_bound_Y1[i], upper_bound_Y1[i], (1, 1))
    else:
        Y[:, :1] = np.random.uniform(lower_bound_Y1, upper_bound_Y1, (n, 1))

    for t in range(1, T):
        Y[:, t] = C[:, t - 1]
        Y[:, t] += phi @ Y[:, t - 1]
        Y[:, t] += epsilon[:, t - 1]

    return Y

Finally, the **VAR(1) model** can be generated with just one click using the following function.

In [ ]:
def VAR1(T, phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, seed=None):
    """
    Generates a VAR(1) model using the before defined functions.
    Parameters:
        T =  desirable length of time series
        phi = adjacency matrix for the model
    Returns:
        Y = dataset
    """
    if seed is not None:
        np.random.seed(seed)
        
    Epsilon, Sigma = Generate_Error(phi, T, standard=standard, seed=seed)
    C = Generate_Constant(phi, T, constant=constant, seed=seed)
    Y = Generate_Time_Series(T, phi, C, Epsilon, lower_bound_Y1=lower_bound_Y1, upper_bound_Y1=upper_bound_Y1, seed=seed)
    
    return Y

To be able to work with the dataset, define a function that generates a pandas DataFrame and a Tigramite DataFrame using the same indeces as Schultz in her paper.

In [ ]:
def GenerateDF(data, start_year=1950, yearly=False, freq='ME', column_names=['WPAC','CPAC','EPAC','TW']):
    """
    This function generates a dataframe that we can use easily. 
    It will transpose the dataset so that we have rows of years and columns of variables, instead of the other way around.
    Parameters:
        data = set of raw data
        yearly = TRUE will generate yearly indeces that end in 2015, FALSE will generate monthly indeces that start in January 1950
    """
    
    # Make it a df and transpose the dataset (needed for Tigramite)
    df = pd.DataFrame(data)
    df = df.T
    
    if yearly == True:
        rows = list(range(2015-len(df), 2015))
    # Generate the names for the columns and rows
    #rows = list(range(start_year, start_year + df.shape[0]))  <--- yearly
    else:
        if len(df) > 1000:  # Use numeric indices if the length is very large
            rows = list(range(len(df)))        
        else:
            rows = pd.date_range(start=f'1950-01', periods=df.shape[0], freq=freq)  
        
    # Assign the names to the columns and rows
    df.index = rows
    df.columns = column_names
    
    # We will also be using a Tigramite df so let's return that as well
    # Note that a Tigramite dataframe needs to be of size Txn 
    dfT = pp.DataFrame(df.values, var_names=df.columns.tolist(), datatime=df.index.tolist())

    return df, dfT

## Example

This subsection gives an example model to show how it works and what a randomly generated VAR(1) dataset could look like.

In [ ]:
random_phi, random_phi_eigenvalues = Random_phi(4, 0.7, beta=0.5, gamma=0.01, seed=611) # with n=4, s=0.7 and seed=612
random_VAR1 = VAR1(864, random_phi, standard=True, constant=False, lower_bound_Y1=[20, 20, 20, 60], upper_bound_Y1=[40, 40, 40, 80], seed=612)
df_random_VAR1, dfT_random_VAR1 = GenerateDF(random_VAR1)

In [ ]:
def PlotDF(df, model_name = ''):
    fig = px.line(df, x=df.index, y=df.columns, title=f'Time Series Plot of Variables - {model_name}')
    fig.show()
    fig = go.Figure()
    for column in df.columns:
        fig.add_trace(go.Histogram(x=df[column], name=column, opacity=0.75))
    fig.update_layout(
        barmode='overlay',
        title=f'Distribution of Variables - {model_name}',
        xaxis_title='Value',
        yaxis_title='Count',
        legend_title='Variable'
    )
    fig.show()

In [ ]:
PlotDF(df_random_VAR1, model_name = 'Random VAR(1) model data')

## Parameter Estimation

This subsection will focus on the estimation of parameters in VAR models. Generally, we use **Multivariate Least Squares** when dealing with VAR models. In Lütkepohl it is shown that this is equivalent to OLS estimation for each variable in $Y_t$ seperately. For a VAR model, define the equations as in $(3.2.1)$ in Lütkepohl. That means that $Y := (y_1, \ldots, y_T)$ with each $y_t = (y_{1t}, \ldots, y_{Kt})^T$ and $Z := (Z_0, \ldots, Z_{T-1})$ with each $Z_t = (1, y_t)^T$. Then the MLS estimator is defined as follows:

$\hat{\beta} = ((ZZ')^{-1} \otimes I_K)\textbf{y}$

Or equivalently (using derivation from postmultiplication):

$\hat{B} = YZ'(ZZ')^{-1}$

Here we defined $B:= (\nu, A_1)$ or, since we are working with only one lag and thus the $A$ matrices do not need to have a subscript, $B:= (\nu, A)$. $\nu$ is the constant term of the model (which in the simulation sets will mainly be set equal to zero).

Note that the $K$ will mainly be set to four ($K=4$) because of the four climate variables and simplicity in the simulation sets.

In [ ]:
def MLS(df):
    """
    Estimate the coefficient matrix for a VAR(1) model using MLS.
    Returns:
        B_hat = the estimated K x K coefficient matrix by the theory of Lütkepohl.
    """
    # First we need the values (not the dataframe itself)
    Y_transpose = df.values # this is now N by T
    T, K = Y_transpose.shape # this is now T by N (or K in the notation in Lutkepohl)
    
    # Then we can simply transpose the values to obtain the Y matrix (so now with the time on the columns and variables on the rows)
    Y_allvalues = Y_transpose.T
    Y = Y_allvalues[:,1:]
    
    # Construct Z by filling the first row with 1's and then filling the rest of the rows with the v
    Z = np.ones((K+1, T-1))
    for i in range(1, K):
        for j in range(T-1):
            Z[i,j] = Y_allvalues[i-1, j]
    
    # Now use the formula for the estimator B to construct the MLS estimate for the constant term and the Phi matrix.
    # We will do it in steps so that it is clear we are computing it right
    ZZ_T = np.dot(Z, Z.T)
    ZZ_T_inv = np.linalg.pinv(ZZ_T)
    YZ_T = np.dot(Y, Z.T)
    B_hat = np.dot(YZ_T, ZZ_T_inv)
    
    return B_hat

In [ ]:
print(random_phi)

In [ ]:
MLS_estimate = MLS(df_random_VAR1)
print(MLS_estimate[:, 1:])

# SVAR Model

In this section the Structural VAR Model is defined, according to _equation_ (TILLIE: EQUATIONNUMMER) from the thesis. 

$A Y_t = \Phi Y_{t-1} + \epsilon_t$

A structural model incorporates structural theories and relationships between variables by incorporating an upper triangular matrix with the structural parameters: A. Some differences with the VAR model are highlighted next:
1. The error terms are now not called _innovations_ but _structural shocks_. They are now mutually diagonal (which was not necessarily the case in VAR models, where the sigma may not be diagonal).
2. A is an upper triangular matrix with 1's along the diagonal. Since there is by definition no 0 on the diagonal, and the matrix is upper triangular, the matrix A is by definition invertible (_Lecture Notes 1: Matrix Algebra Part C: Pivoting and Matrix Decomposition. (2014). In University of Warwick, EC9A0 Maths for Economists (pp. 1–46)_).

It is also possible to take the identity matrix as $A$ ($ I_n = A$), which will simply mean that there is no structural relationship between the variables in the same time period. This comes back to a regular VAR model. Therefore, in case we expect the matrix $A$ to be the identity matrix, or when comparing it to a model where it is, then the model can be estimated by using MLS in the regular VAR model.

## A matrix

In [ ]:
def Random_A(n, lower_bound=0.5, upper_bound=1.5, seed=None):
    """
    Generate a random A based on the approach of Kalisch and Bühlmann.
    Made for a VAR(1) model (p=1)
    Parameters:
        n = dimension of the data Y_t, number of variables
        beta = diversion of 0 for the off-diagonal entries
        gamma = diversion of 1 for the diagonal entries (they are unaffected by s)
    Returns:
        Phi = the adjacency matrix for the VAR(1) model
        eigenvalues = corresponding to phi
    """
    if seed is not None:
        np.random.seed(seed)
    
    stable_system = False
    while not stable_system:
        
        Phi = np.triu(np.random.uniform(lower_bound, upper_bound, (n, n)))
        np.fill_diagonal(Phi, 1)
        
        stable, eigenvalues = CompanionFormMatrix(Phi)
        if stable:
            stable_system = True
    
    return Phi, eigenvalues

## Functions

Now to calculate the structural form of the VAR model, extend the function that generates time series in VAR with an **inverse of the A matrix** such that it satisfies the equations as in Bolla et al. (2023). 

In [ ]:
def Generate_Time_Series_SVAR(T, A, phi, C, epsilon, lower_bound_Y1=30, upper_bound_Y1=40, seed=None):
    """
    Generates a time series raw dataset based on chosen specifics, extended with the A matrix that is needed for the SVAR model. 
    Parameters:
        phi = the adjacency matrix of the model.
        T = the length of the time series wished to generate
        constant = FALSE returns only zeros, so that the constant does not influence the time series, TRUE returns random constants
    Returns:
        epsilon = the error terms stored in a nxT matrix
        sigma = the covariance matrix for the Normal distribution fo the error terms
    """
    
    if seed is not None:
        np.random.seed(seed)

    n, _ = phi.shape
    Y = np.zeros((n, T))
    if isinstance(lower_bound_Y1, list):
        for i in range(n):
            Y[i, :1] = np.random.uniform(lower_bound_Y1[i], upper_bound_Y1[i], (1, 1))
    else:
        Y[:, :1] = np.random.uniform(lower_bound_Y1, upper_bound_Y1, (n, 1))
    
    A_inv = np.linalg.inv(A)
    
    for t in range(1, T):
        Y[:, t] = A_inv @ C[:, t - 1]
        Y[:, t] += A_inv @ phi @ Y[:, t - 1]
        Y[:, t] += A_inv @ epsilon[:, t - 1]

    return Y

In [ ]:
def SVAR1(T, A, phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, seed=None):
    """
    Generates a VAR(1) model using the before defined functions.
    Parameters:
        T =  desirable length of time series
        A = structural form matrix
        phi = adjacency matrix for the model
    Returns:
        Y = dataset
    """
    if seed is not None:
        np.random.seed(seed)
    
    Epsilon, Sigma = Generate_Error(phi, T, standard=standard, seed=seed)
    C = Generate_Constant(phi, T, constant=constant, seed=seed)
    Y = Generate_Time_Series_SVAR(T, A, phi, C, Epsilon, lower_bound_Y1=lower_bound_Y1, upper_bound_Y1=upper_bound_Y1, seed=seed)
    
    return Y

## Example

This subsection gives an example of what an SVAR model could look like. One can either choose to generate an A matrix using the before defined function, or to input the identity matrix, since this also satisfies the requirements for an A matrix. However, it _is_ important to make sure that the errors are generated with a diagonal covariance matrix. Therefore, it is advised for an SVAR model to always set **standard=True**. 

In [ ]:
identity_4 = np.eye(4)
identity_SVAR1 = SVAR1(864, identity_4, random_phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, seed=612)
df_identity_SVAR1, dfT_identity_SVAR1 = GenerateDF(identity_SVAR1)

In [ ]:
PlotDF(df_identity_SVAR1, model_name = 'Random SVAR(1) model data with identity matrix (= VAR(1))')

In [ ]:
random_A, _ = Random_A(4, lower_bound=-0.01, upper_bound=0.01, seed=611)
random_SVAR1 = SVAR1(864, random_A, random_phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, seed=612)
df_random_SVAR1, dfT_random_SVAR1 = GenerateDF(random_SVAR1)

In [ ]:
PlotDF(df_random_SVAR1, model_name = 'Random SVAR(1) model data')

## Parameter Estimation

This subsection will focus on the estimation of parameters in SVAR models. Lütkepohl tells us that generally scientists may use _Maximum Likelihood methods_ when dealing with SVAR models. However, this becomes hard quickly due to the feedback of contemporaneous variables. Therefore Kotzé introduces the reduced form. By left-multiplying both sides of our SVAR(1) model equation by the inverse structural parameter matrix $A^-1$, we are left with the following model:

$Y_t = A^{-1} \Phi Y_{t-1} + A^{-1} \epsilon_t$

$Y_t = B Y_{t-1} + u_t$ where $B = A^{-1} \Phi$ and $u_t = A^{-1} \epsilon_t$ with $\Sigma_u = A^{-1}\Sigma_\epsilon(A^{-1})'$ 

First off apply OLS to find the estimate for the matrix $B$. Then to identify $A$ and $\Phi$ seperately, use identifying restrictions on $A$ to estimate the matrix. Once we know $A$, we can simply calculate $\Phi$ using the definition of $B$. Since we know about the upper-triangular structure of $A$, and so of the lower-triangular structure of $A^{-1}$, it is possible to impose the Cholesky decomposition to reflect the recursive nature. For this we will use the equation for $\Sigma_u$.

Because it is easier we will just use a package that fits the SVAR model to the input dataframe and compute the structural moving average matrices using ML methods with the package by statsmodels, which is based on the literature of Hamilton (1994).

**Just a reminder: when comparing to a model in which the $A$ matrix is the Identity matrix, or when it is assumed that the variables do not affect each other in the same time period, only temporal; then the model can be reduced back to a VAR model and the parameter matrix $\Phi$ can be estimated using MLS again.**

In [ ]:
def MLmethod(df, p=1, maxn=10):
    """
    Use the statsmodels package to fit a reduced VAR model to the data and then use those parameter estimates to compute the SVAR parameters.
    Parameters:
        maxn = the maximum number of avergaes 
    """
    var_model = sm.tsa.VAR(df)
    var_result = var_model.fit(maxlags=p)
    
    # Define the A matrix (we will define it as the identity matrix for the decomposition for now)
    N = df.shape[1]
    A = np.eye(N)

    # Use lower triangular Cholesky factor of the sigma_u to find B
    B = np.linalg.cholesky(var_result.sigma_u).T

    # Fit the SVAR model
    svar_model = SVAR(var_model.endog, svar_type='A', A=A, B=B)
    svar_result = svar_model.fit(maxlags=p)

    # Compute Structural MA coefficient matrices
    ma_matrices = svar_result.svar_ma_rep(maxn=maxn)

    return svar_result, ma_matrices

In [ ]:
#try_out_data = np.random.randn(100, 4)
#df = pd.DataFrame(try_out_data, columns=['Y', 'P', 'R', 'M'])

# Fit SVAR model and compute parameter matrices
#svar_result, ma_matrices = MLmethod(df_identity_SVAR1)

# Print the estimated parameters
#print("A matrix:\n", svar_result.A)
#print("B matrix:\n", svar_result.B)
#print("Estimated structural form residuals covariance matrix (Sigma_u):\n", svar_result.sigma_u)
#print("Structural MA coefficient matrices:\n", ma_matrices)

In [ ]:
import numpy as np
import pandas as pd


def MLmethod(df, p=1, maxn=10):
    """
    Use the statsmodels package to fit a reduced VAR model to the data and then use those parameter estimates to compute the SVAR parameters.
    Parameters:
        df: DataFrame containing the time series data.
        p: The number of lags to include in the VAR model.
        maxn: The maximum number of structural MA terms to compute.
    """
    var_model = sm.tsa.VAR(df)
    var_result = var_model.fit(maxlags=p)
    
    # Define the A matrix with some restrictions
    N = df.shape[1]
    A = np.eye(N)
    
    # Use lower triangular Cholesky factor of the sigma_u to find B
    B = np.linalg.cholesky(var_result.sigma_u).T
    # Ensure the B matrix is appropriately constrained (Cholesky decomposition already gives a lower triangular matrix)
    B_mask = np.tril(np.ones((N, N), dtype=bool)).T
    
    # Fit the SVAR model
    svar_model = SVAR(df, svar_type='AB', A=A, B=B)
    svar_model.A = A
    svar_model.A_mask = np.eye(N, dtype=bool)
    svar_model.B = B
    svar_model.B_mask = B_mask
    svar_result = svar_model.fit(maxlags=p, override=True)

    # Compute Structural MA coefficient matrices
    ma_matrices = svar_result.svar_ma_rep(maxn=maxn)

    return svar_result, ma_matrices

# Sample data
n_obs = 1000
n_vars = 4
data = np.random.randn(n_obs, n_vars)
df = pd.DataFrame(data, columns=['Y', 'P', 'R', 'M'])

# Fit SVAR model and compute parameter matrices
svar_result, ma_matrices = MLmethod(df)

# Print the estimated parameters
print("A matrix:\n", svar_result.A)
print("B matrix:\n", svar_result.B)

est_phi = svar_result.A @ svar_result.B
print(est_phi)

In [ ]:
PlotDF(df, model_name = '1111')

In [ ]:
s = SVAR1(864, svar_result.A, est_phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, seed=612)
df2, df2T = GenerateDF(s)
PlotDF(df2)

# CVAR Model

The last important model to identify is the Causal VAR Model. The CVAR(1) model is a novel model introduced by Bolla et al. (2023) and incorporates causality. It is based on the strucural form of VAR, but now orders the variables according to a causal ordering. 

$A Y_t = \Phi Y_{t-1} + \epsilon_t$

The matrix $A$ will now relate to a DAG where the causality is given in the order: $X_{t,i}$ can be caused by $X_{t,j}$ whenever $i < j$, which means there there is a directed edge in the DAG $i \rightarrow j$. So the causal effects are reflected by the upper triangular structure of matrix $A$ from the SVAR model. And the relationships are contemporaneous. 

Note that this is about the same time $t$; the model is also able to reflect intertemporal relations. These are reflected in the matrix $\Phi$. 

The difference between constructing a CVAR and an SVAR model, is that the variables should be **weakly stationary**. Therefore, the CVAR(1) function introduces the Phillips Perron test. If this test is not passed, a new SVAR will be generated using the next seed. That way, the CVAR will finally return a weakly stationary series, as required for the CVAR model.

Furthermore, it is advised for the data to be **jointly Gaussian** by definition (_Bolla et al. (2023)_). This is something that simply needs to be checked. To test the multivariate Normality of the dataset use the Henze-Zikler test from the pingouin package (_pingouin.multivariate_normality — pingouin 0.5.4 documentation. (n.d.)_). This is a powerful test. In _Schultz_ (2023) the univariate Shapiro-Wilk test was used. Although it is a univariate test, it should give a good indication of multivariate Normality if all variables are normal. However, applying the Henze-Zikler test to the climate data it becomes clear that the data is not _jointly_ Normal after all. 

## Functions

Not much needs to be changed from the CVAR(1) model. Include a while loop that checks the multivariate Normality of the dataset. Only if the data is multivariate normal, let it be returned. The function also returns the seed that is eventually used to produce the returned data. This might be necessary for reproduction.

In [ ]:
def CVAR1(T, A, phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, pp_alpha=0.05, seed=None):
    """
    Generates a CVAR(1) model using the before defined functions. Mainly based on the SVAR model, but adds the condition that the data is multivariate Normally distributed.
    Uses the Phillips-Perrontest to test for weak Stationarity.
    Parameters:
        T =  desirable length of time series
        A = structural form matrix
        phi = adjacency matrix for the model
        pp_alpha = significance level for Philips-Perron test
    Returns:
        Y = dataset
    """
    stationarity = False
    n, _ = phi.shape
    seed = seed - 1 if seed is not None else None
    while stationarity == False:
        if seed is not None:
            seed += 1        
        Y = SVAR1(T, A, phi, standard=standard, constant=constant, lower_bound_Y1=lower_bound_Y1, upper_bound_Y1=upper_bound_Y1, seed=seed) # returns N by T data
        df = pd.DataFrame(Y.T, columns=[f'Var{i+1}' for i in range(n)]) # DataFrame with T by N data
        stationarity = True
        for column in df.columns:
            pp_test_result = PhillipsPerron(df[column])
            p_value = pp_test_result.pvalue
            if p_value >= pp_alpha:
                stationarity = False
                break
                
    return Y, seed

## Example

This subsection will again provide example datasets, this time for the Causal VAR(1) model. The seed used is the same as the seeds before and is chosen because it generates multivariate Normal data according to the Henze Zirkler test.

In [ ]:
identity_CVAR1, seed_identity_CVAR1 = CVAR1(864, identity_4, random_phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, pp_alpha=0.05, seed=611)
df_identity_CVAR1, dfT_identity_CVAR1 = GenerateDF(identity_CVAR1)

In [ ]:
print(seed_identity_CVAR1)

In [ ]:
PlotDF(df_identity_CVAR1, model_name = f'Random CVAR(1) model data with identity matrix (= VAR(1)) with seed={seed_identity_CVAR1}')

In [ ]:
#random_CVAR1, seed_random_CVAR1 = CVAR1(864, random_A, random_phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, pp_alpha=0.05, seed=611)
#df_random_CVAR1, dfT_random_CVAR1 = GenerateDF(random_CVAR1)

In [ ]:
#print(seed_random_CVAR1)

In [ ]:
#PlotDF(df_random_CVAR1, model_name = f'Random CVAR(1) model data with seed={seed_random_CVAR1}')

## Parameter Estimation

This subsection will focus on the estimation of the parameters for a CVAR model. When trying to fit the data to a CVAR model, we cannot use the regular (M)LS or ML methods since we are dealing with causality in the parameters. A specialized approach is necessary; explained entirely in _Bolla et al._ (2023). For this we import the entire function as given on their GitHub. The function will fit the model data to a CVAR model according to their very own definitions. The parameters are estimated using Cholesky block decomposition, which relates to the ordering of the variables. Next, my own functions are defined that will make it easier to run the provided functions and give the option to plot the outcomes. 

The returned model dictionary contains the estimates for the $A$ and $\Phi$ matrices according to the CVAR(1) model. **Note** that you have to fill in $p=1$ yourself to make sure it runs correctly. The function called **'CVAR_ParEst'** will then return 'model, JT_dict'. The returned **model** (first return option) stores the matrices $A$ and $\Phi$ of the CVAR model. Access them as follows:
- $A$: model['A']
- $\Phi$: model['B']

These outputs are the estimated parameters of the model. They can potentially be compared to the (M)LS or ML estimates from earlier. The CVAR_ParEst function takes the following arguments: 
(df, p, start_var, restricted=True, plot_skeleton=False, plot_reordered=False, RZP_before_ordering=False, RZP_after_ordering=True, print_JT_dct=False, choose_own_order=False)

In [ ]:
def CVAR_fitting(data, p, ordering=None, JT=None):
  """
  Fits a Causal VAR(p) model and reports the related matrices in package as a 
  dict. Valid keys for the resulting dictionary include:
    'block covariance', 'partial correlations', 'K', 'K-original' (only exists
    when restricted == True), 'A', 'B', and 'Delta'
  For details, please refer to the paper.

  Parameters
  ----------
  data: array_like
    A multi-dimensional time series.
  p: int
    A positive integer indicating the order of the desired VAR model.
  ordering: list
    An ordered list specifying the desired variable ordering for the data
  JT: dict, default None
    A pseudo-graph object that describes the junction tree structure based on
    a decomposable graph generated with contemporaneous variables in the data.  
    
    Setting JT to a non-None value will enable covariance selection and lead to
    a restricted VAR model (zeros will be set to the corresponding positions in
    the based on the given junction tree).

    Example: 
    >> JT_Fin_VAR1={'cliques': 
                    [['BVSP', 'EM', 'NIK'], 
                      ['BVSP', 'DAX', 'EM', 'FTSE', 'ISE', 'SP'], 
                      ['BVSP', 'DAX', 'EU', 'FTSE', 'ISE']],
                  'separators': 
                    [['BVSP', 'EM'], 
                      ['BVSP', 'DAX', 'FTSE', 'ISE']]}

  Returns
  -------
  outputs: dict
    A collection of important matrices for the causal VAR model stored as
    {'matrix_name' : matrix} pairs.
  
  """
  if ordering is not None:
    data = data.reindex(columns=ordering)
  else:
    ordering = data.columns

  X = np.array(data)
  n, d = X.shape
  # For VAR(p), we need the block Toeplitz covariance matrix C_{p+1}
  CC = block_covariance(data, p+1, keep_label=True)
  K = pd.DataFrame(np.linalg.inv(CC), columns=CC.columns, index=CC.columns)
  cond_C_inv = K.iloc[0:d, 0:d]

  outputs = {
      'block covariance': CC,
      'partial correlations': partial_cor(cond_C_inv, keep_label=True),
    }

  if JT is not None:
    # conduct covariance selection and build a restricted CVAR model
    outputs['K-original']= K # store the original concentration matrix K
    K = cov_selection(data, p, JT, keep_label=True) 

  L, D = block_LDL(K, p, d)
  outputs['K'] = K
  outputs['A'] = pd.DataFrame(L[:d,:d].T, columns=ordering, index=ordering)
  outputs['B'] = pd.DataFrame(L[d:,:d].T, columns=CC.columns[d:], index=ordering)
  outputs['Delta'] = np.linalg.inv(D[:d,:d])  
  
  return outputs 



# In[ Section 1: functions for analyzing multi-variate time series data; ]
## primarily used to construct graphs for our casual VAR model along with the
## NetworkX package.
def covariance_func(data, h, keep_label=False):
  """
  Computes the ergodic estimate for C(h) (the covariance function for the 
  VAR model)
  
  Parameters
  ----------
  data: array_like
    A multi-dimensional time series.
  h: an integer
   An integer, can be negative.
  keep_label: bool, default False
    If (keep_label == True), return result as a pd.DataFrame with readable
    labelling (extra columns are labelled as 'X_{i}-p' indicating the 
    respective lag-p variables in the context of autoregression).  

  Returns
  -------
  C: np.array (default) or pd.DataFrame
    A d x d matrix (with the same variable labelling as data) that serves as the
    the ergodic estimate for C(h).
  """
  if keep_label and (type(data) is not pd.DataFrame):
    raise TypeError("data (first argument) has to be a pd.DataFrame"
    " in order to carry variable labels.")

  X = np.array(data) 
  n, d = X.shape 
  h_abs = abs(h) # in case when h is negative

  # compute Xbar, a row vector containing the sample means for each variable.
  Xbar = np.zeros((1,d)) 
  for i in range(d):
    Xbar[0, i] = np.mean(X[: , i])

  # construct C based
  C = np.zeros((d, d))
  for i in range(n - h_abs):
    Z0 = np.transpose(X[i, :] - Xbar) # column vector (temp)
    Z1 = X[i + h_abs, :] - Xbar # row vector (temp)
    C = C + np.matmul(Z0, Z1) # running sum of outer products

  C = np.true_divide(C, n) # divde every entry in C by the sample size n.

  # return C^T(h) if h is negative (see page 4, ver.0727_2021, above Eq(4))
  if h < 0:
    return C.T
  # otherwise, return C(h) (h is positive).
  return C


def block_covariance(data, k, keep_label=False):
  """
  Computes the block Toeplitz covairnce matrix C_k (intended for VAR(k-1)) using 
  the values of the covariance function C(h). 

  Parameters
  ----------
  data: array_like
    A multi-dimensional time series
  k: an integer
    A positive integer.
  keep_label: bool, default False
    If (keep_label == True), return result as a pd.DataFrame with readable
    labelling (extra columns are labelled as 'X_{i}-p' indicating the 
    respective lag-p variables in the context of autoregression).  

  Returns
  -------
  CC, np.array (default) or pd.DataFrame
    CC is a symmetric, positive definite block Toeplitz matrix with 
    p x p blocks which are d x d covariance matrices C(h)'s.

  Remarks
  -------
    CC is the concentration matrix K that should be used for the VAR(k-1) model. 
    
    To avoid off-by-one error use the concentration_mat() function below.
  """
  if keep_label and (type(data) is not pd.DataFrame):
    raise TypeError("data (first argument) has to be a pd.DataFrame"
    " in order to carry variable labels.")

  X = np.array(data)
  n, d = data.shape

  # construct the block Toeplitz covariance matrix row-by-row, block-by-block
  CC = []
  for i in range(k):
    row_block_vec = []
    for j in range(k):
      h = i - j
      row_block_vec.append( covariance_func(data, h) )
    CC.append(row_block_vec) # add the next row of blocks to CC
  
  CC = np.block(CC) # "unwrap" the blocks
  
  if keep_label:
    # extend the labels to fit the matrices for VAR(k-1)
    labels = extend_labels(data, k-1)
    return pd.DataFrame(CC, index = labels, columns = labels)
    
  return pd.DataFrame(CC)

      
  
def concentration_mat(data, p, keep_label=False):
  """
  Computes the concentration matrix K, inverse of the block Toeplitz co-
  variance matrix, for the VAR(p) model.
  
  Parameters
  ----------
  data: array_like
    A multi-dimensional time series
  p: an integer
    A positive integer indicating the order of the desired VAR model.
  keep_label: bool, default False
    If (keep_label == True), return result as a pd.DataFrame with readable
    labelling (extra columns are labelled as 'X_{i}-p' indicating the 
    respective lag-p variables in the context of autoregression).  

  Returns
  -------
  K: np.array (default) or pd.DataFrame
    K is the inverse of the symmetric, positive definite block Toeplitz 
    matrix with p x p blocks which are d x d covariance matrices C(h)'s.
    
    The upper left d x d block of K is the inverse of the conditional
    covariance matrix C(p|1,..., p-1).
  """
  CC = block_covariance(data, p+1, keep_label)

  if keep_label:
    return pd.DataFrame(np.linalg.inv(CC), index=CC.columns, columns=CC.columns)
  
  return np.linalg.inv(CC)


def inv_cond_cov(data, p, keep_label=False):
  """
  Computes the conditional covariance matrix C(p|1,...,p-1) using the K matrix 
  (i.e., inverse of the block covariance matrix) (maybe inefficient)
  
  Parameters
  ----------
  data: array_like
    A multi-dimensional time series
  p: an integer
    A positive integer indicating the order of the desired VAR model.
  keep_label: bool, default False
    If (keep_label == True), return result as a pd.DataFrame with readable
    labelling (extra columns are labelled as 'X_{i}-p' indicating the 
    respective lag-p variables in the context of autoregression).  

  Returns
  -------
  matrix: np.array (default) or pd.DataFrame
    The inverse of the conditional covariance matrix C(p|1,..., p-1) for the 
    time series data.
  """
  n,d = data.shape
  K = concentration_mat(data, p, keep_label)
  if keep_label:
    return K.iloc[:d, :d]
  return K[:d, :d]


def partial_cor(C_inv, keep_label=False):
  """
  Computes the partial correlation matrix based on the concentration matrix K.

  Parameters
  ----------
  C_inv: array_like
    A conentration matrix (inverse of a covariance matrix)
  p: an integer
    A positive integer indicating the order of the desired VAR model.
  keep_label: bool, default False
    If (keep_label == True), return result as a pd.DataFrame with readable
    labelling (extra columns are labelled as 'X_{i}-p' indicating the 
    respective lag-p variables in the context of autoregression).   

  Returns
  -------
  rr: np.array (default) or pd.DataFrame
    rr is a d x d matrix (same shape as K for the VAR model).

    rr[i][j] is the partial correlation coefficient between X_i and X_j (as 
    specified in K) given all other variables (ceteris paribus). As a result, 
    rr keeps the same varaible ordering as the corresponding covariance matrix
    (i.e., inverse of K). 
  """
  if keep_label and (type(C_inv) is not pd.DataFrame):
    raise TypeError("C_inv (first argument) has to be a pd.DataFrame"
    " in order to carry variable labels.")

  K = np.array(C_inv)
  _, d = K.shape 
  rr = np.zeros((d,d))

  for i in range(d):
      for j in range(d):
          if i == j:
              rr[i,j]= np.nan # diagonal entry: undefined
          else:
              rr[i,j] = np.true_divide(-K[i,j], np.sqrt(K[i,i]*K[j,j]))

  if keep_label:
    return pd.DataFrame(rr, index=C_inv.columns, columns=C_inv.columns)
  return rr

def has_RZP(adj_matrix):
  """
  Checks if an adjacency matrix has a reducible zero pattern.
  """
  M = np.array(adj_matrix)
  nrow, ncol = M.shape

  for j in range(nrow):
      for i in range(ncol):        
          if (j < i) and (M[j,i] == 0):
              for h in range(j):
                  if (M[h,j] == 1) and (M[h,i] == 1):
                      return False
  return True 


def extend_labels(data, p):
  """
  Returns an extened list of indices (from data) with lagged-varaibles up to
  order p.
  """
  assert type(data) is pd.DataFrame, "input data have to be a pd.DataFrame"
  labels = data.columns
  for i in range(1, p+1):
    labels = labels.append(pd.Index(map(lambda x: x + '-' + str(i),
                                        data.columns)))
  return labels

def max_card_order(G, start_node):
  """
  Returns an ordering (list of variable names) of graph G's vertices based on
  the max cardinality search algorithm according to the pseudocode in p.312 of 
    [11] Koller, D., Friedman, N., Probabilistic Graphical Models. Principles
    and Techniques. MIT Press (2009). 
  (with slight modificiation).
  """
  assert start_node in G, "start_node should be in G"
  
  N = len(G)
  order = [0] * (N-1) + [start_node]
  weight = {v: 0 for v in G.nodes}
  # setting weights of used nodes to -N ensures the priority of unused nodes. 
  weight[start_node] = -N


  for u in G.neighbors(start_node):
      weight[u] += 1

  for k in range(N-2, -1, -1):
    v = max(weight, key=weight.get)
    order[k] = v
    weight[v] = -N
    for u in G.neighbors(v):
      weight[u] += 1

  return order

def max_card_order_v1(G, start_node):
  """
  This is the same as the previous function but allows users to break ties
    manually.
  """
  assert start_node in G, "start_node should be in V(G)"
  
  N = len(G)
  order = ['?'] * (N-1) + [start_node]
  weight = {v: 0 for v in G.nodes}
  # setting weights of used nodes to -N ensures the priority of unused nodes. 
  weight[start_node] = -N

  for u in G.neighbors(start_node):
      weight[u] += 1

  for k in range(N-2, -1, -1):
    v = max(weight, key=weight.get)
    maxVal = weight[v]
    keys = [key for key, val in weight.items() if val == maxVal]
    if len(keys) != 1:
      print('Current ordering:', order)
      v = input("Break the tie between " + str(keys) + '\n  Enter: ')

    order[k] = v
    weight[v] = -N
    for u in G.neighbors(v):
      weight[u] += 1

  return order

# In[ Section 2: Covariance Selection ]
## to build a restricted model in one-click given a junction tree  
def lagged_sample(data, p, keep_label=False):
  """
  Generates a (n-p)-element (p+1)d-dimensional sample of all contemporary 
  and lagged variables based on the original d-dimensional sample. Intended for
  a vector autoregression (VAR) model of order p.
  
  Parameters
  ----------
  data: array_like
    A multi-dimensional time series.
  p: int
    A positive integer indicating the order of the desired VAR model.
  keep_label: bool, default False
    If (keep_label == True), return result as a pd.DataFrame with readable
    labelling (extra columns are labelled as 'X_{i}-p' indicating the 
    respective lag-p variables in the context of autoregression).  
    
  Returns
  -------
  lagged_data: np.array (default) or pd.DataFrame
    An  (n-p)-element (p+1)d-dimensional sample of all contemporary and lagged
    variables based on the original d-dimensional sample. 
  
  """
  if keep_label and (type(data) is not pd.DataFrame):
    raise TypeError("data (first argument) has to be a pd.DataFrame"
    " in order to carry variable labels.")

  X = np.array(data)
  n, d = data.shape
  
  block_vector = [X[(p-i) :(n-i), :] for i in range(p+1)]
  X_lagged= np.block(block_vector)

  if keep_label:
    labels = extend_labels(data, p)
    return pd.DataFrame(X_lagged, columns=labels)
  
  return X_lagged # default, return type: np.array


def blowed_S_inv(lagged_data, indices, keep_label=False):
  """
  blowed_S_inv will take a low-dimensional sample covariance matrix S and insert
  the inverse of S into the proper row/column of a larger (sparse) matrix with
  the same shape as K.

  Parameters
  ----------
  lagged_data: array_like
    An (n-p)-element (p+1)d-dimensional sample of all contemporary and lagged
    variables based on the original d-dimensional sample.
  indices: list of int
    A list of indices corresponding to the set of "valid" variables included in
    a clique/separator.
  keep_label: bool, default False
    If (keep_label == True), return result as a pd.DataFrame with readable
    labelling (extra columns are labelled as 'X_{i}-p' indicating the 
    respective lag-p variables in the context of autoregression).  

  Returns
  -------
  S_inv, np.array (default) or pd.DataFrame
    A pd x pd matrix with nonzero entries in the indices of S with zeros in
    indices not contained in S.
  
  """
  if keep_label and (type(lagged_data) is not pd.DataFrame):
      raise TypeError("data (first argument) has to be a pd.DataFrame"
      " in order to carry variable labels.")

  X_lagged = np.array(lagged_data)
  n, m = X_lagged.shape # m = (p+1) * d
  # S0 is the original dense sample covariance matrix 
  S0 = np.zeros((len(indices), len(indices)))
  for i0, i in enumerate(indices):
    for j0, j in enumerate(indices):
      Z1 = X_lagged[:, i] - np.mean(X_lagged[:, i])
      Z2 = X_lagged[:, j] - np.mean(X_lagged[:, j])
      S0[i0, j0] = np.dot(Z1, Z2)

  S0 = np.linalg.inv(S0)
  S_inv = np.zeros((m, m))
  for i0, i in enumerate(indices):
    for j0, j in enumerate(indices):
      S_inv[i, j] =  S0[i0, j0]

  if keep_label:
    labels = extend_labels(lagged_data.columns)
    return pd.DataFrame(S_inv, index=labels, columns=labels)

  return S_inv 
 

def cov_selection(data, p, JT, keep_label=False):
  """
  Computes K-hat, a re-estimation of the concentration matrix  for a restricted
  VAR(p) model via covariance selection to implement contemperaneous causallity 
  w.r.t. to a junction tree (JT) specified by the user.

  Parameters
  ----------
  data: array_like  
    A multi-dimensional time series. 
  p: int
    A postive integer specifying the order of the VAR model.
  JT: dict 
    A pseudo-graph object that describes the junction tree structure based on
    a graph genereated with contemporaneous variables in the data. 

    Example: 
    JT_Fin_VAR1={'cliques': 
                    [['BVSP', 'EM', 'NIK'], 
                      ['BVSP', 'DAX', 'EM', 'FTSE', 'ISE', 'SP'], 
                      ['BVSP', 'DAX', 'EU', 'FTSE', 'ISE']],
                  'separators': 
                    [['BVSP', 'EM'], 
                      ['BVSP', 'DAX', 'FTSE', 'ISE']]}
  keep_label: bool, default False
    If (keep_label == True), return result as a pd.DataFrame with readable
    labelling (extra columns are labelled as 'X_{i}-p' indicating the 
    respective lag-p variables in the context of autoregression).  

  Returns
  -------
    K_hat: np.data pd.DataFrame
      numpy pd x pd K concentration matrix 
  """
  lagged_data = lagged_sample(data, p, keep_label)
  X_lagged = np.array(lagged_data)
  
  n, d = data.shape
  n_adj, d_adj= X_lagged.shape # n_adj = n-p; d_adj = (p+1) * d

  # create a mapping of (contemporaneous) variables to indices 
  dct_var_to_index = {var:i for i,var in enumerate(data.columns)}

  K_hat = np.zeros((d_adj, d_adj)) # K_hat is a (p+1)d x (p+1)d matrix

  # all lagged variables (columns > d) needs to be included for covariance 
  # selection as our causal VAR model only consider comtemporaneous causality.
  ind_lagged_var = list(range(d, d_adj))
  for clique in JT['cliques']:
    indices = [dct_var_to_index[var] for var in clique]
    K_hat += blowed_S_inv(X_lagged, indices + ind_lagged_var)

  for sep in JT['separators']:
    indices = [dct_var_to_index[var] for var in sep]
    K_hat -= blowed_S_inv(X_lagged, indices + ind_lagged_var)

  K_hat *= n_adj # multiply sum & diff. of product moments by (n-p). 

  if keep_label:
    labels = extend_labels(data, p)
    return pd.DataFrame(K_hat, index=labels, columns=labels)
  
  return K_hat


# In[ Section 3: Block LDL Decomposition ]
## To obtain the model equations from the concentration matrix
def block_LDL(conc_mat, p, d):
  """
  Performs the proposed block LDL decomposition on the postiive definite
  concentration matrix for the novel causal VAR model. For more details, 
  please see the paper. 

  Parameters
  ----------
  conc_mat: array_like
    A positive-definite concentration matrix for the causal VAR(p) model.
  p: int
    A positive integer indicating the order of the VAR model.
  d: int
    A positive integer indicating the dimension of the VAR process.

  Returns
  -------
  L: np.array 
    A block-triangular matrix.
  D: np.array
    A block-diagonal matrix.
  """
  K = np.array(conc_mat)
  L = np.zeros(((p+1)*d,(p+1)*d))
  D = np.zeros(((p+1)*d,(p+1)*d))

  for j in range(d):
    if j == 0:
      D[0,0] = K[0,0]
    else:
      sum_ = 0
      for h in range(j):
        sum_ += L[j,h]*D[h,h]*L[j,h]
      D[j,j] = K[j,j] - sum_

    for i in range(j+1,d):
      sum_ = 0
      for h in range(j):
        sum_ += L[i,h]*D[h,h]*L[j,h]
      L[i,j] = (K[i,j] - sum_)/D[j,j]

    L[d:,j] = K[d:,j]
    for h in range(j):
      L[d:,j] -= D[h,h]*L[j,h]*L[d:,h]
    L[d:,j] = L[d:,j]/D[j,j]

  for i in range(2*d):
    L[i,i] = 1
  
  return L, D   

def get_A(L,d):
  """
  Retrives the A matrix (part of model equation) from the result of the block
  LDL decomposition.
  """
  return np.transpose(L[:d,:d])

def get_B(L,d):
  """
  Retrives the B matrix (part of model equation) from the result of the block
  LDL decomposition. Notice that B = (B_1, B_2, ..., B_p).
  """
  return np.transpose(L[d:,:d])

def get_Delta(D,d):
  """
  Retrives the Delta matrix from the result of the block LDL decomposition.
  """
  return np.linalg.inv(D[:d,:d])

# In[ Section 4: Order selection ]
## To obtain the optimal order p of the model and get according important matrices 
def cli_sep_choose_2 (JT,p,n,d):
    '''
    Computes the C_choose_2 and S_choose_2 where C is a clique and S a separator in the 
    junction tree decomposition.
    
    Parameters
    ----------
    JT: dict, default None
      A pseudo-graph object that describes the junction tree structure based on
      a decomposable graph generated with contemporaneous variables in the data.
    p: integer
      Order of the CVAR model
    n,d: integers
      Dimensions of the multi-dimentional time series
    
    Returns
    -------
    C_choose_2: ndarray containing each Cj_choose_2 for j=1,2...k where k is the number of cliques
    S_choose_2: ndarray containing each Sj_choose_2 for j=1,2...k where k is the number of cliques
    ''' 
    C = [len(JT['cliques'][0])]
    S = [0]
    for j in range(1,len(JT['cliques'])):
        C.append(len(JT['cliques'][j]))
        S.append(len(JT['separators'][j-1]))
    C = np.array(C)
    S = np .array(S)
    C_choose_2=C*(C-1)/2
    S_choose_2=S*(S-1)/2
    return(C_choose_2,S_choose_2)

def AICC_penalty_restricted(JT,p,n,d):
    '''
    Computes the penalty term of AICC in the restricted case.
    
    Parameters
    ----------
    JT: dict, default None
      A pseudo-graph object that describes the junction tree structure based on
      a decomposable graph generated with contemporaneous variables in the data.
    
    p: integer
      Order of the CVAR model
    n,d: integers
      Dimensions of the multi-dimentional time series
    
    Returns
    -------
    penalty_r: float
      penalty term of AICC for the restricted case.
    ''' 
    C_choose_2,S_choose_2 = cli_sep_choose_2(JT,p,n,d)
    penalty_r=2*(p*d**2+C_choose_2.sum()+S_choose_2.sum())*(n-p)*d/((n-p)*d-p*d**2-C_choose_2.sum()-S_choose_2.sum()-1)
    return(penalty_r)

def AICC_penalty_unrestricted(p,n,d):
    '''
    Computes the penalty term of AICC in the unrestricted case.
    
    Parameters
    ----------
    p: integer
      Order of the CVAR model
    n,d: integers
      Dimensions of the multi-dimentional time series
    
    Returns
    -------
      penalty term of AICC for the unrestricted case.
    ''' 
    d_choose_2= d*(d-1)/2
    return((2*(p*d**2+d_choose_2)*(n-p)*d)/((n-p)*d-p*d**2-d_choose_2-1))

def order_selection (data,ordering=None,JT=None):
    """
    Computes several order selection criteria for p=1,...,9 along with an estimation of each important matrices of 
    the CVAR_fitting(p) model for each p and for both restricted and unrestricted case. Also provides an estimation of the 
    time series.
    
    Parameters
    ----------
    data: array_like
      A multi-dimensional time series.
    ordering: list
      An ordered list specifying the desired variable ordering for the data
    JT: dict, default None
      A pseudo-graph object that describes the junction tree structure based on
      a decomposable graph generated with contemporaneous variables in the data.  
      Setting JT to a non-None value will enable covariance selection and lead to
      a restricted VAR model (zeros will be set to the corresponding positions in
      the based on the given junction tree).
     
    Returns
    -------
    df: dataframe
      A dataframe containing for each value of p the log-likelihood, log-determinant of the error 
      covariance matrix, and the penalization terms for each criterion and the AIC, AICC, BIC and HQ 
      computed thanks to the preceding quantities.
    A_p: list of matrices
      Contains the matrices A for each p
    Bs_p: list of list of matrices
      Contains the list of matrices B_1,...,Bp for each p
    X_hat_p: dataframe
      Estimation of the data, the time series (warning: has to be rearranged given the paper's ordering)
    """
    n,d = data.shape
      
      #Rearanging the raw data to stick with the format proposed in the paper
    if ordering==None:
        data=data.reindex(columns=['NIK', 'EU', 'ISE', 'EM', 'BVSP','FTSE','DAX','SP'])
       
      #Creating arrays of values to be return in the dataframe
    AICC_p, AIC_p, BIC_p,HQ_p = [], [], [], []
    m2_LogLik, Log_det_Delta = [], [] 
    AICC_penalty, AIC_penalty, BIC_penalty, HQ_penalty = [], [], [], []
        
        #Creating arrays of important matrices to be return in the dataframe
    A_p, Bs_p, X_hat_p = [], [], []
        
    for p in range (1,10):
        CVAR_matrices_dict = CVAR_fitting(data, p, ordering=ordering, JT=JT)
        K = CVAR_matrices_dict['K'] # (pd,pd) dimensional
        A = CVAR_matrices_dict['A'] # (d,d) dimensional 
        B = CVAR_matrices_dict['B'] # (d,pd) dimensional
        Delta = CVAR_matrices_dict['Delta']
        
        deltas = Delta.diagonal()          
        Bs = np.hsplit(B,p) # splitting B into p (d,d) dimensional matrices       
        X_lagged = lagged_sample(data, p, keep_label=False) # All lagged values up to t-p (n,pd) dim
        X_lags = np.hsplit(X_lagged,p+1) # splitting by lags
        X_hat = np.zeros(X_lags[0].shape)
        X_hat = X_hat.T
        
        #Computing U matrix and estimation of X_t
        U = A@np.transpose(X_lags[0])
        for i in range (len(Bs)):
            U += Bs[i]@np.transpose(X_lags[i+1])
            X_hat += -np.linalg.inv(A)@Bs[i]@np.transpose(X_lags[i+1]) #computing X_hat, estimation of X
            
        #Computing the mean squared errors (global and component wise)
        sum1 = np.zeros(d)
        sum2 = np.zeros(d)
        MSE1 = 0 #global MSE
        MSE2 = 0 #global MSE (other formula)
        MSEj = np.zeros(d) #component wise MSE (MSEj for j in preffered order)
            
        for t in range(n-p):
            sum1+= (U[t])**2
            sum2 += 1/(n-p)*(np.dot(A,(pd.DataFrame(data.iloc[p:,].to_numpy().T)-X_hat)[t])**2)
        for j in range(d):
            MSE1 += 1/(n-p)*sum1[j]
            MSE2 += sum2[j]/deltas[j]
            MSEj[j]= (sum2[j])**(1/2)
            
        #Uncomment to print MSE :
        #print('MSE1: ', MSE1)
        #print('Global MSE:', MSE2 )
        #print('C-p wise MSE: ', MSEj)
            
        #Appending the computed value for log-likelihhod, log(det(delta)), penalty terms and criteria for each p
            
        m2_LogLik.append(-2*(-(n-p)*d/2*np.log(2*np.pi)-(n-p)/2*np.log(deltas).sum()-1/2*(n-p)*MSE1))
        Log_det_Delta.append(np.log(deltas).sum())

        if JT is not None: #restricted case
            C_choose_2,S_choose_2 = cli_sep_choose_2 (JT,p,n,d)
            AICC_penalty.append(AICC_penalty_restricted(JT,p,n,d))
            AIC_penalty.append(2*(p*d**2+C_choose_2.sum()+S_choose_2.sum())/(n-p))
            BIC_penalty.append(np.log(n-p)*(p*d**2+C_choose_2.sum()+S_choose_2.sum())/(n-p))
            HQ_penalty.append(2*np.log(np.log(n-p))*(p*d**2+C_choose_2.sum()+S_choose_2.sum())/(n-p)) 
        else: #unrestricted case
            d_choose_2= d*(d-1)/2
            AICC_penalty.append(AICC_penalty_unrestricted(p,n,d))
            AIC_penalty.append(2*(p*d**2+d_choose_2)/(n-p))
            BIC_penalty.append(np.log(n-p)*(p*d**2+d_choose_2)/(n-p))
            HQ_penalty.append(2*np.log(np.log(n-p))*(p*d**2+d_choose_2)/(n-p)) 
            
        #Computing of the criteria
        AICC_p.append(m2_LogLik[p-1] + AICC_penalty[p-1])
        AIC_p.append(Log_det_Delta[p-1] + AIC_penalty[p-1])
        BIC_p.append(Log_det_Delta[p-1] + BIC_penalty[p-1])
        HQ_p.append(Log_det_Delta[p-1] + HQ_penalty[p-1])

        #Appending as dataframe computed matrices A, Bs and X_hat for each p 
        A_p.append(A)
        Bs_p.append(Bs)
        X_hat = X_hat.T
        X_hat_df = pd.DataFrame(X_hat)
        if ordering==None:
            X_hat_df.columns = data.columns
        else:
            X_hat_df.columns = ordering
        n1,d1 = X_hat_df.shape
        X_hat_df.index=data.index[n-n1:n]
        X_hat_p.append(X_hat_df)

    #Creating the dataframe to be returned
    tab={'AICC': AICC_p,'-2*Log-likelihood':m2_LogLik,'AICC_penalty': AICC_penalty,
         'Sum_log_deltas':Log_det_Delta, 'AIC':AIC_p,'AIC_penalty':AIC_penalty, 
         'BIC':BIC_p,'BIC_penalty':BIC_penalty,'HQ':HQ_p,'HQ_penalty':HQ_penalty}
    df=pd.DataFrame(tab, index=['p = '+ str(i) for i in range(1,10)])
        
    return (df,A_p,Bs_p,X_hat_p)

In [ ]:
def plot_graph(graph, title):
    nx.draw_shell(graph, node_size=1600, node_color='c', with_labels=True)
    plt.title(title)
    plt.show()

In [ ]:
def check_rzp(matrix, message):
    if has_RZP(matrix):
        print(f"The adjacency matrix for {message} has an RZP.")
    else:
        print(f"WARNING: the adjacency matrix for {message} does not have an RZP.")

In [ ]:
def CVAR_ParEst(df, p, start_var, restricted=True, plot_skeleton=False, plot_reordered=False, 
                   RZP_before_ordering=False, RZP_after_ordering=True, print_JT_dct=False, choose_own_order=False):
    
    """
    Fit a Conditional Vector Autoregressive (CVAR) model.     
    Paste all the functions after each other so we can easily fit a model with just one click.
        Parameters:
    df (pd.DataFrame): DataFrame containing the data to fit.
    p (int): Maximum time lag for the model.
    start_var (str): Starting node for the order.
    restricted (bool): If True, compute the Junction Tree and pass to the model.
    plot_skeleton (bool): If True, plot the skeleton graph.
    plot_reordered (bool): If True, plot the reordered graph.
    RZP_before_ordering (bool): If True, check RZP before ordering.
    RZP_after_ordering (bool): If True, check RZP after ordering.
    print_JT_dct (bool): If True, print the Junction Tree dictionary.
        Returns:
    model (CVAR): Fitted CVAR model.
    JT_dct (dict): Junction Tree dictionary.
        Optional:
    You could also choose to add to return: 
    C_1_given_0, pcor, JT, X_reordered, G, G_reordered, G_reordered_mat, preferred_order
    """
    
    "TESTS AND SKELETON"
    # Independence tests and skeleton graph
    C_1_given_0 = inv_cond_cov(data=df, p=p, keep_label=True)
    pcor = partial_cor(C_1_given_0, keep_label=True)
    G_mat = (abs(pcor) > 0.04).astype(int)
    G = nx.Graph(G_mat)
    
    if plot_skeleton:
        plot_graph(G, 'Skeleton Graph')
    
    if not nx.is_chordal(G):
        print("WARNING: the skeleton graph is not chordal.")
    #else:
        #print("The skeleton graph is chordal!")
    
    if RZP_before_ordering:
        check_rzp(G_mat, 'G')

    "ORDERING"
    if choose_own_order == True:
        preferred_order = max_card_order_v1(G, start_node=start_var)
    else:
        preferred_order = max_card_order(G, start_node=start_var)
    #print(f"The preferred order we will work with: {preferred_order}")
    
    # Reorder the graph
    G_dict = nx.to_dict_of_lists(G)
    G_reordered = nx.Graph({node: G_dict[node] for node in preferred_order})
    
    if plot_reordered:
        plot_graph(G_reordered, 'Reordered Graph')
    
    G_reordered_mat = nx.to_pandas_adjacency(G_reordered, dtype=int)
    X_reordered = df.reindex(columns=preferred_order)
    
    if RZP_after_ordering:
        check_rzp(G_reordered_mat, 'G_reordered')

    "JUNCTION TREE"
    JT = nx.junction_tree(G_reordered)
    JT_dct = {'cliques': [], 'separators': []}
    for node, node_type in JT.nodes(data='type'):
        if node_type == 'clique':
            JT_dct['cliques'].append(node)
        else:
            JT_dct['separators'].append(node)
    
    if print_JT_dct:
        print(JT_dct)
    
    "MODEL"
    model_type = "restricted" if restricted else "unrestricted"
    #print(f"This function returns the {model_type} CVAR({p}) model.")
    
    model = CVAR_fitting(data=df, p=p, ordering=preferred_order, JT=JT_dct if restricted else None)
    
    return model, JT_dct

In [ ]:
CVAR_estimate, _ = CVAR_ParEst(df_identity_CVAR1, 1, 'TW')
print("")
print(CVAR_estimate['B'])
print(CVAR_estimate['B']['CPAC-1'])

In [ ]:
try_out = np.zeros((4,4))
print(try_out)
try_out[:, 0] = CVAR_estimate['B']['WPAC-1']
try_out[:, 1] = CVAR_estimate['B']['EPAC-1']
try_out[:, 2] = CVAR_estimate['B']['CPAC-1']
try_out[:, 3] = CVAR_estimate['B']['TW-1']
print(try_out)

# Tigramite's PCMCI

The last, though, most important estimator we will look at is the **PCMCI algorithm** as defined in the Tigramite package. This package is not related to a certain model but to the theory of **Directed Acyclic Graphs**. The well-known PCMCI algorithm develops estimates for the auto-MCI and cross-MCI, which describe the nodes and linkstrengths in the graph. According to the theory in Bolla (2023) this can be related to a CVAR model, only instead of their provided code for estimating the parameters, we estimate it using Tigramite. The output val_matrix gives the estimates for $\Phi$. 

First of all we set the conditional independence test to be like the one Schultz used.

## Functions

In [ ]:
parcorr = ParCorr(confidence='analytic', mask_type='y')

In [ ]:
def RunPCMCI(df, test, tau_max=1, tau_min=1, pc_alpha=0.05, alpha_level=0.01, with_mask = True, model = 'model', show_DAG=True, var_names = ['WPAC', 'CPAC', 'EPAC', 'TW']):
    """
    Run the PCMCI algorithm to estimate the parameters in a CVAR(1) model (or another dataset).
    Parameters:
        df = the dataframe of the data; detrended and deseazonalized!
        test = the partial correlation test that will be used to find the links
        tau_max = maximum lag in the model (take 1 if we want to link it to a CVAR(1) model for example)
        tau_min = minimum lag in the model (generally set to 1)
        pc_alpha = significance level in algorithm
        alpha_level = significance level at which the p_matrix is thresholded to get graph
        with_mask = TRUE if we want to apply the spring barrier mask
        model = string with name of the model
        show_DAG = TRUE if we want to plot the DAG at the end
    """
    # Construct mask for spring barrier
    cycle_length = 12
    springbarrier_mask = np.ones(df.shape, dtype=bool)
    for i in [4, 5, 6, 7, 8, 9, 10, 11, 0]:
        springbarrier_mask[i::cycle_length, :] = False
    
    # Run PCMCI
    if with_mask == True:
        dfT = pp.DataFrame(np.copy(df), datatime = {0:np.arange(len(df))}, var_names= var_names, mask = springbarrier_mask)
    else:
        dfT = pp.DataFrame(np.copy(df), datatime = {0:np.arange(len(df))}, var_names= var_names)

    pcmci = PCMCI(dataframe=dfT, cond_ind_test=test, verbosity=0)
    results = pcmci.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=pc_alpha, alpha_level=alpha_level, fdr_method = 'fdr_bh')
    
    if show_DAG == True:
    # Plot the graph
        tp.plot_graph(
            val_matrix=results['val_matrix'],
            graph=results['graph'],
            var_names=var_names,
            link_colorbar_label='cross-MCI',
            node_colorbar_label='auto-MCI')
        plt.title(f'Causal networks based on complete history of {model} data', fontsize = 18)
        plt.show()
    
    return results

## Example

Now the above function RunPCMCI is actually the parameter estimation function. The model itself is simply the CVAR(1) model as before again. The example will be continued below by estimating this model using PCMCI and converting the val_matrix into a $\Phi$ matrix for the CVAR(1) model.

In [ ]:
PlotDF(df_identity_CVAR1, model_name = f'Random CVAR(1) model data with identity matrix (= VAR(1)) with seed={seed_identity_CVAR1}')

## Parameter Estimation

The parameters of the model can simply be estimated by running the PCMCI algorithm and converting the outcome val_matrix into a $\Phi$ matrix that fits the CVAR(1) model. Note that if we use CVAR(p) models with $p>1$ we will need to adjust the maximum lag in the algorithm. The val_matrix then becomes bigger and the different lags in the model will correspond to $\Phi_{t-1}$, $\Phi_{t-2}$, $\ldots$, $\Phi_{t-p}$. 

**Important comparison note:** Schultz looks at the minimal time lag of 1, not 0, meaning the PCMCI algorithm does not compute the matrix $A$ as defined in the CVAR model. Therefore, this model takes $A = I$ again, with $I$ being the identity matrix. 

In [ ]:
def ConstructPhiMatrix(val_matrix):
    "Construct a Phi Matrix for the pcmci option based on an input val_matrix"
    n, _, _ = val_matrix.shape # Or just fill in 4
    Phi = np.zeros((n,n))
    for i in range(0,n): 
        for j in range(0,n): 
            Phi[i,j] = val_matrix[j,i,1]
    return Phi

Now the example is continued by estimating the CVAR(1) model with the PCMCI algorithm and then setting $\mathbf{val\_matrix} = \Phi$. 

In [ ]:
Example_Results = RunPCMCI(df_identity_CVAR1, parcorr, tau_max=1, tau_min=1, pc_alpha=0.05, alpha_level=0.01, with_mask = True, model = 'Example with CVAR(1) model', show_DAG=True)
Example_PCMCI_Phi = ConstructPhiMatrix(Example_Results['val_matrix'])

# Tigramite's Bootstrap Aggregating (Bagging)

Tigramite does not only contain the PCMCI algorithm, but also the bootstrapped version of this. We can easily run this with the following function.

## Functions

In [ ]:
def RunBagging(df, test, tau_max=1, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, pc_alpha=None, plot=True, model='', with_mask=True, var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None):
    """
    Run Bootstrap Aggregating to find the parameter estimates for a CVAR(1) model or a different dataset.
    Parameters:
        df = the dataframe of the data; detrended and deseazonalized!
        test = the partial correlation test that will be used to find the links
        
        tau_max = maximum lag in the model (take 1 if we want to link it to a CVAR(1) model for example)
        pc_alpha = significance level in algorithm
        alpha_level = significance level at which the p_matrix is thresholded to get graph
        with_mask = TRUE if we want to apply the spring barrier mask
        model = string with name of the model
        show_DAG = TRUE if we want to plot the DAG at the end
    """
    
    if seed is not None:
        np.random.seed(seed)
    
    # Construct mask for spring barrier
    cycle_length = 12
    springbarrier_mask = np.ones(df.shape, dtype=bool)
    for i in [4, 5, 6, 7, 8, 9, 10, 11, 0]:
        springbarrier_mask[i::cycle_length, :] = False
    
    if with_mask == True:
        dfT = pp.DataFrame(np.copy(df), datatime = {0:np.arange(len(df))}, var_names= var_names, mask = springbarrier_mask)
    else:
        dfT = pp.DataFrame(np.copy(df), datatime = {0:np.arange(len(df))}, var_names= var_names)
    
    #Now let's run it:
    
    pcmci = PCMCI(dataframe=dfT, cond_ind_test=test, verbosity=0)
    
    results = pcmci.run_bootstrap_of(
        method='run_pcmciplus', 
        method_args={'tau_max':tau_max, 'pc_alpha':pc_alpha_bootstrap}, 
        boot_samples=boot_samples,
        boot_blocklength=boot_blocklength,
        seed=seed)
    
    if plot == True:
        # Output graph, link frequencies (confidence measure), and mean test statistic values (val_mat)
        boot_linkfreq = results['summary_results']['link_frequency']
        boot_graph = results['summary_results']['most_frequent_links']
        val_mat = results['summary_results']['val_matrix_mean']
    
        tp.plot_graph(
            graph = boot_graph,
            val_matrix= val_mat,
            link_width = boot_linkfreq,
            var_names=dfT.var_names,
            )
        plt.title(f'Causal networks using Bagging based on {model} data', fontsize=16)
        plt.show()
        
    return results

## Example

Now the above function RunPCMCI is actually the parameter estimation function. The model itself is simply the CVAR(1) model as before again. The example will be continued below by estimating this model using PCMCI and converting the val_matrix into a $\Phi$ matrix for the CVAR(1) model.

## Parameter Estimation

In [ ]:
def ConstructPhiMatrixBagging(mean_val_matrix):
    "Construct a Phi Matrix for the bagging option based on an input mean val_matrix"
    n, _, _ = mean_val_matrix.shape # Or just fill in 4
    Phi = np.zeros((n,n))
    for i in range(0,n): 
        for j in range(0,n): 
            Phi[i,j] = mean_val_matrix[j,i,1]
    return Phi

In [ ]:
Example_Results_bagging = RunBagging(df_identity_CVAR1, parcorr, tau_max=1, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, pc_alpha=None, plot=True, model='', with_mask=True, var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)
#RunPCMCI(df_identity_CVAR1, parcorr, tau_max=1, tau_min=1, pc_alpha=0.05, alpha_level=0.01, with_mask = True, model = 'Example with CVAR(1) model', show_DAG=True)

In [ ]:
Example_Bagging_Phi = ConstructPhiMatrixBagging(Example_Results_bagging['summary_results']['val_matrix_mean'])

# DATA: Jointly Normal Random Set

Now we can test the three models by generating a Normal Random Set, estimating the parameters for each model using the provided parameter estimation, and then generating a random time series based on these parameters. Do we see a difference in their performance? And what happens if we, later on, throw the Tigramite estimation in the mix?

In [ ]:
def JointlyNormalDataset(T, mean = np.zeros(4), cov=np.eye(4), hz_alpha=0.05, seed=None):
    """
    Generate a dataset that is jointly normally distributed.
    Parameters:
        T = number of time points.
        mean = mean of the multivariate normal distribution.
        cov = covariance matrix of the multivariate normal distribution.
        hz_alpha = significance level for the Henze-Zirkler test.
    Returns:
        data = generated dataset.
        seed = seed used for the generation.
    """
    np.random.seed(seed)
    multi_normal = False
    seed = seed - 1 if seed is not None else None

    while not multi_normal:
        seed += 1
        np.random.seed(seed)
        data = np.random.multivariate_normal(mean, cov, T)
        df = pd.DataFrame(data, columns=[f'Var{i+1}' for i in range(len(mean))])
        henze_zirkler_result = pg.multivariate_normality(df, alpha=hz_alpha)
        if henze_zirkler_result[2] == True:
            multi_normal = True
    
    return data, seed

In [ ]:
covY = [[1, 0.8, 0.2, 0.6], 
       [0.8, 1, 0.3, 0.7], 
       [0.2, 0.3, 1, 0.4], 
       [0.6, 0.7, 0.4, 1]]
data, seed_for_jointly_normal_data = JointlyNormalDataset(864, cov=covY, seed=611)
Y = data.T
# (pingouin wants n_samples by n_features so T by N
henze_zirkler_result = pg.multivariate_normality(data, alpha=0.05)
print(henze_zirkler_result)

In [ ]:
dfY, dfTY = GenerateDF(Y, start_year=1950, yearly=False, freq='ME', column_names=['WPAC','CPAC','EPAC','TW'])

In [ ]:
PlotDF(dfY, model_name = 'Random Jointly Normal data')

When fitting the models, some statistics are handy to keep in mind. The means can tell us something about the place of the data on the Y-axis, whereas the lower and upper bound of the data will tell is the width. 

In [ ]:
means = dfY.mean()
print(means)

In [ ]:
minimalYvalues = dfY.min()
print(f'Minimum values in the dataframe:')
print(minimalYvalues)
minY = minimalYvalues.min()
print(f'Minimal value in the dataframe:')
print(minY)

In [ ]:
maximalYvalues = dfY.max()
print(f'Maximal values in the dataframe:')
print(maximalYvalues)
maxY = maximalYvalues.max()
print(f'Maximal value in the dataframe:')
print(maxY)

## Fit VAR Model

First try fitting the regular VAR model. Remember; this uses MLS to estimate $\Phi$ and leaves out the $A$ matrix. The model aks for a lower bound and upper bound to generate the first iteration $Y_1$ between. These are set to the minimum and maximum of the dataset.

In [ ]:
MLS_Phi_hat = MLS(dfY)
print(MLS_Phi_hat[:, 1:])

In [ ]:
Y_VAR1 = VAR1(864, MLS_Phi_hat[:, 1:], standard=True, constant=False, lower_bound_Y1=minY, upper_bound_Y1=maxY, seed=None)

In [ ]:
dfY_VAR1, dfTY_VAR1 = GenerateDF(Y_VAR1, start_year=1950, yearly=False, freq='ME', column_names=['WPAC','CPAC','EPAC','TW'])

In [ ]:
PlotDF(dfY_VAR1, model_name = 'Random Jointly Normal data - Estimated with VAR(1) model')

## Fit SVAR Model

Next up the model is fitted to the SVAR model, that extends the theory of a VAR model with the structural component matrix $A$. Therefore, two parameters are now estimated; $A$ and $\Phi$. Instead of MLS, now ML methods are used to estimate the model parameters. Then a new random dataset is generated again based on these parameters to check the correctness.

## Fit CVAR Model

Next up the model is fitted to the CVAR model, that extends the theory even further by estimating the matrices $A$ and $\Phi$ based on causal inference theory, using Cholesky block decompositions. 

In [ ]:
CVAR_estimates, _ = CVAR_ParEst(dfY, 1, 'TW')
Causal_A = CVAR_estimates['A']
Causal_Phi = CVAR_estimates['B']

In [ ]:
Y_CVAR1, seed_Y_CVAR1 = CVAR1(864, Causal_A, Causal_Phi, standard=True, constant=False, lower_bound_Y1=minY, upper_bound_Y1=maxY, pp_alpha=0.05, seed=1)

In [ ]:
dfY_CVAR1, dfTY_CVAR1 = GenerateDF(Y_CVAR1, start_year=1950, yearly=False, freq='ME', column_names=['WPAC','CPAC','EPAC','TW'])

In [ ]:
PlotDF(dfY_VAR1, model_name = 'Random Jointly Normal data - Estimated with CVAR(1) model')

# DATA: Load Monthly Climate Data

Next up, load the dataset that Schultz uses in her work; data on the variables in the Bjerknes feedback on a monthly basis. We will import them in the same way she did using the same transformations.

In [ ]:
start_year = 1950
length = 12*(2022 - start_year)

## WPAC = Sea level pressure (in Pa) in the West Pacific

In [ ]:
WPAC = np.loadtxt('iera5_slp_130-150E_-5-5N_n.dat.txt', skiprows=20)[:, 1:].reshape(-1)[:length]

In [ ]:
mean_WPAC = np.mean(WPAC)
std_dev_WPAC = np.std(WPAC)
median_WPAC = np.median(WPAC)
min_val_WPAC = np.min(WPAC)
max_val_WPAC = np.max(WPAC)

print("Mean:", mean_WPAC)
print("Standard Deviation:", std_dev_WPAC)
print("Median:", median_WPAC)
print("Minimum Value:", min_val_WPAC)
print("Maximum Value:", max_val_WPAC)

In [ ]:
start_year = 1950
end_year = 2022
plot_length = 12 * (end_year - start_year + 1)
time_index = pd.date_range(start=f'{start_year}-01', periods=plot_length, freq='MS')
time_index = time_index[:len(WPAC)]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_index, y=WPAC, mode='lines', name='WPAC'))
fig.update_layout(
    title='Time Series of WPAC',
    xaxis_title='Time',
    yaxis_title='Near surface temperature<br>(in K)<br>in the West Pacific',
    margin=dict(l=140), 
    yaxis=dict(
        title_standoff=10,
        automargin=True
    )
)
fig.show()

## CPAC = Near surface temperature (in K) in the Central Pacific

In [ ]:
CPAC = np.loadtxt('iera5_t2m_-150--130E_-5-5N_n.dat.txt', skiprows=20)[:, 1:].reshape(-1)[:length]

In [ ]:
mean_CPAC = np.mean(CPAC)
std_dev_CPAC = np.std(CPAC)
median_CPAC = np.median(CPAC)
min_val_CPAC = np.min(CPAC)
max_val_CPAC = np.max(CPAC)

print("Mean:", mean_CPAC)
print("Standard Deviation:", std_dev_CPAC)
print("Median:", median_CPAC)
print("Minimum Value:", min_val_CPAC)
print("Maximum Value:", max_val_CPAC)

In [ ]:
time_index = time_index[:len(CPAC)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_index, y=CPAC, mode='lines', name='CPAC'))
fig.update_layout(
    title='Time Series of CPAC',
    xaxis_title='Time',
    yaxis_title='Near surface temperature<br>(in K)<br>in the Central Pacific',
    margin=dict(l=140), 
    yaxis=dict(
        title_standoff=10,
        automargin=True
    )
)
fig.show()

## EPAC = Near surface temperature (in K) in the East Pacific

In [ ]:
EPAC = np.loadtxt('iera5_t2m_-100--80E_-5-5N_n.dat.txt', skiprows=20)[:, 1:].reshape(-1)[:length]

In [ ]:
mean_EPAC = np.mean(EPAC)
std_dev_EPAC = np.std(EPAC)
median_EPAC = np.median(EPAC)
min_val_EPAC = np.min(EPAC)
max_val_EPAC = np.max(EPAC)

print("Mean:", mean_EPAC)
print("Standard Deviation:", std_dev_EPAC)
print("Median:", median_EPAC)
print("Minimum Value:", min_val_EPAC)
print("Maximum Value:", max_val_EPAC)

In [ ]:
time_index = time_index[:len(EPAC)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_index, y=EPAC, mode='lines', name='EPAC'))
fig.update_layout(
    title='Time Series of EPAC',
    xaxis_title='Time',
    yaxis_title='Near surface temperature<br>(in K)<br>in the East Pacific',
    margin=dict(l=140), 
    yaxis=dict(
        title_standoff=10,
        automargin=True
    )
)
fig.show()

## TW = Near surface winds (in m/s) in the Central Pacific

In [ ]:
TW = np.loadtxt('iera5_u10m_-180--130E_-5-5N_n.dat.txt', skiprows=20)[:, 1:].reshape(-1)[:length]
TW *= -1 

In [ ]:
mean_TW = np.mean(TW)
std_dev_TW = np.std(TW)
median_TW = np.median(TW)
min_val_TW = np.min(TW)
max_val_TW = np.max(TW)

print("Mean:", mean_TW)
print("Standard Deviation:", std_dev_TW)
print("Median:", median_TW)
print("Minimum Value:", min_val_TW)
print("Maximum Value:", max_val_TW)

In [ ]:
time_index = time_index[:len(TW)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_index, y=TW, mode='lines', name='TW'))

fig.update_layout(
    title='Time Series of TW',
    xaxis_title='Time',
    yaxis_title='Near surface winds<br>(in m/s)<br>in the Central Pacific',
    margin=dict(l=140), 
    yaxis=dict(
        title_standoff=10,
        automargin=True
    )
)

fig.show()

## Extended Dataset and lists

In [ ]:
extended_data = np.vstack((
    WPAC,
    CPAC,
    EPAC,
    TW
                )).T

In [ ]:
df_extended_data, dfT_extended_data = GenerateDF(extended_data.T)

In [ ]:
PlotDF(df_extended_data, model_name = 'Climate Dataset Complete')

In [ ]:
corr_extended_data = df_extended_data.corr()
print(corr_extended_data)
plt.figure(figsize=(10, 8))
sns.heatmap(corr_extended_data, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.1)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# FOR THE DSCRIPTIVE STATISTICS I use the untransformed data:
descriptive_stats = df_extended_data.describe()
print(descriptive_stats)
print(descriptive_stats.to_latex())

In [ ]:
#from pandas_profiling import ProfileReport



## Detrending and deseazonalizing

In [ ]:
def deTrend_deSeasonalize(data, period=12, show_plot = True):
    # step 1: seasonal component
    decomposition = seasonal_decompose(data, model='additive', period=period) # 1 for annual, 12 for monthly, 52 for weekly and 365 for daily
    data = data - decomposition.seasonal
    # step 2: Trend based on Gaussian kernel smoother
    #smoothed = pp.smooth(np.copy(data), smooth_width=15 * 12, residuals=False)
    index = range(0, len(data))

    if show_plot == True:
        plt.plot(index, data)
        #plt.plot(index,smoothed)
        plt.legend(['data', 'Smoothed trend'])
        plt.title("Deseasonalized data with smoothed trend")
        plt.show()

    else:
        result = data #- smoothed
        return result

In [ ]:
cpac_detrend = deTrend_deSeasonalize(df_extended_data['CPAC'], show_plot = False)
epac_detrend = deTrend_deSeasonalize(df_extended_data['EPAC'], show_plot = False)
wpac_detrend = deTrend_deSeasonalize(df_extended_data['WPAC'], show_plot = False)
tw_detrend = deTrend_deSeasonalize(df_extended_data['TW'], show_plot = False)
        
df_extended_data_dsdt = pd.DataFrame({'WPAC': wpac_detrend, 'CPAC': cpac_detrend, 'EPAC': epac_detrend, 'TW': tw_detrend})

In [ ]:
corr_extended_data_dsdt = df_extended_data_dsdt.corr()
print(corr_extended_data_dsdt)
plt.figure(figsize=(10, 8))
sns.heatmap(corr_extended_data_dsdt, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

## Data Statistics 

In [ ]:
means_climate = [mean_WPAC, mean_CPAC, mean_EPAC, mean_TW]
variances_climate = [(std_dev_WPAC**2), (std_dev_CPAC**2), (std_dev_EPAC**2), (std_dev_TW**2)]

In [ ]:
print(means_climate)
print(variances_climate)

In [ ]:
minima_climate = [min_val_WPAC, min_val_CPAC, min_val_EPAC, min_val_TW]
maxima_climate = [max_val_WPAC, max_val_CPAC, max_val_EPAC, max_val_TW]

In [ ]:
print(minima_climate)
print(maxima_climate)

# Bias Dynamics

The statistic of interest here is the bias, defined as follows:

${\displaystyle \operatorname {Bias} ({\hat {\theta }},\theta )=\operatorname {Bias} _{\theta }[\,{\hat {\theta }}\,]=\operatorname {E} _{x\mid \theta }[\,{\hat {\theta }}\,]-\theta =\operatorname {E} _{x\mid \theta }[\,{\hat {\theta }}-\theta \,]}$

Where $\theta$ is the true parameter matrix and $\hat{\theta}$ is the estimator for the true parameter matrix. In this thesis, several different estimators are used, like MLS, PCMCI and Bootstrap Aggregating (bagging). 

The goal here is to find the convergence dynamics of the bias of the estimator as the length of the time series grows. Thus, the bias is computed as follows (for one stable length of the time series T);
1. Start with a 'true parameter matrix' $\phi$ which can either be simulated randomly or be assumed to be true from a certain outcome.
2. Simulate multivariate Normal time series based on a stable VAR(1) model for simplicity (of length T).
3. Estimate $\hat{\phi}$ using the estimator with the data from (2) as input.
4. Repeat steps 2 and 3 for M times, storing all estimates in a list.
5. Compute the mean of the estimated matrices (with each entry of the mean estimate being the mean of those entries in the estimated matrices.
6. Compute the bias by subtracting the true parameter matrix from the mean estimated parameter matrix.

This can be repeated for any lenght of the time series T. The wish is to create a graph that plots, for any lenght of T, the bias as based on many simulations. 

## MLS

The first parameter estimator we look at is the Multiple Least Squares estimator, which corresponds to the VAR(1) model:

$Y_t = \Phi Y_{t-1} + \epsilon_t$

Note that this is the same as the SVAR(1) model with $A = I$. The parameter $\Phi$ is estimated using MLS. 

In [ ]:
def Bias_MLS(true_parameter, T, M, lower_bound, upper_bound, standard=True, constant=False, show_progress=True, seed=None):
    """
    Calculate the bias of the estimator for datasets of T using MLS as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        T = lenght of the generated dataset that will estimate the parameter using MLS
        M = number of simulations to calculate the mean parameter
        standard = TRUE uses a standard Normal distribution for generating the errors
        constant = FALSE returns only zeros, so that the constant does not influence the time series, TRUE returns random constants
    Returns:
        bias in the form of the mean estimate for the parameter minus the true parameter
    """
    if seed is not None:
        np.random.seed(seed)
    
    estimates = []
    norms_per_sim = []
    
    iterator = tqdm(range(M), desc="Running MLS simulations") if show_progress else range(M)
    
    for i in iterator:        # Step 2: simulate the data
        Y = VAR1(T+100, true_parameter, standard=standard, constant=constant, lower_bound_Y1=lower_bound, upper_bound_Y1=upper_bound, seed=seed)
        df_full, _ = GenerateDF(Y)
        df = df_full.iloc[100:]  # Skip the first 100 data points for a .. break (niet de berlin maar die andere naam haha)

        # Step 3: estimate the parameter
        try:
            MLS_estimate = MLS(df)
        except np.linalg.LinAlgError:
            print(f"SVD did not converge for T={T}, seed={seed}. Retrying with a different seed.")
            if seed is not None:
                seed += 1
            continue        
        # Step 4: store information and repeat 
        estimates.append(MLS_estimate[:, 1:])
        if seed is not None:
            seed += 1
        # Later on we might want to plot the norms per simulation as well so that we can see what is happening
        bias_per_sim = MLS_estimate[:, 1:] - true_parameter
        norm_per_sim = np.linalg.norm(bias_per_sim, 'fro')
        norms_per_sim.append(norm_per_sim)
        
    mean_estimate = np.mean(estimates, axis=0)
    bias = mean_estimate - true_parameter
    bias_value = np.linalg.norm(bias, 'fro')

    return bias, bias_value, norms_per_sim

In [ ]:
bias, bias_value, norms_per_sim = Bias_MLS(climate_pcmci_phi, 864, 10000, minima_climate, maxima_climate, standard=True, constant=False, seed=None)
#def Bias_MLS(true_parameter, T, M, lower_bound, upper_bound, standard=True, constant=False, seed=None)
print(bias)
#print(norms_per_sim)
#bias2, bias_value2, norms_per_sim2 = Bias_MLS(climate_pcmci_phi, 864, 10, minima_climate, maxima_climate, standard=True, constant=False, seed=21)
#all_norms = [norms_per_sim, norms_per_sim2]
#print(all_norms)
#print(all_norms[0])

In [ ]:
print("bias value", bias_value)

In [ ]:
print("norms per sim", norms_per_sim)

In [ ]:
bias2, bias_value2, norms_per_sim2 = Bias_MLS(climate_pcmci_phi, 2000, 10000, minima_climate, maxima_climate, standard=True, constant=False, seed=None)


In [ ]:
print(bias2)

In [ ]:
print("bias value", bias_value2)

In [ ]:
print("norms per sim", norms_per_sim2)

In [ ]:
bias3, bias_value3, norms_per_sim3 = Bias_MLS(climate_pcmci_phi, 600, 10000, minima_climate, maxima_climate, standard=True, constant=False, seed=None)


In [ ]:
print(bias3)

In [ ]:
print("bias value", bias_value3)

In [ ]:
print("norms per sim", norms_per_sim3)

In [ ]:
bias4, bias_value4, norms_per_sim4 = Bias_MLS(climate_pcmci_phi, 10000, 10000, minima_climate, maxima_climate, standard=True, constant=False, show_progress=True, seed=None)


In [ ]:
print(bias4)

In [ ]:
print("bias value", bias_value4)

In [ ]:
print("norms per sim", norms_per_sim4)

In [ ]:
bias5, bias_value5, norms_per_sim5 = Bias_MLS(climate_pcmci_phi, 432, 10000, minima_climate, maxima_climate, standard=True, constant=False, show_progress=True, seed=None)


In [ ]:
print(bias5)

In [ ]:
print("bias value", bias_value5)

In [ ]:
print("norms per sim", norms_per_sim5)

Now that the bias can easily be computed, let us iterate over different lengths for the time series T to see how the MLS estimator behaves as the dataset becomes bigger.

In [ ]:
def SimulateBiasMLS(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None):
    """
    Calculate the bias of the estimator for datasets of T using MLS as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        min_T = the starting point of the time intervals (integer)
        max_T = the ending point of the time intervals (integer)
        intervals = the size of the intervals (integer)
        M = the number of simulations used to calculate the mean bias and norm for each length T
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        standard = if TRUE then the errors of the randomly simulated dataset are from a standard Normal distribution; if FALSE then from a random Normal distribution
        constant = if FALSE then the constant is left out of the VAR(1) model
        plot_graph = if TRUE then the graph is plotted at the end
        plot_sims = if TRUE then the norms for all the simulations are also shown in the graph
    Returns:
        biases = the matrices containing the bias for each entry of the matrix, for each time T
        norms = the norm of the bias matrices
        df_norms = the dataframe containing the norms for each T (also used for the plot)
    """
    l = (max_T - min_T)//intervals
    biases = []
    norms = np.zeros((l,2))
    all_norms = []
    i = 0
    
    
    "Running the simulations"
    for t in tqdm(range(min_T, max_T, intervals), desc="Simulating Bias"):
        bias, norm, all_norms_T = Bias_MLS(true_parameter, t, M, lower_bound, upper_bound, standard=standard, constant=constant, show_progress=False, seed=seed)
        biases.append(bias)
        norms[i,0] = t
        norms[i,1] = norm
        all_norms.append(all_norms_T)
        i += 1
    
    
    "Making the graph"
    df_norms = pd.DataFrame(norms, columns=['T','Frobenius Norm'])
    df_norms.set_index('T', inplace=True)
    
    fig = go.Figure()
    # Add the Frobenius Norm line in red
    fig.add_trace(go.Scatter(
        x=df_norms.index,
        y=df_norms['Frobenius Norm'],
        mode='lines',
        name='Frobenius Norm',
        line=dict(color='red')
    ))    # if we would like to plot the norms that we retrieved for the M iterations:
    if plot_sims == True: 
        for idx, all_norms_T in enumerate(all_norms):
                t = min_T + idx * intervals
                fig.add_trace(go.Scatter(x=[t]*len(all_norms_T), y=list(all_norms_T), mode='markers', marker=dict(color='blue', opacity=0.05), showlegend=(idx == 0), name='Norms of Simulations' if t == min_T else None))
            
    fig.update_layout(yaxis_title='Frobenius Norm of Bias')
    fig.update_layout(xaxis_title='Number of time periods T in dataset')
    # Add the trend line to the plot
    x = df_norms.index.values
    y = df_norms['Frobenius Norm'].values
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    slope = z[0]
    fig.add_trace(go.Scatter(x=x, y=p(x), mode='lines', name='Trend Line', line=dict(color='black', dash='dash'))) # achter grey: , dash='dash'
    fig.add_annotation(x=(((max_T-min_T)/2)+min_T), y=p(((max_T-min_T)/2)+min_T), text=f"Trend Slope: {slope:.4f}", showarrow=True, arrowhead=2, ax=-40, ay=-30, bgcolor="white")

    if plot_graph == True:
        fig.show()
    
    
    return biases, norms, df_norms

In [ ]:
def SimulateBiasMLS2(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None):
    """
    Calculate the bias of the estimator for datasets of T using MLS as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        min_T = the starting point of the time intervals (integer)
        max_T = the ending point of the time intervals (integer)
        intervals = the size of the intervals (integer)
        M = the number of simulations used to calculate the mean bias and norm for each length T
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        standard = if TRUE then the errors of the randomly simulated dataset are from a standard Normal distribution; if FALSE then from a random Normal distribution
        constant = if FALSE then the constant is left out of the VAR(1) model
        plot_graph = if TRUE then the graph is plotted at the end
        plot_sims = if TRUE then the norms for all the simulations are also shown in the graph
    Returns:
        biases = the matrices containing the bias for each entry of the matrix, for each time T
        norms = the norm of the bias matrices
        df_norms = the dataframe containing the norms for each T (also used for the plot)
    """
    l = (max_T - min_T) // intervals
    biases = []
    norms = np.zeros((l, 2))
    all_norms = []
    i = 0
    
    colors = sns.color_palette("colorblind")
    purple = colors[3]
    green = colors[2]
    orange = colors[1]
    lightblue = colors[0]
    
    # Running the simulations
    for t in tqdm(range(min_T, max_T, intervals), desc="Simulating Bias"):
        bias, norm, all_norms_T = Bias_MLS(true_parameter, t, M, lower_bound, upper_bound, standard=standard, constant=constant, show_progress=False, seed=seed)
        biases.append(bias)
        norms[i, 0] = t
        norms[i, 1] = norm
        all_norms.append(all_norms_T)
        i += 1

    # Making the graph
    df_norms = pd.DataFrame(norms, columns=['T', 'Frobenius Norm'])
    df_norms.set_index('T', inplace=True)

    # Plotting with Seaborn and Matplotlib
    if plot_graph:
        plt.figure(figsize=(12, 6))
        
        # Plot the Frobenius Norm line in red
        sns.lineplot(data=df_norms, x=df_norms.index, y='Frobenius Norm', color=green, label='Frobenius Norm')
        
        if plot_sims:
            for idx, all_norms_T in enumerate(all_norms):
                t = min_T + idx * intervals
                sns.scatterplot(x=[t] * len(all_norms_T), y=all_norms_T, color=lightblue, alpha=0.05, label='Norms of Simulations' if idx == 0 else "")

        # Add the trend line to the plot
        x = df_norms.index.values
        y = df_norms['Frobenius Norm'].values
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        slope = z[0]
        sns.lineplot(x=x, y=p(x), color=orange, linestyle='--', label='Trend Line')

        plt.title('Frobenius Norm of Bias')
        plt.xlabel('Number of time periods T in dataset')
        plt.ylabel('Frobenius Norm of Bias')
        plt.legend()
        plt.annotate(f"Trend Slope: {slope:.4f}", xy=(x[len(x) // 2], p(x[len(x) // 2])), xytext=(0, 50),
                     textcoords='offset points', arrowprops=dict(arrowstyle='->', color='black', lw=1.5), backgroundcolor='white', fontsize=12)
        plt.tight_layout()
        plt.show()

    return biases, norms, df_norms

In [ ]:
biasesMLSgraph, normsMLSgraph, df_normsMLSgraph = SimulateBiasMLS2(climate_pcmci_phi, 600, 4000, 50, 1000, minima_climate, maxima_climate, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None)
#def SimulateBiasMLS(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None):

In [ ]:
biasesMLSgraph, normsMLSgraph, df_normsMLSgraph = SimulateBiasMLS2(climate_pcmci_phi, 600, 4000, 50, 1000, minima_climate, maxima_climate, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None)
#def SimulateBiasMLS(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None):

In [ ]:
colors = sns.color_palette("colorblind")
purple = colors[3]
green = colors[2]
orange = colors[1]
print(orange)    

## CVAR

Here data is simulated based on the true parameter in a CVAR(1) model, which is an extension of the SVAR model with weakly stationary data. The $A$ matrix could in theory be $I$, but since we wish to compare it to the MLS and PCMCI bias-output in the end, we take $A = I$ again. 

In [ ]:
def Bias_CVAR(true_parameter, T, M, lower_bound, upper_bound, start_var, pp_alpha=0.05, standard=True, constant=False, p=1, restricted=True, choose_own_order=False, disable_tqdm=False, seed=None):
    """
    Calculate the bias of the estimator for datasets of T using the estimation for CVAR models.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        T = lenght of the generated dataset that will estimate the parameter using MLS
        M = number of simulations to calculate the mean parameter
        standard = TRUE uses a standard Normal distribution for generating the errors
        constant = FALSE returns only zeros, so that the constant does not influence the time series, TRUE returns random constants
        restricted = if TRUE we make use of the restricted CVAR(1) model
    Returns:
        bias in the form of the mean estimate for the parameter minus the true parameter
    """
    if seed is not None:
        np.random.seed(seed)
    
    estimates = []
    norms_per_sim = []
    
    N, _ = true_parameter.shape
    
    for i in tqdm(range(M), desc=f"Simulating Bias for T={T}", disable=disable_tqdm):
        # Step 2: simulate the data
        Y, _ = CVAR1(T, np.eye(N), true_parameter, standard=standard, constant=constant, lower_bound_Y1=lower_bound, upper_bound_Y1=upper_bound, pp_alpha=pp_alpha, seed=seed)
        df, _ = GenerateDF(Y)
        # Step 3: estimate the parameter
        model, JT_dict = CVAR_ParEst(df, p, start_var, restricted=restricted, plot_skeleton=False, plot_reordered=False, RZP_before_ordering=False, RZP_after_ordering=False, print_JT_dct=False, choose_own_order=choose_own_order)
        # Step 4: store information and repeat 
        CVAR_estimate = np.zeros((N,N))
        CVAR_estimate[:, 0] = model['B']['WPAC-1']
        CVAR_estimate[:, 1] = model['B']['CPAC-1']
        CVAR_estimate[:, 2] = model['B']['EPAC-1']
        CVAR_estimate[:, 3] = model['B']['TW-1']
        estimates.append(CVAR_estimate)
        # And again store the results in a list
        bias_per_sim = CVAR_estimate - true_parameter
        norm_per_sim = np.linalg.norm(bias_per_sim, 'fro')
        norms_per_sim.append(norm_per_sim)
        if seed is not None:
            seed += 1
            
    mean_estimate = np.mean(estimates, axis=0)
    bias = mean_estimate - true_parameter
    bias_value = np.linalg.norm(bias, 'fro')

    return bias, bias_value, norms_per_sim

In [ ]:
biasCVAR, bias_valueCVAR, norms_per_simCVAR = Bias_CVAR(climate_pcmci_phi, 864, 10000, minima_climate, maxima_climate, 'CPAC', standard=True, constant=False, seed=None)
#bias_try, norm_try, norms_per_sim = Bias_CVAR(climate_pcmci_phi, 864, 50, minima_climate, maxima_climate, 'CPAC', pp_alpha=0.05, standard=True, constant=False, p=1, restricted=True, choose_own_order=False, disable_tqdm=False, seed=None)
#print(norm_try)
#print(bias_try)
#print(norms_per_sim)

In [ ]:
print(biasCVAR)

In [ ]:
print(bias_valueCVAR)

In [ ]:
print(norms_per_simCVAR)

The above function will create the bias estimates again, this time using the CVAR parameter estimator. The bias for each iteration in M and the mean bias along with its Frobenius Norms are computed and stored to return as output. Just like for the MLS case, the following function will run this bias computing function for different lengths T of the time series to find out the dynamics. 

In [ ]:
def SimulateBiasCVAR(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, start_var, pp_alpha=0.05, standard=True, constant=False, plot_graph=True, plot_sims=True, restricted=True, choose_own_order=False, seed=None):
    """
    Calculate the bias of the estimator for datasets of T using MLS as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        min_T = the starting point of the time intervals (integer)
        max_T = the ending point of the time intervals (integer)
        intervals = the size of the intervals (integer)
        M = the number of simulations used to calculate the mean bias and norm for each length T
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        start_var = string with the name of the variable we would like to start with (based on climate experts)
        pp_alpha = the significance level of the Pearson-Perron stationarity test
        standard = if TRUE then the errors of the randomly simulated dataset are from a standard Normal distribution; if FALSE then from a random Normal distribution
        constant = if FALSE then the constant is left out of the VAR(1) model
        restricted = if TRUE we make use of the restricted CVAR(1) model
        choose_own_order = if FALSE, the system will choose its own order
        plot_graph = if TRUE then the graph is plotted at the end
        plot_sims = if TRUE then the norms for all the simulations are also shown in the graph
    Returns:
        biases = the matrices containing the bias for each entry of the matrix, for each time T
        norms = the norm of the bias matrices
        df_norms = the dataframe containing the norms for each T (also used for the plot)
    """
    l = (max_T - min_T)//intervals
    biases = []
    norms = np.zeros((l,2))
    all_norms = []
    i = 0    

    "Running the simulations"
    for t in tqdm(range(min_T, max_T, intervals), desc="Simulating Bias"):
        bias, norm, all_norms_T = Bias_CVAR(true_parameter, t, M, lower_bound, upper_bound, start_var=start_var, pp_alpha=pp_alpha, standard=standard, constant=constant,  p=1, restricted=restricted, choose_own_order=choose_own_order, disable_tqdm=True, seed=seed)
        biases.append(bias)
        norms[i,0] = t
        norms[i,1] = norm
        all_norms.append(all_norms_T)
        i += 1      
        
    "Making the graph"
    df_norms = pd.DataFrame(norms, columns=['T','Frobenius Norm'])
    df_norms.set_index('T', inplace=True)
    
    fig = go.Figure()
    fig = px.line(df_norms, x=df_norms.index, y=df_norms.columns, title=f'CVAR estimation - Frobenius Norm of Bias Matrix per time series length T - {M} simulations per T')
    # if we would like to plot the norms that we retrieved for the M iterations:
    if plot_sims == True: 
        for idx, all_norms_T in enumerate(all_norms):
                t = min_T + idx * intervals
                fig.add_trace(go.Scatter(x=[t]*len(all_norms_T), y=list(all_norms_T), mode='markers', marker=dict(color='lightblue', opacity=0.05), showlegend=(idx == 0), name='Norms of Simulations' if t == min_T else None))
            
    fig.update_layout(yaxis_title='Frobenius Norm of Bias')
    fig.update_layout(xaxis_title='Number of time periods T in dataset')
    # Add the trend line to the plot
    x = df_norms.index.values
    y = df_norms['Frobenius Norm'].values
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    slope = z[0]
    fig.add_trace(go.Scatter(x=x, y=p(x), mode='lines', name='Trend Line', line=dict(color='black'))) # achter de kleur, ", dash='dash'"
    fig.add_annotation(x=(((max_T-min_T)/2)+min_T), y=p(((max_T-min_T)/2)+min_T), text=f"Trend Slope: {slope:.4f}", showarrow=True, arrowhead=2, ax=-40, ay=-30, bgcolor="white")

    if plot_graph == True:
        fig.show()
    
    
    return biases, norms, df_norms

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def Bias_CVAR_with_timeout(*args, **kwargs):
    return Bias_CVAR(*args, **kwargs)

def SimulateBiasCVAR2(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, start_var, pp_alpha=0.05, standard=True, constant=False, plot_graph=True, plot_sims=True, restricted=True, choose_own_order=False, seed=None, timeout=120):
    """
    Calculate the bias of the estimator for datasets of T using MLS as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        min_T = the starting point of the time intervals (integer)
        max_T = the ending point of the time intervals (integer)
        intervals = the size of the intervals (integer)
        M = the number of simulations used to calculate the mean bias and norm for each length T
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        start_var = string with the name of the variable we would like to start with (based on climate experts)
        pp_alpha = the significance level of the Pearson-Perron stationarity test
        standard = if TRUE then the errors of the randomly simulated dataset are from a standard Normal distribution; if FALSE then from a random Normal distribution
        constant = if FALSE then the constant is left out of the VAR(1) model
        restricted = if TRUE we make use of the restricted CVAR(1) model
        choose_own_order = if FALSE, the system will choose its own order
        plot_graph = if TRUE then the graph is plotted at the end
        plot_sims = if TRUE then the norms for all the simulations are also shown in the graph
        timeout = maximum allowed time (in seconds) for each iteration
    Returns:
        biases = the matrices containing the bias for each entry of the matrix, for each time T
        norms = the norm of the bias matrices
        df_norms = the dataframe containing the norms for each T (also used for the plot)
    """
    l = (max_T - min_T)//intervals
    biases = []
    norms = np.zeros((l,2))
    all_norms = []
    i = 0    

    "Running the simulations"

    for t in tqdm(range(min_T, max_T, intervals), desc="Simulating Bias"):
        bias = None
        norm = None
        all_norms_T = None
        attempt = 0
        
        while attempt < 5:
            with ThreadPoolExecutor(max_workers=1) as executor:
                future = executor.submit(Bias_CVAR_with_timeout, true_parameter, t, M, lower_bound, upper_bound, start_var=start_var, pp_alpha=pp_alpha, standard=standard, constant=constant, p=1, restricted=restricted, choose_own_order=choose_own_order, disable_tqdm=True, seed=seed)
                
                try:
                    bias, norm, all_norms_T = future.result(timeout=timeout)
                    break
                except Exception as e:
                    attempt += 1
                    print(f"Iteration for T={t} took too long or failed, retrying with a different seed (Attempt {attempt})")
                    if seed is not None:
                        seed += 1

        if bias is None or norm is None or all_norms_T is None:
            print(f"Failed to compute Bias for T={t} after 5 attempts.")
            continue

        biases.append(bias)
        norms[i,0] = t
        norms[i,1] = norm
        all_norms.append(all_norms_T)
        i += 1      
        
    "Making the graph"
    df_norms = pd.DataFrame(norms, columns=['T','Frobenius Norm'])
    df_norms.set_index('T', inplace=True)
    
    fig = go.Figure()
    fig = px.line(df_norms, x=df_norms.index, y=df_norms.columns, title=f'CVAR estimation - Frobenius Norm of Bias Matrix per time series length T - {M} simulations per T')
    # if we would like to plot the norms that we retrieved for the M iterations:
    if plot_sims == True: 
        for idx, all_norms_T in enumerate(all_norms):
                t = min_T + idx * intervals
                fig.add_trace(go.Scatter(x=[t]*len(all_norms_T), y=list(all_norms_T), mode='markers', marker=dict(color='blue', opacity=0.05), showlegend=(idx == 0), name='Norms of Simulations' if idx == 0 else None))
            
    fig.update_layout(yaxis_title='Frobenius Norm of Bias')
    fig.update_layout(xaxis_title='Number of time periods T in dataset')
    # Add the trend line to the plot
    x = df_norms.index.values
    y = df_norms['Frobenius Norm'].values
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    fig.add_trace(go.Scatter(x=x, y=p(x), mode='lines', name='Trend Line', line=dict(color='black', dash='dash')))
    
    if plot_graph == True:
        fig.show()
    
    return biases, norms, df_norms

In [ ]:
biases, norms, df_norms = SimulateBiasCVAR(climate_pcmci_phi, 800, 5200, 50, 100, minima_climate, maxima_climate, 'CPAC', standard=True, constant=False, plot_graph=True, plot_sims=True, restricted=True, choose_own_order=False, seed=1`)
# def SimulateBiasCVAR(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, start_var, standard=True, constant=False, plot_graph=True, plot_sims=True, restricted=True, choose_own_order=False, seed=None):


In [ ]:
biases, norms, df_norms = SimulateBiasCVAR2(climate_pcmci_phi, 800, 1600, 50, 100, minima_climate, maxima_climate, 'CPAC', standard=True, constant=False, plot_graph=True, plot_sims=True, restricted=True, choose_own_order=False, seed=None)


## PCMCI

The one-to-final estimator of which the bias is investigated is the PCMCI algorithm output. Here data is based on a CVAR(1) model again, meaning it will be generated by an SVAR(1) model with the restriction of weak stationarity. Since the PCMCI algorithm will return $\mathbf{val\_matrix} = \Phi$, we say that the matrix $A$ which shows the structural relations at timelag zero is equal to the identity matrix $A = I$. 

In [ ]:
def Bias_PCMCI(true_parameter, T, M, lower_bound, upper_bound, test, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask = True, standard=True, 
               constant=False, show_DAG=True, disable_tqdm=False, seed=None):
    """
    Calculate the bias of the estimator for datasets of T using the estimation for CVAR models.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        T = lenght of the generated dataset that will estimate the parameter using MLS
        M = number of simulations to calculate the mean parameter
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        test = conditional independence test we use (string)
        pp_alpha = the significance level of the Pearson-Perron stationarity test
        pc_alpha = significance level in algorithm
        alpha_level = significance level at which the p_matrix is thresholded to get graph
        with_mask = TRUE if we want to apply the spring barrier mask
        standard = TRUE uses a standard Normal distribution for generating the errors
        constant = FALSE returns only zeros, so that the constant does not influence the time series, TRUE returns random constants
    Returns:
        bias in the form of the mean estimate for the parameter minus the true parameter
    """
    if seed is not None:
        np.random.seed(seed)
    
    estimates = []
    norms_per_sim = []
    
    N, _ = true_parameter.shape

    for i in tqdm(range(M), desc=f"Simulating Bias for T={T}", ncols=100, disable=disable_tqdm):
        # Step 2: simulate the data
        Y, _ = CVAR1(T+100, np.eye(N), true_parameter, standard=standard, constant=constant, lower_bound_Y1=lower_bound, upper_bound_Y1=upper_bound, pp_alpha=pp_alpha, seed=seed)
        df_full, _ = GenerateDF(Y)
        df = df_full.iloc[100:]  # Skip the first 100 data points for a .. break (niet de berlin maar die andere naam haha)
        # Step 3: estimate the parameter
        results = RunPCMCI(df, test, tau_max=tau_max, tau_min=tau_min, pc_alpha=pc_alpha, alpha_level=alpha_level, with_mask = with_mask, model = '', show_DAG=show_DAG, var_names = ['WPAC', 'CPAC', 'EPAC', 'TW'])
        # Step 4: store information and repeat 
        PCMCI_estimate = ConstructPhiMatrix(results['val_matrix'])
        estimates.append(PCMCI_estimate)
        # And again store the results in a list
        bias_per_sim = PCMCI_estimate - true_parameter
        norm_per_sim = np.linalg.norm(bias_per_sim, 'fro')
        norms_per_sim.append(norm_per_sim)
        if seed is not None:
            seed += 1
        
    mean_estimate = np.mean(estimates, axis=0)
    bias = mean_estimate - true_parameter
    bias_value = np.linalg.norm(bias, 'fro')

    return bias, bias_value, norms_per_sim

In [ ]:
bias_tryPCMCI, norm_tryPCMCI, norms_per_simPCMCI = Bias_PCMCI(climate_pcmci_phi, 864, 10000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask = True, standard=True, constant=False, show_DAG=False, disable_tqdm=False, seed=None)
#def Bias_PCMCI(true_parameter, T, M, lower_bound, upper_bound, test, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask = True, standard=True, 
      #         constant=False, show_DAG=True, disable_tqdm=False, seed=None)
    
##bias_try, norm_try, norms_per_sim = Bias_CVAR(climate_pcmci_phi, 864, 50, minima_climate, maxima_climate, 'CPAC', pp_alpha=0.05, standard=True, constant=False, p=1, restricted=True, choose_own_order=False, disable_tqdm=False, seed=None)

In [ ]:
print(norm_tryPCMCI)

In [ ]:
print(bias_tryPCMCI)

In [ ]:
print(norms_per_simPCMCI)

In [ ]:
bias_tryPCMCI2, norm_tryPCMCI2, norms_per_simPCMCI2 = Bias_PCMCI(climate_pcmci_phi, 2000, 10000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask = True, standard=True, constant=False, show_DAG=False, disable_tqdm=False, seed=None)


In [ ]:
print(norm_tryPCMCI2)

In [ ]:
print(bias_tryPCMCI2)

In [ ]:
print(norms_per_simPCMCI2)

In [ ]:
bias_tryPCMCI3, norm_tryPCMCI3, norms_per_simPCMCI3 = Bias_PCMCI(climate_pcmci_phi, 600, 10000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask = True, standard=True, constant=False, show_DAG=False, disable_tqdm=False, seed=None)


In [ ]:
print(norm_tryPCMCI3)

In [ ]:
print(bias_tryPCMCI3)

In [ ]:
print(norms_per_simPCMCI3)

In [ ]:
bias_tryPCMCI4, norm_tryPCMCI4, norms_per_simPCMCI4 = Bias_PCMCI(climate_pcmci_phi, 10000, 10000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask = True, standard=True, constant=False, show_DAG=False, disable_tqdm=False, seed=None)


In [ ]:
print(norm_tryPCMCI4)

In [ ]:
print(bias_tryPCMCI4)

In [ ]:
print(norms_per_simPCMCI4)

In [ ]:
bias_tryPCMCI5, norm_tryPCMCI5, norms_per_simPCMCI5 = Bias_PCMCI(climate_pcmci_phi, 432, 10000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask = True, standard=True, constant=False, show_DAG=False, disable_tqdm=False, seed=None)


In [ ]:
print(norm_tryPCMCI5)

In [ ]:
print(bias_tryPCMCI5)

In [ ]:
print(norms_per_simPCMCI5)

In [ ]:
def SimulateBiasPCMCI(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, test, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None):
    """
    Calculate the bias of the estimator for datasets of T using MLS as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        min_T = the starting point of the time intervals (integer)
        max_T = the ending point of the time intervals (integer)
        intervals = the size of the intervals (integer)
        M = the number of simulations used to calculate the mean bias and norm for each length T
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        start_var = string with the name of the variable we would like to start with (based on climate experts)
        pp_alpha = the significance level of the Pearson-Perron stationarity test
        standard = if TRUE then the errors of the randomly simulated dataset are from a standard Normal distribution; if FALSE then from a random Normal distribution
        constant = if FALSE then the constant is left out of the VAR(1) model
        restricted = if TRUE we make use of the restricted CVAR(1) model
        choose_own_order = if FALSE, the system will choose its own order
        plot_graph = if TRUE then the graph is plotted at the end
        plot_sims = if TRUE then the norms for all the simulations are also shown in the graph
    Returns:
        biases = the matrices containing the bias for each entry of the matrix, for each time T
        norms = the norm of the bias matrices
        df_norms = the dataframe containing the norms for each T (also used for the plot)
    """
    l = (max_T - min_T)//intervals
    biases = []
    norms = np.zeros((l,2))
    all_norms = []
    i = 0    

    
    "Running the simulations"
    for t in tqdm(range(min_T, max_T, intervals), desc="Simulating Bias"):
        bias, norm, all_norms_T = Bias_PCMCI(true_parameter, t, M, lower_bound, upper_bound, test, pp_alpha=pp_alpha, pc_alpha=pc_alpha, alpha_level=alpha_level, 
                                             tau_max=tau_max, tau_min=tau_min, with_mask = with_mask, standard=standard, constant=constant, show_DAG=False, disable_tqdm=True, seed=seed)
        biases.append(bias)
        norms[i,0] = t
        norms[i,1] = norm
        all_norms.append(all_norms_T)
        i += 1   

        
    "Making the graph"
    df_norms = pd.DataFrame(norms, columns=['T','Frobenius Norm'])
    df_norms.set_index('T', inplace=True)
    
    fig = go.Figure()
    fig = px.line(df_norms, x=df_norms.index, y=df_norms.columns, title=f'Tigramite estimation - Frobenius Norm of Bias Matrix per time series length T - {M} simulations per T')
    # if we would like to plot the norms that we retrieved for the M iterations:
    if plot_sims == True: 
        for idx, all_norms_T in enumerate(all_norms):
                t = min_T + idx * intervals
                fig.add_trace(go.Scatter(x=[t]*len(all_norms_T), y=list(all_norms_T), mode='markers', marker=dict(color='lightblue', opacity=0.05), showlegend=(idx == 0), name='Norms of Simulations' if t == min_T else None))
            
    fig.update_layout(yaxis_title='Frobenius Norm of Bias')
    fig.update_layout(xaxis_title='Number of time periods T in dataset')
    # Add the trend line to the plot
    x = df_norms.index.values
    y = df_norms['Frobenius Norm'].values
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    slope = z[0]
    fig.add_trace(go.Scatter(x=x, y=p(x), mode='lines', name='Trend Line', line=dict(color='black'))) #, dash='dash'
    fig.add_annotation(x=(((max_T-min_T)/2)+min_T), y=p(((max_T-min_T)/2)+min_T), text=f"Trend Slope: {slope:.4f}", showarrow=True, arrowhead=2, ax=-40, ay=-30, bgcolor="white")

    if plot_graph == True:
        fig.show()
    
    
    return biases, norms, df_norms

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

def SimulateBiasPCMCI2(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, test, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None):
    """
    Calculate the bias of the estimator for datasets of T using PCMCI as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        min_T = the starting point of the time intervals (integer)
        max_T = the ending point of the time intervals (integer)
        intervals = the size of the intervals (integer)
        M = the number of simulations used to calculate the mean bias and norm for each length T
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        pp_alpha = the significance level of the Pearson-Perron stationarity test
        standard = if TRUE then the errors of the randomly simulated dataset are from a standard Normal distribution; if FALSE then from a random Normal distribution
        constant = if FALSE then the constant is left out of the VAR(1) model
        plot_graph = if TRUE then the graph is plotted at the end
        plot_sims = if TRUE then the norms for all the simulations are also shown in the graph
    Returns:
        biases = the matrices containing the bias for each entry of the matrix, for each time T
        norms = the norm of the bias matrices
        df_norms = the dataframe containing the norms for each T (also used for the plot)
    """
    l = (max_T - min_T) // intervals
    biases = []
    norms = np.zeros((l, 2))
    all_norms = []
    i = 0  
        
    colors = sns.color_palette("colorblind")
    purple = colors[3]
    green = colors[2]
    orange = colors[1]
    lightblue = colors[0]

    # Running the simulations
    for t in tqdm(range(min_T, max_T, intervals), desc="Simulating Bias"):
        bias, norm, all_norms_T = Bias_PCMCI(true_parameter, t, M, lower_bound, upper_bound, test, pp_alpha=pp_alpha, pc_alpha=pc_alpha, alpha_level=alpha_level, 
                                             tau_max=tau_max, tau_min=tau_min, with_mask=with_mask, standard=standard, constant=constant, show_DAG=False, disable_tqdm=True, seed=seed)
        biases.append(bias)
        norms[i, 0] = t
        norms[i, 1] = norm
        all_norms.append(all_norms_T)
        i += 1   

    # Making the graph
    df_norms = pd.DataFrame(norms, columns=['T', 'Frobenius Norm'])
    df_norms.set_index('T', inplace=True)

    # Plotting with Seaborn and Matplotlib
    if plot_graph:
        plt.figure(figsize=(12, 6))
        
        # Plot the Frobenius Norm line in green
        sns.lineplot(data=df_norms, x=df_norms.index, y='Frobenius Norm', color=green, label='Frobenius Norm')
        
        if plot_sims:
            for idx, all_norms_T in enumerate(all_norms):
                t = min_T + idx * intervals
                sns.scatterplot(x=[t] * len(all_norms_T), y=all_norms_T, color=lightblue, alpha=0.05, label='Norms of Simulations' if idx == 0 else "")

        # Add the trend line to the plot
        x = df_norms.index.values
        y = df_norms['Frobenius Norm'].values
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        slope = z[0]
        sns.lineplot(x=x, y=p(x), color=orange, linestyle='--', label='Trend Line')

        plt.title('Frobenius Norm of Bias')
        plt.xlabel('Number of time periods T in dataset')
        plt.ylabel('Frobenius Norm of Bias')
        plt.legend(fontsize='large')
        plt.annotate(f"Trend Slope: {slope:.4f}", xy=(x[len(x) // 2], p(x[len(x) // 2])), xytext=(0, 50),
                     textcoords='offset points', arrowprops=dict(arrowstyle='->', color='black', lw=1.5), backgroundcolor='white', fontsize=12)
        plt.tight_layout()
        plt.show()

    return biases, norms, df_norms

In [ ]:
biases, norms, df_norms = SimulateBiasPCMCI2(climate_pcmci_phi, 600, 3000, 50, 1000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None)
# DEZE IS NOG MET DE 
#def SimulateBiasPCMCI(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, test, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None):
#biasesMLSgraph, normsMLSgraph, df_normsMLSgraph = SimulateBiasMLS(climate_pcmci_phi, 500, 4000, 50, 5000, minima_climate, maxima_climate, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None)

In [ ]:
biases, norms, df_norms = SimulateBiasPCMCI2(climate_pcmci_phi, 600, 4000, 50, 1000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None)
#def SimulateBiasPCMCI(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, test, pp_alpha=0.05, pc_alpha=0.05, alpha_level=0.01, tau_max=1, tau_min=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None):
#biasesMLSgraph, normsMLSgraph, df_normsMLSgraph = SimulateBiasMLS(climate_pcmci_phi, 500, 4000, 50, 5000, minima_climate, maxima_climate, standard=True, constant=False, plot_graph=True, plot_sims=True, seed=None)
# TILLIE IDK OF DEZE OOK RUNNEN ALS TIJD!!!

In [ ]:
print(df_norms)

In [ ]:
colors = sns.color_palette("colorblind")
purple = colors[3]
green = colors[2]
orange = colors[1]
lightblue = colors[0]
    
plt.figure(figsize=(12, 6))
        
        # Plot the Frobenius Norm line in green
sns.lineplot(data=df_norms, x=df_norms.index, y='Frobenius Norm', color=green, label='Frobenius Norm')
        
for idx, all_norms_T in enumerate(all_norms):
    t = 600 + idx * intervals
    sns.scatterplot(x=[t] * len(all_norms_T), y=all_norms_T, color=lightblue, alpha=0.05, label='Norms of Simulations' if idx == 0 else "")

        # Add the trend line to the plot
x = df_norms.index.values
y = df_norms['Frobenius Norm'].values
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
slope = z[0]
sns.lineplot(x=x, y=p(x), color=orange, linestyle='--', label='Trend Line')

plt.title('Frobenius Norm of Bias')
plt.xlabel('Number of time periods T in dataset')
plt.ylabel('Frobenius Norm of Bias')
plt.legend(fontsize='large')
plt.annotate(f"Trend Slope: {slope:.4f}", xy=(x[len(x) // 2], p(x[len(x) // 2])), xytext=(0, 50),
                     textcoords='offset points', arrowprops=dict(arrowstyle='->', color='black', lw=1.5), backgroundcolor='white', fontsize=12)
plt.tight_layout()
plt.show()

## Bagging

The final estimator of which the bias is investigated is the Bagging algorithm output. Just like with PCMCI, the data is based on a CVAR(1) model, meaning it will be generated by an SVAR(1) model with the restriction of weak stationarity. Since the PCMCI algorithm will return $\mathbf{val\_matrix} = \Phi$, we say that the matrix $A$ which shows the structural relations at timelag zero is equal to the identity matrix $A = I$. 

In [ ]:
def Bias_Bagging(true_parameter, T, M, lower_bound, upper_bound, test, pp_alpha=0.05, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, pc_alpha=None, 
                 alpha_level=0.01, tau_max=1, with_mask = True, standard=True, constant=False, disable_tqdm=False, var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None):
    """
    Calculate the bias of the estimator for datasets of T using the estimation for CVAR models.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        T = lenght of the generated dataset that will estimate the parameter using MLS
        M = number of simulations to calculate the mean parameter
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        test = conditional independence test we use (string)
        pp_alpha = the significance level of the Pearson-Perron stationarity test
        pc_alpha = significance level in algorithm
        alpha_level = significance level at which the p_matrix is thresholded to get graph
        with_mask = TRUE if we want to apply the spring barrier mask
        standard = TRUE uses a standard Normal distribution for generating the errors
        constant = FALSE returns only zeros, so that the constant does not influence the time series, TRUE returns random constants
    Returns:
        bias in the form of the mean estimate for the parameter minus the true parameter
    """
    if seed is not None:
        np.random.seed(seed)
    
    estimates = []
    norms_per_sim = []
    
    N, _ = true_parameter.shape

    for i in tqdm(range(M), desc=f"Simulating Bias for T={T}", ncols=100, disable=disable_tqdm):
        # Step 2: simulate the data
        Y, _ = CVAR1(T+100, np.eye(N), true_parameter, standard=standard, constant=constant, lower_bound_Y1=lower_bound, upper_bound_Y1=upper_bound, pp_alpha=pp_alpha, seed=seed)
        df_full, _ = GenerateDF(Y)
        df = df_full.iloc[100:]  # Skip the first 100 data points for a .. break (niet de berlin maar die andere naam haha)
        # Step 3: estimate the parameter
        results = RunBagging(df, test, tau_max=tau_max, boot_samples=boot_samples, boot_blocklength=boot_blocklength, pc_alpha_bootstrap=pc_alpha_bootstrap,
                             pc_alpha=pc_alpha, plot=False, model='', with_mask = with_mask, var_names=var_names, seed=seed)
        # Step 4: store information and repeat 
        Bagging_estimate = ConstructPhiMatrixBagging(results['summary_results']['val_matrix_mean'])
        estimates.append(Bagging_estimate)
        # And again store the results in a list
        bias_per_sim =  Bagging_estimate - true_parameter
        norm_per_sim = np.linalg.norm(bias_per_sim, 'fro')
        norms_per_sim.append(norm_per_sim)
        if seed is not None:
            seed += 1    
        
    mean_estimate = np.mean(estimates, axis=0)
    bias = mean_estimate - true_parameter
    bias_value = np.linalg.norm(bias, 'fro')

    return bias, bias_value, norms_per_sim

In [ ]:
bias_tryBagging, norm_tryBagging, norms_per_simBagging = Bias_Bagging(climate_pcmci_phi, 864, 1000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, boot_samples=100, boot_blocklength=1, pc_alpha_bootstrap=0.05, pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask = True, standard=True, constant=False, disable_tqdm=False, var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)
#Bias_Bagging(true_parameter, T, M, lower_bound, upper_bound, test, pp_alpha=0.05, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, pc_alpha=None, 
       #          alpha_level=0.01, tau_max=1, with_mask = True, standard=True, constant=False, disable_tqdm=False, var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None):

In [ ]:
print(bias_tryBagging)

In [ ]:
print(norm_tryBagging)

In [ ]:
print(norms_per_simBagging)

In [ ]:
bias1Bagging, norm1Bagging, norms_per_sim1Bagging = Bias_Bagging(climate_pcmci_phi, 432, 1000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, boot_samples=100, boot_blocklength=1, pc_alpha_bootstrap=0.05, pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask = True, standard=True, constant=False, disable_tqdm=False, var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)


In [ ]:
print(bias1Bagging)

In [ ]:
print(norm1Bagging)
# met -100 dingen; 0.2059932982095331055

In [ ]:
print(norms_per_sim1Bagging)

In [ ]:
bias2Bagging, norm2Bagging, norms_per_sim2Bagging = Bias_Bagging(climate_pcmci_phi, 2000, 1000, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, boot_samples=100, boot_blocklength=1, pc_alpha_bootstrap=0.05, pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask = True, standard=True, constant=False, disable_tqdm=False, var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)


In [ ]:
print(bias2Bagging)
# met -100 dingen: 0.19859620377587203

In [ ]:
print(norm2Bagging)

In [ ]:
print(norms_per_sim2Bagging)

In [ ]:
 def SimulateBiasBagging(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, test, pp_alpha=0.05, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, 
                        pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, 
                        var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None):
    """
    Calculate the bias of the estimator for datasets of T using MLS as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        min_T = the starting point of the time intervals (integer)
        max_T = the ending point of the time intervals (integer)
        intervals = the size of the intervals (integer)
        M = the number of simulations used to calculate the mean bias and norm for each length T
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        start_var = string with the name of the variable we would like to start with (based on climate experts)
        pp_alpha = the significance level of the Pearson-Perron stationarity test
        standard = if TRUE then the errors of the randomly simulated dataset are from a standard Normal distribution; if FALSE then from a random Normal distribution
        constant = if FALSE then the constant is left out of the VAR(1) model
        restricted = if TRUE we make use of the restricted CVAR(1) model
        choose_own_order = if FALSE, the system will choose its own order
        plot_graph = if TRUE then the graph is plotted at the end
        plot_sims = if TRUE then the norms for all the simulations are also shown in the graph
    Returns:
        biases = the matrices containing the bias for each entry of the matrix, for each time T
        norms = the norm of the bias matrices
        df_norms = the dataframe containing the norms for each T (also used for the plot)
    """
    l = (max_T - min_T)//intervals
    biases = []
    norms = np.zeros((l,2))
    all_norms = []
    i = 0    

    
    "Running the simulations"
    for t in tqdm(range(min_T, max_T, intervals), desc="Simulating Bias"):
        bias, norm, all_norms_T = Bias_Bagging(true_parameter, t, M, lower_bound, upper_bound, test, pp_alpha=pp_alpha, boot_samples=boot_samples, boot_blocklength=boot_blocklength, 
                                               pc_alpha_bootstrap=pc_alpha_bootstrap, pc_alpha=pc_alpha, alpha_level=alpha_level, tau_max=tau_max, with_mask = with_mask, 
                                               standard=standard, constant=constant, disable_tqdm=True, var_names=var_names, disable_tqdm=True, seed=seed)
        biases.append(bias)
        norms[i,0] = t
        norms[i,1] = norm
        all_norms.append(all_norms_T)
        i += 1   

        
    "Making the graph"
    df_norms = pd.DataFrame(norms, columns=['T','Frobenius Norm'])
    df_norms.set_index('T', inplace=True)
    
    fig = go.Figure()
    fig = px.line(df_norms, x=df_norms.index, y=df_norms.columns, title=f'Bootstrap Aggregation estimation - Frobenius Norm of Bias Matrix per time series length T - {M} simulations per T')
    # if we would like to plot the norms that we retrieved for the M iterations:
    if plot_sims == True: 
        for idx, all_norms_T in enumerate(all_norms):
                t = min_T + idx * intervals
                fig.add_trace(go.Scatter(x=[t]*len(all_norms_T), y=list(all_norms_T), mode='markers', marker=dict(color='blue', opacity=0.05), showlegend=(idx == 0), name='Norms of Simulations' if t == min_T else None))
            
    fig.update_layout(yaxis_title='Frobenius Norm of Bias')
    fig.update_layout(xaxis_title='Number of time periods T in dataset')
    # Add the trend line to the plot
    x = df_norms.index.values
    y = df_norms['Frobenius Norm'].values
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    slope = z[0]
    fig.add_trace(go.Scatter(x=x, y=p(x), mode='lines', name='Trend Line', line=dict(color='black'))) #, dash='dash'
    fig.add_annotation(x=(((max_T-min_T)/2)+min_T), y=p(((max_T-min_T)/2)+min_T), text=f"Trend Slope: {slope:.4f}", showarrow=True, arrowhead=2, ax=-40, ay=-30, bgcolor="white")

    if plot_graph == True:
        fig.show()
    
    
    return biases, norms, df_norms

In [ ]:
def SimulateBiasBagging2(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, test, pp_alpha=0.05, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, 
                        pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, 
                        var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None):
    """
    Calculate the bias of the estimator for datasets of T using Bagging as an estimator.
    Parameters:
        true_parameter = the input parameter that is used to generate the dataset and to compare the estimated parameter to
        min_T = the starting point of the time intervals (integer)
        max_T = the ending point of the time intervals (integer)
        intervals = the size of the intervals (integer)
        M = the number of simulations used to calculate the mean bias and norm for each length T
        lower_bound = the starting lower bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        upper_bound = the starting upper bound of the randomly simulated dataset (based on VAR(1) model) (integer or list)
        pp_alpha = the significance level of the Pearson-Perron stationarity test
        standard = if TRUE then the errors of the randomly simulated dataset are from a standard Normal distribution; if FALSE then from a random Normal distribution
        constant = if FALSE then the constant is left out of the VAR(1) model
        plot_graph = if TRUE then the graph is plotted at the end
        plot_sims = if TRUE then the norms for all the simulations are also shown in the graph
    Returns:
        biases = the matrices containing the bias for each entry of the matrix, for each time T
        norms = the norm of the bias matrices
        df_norms = the dataframe containing the norms for each T (also used for the plot)
    """
    l = (max_T - min_T) // intervals
    biases = []
    norms = np.zeros((l, 2))
    all_norms = []
    i = 0    
    
    colors = sns.color_palette("colorblind")
    purple = colors[3]
    green = colors[2]
    orange = colors[1]
    
    # Running the simulations
    for t in tqdm(range(min_T, max_T, intervals), desc="Simulating Bias"):
        bias, norm, all_norms_T = Bias_Bagging(true_parameter, t, M, lower_bound, upper_bound, test, pp_alpha=pp_alpha, boot_samples=boot_samples, boot_blocklength=boot_blocklength, 
                                               pc_alpha_bootstrap=pc_alpha_bootstrap, pc_alpha=pc_alpha, alpha_level=alpha_level, tau_max=tau_max, with_mask=with_mask, 
                                               standard=standard, constant=constant, disable_tqdm=True, var_names=var_names, seed=seed)
        biases.append(bias)
        norms[i, 0] = t
        norms[i, 1] = norm
        all_norms.append(all_norms_T)
        i += 1   

    # Making the graph
    df_norms = pd.DataFrame(norms, columns=['T', 'Frobenius Norm'])
    df_norms.set_index('T', inplace=True)

    # Plotting with Seaborn and Matplotlib
    if plot_graph:
        plt.figure(figsize=(12, 6))
        
        # Plot the Frobenius Norm line in green
        sns.lineplot(data=df_norms, x=df_norms.index, y='Frobenius Norm', color=purple, label='Frobenius Norm')
        
        if plot_sims:
            for idx, all_norms_T in enumerate(all_norms):
                t = min_T + idx * intervals
                sns.scatterplot(x=[t] * len(all_norms_T), y=all_norms_T, color=orange, alpha=0.05, label='Norms of Simulations' if idx == 0 else "")

        # Add the trend line to the plot
        x = df_norms.index.values
        y = df_norms['Frobenius Norm'].values
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        slope = z[0]
        sns.lineplot(x=x, y=p(x), color='grey', linestyle='--', label='Trend Line')

        plt.title('Frobenius Norm of Bias')
        plt.xlabel('Number of time periods T in dataset')
        plt.ylabel('Frobenius Norm of Bias')
        plt.legend(fontsize='large')
        plt.annotate(f"Trend Slope: {slope:.4f}", xy=(x[len(x) // 2], p(x[len(x) // 2])), xytext=(0, 50),
                     textcoords='offset points', arrowprops=dict(arrowstyle='->', color='black', lw=1.5), backgroundcolor='white', fontsize=12)
        plt.tight_layout()
        plt.show()

    return biases, norms, df_norms

In [ ]:
biases, norms, _ = SimulateBiasBagging(climate_pcmci_phi, 800, 5000, 100, 8, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, 
                        pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, 
                        var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)

In [ ]:
biases, norms, _ = SimulateBiasBagging2(climate_pcmci_phi, 800, 1000, 50, 100, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, boot_samples=100, boot_blocklength=1, pc_alpha_bootstrap=0.05, 
                        pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, 
                        var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)

#SimulateBiasBagging(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, test, pp_alpha=0.05, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, 
            #            pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, 
             #           var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)

In [ ]:
biases, norms, _ = SimulateBiasBagging2(climate_pcmci_phi, 600, 3000, 100, 100, minima_climate, maxima_climate, parcorr, pp_alpha=0.05, boot_samples=100, boot_blocklength=1, pc_alpha_bootstrap=0.05, 
                        pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, 
                        var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)

#SimulateBiasBagging(true_parameter, min_T, max_T, intervals, M, lower_bound, upper_bound, test, pp_alpha=0.05, boot_samples=200, boot_blocklength=1, pc_alpha_bootstrap=0.05, 
            #            pc_alpha=None, alpha_level=0.01, tau_max=1, with_mask=True, standard=True, constant=False, plot_graph=True, plot_sims=True, 
             #           var_names=['WPAC', 'CPAC', 'EPAC', 'TW'], seed=None)

## Z-test

The bias norms can be tested to be statistically significantly different from zero by a simple z-test. The test is conducted by setting the 'true value' to zero, and checking whether the z-statistic is smaller than 2.576 (the critical z-value at 99% confidence). We find that this holds for every estimator.

$z = \frac{\bar{x} - \mu}{\sigma / M}$

In [ ]:
M = 10000
m = np.sqrt(M)
print(m)

### MLS

In [ ]:
norms_MLS = [0.509639613483956, 0.5151823569364306, 0.4979804166713629, 0.49154706128874726, 0.5226031753655092, 0.5017990824791546, 0.48779307656052184, 0.47817978800201516, 0.5022884284384644, 0.5197338226528508, 0.5218500802580341, 0.49146678966207386, 0.5227940310445647, 0.5157407386289622, 0.4814629540981384, 0.5170709486299068, 0.4761323676266077, 0.46080764011447595, 0.4841242357204462, 0.5071909832099651, 0.49390526722183, 0.4940860734230168, 0.5098858385710504, 0.5168595911859167, 0.4976706881897352, 0.46739491423922236, 0.5038202377429424, 0.4672379169151019, 0.5180735330009136, 0.5359617467125093, 0.5130001051855592, 0.5026617458375231, 0.513809985545983, 0.5074926065109961, 0.4603475211346464, 0.4896651626795731, 0.49363796975573354, 0.5159269735909903, 0.4468952597349325, 0.48595371820649086, 0.48644819874916806, 0.47517119537709945, 0.5248141410211179, 0.5417838445848845, 0.47192486261884325, 0.4578567477361566, 0.5018957982182174, 0.516154066037057, 0.5138650832118579, 0.48478898219478217, 0.5282584861571267, 0.541012377341302, 0.5146270441828036, 0.4962644849178571, 0.4972535195452294, 0.5060388800663919, 0.5062758835992305, 0.5153557035119484, 0.4767588380154091, 0.5068105402253376, 0.4872202948720744, 0.5047824104216245, 0.5073346675895699, 0.47971138200453395, 0.5642526468682842, 0.4989598939336626, 0.4574849995629738, 0.5412901391713443, 0.47439527231977613, 0.5097035515490207, 0.47223213308774786, 0.4841032499319461, 0.5004870836527453, 0.5181774181938194, 0.5017054321084795, 0.49947169328392865, 0.5182110724635167, 0.47290976142591595, 0.44603785445696204, 0.5451896024687861, 0.5342724868215594, 0.4712712153060295, 0.46857934500500514, 0.47053011556761065, 0.5123189145400756, 0.5221155973067982, 0.5383568308167541, 0.5185840022881112, 0.5171012372275494, 0.48534469878691205, 0.5005344040717603, 0.4934446091854696, 0.5283809694628522, 0.5540372248051089, 0.4527919545419394, 0.5169996775438465, 0.549701319592862, 0.47440748874601935, 0.5175913716176086, 0.5321940912907904, 0.4631607710998482, 0.48791071886418863, 0.4991550700378523, 0.5010289035821414, 0.5256807352223349, 0.4658211131312813, 0.47345881111936416, 0.48369930017914026, 0.5103059268019687, 0.5152498332474121, 0.5006784378175728, 0.4790354896416432, 0.4638791057685744, 0.5073116731892569, 0.493666780108529, 0.5010857581046502, 0.46337737236388993, 0.47254102983812735, 0.4529165840682385, 0.5001783183798311, 0.5073734558041703, 0.47405060824897866, 0.4815980764139174, 0.5621380279292169, 0.48226356648694846, 0.5079314461982942, 0.4894073454435678, 0.4649981643838299, 0.4538204163373401, 0.5225685218616949, 0.48127336034397994, 0.5175259956164321, 0.5077440288074235, 0.529945996043059, 0.4778502164694691, 0.5435480922683747, 0.49029596706001577, 0.48864863466025016, 0.4308523876247655, 0.5255597681268577, 0.5086659319846584, 0.48942915957015337, 0.46174365751487406, 0.47932797169581, 0.48777641818575973, 0.4891117321645068, 0.52125098385332, 0.4871393701193899, 0.5148353868806459, 0.4795448339284236, 0.48577117452858176, 0.4821532454016567, 0.4943630492556166, 0.4917568299698228, 0.5333355535953759, 0.5022547145067084, 0.5334152820605546, 0.464931496120296, 0.4823775608181214, 0.47692039835371886, 0.510500401222014, 0.48653064121558676, 0.4953304991724202, 0.47617498553704896, 0.502559648377147, 0.5214583130321743, 0.4266832043603202, 0.47085378293999053, 0.4972746322726199, 0.46394672102917955, 0.492373731525812, 0.47291307916777614, 0.502318895161864, 0.5271662959904047, 0.5099010236523439, 0.49662819071463987, 0.5198124685616358, 0.5078101829316536, 0.49907096459196054, 0.4866817201320715, 0.5015864194518906, 0.5148435662649724, 0.46726356271497604, 0.4763435564304297, 0.48874450292120564, 0.5117059030124208, 0.5382997982492285, 0.4930235726797209, 0.5452694665667938, 0.5360999003330434, 0.544473905163938, 0.5080067838302786, 0.5367292821556842, 0.4715083784065136, 0.4585105448345174, 0.46160963387720977, 0.476825011700407, 0.48677699724405377, 0.46488524647038026, 0.4900958870837132, 0.48501304013776403, 0.492040539888385, 0.5049111918058963, 0.4724307843235433, 0.4724954090322196, 0.49241481442038604, 0.5026959048762435, 0.503289394812832, 0.4520821323151231, 0.5086938264808881, 0.4636792061334519, 0.4667785002622052, 0.4965277263522264, 0.5289858793390899, 0.48713619369127176, 0.48391554574138085, 0.4938101989208228, 0.47326528470100443, 0.4897908121594883, 0.5200650436542809, 0.4848679166115489, 0.49230648453039394, 0.5047648940876222, 0.5234383702405452, 0.5181838706555358, 0.500946151687202, 0.5325629642739076, 0.5284804650769398, 0.5186576184329993, 0.5031000193720055, 0.49162600898751696, 0.4911344648792971, 0.4893754685704081, 0.4883862914761743, 0.4938315032125873, 0.49409518624951043, 0.4978319044234477, 0.5022726837759223, 0.5167890640881068, 0.47628811033413554, 0.44792001520996266, 0.5288544632364534, 0.49540042482054386, 0.5371619727400774, 0.4923332541471515, 0.5320590474183875, 0.5211202499693494, 0.48617216212794145, 0.49994856899993695, 0.5190560737033447, 0.5178824055257789, 0.4691420093943165, 0.5336014194943858, 0.4936978315558379, 0.4848499473741176, 0.5122221809656076, 0.4725043718892173, 0.5153891798300355, 0.46452374678117236, 0.5317434204835593, 0.4981886259199696, 0.48219727130127704, 0.4635816378400697, 0.5141057442349907, 0.5186932417369671, 0.5070985138174319, 0.4966971751810086, 0.5018603800664094, 0.452069254696644, 0.5159258359414726, 0.5012286630670119, 0.45261709832347913, 0.5020618869980225, 0.4881741413680981, 0.5225031641114211, 0.5127293246046163, 0.5348225603382248, 0.4628692036476925, 0.5405857474565043, 0.539648440210849, 0.45236163387194717, 0.5044839801217768, 0.5199121821486102, 0.5053503355882704, 0.4785518982526195, 0.5570462684615638, 0.4773041370382738, 0.5228633239830027, 0.4954039592506022, 0.5354055880660199, 0.5462420200132835, 0.5097977670344095, 0.48031005346315475, 0.4474274708389047, 0.5102063421735102, 0.4902114063894298, 0.48235256444196967, 0.5087684780939089, 0.4746829230985078, 0.46808389600506944, 0.5593564002282662, 0.4715696638270988, 0.48128874698538504, 0.48907008743722546, 0.46954816339216676, 0.4785290612373109, 0.5396613602828332, 0.5026845829699402, 0.4731470275678765, 0.5047735505167222, 0.48183315929533926, 0.5425600455592382, 0.4959135931679013, 0.5153562735136851, 0.47934928646244124, 0.4930445177155119, 0.5360101585805037, 0.4503137347468487, 0.4961429933614105, 0.4730692528787193, 0.4673917938410766, 0.5435852523449625, 0.46448119877337907, 0.46714291801028746, 0.4895442680152324, 0.48105610207525096, 0.5013922187258004, 0.48282915987712366, 0.46638150798657074, 0.4867511995891278, 0.4999791731445239, 0.49672363228735106, 0.516656725190586, 0.5129896511259996, 0.4567570952942767, 0.4970065044970094, 0.4918428483708286, 0.47657918593086, 0.5007067486098136, 0.5272400423765373, 0.516693084185187, 0.48675503099181583, 0.5413406300045484, 0.5262982839124051, 0.5134086695121483, 0.4890192105238079, 0.4812733498645666, 0.4867362899765686, 0.46636972308872143, 0.4897670084302459, 0.5473447723960663, 0.5423424328651311, 0.4998472890033875, 0.47172289271789836, 0.48701184237, 0.5396800371635926, 0.5237964875201306, 0.47759403485128915, 0.46821593390928157, 0.46797448834211336, 0.5047313453204778, 0.4808338694021519, 0.5384816921785471, 0.4965786982163958, 0.5121763922551753, 0.4798691999697587, 0.46508059027043785, 0.4755361503130281, 0.4691096125026791, 0.48165296273797026, 0.499147701841338, 0.49107617774348217, 0.5092589307521798, 0.5439766851254313, 0.4810353517142632, 0.4979614954470792, 0.48771624453048945, 0.49252679558134993, 0.49952682695744277, 0.5361170607456027, 0.4745614129755903, 0.4709546013073145, 0.5264901083101784, 0.5111763662759988, 0.4994506628862594, 0.5129690519598625, 0.5174298045155521, 0.49793007844028214, 0.4939759156978377, 0.505751332100423, 0.49305535047451954, 0.4983043501000397, 0.5584857064099275, 0.48154293763100275, 0.5159905155690307, 0.4883127296841377, 0.4872992376736218, 0.5148751560735202, 0.5257424702494307, 0.535750421030691, 0.5066280946884597, 0.516263917105452, 0.45590996419091806, 0.49854247752899566, 0.5440162198812685, 0.49802546825040733, 0.4949056581916496, 0.5128750013867086, 0.49661939960379675, 0.4885899862663541, 0.4776351926157899, 0.44640159702635507, 0.4834993026031378, 0.47610748602147224, 0.5216148066246153, 0.5324672613020429, 0.502395206694991, 0.5095294927274905, 0.5152162666321665, 0.4904181076938761, 0.5114314630950336, 0.4638567105067617, 0.5058881125749315, 0.4898103480469102, 0.4933280152532323, 0.492673717404673, 0.5054749070836476, 0.5001953117765318, 0.5079911859493823, 0.48301905167394066, 0.5166788472091193, 0.5001708975954385, 0.5284358355097135, 0.5169962819995677, 0.4841494990211134, 0.491793488135495, 0.4791825170275653, 0.506077117318972, 0.5186528117045713, 0.47551405072421604, 0.5112634146655283, 0.44195634794382127, 0.5111590759910103, 0.49414057383655374, 0.5103133096742413, 0.502999427063527, 0.5007891709963083, 0.4687783940181207, 0.49017305889870977, 0.4893103384964776, 0.48934814851699016, 0.49458261757914584, 0.5145570089301907, 0.5279286194627892, 0.5030559927313567, 0.4494115289323349, 0.4889241399488398, 0.4846801667447188, 0.5129378483107212, 0.5297568608816909, 0.4988538484600222, 0.4903568192635638, 0.4756565020274002, 0.5075077926894781, 0.48645417079387093, 0.5049296664493512, 0.4936934969103254, 0.5260806259846104, 0.504543685789744, 0.529083214568238, 0.48823159236253805, 0.5197291714010004, 0.4986858331071905, 0.4770681229590306, 0.5195186851924434, 0.47879017159797393, 0.49205412032496443, 0.4763260455096421, 0.5829711347874491, 0.4824998436781363, 0.5204910078809257, 0.49039350044894475, 0.5052870113991424, 0.47414008496795995, 0.5528934676526527, 0.4949824889696901, 0.5098026272969907, 0.4851654519718651, 0.5027436625511374, 0.4888190366355975, 0.490634493592202, 0.5044246579291917, 0.5020618637977011, 0.5152879797521491, 0.5127150796653246, 0.5060753122000756, 0.4723260320523462, 0.520548084873033, 0.5032113296657956, 0.45785749214855126, 0.47633298820543324, 0.5122431011675596, 0.5142676060029828, 0.4830758493633482, 0.4520384108225779, 0.5558830798608031, 0.4896209669961432, 0.45773731084828734, 0.4851817680139171, 0.4873743944499234, 0.4790102363388399, 0.5433368886268324, 0.5026668868533808, 0.5105901921856515, 0.4692249867086962, 0.5120797124118025, 0.5116156990754235, 0.494348894186606, 0.461845003951357, 0.47704881675714284, 0.5117858321592242, 0.45125450994672045, 0.492816022950754, 0.45641340599344304, 0.5040431399927838, 0.5010484819328812, 0.5209708921128157, 0.4473843953541051, 0.4808543677463646, 0.511776527482353, 0.49140326457194733, 0.49356266371586127, 0.46421543021762335, 0.4781998743050102, 0.47690729533777454, 0.5134628384499237, 0.4889000008430952, 0.5074641038168131, 0.4985352202001941, 0.5102068355235171, 0.504556773397262, 0.5087735513555054, 0.49733864938998895, 0.5044375563148495, 0.49035567041615274, 0.49851478190112014, 0.5159571397894006, 0.5008256934577912, 0.5138922948070177, 0.4950195656845065, 0.49490424462537513, 0.5169398060567459, 0.5120071238811222, 0.5232521523677524, 0.4875648898188721, 0.5098755036115218, 0.5056276271198296, 0.5049182259958467, 0.51638932820304, 0.5325814042323481, 0.549100365370758, 0.46637278459161413, 0.49560419528217103, 0.49156394114279545, 0.5080598156934809, 0.4756771354865503, 0.48210036848179705, 0.44812253344352837, 0.4808043340870551, 0.486109878924877, 0.490608637923668, 0.5244580018231997, 0.48778968188582505, 0.47449099153869573, 0.4515648524153527, 0.5189083151274799, 0.5053580294208748, 0.51077497530646, 0.48955571570338563, 0.5059737220576171, 0.5187799434699668, 0.49809156799282944, 0.522877453299402, 0.48512862319507005, 0.4739304021830637, 0.5113306048699483, 0.4664896267011688, 0.5122796056328865, 0.5527090859466659, 0.5031425451237855, 0.5195376489745929, 0.5034394417354433, 0.4794217354609914, 0.44341540576493627, 0.4835269702235294, 0.5118440759431596, 0.5522854877933248, 0.50971274692535, 0.49769010715642736, 0.5099533176425808, 0.5157996870377995, 0.49595641339699686, 0.514059116186625, 0.5138539899411879, 0.5044273008621066, 0.5295179668233111, 0.48495368161555763, 0.49509139895362086, 0.49731842931851233, 0.525064735685958, 0.4979434250935847, 0.5021420240763405, 0.49828288032275037, 0.4613633106416954, 0.48636025544701805, 0.47309521471052424, 0.47459311684433614, 0.49662753151921246, 0.5017500513371285, 0.5119543069568623, 0.46282033897597413, 0.4965168795207139, 0.5124161696830595, 0.5222403313348564, 0.5147915869109928, 0.4899240022144692, 0.5394155172612, 0.5096550503662144, 0.4959216362910018, 0.540752819884726, 0.45939829272957566, 0.5032253684858842, 0.5459560071978287, 0.4846111809055103, 0.5249297004552431, 0.528707399354133, 0.5175677305584031, 0.5038304170976343, 0.49279191257076427, 0.5176811618851205, 0.49638056810743547, 0.4891662967212726, 0.5137161972786737, 0.4898120293863516, 0.4853138470517232, 0.5211547007446279, 0.5312335346726543, 0.4757424233194572, 0.5023051640954806, 0.5054804426852966, 0.49835227111935676, 0.500230802394209, 0.4825654309049189, 0.541468791842577, 0.5062065733726507, 0.5176580524531654, 0.5436969657805901, 0.4945503805592406, 0.503828805434831, 0.49464412346892345, 0.47069448499851874, 0.46604143408596715, 0.5066233752164212, 0.4905116103114046, 0.5258610068181798, 0.48997225189039295, 0.4988541570400964, 0.5060507909053283, 0.4905916437302719, 0.48873582680932004, 0.5161279747696609, 0.5165603163451392, 0.4828566258111311, 0.5376596102551827, 0.48295563945086833, 0.4874349941044669, 0.46732456362460545, 0.5035748808237019, 0.5148432385800727, 0.5604812674913126, 0.5195118294894555, 0.5061258278310814, 0.5009405185261664, 0.49840553442969554, 0.46632680820103856, 0.4675365414405044, 0.498060648515474, 0.4862026829140151, 0.5206731491022785, 0.5016636099401462, 0.48686025165104346, 0.5413136142121456, 0.5061737907762159, 0.5082561363847545, 0.5092840609940871, 0.48881454582356937, 0.4910598485993161, 0.47840337679784894, 0.5047494833298884, 0.49458322121803194, 0.5186480474628958, 0.45846220591730946, 0.5128985886541971, 0.5772483489539041, 0.5072169728306078, 0.5503207341948634, 0.49157462582188094, 0.5176011948442493, 0.4851135620746197, 0.4937100929698557, 0.512626453819973, 0.4915646829308141, 0.5085236824005049, 0.47071899748058393, 0.48143428090227436, 0.48648759746208065, 0.477386632387893, 0.4798408597591962, 0.4797664806503258, 0.5182515793345874, 0.4824674210062849, 0.4806849950287661, 0.4906116033815253, 0.5149294607575091, 0.5365584941314291, 0.4797752397548511, 0.47629630578813, 0.5072585813352755, 0.4861995520950507, 0.4754083718584899, 0.4951549635261719, 0.48491857040574665, 0.4579938923582074, 0.5306326649180053, 0.4914402391044621, 0.4879024437542936, 0.518139626215902, 0.5058650676998326, 0.5133066263774394, 0.48279576715548855, 0.5027744791077058, 0.48149742022122966, 0.49831520653460465, 0.4595986078153166, 0.5295279098044171, 0.4459961528959324, 0.5257592431106294, 0.5130247009844403, 0.4880554396378844, 0.5010280177186892, 0.5088051208346213, 0.4954363634229704, 0.5139174331850384, 0.49874794667080935, 0.4901579762585724, 0.4899615753629431, 0.48624323392155605, 0.4924212617561236, 0.5159785613400709, 0.4852381565565632, 0.49690324678167347, 0.5237697386005087, 0.514080404091713, 0.5008767313299299, 0.5416478439019173, 0.5062022484088478, 0.5578400432691465, 0.484915785809796, 0.4731250520406573, 0.5385256454119426, 0.5183084632239809, 0.47501742125072777, 0.488544285793524, 0.4866682959910328, 0.5067105693873195, 0.46420927719959454, 0.4479078462933752, 0.5111562291216203, 0.4846118753520387, 0.48927954788538475, 0.48403875046571904, 0.5211032798284003, 0.5261157688179525, 0.5271211678049748, 0.47580707865198774, 0.4898530056628282, 0.483260320473588, 0.48631318636498694, 0.4852910699720906, 0.5080200272904799, 0.4590124265908261, 0.524740178751044, 0.4802936964892521, 0.4854548924518155, 0.504589770519092, 0.4853013195009595, 0.5127753745285892, 0.5487760404247048, 0.4751938825872939, 0.4889879845964211, 0.46448310406821836, 0.5231768010690333, 0.5236662108996859, 0.5169333190434258, 0.4921557618412144, 0.4939535056737566, 0.4757537484282906, 0.4828152231571081, 0.5296090599666887, 0.5038657391317056, 0.4993549163914686, 0.49131681284292666, 0.5311567159103235, 0.5046455140926958, 0.4864357559152065, 0.5399751692033107, 0.45611136412885456, 0.5121218889190142, 0.5004383617988852, 0.5189228660107466, 0.48962087665313897, 0.47141248209358494, 0.4936718267250076, 0.4815504148179069, 0.4457984611447583, 0.5458934938980806, 0.5075048749621889, 0.5048229361770129, 0.519683445553913, 0.4777639278679319, 0.5200108875179851, 0.5419239670583139, 0.505403813244207, 0.5115683171356806, 0.5515434970872811, 0.48480671947759446, 0.501376758299699, 0.4943229007224004, 0.4766969212408566, 0.4951800557240046, 0.4881526461098638, 0.4826681352067062, 0.4728330490712245, 0.506000960331321, 0.49096282952692794, 0.49360710402637775, 0.5072048018663023, 0.46831859141053594, 0.48970314959053673, 0.49872081891774844, 0.4690592410352637, 0.4874628983507962, 0.4436408097064449, 0.4832974800690968, 0.49371180471214776, 0.5133112211544688, 0.5354042961636792, 0.5003204902936925, 0.4384686900586339, 0.5068178043337676, 0.5133027182193024, 0.5050892125906921, 0.4830459491351407, 0.5175903753228317, 0.48846679953075894, 0.4875971439312548, 0.5049731434653101, 0.44607722301695046, 0.5264975863456859, 0.5278445213680966, 0.5124098768892067, 0.4819670905254012, 0.48813111043527857, 0.48247082736128144, 0.474388714916658, 0.47545620926003007, 0.5015671965260694, 0.48458586764226397, 0.49568042759734293, 0.4525089766541735, 0.49063716138721886, 0.4695697202957133, 0.5087797979537644, 0.49665378568487395, 0.5372260894556788, 0.49371767182846593, 0.4909626712788498, 0.48721598780989817, 0.5239169781291428, 0.49259814141665287, 0.5360664244112963, 0.536569990788558, 0.4899365020406261, 0.4914953552351533, 0.46996860740949614, 0.49277205173782074, 0.5405716358560807, 0.46480397193860684, 0.5216866430968602, 0.5047128615875464, 0.5008456760925093, 0.5429903794461546, 0.502270206045496, 0.5123361366994192, 0.4870493179778859, 0.4758661383947786, 0.5092413345389555, 0.5144898610772706, 0.5107540824027509, 0.4849075279300208, 0.4932404123776352, 0.5122346616348737, 0.5284532272607964, 0.472264649779327, 0.4733885196829961, 0.5278409251947639, 0.4774617397653339, 0.4919662390232169, 0.48676821198117076, 0.4637529349375209, 0.49159922814789725, 0.48733637636204635, 0.5168838913245112, 0.47054590313373235, 0.437344844995951, 0.4945965887949957, 0.5119566783636774, 0.5013702718790317, 0.520818414093504, 0.4690681587616058, 0.46087201564882135, 0.4793291627928925, 0.5136158921844335, 0.47740648994542617, 0.4855755712940297, 0.49034678003882, 0.49534341640605956, 0.5448128141696903, 0.49490623604020223, 0.4823134878073463, 0.5164502027683002, 0.5080041939134662, 0.5005548231611817, 0.49241663817367076, 0.48630420988919004, 0.5085611580696129, 0.5079661195210811, 0.5008558591290692, 0.498601313875764, 0.5364776890593652, 0.47949044290874937, 0.4990856671299101, 0.5434796333280721, 0.5194184746664421, 0.49403242101685835, 0.5021069047397136, 0.48490000543590117, 0.5304034534680121, 0.45933736555275273, 0.48536654033263726, 0.5075526650072455, 0.5492047983206493, 0.5430756202328553, 0.5175871748561174, 0.48598946541991006, 0.4995989398444537, 0.5271914249984507, 0.4960856263972032, 0.49466271866247163, 0.4868145488072037, 0.45742757650387994, 0.5016687841191029, 0.4745602436798286, 0.5071606091757271, 0.4633266125038328, 0.49600372752083516, 0.4849903157329118, 0.4826095348916495, 0.444769576844927, 0.5195939369941005, 0.4742998227721062, 0.4444481257772176, 0.4907004557928964, 0.49563223757474145, 0.47079433188550823, 0.49099308416386983, 0.5262072161947579, 0.4578359073866738, 0.5090660761674064, 0.4804048832498526, 0.4997427847460062, 0.48818800445691884, 0.5140476121844944, 0.4813784654351992, 0.5151071648642428, 0.5070242730873893, 0.44193701203754704, 0.4454924766472747, 0.50919269641456, 0.5085112500310842, 0.5286467791185845, 0.4804819305176133, 0.542040824619507, 0.5042726518324403, 0.4607280595904663, 0.5096216862644156, 0.4913290021687524, 0.5192652123876598, 0.5330512513325516, 0.49780174279139017, 0.5298320897741478, 0.5101076172201601, 0.48779409781945854, 0.5202367197205654, 0.49222945462371853, 0.4793855092288983, 0.4846279664991435, 0.44455086172267644, 0.5193907822017445, 0.5455746103066522, 0.5030229810550194, 0.49337447350650604, 0.5159670842864252, 0.4673933975541406, 0.507718298287846, 0.5250134899143286, 0.44595105521338985, 0.49973993947178563, 0.5411173489731435, 0.5360986673257054, 0.5020500597922691, 0.4788208837503241, 0.4620957285754605, 0.5067129243923923, 0.47992429908708, 0.4809371409331225, 0.5075760980643614, 0.5085686714869916, 0.5362746289723083, 0.5101804931412786, 0.4792733283184464, 0.4955384660811785, 0.5058181774905096, 0.49569305173548217, 0.5228368036435208, 0.4273134213788579, 0.5073513887900559, 0.4920154614291138, 0.5476509986861795, 0.48791803818465507, 0.5049307647871586, 0.499337137041502, 0.5109642468424627, 0.47914096546228524, 0.48591340424594487, 0.5091449797819994, 0.5380986576373136, 0.48200224198245467, 0.514207402774579, 0.49507221726315326, 0.5170105898260569, 0.4968954851695997, 0.5149876236485265, 0.5384553275968101, 0.5217291015092653, 0.5056448054973554, 0.5130352883748966, 0.4844122290793778, 0.46629512115764005, 0.45981948934538275, 0.5002939329049505, 0.4995693217010422, 0.5097806011060178, 0.5314579461061397, 0.4875327450570112, 0.4635636320621223, 0.45006324136693093, 0.47628326679121274, 0.5149824696332531, 0.5043418991114916, 0.48935846471529504, 0.4757743347909339, 0.4958530781914506, 0.5146588816917769, 0.503552862539648, 0.49328041164631437, 0.479799033649009, 0.5219042611561469, 0.5100099964093736, 0.4998441811672129, 0.5476518120054692, 0.4911793215728082, 0.5271639833594144, 0.47844918885104054, 0.5040632920733249, 0.5204970274456721, 0.532618723397809, 0.48765373059472306, 0.5033606226648052, 0.47590852444170195, 0.564477945239208, 0.47839186973452114, 0.48768208130709956, 0.5098930311343148, 0.5071904186674938, 0.49245617549580417, 0.5202368259908158, 0.5094795816797721, 0.48627548101043866, 0.4824061821457974, 0.5158067074314452, 0.46495973190459416, 0.4868251991838953, 0.5049942386330014, 0.49967543164103023, 0.5065658287259693, 0.5147374265176844, 0.538379296759284, 0.48114971482033686, 0.44998166044373905, 0.5269175060532381, 0.5226846927988916, 0.4913551694302638, 0.5231317111612779, 0.5294319872782466, 0.4952268974105154, 0.49060692159296904, 0.47276411579243993, 0.5076954514429285, 0.4973590058181331, 0.5141110713437419, 0.4817708058201551, 0.47231734719209484, 0.4859274660325925, 0.4968029079814583, 0.4786198746087369, 0.4822200599403199, 0.5329879052342491, 0.4638011229527708, 0.487183736633949, 0.5043209153986514, 0.5224560526808113, 0.47029595741433705, 0.4664193689691931, 0.5551988717511254, 0.4944818387275202, 0.5068318964554496, 0.5120260420661482, 0.5303903273349028, 0.4820985637314262, 0.5170622031106091, 0.49230335206371556, 0.5366419820219858, 0.46095617417943624, 0.4792093850697176, 0.5103536869671798, 0.5125990166388124, 0.4902629002322441, 0.4790121474351209, 0.47704952076477153, 0.5202964802664937, 0.5178359165468674, 0.47472175461852284, 0.4712498521343585, 0.48936274783040223, 0.4866922597872632, 0.5245463140907114, 0.5023961271269526, 0.4826156941209244, 0.5333746237324115, 0.47956369047982345, 0.5284218142895838, 0.471502167096134, 0.5061151982725922, 0.4762981746378697, 0.4661928367028487, 0.4912438159421647, 0.4840231785656409, 0.5050282128811391, 0.5171221194284318, 0.4663140154802448, 0.4912526748848267, 0.468254245280311, 0.4990833088059376, 0.5209521506507941, 0.49633228567152105, 0.5735978060751734, 0.48880708600089107, 0.4884272058908236, 0.4610696215936358, 0.5196753378551656, 0.511554154956413, 0.5141573697746722, 0.5291224433071663, 0.5078858283208307, 0.4980829602574181, 0.4949697835367229, 0.5328450906707201, 0.47470822364450954, 0.543526198715762, 0.4367287870553458, 0.4959292880509095, 0.503401565732043, 0.49363509733805855, 0.5203620284200431, 0.5038549736752668, 0.45353722167356975, 0.5142882138925131, 0.5002124974663394, 0.48352070118651064, 0.458617525143759, 0.4886620467030503, 0.5503919117278814, 0.4273218128814948, 0.48881573457174143, 0.5228566853905514, 0.47758618813273346, 0.48928708309668395, 0.4899789466720939, 0.5002912525298036, 0.47705287003880953, 0.48644204715168377, 0.48651948577547616, 0.44881679338029856, 0.4735647818009027, 0.48353747416600035, 0.49213043666855133, 0.5335854529414606, 0.49999831930962113, 0.4763320259986487, 0.4501889722117427, 0.4583546695443903, 0.4876443855435909, 0.4854235511649527, 0.47227855824937676, 0.49866586280310965, 0.465247329154479, 0.48344851150394647, 0.4403975846434867, 0.4997834217462553, 0.5316918373688231, 0.5232111796610385, 0.5027005858516577, 0.4754634500616682, 0.4932074326542161, 0.5366431059561134, 0.5206149229301467, 0.5239716847595995, 0.5070199741728225, 0.45113312093052466, 0.5070489644479049, 0.45604266879508343, 0.509009677292889, 0.49997754783896686, 0.47037444647641063, 0.4889221834481368, 0.5071114277088893, 0.49298941589833006, 0.4923464344318742, 0.530760856869651, 0.44393927150841817, 0.49332239015121454, 0.47908534500498107, 0.499776992367288, 0.4808284476023067, 0.5696149297309637, 0.5190756105785067, 0.5068584140796456, 0.48689592282843625, 0.48894450484877866, 0.5216354154876214, 0.5211019079431507, 0.4811416733834643, 0.5087420267148112, 0.5187607248766117, 0.48259368737076863, 0.5608756779726665, 0.5133243157244792, 0.4912259481178853, 0.5112325235356091, 0.49788998266661977, 0.512099066897473, 0.5380119529360318, 0.556803403248033, 0.5289688930814046, 0.4558022430097048, 0.4987374241868918, 0.503521705101864, 0.4853495216386423, 0.4932914729016689, 0.5126333557972597, 0.4878860573629742, 0.5000494780990973, 0.4999166123744338, 0.4797327023844406, 0.533714631561771, 0.48283749036886475, 0.4900452653480964, 0.4668400825311239, 0.5094583840255947, 0.4654914716014584, 0.4960712893138934, 0.5049542542768327, 0.44180748668876313, 0.4854872690316802, 0.5591951516967297, 0.5127630608896925, 0.4536846277102134, 0.4911821969210452, 0.4427970662319783, 0.517270694802713, 0.45363599897889395, 0.4745581015216782, 0.518911573871164, 0.533996793265025, 0.49856736697348686, 0.5221412278511722, 0.514396203120687, 0.4403042023088163, 0.5067678875313738, 0.52120346684655, 0.47313288445431806, 0.47435197154262687, 0.507323613229035, 0.4827374614874647, 0.482126843278457, 0.4925733703197625, 0.5226441657680705, 0.4793867709198224, 0.4543345353174793, 0.46456815060031803, 0.5174701228411839, 0.4942594778094093, 0.5169799029518807, 0.4415181396667406, 0.46979352079997994, 0.5059316633270032, 0.5156159731929717, 0.5154980486421699, 0.5228717201180736, 0.4674883659392921, 0.4487269648482292, 0.5034304505929638, 0.4855854303984152, 0.49222038874186436, 0.48017094059503135, 0.5569892131118057, 0.5052975793462975, 0.47452166511353455, 0.5065059217787411, 0.5250464561663755, 0.4956001933587053, 0.4970793198497714, 0.5213435869007874, 0.5280779754677782, 0.5072065222297725, 0.4988671946687111, 0.48272663787303943, 0.4934093640293411, 0.47137941569688413, 0.46533066912502274, 0.4962264287428716, 0.48847702980381447, 0.5464312443832514, 0.49191869068697114, 0.5071448124950158, 0.5716461803020767, 0.46890523187919225, 0.4715734153632696, 0.5375559693422347, 0.48828655731261256, 0.5029697316339358, 0.49734184352909927, 0.5203155340009182, 0.48700941816482807, 0.4917432847969094, 0.5298218661472949, 0.44013952507413934, 0.5080086479220963, 0.5105099038761124, 0.5094786395620111, 0.5133645357312383, 0.5372819502368541, 0.4409186436026607, 0.5103233175533309, 0.5266370825470028, 0.509960419104765, 0.49044194839444455, 0.5096333567485217, 0.5324709282796504, 0.4886700512268051, 0.5018435699447009, 0.48074825052353976, 0.5074999133829742, 0.457718018022455, 0.513448666939537, 0.5141602683107037, 0.5259465686549013, 0.4949766364301577, 0.529417431673365, 0.4820232609206151, 0.5126835247555104, 0.5200781217492536, 0.4690546752765111, 0.4570212910555403, 0.526055895899259, 0.4979627871509768, 0.5420893427062811, 0.49085403556638524, 0.5265354316331086, 0.47652812303190456, 0.49113432327659645, 0.5205624556230218, 0.5499063373421269, 0.528389894506729, 0.48684835842872126, 0.5125987752113219, 0.48620155515562313, 0.4787204066153863, 0.5204492622410993, 0.4893784839266008, 0.48732008871595045, 0.5386495243277292, 0.5019655196291403, 0.48374107043909276, 0.49087158414122295, 0.4863531522979973, 0.4814123310118116, 0.5121737674726594, 0.5112799217187264, 0.5250792980997336, 0.491517195502108, 0.47559191138245255, 0.5085615308575248, 0.4976779140222165, 0.4737725951041143, 0.4742065229839684, 0.44546917579520473, 0.5148789517377863, 0.48801846541579097, 0.5036246244651061, 0.5228639863551342, 0.5202921666565279, 0.5146623451725293, 0.4989874822771287, 0.4811476902194965, 0.5213346266837372, 0.4877190235899827, 0.5106185568119834, 0.4909125711830363, 0.5565687966498338, 0.5210906799063829, 0.4423220724784454, 0.47525089457669434, 0.5050283236703967, 0.5193466004114423, 0.4878534642212832, 0.5042211596449115, 0.478223619443014, 0.46046571206800707, 0.5063571289789418, 0.4896412263754959, 0.49117593063873155, 0.5406187011527289, 0.512862078212167, 0.5188636181314635, 0.48100131971540483, 0.5147120318146503, 0.517932383988562, 0.5033879780630561, 0.496934758902481, 0.49748186572702385, 0.47309110588338993, 0.48810868741953434, 0.48925791822259335, 0.4587746587314578, 0.5079290299449669, 0.4921247049802194, 0.508968571437017, 0.5126150455067559, 0.472967303425544, 0.47730896734865697, 0.5256416976546003, 0.512203681687333, 0.4543835382256522, 0.5161030850537486, 0.5282222023470804, 0.4904043396687506, 0.5016832118361026, 0.49635787090837596, 0.500839532729933, 0.4754542244253536, 0.5154938306471901, 0.4680547783970354, 0.49010249855652865, 0.5330219063882325, 0.49327632752971473, 0.48320333456261316, 0.5217174509391359, 0.5507720473488538, 0.4811869160467986, 0.4943036415236841, 0.5511750337740372, 0.5136933918452214, 0.4911679210005668, 0.42735653281357006, 0.4526401401152454, 0.5221879072467052, 0.48400839356007197, 0.4857296796183002, 0.5369565881272312, 0.5068385107457181, 0.5467869055407297, 0.4813469260550053, 0.49697872546657096, 0.5003028270969108, 0.5218391525531784, 0.5317542873783607, 0.4884766112617603, 0.514663637909673, 0.46166810390216995, 0.45808069162701565, 0.43858999781190233, 0.5133616561910879, 0.4867988868312854, 0.49836849956526724, 0.49255686976550916, 0.5228414851336434, 0.5331839316493344, 0.5126491854952595, 0.5034713099900093, 0.49679759318421607, 0.48159822496055427, 0.4928665102126231, 0.5078352199065022, 0.487777118144626, 0.4814220882511646, 0.511958442987529, 0.5287242543511902, 0.510128537797507, 0.49595357365787707, 0.5023935948633351, 0.49336786042086134, 0.5403625036623554, 0.5397836328408915, 0.5363001042146939, 0.4693630639106306, 0.47415635056745875, 0.49041829925240915, 0.5103934641581607, 0.5292758902513575, 0.47266923096138275, 0.5233903314120341, 0.5068034605673011, 0.5091856326715757, 0.5139153087468381, 0.4734922542974914, 0.49779998484058297, 0.4363809957503606, 0.4620627512329721, 0.5153588205924974, 0.4644427210704068, 0.5008401533374223, 0.5463945797663231, 0.5621335267156722, 0.4902742822796634, 0.4854787164719737, 0.4876954011740929, 0.49196156487867065, 0.505711492704817, 0.49226846210909664, 0.523466018792349, 0.4710779172061635, 0.5344450619405713, 0.519086839361572, 0.511579345697288, 0.5263044153654824, 0.49193165542907613, 0.4592361005366255, 0.4682311064915497, 0.49555807469238894, 0.5036032456847321, 0.5085925970688995, 0.4860861494596183, 0.5063916871513897, 0.4832264560108148, 0.5274785271172681, 0.4790107897156639, 0.5056521494368702, 0.4957175914580773, 0.4806473047897956, 0.5475370168339401, 0.524007266826342, 0.5428483504840084, 0.46296855419926325, 0.5501969014147627, 0.47033690225023916, 0.46845428044029236, 0.5034405314622351, 0.49870413712496453, 0.5330011176519379, 0.5165137357043755, 0.47260265392543954, 0.5097881967241525, 0.47248586629639366, 0.535804303426547, 0.5012907370316931, 0.5339697224644666, 0.5162471307130129, 0.48027742315191674, 0.4826140139704728, 0.5231412305785155, 0.4868518690570659, 0.4835126695745407, 0.48769614429433644, 0.4589212872300832, 0.48049806710556425, 0.5080450447383231, 0.5119850831958739, 0.48529713330080054, 0.4957545451920828, 0.4907969872237544, 0.4840877454957852, 0.4697352497343593, 0.505401319074943, 0.5039545108961906, 0.4481004120291305, 0.49345664954229257, 0.5292739180132039, 0.4833905211240689, 0.4980851705543552, 0.4696754746893091, 0.4836864754814919, 0.5183425439993953, 0.47504824793970674, 0.4846902977644627, 0.5120229269113636, 0.5221099826717005, 0.48094288507405186, 0.5109846736631916, 0.49621138964609585, 0.49782334203490197, 0.44509319715901785, 0.4741079973496287, 0.49213644910410964, 0.5371538834090474, 0.5398162298183585, 0.49409582245395467, 0.514467541522348, 0.5141056784222738, 0.5209272224073423, 0.47594807277188655, 0.4616059031363392, 0.5271545673472049, 0.4998884183790378, 0.47844991570413214, 0.4891508734864894, 0.4681495306599789, 0.49466341174540884, 0.49612230316936246, 0.5385551915020178, 0.48839037852426603, 0.49753707720120743, 0.5500610355557293, 0.5399092511325495, 0.49961203578288405, 0.5026646050337166, 0.49544312815476044, 0.4776434842398574, 0.48631696322741624, 0.47945727048865916, 0.4961930559620735, 0.4823865685940736, 0.5073451090020397, 0.4675990056856734, 0.49158257468998123, 0.5028322016278888, 0.5216897855112643, 0.5051833801114497, 0.4705604559189496, 0.49210031868278725, 0.47203412681658974, 0.5298497337313536, 0.5333992364583675, 0.510939417170532, 0.48342093127891733, 0.5013414750372884, 0.5502528022919895, 0.4404043268609333, 0.5035252585666324, 0.46214249357124293, 0.4566998950101543, 0.511388764128071, 0.5335704697010092, 0.49798994932083385, 0.5218876446665737, 0.4608282698368654, 0.4633439307510296, 0.5260996640144758, 0.5081533581428619, 0.4898772724408851, 0.4353959507371102, 0.5158679860643738, 0.5348081248792462, 0.5126978171587953, 0.49149344837320613, 0.47797683726468204, 0.5034339409238074, 0.497561170143226, 0.5126635011356081, 0.46651274313422836, 0.48981523760713896, 0.4923551812614603, 0.48197508684681145, 0.4891406501003366, 0.5373779568844967, 0.4814270125921034, 0.4636755397689647, 0.5127010569900362, 0.5009044877042612, 0.5039673858649244, 0.491361275006053, 0.4770403069771083, 0.5338355189749072, 0.4739118093877005, 0.563805657648151, 0.507796799654514, 0.4665137777826208, 0.4954602357949183, 0.5257316504731943, 0.5196430562336495, 0.5025352228587079, 0.4727016916839933, 0.4981177209697848, 0.48178077586522816, 0.5094645751922761, 0.4932409523981901, 0.4879264172791279, 0.5015481591173224, 0.4554873511350155, 0.5146753504522695, 0.49793937962411744, 0.5399761233076652, 0.5373472849615321, 0.4365803241626779, 0.4892281783304897, 0.4176810453259357, 0.4703158563389628, 0.49406439476841907, 0.478318567488877, 0.4478964004642974, 0.4916195594952412, 0.4734759351453016, 0.49927364578701167, 0.4746833392087843, 0.4993416626335437, 0.4894233930125702, 0.4603399933326711, 0.5194504039379622, 0.5261653785204462, 0.502656646028322, 0.5131807120508524, 0.5108609462048618, 0.45725750781558333, 0.4911174081850109, 0.5006044226603126, 0.4969355367288508, 0.5008660150177193, 0.4501235217121497, 0.517708831410586, 0.5222365643410499, 0.4848833601400866, 0.5027985983229981, 0.4946334338197601, 0.472496484495549, 0.4933121238128051, 0.4971621947866597, 0.5139465866039057, 0.5148633957939971, 0.5030485636873548, 0.5128569073535316, 0.4915251109116614, 0.4773367640223699, 0.5415242291175174, 0.5024506029482008, 0.47793678998561134, 0.4700831892057701, 0.4802365333278254, 0.4973034922554859, 0.5033397271556653, 0.4910439919345993, 0.5192435467609916, 0.45164073053492726, 0.4991235102514182, 0.48978465855382053, 0.4858368200579533, 0.5177453819713533, 0.5008399754739404, 0.5192727490223137, 0.4751956178199282, 0.5322254741455887, 0.516403074392267, 0.5015118831332643, 0.5288501162622561, 0.4677389955321894, 0.47385269313613537, 0.4753451345381807, 0.475895667549978, 0.526734380000166, 0.5189276195208229, 0.5165076707315897, 0.48267032758070655, 0.5306749301087492, 0.4780783145324484, 0.48014607365642853, 0.5391030295994746, 0.4888476267784785, 0.4752180376339054, 0.5556491069855191, 0.5124368130188195, 0.5442851390039984, 0.4997500965289757, 0.46668395099522, 0.48191234603535626, 0.47494150203890056, 0.49298539147756293, 0.4632931697445222, 0.4800045103706408, 0.47151231938184934, 0.48228276397514946, 0.4474429830176153, 0.49967731453307707, 0.4827178404919593, 0.5014489339265952, 0.511982467055061, 0.5078662808661207, 0.4927148178713837, 0.5135367665766645, 0.5143810171877556, 0.5237750279502581, 0.4979455709734444, 0.49993211823277417, 0.5087339215599942, 0.5012563599837574, 0.475605680084578, 0.49106393805717363, 0.5230303671551664, 0.4840740769785171, 0.49753363294035136, 0.47687955154261324, 0.5175305996082441, 0.4718126392436182, 0.495266119603503, 0.5187404621999033, 0.527937076039179, 0.4813468432313619, 0.5390953635069239, 0.5048767812899562, 0.5542558250991181, 0.4896782867277773, 0.45262517411687714, 0.5037410427209138, 0.5030960010273815, 0.4134675862727111, 0.5275019194233993, 0.5352789650352301, 0.44220957715466797, 0.5225228318004949, 0.4666947030469482, 0.5019157384109587, 0.4725160838082266, 0.5125813278906783, 0.5506633140561855, 0.50935423772431, 0.5346999465657432, 0.5349931174788234, 0.48726404285716274, 0.505564604422262, 0.5046152989980108, 0.48618987882397163, 0.4908836998012236, 0.4861178082150001, 0.48851606464829267, 0.4674166140454122, 0.45158187999540683, 0.5089280070117989, 0.5401550116680399, 0.48958092531416875, 0.49309350787332545, 0.5200079988556343, 0.5011720619259928, 0.49952656654531447, 0.4391715162855325, 0.46829504685848516, 0.4821450368401016, 0.5146886984893906, 0.5183428795304572, 0.501065159973027, 0.4495585933186354, 0.45655231720801137, 0.5234686784289724, 0.5062291230780759, 0.49494272239993614, 0.4642665076672145, 0.4857546856180591, 0.4971245800868355, 0.4596643182112106, 0.570728943919129, 0.48558014265059746, 0.4953613314905553, 0.5107066356742024, 0.5314064124291572, 0.4674604038965487, 0.4930112966121108, 0.5048893645589629, 0.45630078308455607, 0.5354967616771211, 0.5097408213150972, 0.5077778375868232, 0.5044424000976876, 0.5556820114955459, 0.47700926726620174, 0.4897864051213529, 0.4881416511599369, 0.4783977368892044, 0.45406790598970564, 0.5492746923024457, 0.5026292007523906, 0.4850173493606538, 0.5311318617481031, 0.50100033166926, 0.5126094260521946, 0.4897727669713624, 0.49460579692532947, 0.49141158566780235, 0.4647855252877456, 0.5044763324573308, 0.5442756528295392, 0.48504004225305974, 0.5058659886367688, 0.449215665103856, 0.4883623048404948, 0.5075141145180698, 0.5165097310144425, 0.5330216173063926, 0.5164241935772959, 0.5170134007190246, 0.48612843872576705, 0.46851577366827724, 0.5189088803744174, 0.4409443500280684, 0.4671261027952369, 0.47558316958602853, 0.5001285388578499, 0.4753243297024068, 0.5221312471126153, 0.47940370895368395, 0.5074183475397438, 0.5109455449641372, 0.4897052161184244, 0.49905069711473865, 0.5088977998031439, 0.5395452298963441, 0.5123289227647319, 0.47988161825336423, 0.49987855753283134, 0.5338768834366828, 0.5349671955256808, 0.47954789040278806, 0.5236318181585374, 0.48691610307366096, 0.5142906929848879, 0.5397631875113464, 0.48610100029275816, 0.41813619596879725, 0.4473047154135233, 0.5185669110998928, 0.493877263912204, 0.4490789321978857, 0.5631138047474529, 0.5010098810429415, 0.44867795195990756, 0.5174190596105565, 0.48660870658049066, 0.5054670506813999, 0.5143480823044632, 0.45434719975113086, 0.5110128077196221, 0.5111291454405811, 0.5114508833994614, 0.5563546513153881, 0.48870624630572984, 0.5040191653412041, 0.5258700701262341, 0.44393106229926105, 0.5098188187871769, 0.5084002134062825, 0.5373785780500409, 0.5212251932930346, 0.5070725379948468, 0.4744437375417826, 0.5047789763458268, 0.5157614412103234, 0.5263946562249685, 0.499968835604984, 0.4798088574539292, 0.5255163167199531, 0.5284170268921391, 0.4801441440831651, 0.4425676772989895, 0.47512815745205467, 0.4959848513134129, 0.5444293096145862, 0.49796973215333956, 0.4627903815974611, 0.4858156244956809, 0.4799315696254458, 0.5039594400671303, 0.4767754511545224, 0.4759882306547093, 0.47855794177859046, 0.4401028780222739, 0.4929702089927914, 0.4773396886528021, 0.4739735188579846, 0.49388928529061116, 0.4555422484146943, 0.497902333460347, 0.4611735198584546, 0.47263820677063045, 0.5564394791618748, 0.5378487423388291, 0.5378078117515974, 0.5210189681045934, 0.502502095113563, 0.5412650532783414, 0.4952690118521821, 0.4983028954484475, 0.491598872699038, 0.5179738252682491, 0.5165684471528202, 0.48905547690395806, 0.4498771817261324, 0.47639596595061984, 0.49593390407542093, 0.4580942488937243, 0.45567036944005374, 0.49352893454650076, 0.469626541619021, 0.4676379875405473, 0.49558265224207276, 0.5448461093434314, 0.4647168554673924, 0.5326670664094385, 0.4569762059135121, 0.52911814878108, 0.47860334811174143, 0.5348042604850065, 0.4799581998152758, 0.48562495597502753, 0.4979614425732112, 0.5325549786038619, 0.5121321487974817, 0.516856574544646, 0.5299023597078859, 0.485710219891351, 0.5512630938549123, 0.50483085210606, 0.5047011597984411, 0.5009788469752908, 0.4491366405722483, 0.49936699049337213, 0.4858458529865987, 0.49816185837809784, 0.5076919941041167, 0.4820195639485366, 0.5172802738386875, 0.4858463078926649, 0.48536594117636833, 0.479662071409023, 0.48058799613720476, 0.5045400989362547, 0.48561868791620644, 0.451783970186072, 0.4867160385741578, 0.49127411263451226, 0.4189755221172186, 0.49744897804477345, 0.5050634748929314, 0.4840718943600027, 0.4881097777157032, 0.5348343566045534, 0.4752682580937414, 0.4864230530555989, 0.5234968484845116, 0.458546688515703, 0.4763655259056023, 0.529838752369895, 0.5351095523572004, 0.5147427973397545, 0.4723156789618759, 0.4923434747306507, 0.47061903890237833, 0.4628908534764586, 0.5063214853105611, 0.5564288603813934, 0.5261451161738485, 0.514947338393357, 0.47890441370303366, 0.5147269258160876, 0.5027162976861591, 0.4617441381539221, 0.4666675634147289, 0.5275238424646215, 0.5001920143720424, 0.505186714329848, 0.492295234124953, 0.5082055739342399, 0.4877660467021813, 0.49407614592827637, 0.5057993207136078, 0.5331087457386857, 0.5104164478207954, 0.49619988175285656, 0.5048762013570444, 0.49516330441507433, 0.4875648276528899, 0.4991017596320288, 0.5121398359694874, 0.5006880462908233, 0.4674522268819345, 0.4719006615590712, 0.5243872703879779, 0.4617308136802743, 0.48876964497948705, 0.4889972851884906, 0.4918195547484188, 0.5246110108664662, 0.5245599055859597, 0.5480922272908164, 0.547705652545798, 0.5278456454848318, 0.5217865746489221, 0.43445545037587907, 0.477404897346484, 0.5153208545642196, 0.47379955934939094, 0.4954762841573309, 0.5548952185962994, 0.4695719237218271, 0.5181608988245617, 0.48535034427094326, 0.46933864404698417, 0.5000412634162091, 0.4717753584278531, 0.5089952301728814, 0.4575510054028196, 0.49958702144538164, 0.49736266381576033, 0.49321769913809754, 0.5075208993329149, 0.5183436737885089, 0.50223592313204, 0.5077280928879008, 0.5387745872156129, 0.4947848759259124, 0.4943882397668865, 0.49944985323113644, 0.45431937774104864, 0.5054933502117385, 0.5260193923873071, 0.530555559784405, 0.4738864558127367, 0.4766670746543225, 0.486844711740047, 0.4796280366567673, 0.48333306954627503, 0.5327052322337076, 0.537550490313751, 0.4772870082201714, 0.5146833723498582, 0.47754417820884476, 0.5202583202922537, 0.4984442461530567, 0.5065435793563609, 0.5359589645124871, 0.5068878948880892, 0.5004490466242159, 0.4836490360189986, 0.47972137317673036, 0.472923531808002, 0.4500969437326424, 0.533840382842354, 0.5389345660679842, 0.4356233577502586, 0.4868696498744183, 0.5242807461745691, 0.5304387597418673, 0.5318971181172903, 0.5122067748987086, 0.4969804758428711, 0.486717852308931, 0.48286565773552126, 0.4994686167235932, 0.49977211806449606, 0.49714424090750386, 0.4999360445674375, 0.5129932269323201, 0.5138251826945623, 0.5021684875203009, 0.4835215399500211, 0.4921850267237154, 0.44951950291640386, 0.47026778655070745, 0.5274301854231386, 0.5254791911595735, 0.4570287804680376, 0.5398740393494759, 0.451151007503664, 0.4765485577973477, 0.47427881749779477, 0.46468856187903423, 0.5075468366333122, 0.5432258268142464, 0.50915893801523, 0.49355728663758636, 0.49427725585628784, 0.4710285060958485, 0.5001465787698557, 0.49804923873069457, 0.46451710119541034, 0.5017347737561386, 0.5389836538296435, 0.5281457975752225, 0.4894272634058613, 0.5084478397834872, 0.5237391175102736, 0.4920092467498599, 0.49129804854191206, 0.5343535557274265, 0.4935591126883674, 0.46690975168909277, 0.5031144633126049, 0.5106871505482862, 0.4887212385716276, 0.5396708318689876, 0.4737201718455602, 0.4928725859768152, 0.513961693749332, 0.5045733698372314, 0.5181942260189185, 0.518251088049548, 0.49291958848776496, 0.5081037546582413, 0.4984702204720245, 0.5204968665784061, 0.5366819097825984, 0.5111082807840045, 0.4848897987154909, 0.5169608503870818, 0.4930597335192841, 0.5371940014389903, 0.480447431418016, 0.48424570721613724, 0.48741649945096727, 0.488198993568103, 0.47411006038560066, 0.5561164377463437, 0.47733314493552653, 0.5233337221318665, 0.4790966918443399, 0.5575342479778947, 0.5298466213523241, 0.49630832123922536, 0.5305967333097933, 0.48185403830915685, 0.4526421179800221, 0.4965948849065247, 0.5117119599635188, 0.5132778984421235, 0.4984726899601612, 0.4959790514880155, 0.49002586507832707, 0.4806715800750249, 0.5095450930705795, 0.47078386476954176, 0.5363280699971547, 0.4832447949778617, 0.45860446034727625, 0.49464160182111216, 0.48415382981263316, 0.5077223596817799, 0.5104926336506287, 0.4309508123058758, 0.5246850807546992, 0.4876607083277398, 0.4903741721522771, 0.4925705851156081, 0.4597231098618698, 0.48987381960139736, 0.4585515270896519, 0.46677197260820413, 0.5010121910774507, 0.5416375211970985, 0.5085232750032251, 0.44603424820851506, 0.5298578161917404, 0.482747454894187, 0.5005824936774406, 0.4971166929605535, 0.5147079189985122, 0.4580472869326299, 0.4574072372819369, 0.48038222038811995, 0.48957195752420174, 0.4443991369765376, 0.5097792978861906, 0.523817560832814, 0.47933696313524726, 0.45237662153441927, 0.4894792069493246, 0.4661577109584323, 0.5090107888190939, 0.4851297160611689, 0.4915956727292116, 0.4568147868001233, 0.5123931345432455, 0.5177656989936851, 0.5259458580398356, 0.4965682279137664, 0.5580843064709896, 0.4875987345550188, 0.5041369535995655, 0.4805445732392123, 0.5209677607547225, 0.5084218852107191, 0.5035395448129992, 0.47641971065005706, 0.5579775141083544, 0.5149062707326113, 0.44531661808217393, 0.5316168645224991, 0.5137207874614544, 0.5185665450588813, 0.45265551856524516, 0.5028242675307734, 0.5127373114691308, 0.5147911954332184, 0.5091366757752053, 0.5173813359431735, 0.4840316827757993, 0.48930349559904357, 0.4802881434601442, 0.5354370326413629, 0.5216875854059578, 0.5340482863595816, 0.5136969780735527, 0.5155564517961937, 0.5148129973131785, 0.4728276463770764, 0.4719067348148122, 0.48110858124671485, 0.5086045754114019, 0.4342354606994085, 0.49685242767010834, 0.5314553579123407, 0.5178203537755229, 0.5313375943065242, 0.4925124391799874, 0.511935850459011, 0.5269019643872892, 0.44805743680748983, 0.5022542714157721, 0.47955934466141165, 0.5077233895643288, 0.5165877960324978, 0.5122610645032876, 0.48421910900490345, 0.47186520996811776, 0.5305486563511178, 0.4827600973159437, 0.5150772742807525, 0.45093155720587824, 0.48704127979478795, 0.44028258364309625, 0.47587014500134933, 0.5037959477268386, 0.5511883523990765, 0.5209117680232981, 0.47974838471469783, 0.5277704595958179, 0.5529005990536882, 0.4728725481007967, 0.5110802401432852, 0.4871876240068508, 0.5000887787827002, 0.4896233269518414, 0.5045527789728714, 0.47827669455280264, 0.4502977101031963, 0.5097407288352237, 0.49570281252144704, 0.45604785623890687, 0.5097475885308883, 0.4751764892542204, 0.47124042123138177, 0.5593093235251698, 0.49906153792337554, 0.4741095158675231, 0.5250980133044696, 0.5412354958993538, 0.49031783402867996, 0.47734579987002984, 0.5174740184488451, 0.49047389114312795, 0.4955079164765852, 0.44437532004759067, 0.49628270830963106, 0.5135972813784799, 0.4849214221224675, 0.4765010234629782, 0.48246224997375975, 0.540579928384529, 0.5463784911455551, 0.522247693629515, 0.5007526377618939, 0.4310197079185632, 0.5067684195765068, 0.5178049623461094, 0.5457252384925798, 0.5035503731055525, 0.4944976783610252, 0.4765304145140999, 0.5118377208031096, 0.5347256177318339, 0.4749959578862497, 0.4715237698836838, 0.5093800232502473, 0.524267183762801, 0.5327050432209173, 0.4617334059080803, 0.5180104597975576, 0.5199566042202134, 0.5201516176643267, 0.5016277389735899, 0.4987575580578503, 0.5131986719565291, 0.5090204404984866, 0.5484807440474934, 0.47003819422858223, 0.5197994735993722, 0.5018898447905201, 0.5296176985606279, 0.46274226471824953, 0.4981534561267772, 0.5231379761923368, 0.5085865851758813, 0.4912152118864091, 0.5267358195778087, 0.5078010120940213, 0.49693739341769877, 0.4838742925642349, 0.5158180448958307, 0.5011539163574161, 0.5037036814497061, 0.47696569391534543, 0.48219893622421145, 0.503071529891178, 0.5286224086205216, 0.5229903568222509, 0.5196589830247939, 0.5078583778963874, 0.501887159361437, 0.49645116783447685, 0.508451106332493, 0.4918222521683149, 0.4990388113882814, 0.4892828755067703, 0.5153413873832146, 0.4976634133627012, 0.525027835224838, 0.5106131403289936, 0.5130513160338454, 0.507225212105877, 0.4835231385804096, 0.5475184131419152, 0.4824186976427216, 0.5099039101637793, 0.46904134490016747, 0.4694227722867398, 0.5181549834555303, 0.4701993507017761, 0.488067214404852, 0.4971679206805415, 0.4717318772068501, 0.5097007724802969, 0.4815673842315632, 0.4343102173882438, 0.4377624788228596, 0.5074125135958961, 0.5220377291478625, 0.46171262667931684, 0.5096443484812, 0.47723180294711715, 0.48784317960367984, 0.4938720786899132, 0.47895538633972146, 0.5439039989540407, 0.5077647196701238, 0.48350817357135506, 0.5304802972101372, 0.49970664333794074, 0.458310270526473, 0.5205115096440307, 0.4996749276394603, 0.49367138254148013, 0.4653189784068146, 0.4967797327485729, 0.5114685165096209, 0.5314338223586133, 0.5563503932242624, 0.47048980864325907, 0.520492496328872, 0.4987754056960291, 0.4971525699122567, 0.5302246386491944, 0.5214552877721986, 0.5130916298353085, 0.48317111300449234, 0.4654922467657856, 0.5104423842282627, 0.5300512648154436, 0.5136671853738987, 0.512962851115442, 0.49774690215287337, 0.5545757930520814, 0.49778030229274195, 0.4454570381545388, 0.5219713513910157, 0.48447222161603265, 0.5360302119784494, 0.5170233394231077, 0.48750981987151776, 0.5009705513588537, 0.5033024852755752, 0.4840632901568221, 0.5190025409210963, 0.5085776814909488, 0.5015775718726184, 0.5379882802329563, 0.49449702865451095, 0.5161116340322799, 0.5168059429104789, 0.5318217810375429, 0.5187313281346199, 0.4625345766813677, 0.4901182568332468, 0.48089023684281407, 0.47935024163922274, 0.5275564042613352, 0.4908243105069265, 0.49934426533005205, 0.5309781144380824, 0.5107617046726227, 0.4626865713650613, 0.4761950497375337, 0.5049560273925641, 0.5002019562784664, 0.5109901124332575, 0.5135060209189963, 0.4586454626491345, 0.479386767727783, 0.5282362684699341, 0.4684211469101811, 0.5080869300442593, 0.4786870011047757, 0.4817748984128522, 0.5288219926066491, 0.4962163255242696, 0.4787295980997282, 0.4490103642222369, 0.49325536666430814, 0.4816784773508421, 0.530784313071496, 0.5156271983850558, 0.49839058096194233, 0.5169083874014628, 0.5119414661621859, 0.5392992355652638, 0.5053696067571718, 0.5099897363587199, 0.4571762006779238, 0.5348576441609768, 0.49374797069439513, 0.4454239449130612, 0.5092175160933362, 0.5341729430610508, 0.4621416441840656, 0.4964418669121149, 0.5036358613702054, 0.4861368735328405, 0.5431372161714677, 0.47572480639633635, 0.5092121685217216, 0.5181309579851912, 0.5002386472064935, 0.5283064592487384, 0.49852154718559094, 0.4774248712674753, 0.5143306133464347, 0.4949157734879569, 0.517757612294552, 0.5128806759094018, 0.5190483024266942, 0.5102445898283914, 0.49336222062465035, 0.5185595310385371, 0.5392422864451336, 0.4885493502530322, 0.5125191894345101, 0.46643067810881583, 0.5144936242532312, 0.5221133922384903, 0.5187733228621655, 0.4690648125504022, 0.4796398676432498, 0.5120557172017987, 0.49476484535659787, 0.444155996983423, 0.46013805661428797, 0.49421014043986244, 0.518664732749501, 0.46990360800053993, 0.520311120257919, 0.5450449704954594, 0.5273576447080637, 0.4904252317539286, 0.5274757656451097, 0.5076860536521819, 0.49377504614057366, 0.4932437237279362, 0.48849202030950395, 0.5292100186637273, 0.46735398070616957, 0.497059891496568, 0.4919792784137144, 0.49584493518635886, 0.4744555396344937, 0.48752750136420203, 0.5314444459142967, 0.49003739988983547, 0.4797668921773179, 0.49580670567567997, 0.513032746490568, 0.5295137019695839, 0.472230553558626, 0.4860735526457594, 0.46775580581775034, 0.4771830817985144, 0.5281826384371547, 0.4935305014811637, 0.5071191782052715, 0.49687255337672853, 0.5152867583498335, 0.485316978859108, 0.5006138297142366, 0.46637117126684957, 0.4622974132268516, 0.49677655927426834, 0.5034494109892856, 0.5105839640968559, 0.4669889873234763, 0.5155082105516392, 0.4836665596679905, 0.5067496632292596, 0.4945637302288457, 0.5373247280165505, 0.421705560618407, 0.509261049404674, 0.4733729019422434, 0.49642945932458665, 0.49892173473447304, 0.5028932630297548, 0.5244669756575897, 0.4553935246319634, 0.5295112484810314, 0.4769605352647, 0.4856261221030974, 0.476394984191526, 0.5028410243621158, 0.5220936034915206, 0.5227367747870029, 0.5018863057255049, 0.5142469488696253, 0.46646133884360547, 0.5479824525546129, 0.49148523222209495, 0.4939674759386681, 0.47957278576481954, 0.51127928389758, 0.5025743434924586, 0.5301463885717045, 0.5124720907414538, 0.49318931343327754, 0.47288655173977123, 0.4935208255427165, 0.5375423891965764, 0.5246880252488645, 0.49318870731995085, 0.4920466035955119, 0.5287405120440746, 0.5047445258365058, 0.5274266190612299, 0.4981623584629719, 0.5031373137094851, 0.475845791831834, 0.5066809099382339, 0.459129411085861, 0.5273284687408614, 0.5325622768418381, 0.45266487216675355, 0.5147378905628688, 0.49152433737965723, 0.48251702886805975, 0.5414041444871797, 0.48188200791683233, 0.5135631540212813, 0.5304610854192244, 0.48826165376483527, 0.5055793091759825, 0.4906502623541863, 0.5115450754250885, 0.5009841900184392, 0.512381679638328, 0.5220540496733416, 0.5033584724004062, 0.4801726588612002, 0.4543136434586813, 0.49032657998689294, 0.47013577347412994, 0.48963673449901235, 0.5231546238485989, 0.5406888783895278, 0.5078281427762498, 0.49196263939548956, 0.4706783226301055, 0.4406068379030414, 0.5107230206073236, 0.47308865251285, 0.5305392247583979, 0.5329678647385051, 0.5202753162962694, 0.5232488899391625, 0.49691551674650186, 0.47937477176978016, 0.5111546813646723, 0.4950777780390067, 0.4958572653016162, 0.48828802797045406, 0.46396294708627916, 0.4998732862533612, 0.47689640961037105, 0.4743209305316069, 0.5202796230641691, 0.5005247352369224, 0.4979995963319889, 0.5052784571242196, 0.5711128771215164, 0.5048644659642105, 0.44711173208533095, 0.5481026395827415, 0.5068176410818054, 0.5438367355585632, 0.48638642689690786, 0.48718614706416674, 0.4821288869850239, 0.4463144420153909, 0.4652011983235265, 0.5091065994412548, 0.475344691683374, 0.547151065742488, 0.457607042745929, 0.520938913986365, 0.4719373621491877, 0.5092953951035643, 0.48052468533896253, 0.5217718722219112, 0.4657661890286018, 0.5364255338513495, 0.4813206485356255, 0.5000090992397032, 0.4927522640715917, 0.46827993525065936, 0.5417713095145967, 0.49546618391910296, 0.490255269616707, 0.5127181569195514, 0.47354957046716384, 0.5022121956091564, 0.5033068618145845, 0.47642159643279997, 0.49282278111832795, 0.5139586875617611, 0.5304467005795078, 0.4522044996195291, 0.49868518613538454, 0.5425119293539895, 0.49963895847898987, 0.49340802728389127, 0.4947023626010957, 0.47480942703285334, 0.5032885539779745, 0.5169065845191214, 0.4831741453847327, 0.4909801215585121, 0.505298493189254, 0.5115793211556683, 0.47119705393833416, 0.4596208224847323, 0.47103817339305115, 0.5409966621817918, 0.5104776704162032, 0.46059083612300566, 0.47920122233531964, 0.5179637793898555, 0.4689710359968366, 0.49791852460190034, 0.4707637561226821, 0.4806986040420833, 0.482650621008163, 0.4573958456105089, 0.47365903973736495, 0.4789937563127788, 0.48852599219443094, 0.5149356504678116, 0.5317784879595303, 0.46548898579374454, 0.5189821454036793, 0.5112770749610382, 0.5042686539751192, 0.5116195195658333, 0.5334732597907453, 0.4583949885748972, 0.4549709673971298, 0.49484701905240785, 0.5059178225450143, 0.4803016027969003, 0.4839297850107536, 0.5514862430609955, 0.47605137065958447, 0.48619564277324173, 0.497141230187661, 0.4872190451387735, 0.4790706933386057, 0.5048931566597131, 0.5488196805493629, 0.49200871529852436, 0.4863778711951994, 0.5003864385830052, 0.5214190947716286, 0.5109984010889961, 0.5209772260421334, 0.4932100795469974, 0.48691317012080043, 0.5140923244003288, 0.49884744195378433, 0.5414226083479196, 0.5098534251956436, 0.5155155153007094, 0.4869004621003241, 0.4991885059121368, 0.5112089395544775, 0.5046067048705057, 0.46841973626420325, 0.49243550269877623, 0.44975558367262797, 0.46892674425948255, 0.4992739400775242, 0.47141408455641165, 0.5050858151460885, 0.4928859106161722, 0.4430878976867962, 0.49922438835619115, 0.5516488252394317, 0.48807687588969345, 0.4630911273404736, 0.4951762238278913, 0.5180254234911176, 0.4848156543739896, 0.47697497460276134, 0.5116865970199965, 0.45291281906044034, 0.49729572887871126, 0.5144315521725955, 0.4689966786425105, 0.48448166408616045, 0.5135786589826066, 0.5030235366070215, 0.505292675139914, 0.4929586818928795, 0.4979116157376299, 0.5080302873582339, 0.5015368915829964, 0.48995386889187226, 0.4817326522809866, 0.5280314748310309, 0.46230835413079635, 0.49819085482591097, 0.4674091026977401, 0.5091205961311568, 0.44919217988302534, 0.515528210656465, 0.4843466158809706, 0.5414418027626969, 0.4781500879822259, 0.5156044661956464, 0.5111949353064863, 0.5231850273903864, 0.5246408900080739, 0.47048147408290386, 0.49748944730555694, 0.5318296334047629, 0.4844787675732433, 0.5154058607419901, 0.4765922690980101, 0.5261938654449609, 0.5203501459847143, 0.4673545675703614, 0.533121433956551, 0.4661453925597387, 0.47426982243249094, 0.529293905487058, 0.4440146118157494, 0.5101812226506948, 0.5227463946277604, 0.513815831995673, 0.5159880983639957, 0.535897720307528, 0.4989564463624747, 0.5036333716079066, 0.4553947238278151, 0.49806790683224716, 0.4917191568679283, 0.4819597638744989, 0.47114239117146084, 0.508670206766734, 0.4881628368551116, 0.5249557463047273, 0.5043623641854217, 0.5046592441426089, 0.4961398752025607, 0.48006675329632, 0.5097939796751374, 0.498684948422965, 0.4873439573055764, 0.5737312862740652, 0.5070364960277784, 0.4961122626991059, 0.47401523548022273, 0.5046393785769624, 0.5150589635751792, 0.5253065055718533, 0.4569245620350313, 0.47694008758115136, 0.4901211096756647, 0.5020193855944888, 0.45960114745357494, 0.49043786793810634, 0.49544095289351064, 0.4637717515557526, 0.5160422644280503, 0.48015430174052265, 0.49573899543643185, 0.4781936164909176, 0.493582961692839, 0.5420011848978065, 0.4546451299434169, 0.5016358285069825, 0.48741314835936883, 0.5165774243063709, 0.49677589958340673, 0.47614308483490797, 0.49232305265909143, 0.5155832590943441, 0.49492139593418755, 0.5363852110954782, 0.5204039368608124, 0.49960278868091806, 0.5094456920976609, 0.510732994133523, 0.5308450867627299, 0.5053543422092701, 0.5363050407985076, 0.45461938126655316, 0.511782448447117, 0.49045760370293157, 0.48158323679154363, 0.5589429336457149, 0.4966864397727483, 0.46877410883442944, 0.4988286018617122, 0.5394164216838425, 0.4566234707442346, 0.504912867814105, 0.4579920007283244, 0.5108079403708932, 0.5010408721895583, 0.4953828755225737, 0.5073448752082764, 0.5158564032696945, 0.5225636598954395, 0.5082594983672771, 0.5329929223155653, 0.4768845432060947, 0.49828823963756574, 0.46638046594878746, 0.4542884047754205, 0.4862473723989493, 0.5355069712652796, 0.4425240442743492, 0.5252048388266721, 0.48400347436955576, 0.5101931831432658, 0.5320132634146996, 0.48256224950325616, 0.5170189293305617, 0.5353647890841269, 0.5049830825207798, 0.4520204146480613, 0.5239003779857555, 0.48426758585599666, 0.49947450607517585, 0.49477263021057744, 0.4629941879743649, 0.4783080765032665, 0.5586006364152617, 0.49004399926985176, 0.4636438493721776, 0.4537261017786558, 0.5240192867408403, 0.45088236005905263, 0.4941084741968556, 0.4847717304259661, 0.4241793303791885, 0.49021174636628334, 0.515010051222918, 0.4944616843315458, 0.5162934392539791, 0.466311729884509, 0.489558511037735, 0.5271004034396499, 0.5088103321642896, 0.501499122034755, 0.4628527192937304, 0.48830459898284256, 0.4871040649197714, 0.5108451340451217, 0.4559805918894815, 0.500848111017408, 0.46611322618314355, 0.44037932367855215, 0.5242072738659276, 0.5406297131569934, 0.4829057485608834, 0.5059423857671826, 0.5059775963164793, 0.4954172777080713, 0.5187312897345299, 0.5159203168289659, 0.5159430157971118, 0.4916167324505651, 0.5329047899084545, 0.490219971486652, 0.47715386535215093, 0.43953436163968806, 0.4769694578045584, 0.49897038748051803, 0.5376489103860861, 0.5245874442774964, 0.4950215575515164, 0.49183752607725595, 0.4590292573350954, 0.5142298312928216, 0.4944981267388646, 0.4879618279513363, 0.4909820846698642, 0.48314215486241036, 0.5309958927584342, 0.5388130472381346, 0.5164355847706302, 0.5037409872523447, 0.5161701646283953, 0.4896462749831301, 0.49165056694611103, 0.5194893935738008, 0.531399687598112, 0.5117015605550868, 0.487338284976286, 0.5172230225218087, 0.5161668980066245, 0.47533898563357424, 0.47212035272373437, 0.4877564160913132, 0.4995547314374593, 0.5194074838195153, 0.5002471368548924, 0.4760114121815559, 0.5172087644474163, 0.5151868616501449, 0.4883918860481952, 0.5053032587882703, 0.4621040949969015, 0.4899657369974251, 0.5114569371438127, 0.552135831242082, 0.5039145219789085, 0.4833390311069501, 0.49521749860964964, 0.46117611008555937, 0.47993184320514043, 0.47764236883165057, 0.4591272207902733, 0.5117454420633648, 0.5230152704075125, 0.5087817762443821, 0.48102195618045096, 0.5191160143411241, 0.4861757563551871, 0.48914915754901417, 0.47953747174342215, 0.48514582618407903, 0.4946735324776783, 0.47726016596361653, 0.5105457898546998, 0.46229938694832623, 0.5408064489906109, 0.4570196595925724, 0.5431318268164772, 0.5098668952729303, 0.48158057848053465, 0.4819311369903588, 0.5036497712083092, 0.5007528810111302, 0.5142305540195531, 0.49859879554215547, 0.5058747369264773, 0.5400673969814739, 0.4763152550291574, 0.5188492559762298, 0.46342565733873453, 0.5007290898111509, 0.5230070526446787, 0.4876815652735392, 0.5062312671928951, 0.47815169200705526, 0.5071526462231426, 0.5059500124756533, 0.49944614992559716, 0.4818866169665848, 0.4666308665469734, 0.5042669816696896, 0.4883536254845238, 0.5159478920767193, 0.4982965917896215, 0.5038070528942833, 0.5082202095691272, 0.5186839340988368, 0.4979105975217264, 0.4625552917357581, 0.49246964049954967, 0.5018556151929013, 0.4689280051651101, 0.5122953928812463, 0.5040432898222733, 0.49286920255261063, 0.5148095824852982, 0.4745682746759505, 0.5126339621352921, 0.5069652759472061, 0.5195710082571146, 0.4826304633021882, 0.5277392768207223, 0.47851061551896173, 0.4731462962395844, 0.5234282638007639, 0.4940655535802101, 0.4863200751207132, 0.4708645570900111, 0.5121036010651566, 0.46543515805752744, 0.4962098180310844, 0.5381212564551929, 0.47954211369074334, 0.4918368141351917, 0.5293150904860007, 0.5222683210015726, 0.5140685302417602, 0.48250162200043606, 0.508439904605879, 0.5222706584446215, 0.5321989249698684, 0.5120117829004528, 0.5067372067162823, 0.4744379519737764, 0.508730303337712, 0.5364552580022277, 0.4803990603573522, 0.5232007031608543, 0.4845826019524847, 0.45592758037328995, 0.4977924421439272, 0.5068961535248068, 0.501475079475994, 0.5275928679887062, 0.4990210188659542, 0.455588147008178, 0.5101262782777852, 0.5431795242560136, 0.49431462552144795, 0.4870131019817464, 0.4896753903470359, 0.47524279117745016, 0.5166946677405755, 0.4856017963227172, 0.5196503277856154, 0.5003861841971412, 0.5289878277078356, 0.4882431665344692, 0.49872559642660264, 0.513817537098537, 0.4737776492371359, 0.5023786876324984, 0.5013505444946283, 0.4884388896214375, 0.5018063693334586, 0.4992187759002681, 0.5154870472291492, 0.4441282596698114, 0.5013569371314447, 0.46840277014430015, 0.48745608418286174, 0.4622467420734537, 0.5032328567451394, 0.5138011726866328, 0.5281603735468547, 0.4827406333992728, 0.4543420743214212, 0.5138783453183212, 0.4738757218895412, 0.4880252866577617, 0.4837511976758388, 0.4949586279796965, 0.5103011480404643, 0.48987091679635336, 0.46386887450471914, 0.48040393948458093, 0.4937068508453006, 0.5152422276664125, 0.4999885717790499, 0.4897036603857507, 0.4509931347483164, 0.4954142714248202, 0.47586355461290053, 0.48337607964676993, 0.47870604717164783, 0.4438336899324507, 0.5267095424660734, 0.5030394460317901, 0.5146161069452335, 0.4945304004823857, 0.46722733710714554, 0.48192134458027136, 0.4952954938478604, 0.48266937232687557, 0.5314898860089126, 0.49057414592524157, 0.5021943666765732, 0.4814957201039675, 0.48059751584188026, 0.5181404871209929, 0.5024050415537844, 0.4774235309680711, 0.5272239270649703, 0.4645761030535708, 0.5325830087036207, 0.5383658911276107, 0.5018484463409356, 0.4825693464301988, 0.5195620155152387, 0.5106574281467192, 0.5327137203443773, 0.5259111479087346, 0.44953074033923757, 0.49476597355987706, 0.5491698539655572, 0.4585866008254402, 0.47387180736171064, 0.4853918948278191, 0.5045620925164443, 0.5046900221734355, 0.4752676807233119, 0.4946293600945957, 0.5295920014523954, 0.5458161040267916, 0.48924243737517, 0.5025504379041351, 0.4916706282526125, 0.45421260117517537, 0.48179702127823626, 0.49499165363199255, 0.5380684327881198, 0.49022071478369905, 0.4854139885829525, 0.47273298749386167, 0.4856097835965103, 0.5043293576645951, 0.5194114674273477, 0.5291359339120295, 0.5301955918280051, 0.5058823568803271, 0.5218303818429657, 0.5235641926785353, 0.507495290396778, 0.4689956610041976, 0.4843478137892879, 0.5454231337246987, 0.5093245485567379, 0.48299751230943544, 0.46384397577161923, 0.4787419905843488, 0.5090135758769646, 0.49587384881542923, 0.48193990509988704, 0.46066743288124634, 0.4950612615313998, 0.4800644930441978, 0.5146390039634061, 0.4964542531489548, 0.5290382553942614, 0.49466504587682947, 0.4777192031708036, 0.5140341758908178, 0.5206675604573506, 0.48362527883523054, 0.49832918897676326, 0.49434295055163224, 0.5049049476562707, 0.4657848069293003, 0.4856537638068719, 0.48371989902197104, 0.4948696007478526, 0.4854692506001405, 0.4840998219545914, 0.4670379013523503, 0.5259334439062942, 0.5289179889828763, 0.5223495127546928, 0.4529153085037728, 0.5109453200892672, 0.4887996599634533, 0.5027229364660847, 0.48620238564107626, 0.4652532531002387, 0.489505705639111, 0.5083358067952584, 0.5225847556854843, 0.5100455507124807, 0.5395537693049665, 0.49467334877916425, 0.49595448764091826, 0.5005961292771023, 0.5122825542484374, 0.4971171664207507, 0.48508165503665734, 0.5059951164716806, 0.5156034014314234, 0.50483691831507, 0.49163093801386704, 0.4724133450291688, 0.4645381377035303, 0.49358327810659125, 0.5345453730507268, 0.5627067165554979, 0.46482689816275025, 0.5307061093124901, 0.48910853475910815, 0.48415219203129345, 0.5038464286739909, 0.5185981436898118, 0.4639250009626671, 0.4872806251275116, 0.5106898480177361, 0.541155364990638, 0.5102056239780001, 0.4791639013541224, 0.5167197023030917, 0.4862202330982383, 0.4823275129259724, 0.5333960751832516, 0.49618941955109197, 0.5088728308142688, 0.5022894307566715, 0.4955826989740163, 0.45291963892274956, 0.51918774989686, 0.46587584122799464, 0.46708924748319636, 0.4617628803898636, 0.48762126722898036, 0.5199154264834734, 0.5029181303073755, 0.4257094033501615, 0.505008895845108, 0.49385404148950973, 0.5248684989131651, 0.45338460741512066, 0.49250403918812946, 0.49019137647283545, 0.5410249789933914, 0.46478996084880153, 0.4844996929028485, 0.5114458592172294, 0.4906310301875551, 0.48347653589049816, 0.4485751524426006, 0.48544657701486454, 0.4995244898968034, 0.5037259345305249, 0.5137826302272904, 0.49688019001254824, 0.5129392381733617, 0.4587192761288862, 0.5439208365551348, 0.4737268647772438, 0.4835858306621337, 0.45698513706195293, 0.4777785935496588, 0.4923384639875912, 0.5156514156403117, 0.5060740945973798, 0.4813433158212854, 0.49609928183572394, 0.45534085030824456, 0.5046849335474259, 0.45958403078347465, 0.47987820920714275, 0.4698830563138417, 0.48854298821637737, 0.4854789074272464, 0.4951951297214436, 0.5126006379013953, 0.453946143952969, 0.47409811965610177, 0.505042653053282, 0.47754867831227277, 0.47564508268406075, 0.4772648682280095, 0.543056672041245, 0.5568056475096821, 0.5221772345930801, 0.4967379467397616, 0.4880135603833883, 0.4796650923755378, 0.5061097754648038, 0.4698237841714822, 0.4587404954500233, 0.5108964856760364, 0.4761146293463924, 0.489954150721609, 0.5177630671923495, 0.49103330049216903, 0.48723465453998305, 0.5501333989539966, 0.5184385620944868, 0.5113259594994057, 0.5224748071607256, 0.51696631043247, 0.49280619518866536, 0.5151113311244727, 0.4868108672330455, 0.4647916962415101, 0.5220623185607255, 0.475754508709434, 0.5052468799589384, 0.4897638981592038, 0.5152857688692013, 0.5178891614087298, 0.5166969509574553, 0.466947992289875, 0.47070015281072736, 0.5430299279419429, 0.4958500528686499, 0.4564032962361496, 0.5081178665101591, 0.5038644985696107, 0.5142581882605802, 0.49760191442319307, 0.5255876881581698, 0.532643750724806, 0.4966026398343994, 0.4476557883573627, 0.49837048394785816, 0.4879308857915928, 0.48851665839833736, 0.5340755441354825, 0.5271360372177211, 0.48987945691749896, 0.5013148397900978, 0.4745822180844922, 0.5054896104771229, 0.4986715883635492, 0.501350816793944, 0.4461754703579533, 0.48903313973235474, 0.49744761413870553, 0.5044178077908033, 0.49829550771538694, 0.509017709968913, 0.5074982841137474, 0.49051995434720586, 0.45975410028665786, 0.4587748716293059, 0.49397296947323194, 0.5055928462625093, 0.44512446497025815, 0.49499090027832565, 0.516802138816175, 0.49159557477788013, 0.4767255265094645, 0.5043581041913715, 0.501700039249729, 0.4691878656975137, 0.5435153226086504, 0.510199438846643, 0.5019098078729022, 0.5230719553679917, 0.4933818124182762, 0.508163816270035, 0.47480481325941987, 0.4628149064541333, 0.5375728175779625, 0.45985450915769194, 0.48620502491865725, 0.5276892409992101, 0.4889583215130658, 0.4840008408050687, 0.5029770137422622, 0.5317289467234696, 0.4764203680413823, 0.5018731389528835, 0.48899622565490375, 0.5027382623387612, 0.5123760441776113, 0.47261752274290625, 0.47434033462709063, 0.4655543752301697, 0.5347684858899224, 0.4799942683259327, 0.5208531097750125, 0.4868401422644918, 0.5098998629826041, 0.5085444074866685, 0.5079559581160068, 0.5321290137400057, 0.5004869991383494, 0.47657881341312847, 0.4874494267943855, 0.5275433221960685, 0.4884224298772536, 0.5066989063558207, 0.48755631568757746, 0.512412630117654, 0.5286019961371313, 0.5082807303335787, 0.49303426819345636, 0.47711846616285875, 0.4775036818580939, 0.5011401164965945, 0.49474330921966486, 0.5003923685596263, 0.5062255865915084, 0.4647888085104782, 0.4871266539212638, 0.5165987199985629, 0.43913779046087054, 0.5074560013094273, 0.5189875052635421, 0.5250687730944603, 0.47035868975286804, 0.5127994892257381, 0.568585734921922, 0.467515635184222, 0.5120884371074559, 0.4749292655260221, 0.5287766336782325, 0.4963913520541606, 0.5019184829050527, 0.5064891063151549, 0.5013841441058863, 0.4907494675773096, 0.4795358365675978, 0.4985886785768612, 0.5187245187023418, 0.5435801647177406, 0.520883360463956, 0.517862856316892, 0.4817881731817676, 0.5240208882204919, 0.4482540329268136, 0.4900627349063161, 0.4639072926635156, 0.5126338612498933, 0.4427757476080703, 0.48758301767413265, 0.4720546022534434, 0.4984625893843907, 0.540171359905838, 0.4549605668619571, 0.46417680555731455, 0.4329308559681466, 0.4962489364377655, 0.4691986235584609, 0.4976562494393577, 0.47538768100490547, 0.5266930806456404, 0.49667813845613723, 0.4688893076672838, 0.49121221047705904, 0.47682081314223657, 0.4559813686937176, 0.49597065173890353, 0.5146721456726573, 0.48985602026224134, 0.4917339052923322, 0.49401170332552935, 0.5525639874042931, 0.49926290181125016, 0.5160526128478202, 0.5099117599949645, 0.4855754443491476, 0.4834465306426869, 0.5001727486212039, 0.48711473201236405, 0.512256706110972, 0.5309650371974037, 0.5126600392813503, 0.474164117677277, 0.4784797630031065, 0.430355320128695, 0.47253694719356293, 0.5132261442772748, 0.49602035858583665, 0.5107702953030399, 0.5191086903806762, 0.5143148288171053, 0.46030581903058193, 0.47413336652543353, 0.5067231309837229, 0.5047819928192921, 0.4910071457772089, 0.4962329127392236, 0.49448037406937084, 0.5292998602470557, 0.44564184371737486, 0.5137092382114318, 0.4953621146757761, 0.46974321424341997, 0.4763192369704403, 0.4844932159405862, 0.4895170222489057, 0.49381155185742276, 0.5308992248454428, 0.4970619406316861, 0.4480107409315981, 0.6080741705404326, 0.5086272723045423, 0.5067759461343679, 0.4898528365911754, 0.4759775697141454, 0.5116952454872681, 0.47967083483310025, 0.47225527280502494, 0.49953387417202694, 0.5406786136987293, 0.4708466473337479, 0.4719025113093851, 0.4884364940697216, 0.5063000013172666, 0.4403104107446355, 0.5039397858606569, 0.4537611018746419, 0.48091338183633486, 0.5153874546459644, 0.48155999965882035, 0.5116700391686093, 0.48243570990408186, 0.4877496990324975, 0.47994847447965605, 0.49713628860390846, 0.49778877927820797, 0.4571180111931807, 0.5288027548557269, 0.5040310054340683, 0.5356306406666667, 0.48167475551439276, 0.4984509697527016, 0.4925969923690255, 0.47277211769975774, 0.4942201643294088, 0.49655904311638627, 0.521742677339927, 0.4905096293545794, 0.5021186580976796, 0.49448761052812695, 0.4633925740961069, 0.4796777027279198, 0.5266495387578245, 0.4945068571985631, 0.4679425238251603, 0.4600150697155229, 0.5130299444647013, 0.4808763904245217, 0.4724333749215284, 0.5254052284172627, 0.4834062817478223, 0.5147667178872302, 0.49958313054432046, 0.4833159849092609, 0.508367792769603, 0.5067879198984728, 0.5023208972291053, 0.48014812359522185, 0.5015680390707699, 0.4974241104134273, 0.4828411428738579, 0.4670832232802901, 0.4870114226583913, 0.49621218262453387, 0.4840311832042667, 0.4699067170470229, 0.4489322888510123, 0.4648058713656133, 0.49808902291602924, 0.5105138084256539, 0.46088878307684356, 0.48999502630358127, 0.5116502947914322, 0.4938228437731877, 0.5167739371284491, 0.5279228894445163, 0.540589757260713, 0.4623013514654372, 0.5286329628605666, 0.49175415654094917, 0.48876078365722986, 0.4416643211349113, 0.4775824677787554, 0.5019839622359958, 0.5137362111889515, 0.5034377352669106, 0.49555247844890377, 0.5230960409786833, 0.48566021062465076, 0.5029428705609768, 0.46506284170565404, 0.5275533760585895, 0.5130179846502906, 0.47492698103005965, 0.45669031855994857, 0.4649219987096513, 0.45677663718231976, 0.5209437509468713, 0.48449436813560176, 0.4835528109146942, 0.5037355135415407, 0.5269892798086573, 0.4756576324697926, 0.5262593760422737, 0.4990710266850178, 0.48138977817368994, 0.4742454689015228, 0.5351110806922441, 0.4773539816900106, 0.4662997536140669, 0.48591513098701083, 0.45799227998415404, 0.5094220360623328, 0.4918217999706469, 0.4944631499255331, 0.45998715724599726, 0.49694332568946303, 0.47211530307082455, 0.5056379503066096, 0.48782622380525265, 0.5149294831885779, 0.47567456847619377, 0.4879769273491358, 0.47431110798034576, 0.4682666672706926, 0.4860446414310723, 0.47828663868742627, 0.5144981363892044, 0.5272360649351344, 0.5047440354786779, 0.4944600516907035, 0.5227174327008807, 0.4843187743595648, 0.49846619212966187, 0.4625960468626848, 0.5229627908407916, 0.4622310483798452, 0.5334716674639861, 0.4817233877599536, 0.5059934756959451, 0.5152127878713383, 0.45979834455810553, 0.448719864766327, 0.5046057051910305, 0.4982383245292447, 0.5174093268242929, 0.5112611212389702, 0.5143143424208728, 0.5221487686442045, 0.5116557020039609, 0.4748618437524932, 0.5022119467350333, 0.5159840475159881, 0.4804831224963659, 0.5354293260355104, 0.5223320286758149, 0.5080734981576728, 0.5081145718943655, 0.5251923465597711, 0.4954403499678814, 0.5217749825645459, 0.4602182932963028, 0.5049104839086063, 0.4787990886408594, 0.5229415092325754, 0.46134325686460537, 0.5071787989951669, 0.540425085537232, 0.4782772663348869, 0.49346441596706664, 0.5258852108065241, 0.520263726191784, 0.48295513840707793, 0.5233367347103519, 0.5131428667933972, 0.4874817343231768, 0.4699783364014123, 0.5243635095948029, 0.5020874732751626, 0.4576941864088706, 0.5121329998377581, 0.5014329954367396, 0.4749488895725683, 0.4812674026203742, 0.5052564411763433, 0.4517139974843681, 0.5374781096481801, 0.4854993500482712, 0.5424156716044096, 0.5103715871966, 0.49088176539328016, 0.4740153372814934, 0.48907718060152877, 0.49468837111071595, 0.5080193953051843, 0.5150723490922248, 0.5100247240780925, 0.5272591329304509, 0.5075610975000183, 0.46033042001506635, 0.5049834190232558, 0.5346150667887747, 0.4851094905331591, 0.4954289295836116, 0.5032548939966559, 0.5202242561552963, 0.48820057321919186, 0.5184227427794494, 0.49199205918371963, 0.49118216210505883, 0.489425536042445, 0.5384459471623212, 0.4811225968977423, 0.45224698228557236, 0.48676370042178363, 0.500957914759156, 0.5106194819281414, 0.5376229736847156, 0.40999279444461767, 0.49641213851698923, 0.4803603460425423, 0.4936079576164153, 0.5106721677445462, 0.531186123393988, 0.47995127364370127, 0.5456877671051062, 0.438548519077936, 0.48802982624901553, 0.5198350202101695, 0.5478220533103999, 0.5049685230398602, 0.5193304430238048, 0.4921251827265333, 0.4636538215300093, 0.5234814138285283, 0.5161373528937515, 0.5122848829275881, 0.48503146849047635, 0.531147996646127, 0.486550524029966, 0.4503264824254182, 0.5174360110130998, 0.5141387869193768, 0.5602800831528941, 0.44765015677641895, 0.5219789354356561, 0.4818909331196098, 0.5136884676347313, 0.4717410582034351, 0.554336925747507, 0.48664108754654867, 0.5082757101160371, 0.5293248540964577, 0.5041425852987411, 0.5231384689180516, 0.5028426091181636, 0.4863202613028006, 0.4815178387606371, 0.4948510028644003, 0.5026674961826442, 0.49546214673961675, 0.5027067357930841, 0.501335121147752, 0.45332213819121064, 0.4848645738712877, 0.4602737397273456, 0.48967546578935855, 0.4590840765737035, 0.5055992337898579, 0.5286882955508516, 0.4790895299035346, 0.4607178988769989, 0.5340312297952623, 0.5116788630426248, 0.5331216513259235, 0.5075608931027992, 0.4785298962166085, 0.5187123184741356, 0.4963316009625436, 0.5351217993868286, 0.48640582750467376, 0.5174373346799112, 0.49299399964540325, 0.4846881919271899, 0.4926045814921986, 0.5310833709077737, 0.5182826989523855, 0.46578730872050395, 0.4811317204322346, 0.500045855882594, 0.48924632051301375, 0.5394484750276126, 0.49296197176444556, 0.5206372468928653, 0.5190517602129279, 0.4893258258577983, 0.4962083798402057, 0.55176882144912, 0.5207881456810829, 0.5004727311795488, 0.4709467767497972, 0.45233642075075425, 0.4936788694998265, 0.523229740064931, 0.513616531359548, 0.4690320796935605, 0.4860357401544245, 0.5000963130673481, 0.48799637069642016, 0.5143192048363305, 0.4950016418679823, 0.48550106984160757, 0.4569603218457346, 0.5110889212250259, 0.5119367641355403, 0.49571185289867664, 0.4925351701036672, 0.4920082043024518, 0.5313393762374043, 0.5026369585111399, 0.5190603306248949, 0.4752736756800002, 0.50494062399113, 0.5044229032222994, 0.4694958196940609, 0.4689891255588959, 0.48447598183255974, 0.4600621556160152, 0.5184558074865846, 0.5298873965743848, 0.47268989176361503, 0.5230215597662827, 0.4776112470145407, 0.4511349337522639, 0.5246173877819064, 0.49856283993828354, 0.49910605301160604, 0.4970166844197374, 0.5057211874104385, 0.5166183520378168, 0.5104524596224315, 0.4912790770671027, 0.48281529759203196, 0.5387317608941287, 0.5257282697005622, 0.49104406063312517, 0.46719145098953885, 0.44629453922143564, 0.5220399716443898, 0.4874669645613606, 0.47989706060163917, 0.503094806183974, 0.4865189762372571, 0.49231664492111316, 0.4938309150164703, 0.4401871104459998, 0.5171092869857233, 0.4540966048973464, 0.48260188656763525, 0.49033934817526365, 0.4762379903250979, 0.48270923086286505, 0.5056514396749974, 0.5031438815135887, 0.4725568511595859, 0.5241703223203691, 0.5268788672732491, 0.48831338289957327, 0.5065096304244799, 0.5246727300238802, 0.508346220976899, 0.5007590862073821, 0.5033166372132726, 0.4781128182323675, 0.5023931899653071, 0.4655228544820719, 0.5091724661407663, 0.48568861271201236, 0.5170793372829517, 0.4895346070744018, 0.4747987867639482, 0.4922797720795851, 0.483222083671475, 0.4886554809311672, 0.47002661597500156, 0.48497574975562335, 0.5005333372898878, 0.49638447784090045, 0.506220914422523, 0.5355838453185398, 0.4962614788126714, 0.5189191766040905, 0.4746675800264282, 0.5149218653336458, 0.5215595254502127, 0.4912901029060125, 0.4719409757635415, 0.5434601922471822, 0.495645056711897, 0.5324844071746367, 0.5063968784602851, 0.5203421514699653, 0.5088675937264486, 0.5229967147904118, 0.5033334370076965, 0.4874241661930093, 0.48647850888452376, 0.4921641060237847, 0.4747937674345835, 0.5344984193242345, 0.5039969142177136, 0.5082721799892669, 0.5183893492205389, 0.4873724399250251, 0.5164176901134595, 0.4875954430565674, 0.47271143605358745, 0.4706394479853375, 0.4978628595290251, 0.4873541569357304, 0.5080537183185766, 0.5338811957836125, 0.4859827264469067, 0.48193593768635007, 0.47809325989209855, 0.49280203819342056, 0.48696732881407906, 0.5299741107414085, 0.47851983006931276, 0.5039852081108219, 0.4735792572513106, 0.4606803906521657, 0.47087236282611294, 0.4745643379919591, 0.5078452517640668, 0.4671301974948217, 0.4951154600979084, 0.44419034255769896, 0.4855470545157333, 0.531139074207453, 0.47462063000853966, 0.4618077383599884, 0.46671487689613583, 0.49857726984560696, 0.44593481195960805, 0.4800797717672494, 0.47373941903411004, 0.4525345487090043, 0.5277669874928304, 0.5290125017708573, 0.49665029847201975, 0.5286945370025675, 0.49596459813956206, 0.5308203803373516, 0.4794254062703417, 0.5063202959752899, 0.4646481123750112, 0.47587904020947336, 0.5032575402191151, 0.4851030927442845, 0.4434556744231921, 0.5322340717460475, 0.499202464572028, 0.48809243120402823, 0.5425478894067562, 0.4981478195310447, 0.4462482692754786, 0.5177785576121118, 0.4865305003467433, 0.5314374066415073, 0.5622936678502063, 0.49568561722576654, 0.45878668715933074, 0.49238233974131446, 0.5441279175301937, 0.4935210167893673, 0.5274365496537259, 0.458899449326212, 0.530794288758472, 0.4963207994711512, 0.5083886816951506, 0.49141460362865175, 0.50452024684677, 0.4762446264545634, 0.49183636798647995, 0.4664283296917486, 0.5068255849789629, 0.5240338189022813, 0.5179207138791379, 0.478032459491355, 0.4790210241328824, 0.4991312617173894, 0.49083774693515686, 0.5070853315144969, 0.47705273771010703, 0.48670526739478626, 0.5094549280725118, 0.4819073024720634, 0.48454176638398133, 0.5194659673262702, 0.48693590279407467, 0.4954726585189851, 0.4790535240712069, 0.4963634940357258, 0.47988895901767853, 0.5094222341469248, 0.452962004770876, 0.5131218437332208, 0.5260258905362979, 0.5160126092686804, 0.4894077362182954, 0.46068230425301226, 0.5329126438923596, 0.4647918950506139, 0.500144338531015, 0.5054320268154889, 0.5019735806011142, 0.4779985750239995, 0.5248453882875809, 0.5086638825810564, 0.4846343111847447, 0.5050718667411022, 0.4732759827039803, 0.4828818942706677, 0.44802388149638006, 0.5272570666206373, 0.48384351073372894, 0.4991423808951366, 0.5090369954827951, 0.48196304635768017, 0.50941706803399, 0.5290154553519456, 0.5092753038481211, 0.49621713880153595, 0.522806918476931, 0.48076392422241876, 0.5124045918572498, 0.5413562712200927, 0.4745791685924217, 0.515491102763123, 0.4724924273426195, 0.47572034863601304, 0.4937498789123727, 0.4290074278273622, 0.5074336321785742, 0.4984468074922906, 0.43931026682173496, 0.523960552072908, 0.4776652840201418, 0.48325603574146736, 0.48175187728257357, 0.5181645484724638, 0.4942651487896726, 0.5150676780823052, 0.49738394650913664, 0.5191894343089671, 0.4951965033493738, 0.4726990976403631, 0.528670868465567, 0.4880513522399342, 0.4850127351699823, 0.4667680593010036, 0.5034877048773642, 0.5186929700055337, 0.5074047137822891, 0.49350104940281003, 0.4974629541483673, 0.5082764111276318, 0.48950048532110385, 0.5026409536493769, 0.5168684568401727, 0.4898138983577181, 0.5040799381431528, 0.46503156382075683, 0.4943099301639688, 0.49389266325881864, 0.5364327333073005, 0.46332282480226544, 0.45607301713897724, 0.4953660201624973, 0.5150218430259756, 0.46895844699976, 0.5360934056805092, 0.4653682686201417, 0.5236332934349708, 0.5707814517594959, 0.49922999343134755, 0.4890306443787411, 0.46585351339895725, 0.4900878282947124, 0.5289504202679746, 0.49145918360720114, 0.5054822369524681, 0.47181992076741847, 0.4666746779577295, 0.4367303825947144, 0.4923022952474039, 0.49722626969148376, 0.5471583541436481, 0.5089954554562115, 0.4429808041053383, 0.5004152563491319, 0.5064639295606262, 0.5070101746701238, 0.491426086257775, 0.5050523270134064, 0.5336727365471635, 0.5049973666863952, 0.4642646502934065, 0.4944064100996925, 0.47302762396531606, 0.5169017203458963, 0.4937436800040004, 0.46829792083288646, 0.5594343867505381, 0.5236045144903263, 0.49914700969715164, 0.478698261585875, 0.49344257002627995, 0.49223856899015295, 0.49659610972753715, 0.514253622704022, 0.4828243292659988, 0.5206441203417367, 0.4552853080422863, 0.513588179749741, 0.5437882845941119, 0.4959278454733169, 0.5059948404827748, 0.4868634202032336, 0.5299011882457382, 0.43760097235501433, 0.4990660191571665, 0.5645327453846425, 0.5106592186335034, 0.4819890150351815, 0.5270184490465715, 0.5005669318442033, 0.47764165458447133, 0.5264529528263475, 0.4928516118286133, 0.5351391556432504, 0.4692003772326442, 0.4811559951823234, 0.5337340669512767, 0.5577288135678075, 0.4915471085211564, 0.5028743309977316, 0.5095236068896597, 0.4960557221522223, 0.4443663675390529, 0.5499361810817389, 0.5509112096888418, 0.5108838428332152, 0.4805747692387166, 0.5119495229227611, 0.4970877174605978, 0.5123616019920237, 0.4782107068939914, 0.49781345160778345, 0.4853415552152252, 0.494843728330182, 0.503014519269443, 0.4773482195094377, 0.4600100777742284, 0.5053506900220469, 0.5360986684630539, 0.5284877906589179, 0.43931202748548154, 0.513106090226977, 0.4860901499512959, 0.4482024194658413, 0.4970473384456506, 0.4918659410832805, 0.47272394997339096, 0.5379279037341862, 0.49274348945546464, 0.4644377018159694, 0.4822442913873824, 0.5536377345950484, 0.47723422068957444, 0.48872759703173974, 0.4829875594094475, 0.45764816109068096, 0.4931751828171603, 0.5487876216587837, 0.5320122505654248, 0.4523886657759554, 0.47105026476025724, 0.4432530852675979, 0.466864216565588, 0.48422203935141644, 0.4924583310858393, 0.4948653497303528, 0.4823227647563149, 0.5091423876480995, 0.45326874955690055, 0.5133056234896126, 0.5203944934253166, 0.5449371465538742, 0.515646343635186, 0.5140053665810648, 0.49938674489031465, 0.48414284391879114, 0.4926679077930887, 0.49165421498311, 0.5526865438119158, 0.5043131966521497, 0.5042437038640257, 0.4778634328601583, 0.508862690421441, 0.5194180649721761, 0.4864190453385931, 0.4897267836464681, 0.47288966462600013, 0.5131472660691958, 0.4716739498847385, 0.5042078641469725, 0.47881260450214863, 0.45845728431774796, 0.49480768744906983, 0.5171194449361276, 0.5135597031990542, 0.4938907386253643, 0.5188072165381482, 0.4765799363244543, 0.5047655024341108, 0.49021742653666167, 0.4716802610212223, 0.48131270498742285, 0.5318967504738786, 0.5257280121972975, 0.509139951449334, 0.4840249275630536, 0.493633517191919, 0.5017504041710056, 0.4674746885311177, 0.500608784464251, 0.4982636187360263, 0.47852779033248527, 0.5065019073525118, 0.5219920336023925, 0.5168643661749437, 0.49801291020432414, 0.5145740231947724, 0.49551438625854155, 0.5435199153694598, 0.45990386522369997, 0.5101890198454668, 0.5312734105978805, 0.4961582857470427, 0.49974987873708066, 0.5098990851003253, 0.5275437425189297, 0.4666684345775021, 0.5146988095639742, 0.4791941422786035, 0.49176603934634955, 0.559003675722203, 0.4763404213057003, 0.4844977982004605, 0.4574045317923424, 0.5286069980177504, 0.47064748992365946, 0.500992751723281, 0.47798134191033753, 0.48705861522344834, 0.44795027236023105, 0.5118435023503146, 0.5054967549644114, 0.49011884610553635, 0.49100433116415976, 0.5050507896691933, 0.508866059767444, 0.4710404265312715, 0.5084863663230019, 0.49029981210036067, 0.4817036466097887, 0.48618444581219866, 0.47326599640557515, 0.46770535096079746, 0.4911401241835946, 0.5150076283098495, 0.5049969122645424, 0.5150934891520276, 0.4511654861638964, 0.4627345506197077, 0.47234806027359005, 0.43619115311304113, 0.48567339062335474, 0.5262906064829553, 0.460186546275274, 0.5360720572726982, 0.5178297852817908, 0.5067979034934866, 0.5026202170886764, 0.4682344186108553, 0.4926706329450863, 0.5025411502585013, 0.5417188637575828, 0.48977855933682773, 0.5453825248884269, 0.5056940145333566, 0.4894400507890705, 0.48903229236269524, 0.476314314194669, 0.5330402907745284, 0.5366014704210192, 0.4959810417826304, 0.4478135487212325, 0.4621397047396121, 0.4772066365204953, 0.462546737829164, 0.4743478317056272, 0.43674001964083303, 0.46291422598829973, 0.47417745231585395, 0.5304680547736235, 0.502647812198301, 0.47652832508341014, 0.4836324483529808, 0.5205966340227911, 0.5018078163240283, 0.486676899107612, 0.47320726171388156, 0.5185268692591605, 0.48592789190359864, 0.4984949107324724, 0.4805169043750014, 0.478260144830685, 0.500720133624726, 0.5269678165264038, 0.46826411873528806, 0.5090666144940664, 0.45791970856495007, 0.47887513036150714, 0.4937816715609808, 0.5335533898314573, 0.48983321191807, 0.5548058192747696, 0.452195237237067, 0.4917151342451284, 0.49882366715114673, 0.5265800496918075, 0.48946216268152604, 0.5043667782482587, 0.5031128500017513, 0.4929084963782079, 0.4844035397530875, 0.45825521499877225, 0.5018469423788744, 0.5038798189096307, 0.49467852417167674, 0.4923146555138981, 0.5377695630921963, 0.5261811695231173, 0.5263409015148063, 0.46576903338039755, 0.5214072955777949, 0.5286314007678736, 0.440433642418782, 0.45660649373379136, 0.5016283479936114, 0.4824626046845826, 0.48354411094125765, 0.5016092950468268, 0.46483761721387834, 0.521904941839611, 0.5585925149604304, 0.5170600428983201, 0.4879436512136653, 0.49182877885899395, 0.5144806323779034, 0.5098322267923586, 0.5285046446013132, 0.4901024287160935, 0.48178459797613443, 0.49513449981094854, 0.4613703910125919, 0.48974312643156376, 0.5484734894644305, 0.4485155644910599, 0.5173526996361119, 0.4910492219354835, 0.5004593499970816, 0.5722630135347635, 0.46174732429384885, 0.5432517905455599, 0.4998839062466232, 0.5138377533323323, 0.5015075792684223, 0.4723778390708752, 0.53219594777214, 0.5017504451464962, 0.5635429041529473, 0.47368686209658506, 0.4993276410522214, 0.4950331473830519, 0.5018821002011973, 0.5102760573025196, 0.5155923641431038, 0.5200530507243886, 0.48638065347626447, 0.4690243968368677, 0.5117282241676457, 0.5166394762271159, 0.4833456134999608, 0.5266871724206517, 0.5168560236210908, 0.4928589070421656, 0.4815494097578108, 0.5230527094404861, 0.4861440861725334, 0.49587055664170354, 0.498912817578488, 0.45816894665580205, 0.4952505969599637, 0.5357924046657002, 0.5222685633276396, 0.48186461102996925, 0.5049698679805624, 0.47274593918097607, 0.527193020971424, 0.48054200311671946, 0.5367728522160337, 0.5125462439121065, 0.5246895017178796, 0.5176281538275161, 0.4880092914916761, 0.49116892054410005, 0.47396199762311453, 0.4717427122881217, 0.4956885748784866, 0.47745183690688053, 0.512296953449236, 0.5019613506091777, 0.4860411902276691, 0.4761834966664427, 0.4296890369004245, 0.5153034103067202, 0.4941567520909443, 0.4836724137995819, 0.4866960928615728, 0.4732700477756707, 0.5093851771865098, 0.4822341706164346, 0.4666473731870418, 0.48922865655105374, 0.4812720258916348, 0.4834566934699567, 0.5168681468654696, 0.48486812592651185, 0.45414352610993663, 0.4862141296255525, 0.5300298603332956, 0.500155404945519, 0.4848948111882244, 0.5202228731886548, 0.48753503351236926, 0.47554577931207603, 0.4960938684985553, 0.5093259006665667, 0.5090967772902559, 0.5335680157345538, 0.522869044734686, 0.5548027788661021, 0.48157748574462855, 0.4978406760066678, 0.47948774901533964, 0.5342511042489262, 0.4939068821963864, 0.5218361178302836, 0.4907229822243267, 0.5344674356044041, 0.45345092570627277, 0.4785882312006586, 0.5192823756355439, 0.5211343304438237, 0.4884872972689066, 0.4914378030509209, 0.5406707605844745, 0.46433812287969617, 0.48568632849797694, 0.47407118187756025, 0.49048908508863975, 0.5359775919826765, 0.4813885101558183, 0.4835047227307463, 0.46763719567481804, 0.5145908751173404, 0.46949338207700875, 0.4579657665456375, 0.4755037120434889, 0.46662550392711905, 0.4670148038084669, 0.5222814821445622, 0.4963159625648992, 0.4549633128548731, 0.45994096818276975, 0.5070254059791738, 0.4914483618539558, 0.5031206325934584, 0.5133536882212449, 0.4591794874125369, 0.5149762594153036, 0.5151428248527499, 0.498260942343992, 0.49582218680809537, 0.48082737495787004, 0.49264014737722217, 0.45319454967893785, 0.49431204002056284, 0.5091185099754172, 0.5078053852055957, 0.53645648811125, 0.5016583857339814, 0.48584548934434585, 0.5079249755623519, 0.477934104112632, 0.4951111673902321, 0.5073601909366723, 0.5101190665617955, 0.4812905179872927, 0.4661498700875343, 0.5107027600065686, 0.5302873915262483, 0.4989204489581795, 0.5075534200651958, 0.5176131214243198, 0.49771532537621144, 0.5277294321254958, 0.5069344968939604, 0.475478740551905, 0.5190340239431652, 0.4728951955046152, 0.49550239307550187, 0.5194702445022394, 0.48164417925211817, 0.4827364236242184, 0.4825003142913792, 0.45212562677252666, 0.48098691025628454, 0.4753995541898035, 0.4813435036940387, 0.5247333947476482, 0.5007733676215796, 0.5055440898168959, 0.5344829144119281, 0.5788744381782315, 0.4804855700339394, 0.4944260395475433, 0.496211911919729, 0.5426319099430719, 0.5270299236688931, 0.4937130181061535, 0.5055607346178691, 0.4668238059461762, 0.5311003362568489, 0.5294517395273489, 0.48505751226944976, 0.49774174317254033, 0.5012096260392349, 0.5319613791720601, 0.4843227682249333, 0.5157127177507771, 0.5358535700792344, 0.5044666552382056, 0.5335267106888468, 0.49508908577303035, 0.5159563253802444, 0.5102543931712697, 0.5012851764818816, 0.5006458344646275, 0.513664482186727, 0.48287753486345775, 0.5117540910137088, 0.5199987331775144, 0.4609012636647754, 0.473736356072376, 0.5292633703408403, 0.5052559744298035, 0.5017052769008713, 0.48450317725853437, 0.49334563639493806, 0.525992149070441, 0.4986284634624812, 0.5106013077567659, 0.4799173437387958, 0.4946045488426521, 0.5161590955217983, 0.5029816851319954, 0.5309276132002921, 0.48199490452567445, 0.5308151119103071, 0.5135800999543988, 0.45014652708937863, 0.5122643662783337, 0.5161133943766534, 0.48092698501573655, 0.4898056683267788, 0.4613830625140136, 0.5338247375679551, 0.49502557014571524, 0.4955279413696196, 0.523202700448486, 0.5328936764303656, 0.4618076681963823, 0.5178321403664893, 0.46899203981833776, 0.46273664939586, 0.5119355348438225, 0.5212542023257632, 0.4890840322085916, 0.5346343595473919, 0.4997186588137651, 0.5121583073103477, 0.5311493528211281, 0.49744528343308236, 0.5175693052994996, 0.47402985932314606, 0.5561873470785788, 0.4985555278052728, 0.5293740781698559, 0.5189502901524354, 0.5171376981389378, 0.4833962364073813, 0.5025947988306662, 0.4727872261418149, 0.45976953902810924, 0.49125504029225864, 0.49173323580157646, 0.5183445784675125, 0.4629835266128764, 0.5265178409485677, 0.5715731991414158, 0.4833052078498004, 0.5308368994640992, 0.4824775515191881, 0.47672559100548423, 0.5168955722647732, 0.5154089051309633, 0.4861239836314589, 0.44825729359163813, 0.4827199666755242, 0.5009139828720743, 0.46770465650527954, 0.5078665442573734, 0.48824984869401894, 0.5007217726781096, 0.47916291257643845, 0.5324907741775304, 0.5023085479125442, 0.4855100799193874, 0.48713697180344817, 0.5162364399030874, 0.5120527642907394, 0.4822334805165658, 0.5347185948369481, 0.522206159574377, 0.504727065548962, 0.5035562373097727, 0.5023852092855153, 0.4488076267774999, 0.49680424190427913, 0.4876373968862589, 0.509015136599549, 0.48555656925038365, 0.46530931458851466, 0.5196286935091642, 0.5073455936256458, 0.4599159685309638, 0.48459036813321044, 0.5343413061850241, 0.4893507025008391, 0.49637142322257055, 0.508359364376623, 0.5058034557722152, 0.49441538284010234, 0.5033600019872129, 0.4809596097910472, 0.48283208557007135, 0.4700283841168834, 0.48840249188368695, 0.520964110944975, 0.48124963026233264, 0.4933664671246944, 0.446484222073945, 0.46435132234230103, 0.4719122018943276, 0.4932662257174829, 0.498785511567393, 0.48826585905834974, 0.46908513508004945, 0.46757950592497943, 0.5195961216804741, 0.4939610986336955, 0.4723257595670658, 0.48684626246043466, 0.4956254653982702, 0.4632013872939723, 0.48011948125597104, 0.5251218635781366, 0.5065708741722036, 0.49991783380928456, 0.4991774448106622, 0.5539201443630111, 0.5295451652776015, 0.49795187623202797, 0.49459780322344365, 0.4789598131822203, 0.5258635665784293, 0.4385579768023424, 0.525325425640423, 0.4799673833853387, 0.5141758679776445, 0.5238788779802277, 0.5348265090147634, 0.5215422449358948, 0.46742436613496163, 0.4743156207917558, 0.47884100910545613, 0.4686888050037754, 0.47281498555369184, 0.48870940088385395, 0.4996906184145903, 0.4997649833111641, 0.4767337245197377, 0.457427485396994, 0.5179447518746942, 0.46729987366535264, 0.4782573314950289, 0.5197534070344488, 0.48061296015393473, 0.4968993883358342, 0.5214247726876416, 0.47499111418474166, 0.45141829916989207, 0.5045416800820612, 0.5119420579197524, 0.47403600079359637, 0.5094886779141937, 0.5181509129364315, 0.4825444405839711, 0.4701663368013908, 0.45479912737497236, 0.5150551072506739, 0.49652801572850325, 0.46925243457549576, 0.4796930516688592, 0.4987168802576547, 0.5417115114379296, 0.49605842248014764, 0.4876031191329317, 0.4588074224971831, 0.5154977039513875, 0.49293596842633536, 0.5452409925625515, 0.4621129178273588, 0.46698640627670107, 0.5244370064324149, 0.516215241928087, 0.5025418130480143, 0.4984098120738497, 0.5420327318603418, 0.4931546263999096, 0.5201842871997487, 0.5099266088067441, 0.4942414426827819, 0.47656115375970415, 0.48684824218055983, 0.5182860136049923, 0.4723903569482151, 0.4700179368225854, 0.4765872643682852, 0.45078665914757865, 0.5540866591089552, 0.5333720407238937, 0.49134489262062375, 0.5129867387321719, 0.5297858003918611, 0.527798727043794, 0.4670895822465211, 0.4894837660445729, 0.4933732662464572, 0.4956032168342062, 0.46530859404135533, 0.5003510566119549, 0.4883908523149507, 0.4948045028130518, 0.5321968201238937, 0.4916790992306754, 0.4975320392589571, 0.4705827004017943, 0.5040716471202957, 0.504882196193003, 0.5151830446269815, 0.48324050763480997, 0.4629512281664863, 0.48157902278858544, 0.49733796891698867, 0.47083580384822094, 0.48247542637891117, 0.5079948902421387, 0.45183422521161737, 0.5098278809761757, 0.48840107481932016, 0.4737546295277526, 0.5098609671896127, 0.5373551942997472, 0.49034264208946116, 0.5141144911687994, 0.4740124965322506, 0.4724999694721883, 0.46820984874799393, 0.4870041718370142, 0.49660346598332616, 0.46652419367439957, 0.5390093783827585, 0.4907110006403014, 0.5275140225049019, 0.486154538115189, 0.4889420174361447, 0.5160110638499588, 0.4931665034342868, 0.5183781483690809, 0.47802462803815876, 0.47368805627154936, 0.5298841744502146, 0.49389500685842697, 0.4592139580997642, 0.4934178038704627, 0.4914697768490617, 0.501918533948979, 0.4948082235913132, 0.4922924162943116, 0.48315457443941434, 0.537977783386349, 0.5459230739053139, 0.4668860733729132, 0.45959361185609116, 0.4809548756386194, 0.4684738275426778, 0.4842712413617141, 0.4677645702492775, 0.46707475990566305, 0.4808497812304323, 0.45895251184044106, 0.5260357057284799, 0.5134977803936298, 0.5081665083540055, 0.4970436701464221, 0.5779147841546199, 0.4803056477756491, 0.5097895123167898, 0.47378843235442, 0.4823552946163659, 0.5250873094274668, 0.44117606538176235, 0.5122967571232933, 0.4998541415677078, 0.4668728530076897, 0.5240104337123291, 0.4759777059613963, 0.46292866992418885, 0.5090007066584439, 0.5074588714307491, 0.5077731100664743, 0.4989012797758646, 0.47412323990154204, 0.5038634857451617, 0.48953973439807014, 0.5021940684035644, 0.5045000485791032, 0.4904966858297797, 0.46246043472301374, 0.47121378905316635, 0.4355788550960911, 0.49112688978741664, 0.4610214752135202, 0.5121443015509586, 0.5232332660687815, 0.5282906459873808, 0.4538992366703017, 0.48841725726477386, 0.5131158439329488, 0.4918157715303414, 0.5165924804379823, 0.4729052792912483, 0.4937936121883108, 0.5004160844234328, 0.4803376156386287, 0.5309420966054443, 0.5092668627752481, 0.5223351230316035, 0.4712490819985143, 0.5100568989491566, 0.46237916969072773, 0.4825691846532946, 0.47560268313298215, 0.47892770744607965, 0.4587028998416462, 0.5092523004529557, 0.48096985473309745, 0.5126836746739887, 0.48433537941617666, 0.4458333982441146, 0.49013327485418606, 0.4867914686227168, 0.4780287997443753, 0.5001260046984357, 0.48288977823590307, 0.4646011766865556, 0.48377287642695976, 0.5009255253684246, 0.49090305302221104, 0.4426756333289868, 0.49242078210804074, 0.48296378004063173, 0.46635062139959776, 0.5400552315570851, 0.4799608378437589, 0.49049767281468537, 0.5266837451299875, 0.5267373305166599, 0.4634858486061967, 0.4625108910595119, 0.5267582549270847, 0.49086146395425795, 0.5245492421648806, 0.5234056080911541, 0.49287162591704514, 0.47424375926896295, 0.5075628841203943, 0.5091410288299001, 0.5074859894919865, 0.48137893083018746, 0.4994933612037536, 0.5028311461739965, 0.49372824263324966, 0.474361785234302, 0.48039796451298183, 0.5401882380318851, 0.5199105324850434, 0.5337734390059022, 0.4617203567573727, 0.5237857560564827, 0.5117741973756703, 0.4885803342938858, 0.518930537057114, 0.4336087491319338, 0.5010489389707693, 0.4984326044015157, 0.5023313309406915, 0.4951575788838888, 0.4992148925611354, 0.4959354352290061, 0.48042877836482956, 0.5057260741676701, 0.5152743990299236, 0.4929270951768983, 0.5055283937683523, 0.464501864582836, 0.4739091004426952, 0.5469297976807046, 0.4827299770401986, 0.49697254818884695, 0.46116227302869683, 0.45639033755386055, 0.4791484886465173, 0.5533586962159718, 0.5156820120911784, 0.43684640720410617, 0.4563444638696443, 0.5125115283409121, 0.49932633252225733, 0.4711120525534945, 0.4710098485801375, 0.49443260059924127, 0.5293780264220623, 0.5245405815310131, 0.48498628177497427, 0.48216345018120077, 0.48537855140731745, 0.5164234388381835, 0.5162818620262379, 0.5103432153648537, 0.505368180199998, 0.45334411456143187, 0.458633736335393, 0.4864358531264363, 0.47506150610252723, 0.5029239729149865, 0.48954701751620266, 0.5022444633370987, 0.516567059938995, 0.48941494843104616, 0.4678102339719958, 0.46782820926130353, 0.5036685802413362, 0.4858410568804317, 0.4694171378731684, 0.48804476757454857, 0.5161830350199216, 0.5242181912018299, 0.49654808698036396, 0.5456429459378132, 0.49371159563032013, 0.5174191594401879, 0.5021305264570077, 0.529370310301686, 0.5026870088550389, 0.5224395045796268, 0.5194360988237953, 0.4857879555104116, 0.48327923748499674, 0.5051898049677694, 0.48904007319669024, 0.5367497000917277, 0.46629943511854943, 0.49273872867273644, 0.5144379182427625, 0.5155354482293337, 0.49846841069149966, 0.46511103933243175, 0.4941445877185683, 0.5031168300174946, 0.49599342517454853, 0.4927792573041676, 0.49793674219510553, 0.5343140783924195, 0.49590367599275514, 0.527545153261389, 0.5394632247567497, 0.5383595584026084, 0.4999723317458878, 0.48958011217782654, 0.5093877039561352, 0.5491688579012224, 0.5424626031133536, 0.5089586367150346, 0.535321934852791, 0.5306517883835158, 0.5329908786208419, 0.5247792936232685, 0.46199122446968743, 0.5043851268162308, 0.49297080234103136, 0.4505469010406813, 0.4703027384735401, 0.4831127653328277, 0.4949276132264103, 0.502411220206231, 0.4898752640293196, 0.5164317387594566, 0.49708698117742184, 0.4960014498651909, 0.5002002433373944, 0.5226722891778182, 0.5118714875695322, 0.5050686716657568, 0.5088302567859371, 0.5131139074432326, 0.4959253042098088, 0.49604544514494486, 0.5098626238614261, 0.4956952406645917, 0.47459928394252726, 0.45829878024922605, 0.5007346046077726, 0.5312359449617561, 0.5616069435157816, 0.5227246870050676, 0.5236303282360616, 0.50627165157878, 0.4991259970311983, 0.524756989772525, 0.5184867391040859, 0.47460443746690184, 0.5154386200240563, 0.4866793758675005, 0.540897628983069, 0.516899918706535, 0.5511437824862411, 0.46169080899214077, 0.5222441044007398, 0.5259246878319078, 0.5179355030794675, 0.5190000688328377, 0.5036763106939178, 0.44594112059777774, 0.5121289597974187, 0.4923684355108596, 0.5076361157930765, 0.5224806426940292, 0.4244608170177579, 0.5559745805991672, 0.5146000841012973, 0.4607881114944486, 0.4729148573201655, 0.5094207040789178, 0.5005184524151531, 0.4809850138808691, 0.4857302525464185, 0.4663742936677416, 0.507824906285892, 0.49527875989313835, 0.5065726546840572, 0.48246852328474754, 0.47047326000691136, 0.5374792649646755, 0.5243839938673014, 0.5045342332429059, 0.5104525263789148, 0.5072162721682297, 0.450753297270001, 0.49230079718304626, 0.45915346583686084, 0.5076425937318703, 0.4852871220593842, 0.48868603933345817, 0.5019286506771362, 0.48363754788906904, 0.497885597653358, 0.46616671368807316, 0.5382132108774629, 0.5277381982879943, 0.4489106846756725, 0.45250109036215425, 0.48188955544557555, 0.47103461403966096, 0.5265004611368672, 0.5309903909219046, 0.4977945938011483, 0.5207028571486226, 0.4993306638541093, 0.4879223494880343, 0.4908078068090963, 0.5041617623987558, 0.46234243455875224, 0.5268059166007134, 0.4596013775047974, 0.4827180844757871, 0.49568024438712466, 0.5242753636554923, 0.4604905179027243, 0.5293631109256359, 0.5194675614214407, 0.4808428054211971, 0.49366818962490505, 0.4995649456200685, 0.5105416621644421, 0.44515112051377137, 0.4825518749147267, 0.48095391187407444, 0.49900898354931456, 0.5068628959803883, 0.518680825321295, 0.44775149062494385, 0.5079866553730943, 0.5051122507196243, 0.5037292080119831, 0.4789603349534572, 0.49520576399146893, 0.4832419816652867, 0.4987387704036415, 0.49049356584551757, 0.48351532277473963, 0.4559393926506849, 0.47850134894415003, 0.5199311822810375, 0.45730570282341176, 0.45144137863561395, 0.5547887275751281, 0.5143895237676162, 0.5102366131654469, 0.5179266877215215, 0.5153402767388243, 0.4967998163700431, 0.48388493759246476, 0.5167983360717647, 0.5289189415336681, 0.5029342915827612, 0.4861272615453292, 0.5204800601785357, 0.4998593913723623, 0.4915551902939511, 0.5241479052620385, 0.5028421724990643, 0.4161463279914962, 0.49010840099168135, 0.5202315405190252, 0.48000115601748833, 0.5291908258079622, 0.4979665909860292, 0.5185282531351944, 0.5087909935299761, 0.4915055490564926, 0.5085120860284369, 0.48896518036032544, 0.5257788307232744, 0.4576404935281609, 0.5108908926681817, 0.477906570166922, 0.5083017941890933, 0.5094398994583775, 0.4880275841128829, 0.4951041278146192, 0.47161455604292574, 0.5221537893489031, 0.5057904083356198, 0.48324023771791713, 0.4919242602445718, 0.459841530992156, 0.48698338111431383, 0.5119505016673667, 0.5024886477171715, 0.49853964071996093, 0.48701283665991607, 0.4959794160152526, 0.478506128361571, 0.5064905351405866, 0.5186579276925045, 0.4971638006903344, 0.45989707515762906, 0.5162538713099906, 0.467678742498051, 0.485926996847514, 0.46898474124434125, 0.5010835554773451, 0.4909921500286234, 0.47783983707619876, 0.48099854041136364, 0.48841402737174244, 0.5132325556083764, 0.43501780581226845, 0.440293572236521, 0.48617137357676427, 0.5181923668295195, 0.4841143351331802, 0.4690243890947864, 0.4997263517173415, 0.5577204067110108, 0.4717941787950505, 0.4694469105769949, 0.522303106701979, 0.4307088348152734, 0.5111277903995824, 0.5271938757729001, 0.48644949218057204, 0.530356380912099, 0.5191394141077658, 0.5202513798443443, 0.4947236792060106, 0.46853157689753255, 0.4967488000653499, 0.4932502863442726, 0.474426287731994, 0.4641470720256486, 0.5026255750163534, 0.4876699223214138, 0.5185536454041794, 0.47578793198043545, 0.5128124583989025, 0.4788483544953791, 0.5127271384345425, 0.4767794205668595, 0.512720276407047, 0.5046695773621754, 0.4979778702041554, 0.46194100182039544, 0.5006811417686727, 0.5170333040403283, 0.4891652411711451, 0.46290038638877345, 0.5143792222829957, 0.4588554230949661, 0.4669246250555072, 0.4932022846334323, 0.4998031946143271, 0.4913261731061495, 0.5143107802009448, 0.49952854497114585, 0.4707701331545497, 0.5315274252040434, 0.49158733706209584, 0.4911061320179158, 0.5268410923441976, 0.5080089367674223, 0.4850231761649931, 0.46701715282940875, 0.4967013254080944, 0.49175292925425496, 0.49628132601794306, 0.4519862936751216, 0.46488758447973155, 0.47769223624084656, 0.4530885544136851, 0.531912111093883, 0.5136774153894803, 0.4702645224000158, 0.533735983978198, 0.5420161264547918, 0.47147389565918785, 0.4799588781890046, 0.4366512358387762, 0.48374367437153304, 0.48466527718960417, 0.4978963099291882, 0.49960260653531086, 0.5136170500454407, 0.5074167233115027, 0.5131659070517742, 0.48061785747190955, 0.5436998288968434, 0.5032479254303475, 0.5185076751556837, 0.5065299760067599, 0.46493899534278865, 0.48388068143140084, 0.5042613356505063, 0.5044384213571177, 0.4740170582477898, 0.5350672893029708, 0.48183772715669887, 0.4960401123292493, 0.519601088974213, 0.5650065178988761, 0.5173261856569488, 0.4721575252367462, 0.511038471793294, 0.5076032549785741, 0.5176711392588613, 0.4511257131483603, 0.5024553964297076, 0.5022136390388414, 0.5044443161583483, 0.5017743974752971, 0.4757900046912784, 0.49291193959754254, 0.47415537050098144, 0.5027028765390273, 0.5093098590982328, 0.5171736252217033, 0.502041009455816, 0.4718222020640631, 0.4899484909904706, 0.5111338691126517, 0.492740799652038, 0.5035418029219757, 0.4932149928433492, 0.4955298877489311, 0.48349992340810727, 0.5430864658367208, 0.502673523739085, 0.5137744694107331, 0.4801560896605095, 0.4300585758707418, 0.4447399259701796, 0.5160951446726538, 0.5033318892775369, 0.48272642353302503, 0.4813384354034168, 0.5062235442380665, 0.5183634520278722, 0.5211037757818324, 0.49782071947704704, 0.5378467507297746, 0.47741992242576153, 0.5128486845816767, 0.5029659004540112, 0.49544015424920035, 0.470458184886736, 0.5190955122803993, 0.5107733966253134, 0.5142839564583238, 0.505846916095848, 0.4799702199722948, 0.5130222991741235, 0.511650011830702, 0.4694941760618622, 0.527663836177541, 0.48956509441039786, 0.4900872586174176, 0.5512379913456351, 0.49444262205677286, 0.4942024790086526, 0.49279061596791357, 0.49754570763483097, 0.4781371528513317, 0.49034196844772315, 0.46429451022757345, 0.48741803994756533, 0.5092986127410932, 0.5118252844198049, 0.4982820804649203, 0.4991382805364429, 0.5104381121188352, 0.4908347817767972, 0.48960101576979836, 0.4863376903481101, 0.4816657967415127, 0.49744008976181836, 0.5573768788513964, 0.4817686936678703, 0.4659119756741455, 0.44811641938458224, 0.5170970102555644, 0.5047091553061566, 0.5052033061072388, 0.49681252825687516, 0.4796891170089968, 0.5120720404302714, 0.5089053977078969, 0.5268110338876715, 0.4738101847335637, 0.4974883550870556, 0.49156093511877685, 0.49791991688389686, 0.4698617777495442, 0.5708590037736148, 0.5056477117648139, 0.4747841516740687, 0.4968452749893662, 0.5129692550030963, 0.4939566140387421, 0.506747445460125, 0.4936960605502448, 0.4514168375934179, 0.5143044122678421, 0.4739861766118409, 0.4891563424597765, 0.4788964297770213, 0.4804881836905683, 0.4927394809357434, 0.486704849363688, 0.46753310259914355, 0.4682287800052414, 0.5506274940388318, 0.4384388515219164, 0.4914272534271435, 0.4825497539355725, 0.48131518628889236, 0.5272524922662729, 0.49957457569716357, 0.5213207432221061, 0.518333543535283, 0.46678190930664254, 0.5000718225765255, 0.5040691927818948, 0.5204464084966919, 0.4990376816722259, 0.46262181427465654, 0.508696839573224, 0.5177343888929762, 0.5128605170449497, 0.47244116810175546, 0.4961966874358386, 0.4875925170422286, 0.4816615749324253, 0.5131821182390018, 0.5378339876255249, 0.5234275022030717, 0.5235577591600029, 0.43193022440182494, 0.47087454040224763, 0.516544813989929, 0.5279433176169639, 0.5385350578693442, 0.49511219771943926, 0.47062450255374577, 0.5091736743011324, 0.5113886596271521, 0.44912642688946824, 0.4917993523536429, 0.4559311070989873, 0.5020922405040614, 0.489656765608704, 0.5377180299004215, 0.5066610372224958, 0.5114359104490325, 0.4994066701691705, 0.4448075056347509, 0.5124232272261042, 0.49771554744396423, 0.49445417947980025, 0.4693012765840248, 0.5069336730252427, 0.5431718734396055, 0.482913058174517, 0.48874631264461266, 0.4894264914494224, 0.515352912225422, 0.5341678344702797, 0.5015613368313129, 0.49633195907236016, 0.49674547928553486, 0.4455200295352823, 0.5204312368360933, 0.5586688072201833, 0.5348729998590838, 0.5141624899108814, 0.4882842855244417, 0.4928634932465788, 0.4807024578634285, 0.5236449098810576, 0.5021601984503353, 0.496420436347523, 0.458176265416926, 0.5170813717666313, 0.44232523406837976, 0.49707242496282544, 0.519278268992732, 0.5038759963896854, 0.521901435817576, 0.49375176365885703, 0.47271601664805973, 0.4966519148717314, 0.48200945609072643, 0.4820927783545567, 0.5049490136749133, 0.5176550156598896, 0.4854743049871061, 0.4650651825551802, 0.5213209693540989, 0.5272940263584278, 0.4665946642692289, 0.5017344922807181, 0.4856762378672162, 0.5173624610429376, 0.47746633894512136, 0.4617635019794206, 0.5778059894075186, 0.4797253889240303, 0.5514679392739332, 0.45750566132569875, 0.4962650758770528, 0.513810338811234, 0.5123978525170084, 0.5085596118406479, 0.5094429347309757, 0.46806221408081417, 0.5127073795455594, 0.4995226296468273, 0.5100260396295593, 0.4846127066635046, 0.45036059336220047, 0.5116491568445947, 0.5000115517916154, 0.4944370662177306, 0.5218931885514223, 0.507476599910895, 0.47632721772892855, 0.4823491063535398, 0.5644342553229018, 0.5211385784829233, 0.4712776221961223, 0.5021045969945993, 0.5208889703605124, 0.4755907417577772, 0.4930864474028183, 0.5025838149954437, 0.4826472644304883, 0.48607119827056317, 0.5179808556134631, 0.5151119490129014, 0.5132369357968052, 0.48472702900500086, 0.5195792004443339, 0.47928003658760115, 0.492344943612317, 0.48545030783458154, 0.48930157741153635, 0.5040136675378403, 0.4988591544992657, 0.44719809161234736, 0.4535764285340888, 0.4859140722267364, 0.4862878670457957, 0.5285278133651636, 0.5233101072742143, 0.4667155033827516, 0.5100375713954832, 0.5261207019826628, 0.5036909614061178, 0.4749844842242341, 0.5055572490705774, 0.5049164854043325, 0.5150246861317392, 0.5387153018426211, 0.5323734626131036, 0.480126631034155, 0.5010134442059186, 0.46066157116283174, 0.5019032894187204, 0.5153798731967851, 0.4686682203846634, 0.4498409345594561, 0.46387448649361257, 0.5038384628308693, 0.49989337491714364, 0.4974721475656899, 0.49576554852987575, 0.4456883514109369, 0.47406444517951346, 0.4975036280770525, 0.5117347716441045, 0.49547685637659494, 0.47534405103193944, 0.49145782797463816, 0.4652094088763414, 0.48959640559961165, 0.4799759331367362, 0.4780756528077456, 0.48477848606576635, 0.47630566501659927, 0.5172641608435845, 0.4835945477439058, 0.4882256324349322, 0.5151227720202857, 0.5047858305022038, 0.4881633785555939, 0.4960523102061861, 0.4796082776701976, 0.5347883966556805, 0.4837855975624075, 0.4749094246758226, 0.49627230433322356, 0.5058138765635459, 0.4549143106854232, 0.4784115345259468, 0.5303152528589339, 0.481711333998432, 0.4964030182255132, 0.514126394801769, 0.5183117160433627, 0.527206533972744, 0.4793307957094091, 0.5130791138960512, 0.4773397055024605, 0.4477543556983017, 0.4906630089617739, 0.47566739438022215, 0.5074509020026577, 0.5235958928027861, 0.4290683921866351, 0.49233541094337413, 0.5017519227599352, 0.499646973279238, 0.4980374760107882, 0.4846796174587709, 0.5110881386151777, 0.534970085054225, 0.520512279624397, 0.5046790459017769, 0.480619811873953, 0.49597042508867234, 0.5057779348360412, 0.5075016746477798, 0.46749210489687865, 0.4790864552969414, 0.5261089410383925, 0.4840423675560171, 0.5144281575543802, 0.4999233571332564, 0.49560935809631534, 0.5167892217473012, 0.47741984538809157, 0.5486745495108403, 0.4991288443513113, 0.47969167606472907, 0.4843413306576376, 0.49601637483708266, 0.5003048062763331, 0.5259299061264069, 0.5006362508758755, 0.47401878913280365, 0.5105021851737054, 0.5483199316439031, 0.5005430260784289, 0.4992929579030854, 0.4922025361878601, 0.5285847482714283, 0.5009869656296707, 0.5338268130920274, 0.5229670597575362, 0.5080833772125418, 0.4803538549672047, 0.4742861087803463, 0.5265075076343754, 0.5013001624237582, 0.4854346999557474, 0.48224963645795477, 0.5187530614625494, 0.47930071734806823, 0.5159112065624886, 0.48290431100459813, 0.530541252932404, 0.47223942116390993, 0.48364277327141314, 0.4817178815389259, 0.48440159290822127, 0.4956598639063709, 0.47766965552018226, 0.5301099271880151, 0.5055301720695989, 0.51208268067646, 0.5243064880800552, 0.5063449909486295, 0.48633725496782687, 0.5031699775307582, 0.502955193255994, 0.5214605871556118, 0.4808448785557127, 0.48014032559645936, 0.4938380725736791, 0.5068375281803721, 0.48537066957092057, 0.45529864788505675, 0.5086487698296867, 0.5006592046559846, 0.5046965088031252, 0.5165907163857919, 0.49123590439413767, 0.4836280158181572, 0.5051157857323761, 0.5216273250085413, 0.49645309911055024, 0.5063847647340555, 0.4989613754943902, 0.5428259130387856, 0.5306624929306678, 0.5230973657484668, 0.4683255854142236, 0.4738663836683362, 0.4699121292280685, 0.499469140655634, 0.44811852116858225, 0.5019391918101712, 0.49881781906580713, 0.5087329267189669, 0.48760499588771045, 0.4623417833304684, 0.4980646695243699, 0.4733495060416919, 0.5137204929597168, 0.5009072746322251, 0.4678693179621365, 0.515805159528252, 0.45149318294640256, 0.49593501139067825, 0.49933311364051974, 0.4774783968264331, 0.5010478951029892, 0.5093729524956513, 0.47995489970315647, 0.5111149812978739, 0.5035666671752632, 0.4471620207346391, 0.5382546716770887, 0.4797243780594633, 0.4864716230555434, 0.501993589696778, 0.5077669365081361, 0.501228814398094, 0.5183501572833518, 0.47308776438014183, 0.4877511654623747, 0.5127792947513133, 0.520063195433074, 0.5279116313774032, 0.4673626295090539, 0.49954827281498987, 0.4842286085312704, 0.5208512662375141, 0.5199501238475726, 0.5035211898953491, 0.49442809946688027, 0.47327068563367486, 0.5174055727644179, 0.49058498043194937, 0.503560136084491, 0.4742000854538752, 0.5178916422480491, 0.48032388525710673, 0.4850421285818939, 0.4858796294612882, 0.504872444341333, 0.5193510230801431, 0.5169040917340857, 0.5065326070573979, 0.4820561785815771, 0.5260566856119144, 0.49020218386718034, 0.5224246018034939, 0.4622191941206864, 0.4993195009258881, 0.5060001682267101, 0.48421763546818924, 0.4850732306909372, 0.5268429647922398, 0.5082103914994376, 0.47460352883382606, 0.4926398020530002, 0.4729536561496167, 0.4586962654598918, 0.5249908469697624, 0.4629070985544252, 0.4422751216552128, 0.5146971795925813, 0.5000403320641991, 0.4888182908813898, 0.48271856794907897, 0.5521527868658007, 0.4440078250186958, 0.4941095127009276, 0.5160290988862057, 0.4711210843227979, 0.5120217956223152, 0.4741904031036043, 0.47392197529116115, 0.48067267373501177, 0.5006390235389653, 0.5308529432165432, 0.5013277355887602, 0.5087349906181461, 0.5389211116523424, 0.4915331748897758, 0.4701431848569252, 0.49946720685280777, 0.5072172743762293, 0.5191130905980703, 0.5486494757659908, 0.5208867982156306, 0.5258756380375115, 0.5081646397093939, 0.4990944593445063, 0.4902536463039925, 0.5102623411928396, 0.4614311324760845, 0.5180317373113755, 0.5052354009697919, 0.5412762771720655, 0.4719571967195488, 0.5064940233031593, 0.45368915237046836, 0.4953564328918201, 0.5215631783785658, 0.5069754230762342, 0.5508073959765991, 0.4932062307866457, 0.47577284921437873, 0.48065247500038316, 0.5215459147055929, 0.5287584445012188, 0.49142999662835457, 0.5008865499485857, 0.5018282731224075, 0.49890684143934066, 0.5046132966835488, 0.49487176923901277, 0.46685237543398606, 0.4732339640942736, 0.5445911504906156, 0.490110322298526, 0.4884729326763032, 0.5063226019333028, 0.4900876320295428, 0.4560086710531266, 0.47517668795987233, 0.47566610815925153, 0.49472867352675276, 0.47094908990944406, 0.4954752098995262, 0.4941513216445749, 0.5023610791525055, 0.4625107334274565, 0.4939204515213121, 0.5062967489236653, 0.4740330429299879, 0.5290686477002677, 0.4997679762603342, 0.47645134326736477, 0.520446344518635, 0.5389025983043358, 0.5046705977806616, 0.49613293264705005, 0.486669296995186, 0.5371652305717108, 0.471588192964085, 0.4625435859380153, 0.5085296190916078, 0.4766818756490364, 0.4929313636959368, 0.5071191215476027, 0.490690661512574, 0.5214271511617142, 0.49118737072703644, 0.48868623653930604, 0.5228883837253315, 0.4899425738733022, 0.5018508197082994, 0.48502604097095847, 0.5118643447921113, 0.43426541245094646, 0.4864152637142815, 0.5210747925273236, 0.5190166805076657, 0.47665802908768046, 0.4929537279658177, 0.5214627805579956, 0.4898211359750156, 0.5103666767672597, 0.5320448783095787, 0.49158835227293374, 0.4716141667791802, 0.4574094562571464, 0.4738035178009465, 0.48918515445795546, 0.4994996349788749, 0.4471637153172856, 0.51603661331831, 0.4881442366245552, 0.5052891666924769, 0.5018352973153019, 0.4732367469773673, 0.5511601135266606, 0.5205395722915646, 0.48685358852332816, 0.46968043389390124, 0.5305736457323585, 0.4472365477146956, 0.4445873067868861, 0.5232037467566549, 0.49140889594785375, 0.5126831767928164, 0.44337194731242674, 0.47644414470561125, 0.5150557153744121, 0.4929049779855138, 0.5200657260080691, 0.4949812436419718, 0.5336179717643774, 0.5168781884647168, 0.5118781457999793, 0.5188876697821191, 0.5000023147802332, 0.5001030497872563, 0.48096005936683056, 0.5015457417241224, 0.4730712098814148, 0.5016627676109955, 0.4562451198889343, 0.49512148372059633, 0.4870024278026194, 0.5009712595381264, 0.5351590622713189, 0.5306976633894658, 0.4812471528726417, 0.5046912370260018, 0.5287646494448041, 0.4841233383607872, 0.5316226438615602, 0.4894858396015107, 0.5109639660758298, 0.5230213901802439, 0.5298906816008258, 0.4687831356441362, 0.4786364014621851, 0.4932959548750662, 0.4990310630198939, 0.48728995682855036, 0.4847392242785503, 0.4777477825551558, 0.510779145148834, 0.51112229267434, 0.500495610498556, 0.5082230039786603, 0.4882178462396076, 0.461745481372539, 0.5279946266129444, 0.4968347183644919, 0.504078906886577, 0.5057134476352497, 0.5101141404686836, 0.5006321311827234, 0.5057402132024281, 0.5238884738522271, 0.48977256386239865, 0.44744565894393373, 0.514330144246138, 0.5097137324623938, 0.4690005447394059, 0.49855094749945605, 0.5115397688252548, 0.4783146902380042, 0.5065876640799198, 0.4647324910245501, 0.549929240558615, 0.5040803413881937, 0.5053227901826995, 0.4987799689137589, 0.5218792999367192, 0.4893477549928906, 0.5035162019970095, 0.469802929897679, 0.5492944062105087, 0.458663930978227, 0.5140853091035229, 0.5219362242373949, 0.532662325943208, 0.47774694894438646, 0.4804346980114387, 0.4865834080535056, 0.5037392920676973, 0.4843520896771199, 0.46301502213510803, 0.5277263000977673, 0.46841832558956215, 0.5149888869219958, 0.5158119646347729, 0.4955882919018624, 0.5034077522657168, 0.4920384941879761, 0.4863215889779072, 0.543221936160732, 0.5102538501379528, 0.5071913528204272, 0.4607176282794201, 0.4505173994563008, 0.48422497529839414, 0.4851688648443355, 0.4754555372163684, 0.5033382413302075, 0.4669379491894403, 0.49285831964076127, 0.4813337310575001, 0.4907762829212144, 0.5223477693082699, 0.47757296055704257, 0.49884596021886785, 0.5267380768770678, 0.5283797724361681, 0.5207872489993478, 0.4813417976002101, 0.48697944488159856, 0.4855802409489554, 0.4715578885931532, 0.48024484725394573, 0.5227788104111889, 0.539838056560294, 0.5055248309831568, 0.48451334510676025, 0.5118061574079378, 0.5285830452642156, 0.5179198446039537, 0.4847080470633981, 0.5197298761982075, 0.4489717969767923, 0.5264374399356105, 0.5365413268300148, 0.4594916103359879, 0.5165747395894721, 0.4783681617373213, 0.5144274288119741, 0.4756730938613317, 0.5107301842375691, 0.5152024886101081, 0.5174827531949442, 0.5090579031251483, 0.5079181513052402, 0.4866568871029107, 0.5554474703595773, 0.4816111204174617, 0.4949647268014664, 0.5241077818662906, 0.518493049248088, 0.4895958442694023, 0.48798379276632375, 0.49812073961466996, 0.5088342944335016, 0.4960429390730925, 0.5209929611113541, 0.4877367169052572, 0.478021448625646, 0.5040569009914306, 0.5090420476399474, 0.5154676842151177, 0.4993709089618216, 0.49052272581421696, 0.5139053817352792, 0.5052111179859361, 0.4989823916212396, 0.511082640461922, 0.5211223113225859, 0.5053698792829822, 0.49454125905956364, 0.5050223972794149, 0.46583678488184005, 0.48321709695651743, 0.4912175557951643, 0.5078269723461654, 0.5033309048456439, 0.4907862431385553, 0.5034066923098411, 0.5104544379987783, 0.4705075646566292, 0.5498669967794924, 0.4562557450220953, 0.5080726734005457, 0.4934940877127432, 0.4960902667166637, 0.5203191552467795, 0.5061218467372277, 0.5451468199120957, 0.5097183398076427, 0.5587351078163123, 0.47668607221811116, 0.4633347620746468, 0.5338870728246023, 0.4906586741920846, 0.5065977806062036, 0.5302840471786016, 0.4466777141693245, 0.49176265238368544, 0.48584779637917563, 0.5315978939086278, 0.5036268887665077, 0.49085323245805434, 0.5203104619945305, 0.5355265769821397, 0.5005962690927067, 0.5115220751924704, 0.507039445875743, 0.4895136023484698, 0.4896170272952036, 0.47746680771277983, 0.5167036275730942, 0.4857620302735516, 0.47066519889218156, 0.5061830697393418, 0.5173150330402085, 0.5261620649435008, 0.48819987160151057, 0.4775323514427091, 0.5086943130478883, 0.4867332142538468, 0.4957826914534575, 0.4876410671318141, 0.4837627520349761, 0.5351581555252661, 0.5017074185504662, 0.49677547262365546, 0.5155614771878504, 0.4997245856350356, 0.519968734456838, 0.49188601674052423, 0.4859974759531903, 0.49295635227342854, 0.48140385677322434, 0.4556748862136009, 0.5187082282479343, 0.48561590176380265, 0.5053674386264089, 0.5150106266920526, 0.4529765726885746, 0.5050490340912681, 0.496348402115114, 0.5201153492860431, 0.4847855848234656, 0.524848069033206, 0.5036576673208473, 0.4924748081690698, 0.4930202124971994, 0.47762462551862006, 0.4796933804512345, 0.5150001561489564, 0.5488977236219325, 0.4890946961919116, 0.49790530821492374, 0.4970550036220941, 0.5284294142935255, 0.45726113518100286, 0.5190744163484556, 0.4976813720039016, 0.5232612948544414, 0.5172736310032476, 0.5203874861180702, 0.44067618121836827, 0.5291980099889083, 0.49698303739552746, 0.5474579416471246, 0.49993603227297356, 0.4510979705345353, 0.49120158209913384, 0.5164290524020383, 0.49061371850192503, 0.5026527167747966, 0.5102084108836352, 0.47375941553534423, 0.5494517518439738, 0.5031873820934973, 0.5090982523656222, 0.47345923837569853, 0.5147107101072366, 0.4934212684330306, 0.5077311547788187, 0.5176652578064633, 0.4971765418399353, 0.5029740405786873, 0.5368217993069317, 0.48671094331075443, 0.5106664741690858, 0.5388236845186991, 0.5060933152460484, 0.47334921595559143, 0.49345822665712236, 0.4653516780382232, 0.49810973229500627, 0.5013972569463887, 0.5034105173999378, 0.507559120732772, 0.4919696875670641, 0.47169955534167674, 0.4978393556274431, 0.4718112307277305, 0.5013063954677817, 0.4991842677322765, 0.46475723640542715, 0.5231266220430061, 0.47825157469586954, 0.4780035238522504, 0.48657044796284893, 0.5021230405447165, 0.5135272707358444, 0.5111382906697837, 0.4844207210825882, 0.5031814056843551, 0.47339503163066143, 0.510514309791975, 0.5234969453419973, 0.4977451708346184, 0.5139368972418891, 0.474912288565273, 0.508008465007755, 0.49586940991796635, 0.4595105688682528, 0.4979334224350905, 0.5056351106767688, 0.4928987400226145, 0.49607400723783807, 0.4644405737506188, 0.5365916279095533, 0.5121403094541176, 0.4965069041201771, 0.5073911567485357, 0.5286816557454015, 0.4984814825827525, 0.537449319771778, 0.48529657045282326, 0.5384167555964715, 0.5289704388770823, 0.5081054671805147, 0.4919455945435382, 0.4905861461240143, 0.4893694114372908, 0.4866154272179887, 0.4925395449362742, 0.5169997884257408, 0.4685843511142033, 0.48352145737754215, 0.49646980904090066, 0.5131912297223805, 0.5740474998564469, 0.4841530414434579, 0.5366597551250208, 0.4565389417239097, 0.48864303703195755, 0.45666705782708156, 0.5122764541183719, 0.5066894444419603, 0.49867147939176576, 0.494732350339155, 0.5260382514992145, 0.502037064048717, 0.5383285531914463, 0.4998744050353852, 0.4854356948810154, 0.49375207831288065, 0.4627428053169934, 0.5103349327056901, 0.44766426712663854, 0.5027671763357644, 0.486306131960471, 0.5040288073728428, 0.4665592198663636, 0.5212697881967245, 0.4862805454845756, 0.48588636959986575, 0.5409328352246431, 0.49871508621941063, 0.48542066296445147, 0.524871651044702, 0.4664078235617385, 0.4884336704768173, 0.5017409755020091, 0.5408178176527733, 0.5022457521726165, 0.4873434467599151, 0.5180513515941902, 0.5036115601523792, 0.5359025759212949, 0.5026146832283376, 0.45360855439891967, 0.4953569898797466, 0.4406414865685591, 0.5122330555155158, 0.5399756786591529, 0.5220542884572942, 0.4738612077546511, 0.5054632915453734, 0.5165591027229389, 0.5038607520644964, 0.5134003505442573, 0.5041336177930611, 0.5364243244946639, 0.5292355516608814, 0.5198333809367947, 0.5037511764825574, 0.5035761840589106, 0.47364112061400127, 0.4872951753364275, 0.4720929727706488, 0.5047599647339065, 0.48415432716797363, 0.45572928566413035, 0.49470937474375015, 0.47692840758401395, 0.5216094287798818, 0.5112834962256858, 0.4906234783767035, 0.4802500675294833, 0.5489858969500483, 0.5083503689152723, 0.5252639836474929, 0.5342807443290303, 0.48668140117822467, 0.5124279315326759, 0.5143104841511571, 0.48073804098622946, 0.481807583657121, 0.5032335113688414, 0.4730576991176489, 0.46056599952488975, 0.5051909728178724, 0.48377135130408244, 0.5002748251471943, 0.5181654299825534, 0.47626653531525776, 0.5211050278505804, 0.5171401826651995, 0.4765504801612457, 0.5257561103802956, 0.5085080952751747, 0.47478221743792975, 0.47941580551648544, 0.4938735908819204, 0.4856036563866209, 0.501308017103244, 0.4939958030312201, 0.5218293226771172, 0.45307245875798224, 0.47834656177374385, 0.5319665841969449, 0.5084541138932159, 0.49919047003060085, 0.48775541909730297, 0.4633218965510894, 0.45492438955666237, 0.4782263932458449, 0.5305385089199488, 0.4786288041719523, 0.5001683090802779, 0.5071842332022181, 0.5137319891643022, 0.51934899423711, 0.5104372369490766, 0.4807812609532516, 0.5227056155656296, 0.4818654152784224, 0.5035690571631214, 0.4474348285904018, 0.4891279537822979, 0.5266721718253751, 0.4714286771588369, 0.5006419090250512, 0.44552946920076264, 0.497137728937293, 0.5047713617627183, 0.5030441597385387, 0.5015581032182979, 0.48821968244660524, 0.4792686313543682, 0.44887518213468086, 0.4750921348974551, 0.46292295042221343, 0.4587175512810287, 0.4963912028849844, 0.4791069030974156, 0.5103252637319985, 0.5000577028570008, 0.4841937104901546, 0.4924091171749904, 0.5088259944447469, 0.48784998955751374, 0.521197834488063, 0.5074782601985525, 0.562309778303667, 0.510347773063564, 0.5052684482163182, 0.4705939276232318, 0.4738732107348847, 0.5460413739187958, 0.4619633018190072, 0.5330046191986878, 0.4790735729761248, 0.516408864290254, 0.4701641110778587, 0.4717253994012577, 0.5000773886100461, 0.48301166343424706, 0.49264979780172113, 0.49954098937688396, 0.46086331405912445, 0.46981327777630427, 0.5373649994736925, 0.5085457997233707, 0.5296830960683087, 0.5419402333279592, 0.4659104203559099, 0.5013224212367334, 0.5165584262645762, 0.5052008314214365, 0.510052919986865, 0.5212035655202374, 0.49350271711991334, 0.47830144624937554, 0.5597293291040181, 0.44781354245709415, 0.511460171902194, 0.4840349658817889, 0.5535809389809672, 0.47832382476721125, 0.4936566307212325, 0.48561941701413275, 0.44567357067621927, 0.49128320185367924, 0.49826319265298946, 0.5126807850067535, 0.48551848246406787, 0.5334784629923514, 0.4835258888382498, 0.5263703919965739, 0.46032020494183745, 0.4783476458340176, 0.4610880718482182, 0.5179481822128261, 0.48962913212001347, 0.46810280925167486, 0.47858565135423686, 0.49062251844895316, 0.5161959469968063, 0.4640267960575153, 0.5063774977753813, 0.5490206126712851, 0.5125019571321054, 0.47823473611146045, 0.490387112432969, 0.4836367418443142, 0.48260471523533915, 0.5077074952167422, 0.4727052155161302, 0.483533906454697, 0.5146505699605466, 0.48289292853432575, 0.45487621032445985, 0.5084638271747404, 0.504828034981918, 0.5025414282543603, 0.4662801320418807, 0.4899417106786317, 0.4925075510903138, 0.49525086496992954, 0.4863686056972092, 0.4777572370953531, 0.5168863444462856, 0.4999458004948174, 0.4559501056380632, 0.45409828910111594, 0.4426487304782429, 0.4786482549410503, 0.5171017270482701, 0.4637756609027859, 0.5295920761720222, 0.5087132690787428, 0.4662352512923481, 0.5600562833161098, 0.5758663254616208, 0.4886466636596472, 0.5528322263830321, 0.49335808692151656, 0.4631965929964994, 0.4873469230037323, 0.4819892530554417, 0.5137847590173716, 0.46239914252517667, 0.48495843322184123, 0.5009884854775406, 0.532312036017132, 0.5000541409774462, 0.5076280400224945, 0.5476764822556194, 0.4820409677252882, 0.505375292078244, 0.52403401742407, 0.5140812493781469, 0.54715457591586, 0.5001473411129472, 0.4864090799898171, 0.44994290883993576, 0.5069445628444885, 0.47597329872109656, 0.49158405821124873, 0.46631756060635404, 0.4563213788778321, 0.4434920884632992, 0.49389841227203213, 0.5065800348777681, 0.5130987826439685, 0.5416482860402869, 0.4958967190869766, 0.48053509379267334, 0.5202664226196206, 0.5342866922993813, 0.5091411293290409, 0.5150796801594452, 0.526964391085621, 0.5084898763102499, 0.5044730626699382, 0.5261879313389014, 0.484483439392179, 0.5133157906982371, 0.5314790829749895, 0.49562402703891845, 0.49669084847250206, 0.45490798939540633, 0.46474148705409035, 0.48068063914241044, 0.5032893148036082, 0.4914367833004361, 0.5083441400261034, 0.48539379764969903, 0.515754356024211, 0.5163143970564983, 0.47369467765117956, 0.4735507577724289, 0.5076094336649143, 0.4767488616803267, 0.4622725004874117, 0.5090946676753321, 0.5004756373009536, 0.45023668663212535, 0.4917375910994822, 0.48160830028364493, 0.4925751309695673, 0.4942084287807685, 0.5407092590232976, 0.5192874782192368, 0.5341642115533042, 0.46037866318038645, 0.48361906461468046, 0.47978586278276564, 0.500864028889324, 0.5074568546103095, 0.518685340915482, 0.4778721958744824, 0.5089669032198045, 0.4747781501324962, 0.48853669470417527, 0.45261711757491924, 0.508425285099383, 0.5122208117040264, 0.48331528974644267, 0.5001566522643623, 0.5464717663185696, 0.47793710212370605, 0.49537104780445285, 0.4795177289354201, 0.522510777018898, 0.45289895130157054, 0.4780534878899107, 0.4702832196515977, 0.5026923775054448, 0.5426591222332936, 0.5074778373581857, 0.5573560829409523, 0.4689102756219853, 0.5141278033082428, 0.512789769554108, 0.5462207647086215, 0.49755113996356387, 0.4993976835160826, 0.5003572815085099, 0.528500433753531, 0.5025594474380696, 0.4882175717285031, 0.5336187642245713, 0.5164997840174755, 0.5321626478079446, 0.49728250082706604, 0.46392270491584603, 0.4862147307055372, 0.48215186517130826, 0.5063648236689555, 0.4765932270354985, 0.4609532926467033, 0.4505141009019516, 0.4661128689080361, 0.4719073095825457, 0.4963800610690133, 0.4605029351319996, 0.5139167581411753, 0.4800037575582151, 0.4649879378934307, 0.4870651362833496, 0.5165993708802794, 0.5118330563263375, 0.48561629226028374, 0.5104406304445296, 0.4838172624092749, 0.4721379310692143, 0.44488569918910564, 0.4959590477589639, 0.49062759232029934, 0.5143818861984913, 0.48047372028988444, 0.5256625221591419, 0.5076552394584766, 0.5140110274224069, 0.52264154786505, 0.4339118924719938, 0.5125227424679222, 0.5490395991877124, 0.45914272388199556, 0.5079981201724333, 0.5156027162620782, 0.49027077452923623, 0.493434318798283, 0.4750058494271672, 0.5263285844935922, 0.491182752686038, 0.4577795909163014, 0.5145883851725187, 0.5782975731224789, 0.5316721171066278, 0.48712746713008737, 0.44686653074556953, 0.49716972265426806, 0.47827268605554435, 0.521338511767456, 0.5581072548019671, 0.5098626257763488, 0.4835482779277558, 0.4843580006934833, 0.524002122976619, 0.5067928157157017, 0.48987448354273344, 0.5369816054595338, 0.5600010110305734, 0.44763481632845525, 0.5103018486170012, 0.5383089046535953, 0.4956181050499296, 0.5033905451708666, 0.499294734873388, 0.4666645385943055, 0.48906380499490826, 0.5133837816125126, 0.47742822332694584, 0.4694486105973164, 0.5496291408924157, 0.5430547612233053, 0.5039513755876345, 0.4588159029015156, 0.46017113594599995, 0.47745852887020956, 0.5356178778230904, 0.4989375648424471, 0.49189560645127434, 0.49301368543009494, 0.5054850252305171, 0.4914844349863841, 0.49462392420954754, 0.4799204891119753, 0.4649468331146815, 0.5492371910706745, 0.5184265693754677, 0.5554870589894473, 0.48606380142552114, 0.5394320064392787, 0.5060574722531532, 0.5386644170135338, 0.467803187771266, 0.49028301571210514, 0.48549381650545265, 0.5098649385025793, 0.46212546342106264, 0.5017672268565492, 0.5231289712107292, 0.49614027729778337, 0.5058098044988997, 0.4955792125423809, 0.5130099471438359, 0.5070132051735429, 0.49028488175800594, 0.5136073511610885, 0.5390585280197553, 0.501978313934565, 0.4734998629847222, 0.5308257586774566, 0.49718547440435695, 0.45218166861672177, 0.48712609926614225, 0.4949414546437864, 0.49369928936591595, 0.465952295922704, 0.5124607213777984, 0.5050941919860608, 0.48957780126844036, 0.5125128939821033, 0.5272628583524358, 0.5187811696018062, 0.5590353504741843, 0.5191279991585817, 0.4727603124817721, 0.506437108217515, 0.5139414781911651, 0.5368804905714757, 0.488506102420222, 0.5001015575033242, 0.466228295492045, 0.4778861753031141, 0.4663017678997744, 0.48136232739904167, 0.5054727417370017, 0.47364533537445047, 0.4971785251469732, 0.48172788151511087, 0.49815157915862923, 0.4903045918642144, 0.4841012658786621, 0.42835465680443185, 0.5312565726634699, 0.49258418661143727, 0.5157360329232962, 0.4904725491645875, 0.5005577384567759, 0.499113294955196, 0.5091697350316012, 0.5015825015320992, 0.5405251686228386, 0.4817890296257113, 0.49689769600028355, 0.45962987366297725, 0.4852413114597774, 0.46880349975153723, 0.5150063651125799, 0.48142482144657944, 0.5254200416099861, 0.48109636802099687, 0.48731289496170016, 0.4733107252591771, 0.5053035783390036, 0.4878369586150852, 0.4510348503635458, 0.46322240399582437, 0.4883300608937173, 0.4955404120406356, 0.49182743888144775, 0.49968667566807573, 0.5445616239018037, 0.49513699366078046, 0.5080742403015159, 0.487392717068902, 0.48821985184531, 0.4962442119006419, 0.48077980907611545, 0.4830857039972402, 0.47235753276837994, 0.47815182474799584, 0.5017226347237949, 0.494350813979084, 0.4980288314806301, 0.5109350143190639, 0.5143187989870222, 0.46416456456309174, 0.5430022185155496, 0.48222207823761293, 0.4570145657487081, 0.4990845212783685, 0.4779311302014384, 0.5118682249284635, 0.5165340303114372, 0.49732935447659116, 0.5246324252123392, 0.480905241361141, 0.4954988524581204, 0.48263652956075337, 0.4971262753742437, 0.47210767233609013, 0.48881268508178083, 0.47582489307560566, 0.4887880417259626, 0.556087984685622, 0.5029949250277688, 0.503065829705865, 0.5044671753976311, 0.5254155723268665, 0.47940452569560243, 0.4977355854653595, 0.4982328204686016, 0.49291259855256675, 0.4958145824883068, 0.49224719903288694, 0.487826317743412, 0.489732227356588, 0.503869959069654, 0.4891859107991315, 0.4918947447940447, 0.4900391898424503, 0.5073975736733957, 0.4808378119901811, 0.4993704503521815, 0.481622176524905, 0.47577689431927883, 0.5116768097614856, 0.5100353593312948, 0.5068042129725641, 0.5342760664775635, 0.4808617900912797, 0.4475742226913546, 0.4613996379606861, 0.5140823169308594, 0.5204850805286019, 0.49473121165223133, 0.4374044172270914, 0.47023873088370605, 0.4672306779984735, 0.5130076968855428, 0.48021505522379154, 0.4942618531950053, 0.5009575411039985, 0.4745558947799114, 0.4894227157210653, 0.5217359343915251, 0.5205543350777615, 0.5064331369004363, 0.4823253511410033, 0.4945222583297032, 0.5056182796701021, 0.47902430212325653, 0.4411698447059039, 0.4831504652991438, 0.442658866812537, 0.48608033807499085, 0.513736324377298, 0.49219381241736715, 0.49106038797466917, 0.5017547654445259, 0.5144816174136392, 0.5128118269093179, 0.48482901496526326, 0.4793818506227965, 0.5167958345118006, 0.49915238927983896, 0.4647726713902787, 0.5089170507296478, 0.49540029020100246, 0.47637406223371553, 0.5187959418146155, 0.45549520772034635, 0.4987767820184002, 0.4980968636546499, 0.5019989595206139, 0.4660220300942611, 0.4811583532808998, 0.5090622789696352, 0.46773845286064897, 0.5202237870512975, 0.5187686218739881, 0.5056702700590555, 0.4836774566872991, 0.514553469156237, 0.4878553553722398, 0.5384429857529542, 0.465503536269752, 0.49363192712036447, 0.4977355399348146, 0.500513988810464, 0.5527447074299755, 0.49626419627217516, 0.49745653554388936, 0.49109524414314937, 0.46665238393793557, 0.49243046819593067, 0.49820586446556936, 0.4887615505305815, 0.44245329644289466, 0.4883456320712478, 0.47250141941957924, 0.5036406478977871, 0.45918613198908925, 0.5422070166035582, 0.5168914159315839, 0.5061540730321014, 0.4940272715423226, 0.4964173671148295, 0.47000564369674297, 0.49932771026037426, 0.4757361472926176, 0.44954429262767026, 0.5179434508171095, 0.4643776800705282, 0.4947588268348254, 0.4883016445234003, 0.522405065115183, 0.4999818138025314, 0.5010984944186677, 0.498015061342205, 0.4985663204465725, 0.4797270098830254, 0.5086055250615228, 0.5361368638827344, 0.5011605603858442, 0.5415292259079374, 0.483084223446646, 0.4984036974526457, 0.46102965523025025, 0.4834661314341138, 0.5044139296761833, 0.5229960037308176, 0.4485333319400234, 0.510245124039359, 0.47652775623185045, 0.4659848978896384, 0.49552009852677015, 0.4738014591252026, 0.5184928765490088, 0.49150075313583247, 0.49463423227549863, 0.42943894237017605, 0.469394577524563, 0.5095554956727938, 0.5362658812920928, 0.4468442357772237, 0.4854058425805204, 0.46743684210500525, 0.5665337218425027, 0.4591586318351864, 0.5252682015165336, 0.46600742358505065, 0.4478714499144648, 0.5447773851102994, 0.5517610629035726, 0.4915429080430739, 0.5055434787266608, 0.49260125063234, 0.5040162531378294, 0.5257999866146681, 0.4780808493844075, 0.49739224822450206, 0.530132561626525, 0.5412732373980569, 0.495753241512037, 0.4966266278935757, 0.5157158207241201, 0.49480982372000165, 0.5106805936595779, 0.5310469602207611, 0.523915162804445, 0.479587389602257, 0.4795658044162308, 0.5210981027300783, 0.49918628761886824, 0.49391518379097227, 0.507401841374256, 0.5224791397013562, 0.46033565710077284, 0.49068706120086125, 0.4865831984411677, 0.4931293155214688, 0.461754492118472, 0.4747342333191672, 0.4829756176749143, 0.47083984895673797, 0.5152890755649078, 0.5102734128655674, 0.47005143951493705, 0.44515622815718003, 0.4713547809579839, 0.5252609705921538, 0.5355081558625202, 0.5026200174711919, 0.5018859248174072, 0.49690540155876817, 0.474046254623047, 0.4653166716268592, 0.4755512457398488, 0.5440975817219553, 0.47477003377840354, 0.48319944281002153, 0.47455746168258606, 0.47897466868284766, 0.4858113702155002, 0.48915819048765435, 0.5180627944814848, 0.4908769711867389, 0.5007594937742457, 0.47618085404511307, 0.5166743719617349, 0.520423917752705, 0.4660762704306149, 0.5081676554735468, 0.5134260970646907, 0.511948756776805, 0.4478813315684581, 0.44388769407914447, 0.48461576953755126, 0.49739026235995676, 0.5031811526914787, 0.5096713835465688, 0.5217695429215672, 0.515744750366862, 0.5103379151223059, 0.47308912358186644, 0.4789151508827385, 0.4678251934185299, 0.5350024173860699, 0.5125834989140838, 0.46649645275441903, 0.5045926671036887, 0.4803358114327313, 0.552808562653131, 0.49874053375164945, 0.4782964783511085, 0.5095886252687754, 0.5161270974215247, 0.45651972928299617, 0.5072905298787442, 0.4470837240500708, 0.5079945484716357, 0.5074924799991429, 0.4854395597749908, 0.480282416508274, 0.4843396460969244, 0.513916898360826, 0.48974377121801627, 0.485083635458426, 0.5153777979906122, 0.5009483439518083, 0.519907183176795, 0.4874037315763975, 0.49978091754045134, 0.454790830184034, 0.47570292933886477, 0.4627997959722958, 0.541004417225646, 0.47072848812269513, 0.5191529445170662, 0.5167461561156457, 0.5459081640220903, 0.49895972680652273, 0.4846547798183553, 0.5078970513017297, 0.47930104696168035, 0.5337479461329355, 0.5157749687335086, 0.487062156109237, 0.47117060325517784, 0.52459014536522, 0.4892142713864052, 0.5164562751988676, 0.5358470003182221, 0.4663439657660381, 0.5180117223138513, 0.5228147185879225, 0.46154220846420374, 0.5028378764390992, 0.47059670304642326, 0.46835324077534474, 0.48195018630302366, 0.5047102274809261, 0.5296999460590953, 0.4650005713380372, 0.5263624693859524, 0.4597869285615747, 0.5275868500473956, 0.484540660048217, 0.47574502666397417, 0.45294414316680337, 0.528192456010374, 0.49005183820384374, 0.4764221972635695, 0.5129422779553549, 0.5505293328529904, 0.50887060864181, 0.5264607385526795, 0.5046263613792431, 0.5127216072884903, 0.4776314673177316, 0.48445434278781196, 0.48023080430248327, 0.4605749596156118, 0.5054139729156634, 0.4791997497652435, 0.4780952120543052, 0.45605725388219454, 0.5214809556741135, 0.49020409341501486, 0.5391529248949903, 0.5278333198265668, 0.5178245287433673, 0.5579037891513654, 0.5049121022807547, 0.46877112471584514, 0.49346297831015307, 0.43943112761357633, 0.5060982653507848, 0.5118328731265449, 0.4836614298678378, 0.48418002284534284, 0.5021221267547108, 0.5227902870962731, 0.4755933441018976, 0.4741764293427882, 0.4817289109572133, 0.4934448405764933, 0.534012440100508, 0.5155852562009562, 0.5229151365214094, 0.49049678974455957, 0.4823093750377838, 0.4999097765936847, 0.506935282920249, 0.5384692593341078, 0.5046154614060101, 0.5096983571066103, 0.4757769267280555, 0.5348536883861639, 0.4786068384572648, 0.4844598922998309, 0.5043472528600871, 0.5106772349963602, 0.4964303628353002, 0.5190785602563742, 0.550507378003878, 0.4859357526195421, 0.4569839157575281, 0.48131124866629893, 0.5031252257898843, 0.506999134438011, 0.4915661077524607, 0.5316994603358561, 0.48725755037933294, 0.4703991029846071, 0.5139967474578334, 0.46084271577650116, 0.5539557925237284, 0.507231861473116, 0.4568527123066753, 0.5104491457932601, 0.5043367326347697, 0.5162471202165323, 0.46085928358722467, 0.48874528528322464, 0.4900272506991963, 0.5057861398770764, 0.4943551776003504, 0.500526958709532, 0.46304987144077353, 0.49797771174076605, 0.48195243810853056, 0.5483544908588205, 0.5429903321122753, 0.4857512882331588, 0.48448469985298914, 0.5692751171408272, 0.48109927100972877, 0.4964133736638695, 0.46623142273203416, 0.4843115785866789, 0.5143988178559098, 0.5124977754931106, 0.4758842025566146, 0.5228116236351635, 0.4439787469837337, 0.49772697650996084, 0.4790738899219478, 0.5278495900119659, 0.44080843090839655, 0.4765802212850313, 0.48820525804271325, 0.5157030469810846, 0.527915226692381, 0.5039954986655065, 0.47072917142586373, 0.5331515274480911, 0.5134064762779248, 0.5200405015194701, 0.5182606861323465, 0.5084087655827965, 0.5023157467839484, 0.48203532871157084, 0.461658327701127, 0.5413751480499867, 0.4938581534265151, 0.5183608597733391, 0.5341097088408187, 0.4919004595872817, 0.522691820072709, 0.4985264046549983, 0.5061233325258551, 0.5068265880520324, 0.5138673325902828, 0.5119429579356533, 0.5057905528777888, 0.49989405652557156, 0.5079553704150681, 0.5154656850278103, 0.5085634593323242, 0.5032652556446856, 0.4926523863236722, 0.5030302023541618, 0.4908578994915492, 0.5078569457881167, 0.4807966934828061, 0.4652657719480426, 0.519675521167741, 0.5153891199292651, 0.4543768861477599, 0.5020880236430175, 0.4774474696393081, 0.513393810051364, 0.5158855477962333, 0.4803822452230529, 0.5201452470800553, 0.45134187195507347, 0.4830591360995435, 0.47365661597046904, 0.5009496165098394, 0.46372261528028924, 0.49001740452207015, 0.5093847421047097, 0.5170376546042033, 0.4806195238071347, 0.46983223449598527, 0.476688875498283, 0.47802856913528874, 0.4822372865337236, 0.4581813111924746, 0.4598366811204911, 0.478055879827009, 0.47369663159116554, 0.4980258264870626, 0.48725182055846883, 0.5228116749517696, 0.5208271763104226, 0.4699998007930334, 0.4919727743586958, 0.518663420910464, 0.5234813559519877, 0.5451852066836265, 0.4977194028063034, 0.48719624726794486, 0.5298543136067314, 0.49777566283438396, 0.4666688393373184, 0.5154433810694041, 0.45052961815217163, 0.5178073150598973, 0.497992377713724, 0.5062833091066936, 0.5033014446823527, 0.47362485854094516, 0.46252553700093907, 0.480883605592953, 0.5289947964794763, 0.5071425886345219, 0.5146322418372071, 0.4965347407674854, 0.5363845779815635, 0.49865919976601014, 0.4715672645787377, 0.5134847183241092, 0.45983092040621615, 0.4619136877591052, 0.5432587150035895, 0.5125277345263436, 0.5421554044506934, 0.485790815218634, 0.5425275912358059, 0.5049187630000731, 0.5286974226545843, 0.5107778590195315, 0.49118030078988867, 0.47424751056516223, 0.45698221252020854, 0.4597051401371515, 0.46881698939858524, 0.4651934134650595, 0.49330381154776953, 0.5006608999130719, 0.48524359130167666, 0.45328608430710543, 0.5047969800007812, 0.4773978647663037, 0.4912637250986932, 0.47876670921025893, 0.5112786833260086, 0.49923693776145583, 0.48918257904109214, 0.5008676017402984, 0.46190696742018517, 0.4834225937010925, 0.48561316653295217, 0.46854411260110046, 0.5224414662503026, 0.49388308651401563, 0.5406124504289394, 0.4866653408173564, 0.4906424359702069, 0.5205120851706154, 0.4955361565348318, 0.49607558970283255, 0.5097177842991493, 0.4617308551835855, 0.5058867870637003, 0.47921505239503887, 0.47312038309166177, 0.4829766974428596, 0.49102414106488745, 0.48554221756568766, 0.4822970408709533, 0.5346819278393269, 0.5537581830831659, 0.4834609186066347, 0.5577122355193966, 0.5190298314015981, 0.5164238075681362, 0.5259676412761917, 0.5131634135725778, 0.49845121636493306, 0.5346518075166312, 0.5160249333362892, 0.5072838540835467, 0.5327400549763104, 0.4920384517427681, 0.48733134271145667, 0.4668317599147483, 0.49319897599282114, 0.5323679288110504, 0.5146926721424898, 0.5311112734809414, 0.5245424812135059, 0.5208438610583022, 0.49838075995666675, 0.5350569251623748, 0.46297526057118715, 0.5144853853728719, 0.5336798766879082, 0.5261942276528838, 0.5249532350525897, 0.4855669517507387, 0.49973922916094415, 0.507745756052509, 0.48899916623010603, 0.4885880594738805, 0.4893715098104286, 0.5161542445484008, 0.4703855751313299, 0.45332064503197234, 0.5048470756394027, 0.5006117610537956, 0.44741529441144207, 0.5039073325069987, 0.5313975438413017, 0.4541087203784608, 0.5127169335793166, 0.48309023651146354, 0.4412953220876107, 0.4876024971877466, 0.5015551745127861, 0.5005047468967848, 0.4832553943973032, 0.5310514157440195, 0.4972930983315128, 0.5110595719256718, 0.5145490140657932, 0.5120051361830311, 0.49125653335982267, 0.5273818820821891, 0.5123504048050384, 0.4876102741091668, 0.512216652952739, 0.45651637724103367, 0.5243695657193365, 0.4906892933383637, 0.4983734222232135, 0.5009247245731198, 0.5289152526158689, 0.4984141907700613, 0.4947720653607253, 0.512144931753577, 0.4901611774336727, 0.5205731319863274, 0.47336782232158564, 0.5078773038258033, 0.47148000050147365, 0.46978197294776175, 0.46759280325333535, 0.5355192779953918, 0.4758862255599608, 0.5014436944309429, 0.5130611700699105, 0.49842917693692834, 0.49279320419590616, 0.5058539460204727, 0.5046677937311977, 0.5006560580090635, 0.4910798803728382, 0.49038224244574796, 0.49636778596567926, 0.5117270780662051, 0.5174940499081434, 0.4907527390567141, 0.4888032329305455, 0.47224242230618174, 0.5058930888688743, 0.4855200270373444, 0.48366947500427393, 0.5118899968013739, 0.5046814549576087, 0.5582875418305978, 0.4524258054521665, 0.49833303752181274, 0.4857585473031911, 0.5280840506390243, 0.5027714940442053, 0.50830436976034, 0.5096203153657597, 0.515059830173113, 0.5339669672489488, 0.4606789467734571, 0.5173690008689269, 0.5522777019892601, 0.5039445374237176, 0.48803706146886583, 0.5097888328681212, 0.5046014084536011, 0.5042190172266482, 0.4837447425639466, 0.5097325468814106, 0.48962707410740997, 0.4965070283427616, 0.49381744505685493, 0.4787137423395537, 0.4685470171865581, 0.4616808477191847, 0.5048447798623452, 0.4750234064082079, 0.44996752292578107, 0.5339058551551559, 0.5590161696827759, 0.4953918824113658, 0.4875460388952158, 0.5387606762436135, 0.514269975758177, 0.4975483411693742, 0.5093488383524186, 0.5146224385387003, 0.5162369715517822, 0.4838698216865707, 0.5516868730777902, 0.49564672567729057, 0.50225475988759, 0.4439672582479398, 0.484964826994334, 0.4421176321328768, 0.5082346895312515, 0.464593689551501, 0.4990836063102222, 0.509400305205003, 0.4788293082475354, 0.4744366782775456, 0.4912280882501564, 0.5335666261883228, 0.46412773473070534, 0.5329494749993741, 0.5315309596856226, 0.49610649564312587, 0.5076449943057207, 0.4579731143021318, 0.5241251215178716, 0.5195334944845575, 0.5052072796310718, 0.447828631026714, 0.4706670973126617, 0.5015853861386778, 0.4814032469251712, 0.5133489368398233, 0.4711054898814366, 0.4775403959256101, 0.547368601975253, 0.5233239555839191, 0.47639815772946953, 0.43799535061822864, 0.507420427214355, 0.547922454979784, 0.5348880091319436, 0.5080218120358495, 0.5064149152279358, 0.49573615037432045, 0.5166766752961984, 0.4755167818646706, 0.5118311792628355, 0.5251666155625317, 0.5303804862211704, 0.514948495432942, 0.5272002447049748, 0.5330891250403909, 0.5281461121617975, 0.5246003398113508, 0.5012152593822709, 0.4830461368390326, 0.5019586807606344, 0.45959266948839694, 0.5134066303622186, 0.5119154075611687, 0.530301261220996, 0.5065357854906298, 0.46332060391236857, 0.46169877801483866, 0.474116502090428, 0.4690915645217834, 0.5420852603273061, 0.5182632663882816, 0.529078424792973, 0.4855593496185045, 0.49079208814737246, 0.4264996568309932, 0.5232791032916053, 0.48342710387315085, 0.4790292134965613, 0.46226632536169104, 0.48933402334879883, 0.48217861188315264, 0.5190538423858044, 0.5335977614956123, 0.5261351048349415, 0.5106066274697765, 0.5074344926627666, 0.5402301434538064, 0.4907335310157154, 0.5116073382642887, 0.45680134392357685, 0.46157689354435305, 0.4927383196002628, 0.48569815155344703, 0.5216814250893982, 0.5324974034673448, 0.47272825859965817, 0.5295317660545097, 0.49430070511881075, 0.4675870062540219, 0.5334556782832909, 0.5087682947471239, 0.4810879306565946, 0.48625114799957575, 0.5474174506543789, 0.49682129011062515, 0.49329068618522803, 0.48984294247391336, 0.4873136528628456, 0.49024333374473866, 0.5482508483295786, 0.5290972521789801, 0.4428497008410947, 0.4679239889823134, 0.5040511668418559, 0.48425084024375215, 0.5253729674923937, 0.518384644730573, 0.558555894155626, 0.4983582440978393, 0.4671309047890896, 0.4805983383853673, 0.47312060530231553, 0.5077994330734142, 0.5210367224535181, 0.5130190625122447, 0.5002611288444229, 0.4566080532052308, 0.4702704426466034, 0.5182563836370692, 0.48726107127508417, 0.48013939408879214, 0.4590751308837558, 0.4914324557405508, 0.5153810122692227, 0.48480585697345, 0.4610404051717915, 0.44471878835930684, 0.508646153619947, 0.4939281514309704, 0.466389499883153, 0.4623520023196942, 0.5071265715443558, 0.5074489387735729, 0.536207500695316, 0.5086804225088655, 0.5118865584240133, 0.5090097974384566, 0.4589140004943847, 0.5258542224675489, 0.487689382693667, 0.4977689619160304, 0.44289144752413234, 0.5098765164314761, 0.484608325604718, 0.4913153536531608, 0.4914244981406327, 0.5100032907849027, 0.4890099910601048, 0.483825619592436, 0.49441442396260055, 0.5077536030879805, 0.4648523173260809, 0.5215660682060231, 0.5199769116111743, 0.5393802835997333, 0.49317836575262297, 0.5035782701335256, 0.537686212314919, 0.5164262160841607, 0.49092289458986027, 0.47173853824418344, 0.4494108073733871, 0.4482269724168639, 0.45252023365695077, 0.5019315507249148, 0.5034493728787284, 0.48210698507140065, 0.4798710914678637, 0.5107534815032219, 0.4924967037383432, 0.465600372021844, 0.45522593002190237, 0.5027216109347147, 0.4833885258315854, 0.4809683027924109, 0.47120132029085515, 0.478572888821886, 0.4990998538949727, 0.562579896816829, 0.5188734936569432, 0.4875601105736894, 0.4482715074086314, 0.504795296683815, 0.5012859265526165, 0.49153066339263696, 0.46067783399669165, 0.48452147653742744, 0.5124553351024931, 0.4815856530924935, 0.48758754545689176, 0.47817690406868824, 0.4865386121512176, 0.4943687111279391, 0.5313507576827857, 0.46603692289300774, 0.4961824705647365, 0.43887739736792103, 0.49301332835587447, 0.4693733259610912, 0.5491298446426651, 0.5093795461877636, 0.5355286086978431, 0.5088445274588713, 0.4921684203303229, 0.4485555807643267, 0.46147134877260465, 0.503148585955365, 0.48866517931500814, 0.53636589119856, 0.4922893658742744, 0.5000528988406411, 0.48543217273986067, 0.49641387464127135, 0.4588954388100049, 0.4236614082871806, 0.5006823393624314, 0.5039624475585125, 0.5202805142390902, 0.47592744423268685, 0.4815590817526558, 0.4915834200662846, 0.5019076133619748, 0.5152770495422048, 0.5071634007801422, 0.48900254665656273, 0.5119852236987629, 0.500066734134859, 0.5001579436354712, 0.49103557381739754, 0.5359624325554087, 0.4813820381264379, 0.49270365415156836, 0.4813407391213931, 0.4751200356584538, 0.42603629724805, 0.5245450920380044, 0.5161130954734918, 0.45354729677396205, 0.49896755230892115, 0.4860556598530723, 0.46537485137563495, 0.4983510445178556, 0.5300719351227583, 0.5229379916620861, 0.5098298392838893, 0.5310440855229751, 0.5468337906554662, 0.49252489801367466, 0.4702600454359737, 0.4769925563568156, 0.5259852967177561, 0.4691843748148675, 0.5229307412812116, 0.4837425446036555, 0.5479153244099345, 0.4951760824431083, 0.4754893510657667, 0.49452133772369905, 0.529598370555175, 0.5376429571083339, 0.4857023855553837, 0.5139941843803644, 0.4932096981560612, 0.5224034558901222, 0.48676613126446355, 0.5018108888737065, 0.4872481306889434, 0.47078498871635505, 0.5491829360876156, 0.520022749449396, 0.4716968027051082, 0.44002660627555634, 0.4654445505501264, 0.4563937620437415, 0.5214067012591291, 0.48529397455225387, 0.48214852680582443, 0.548122702006123, 0.48089471931902183, 0.5069500154788409, 0.5084691338727508, 0.49069861144839566, 0.5582397874136624, 0.4578802293328293, 0.4594496331151448, 0.5191426414937029, 0.5194781034770449, 0.49609146362940876, 0.5139674980507994, 0.41278258607806273, 0.5271447323534828, 0.515547756078756, 0.4296265147186371, 0.47041618569006455, 0.513991058201597, 0.4642267983288631, 0.46376639413916015, 0.49004663170523394, 0.49906044747402417, 0.4672977029477724, 0.48352868378888597, 0.5214936812501795, 0.4857184532569127, 0.496269523320325, 0.4632204866497162, 0.5209955820475678, 0.5314464479501398, 0.5148675629784785, 0.5080531665376461, 0.5043926740282072, 0.4931795249720497, 0.4832798002814313, 0.4843685857100255, 0.4900701868800212, 0.4618991477216476, 0.5206317152951455, 0.47667026069058266, 0.45551867239724997, 0.4804951093526007, 0.4557140899687861, 0.47889827490496895, 0.4958051424486134, 0.5301840031352552, 0.4865680095218091, 0.49271338553945254, 0.497652123062305, 0.48782666292271604, 0.5015825272636358, 0.5022684477941477, 0.507651740288097, 0.5274334519970166, 0.5274537561938661, 0.48097499739092203, 0.4987053338664981, 0.5116130529367666, 0.5500360487630198, 0.5140733904934549, 0.4931232967716797, 0.43850072368527954, 0.5023985277841944, 0.5016779267716353, 0.48303156919768675, 0.5142104572539176, 0.4766090673344279, 0.49953338836532546, 0.478385417565812, 0.4816156809047399, 0.48650212282937527, 0.524196422583876, 0.4965007211675146, 0.48297331216926503, 0.5069921687165448, 0.4798447506433834, 0.5114441393735792, 0.4847275474780545, 0.47420210510125516, 0.49811360627045137, 0.4697057817390406, 0.4422494461355342, 0.4743957781647526, 0.4854794923553535, 0.456666951211075, 0.4947823144988648, 0.49306726285774, 0.5009526866696478, 0.5152987410318802, 0.5059343095775267, 0.4886187952009543, 0.46952870953949805, 0.5127728988823317, 0.5138463541630408, 0.4950294007701702, 0.5065028004264082, 0.4437093187589662, 0.5190797626459199, 0.5165845198501556, 0.49618065336235995, 0.5114298320493537, 0.5011771165738189, 0.5123468814630319, 0.5064431305126175, 0.5012502082702388, 0.5129234360420313, 0.5235134650892022, 0.5104549604162179, 0.49808194688776464, 0.49638013089952576, 0.4934907135725403, 0.479397965070567, 0.48855033378817747, 0.5086937872153778, 0.45679952682752384, 0.4868653530490651, 0.5222230761818264, 0.455551249111467, 0.45044100546652477, 0.5023493230534938, 0.47759192364381875, 0.538777148607126, 0.5081828906119402, 0.5230569686979977, 0.5029474506016677, 0.47730081042177636, 0.4655340069898477, 0.483705481916925, 0.5248882455122247, 0.46522141413479423, 0.48183713018464497, 0.47307044527215614, 0.5109783960785482, 0.523755538092969, 0.48851984463395753, 0.506627749349617, 0.48691125406891433, 0.5277762285223139, 0.48762694868658396, 0.46673403601538566, 0.5084709068503616, 0.4648216745537807, 0.47763090418412296, 0.5040345466326925, 0.49409220694732664, 0.4725282771086559, 0.4922357039771652, 0.496233002825503, 0.532946818141988, 0.532269671703111, 0.5078058173057823, 0.4814825992056746, 0.5499414420469035, 0.4905326995027683, 0.4652180716049855, 0.48283925992376736, 0.49952543636890456, 0.484190072111463, 0.5032842526859843, 0.5330571994968742, 0.5025980942435527, 0.5266878383039267, 0.48999775920644983, 0.4962867304065961, 0.521666329904866, 0.4854323397171877, 0.4725607750510853, 0.47865418443741375, 0.5185014752834378, 0.46582695001336577, 0.4769650546180644, 0.5427332054041789, 0.49839753381819607, 0.5106373578148661, 0.5130781164166197, 0.5084511157526658, 0.4754728573896074, 0.5158400365806104, 0.5414860547987348, 0.49580535063436093, 0.5159694695794766, 0.531832589079805, 0.5060975652452491, 0.501235344333296, 0.5350694822929416, 0.4502237687194501, 0.4715958340651868, 0.5338973760525155, 0.47173510376022476, 0.5226414686414236, 0.4835183385418906, 0.491302985639377, 0.5079154872829096, 0.5170217640467235, 0.5021531259099954, 0.43348757469022026, 0.5287908090143667, 0.5203494364353759, 0.532433448379754, 0.49727699234376027, 0.5288843412111632, 0.4890070170193402, 0.5231443354395362, 0.575247861592872, 0.525227528752531, 0.5406632733218196, 0.48044835173447176, 0.48757190539802187, 0.46829655172318124, 0.5273219293369564, 0.49175227128778687, 0.480009231206188, 0.5479648955222701, 0.4889925807973061, 0.5305634120221403, 0.48732589588355263, 0.4885395376048668, 0.5322302968077319, 0.4822839286337714, 0.457662288306342, 0.5323696327406202, 0.5173979070114887, 0.5095741174121906, 0.5058061042349667, 0.47387192281303264, 0.4819935937828536, 0.48599755005129297, 0.4911854869733093, 0.45609579279901, 0.49863772671551326, 0.5123622386647203, 0.49363887503370457, 0.5030748487095716, 0.4993194347838286, 0.4581400352801103, 0.4973720854803649, 0.48979516565733744, 0.5154049114886446, 0.516703939443695, 0.5003650812892627, 0.5081296079636327, 0.5189681298629752, 0.47670370491177233, 0.4861612739686854, 0.5128426686853355, 0.4847684270815567, 0.5005048876142868, 0.4788354706132705, 0.5135480092389896, 0.5273563358128042, 0.4741198289957705, 0.5004881905410673, 0.4831734928848743, 0.4694581602766107, 0.4932668035364607, 0.5096658523784016, 0.49566046468882097, 0.5292731890613259, 0.5318692014951658, 0.4768370090721683, 0.4847598834521001, 0.49246976219965877, 0.5015416076316773, 0.5139117618413775, 0.47400799300111646, 0.48244189408056654, 0.5064028956079063, 0.5059746780512905, 0.4882152879417285, 0.5242396159556761, 0.4637361920227755, 0.5009293980055275, 0.5314910118388775, 0.4927933609004073, 0.486947650686964, 0.5069527487004479, 0.49594956092395365, 0.5099234072634812, 0.5176205316128458, 0.4842001299912597, 0.49541544923803815, 0.4729674788723055, 0.5095131580701006, 0.49880188140116133, 0.4975606008199259, 0.5308424386036168, 0.4900182386814906, 0.49054079797769445, 0.4856765617872637, 0.4829693703037268, 0.4650427770015947, 0.4842440792195469, 0.4886690253396635, 0.5004039800514167, 0.521966197666418, 0.5027342189663953, 0.4864816335446684, 0.5322140471861224, 0.4733134688775399, 0.4791422285502176, 0.5182246866362172, 0.5067765391042992, 0.5133433113371277, 0.4793852446315386, 0.43473816034217055, 0.5181424961843846, 0.5037888978120375, 0.533032413288913, 0.48719504244234746, 0.5045628035349787, 0.5159721705399924, 0.49189383303927897, 0.5021515793750356, 0.5178692434541871, 0.5049592961639474, 0.4679272399213366, 0.5002082263993924, 0.5208831570831434, 0.4737953141097916, 0.4933200060947546, 0.5087163703091573, 0.5179174022911051, 0.50443929300949, 0.520564027128424, 0.45466344746325504, 0.46235604441581496, 0.48726540617111197, 0.4601464634595613, 0.526086137324427, 0.4953805088696391, 0.5175803621736514, 0.5010732635891632, 0.4840002831470118, 0.4498153044576061, 0.4930359511675562, 0.48129705873145606, 0.4997763752453419, 0.5117252252763562, 0.47733692483985557, 0.48209931916601456, 0.4947506531058442, 0.4997764346019143, 0.4902261822087176, 0.4846009219183983, 0.45813530455846707, 0.513218805875498, 0.5219838462343829, 0.5342537125205336, 0.5537699437399777, 0.47315824183888494, 0.48370145715856017, 0.44396006066868243, 0.49015949144757726, 0.4682329134623976, 0.5144031533254787, 0.5421106357605281, 0.4428496879485625, 0.5197027150022033, 0.4982747957195714, 0.4768521414220192, 0.5103608238849191, 0.5077767247412338, 0.4826008368267336, 0.45146369388847823, 0.46878565177222103, 0.4876057147958607, 0.5008113252473678, 0.510913819161803, 0.4465262581036942, 0.47468653300736846, 0.49613439612936866, 0.46969445462441445, 0.5101683178826347, 0.532800761706365, 0.5020310112669356, 0.49658643569447375, 0.5008364125736942, 0.5432418041406787, 0.4438480779837505, 0.48728545156376923, 0.49175243051563683, 0.5128400519594658, 0.4813010788915025, 0.5289176030891249, 0.45115253459510657, 0.5114385772281158, 0.49819252649296936, 0.4635592644090745, 0.49303845237051536, 0.4994306841521261, 0.508843664236804, 0.5109867574151535, 0.4827987643766154, 0.4948751817371316, 0.4798400211461095, 0.49237596152919777, 0.48337187369950857, 0.4969800234228947, 0.49478441041847643, 0.48993371321487184, 0.4926854838309122, 0.4881487860721081, 0.47233401748015985, 0.4702268311656383, 0.47308297224613044, 0.5278524392896627, 0.4308403262577809, 0.527506472225951, 0.49913797191615966, 0.46919598042567096, 0.4731024471878392, 0.5256995212333824, 0.49162039503037364, 0.5246593791140232, 0.4821261755860657, 0.5139060961970088, 0.47686383289921963, 0.46259909877824473, 0.47195251642310954, 0.5053328514954196, 0.5409831971774026, 0.5354674811083684, 0.5348889770898059, 0.5277604289689524, 0.5201230443964078, 0.4848713773441952, 0.48705703623229624, 0.4535625670488052, 0.544545004275866, 0.5163732118954046, 0.5088307734183736, 0.47734887085752065, 0.5151609625652299, 0.45835293098648044, 0.4955054815793938, 0.49324650877956017, 0.500611830629861, 0.5346321093633609, 0.5030310605385739, 0.4912013162135142, 0.5018799697535694, 0.4879272896470455, 0.45713583844981237, 0.5197458035659926, 0.49752535696138733, 0.49016114717843723, 0.4495658950904514, 0.4786807950627985, 0.4813581667595441, 0.4490307014730453, 0.4584434217046503, 0.44669861832563634, 0.4927184746940139, 0.5085712149948038, 0.5277541478944076, 0.5334991229083921, 0.5375114153588276, 0.5191973460811267, 0.4926591376478561, 0.5158285660652151, 0.4987970947368595, 0.4970379786914679, 0.5341817235122638, 0.4787591563238223, 0.47120982740019085, 0.4724139932043934, 0.5081360171480639, 0.4698462837932879, 0.5250460556290403, 0.4978656594258894, 0.5037832022772445, 0.5014456788136749, 0.5100115186764017, 0.4771784342734466, 0.5549806053461179, 0.47428072202225924, 0.5026220769116734, 0.46333052725698853, 0.5097642462126315, 0.5192914658587376, 0.5186063931669451, 0.4918788357664486, 0.5212250363424054, 0.5141095212659748, 0.5167752111981273, 0.49104241912961877, 0.4843592915578373, 0.5235101128539206, 0.49767731366499723, 0.45864910126140246, 0.5081564811032399, 0.47644336247817715, 0.4755250275364857, 0.4760477995916258, 0.4980155744845023, 0.48822963653787005, 0.5231340302983868, 0.5121770201655731, 0.5332123212072463, 0.47445352751859826, 0.48077998825218576, 0.4967228445464073, 0.5026075266284498, 0.4593832642506203, 0.48556895930021055, 0.552535068307433, 0.5232989837688985, 0.4852080842626405, 0.5319368160410304, 0.5031023780736653, 0.48801248608708286, 0.4885431553164445, 0.5256312640850053, 0.5052039196075666, 0.4764153387447271, 0.48897760179999805, 0.4863763614943428, 0.5208249121068793, 0.5205604592738013, 0.5057151416694995, 0.46912095853649816, 0.5163622224064662, 0.5369825538037435, 0.5017074857514304, 0.5260241389079594, 0.5167070067527885, 0.4623648957496349, 0.502372209766762, 0.47179835268297066, 0.5022405666655545, 0.500131698712679, 0.4963832498633423, 0.514121147843581, 0.4999854390749992, 0.5075024107641626, 0.49592475955542376, 0.48363074802545897, 0.48899507284091864, 0.5476769964798865, 0.49920909861430357, 0.5012863183672386, 0.49606361582928005, 0.46461697340536473, 0.5441894218042503, 0.5300199877903197, 0.48139497838017253, 0.5059400668273547, 0.5090228458311213, 0.5084357512603525, 0.48972955980703997, 0.48663495721460764, 0.49785797101980483, 0.5039590498405366, 0.5214800154937369, 0.5508812002305143, 0.5087405304863085, 0.46464748905183306, 0.5106983890989119, 0.4958540626639398, 0.4903630754457124, 0.507196750125786, 0.5084430331579903, 0.5183753997352659, 0.5049840169481478, 0.4937931153336804, 0.5341251300060775, 0.514509195629144, 0.5259413855203919, 0.5150020607232907, 0.5223132711420256, 0.506414871405969, 0.4998718421253085, 0.5102501746372802, 0.5346982367716558, 0.5244076531448199, 0.49306233479156086, 0.5302585848354792, 0.5201702182589772, 0.5307400415178233, 0.44832587112151656, 0.5061231599126973, 0.4905386869314819, 0.49160825661323465, 0.5022120742058832, 0.46964532128825404, 0.4965352995985032, 0.4878681108665462, 0.5121492628230202, 0.4930035375394703, 0.49833497468124865, 0.4748144985350005, 0.518927626633391, 0.5000100748121296, 0.5298286492597055, 0.49733792386932135, 0.48082481460317084, 0.547990521134013, 0.5021155092072775, 0.4890531386915356, 0.4632375330506625, 0.4678608937080221, 0.4747308315385668, 0.5056083483842575, 0.4804008059763838, 0.49586785188537036, 0.42105949986726166, 0.4971813462515727, 0.5036873338544141, 0.5128545082227278, 0.4849858587269783, 0.5095750605637522, 0.5109399073037614, 0.5404127336051875, 0.47437101498250933, 0.47447927599359163, 0.5185130841781387, 0.5245118527160559, 0.5045602674476621, 0.48298241352823257, 0.5173328845867367, 0.4834090427646166, 0.4548769811616175, 0.4950234400444921, 0.49932073344360045, 0.48015276993148714, 0.5166891356998216, 0.4814338231166254, 0.4884557912221178, 0.4826156509409809, 0.4493003985001822, 0.512926192157328, 0.47421559376202294, 0.4720026358849017, 0.5461734806641972, 0.49990892057457426, 0.48282485495179833, 0.4852023662769756, 0.48322667871849967, 0.4832770750253397, 0.48983419126802813, 0.4966603441228177, 0.49126796902165143, 0.4737416367297487, 0.48706952056381697, 0.49326612623157423, 0.4835194481300766, 0.4940390542166753, 0.4958849978992727, 0.5059801650155995, 0.4769814844113046, 0.5018087817323561, 0.5109082567372212, 0.5008817612614898, 0.5184524436525648, 0.4723978195036021, 0.5242007602369431, 0.5189298410757066, 0.526177148646829, 0.4400864736242048, 0.498727540787638, 0.4973872967614116, 0.49325951945317187, 0.4840282178872197, 0.5075648154730898, 0.480665727967968, 0.48754802532027475, 0.515020004147051, 0.5033524316879069, 0.4832043793225581, 0.5037036192483948, 0.49309928509450884, 0.5366016350351376, 0.4700105510329773, 0.5400711973429329, 0.4702339085620039, 0.4892012085586186, 0.4941814298756883, 0.4687147735626644, 0.47862791714367653, 0.49090563186437564, 0.47499862570261414, 0.49178551161943274, 0.4838981699830916, 0.5335995033854772, 0.5023287388938924, 0.5782437798896831, 0.47447797649930723, 0.47367024479408665, 0.4700965890918606, 0.5098349728028996, 0.486137849519856, 0.4997532615625693, 0.5346751804191161, 0.5295357355501874, 0.46073836560924875, 0.4647953842899933, 0.43314945940565136, 0.4622582582292062, 0.49435199885846154, 0.4913071646141896, 0.48911718151584804, 0.46051372466448826, 0.493373635984045, 0.5259727286141435, 0.4703019884950891, 0.4711497999691593, 0.5075079430089093, 0.5005766916851548, 0.5058570124913099, 0.5029078691658506, 0.486681010309557, 0.44272476230228874, 0.5083271488867646, 0.5279658035415126, 0.4839890795902107, 0.4915322828609012, 0.5015253795548125, 0.48784441780130505, 0.5167327614607732, 0.4706619343010241, 0.4773043710969805, 0.5113821351990102, 0.46409242846882187, 0.4922705919634599, 0.5062817681854024, 0.5184273253241918, 0.48101309834698514, 0.46624794468952113, 0.4767843717961559, 0.49896181890525826, 0.5212988768795872, 0.48215049818543104, 0.46810604631417574, 0.45773425932764367, 0.5114184202703501, 0.49679400487298664, 0.4574740166301158, 0.47873034528626873, 0.4818373128947078, 0.5197990541187912, 0.5182800718757457, 0.50081289726548, 0.42864968443381035, 0.5371295335937453, 0.5300412076506114, 0.5096213639018617, 0.4404699409374535, 0.48230052032596227, 0.49928395433227785, 0.4802273488763682, 0.511716533019799, 0.49096164659803926, 0.4803052722819334, 0.5418044339214791, 0.49361989265003786, 0.5618102841264511, 0.511354687787638, 0.4836385273096921, 0.459281552909859, 0.5392194691289048, 0.47846169845991543, 0.5194100661174985, 0.47616996553712265, 0.4565351749010518, 0.4604087772051133, 0.48971754666408174, 0.4978569764525778, 0.5216029218142129, 0.4813849501604623, 0.4875573238912833, 0.47086675191811633, 0.49267844559160534, 0.5061450706888793, 0.4618413464382942, 0.5190660667582813, 0.47671156091042294, 0.48125836546678896, 0.5010417842752933, 0.5317901281723312, 0.4859827490701406, 0.5045805893414729, 0.5054624107180938, 0.5276849494684357, 0.5081995527018017, 0.494304226683026, 0.5310927472057013, 0.5280825756862618, 0.5022105889318652, 0.4867282882955424, 0.4771857193461006, 0.5096752851362453, 0.5432732227459489, 0.4997700634481346, 0.4795111950416526, 0.5368632096979713, 0.5023452684093394, 0.48115848645113457, 0.4931035752046675, 0.5348001694558226, 0.4659818586787116, 0.5216017510220904, 0.5038020492268821, 0.47026109064113736, 0.5304164881915906, 0.4550942353161858, 0.5252398722792265, 0.4835144134372666, 0.4901098741795909, 0.43690828291105904, 0.5011883943770701, 0.49800043213296014, 0.5143062583278661, 0.5046229342809273, 0.5090658871426065, 0.48317045561784794, 0.4965128293595007, 0.49404438844346965, 0.4900072130312771, 0.44459772562950495, 0.532575221232357, 0.513716012752465, 0.49843681799366435, 0.5533802665024534, 0.5295225179854806, 0.5151693014960412, 0.48624477410206257, 0.5138946785947957, 0.503672514924035, 0.48796827632499723, 0.4514735829416682, 0.5310389983338678, 0.48780354254218766, 0.5407316741773902, 0.48685162301939316, 0.45850831722017976, 0.47786659694933264, 0.4804964648800226, 0.4472695183178793, 0.5003805327494191, 0.5116846505856335, 0.515317523795383, 0.5343056087535311, 0.5142781185534848, 0.41851670342132347, 0.5077421930318486, 0.5181713652726445, 0.49901762164032204, 0.5250381457457654, 0.47617000680441757, 0.49570032161573274, 0.49962409237142463, 0.46570538070165324, 0.4814115868275661, 0.4772802009147864, 0.49831955942430717, 0.5302682710058341, 0.4804907091069576, 0.46837575673335774, 0.5054823135071919, 0.5034598626618819, 0.4572320919176406, 0.5443961559774074, 0.5163090978197017, 0.4517140054609919, 0.504892176115249, 0.5259368646808358, 0.5165049570364154, 0.4919897510154964, 0.47299804829215153, 0.4861998796050381, 0.46065248769922007, 0.4807635429638999, 0.5154535758640253, 0.5093013781309, 0.5143319426923905, 0.5109855765361567, 0.534007488263002, 0.4848351314179088, 0.5390542470759176, 0.482718502138765, 0.4759172713009296, 0.5138316295232243, 0.44018256089151314, 0.49652648861475285, 0.46043396819212473, 0.49314666411526276, 0.502924642067842, 0.49899841656924265, 0.5076356343996304, 0.5218664768110027, 0.4793479137089911, 0.524168475895088, 0.45769365202751366, 0.5473270806548266, 0.49683830554094266, 0.5301825897241582, 0.4828318640283535, 0.4849163314703906, 0.5301545796970839, 0.5000858001045239, 0.43571960386191866, 0.49120952236622006, 0.48262047731061014, 0.5447395403852224, 0.46054908079313145, 0.4995314342250042, 0.4610625941713304, 0.502141604866343, 0.5080372175107126, 0.46642091568558586, 0.4991128883502097, 0.4993710002217746, 0.52763121376146, 0.4728997266799937, 0.5115928639509918, 0.5483143681892861, 0.5182919177927584, 0.5102465070215453, 0.5230025147370938, 0.4852311135975913, 0.489648657197991, 0.514685873405891, 0.5102715338915528, 0.48414471542577675, 0.5167412030008038, 0.516912742050573, 0.5131159818906219, 0.49369051583380025, 0.45953881424525483, 0.5030791264866168, 0.5052463779021539, 0.46754442518308603, 0.49308609143937554, 0.5244767935625937, 0.47970079739276283, 0.5157445385786625, 0.4677268362910161, 0.5103997383999824, 0.5139209507904868, 0.4645901735502059, 0.5143479499865377, 0.5040567693528791, 0.5153514341768811, 0.5340524198898733, 0.49727609853281657, 0.4810244289074078, 0.5212664138628559, 0.4948002287324838, 0.4708100216261331, 0.5023571109110658, 0.5272330760809653, 0.5254498614499052, 0.4792568566682625, 0.4890935589843366, 0.4888766732904841, 0.4621884329969925, 0.4832400928568706, 0.4987287910083675, 0.5465079561652351, 0.48383897395324094, 0.5218685052092306, 0.4888213460904394, 0.5031449872649769, 0.497697166104724, 0.5054403353335943, 0.5063207084772102, 0.5263651018229195, 0.5358551891246162, 0.4931969221587412, 0.48331985022632573, 0.4576567414513992, 0.4748081230640406, 0.4415255494945015, 0.5143449969029809, 0.5010047877116872, 0.5525281185334614, 0.48792618448019653, 0.47999226597222727, 0.5292174944705345, 0.5032315275509835, 0.5140741225868665, 0.453909511081358, 0.5053784085640415, 0.5348429033013371, 0.49444746774816933, 0.5052393601439388, 0.4906487582155336, 0.43437357664810994, 0.5333801321102811, 0.4898816998023334, 0.47601723048803135, 0.5079193880538004, 0.4702097412897478, 0.4850949954811054, 0.5014013275526522, 0.47396189458895716, 0.4757937325680878, 0.4812645863237536, 0.5128523776578848, 0.4687464739640904, 0.46172431116745444, 0.46693774662361, 0.48863944960574285, 0.4872489968896246, 0.4882756584015169, 0.5150587451188811, 0.5075416429719248, 0.4596733677485433, 0.5029688081285693, 0.5242800124117591, 0.4575778069946083, 0.5152381451605748, 0.5171312771482087, 0.4895036861459092, 0.5329409053937144, 0.4997794159905474, 0.5146976389335619, 0.48021663094539085, 0.4907807658321209, 0.5081879901194617, 0.4736928500990843, 0.46642318850265757, 0.5093431695371051, 0.49987260885073576, 0.504734046193457, 0.4846644663512053, 0.4956676161342473, 0.4874930012672848, 0.5600035009125602, 0.49527672345305196, 0.49468871008108173, 0.4886236236414298, 0.5388734523887178, 0.4849075609206792, 0.5045607079060961, 0.5016007441419181, 0.5170069573031729, 0.5427259617168784, 0.4911448792031579, 0.4614303057196171, 0.4839660536844924, 0.5183492309142614, 0.4907702886042737, 0.47586479929923176, 0.5091179696362067, 0.5013479994688086, 0.5209544773440503, 0.5284833386390668, 0.4569908662757759, 0.5112684935862709, 0.5084246559152702, 0.4642027637473393, 0.5082384964159657, 0.4944040508116545, 0.49293235024708454, 0.46519444170409247, 0.49092210067120506, 0.49248370486612647, 0.5078007195956912, 0.48771336995781667, 0.47270450666432395, 0.5001567480723332, 0.48429057211088017, 0.48445020092824276, 0.4745538115755187, 0.5219959582495203, 0.4828651791780767, 0.4641356600951442, 0.5073843501983604, 0.5156844749103642, 0.44531004171160776, 0.45365174824856735, 0.4620756632413866, 0.4645199827123605, 0.5347397719597573, 0.5016671767749044, 0.46199516460589235, 0.5040268752487644, 0.47155994963765097, 0.5066527675561454, 0.5294727493884466, 0.48230229998107216, 0.5061202466442473, 0.5148713494241804, 0.4578523914676839, 0.496909766898615, 0.5449883702545882, 0.45277700789459835, 0.49454165509948766, 0.517423299913426, 0.4978144525556612, 0.4881100615369999, 0.5164889637126077, 0.4611915379909016, 0.5409497904050491, 0.5280420275988971, 0.5221359718159352, 0.4786546398296167, 0.527658348393449, 0.4582489080052798, 0.48926725428778556, 0.5226335892580857, 0.4897248222474435, 0.4927831059942209, 0.45523717980609224, 0.49779362170559915, 0.4837092462952714, 0.5090215857390584, 0.4913325613906886, 0.5335424346779956, 0.49047294678450903, 0.5326293085329775, 0.5164346000840107, 0.5111797545919992, 0.5020053919321547, 0.5322051038164051, 0.5063152530966987, 0.47621797729263754, 0.48241221786899136, 0.5181582436653552, 0.49002581589078326, 0.49320993927477824, 0.5441212146485906, 0.4917906905989476, 0.5029365264770862, 0.46538881381121644, 0.5170726919414498, 0.4774772071357639, 0.451974376051963, 0.5214989296264742, 0.48143211866267294, 0.4967743671399294, 0.47891116352997243, 0.4851700738078594, 0.4801656641492799, 0.46482453717412936, 0.4665365281708506, 0.49520158748041604, 0.5398955514524922, 0.5027270018277449, 0.5561769719497714, 0.48002732366903333, 0.4987150735593023, 0.42151018372200455, 0.4879513935793549, 0.47945231994834003, 0.49935105865666163, 0.5271834015042518, 0.4788581617572175, 0.47666504355371464, 0.5310847289659872, 0.4933482167659349, 0.4896669599295989, 0.4743656633754251, 0.47490015398821656, 0.45544881087404554, 0.5083951221505252, 0.5160465245146392, 0.4733330837546559, 0.4607027930550945, 0.45516386742406995, 0.50958897045695, 0.5022980229780639, 0.4774910109237127, 0.5172229203831156, 0.5327547483891146, 0.5143317693955369, 0.45173238887433637, 0.507966651196546, 0.4996076110104353, 0.49038031974140256, 0.4639962392151538, 0.4460376213513126, 0.4649066444735484, 0.48556859260162516, 0.4809445983837801, 0.5176655515188524, 0.45196370238786043, 0.4683438492819194, 0.47578311364037784, 0.544686923492845, 0.48480050481239195, 0.49483367692482466, 0.4965388683426233, 0.49599786657733386, 0.5446913900600455, 0.5255650622143363, 0.4816589063858862, 0.4909376681994751, 0.4812410219155757, 0.4655976344609209, 0.5510292672795009, 0.544425445875578, 0.48269327153822517, 0.5046049038601573, 0.4479760968604636, 0.5106846352298413, 0.47860055077669655, 0.47231428238313494, 0.4893160587865278, 0.47737752081053086, 0.4915097422628224, 0.48912879360506495, 0.4884502235648859, 0.499767567827399, 0.5056651534949338, 0.48422957617480517, 0.43866408768293275, 0.56899025205292, 0.5205255120777132, 0.5281582955682115, 0.4939107362190418, 0.5128956400986183, 0.48027927014910193, 0.5088394035656277, 0.5080649333512329, 0.48208733197616305, 0.4894766469360707, 0.4944471552459552, 0.4707721499754036, 0.5007941494102864, 0.4826869763389731, 0.4964394531208532, 0.4954105320688764, 0.48529198980578986, 0.5388519007332969, 0.44197739008570397, 0.5064191467825104, 0.46928287369513405, 0.4972248286786345, 0.45621428419306476, 0.46760490318349635, 0.5172008698840992, 0.47350234841294336, 0.49592481401765415, 0.5119452003842575, 0.4768230437618176, 0.5055725319774232, 0.5381765175841461, 0.46891300814518094, 0.516431401931504, 0.47784274346992334, 0.482412795837018, 0.5023471568228738, 0.47917698238383727, 0.48617206872192276, 0.4931845064509617, 0.4906410976832813, 0.5036053106322002, 0.4869906538068041, 0.50565506034016, 0.5313697196045742, 0.5085813317346962, 0.5026028753165201, 0.4605554614908069, 0.4589944748497382, 0.5273327633318331, 0.48684923443231753, 0.492336141974149, 0.49438641206175016, 0.5144348114161902, 0.5089025179985247, 0.5197285839583019, 0.45747080446046234, 0.4399188533422482, 0.5037312291167317, 0.4953110392299213, 0.5146343743344984, 0.5040419863403812, 0.49264810750621457, 0.5092605030888875, 0.495022018248615, 0.516739292862505, 0.5143281388478997, 0.5101115397564299, 0.5000243103350043, 0.5316060297747719, 0.47743205898208674, 0.5154041946787528, 0.5077741788952805, 0.5331630057914089, 0.4878458210436747, 0.5141073523726802, 0.4519662005640443, 0.5201358854642494, 0.49082584185520667, 0.5308477296944809, 0.5046924921439827, 0.4993647866437679, 0.4892316993603919, 0.4773164460813841, 0.4976062547503603, 0.5153258642905023, 0.5005477275241231, 0.5165147940746493, 0.47250493919955, 0.496232068380363, 0.4763728383906843, 0.49675924508960706, 0.5273235238208351, 0.48581476326511097, 0.5115328697602411, 0.48770296025634124, 0.5116288981504342, 0.4743992488245412, 0.5012231487474795, 0.5087938056491184, 0.46166249610932314, 0.4818305096383536, 0.44674973024137205, 0.5278314720199787, 0.47886927181664807, 0.5207261406019933, 0.5024798580620636, 0.46661344646395636, 0.4609995093399852, 0.5127909624282955, 0.4501412086746361, 0.5287798757644915, 0.5358992821571879, 0.47794909373445105, 0.4935863690936944, 0.5060193392707479, 0.4904822095243461, 0.4884482384001891, 0.4707253525032788, 0.49221858274440444, 0.5259938551017206, 0.5037363963182396, 0.53178789188453, 0.5204315167467825, 0.5084075429442623, 0.5056678883903368, 0.5084396015424728, 0.48561948949408, 0.5245605716624538, 0.4279602534706261, 0.4806491379749693, 0.5379212587110356, 0.5489163376741905, 0.5074473973193255, 0.5036591140007424, 0.5167208261674554, 0.48346714951244496, 0.495976778506668, 0.4771927761836321, 0.48285084000369494, 0.5096505932521529, 0.46286649813532754, 0.4874694056477258, 0.5069723903344956, 0.5005854288380152, 0.4926832583070598, 0.5091853849836703, 0.48180004243554675, 0.46863180775191193, 0.540842728846017, 0.5071447533954372, 0.48160796848152154, 0.5093285025216615, 0.49818734650590313, 0.5066912861852614, 0.4957118808976059, 0.5011787434115379, 0.5127817001001252, 0.4587537854889352, 0.5159919531347872, 0.5016263264261716, 0.5187071724169133, 0.5145535144529269, 0.5053426758491707, 0.4287752819480115, 0.5217041421778952, 0.5139112009497045, 0.5145399458490216, 0.4645795953478943, 0.4624334132597202, 0.4806506636493135, 0.5166417377653914, 0.5263040891033661, 0.5018164233854646, 0.4848249737478251, 0.5063932334830362, 0.5036395730906162, 0.4809176092122108, 0.4674864335574164, 0.47487257000755695, 0.47582665195040885, 0.4997123463317642, 0.49913123904355267, 0.47999795413977253, 0.5367372969738463, 0.502177157396737, 0.5009830604550061, 0.49963485974474886, 0.4366507788827713, 0.4593251543263777, 0.4406912481956835, 0.48739636382216794, 0.5540168863118263, 0.5427404253510276, 0.5203854763830473, 0.47597893471888486, 0.49241016582256, 0.4864838092679251, 0.4898345133310153, 0.5262787595005529, 0.46873513013577256, 0.46817429509472175, 0.5201653056455086, 0.5012659006932881, 0.5132642103330556, 0.48195445593439074, 0.533470707175701, 0.532906236680876, 0.5328632275187906, 0.5110419122152018, 0.49127235814761416, 0.4980799596192564, 0.4670010743472177, 0.4808500965366778, 0.49846841397532204, 0.4844499961943466, 0.464127489316143, 0.4912634706020505, 0.5163901514890774, 0.5478878609258825, 0.4450908986006898, 0.4759078833835025, 0.5130864209964624, 0.5094855437384427, 0.502090558479847, 0.4862614882391575, 0.47338518066693963, 0.47073970098959506, 0.5389677418851015, 0.4985307240293796, 0.47555295693711813, 0.4676391436292479, 0.5469634143866424, 0.4794250287915296, 0.4935825437693695, 0.5186322314262085, 0.5053590096041837, 0.47908607612209425, 0.45877263540852525, 0.47049924229870943, 0.5004542791812826, 0.4988342760970035, 0.48416208630914753, 0.4967598038831291, 0.49002157317264566, 0.4870125748904496, 0.5488833408638886, 0.49134940216251266, 0.5415884992977459, 0.5278019912346136, 0.4925675955812722, 0.4782179734404385, 0.49159724802557037, 0.4796925419665372, 0.5211411824051783, 0.5139919754530238, 0.5031145917040057, 0.49907739841074783, 0.511596944349432, 0.4709255239213944, 0.5352836186340602, 0.5146921849549552, 0.473502778447225, 0.5248413535446336, 0.48223414612695, 0.4372405886171421, 0.47062367332114136, 0.5086308663542617, 0.5241185670039092, 0.5112646727360484, 0.47391482183545147, 0.5045344211667279, 0.5221713280338743, 0.4996364433824107, 0.5603495161053971, 0.5255720502756336, 0.4738425688902636, 0.4922904487005548, 0.4887385411017219, 0.5280032505644946, 0.5628829999268578, 0.4903472177438644, 0.5125939474368447, 0.4707201116238721, 0.4881449766086334, 0.48388373128391693, 0.528069017051243, 0.4693687810306897, 0.48726689136428736, 0.5127319452080967, 0.47495147724712083, 0.4840228479950695, 0.47029113856317956, 0.5055479496057804, 0.47974729106387226, 0.48440503193515255, 0.5168523337351001, 0.5363079546342729, 0.5099976277131455, 0.4977697743103301, 0.4721495783171093, 0.5293508191257108, 0.5274738681391509, 0.48942289070547706, 0.519843111770271, 0.49939102176022304, 0.4661665021039976, 0.5116220785783503, 0.4835796798780285, 0.4707358369888121, 0.4842100263187217, 0.48523302297873067, 0.46694211006457037, 0.46509443970636977, 0.48556941557697225, 0.47218603295254424, 0.48787559791429713, 0.47836422104471715, 0.5075728938243682, 0.4697873809848594, 0.5672294270878017, 0.4967049172358882, 0.47660500618675455, 0.49170793347171016, 0.5087370075231399, 0.5126471720091305, 0.5351947823288747, 0.5039758649453345, 0.4992313227631832, 0.5033817150412943, 0.4912365804310971, 0.48607728634417563, 0.5114352755703401, 0.5220420305818285, 0.48689343950452796, 0.524147096290734, 0.5092614551260776, 0.4993082983762783, 0.4796090486278654, 0.484581926951771, 0.5182451093338194, 0.5035318390408051, 0.49595349463108124, 0.5195344227500146, 0.4849820707432198, 0.5049138350845188, 0.5150866028089524, 0.5348101792439772, 0.4673792531389881, 0.5108403276126228, 0.5435645350602843, 0.49101638512131657, 0.5150199844312384, 0.49038651972454456, 0.46002108727023383, 0.5011682160756616, 0.527815840272482, 0.4953650354206348, 0.49611085586824, 0.5249012789706354, 0.518961004298052, 0.4961972900417519, 0.48493808980332576, 0.47612391578862295, 0.4938164967178273, 0.5571385581735904, 0.5113945979951301, 0.48718764766365186, 0.4781234446883183, 0.4705594742538574, 0.48557888029322904, 0.4924599437194846, 0.4870770841820652, 0.4962264583358218, 0.49546338302426013, 0.5071620325119424, 0.44649862939215035, 0.5178281856769675, 0.5194060888185723, 0.4953364120062053, 0.5043986270121799, 0.5172637456169354, 0.4754343160689865, 0.469290934319973, 0.45754996580523183, 0.46647378203838324, 0.5657553428490842, 0.5135124978255975, 0.5143601586418435, 0.46545376776885705, 0.49527609665309535, 0.4978732771129574, 0.5162333603763535, 0.558262201090936, 0.49848930587344487, 0.49694595214669507, 0.4772596014131654, 0.46203361761888717, 0.4858258123355793, 0.5541583403735268, 0.5344574139283602, 0.4906492208887179, 0.5199492415228107, 0.5417615736002708, 0.530357512536254, 0.4850794142267451, 0.5191724880413263, 0.4821841190703418, 0.46734112076909357, 0.5336247623741841, 0.4463951665292713, 0.4984359023609426, 0.5145951014738948, 0.5219890525230628, 0.47417173243841904, 0.5053841793775203, 0.5409789902154483, 0.4852078293571937, 0.5026542393799488, 0.5001666743328642, 0.5564345379054046, 0.5055754855000817, 0.4499943600162118, 0.4914673577286635, 0.4929027640990425, 0.4705000082129826, 0.466942201277295, 0.5040285313909201, 0.45855608278951154, 0.49386554782443487, 0.5116168326172756, 0.49901698204816103, 0.5233617985205962, 0.5195708182462225, 0.48266634931421654, 0.46552981278625033, 0.5616316022169126, 0.4918503704296798, 0.5260710386302748, 0.5117915975057061, 0.45902276812951626, 0.4852790620291422, 0.5168269564712786, 0.4734038874983142, 0.45168598139273564, 0.5290293936577676, 0.487131481319176, 0.47734847992084883, 0.47026888203241635, 0.44352093983651847, 0.47107526718822723, 0.5230148886982827, 0.5482591188263436, 0.4964697247247622, 0.47689872115115295, 0.45500242940063346, 0.5262619184537618, 0.4752465714889568, 0.5166648077726781, 0.4564991274411139, 0.49343574045629063, 0.5101307974194348, 0.5062853745187083, 0.47096557105850756, 0.5154899276905999, 0.4955681185334741, 0.5092078853399574, 0.4776473406592997, 0.5061351280821946, 0.5015541810349776, 0.4726830435428397, 0.4918049451000634, 0.5466098716135328, 0.5149367282732548, 0.5179124295219402, 0.5003770209709655, 0.46769127645963104, 0.46869861116585665, 0.481097314518357, 0.5042627684837632, 0.5132025221040103, 0.4974300291044673, 0.475100862048365, 0.48662008263005546, 0.5137852829900016, 0.49240083697917686, 0.4803024663012996, 0.46854255064003114, 0.5254770010849426, 0.4333382897387051, 0.4770209377304212, 0.49404003741476155, 0.4649933050040809, 0.4726068371717366, 0.4837248791833489, 0.5094216471461478, 0.5385215131947917, 0.5071090327748442, 0.5003968774516233, 0.5393875475570357, 0.4890413284591047, 0.5428225322756773, 0.5379708122756323, 0.4814667574951246, 0.5312230506877341, 0.4947341503119354, 0.465821710769912, 0.4845160215086599, 0.5229259650993112, 0.4891797613842097, 0.5126845510977843, 0.4779261045640084, 0.4998668786234563, 0.5079357645060036, 0.4518997988230008, 0.5186501036110143, 0.5158709000340252, 0.5108454996169793, 0.4769515949111392, 0.5186461853682793, 0.538859470404822, 0.5265580330040821, 0.48993641410072963, 0.5469636757371458, 0.547926163447739, 0.5221109743668687, 0.49119349322880507, 0.46536170431687623, 0.5011052742879357, 0.5315761997364595, 0.47888466424478654, 0.4712921230634191, 0.4696462361591908, 0.4674886985190524, 0.46646090423762004, 0.5115264652622241, 0.5100369571288035, 0.5043318640245229, 0.536228877678165, 0.5112367354976193, 0.5496690308558094, 0.488137289735731, 0.5070819928029136, 0.48181550303956633, 0.5102278180018534, 0.5077140525605399, 0.5108610766635427, 0.4723037201510483, 0.5312022692902387, 0.483396340530042, 0.47349915496658046, 0.5003492520121247, 0.5006129602312335, 0.5039242210851503, 0.5069509385722575, 0.4832327736248932, 0.527706739292082, 0.4635371995416035, 0.5055739434680843, 0.5171972599610138, 0.46057692355438073, 0.491277130199191, 0.4665669395708636, 0.4533859133117834, 0.5153860489308313, 0.45102002923316487, 0.48873736286294994, 0.48929282236321076, 0.5660954002243036, 0.5079685008169965, 0.46984069418681756, 0.4806237430404752, 0.5455446162481504, 0.48730308578352316, 0.5205580893749255, 0.4771361585961364, 0.49868797381695645, 0.4767141705966858, 0.5047864086677633, 0.4542390097381682, 0.5270106581786944, 0.5017382625940575, 0.49777606538505714, 0.4554963878766021, 0.5172244484550238, 0.5048957902458294, 0.5377334916245964, 0.5251306755295881, 0.5208902102918074, 0.48741137066619766, 0.5022864951693943, 0.48710592230587313, 0.5120959437270385, 0.47702253735136935, 0.491531790118657, 0.5033469380799467, 0.47008830131200596, 0.47781383793143856, 0.4872652727287348, 0.5139751102828392, 0.5056520844791511, 0.5191059594613615, 0.5549974577191964, 0.4655948339931038, 0.5105237189436582, 0.5576701862154017, 0.5195288610123605, 0.49977663997058763, 0.5237560498202848, 0.5382688735016654, 0.530166608355045, 0.5195826845253261, 0.5472761323952924, 0.5170997404733706, 0.4745103760110023, 0.5186910641624151, 0.4983355955381915, 0.45116451033943106, 0.5094726699410842, 0.4881777629163013, 0.48596095810260975, 0.483960881754519, 0.521363993132725, 0.4975650951694833, 0.48186777007508674, 0.5179869915204791, 0.5137890965242199, 0.4906554627370069, 0.5085733046552366, 0.47892984100422625, 0.5220282048035575, 0.5024271452215752, 0.4559633744561801, 0.4987055511852238, 0.511220440461228, 0.5046744342311821, 0.49291399412473175, 0.5148236834624089, 0.4821576590778202, 0.5102274118859113, 0.5167663055866566, 0.47973975217793785, 0.5143552606040374, 0.5411467115315584, 0.5090376987822532, 0.5031470281015098, 0.5271493749479609, 0.4906679218762633, 0.5022650529908463, 0.5338157551268049, 0.5008161001209147, 0.4908405976218637, 0.4701083762843309, 0.48983534673269147, 0.4873014808700605, 0.4897342264146629, 0.518836160058685, 0.4587389469016829, 0.4944086170224272, 0.46716624198838574, 0.4955462600652691, 0.4379317733262506, 0.49224895976072547, 0.4981438123638118, 0.5013437840878852, 0.5275926940996342, 0.5111785464135882, 0.47504753937980265, 0.5038246631872288, 0.5276504741551207, 0.5152224370595722, 0.5178849326547654, 0.4997737447008789, 0.5277153041681072, 0.4543106785266587, 0.4454670387531456, 0.48675690356068463, 0.49398086327948926, 0.47688960549474047, 0.5148526146933861, 0.4802521141559883, 0.44990910761303904, 0.5155206417582688, 0.5177133402406101, 0.48290399271964934, 0.4893642382084228, 0.5112801765535976, 0.5415645205287698, 0.5323710964679206, 0.4798197790823309, 0.5092423667507845, 0.48963063236576515, 0.5457348079402046, 0.5027813907308206, 0.5221972834676966, 0.490932051508007, 0.49249617486387115, 0.45239292388276175, 0.5500295363066618, 0.46676338498824266, 0.47968140614195376, 0.49945184351566063, 0.5077972068069863, 0.5161391581046028, 0.5387921498522876, 0.5512374381226237, 0.5072565233639191, 0.48890721837234513, 0.5116505024852961, 0.5108491610443364, 0.4871384170397527, 0.48152234765454405, 0.49729548249798444, 0.5143436537713095, 0.4984974512883608, 0.460093510496734, 0.48605126377701025, 0.4853977601220321, 0.501336783334481, 0.4946088482423591, 0.5197422171063366, 0.5247949466593905, 0.5103288888796609, 0.5103559325299528, 0.47702987969469834, 0.5149693193847257, 0.4422778830984547, 0.5032363918545931, 0.5357687601127987, 0.5013687082768371, 0.5125424051904056, 0.49689963195968934, 0.4884847807059472, 0.5443820121776418, 0.5072008318907301, 0.4936839148227991, 0.4754381302315416, 0.538823246390083, 0.49580817371945696, 0.472597227934044, 0.4549493729062117, 0.5220583325861226, 0.49614186323711407, 0.49534597687284604, 0.4830392646932712, 0.49092491057821713, 0.5191541132348221, 0.5223870692301552, 0.47426772625261837, 0.4954505619045388, 0.49434667455848436, 0.47655412890314736, 0.46681298081139516, 0.5002816757600803, 0.4982758369862087, 0.480444765626855, 0.5088143314992356, 0.545533232599952, 0.5239381335143082, 0.45651656110419886, 0.5076108898816477, 0.48879364409599074, 0.4717880124411156, 0.49922648719172374, 0.5267092870188989, 0.4794780757503867, 0.5551083770597571, 0.45183707461708267, 0.5058422806867021, 0.49376575569777753, 0.5161306121431152, 0.5099522859594481, 0.43516620903894127, 0.47384670307410615, 0.49392548325500235, 0.4945695966503354, 0.5464439121265167, 0.5161948868749343, 0.5058041999413895, 0.517145006310361, 0.4574133659175733, 0.490278981068431, 0.4992947031055913, 0.5259568394656718, 0.4987731588732753, 0.5119158844276299, 0.49626979855699993, 0.4924029599804836, 0.5222124186450627, 0.5490610793924718, 0.4839800984950541, 0.49959170944507525, 0.4860083361115852, 0.4860052193699087, 0.506989679406328, 0.4806828150285796, 0.4751689566517634, 0.508215337950232, 0.5024351056322234, 0.48937279169041614, 0.45919195912931543, 0.527124260689783, 0.48738707675944, 0.48391453020067987, 0.4555314034802922, 0.4929061032780712, 0.5136167041945686, 0.5094448203118978, 0.4911096973725806, 0.4962388901083699, 0.4921084122246516, 0.5334121394780995, 0.5124001935999546, 0.4852981046092862, 0.504378971675785, 0.4790368212140435, 0.4642558946665453, 0.5221056028622766, 0.4784069468808555, 0.5240685459871758, 0.5189089032490473, 0.47850824407549253, 0.5409792420378009, 0.5083550999630668, 0.49026435616392466, 0.4810195772589416, 0.4867567258264948, 0.5347145888367552, 0.45639902526409987, 0.4946313809589639, 0.5084406728240082, 0.4650598948161505, 0.4686602474532275, 0.5012010714608247, 0.4930866179252509, 0.5220737375446645, 0.49767237797205477, 0.4849622822797651, 0.4819022607721021, 0.4903793320583207, 0.5064556863961249, 0.4835857634673525, 0.4795183842033381, 0.5256339567547414, 0.45280113223869045, 0.5347612803454839, 0.4519751760946437, 0.4976012210877849]
mean_norm_MLS = 0.48737127212744585 # bias_value
sd_norms_MLS = np.std(norms_MLS)
print(sd_norms_MLS)

In [ ]:
z_statistic_MLS = mean_norm_MLS / (sd_norms_MLS / m)
print(z_statistic_MLS)

In [ ]:
z_statistic_MLS_432_600 = (0.48681829963000667  - 0.48687663918971297) / (z_statistic_MLS / m)
print(z_statistic_MLS_432_600)
z_statistic_MLS_600_864 = ( 0.48687663918971297 - 0.48737127212744585) / (z_statistic_MLS / m)
print(z_statistic_MLS_600_864)
z_statistic_MLS_864_20000 = (0.48737127212744585 - 0.4872932742209671) / (z_statistic_MLS / m)
print(z_statistic_MLS_864_20000)
z_statistic_MLS_20000_10000 = (0.4872932742209671 - 0.4870540489001939) / (z_statistic_MLS / m)
print(z_statistic_MLS_20000_10000)

In [ ]:
# Z score for T=10,000 as test
norms_big_MLS = [0.48440836416646715, 0.48976791139859516, 0.4794696215845206, 0.48886195288721823, 0.48406317824464706, 0.5010942503063571, 0.4838427993708687, 0.47971863623311056, 0.49032795104319193, 0.4921251154000876, 0.48068105576217107, 0.49664978961461037, 0.4895014975534855, 0.49371217078721125, 0.4818116744795364, 0.4956891009009238, 0.48022415858313305, 0.493212548744332, 0.4920728157023529, 0.4943284858099721, 0.482834228534669, 0.4794774134661014, 0.49740365147339033, 0.4899045574920175, 0.48528088143976966, 0.48519107609015755, 0.4966830004471478, 0.48927577759528346, 0.49743635848752543, 0.4793713276150524, 0.4891975471369331, 0.5035414123019297, 0.48294896586467606, 0.4864430486952114, 0.4959096304736039, 0.4950207687351211, 0.4828802354739467, 0.48781361555193703, 0.49149684018426804, 0.502059526744894, 0.4847129992614664, 0.49052160972456377, 0.5004336610200812, 0.4789297138973772, 0.46836789741902074, 0.4852526037343613, 0.4760509938609195, 0.493904834490387, 0.48773501166982164, 0.49664217566837365, 0.4949079697199989, 0.4961756465811977, 0.4872958979172184, 0.47610615137479945, 0.4695545541091493, 0.47924563412382054, 0.48898267140028884, 0.48323660745478886, 0.4878272758275732, 0.49089894041432663, 0.4860998841133845, 0.48625856187897404, 0.4824565530664861, 0.4922107336711906, 0.484104787396245, 0.47934876425476897, 0.485469296251988, 0.49527261181470295, 0.4838727225099371, 0.5102403441231967, 0.4790522142100298, 0.48833800523723503, 0.4950402725209444, 0.48873229058169143, 0.4974225522727945, 0.4877404694905434, 0.4904674728407499, 0.4801897907371672, 0.4776274085872601, 0.5018127873543023, 0.4964025527166254, 0.469288747277471, 0.48936044981677074, 0.4816137999833679, 0.474986940319788, 0.48563347673999274, 0.48460859281470714, 0.48216659384944893, 0.49104104749909333, 0.49016896558338535, 0.48932273952513883, 0.4882467480634029, 0.48592626065485933, 0.47572369958280214, 0.4874429733999787, 0.486720689189388, 0.49421624761150296, 0.4903094656749481, 0.4937488528443852, 0.4883820176860783, 0.48598670797176197, 0.49131835822662556, 0.4876921053699385, 0.4796014454127188, 0.48289924734264533, 0.4805364393265003, 0.4870113354937324, 0.4953461277111525, 0.4854742832578218, 0.4706527392876375, 0.4954505485532594, 0.49325179820170295, 0.47386735241245137, 0.4870256614788977, 0.485297509830178, 0.485105411547305, 0.4891959232236182, 0.48944668588407586, 0.49627402392120595, 0.4878208062424798, 0.5033914583481562, 0.4860489327256442, 0.4912055978001397, 0.48889861374860566, 0.5084523270256274, 0.4884987791275674, 0.4863554471840459, 0.47922791860727815, 0.484912682059833, 0.48905695401904087, 0.48069975923163566, 0.49906645667742877, 0.4884031595044415, 0.4849009699435882, 0.49012353691356314, 0.4826431868985426, 0.481000000327547, 0.4867202061501049, 0.5009361959258058, 0.4984368081020917, 0.47635310333299846, 0.48903768967971156, 0.503564957583573, 0.49514639676614425, 0.4897113469875414, 0.4765843775356917, 0.4889851667995072, 0.47963238248235335, 0.48658735693253363, 0.49088265860293245, 0.4812695313842583, 0.4988011594145969, 0.49897902774976527, 0.48707410292828773, 0.48657628383080614, 0.4873859385604567, 0.4835201879335823, 0.4947064839519118, 0.49097957672339443, 0.4971162366447176, 0.49783722815048437, 0.49144953997653007, 0.4956100934300971, 0.47752912809589687, 0.48061016275875307, 0.4882758570749506, 0.492228916569824, 0.4829756127240608, 0.48171876562481525, 0.4758930379087141, 0.4895002930486873, 0.48738881638630627, 0.49379177291507004, 0.4886323087166426, 0.4984376224738648, 0.49512643617235674, 0.4969392133938698, 0.4948487474892915, 0.48942610598793224, 0.49483926180829535, 0.48802407190989916, 0.47903499195317817, 0.4965973060171748, 0.4865375057054736, 0.49219477499208475, 0.48577167256928383, 0.49136737753143817, 0.4838760424378501, 0.49770666508864253, 0.477431558780599, 0.48967075016246475, 0.5009642107487542, 0.4833229152468313, 0.48440630966198334, 0.49242012778521066, 0.48955543389881256, 0.4904025951236817, 0.4753085921986951, 0.4961752712092202, 0.4785576905839755, 0.4774811130882475, 0.4919996928063113, 0.4897600157725843, 0.483615505069212, 0.4864172069654272, 0.48937499078578967, 0.4816745041649618, 0.4967357795883902, 0.4875413716945254, 0.48720079844732134, 0.4848677812395545, 0.48892225439269243, 0.4911496795399911, 0.4895816079605526, 0.49619013578541876, 0.4788295678905306, 0.48765064245491, 0.4903441025916599, 0.4875097245298419, 0.49126233600648517, 0.4813180063701576, 0.5001251457353663, 0.4788651706928568, 0.4965303298620825, 0.4864962897396738, 0.48046201672004507, 0.4945764189578721, 0.47909570734435847, 0.4849639894792706, 0.49105438090833553, 0.487338749710233, 0.48700847087874904, 0.4895624823101769, 0.48588379996270886, 0.492316820406633, 0.4893394521222065, 0.4897433519583743, 0.4850381041862751, 0.49487288942211133, 0.49925151485297575, 0.4955195228444366, 0.4780300670663297, 0.4871515774605968, 0.4816563676723798, 0.4817180907454223, 0.49093140263669516, 0.4935960289369508, 0.49272599483857704, 0.4945523260726433, 0.47944623525177743, 0.4834853148678158, 0.48680069666124165, 0.4733136351341729, 0.49732436605824226, 0.4861825157749968, 0.4860139318393392, 0.4890096632795359, 0.48020037442397584, 0.4921079587102299, 0.47518207753073194, 0.48070065831753755, 0.4920440016519951, 0.46304114118119744, 0.49053451362857, 0.49174996855705055, 0.4856314904207599, 0.4790073589516435, 0.4853822902190466, 0.49035054109749576, 0.48625332138709243, 0.49435956362589406, 0.49212721859929026, 0.4815605788205455, 0.4833176654565607, 0.49404148189830593, 0.4763231311239547, 0.48693383693178577, 0.4858499907778676, 0.4859565831707168, 0.4872050226820467, 0.4829571701696644, 0.4900787341237295, 0.4858027339165931, 0.491279564041101, 0.48527269651071214, 0.4872075324581558, 0.4781774759924525, 0.50236733633653, 0.4944052931342916, 0.47774820794727024, 0.49423936461673895, 0.48874677589060644, 0.47522770427261346, 0.47511241449122377, 0.4743404309402786, 0.4810219610791819, 0.49061953044369677, 0.48295509781524615, 0.49715371977352946, 0.4899580582523539, 0.48613613412813883, 0.4922960018274323, 0.4847482644780763, 0.49122169215056377, 0.4860874360260255, 0.48539928684762934, 0.48548623743993863, 0.4878874022682509, 0.4889276927232541, 0.48644988585852045, 0.48338186324121135, 0.48901612073992523, 0.48944992739878845, 0.47488809727113807, 0.4970354570569755, 0.4944437038337605, 0.47752488688145306, 0.4879720525420735, 0.4927592913268969, 0.4794800940485829, 0.48616553194337575, 0.4942334241598564, 0.48531598652375263, 0.4934317393780402, 0.47591931210039035, 0.48994750326816067, 0.4941657271389047, 0.4977484910137808, 0.48472990241737896, 0.4925303236428202, 0.49239354172321675, 0.4855482118938228, 0.49367179448669196, 0.4943005885991502, 0.4941803319296407, 0.4930334415960413, 0.502103501427356, 0.4929951761453671, 0.48281250939787645, 0.4862617472098284, 0.4791374798801572, 0.4886053934191927, 0.4867431261229604, 0.4948481597400096, 0.4860857386227004, 0.4867955560544829, 0.49048560234415384, 0.4933164973304175, 0.4796878680908987, 0.49047809135755815, 0.4770312446403904, 0.4882066362284629, 0.489688499307053, 0.4921016276095662, 0.4910759655371755, 0.4902702272845154, 0.4725217384999982, 0.48523515352119145, 0.49470040975947666, 0.48532293832194384, 0.5021222189361473, 0.4903613337419859, 0.475165919491299, 0.4856706135908633, 0.4823293433309054, 0.4805868224821817, 0.47943063303547534, 0.4859748969123152, 0.4782294636054632, 0.4943646588487099, 0.4729936714809462, 0.49041601578602156, 0.4803519892772223, 0.4807147749347375, 0.49588349124331366, 0.4877094299102728, 0.4920814437374802, 0.4834240782722793, 0.48778422350057843, 0.49144102285621527, 0.4924811588483717, 0.4775426080580182, 0.48102308752573925, 0.4943061511847154, 0.4891059104006561, 0.4756601359887309, 0.48644209101261676, 0.49028942840381023, 0.48137263572298505, 0.47446238560406023, 0.4737336038217071, 0.4931850529754382, 0.4824404001180601, 0.48273172341884535, 0.4937217044941247, 0.4876998548151341, 0.49826978396683935, 0.4848400757621782, 0.4854032460231429, 0.4845783648050914, 0.47775062084647973, 0.48104851556628997, 0.4864244061283735, 0.5057495191918809, 0.4923354007150582, 0.4851255836463025, 0.47718550448191793, 0.4842801562179027, 0.48813352239459934, 0.48608146593704366, 0.48593090296577224, 0.48187709170175674, 0.48969094419885933, 0.4888159691366792, 0.49286221470268277, 0.48484813112880815, 0.502246758275207, 0.4875037346983671, 0.478603863947666, 0.49811235300775814, 0.46078012852400807, 0.49118210035973753, 0.4846990724437738, 0.48202851221001664, 0.49595191768970365, 0.492355002366894, 0.48367632429279406, 0.48864334216329214, 0.4815915801785774, 0.478655636696483, 0.48677467864117185, 0.47744544867950833, 0.48582556632347534, 0.481058867479005, 0.47376104148141135, 0.4856129629443366, 0.49366713104940074, 0.4928914137940408, 0.5022693969327482, 0.47545105714968344, 0.4971525674989119, 0.49679985981944297, 0.5015773206799167, 0.48999446668427604, 0.4924164396675885, 0.47892885967126025, 0.48976072277651017, 0.48488439476427214, 0.487951762117845, 0.4793037224002761, 0.48663794219045253, 0.4891663802158354, 0.4860288346852976, 0.49982231655040626, 0.48820975837857256, 0.49113797251628144, 0.4828432748750361, 0.4881413373060778, 0.49520412854620366, 0.4945527717518973, 0.49103345481797417, 0.4849007169838965, 0.48979416491131444, 0.4989785602403992, 0.48496740360840557, 0.4870735676850899, 0.4919144134153017, 0.47554760948960895, 0.48131396304852214, 0.48524814665958, 0.4883169344650068, 0.4843523684715574, 0.493860113904897, 0.4747825044098849, 0.5010716312661491, 0.47983361573210986, 0.4939273315602271, 0.4864738337402473, 0.4804973638699324, 0.4885834676725731, 0.4780208810892028, 0.4703846102041583, 0.48725883940421766, 0.48824645243370685, 0.48757902496451533, 0.49402142208423966, 0.4895941777000866, 0.49218391091656555, 0.4838011607379211, 0.4828501822410423, 0.49426587878241657, 0.49054195821380664, 0.4754475048496088, 0.4963945374017895, 0.4853667083710786, 0.4836920810236327, 0.4944058424518053, 0.48363546307037053, 0.48340919368203933, 0.4885113592090293, 0.4853105454524058, 0.4917258362944234, 0.4907215433019059, 0.49245601516806203, 0.49157804423062984, 0.49080977451253766, 0.49328574480816306, 0.48223381053049025, 0.5043866078010326, 0.4922942628766093, 0.48503532846307146, 0.4955885148671191, 0.4884281049195122, 0.4778617290827285, 0.4809588461443857, 0.4934936416098733, 0.49185398869612257, 0.4885097068124062, 0.48870544272406335, 0.4876595613565719, 0.48763822473611473, 0.47447789093481746, 0.4951142196195047, 0.48684053996932314, 0.494351940427201, 0.48613468575827196, 0.4850053449823845, 0.48280262333562124, 0.4862055679168259, 0.5055262909343573, 0.4852774495084004, 0.48528842931447413, 0.4803941634655715, 0.4921088375549118, 0.48292092551854476, 0.48323370039608, 0.49228211094164703, 0.485355187618083, 0.4870189967975907, 0.4906011048816616, 0.48304194041510873, 0.4809204350559473, 0.48606801227388924, 0.4842742574281723, 0.4813453835861691, 0.4832354860022999, 0.48887728100912964, 0.48114621775971844, 0.49864324758265804, 0.49130613709291354, 0.48757954015345883, 0.4868738318898903, 0.48858982076444657, 0.4852372500185188, 0.4915724792847998, 0.49195965301320893, 0.4698999354910195, 0.4862182312216943, 0.48524601595234623, 0.4856560173621918, 0.4940907244977763, 0.49091831456083934, 0.4834422419206516, 0.4858043695301136, 0.4773440290315817, 0.49004177759559686, 0.48578559512554903, 0.48932448538046497, 0.488266017387966, 0.47783737609509236, 0.48675047174859504, 0.492054368192752, 0.47800131977940824, 0.48873577037199833, 0.4909951047626288, 0.4779569933620295, 0.4746487897498164, 0.4913415062025225, 0.49293943063258494, 0.4968148706753863, 0.4874363911016624, 0.4801609303473741, 0.49577842400759137, 0.493111956966513, 0.4879371017963062, 0.47885496427919527, 0.4821426223755395, 0.48044493455351817, 0.49006047626669746, 0.4960089664961734, 0.4911464972983975, 0.4927904190909951, 0.47670846616312745, 0.4947699854325417, 0.48537340952749203, 0.4894815910582385, 0.48565658963371, 0.4906057601540433, 0.49191228613419685, 0.4740862069808813, 0.4909902811802031, 0.48034577543704077, 0.4808258961135682, 0.48283788064283195, 0.4846875845478885, 0.48387710366791536, 0.4808264096305738, 0.49798391072768927, 0.49213039142547776, 0.4767602138892565, 0.47935215013311644, 0.4869220086391234, 0.486669935641818, 0.4991816984869198, 0.48530912372141605, 0.4882901174117657, 0.4847111598368887, 0.4879807218951755, 0.48363480794301644, 0.4910312593783804, 0.4937402408903751, 0.48967342193812113, 0.491127065187947, 0.49094189496316876, 0.49141622048640027, 0.5031547329494991, 0.484334470248612, 0.47142507913711723, 0.4790203654909663, 0.4925831669041983, 0.4914395758482141, 0.4813738009677723, 0.48764658554204243, 0.47996757507367116, 0.48747714000384584, 0.485985175295653, 0.4696291729495753, 0.48032131336536205, 0.4899166480782769, 0.4916303743434976, 0.49014144950497335, 0.4855477472088462, 0.494941121240167, 0.47713337992867355, 0.4860481517925638, 0.49501163424905337, 0.4884964552763351, 0.49525589759590055, 0.48834865300126123, 0.4923029833009664, 0.4926899554075272, 0.4819864220287334, 0.4934550465679256, 0.5046549185612422, 0.4999546071715721, 0.4867585881470568, 0.49812429847357237, 0.48891041825979564, 0.4820154688867963, 0.5054863899718188, 0.4854505495660057, 0.4903366542676089, 0.49852648844681213, 0.496294196900583, 0.48189843489518075, 0.48700340723350394, 0.49122146168780645, 0.48583069268052953, 0.48980735664271924, 0.48615520596348993, 0.4920268075088629, 0.48847631500574146, 0.5013271891337985, 0.4831588463797501, 0.48933745564502495, 0.4892027592851872, 0.4829324767484458, 0.4859808615916656, 0.4772459062339787, 0.48382983952888065, 0.4991362023060395, 0.4909672974021233, 0.48229720823296157, 0.48676153735946676, 0.4930566137909033, 0.48628387982991067, 0.4914937218486286, 0.48085522385553253, 0.48499770670356745, 0.47812860742678537, 0.49071822061761255, 0.48577065097289684, 0.4942228923629514, 0.48679539486516826, 0.4891367777791827, 0.4892860855064517, 0.4841947848123319, 0.4953053536004204, 0.4935386731931267, 0.4968754185095721, 0.4855134442935556, 0.49209735911103786, 0.4930495125782793, 0.4837893545674955, 0.48045506852364345, 0.48082762586221156, 0.48753278496622454, 0.4835687112646784, 0.4997275497524721, 0.48143341634255044, 0.4956774423304486, 0.4872078622370728, 0.486032101660784, 0.4963431854466288, 0.48325990366058513, 0.48731913067183796, 0.4994249971002256, 0.5018456621334654, 0.4828090471298454, 0.48576868262170775, 0.483817827121025, 0.4916553555258547, 0.49264075662062173, 0.47861058540582085, 0.4758400853321042, 0.48293265489252923, 0.48604519673075575, 0.4841977861706195, 0.4866963716605708, 0.4787635704739292, 0.4965955217708488, 0.48131433133656865, 0.4937040269914018, 0.4858680179839301, 0.48739039230457387, 0.4899620696839729, 0.4924221788362192, 0.4888813494930448, 0.47504138801660617, 0.4995766774997265, 0.47831368719977246, 0.49564244507903416, 0.4830814238705996, 0.4795293342687677, 0.4890401197762872, 0.4932998271463148, 0.4838068547709057, 0.48363804804426974, 0.48577089439383936, 0.47444449879407385, 0.47381468292333945, 0.4827571462775025, 0.48315621407726483, 0.4793809468320951, 0.4904964770509486, 0.4755052233696291, 0.48667184901446187, 0.4851236520739264, 0.4860364505763595, 0.48524917335789497, 0.4856606484795194, 0.49575264429269406, 0.48648526981827345, 0.4898222895383008, 0.4938700599785593, 0.49106122757579157, 0.4937786146627328, 0.4883965701753752, 0.49316460810099194, 0.4878534519203985, 0.48722082105411924, 0.4890975364844824, 0.4884190698189013, 0.4896946610318948, 0.48912994775147567, 0.48154012667390483, 0.5001823551210633, 0.4897179765775383, 0.4905822521470539, 0.48637587273844357, 0.47746892764780413, 0.48955758401784205, 0.4892866879579638, 0.48654310255578725, 0.4950773716488992, 0.49711624134312127, 0.49928664707877995, 0.4853573908408794, 0.48301272924775485, 0.4902677610232428, 0.4886943810151523, 0.48966414586247514, 0.4771166431302694, 0.49389743589783336, 0.4861704557553813, 0.4830061137368097, 0.47765435617656554, 0.4763095289167384, 0.4881346221490202, 0.48944016131247225, 0.4772517282913785, 0.48061095416834765, 0.4866390777481099, 0.48727108396611896, 0.47932824824234244, 0.49658519508106547, 0.4832822450177089, 0.48280187838947214, 0.49622137013736484, 0.4861933619674033, 0.4873255925047878, 0.4953815608827887, 0.4924441089173812, 0.4838431867436179, 0.47907765538385017, 0.47708765764160393, 0.5049217024728923, 0.4803778328766178, 0.4925249632521947, 0.48139892153490976, 0.48576690722638854, 0.49328803345516614, 0.47509816681109884, 0.49557627692657413, 0.493536297954975, 0.481513872690687, 0.47320260952519005, 0.4929598140370476, 0.5008273537520026, 0.49058421599498847, 0.49515557821139233, 0.48266955727572974, 0.48817373916957485, 0.4887393904034121, 0.47996455335905436, 0.48796368748189095, 0.49085617474638726, 0.49129865515712984, 0.48194344388651245, 0.49950159952375883, 0.479941837957672, 0.4747455363828256, 0.473294197554772, 0.49195805956876676, 0.4992266840634979, 0.49412436358987855, 0.4778659589414597, 0.46907350256126656, 0.4903135747099434, 0.48509005027761937, 0.4864024158208102, 0.4937263723292361, 0.4794239723869307, 0.4814529565799028, 0.4905074007125764, 0.4920222815961024, 0.4801420355977608, 0.5011771673607327, 0.48530334185324664, 0.4963382176864141, 0.48907232566370407, 0.4819443039430025, 0.48788124982056985, 0.4868701609007077, 0.4959521027820512, 0.48715227267987743, 0.4966651515227143, 0.49735543218251455, 0.4838936117225969, 0.49673854689300634, 0.49325069458707765, 0.47783198700193585, 0.4949524770694016, 0.48793197961095375, 0.49253898647542976, 0.48516295443484125, 0.4861339245572466, 0.48375893162176564, 0.4912382710193077, 0.4993139977741156, 0.47934890841147526, 0.48418486085202, 0.5080759298630892, 0.4971840693164002, 0.49797343090922125, 0.49909246840464616, 0.488053784310602, 0.4942819852999976, 0.4681248320114762, 0.48847164069876825, 0.4844502984162917, 0.48273538158317125, 0.4995338751603064, 0.4984182446071723, 0.4911564262757027, 0.49006066977134255, 0.48795829649688804, 0.49047777514057317, 0.4869814149331935, 0.4877771306394293, 0.48483665726272923, 0.4903398695743831, 0.4896343434894861, 0.48132673357713013, 0.4823348700446858, 0.4868975240301217, 0.4783859599191645, 0.48203145102544737, 0.480780679570398, 0.4891183018080647, 0.4834554396430747, 0.48872564238635874, 0.4907132974676933, 0.49253529096356075, 0.4861199297052129, 0.47554768377879647, 0.47752292945328123, 0.49064516415882214, 0.5020152340615656, 0.48425907485687464, 0.4902720246736977, 0.4784471239920412, 0.4900664298050085, 0.47890599748751655, 0.5000441401989871, 0.4824242242014104, 0.5006969904795852, 0.4895730427832169, 0.4963935113090018, 0.48760433190850966, 0.4746973897728477, 0.4904095556774535, 0.4861687255614128, 0.4780531492717196, 0.48862675531063454, 0.4938131188645065, 0.4757921382797796, 0.4878760662901934, 0.48587475086587895, 0.48566531482820924, 0.500191896263775, 0.48737322439395864, 0.4875347928257365, 0.49289583139633336, 0.487771097429924, 0.4983380208490657, 0.49196623641374077, 0.4841207421874287, 0.4873879199026563, 0.49023601496300445, 0.48530521411471533, 0.48888012277152654, 0.47783633580458984, 0.4897634782437886, 0.48794214242012135, 0.47963736590462513, 0.49165855699767924, 0.484433343306325, 0.5039657506302987, 0.4802355416976879, 0.4864810496055901, 0.48850637327434965, 0.4752202121191385, 0.4917496514513879, 0.5035288306199226, 0.49473468323464764, 0.48919033504380866, 0.49201540610689803, 0.4849695466015831, 0.47678381042023393, 0.49280229743677534, 0.4830396903479113, 0.49007258550283794, 0.48276782498312415, 0.4949907003256703, 0.4891977918969023, 0.4932171640508277, 0.49509211047731566, 0.4912071466953561, 0.47897734239702955, 0.48074082443229654, 0.4777140735064535, 0.49021740915259276, 0.5024629482581658, 0.485116807292542, 0.48650706351279027, 0.48831080601480253, 0.49398117982102424, 0.48216165047173326, 0.5015003005768284, 0.482062825886141, 0.4740750816095145, 0.4898419162196866, 0.4835080478067575, 0.49745782363089824, 0.4886689156011072, 0.4829465333230188, 0.4757057524655734, 0.49129852976194327, 0.5015992194448539, 0.49218048125619307, 0.4905363460766633, 0.47892935305108275, 0.48500813603669274, 0.49263452008842384, 0.5041471083545447, 0.4783546827415749, 0.48421282035822394, 0.47644976636556613, 0.4844516217436363, 0.49407819743948583, 0.4893980250400242, 0.4916673083778267, 0.4899243383140953, 0.49963217223795003, 0.5002400498631082, 0.4820917110951598, 0.4988477194353352, 0.4919918505415914, 0.4914211377630479, 0.48480153661217673, 0.48653636981976356, 0.48627223940553144, 0.4917810340221791, 0.4977600069577599, 0.4870158499478642, 0.47506950076097676, 0.4776134375763133, 0.4789302722793727, 0.4824734466643587, 0.49944290335097546, 0.4825912063133391, 0.49122027313701816, 0.4772365236557647, 0.49322838451162787, 0.4980752627937192, 0.4915549376170295, 0.4906434251555459, 0.48051568350692553, 0.4896838978446947, 0.4935059689313183, 0.4773929946171973, 0.4725324243702727, 0.48956029130124834, 0.47346755660052825, 0.5034167937772627, 0.48245586923641454, 0.4830777632361971, 0.48710431913568747, 0.4865197634468363, 0.48228471772744097, 0.48885747613445046, 0.4920983364230872, 0.48822795366355326, 0.493645199728458, 0.4879962787226576, 0.4910628062182456, 0.48837654739633557, 0.485790770226809, 0.4856023728797171, 0.4877245844943057, 0.48258095522268196, 0.4993224322160981, 0.4770436912710934, 0.4911327034109717, 0.4905547865357658, 0.4969491902164314, 0.49478748793120725, 0.49458858906895675, 0.49097491256100784, 0.49427479826359744, 0.48300778468258654, 0.49782607090785613, 0.4863046717631536, 0.4836277957519097, 0.48664766745726823, 0.48374995899464324, 0.47920527543816915, 0.49615756408847794, 0.48594925572262837, 0.4829761417957197, 0.49366285781722735, 0.4973817056211097, 0.4802569957124622, 0.4849282763861981, 0.48670698644430443, 0.495033713945523, 0.4928614983237607, 0.4824951929094412, 0.48592414485074237, 0.49050829184859135, 0.4921240156310785, 0.48450418092298403, 0.4891977108096757, 0.49504914544249673, 0.4842536053875876, 0.4830485275728045, 0.49141446074490597, 0.49206019283691327, 0.4768029070418806, 0.48490097650758673, 0.479671181323828, 0.48224128476496136, 0.4951794425266611, 0.4951873961176073, 0.49651969887984143, 0.48086640819842025, 0.4853393287789969, 0.48662339652678555, 0.48962506706765824, 0.4850040349476092, 0.48843961139931374, 0.47884256372439876, 0.4910698437028388, 0.4856635679942785, 0.475139530861664, 0.4977070875391766, 0.47935057437419754, 0.49360125434563035, 0.4878323136006727, 0.48750133734172113, 0.48705136294889334, 0.48119876078419704, 0.4958433307702763, 0.49244100600144014, 0.4951375574742261, 0.48903550161323395, 0.48724576913153084, 0.4871763731995287, 0.49245678617442223, 0.48485610087771985, 0.49762336588421274, 0.4857918417044472, 0.4967118960741594, 0.49357331302338203, 0.5047606176413459, 0.49206453917014203, 0.4942388042539374, 0.4961367141718484, 0.49551066398105176, 0.4805188594551603, 0.48620414153337393, 0.4921413743143921, 0.4837034236230811, 0.47476718898886205, 0.4985193891740911, 0.49234308130950144, 0.4854951058824521, 0.4811626564227313, 0.48396875796742617, 0.48178538398664833, 0.4842188413420133, 0.5034630799600461, 0.4903114046770333, 0.4836979225966596, 0.48914452672914027, 0.4837623228724823, 0.48392972372207593, 0.5008583691458752, 0.49072282040301823, 0.4930986268414389, 0.4792944685031809, 0.4845779318160174, 0.4826559469808327, 0.5011482449590053, 0.4862236126335513, 0.49027696711605184, 0.47996679486397104, 0.49046876618500257, 0.4775989996256391, 0.48780008325556057, 0.4966990713977024, 0.48864810218671323, 0.4753251183792548, 0.4886614237966687, 0.4830074053027664, 0.4846702252717778, 0.48618322077514764, 0.48855016058206213, 0.49822522678222075, 0.4849556315818996, 0.483331108672648, 0.49586398180909236, 0.49039726661079497, 0.48798742065737655, 0.4903933635116836, 0.4899260841807466, 0.4749392156333131, 0.47510624454075945, 0.48468229070441815, 0.47520132192171816, 0.48460710122006123, 0.4920852145293653, 0.4815874675765263, 0.4879561575830724, 0.48260780577513146, 0.48563625044801756, 0.4894714170702118, 0.49761048205047, 0.4926111878755766, 0.48621088672556767, 0.4760715684854841, 0.48468211903698144, 0.5051553391520436, 0.48911673032582925, 0.48298184597943306, 0.47827770442889644, 0.4812231722042003, 0.49612598142121234, 0.4838117722541784, 0.49258795713975395, 0.49934344632552363, 0.48078314328741334, 0.48683498662292696, 0.4910838947600038, 0.49049470462740713, 0.4794426997201284, 0.5013355352994003, 0.492206395316852, 0.49386845431866144, 0.4893094100055121, 0.47940227201384167, 0.479288877983601, 0.4868318777159542, 0.47723342463218626, 0.4960753883667132, 0.49622242660912863, 0.4861984121240667, 0.48838788220934526, 0.490232216450698, 0.46699629598121317, 0.4883836456129624, 0.49127569877922267, 0.48338908836132155, 0.49475763411859086, 0.49004950369924677, 0.47875852192182217, 0.48651959494566005, 0.49001247601426995, 0.4809361206114796, 0.4889797164145877, 0.4960598242626652, 0.49130930296141356, 0.49311163277232906, 0.48920332124908045, 0.4906128425009406, 0.4896679218149618, 0.500516398231382, 0.4815116452723493, 0.487235587515769, 0.4956140220180601, 0.4850651887060047, 0.48265295120117485, 0.4881386422142981, 0.48342074819965064, 0.4927138231589975, 0.49379819127073943, 0.4827850773831901, 0.4842854579817986, 0.48399579189810776, 0.4811274546825005, 0.49323616029936973, 0.486435451567549, 0.477596225415788, 0.489929592873172, 0.48644997729728834, 0.5023769268507473, 0.4801409805707651, 0.49235752688011714, 0.4812572053964249, 0.48867514877575075, 0.49096038267190156, 0.49287212472024877, 0.4935137608979466, 0.4832336466940522, 0.4904781810335106, 0.4797951176134375, 0.4911033200755687, 0.4897101268326722, 0.48699856355290716, 0.49835328663044764, 0.4732299548903044, 0.48985877251923576, 0.4782648447713439, 0.49681823077527104, 0.4816948539908822, 0.49370262173306356, 0.4869374600750694, 0.49568754623883793, 0.5012669663214424, 0.4858676838203759, 0.4916229803754967, 0.4923874922662264, 0.48453633311896244, 0.48409829972606366, 0.4992097154759003, 0.48209626989946597, 0.5044482929234856, 0.4906959198596147, 0.4876127264372122, 0.49157738551611485, 0.48082643256220764, 0.4892672769064886, 0.47179903091117636, 0.4910601429240554, 0.483141431255324, 0.48648078957962765, 0.48620276792765865, 0.48214322530721265, 0.48099830730945803, 0.482112602706693, 0.4848172110877693, 0.4869953951622649, 0.4870994218999739, 0.4870745464814071, 0.48333580109839214, 0.49183709920512464, 0.4904824719438164, 0.4882221113213906, 0.4801057716681344, 0.47499317073033576, 0.4881387670359824, 0.48857488028231344, 0.4985448121566681, 0.48618732493411865, 0.48653387827212113, 0.48386527395923423, 0.49601856217772905, 0.4949036269826088, 0.4887903496486599, 0.4987245978432491, 0.48107377468907947, 0.48909216235265124, 0.4931240185261944, 0.4879278189453418, 0.4948186995705227, 0.4793292076727622, 0.4942548584269501, 0.5042820933021961, 0.4926934241794958, 0.4876638902391594, 0.4902384584990298, 0.47205178483726296, 0.4801688872367781, 0.48503625202010486, 0.49534892639854894, 0.48759168360305777, 0.4960990762032226, 0.4955158764916101, 0.4943554430637541, 0.48821342798948164, 0.4992953255717348, 0.4727229603621161, 0.48242418022365063, 0.4914421862331386, 0.48161985872115665, 0.49211693460148487, 0.49487368434615514, 0.49851519387010745, 0.47928471988808385, 0.48741796651821845, 0.4967724292608103, 0.4900022671420155, 0.5001809253289424, 0.4861620352036615, 0.47675707388546823, 0.4773372404665972, 0.485952825772672, 0.4896951947825021, 0.4960039879475068, 0.4910449673177838, 0.5025103567814078, 0.4867224613551112, 0.4899998143573027, 0.4884678877475601, 0.4930432448978501, 0.4851835661085287, 0.4815383450849826, 0.4951104067667688, 0.48694959437771085, 0.4821345536140193, 0.47031086838199704, 0.48338375128811434, 0.48268186057645457, 0.4875739656081429, 0.49432002060631397, 0.4869431436935619, 0.4782837715868757, 0.48811761901746076, 0.49663169672233565, 0.4944523627669437, 0.48470232067758556, 0.48341591927207933, 0.48773358360576796, 0.49371574829252984, 0.4893306070022161, 0.4868329958076948, 0.48651831424761194, 0.5065517856990975, 0.4940741059297774, 0.4905734611445056, 0.4824733843025055, 0.4862169136485699, 0.4798170997132847, 0.4868688018243475, 0.48244925534958355, 0.491967887924079, 0.4822701413995567, 0.478324849138561, 0.5034021025605546, 0.4872535195547457, 0.48159787173691304, 0.486675370780006, 0.49264931961282055, 0.4854790224458973, 0.4886103658434085, 0.484627204055967, 0.48673617205105835, 0.49027251804306976, 0.4746935539932117, 0.486744505708615, 0.4950286896729804, 0.4974544718970911, 0.4772088944981364, 0.4924510225967836, 0.48201109314322504, 0.4887422845459509, 0.4989816565317608, 0.4941064756910611, 0.4903143124259723, 0.4947742882662251, 0.4908341750259601, 0.48728788770129766, 0.4836622138344624, 0.47910076279388075, 0.48296268462206116, 0.4889702701388917, 0.47721775541409206, 0.4899776354370287, 0.49354008445886444, 0.47304717685458986, 0.4883434079400049, 0.49569139273542195, 0.48346632500233006, 0.48566944669714085, 0.4773109189334835, 0.48654026003004747, 0.4996851215458569, 0.49169124499345623, 0.4905577589474851, 0.4895675611634213, 0.4867341908668842, 0.47687715187824303, 0.48239186344622165, 0.49320377760704465, 0.4808297067998767, 0.48512636167322304, 0.4935525913775007, 0.4836721574045649, 0.4769051542488427, 0.48370884817189946, 0.48085494293073894, 0.49181128408664243, 0.487478788113325, 0.489105614803169, 0.4924554266305513, 0.47928336628858687, 0.47953325970693467, 0.4951513218592995, 0.49465517520782093, 0.4848095769771848, 0.47725777779265594, 0.47973519722564417, 0.497053442935081, 0.4807190165618999, 0.4842246617519272, 0.4818766022062355, 0.47002648306817557, 0.48166809348223955, 0.4754717229089843, 0.48791076593659255, 0.4988573354835338, 0.4975382957238373, 0.48670483428213906, 0.4880141400880849, 0.48953459199234145, 0.48556123408108176, 0.47820414343854933, 0.4901663190200755, 0.4845684743657245, 0.4854430567334129, 0.4760246810028877, 0.4790147964004418, 0.4845662865791858, 0.4888863017103255, 0.4835328688464237, 0.4834203660186961, 0.48366021898305017, 0.5001487896903191, 0.48682456386612255, 0.4917689799526226, 0.48717958078066204, 0.4885199529550591, 0.4842045089410997, 0.4986615731522645, 0.47666703854276865, 0.49241314219333554, 0.4857201827110494, 0.48362024018211697, 0.4778230223866502, 0.48953285795039686, 0.491222518379594, 0.49154826782570765, 0.4853676204761977, 0.5012258735584977, 0.4890685544023098, 0.47739837130708734, 0.48556463956735124, 0.47866734077839224, 0.48621113894475926, 0.4785312950211595, 0.4914580736097217, 0.4892105228166755, 0.4853847896927051, 0.48704889993306044, 0.4874874931296337, 0.4957521652174843, 0.4850514795251585, 0.48324647764256135, 0.5041033939362837, 0.4924744711218055, 0.49275003626587843, 0.48975515512805884, 0.4856866032018471, 0.4923250678267178, 0.4874385599162459, 0.49445321380244694, 0.4736233509745798, 0.49858396367827923, 0.48039329290556826, 0.46279009448030234, 0.491471221652023, 0.499093115559721, 0.48066323996449584, 0.4899939936560319, 0.492670730315804, 0.48766074361920836, 0.4841741205858148, 0.48615452072954224, 0.49017951993344744, 0.48559587542965493, 0.48516902334026774, 0.48733524527109956, 0.49523431178051863, 0.4838256230337346, 0.485061234832179, 0.49700928590671595, 0.49332624225627747, 0.48689905987806714, 0.4875716258480857, 0.48823703748293534, 0.48415074193291263, 0.49938824768418466, 0.4881030539571926, 0.4862540184273437, 0.48825297651906413, 0.4863214705862156, 0.48453518755496333, 0.48907351032588975, 0.49401475468506223, 0.47270866074559376, 0.47536551150017514, 0.4859946185287464, 0.4846939306611654, 0.48147644209481943, 0.5026449922573781, 0.4839777789805596, 0.492970302526763, 0.4800121519162516, 0.4931863564305735, 0.48788218950880674, 0.505704956869726, 0.4799452945889684, 0.4867234304763078, 0.48545719390986175, 0.4878029102766012, 0.5020831318358051, 0.4920610154439462, 0.48703949649590156, 0.4934167833884833, 0.4920016217845439, 0.47823683489887014, 0.476373286837412, 0.4931631469767686, 0.4867109808400826, 0.48553790902274135, 0.4953448559117803, 0.5030342411265097, 0.48366044316416457, 0.5011076830491794, 0.4856973530479141, 0.4905447423782943, 0.482214041249306, 0.4846233481344754, 0.5057540483098785, 0.48527615180198086, 0.4873964867263018, 0.49882620320938575, 0.49874193826225216, 0.4902089829670227, 0.4735336247520285, 0.4870173508071842, 0.4743390538750099, 0.4898082172200274, 0.4836997240885682, 0.49445596592982266, 0.4874199351531382, 0.4883584924928857, 0.4928168897045405, 0.487080073050765, 0.4900287951888612, 0.5024503696042536, 0.487843091386688, 0.4992881171448741, 0.49463936463182223, 0.4852457194701718, 0.4915440352717503, 0.4847908135062358, 0.4851331396746889, 0.48455261272031686, 0.49137846605104785, 0.4900239545381522, 0.495804407289275, 0.49565248916145516, 0.4880788580606292, 0.4749266648490563, 0.4848086718719499, 0.47473819414738977, 0.49095288513349816, 0.47905700853210015, 0.4904936673747371, 0.4827230255441799, 0.4845818361056516, 0.5012365290780376, 0.4767307801419407, 0.49529227160137684, 0.48806977934557133, 0.4753237160361194, 0.4840190407979923, 0.49328511399259456, 0.49435502333333936, 0.47950472138965766, 0.4856613566054208, 0.4984162515088949, 0.48181396837057183, 0.48468605127661823, 0.4894517366915065, 0.49829472411829634, 0.4912133591716964, 0.47612467322646496, 0.4867167621483605, 0.48394085512377133, 0.4863207824052727, 0.4945730413940368, 0.4852928027802422, 0.4864926998428868, 0.4889082324126346, 0.4864147354027256, 0.4893293554391119, 0.4911615286362546, 0.48784231368661773, 0.48601608557082704, 0.491632199833151, 0.4882798751831324, 0.48449907272264253, 0.49199619068928596, 0.4836903471517499, 0.5036033278693458, 0.48465691619607004, 0.48979481372839945, 0.47757069076971004, 0.4780840929135085, 0.48824546544166414, 0.48079551295078693, 0.48116559287040706, 0.4819511833202681, 0.49190790562488657, 0.4884841546702495, 0.48948485621402976, 0.4819803209147591, 0.48401428130214225, 0.4760351727145735, 0.48715034937116786, 0.4876842490839512, 0.4851431893598556, 0.5003084717153179, 0.4842239879268631, 0.4855826287834055, 0.4952473079582807, 0.5006800022771187, 0.48886828226799556, 0.474853245299809, 0.48068834305205854, 0.48505992746373194, 0.4913863679013651, 0.4879179616766073, 0.4868674009288326, 0.492184535302102, 0.4823642391414594, 0.47695800251108317, 0.48916728504582974, 0.484503482103145, 0.49110500611427604, 0.49627852800761235, 0.4823161861519889, 0.49114726009735626, 0.4903497023016748, 0.4873907382363337, 0.48633447333841123, 0.4774807698698926, 0.4891791430938752, 0.48446617476397924, 0.4757222639281916, 0.4857316333769065, 0.4913379746543806, 0.4845441920435414, 0.4787787053425807, 0.49213472110239503, 0.4933243077874961, 0.48420953468174527, 0.4965391527672458, 0.49254305306062335, 0.493075329921978, 0.49011157291541846, 0.4674658323963891, 0.49095173139212184, 0.4832582781382362, 0.4798701968946429, 0.4841767670605598, 0.483162132633975, 0.4925145496372371, 0.4819310603823294, 0.4811904980938285, 0.4734363335718754, 0.49455618409308366, 0.49199694441470615, 0.48473031647587705, 0.4771131593238892, 0.4804514019521184, 0.49610426966949067, 0.4889030974504887, 0.48862380966219915, 0.48604037989788834, 0.49621449233469117, 0.49234767510882227, 0.48917377066728795, 0.48503967988022123, 0.49056298672043025, 0.47665479563040436, 0.4926194922480836, 0.4931267935795123, 0.48486438427952505, 0.4885711662748322, 0.4846619603356023, 0.4845651108212726, 0.49983876208716427, 0.4990347553826719, 0.49144963954415066, 0.4778532633963965, 0.4883423782352614, 0.4878130590687916, 0.48982558635636103, 0.48445418295330445, 0.4972100132504356, 0.4918509162802824, 0.5057525171133103, 0.47974299174632457, 0.4854574367092795, 0.4768145710401446, 0.47816346372287627, 0.4875208798004751, 0.4966252677895005, 0.4966986298985236, 0.4801608901168245, 0.49075193439055903, 0.48526699266149775, 0.4819714198338021, 0.4817325705246463, 0.4783558520198148, 0.4887619988624445, 0.49041073268748003, 0.49263929384057153, 0.48683932393840423, 0.486938938045745, 0.484504204378302, 0.4775448705261118, 0.4925783486428295, 0.49777148122573467, 0.4864871962635545, 0.4868842274447721, 0.47981425888821944, 0.48871875851867463, 0.48698036048422566, 0.4907595218501757, 0.4823716487389686, 0.4908573606400462, 0.49034074466319666, 0.48836157377935946, 0.4866379687087901, 0.46960363143491296, 0.493315037453575, 0.4921185982971764, 0.49936166696814993, 0.4797817017541044, 0.5039255637734307, 0.47219125626641867, 0.4756259664126304, 0.48827462088145623, 0.4843975513380302, 0.4819605600939189, 0.49546157940074614, 0.4865869524215575, 0.4953486625322402, 0.4938480123485619, 0.4728170913028026, 0.4961097351480159, 0.48795283748542834, 0.4868068646973989, 0.4991471554327273, 0.4834193705298605, 0.48956149533787013, 0.48585714125902196, 0.487291021218559, 0.4842782185836197, 0.488766176751638, 0.47562129450733415, 0.4894530371248307, 0.47792973120399623, 0.48218554691076676, 0.49658970351234893, 0.4965507339598576, 0.4707937718741449, 0.48406965565098325, 0.4898330110786874, 0.5065366231843076, 0.5034919134050109, 0.47933503290983076, 0.47562666375354173, 0.48675599189163843, 0.48844942997001245, 0.4822937646489033, 0.478819640741872, 0.486839338552395, 0.48742255892660163, 0.48714247446080866, 0.4956414008190114, 0.48539767830737024, 0.5006211537627726, 0.48318124482685754, 0.4886370450523167, 0.4887923467785445, 0.4889290214522465, 0.48426015003403744, 0.47996990021970765, 0.4777058953313042, 0.48829817640619483, 0.4924661019380651, 0.4927723008962797, 0.4971073492448098, 0.4969998174500624, 0.48135862912279365, 0.4957152418269605, 0.498615434537407, 0.48354856325897694, 0.49002238095331563, 0.48897848358994644, 0.4826996066987141, 0.48837159913377926, 0.4880093822045049, 0.48042651122924235, 0.4893136122834413, 0.48106605115687767, 0.49328157440350956, 0.4927566390042025, 0.48284304917645043, 0.4756471610646016, 0.4873737255939294, 0.491840529467767, 0.4836997846410035, 0.493053493819442, 0.4810460523707137, 0.48976332113517934, 0.48564065577119464, 0.48538795740759666, 0.4899653177178707, 0.48597880903889196, 0.48876830373338787, 0.4948947455024489, 0.48142307961946024, 0.4832394445588817, 0.485806319671671, 0.4826489502152839, 0.48667716840519626, 0.49040806823178373, 0.4867738235758223, 0.4894034164612811, 0.503079451161044, 0.4887876209116472, 0.49700951104273866, 0.4922090789943815, 0.49270245208970564, 0.4893828499138206, 0.4884070431267048, 0.49401132446552176, 0.4948498762778909, 0.4764358459881151, 0.486826960424408, 0.48796048161176653, 0.4826511949595286, 0.49416449184505196, 0.4934772223663509, 0.4861471170155389, 0.4790456622229695, 0.48944209014398576, 0.48618838448050017, 0.4846891697702644, 0.49506651538969465, 0.4978658876330136, 0.48940842056397676, 0.49063131270957805, 0.48122712610492485, 0.48424899962920837, 0.4894910833553469, 0.48973530943657423, 0.48887177872321497, 0.4837294487433785, 0.47248341873198, 0.4900375620044221, 0.48961189213020234, 0.4889619127180289, 0.49318814211229806, 0.4905416863691837, 0.49179381479993334, 0.49046929655134414, 0.49028567210656326, 0.49811813564135693, 0.48764487351792996, 0.486895641445048, 0.49545124621882974, 0.47905783738544605, 0.47992325500702515, 0.49368534916330303, 0.48257979135563417, 0.4964255035974882, 0.49179150419308626, 0.4815020947422713, 0.48296523243113143, 0.48627964786046485, 0.48328224229736616, 0.48552937784549166, 0.48961641515194004, 0.48502338341019663, 0.4960886152275971, 0.49507111193126996, 0.4943028289625087, 0.4799389862105553, 0.4789665834351955, 0.486197097553011, 0.48850724100482645, 0.50412372224493, 0.4888332655537898, 0.4881236414177452, 0.4899666108001731, 0.49453975912858067, 0.4879792957757667, 0.48743588535975224, 0.4857269811944758, 0.4800229224300127, 0.49482114424441515, 0.4903838014196494, 0.4785234349484455, 0.4691768168369735, 0.4864254753490283, 0.4899211742788682, 0.484415578246441, 0.4920824468704929, 0.4890790486572818, 0.5001268171158842, 0.48309183238409287, 0.4900572500425782, 0.47964676624266644, 0.4886606996675313, 0.4750163364472786, 0.4932499509276534, 0.48921503939863914, 0.4819381437728188, 0.5055506361148806, 0.49399777119782406, 0.5047983923062241, 0.4836717300003379, 0.49755604733930026, 0.4930186531835171, 0.48766212037326034, 0.4771645134900309, 0.4854469503337853, 0.48078306592471454, 0.4883179692531912, 0.4771199966437687, 0.4820746305578473, 0.49162288354088096, 0.49260637621123965, 0.49279290365443323, 0.4843057328457679, 0.4807199672238809, 0.4813902714488424, 0.4948039902234549, 0.5084780205147611, 0.4852339002027031, 0.48070958594733204, 0.4853356627698083, 0.48365436359321395, 0.4816123246869274, 0.4746675225348644, 0.4731470979360409, 0.4833933357327784, 0.4827351350914686, 0.47665482410214083, 0.4895222581858063, 0.4879168949035264, 0.49438386949365287, 0.4906163367104851, 0.4778946199373478, 0.4859298986551289, 0.4938268266331443, 0.4912623082610629, 0.49908685155360893, 0.47869893734086266, 0.49080451209119386, 0.4783802865599345, 0.49121753610993746, 0.48210534694002294, 0.48202682764724797, 0.49699599587352994, 0.4900798196987004, 0.49486220893856425, 0.4872038477224726, 0.48459801517148915, 0.48443689701353104, 0.4900595558988895, 0.4906816975952313, 0.4869622332639195, 0.48626930600366247, 0.4919625246298608, 0.48467267121529034, 0.49919593510989285, 0.4781840731179171, 0.48332660395208343, 0.4943039523245249, 0.48658394612813144, 0.48742152549391654, 0.48925892555091466, 0.49033364349023395, 0.4995800061469174, 0.4832493533081524, 0.47948148496958526, 0.49400604508303947, 0.4867408380859798, 0.48365667155429365, 0.4834828919288541, 0.47462687903825856, 0.4909039031937356, 0.5035946094088763, 0.4948258434692943, 0.4944867663171637, 0.4942085976398709, 0.483536684175174, 0.47988351939514334, 0.49192624118532025, 0.47853889813126516, 0.4877633813942467, 0.47488603048675865, 0.5059048922602426, 0.47665720206929496, 0.4718446537158107, 0.4853330587892507, 0.49655186496230885, 0.4916569650445059, 0.5005037209011258, 0.49701523396839403, 0.49315241664847936, 0.49221871052885857, 0.48845071330041545, 0.5035347444022893, 0.4859687240503188, 0.48782778623218076, 0.48985131341182875, 0.48330579377245225, 0.49415308511777906, 0.4807292651017558, 0.48549347079958277, 0.48553902949877537, 0.49643086840468575, 0.4862224591029003, 0.4916158606303762, 0.4783588183997518, 0.48422788273278095, 0.4789026463958232, 0.4886368436946396, 0.49573994678429323, 0.49045137159878094, 0.4951590641039885, 0.48231191432516884, 0.49897976905373986, 0.49537851535919, 0.4932205174588317, 0.48122421820270517, 0.499067697850627, 0.4825239229621122, 0.4898839167166378, 0.48510056377398997, 0.4901117853449829, 0.48672399932936733, 0.4854750887690796, 0.48276778576223617, 0.47476349974280135, 0.4894409185187237, 0.4913833961623464, 0.4844273458113095, 0.48472625534501407, 0.48802026037690477, 0.49251616969264134, 0.4811119172496028, 0.48135610398200684, 0.481458274570626, 0.4915036314118885, 0.49183885743787714, 0.495305472695781, 0.49302909046683685, 0.49175277954139107, 0.4776143705135971, 0.48679109779275725, 0.4929791341512223, 0.4854544356595423, 0.5038466725562671, 0.4941807488468107, 0.4909008625584019, 0.4943981475102872, 0.48997354882779115, 0.4851811309274515, 0.4940730494590117, 0.5011021502557542, 0.4850221229916229, 0.48783020922242853, 0.47260886508301236, 0.476850618118935, 0.4806274268822921, 0.4988280928201276, 0.4756354864322507, 0.4743107360408464, 0.47592139747085294, 0.48195746021922825, 0.48623142834345223, 0.48317126277609235, 0.4821653421723623, 0.501980439440014, 0.4884553355156811, 0.4807212862194626, 0.49428443823946233, 0.4844494205672054, 0.49978260608902125, 0.4850750221641106, 0.48491957028174626, 0.4842361823060566, 0.5020413061074878, 0.486837553745311, 0.48331602071732227, 0.48534297068560034, 0.48910809879865175, 0.4899144247722153, 0.49528188978469023, 0.4920750953955251, 0.500869199566329, 0.4941687361173556, 0.48167508298322437, 0.4888082048160589, 0.5021001070137716, 0.49993274335177434, 0.4865200706239206, 0.4774199475249628, 0.476645129899645, 0.49274442588258677, 0.48627900689970815, 0.48984061681858004, 0.4844107020179394, 0.4865019540741325, 0.48596405572787826, 0.4888603583010917, 0.49300787564011517, 0.48913266179836506, 0.4792400721502578, 0.4919657426656793, 0.49310607313718574, 0.5015504368727833, 0.48481583642490916, 0.4751963477036816, 0.4957276963149305, 0.48239803769119804, 0.4846508185850559, 0.4971655526534644, 0.49444595555762577, 0.4841036515632289, 0.4904479671208257, 0.46642349707907144, 0.4844956469548326, 0.4845900079822046, 0.4965824875265151, 0.48264948642416633, 0.4924914175827443, 0.4804618268662888, 0.48565530787198874, 0.47753273390691725, 0.4954201032191441, 0.4956270493187918, 0.4924726765562239, 0.4744180137297117, 0.4823980224187729, 0.49194796023352316, 0.48470149215486574, 0.4953880925445818, 0.49670710562595793, 0.48330754651500313, 0.4902982969235505, 0.4956180986654555, 0.48420829238796975, 0.4916545928109504, 0.4869131829188173, 0.47384551985343204, 0.4829891558849494, 0.4893322687618882, 0.48222094524032455, 0.4708870864719353, 0.4801382371099272, 0.48528488898253663, 0.4882065536683145, 0.4979949011017241, 0.48587240118379404, 0.48647119130845884, 0.48959468525348065, 0.4938273637421958, 0.5014770600985021, 0.483988161140642, 0.4872097360951303, 0.4776219762901833, 0.4884126741348153, 0.48720675594123697, 0.4943759529938545, 0.5002415258911278, 0.4952468493450638, 0.47971153764326524, 0.4869827412285492, 0.47549788091037104, 0.49262747229213427, 0.49763470639045315, 0.48407157527828326, 0.4912182368948608, 0.4717259154438973, 0.49140699517571534, 0.4990202068480317, 0.4878980869649761, 0.49673944027526645, 0.49038092750497553, 0.49541888284460617, 0.485266319145462, 0.48828193193271047, 0.4713422762285156, 0.48979377157351744, 0.4838600782506977, 0.4906662321536908, 0.487888489247846, 0.4893070235737965, 0.4778841055240558, 0.4951484955453136, 0.48181609538153375, 0.487901652839114, 0.49236110284677587, 0.4767454785928158, 0.48618544550147974, 0.48379246753850735, 0.4851564785279034, 0.48337151773283304, 0.5011478337689657, 0.4880166013217891, 0.48720240944131415, 0.48964747634793027, 0.4918422366175475, 0.48809142929142757, 0.4887180588594602, 0.48857787933111724, 0.48595704372749815, 0.4814829041732257, 0.49356748160452685, 0.5023721660802847, 0.5008727203891056, 0.4977580117114715, 0.48206352058682594, 0.4941816687112577, 0.47979845397546683, 0.4895363499869071, 0.4828965261194989, 0.4805061151459375, 0.48704638031378705, 0.4861498338798831, 0.4702637644031431, 0.4856122025484916, 0.4824026466557222, 0.4942919461099953, 0.4834651280979203, 0.488880598982745, 0.4852899903159982, 0.4848670635986545, 0.4825745529766751, 0.4798172780794528, 0.4906851364691682, 0.4835571191411758, 0.4839572383119266, 0.47567953378630407, 0.4900776924529738, 0.49224750884665047, 0.4865257917767488, 0.4846788693303246, 0.48823222464785315, 0.48717366462182227, 0.48424067463194287, 0.49261265395320897, 0.48165172107203696, 0.4815918320080788, 0.4878613094599907, 0.48469649378624835, 0.4852799199820208, 0.48923574837006384, 0.4878539357057727, 0.49089698586407937, 0.4926545138375949, 0.4869211163459292, 0.4767764979330927, 0.47992860807944343, 0.487479773462676, 0.4814983687104709, 0.4907200178896938, 0.47565613608756785, 0.4857699613540826, 0.48560044081236425, 0.487020410187686, 0.48605598311214854, 0.4878430970601736, 0.4846605622792969, 0.49248768188980035, 0.48569832433462756, 0.48378553287945425, 0.4882505167795768, 0.4958721413926801, 0.4994305468249981, 0.49230934588987096, 0.47402965678119907, 0.488151917789932, 0.4979934623786062, 0.4908374678400294, 0.4830963751469612, 0.49740449040985596, 0.48135394450172436, 0.48952788451013174, 0.49419849217200074, 0.48827518312376617, 0.48912889165576523, 0.49239844017024464, 0.4835636844957473, 0.49205000450425007, 0.47642364600435944, 0.4856924764486641, 0.48958587220991356, 0.49445188031125187, 0.49314448931218613, 0.4957716340843191, 0.4806759997037956, 0.4920110120538626, 0.48709855277275965, 0.48878304723756705, 0.48676591290352317, 0.48869265661922834, 0.48327594387430145, 0.4750079862165192, 0.48705307728268304, 0.5007844919661895, 0.5013299757118613, 0.4762409227584161, 0.4858917297164783, 0.48054468543860657, 0.4854631110071215, 0.4843193365294101, 0.4858747353164452, 0.4909790497052102, 0.4954452193680081, 0.48530928796573053, 0.4780955478751567, 0.49921592590037284, 0.48813535508870315, 0.49059451345812183, 0.4885097629691994, 0.4940819594869689, 0.4906455838928587, 0.4923653282488312, 0.4796058487312931, 0.4931461416860192, 0.4913056615108762, 0.4879368966138254, 0.48292629085896255, 0.4858773787129805, 0.4852552296191991, 0.488046051912364, 0.5072966807955179, 0.4851551574596053, 0.4916233718994628, 0.5060975955788221, 0.48369450921856955, 0.48068960213371914, 0.48316623579083445, 0.4997447656887547, 0.4878597557249477, 0.481324653005489, 0.47493889059663036, 0.48782762956961184, 0.47560751330453466, 0.4894910646640339, 0.4833047700235642, 0.49028388668407497, 0.4800201625086372, 0.4993345456430617, 0.4992040204345614, 0.4898725947205607, 0.49341241736376723, 0.4911554222068829, 0.48225906671539964, 0.49454796794684963, 0.4855500107945664, 0.49360176745429335, 0.4866104334942055, 0.4848283514482107, 0.4919474505949204, 0.49486470794614934, 0.4923266414664747, 0.48495529138974425, 0.4899443958650392, 0.49222554274386504, 0.49355508240746443, 0.49081146845135837, 0.479660283014524, 0.49357720273151895, 0.4639808444454768, 0.49111275748186944, 0.48818530694265405, 0.4882550218724707, 0.47414377436392985, 0.48703028597698717, 0.4922326465773796, 0.4718650234166086, 0.4852235543948694, 0.48992992173321415, 0.4781899493270635, 0.49148478560627096, 0.4858801194046271, 0.495022363744051, 0.48491459728830916, 0.4709628463424101, 0.48597290479040733, 0.4837252690777529, 0.4866605438653398, 0.48880112741112586, 0.49467890468414866, 0.4912503623554237, 0.4860613548208355, 0.48716219403275285, 0.4933697369540384, 0.4803620611173313, 0.48801313638872784, 0.4933257051948568, 0.4884273812730066, 0.4853598886175408, 0.48894760543045757, 0.48817520258767827, 0.4767071998645717, 0.48169401409292106, 0.4956604899416579, 0.5049774721016466, 0.4926109705161582, 0.4831726865136732, 0.49303118314364963, 0.4920650279987879, 0.49347032855387324, 0.49407030298226884, 0.4973739578700264, 0.48744184290925424, 0.48534277420976185, 0.4838057145058881, 0.4847428755552548, 0.4824047918251279, 0.49911225250179514, 0.4901174346296086, 0.48793444028804916, 0.48662724976361005, 0.48606111048167694, 0.47476780879605546, 0.4870759680242871, 0.4821586131221825, 0.49398817018935737, 0.49024882987625884, 0.4868603751632584, 0.4889637675719462, 0.48785814775294506, 0.49752269757565776, 0.4789258469815488, 0.4768834651294804, 0.4726868533663698, 0.49836496540782416, 0.4888438154286998, 0.4954679129158204, 0.4789574364967563, 0.4825513947117926, 0.4841090782856397, 0.49335253023212094, 0.4835994923236137, 0.4813490130473656, 0.48368814227944984, 0.48412987737681185, 0.49590799848942413, 0.48048040330677116, 0.4780907103520259, 0.48932458301998033, 0.4827367264058182, 0.49347401052341267, 0.49490423661712796, 0.4862627155079288, 0.4972108966617877, 0.4844751754505736, 0.48955512465125384, 0.4803788983649187, 0.5046101010054298, 0.49185816536653243, 0.48333803720094837, 0.4763746188054427, 0.48164144528819947, 0.4890385161436845, 0.4898605044068251, 0.4864368775970298, 0.5070517972400471, 0.4791698368181867, 0.4718106214546965, 0.49172169518771713, 0.48454119856470945, 0.4904831825719311, 0.4778725580445613, 0.48262512025888227, 0.48191549272883505, 0.4879819686707785, 0.4902442605858578, 0.4753044153149579, 0.48702121368776297, 0.4871447862001456, 0.49970224104580874, 0.49545562257435766, 0.49061798532092754, 0.4990048292217112, 0.48181248188830417, 0.4928823571415672, 0.48903879315246607, 0.49021944374266263, 0.48399240322304876, 0.5043148359854894, 0.4908513252768741, 0.4843040720457695, 0.48488779660196263, 0.47628909952381937, 0.48271799919676794, 0.49980291163856233, 0.4875259537733759, 0.4740892357908795, 0.4900683167534571, 0.49783406835204763, 0.506435214928126, 0.49506113485733694, 0.48439146330527144, 0.4888843173081752, 0.4965389167255404, 0.4806364445749926, 0.4819487828504213, 0.49623273358849385, 0.48875448468563437, 0.4932467244884066, 0.4890041727707299, 0.4747903378944471, 0.4888141704993553, 0.4951269597377576, 0.48756824271511195, 0.4702588125468175, 0.48940302610756725, 0.4817722265510287, 0.4811492790656931, 0.48335414956360584, 0.4905783651637405, 0.494544949579788, 0.49996438848052427, 0.47667115300481644, 0.4857191689992035, 0.4824691595578729, 0.4914333085820527, 0.48655432599612003, 0.48044080414946744, 0.4823577432117577, 0.4851427953998707, 0.4906661702438776, 0.4787045597965069, 0.49745833431343, 0.4886226647808696, 0.48941838849654973, 0.48585407253104856, 0.4899524484758421, 0.47945496703401363, 0.4921808323095523, 0.47212261397209987, 0.4758903213226735, 0.4794807928356346, 0.49632892273256546, 0.4918320524778911, 0.4998488366966092, 0.4833423604510288, 0.49590203358321755, 0.4877213334656669, 0.49080762454128235, 0.48992728994336426, 0.4836774535838106, 0.48470695050362195, 0.48783554154394576, 0.46772364723309046, 0.4887649249946956, 0.48110771038091066, 0.498265385559566, 0.47427431667209674, 0.49892314619629213, 0.4877918706920943, 0.4807455973684684, 0.506541260133538, 0.49586351680709145, 0.489641257958693, 0.479164254507533, 0.4933043199746887, 0.4906770787836663, 0.49249989821469914, 0.4910774539938713, 0.4865160918541136, 0.47226331432974356, 0.49288672354042473, 0.4960076932705304, 0.4899005807863584, 0.4906638951941092, 0.48344517439420487, 0.48422667382362783, 0.4742987262899347, 0.5004632735370982, 0.4804674288539189, 0.4802932055803559, 0.4959117358981709, 0.48966383226646043, 0.49497115586154145, 0.4843475644188259, 0.4906872647431184, 0.4814401775381196, 0.48959996667209993, 0.4783120643568619, 0.49255482020904573, 0.4834662057170947, 0.48331820770094475, 0.49284508281874717, 0.48334721442912504, 0.4792600341665722, 0.4958600063649931, 0.4833171105659472, 0.48815179904036055, 0.4928403500715041, 0.5039109285170565, 0.48502207975037653, 0.48268278923669933, 0.49702328530659223, 0.4984205024041346, 0.48742402218414593, 0.4788649685824736, 0.4774959600754931, 0.47828324432505553, 0.4942913045773068, 0.4852030869968731, 0.49277114723713905, 0.4938229066443355, 0.49034393635803303, 0.49025561687569175, 0.4798623238258822, 0.4918906695985952, 0.4924837907294477, 0.4795360766379295, 0.48517424991748404, 0.4872347084504799, 0.48144285406193227, 0.49257864281438607, 0.4837514092795598, 0.4875886176138215, 0.4925840993628707, 0.4802015406406302, 0.4895346163215459, 0.49497535357309785, 0.49592368635046064, 0.4855586957635841, 0.4809101609113575, 0.48812008877979574, 0.4898066911378381, 0.49114127849456723, 0.4833495485187751, 0.48946597558959015, 0.480439342860727, 0.4908496922191444, 0.47474379116055415, 0.49315066966477145, 0.4889095722708606, 0.4989571926567743, 0.4837634281816576, 0.4889711812600423, 0.5015789765913833, 0.47833342474264917, 0.4812094434606591, 0.4821409659132244, 0.48012962926259195, 0.4920744526797021, 0.4898068817042386, 0.4827551862809488, 0.4815368019877945, 0.48814328323389855, 0.496276444540533, 0.5017595302039196, 0.4896361030418901, 0.48638150803190255, 0.49361782903464546, 0.4827725382926192, 0.48952436888204615, 0.49388097047176854, 0.5052815232691087, 0.49425141573861325, 0.48288511233735115, 0.47838725278672106, 0.494530942740317, 0.48302476045371284, 0.4820103595104225, 0.4766678888302218, 0.48182368946871507, 0.4903468319627064, 0.48332879148251134, 0.49358726996204566, 0.5012226020316456, 0.49816541661738284, 0.4839194674011691, 0.4846210458402263, 0.493299526557158, 0.4932861425061851, 0.4898591883704076, 0.4884340354709862, 0.4832609012593571, 0.5010459645005364, 0.47641719902728497, 0.487963570183099, 0.47198215901038854, 0.49859418683481477, 0.48807620361460413, 0.49131999907350543, 0.49104471161522084, 0.486118621894674, 0.48910660190705896, 0.49821280857318845, 0.4810572751456009, 0.4833865088701848, 0.4926212702599816, 0.4840821311039541, 0.48315054968401216, 0.48438536537740745, 0.49132036253897227, 0.4919044733038239, 0.4773728351711804, 0.4873718848115485, 0.4987978328241401, 0.48607710074227684, 0.49054945017971224, 0.48233023258634106, 0.4882702831247258, 0.49410138653463365, 0.487461679227202, 0.4799249707503618, 0.49002336060969426, 0.4831018584182127, 0.4916366014450757, 0.4815583141501179, 0.47963161868263926, 0.48565989313274427, 0.47978290768864984, 0.4896206797566443, 0.49038827875531305, 0.49031313019667666, 0.48460057902305703, 0.48188437214002, 0.48023906054278875, 0.48085595595926073, 0.48152529973301417, 0.4914870205898211, 0.49242972672986046, 0.4973177935475578, 0.4959863095012002, 0.47951542356540927, 0.4891802258206603, 0.48842027949906774, 0.4772629411482642, 0.4912220888575882, 0.4824941851979233, 0.4859664071368917, 0.485142645093264, 0.49054927589141734, 0.4856873699695612, 0.490203807257963, 0.4823180626246658, 0.49046223062127764, 0.4887920970632698, 0.48555423939166814, 0.4846264241417513, 0.4911313312381761, 0.48919380356591025, 0.47427517573510236, 0.5003515555551566, 0.4965164631376634, 0.510171071626457, 0.4931184947855211, 0.4966638978793512, 0.48704863426930756, 0.4841063587036137, 0.48976277705591764, 0.4862562241052998, 0.4900376930016698, 0.48802405807760774, 0.4779352289564684, 0.4870715344360191, 0.4740917322770802, 0.4884004536434333, 0.5017945899122961, 0.474426855307989, 0.49261168533757127, 0.4943109608676885, 0.5012083106626646, 0.48755677712276213, 0.4905538960567478, 0.48418931104931634, 0.4825312772697604, 0.48891395218265365, 0.48897625653696486, 0.5024232295133293, 0.49480873065096, 0.48050324598436006, 0.47924292627057863, 0.4897600220960288, 0.4779710572785056, 0.4960134594985273, 0.4806526051630719, 0.48375943287195944, 0.4874547253766534, 0.48344973029485194, 0.4865596850605671, 0.4918313272106057, 0.4858976099522884, 0.47205904301562734, 0.49860385961887804, 0.4878635814230973, 0.4886533486165757, 0.4799191165152414, 0.48880952030749014, 0.49431744668080047, 0.49092529896845444, 0.49201694713057836, 0.4928629478768565, 0.48906776867991514, 0.48067268557231657, 0.49076519672490115, 0.4843066072734077, 0.490183659107593, 0.48270928205858477, 0.48472339962048644, 0.48577205423593434, 0.48826588785847913, 0.478030024771497, 0.4820843301627493, 0.47704386158514606, 0.4771986520167298, 0.4968095841841703, 0.4868269328529685, 0.4843046171386891, 0.4948514576392022, 0.4944908482235556, 0.47915774104646924, 0.4840766707484031, 0.4921932183669255, 0.48492911161986135, 0.4921490725675972, 0.48152575892465604, 0.4893096711266434, 0.5011274392933665, 0.4852834813514816, 0.4885695330861878, 0.495507953623663, 0.4786615780555985, 0.4967769630847825, 0.4884425725973614, 0.49387908321165336, 0.48367230748195184, 0.48777421999587894, 0.48679399877825724, 0.47649441023529165, 0.49872052143327433, 0.4853793581185662, 0.4818829850504615, 0.48711047771646165, 0.492458906939315, 0.4782338127295419, 0.48250849470789575, 0.4945634215518572, 0.495530172035038, 0.5002719743920282, 0.5037156930081791, 0.4851922703893666, 0.4720271514507418, 0.5037341955522859, 0.47867619820530416, 0.4865303760801033, 0.47537341513804066, 0.5023344299156857, 0.48687996182297577, 0.473958593242223, 0.47305251632557943, 0.48241783594620075, 0.48957596358138855, 0.49731248054564475, 0.48250220148425454, 0.49587401351007326, 0.4844084105364209, 0.48667927055108534, 0.4803341735692983, 0.49634343564080285, 0.47927996346709617, 0.49714080119075715, 0.48237639220366324, 0.4785439843474452, 0.48168006810505226, 0.49294625341736303, 0.49143978307578257, 0.4884961228298638, 0.49057472798649193, 0.4868521567011705, 0.48631416495191815, 0.47761431438232127, 0.4963076947267654, 0.4789999601953654, 0.49138081062099925, 0.47814224828841617, 0.4878944684858059, 0.4850961354180587, 0.491342035066302, 0.47762176068997847, 0.48310082770380425, 0.48363278929978065, 0.49740768055725504, 0.49877687476573934, 0.4893589493315118, 0.48971636222924825, 0.4952485268653167, 0.4886543200699028, 0.4744383996072751, 0.4779295223310121, 0.4731923522102884, 0.4752493853954169, 0.4778573837625064, 0.49119997915438457, 0.4857501611701327, 0.4750037685002765, 0.4858500247473937, 0.4947301092975003, 0.48493239622363543, 0.4809500121626146, 0.4806173671434905, 0.48379819196276186, 0.47787468055472837, 0.48820163856344273, 0.4951173893174823, 0.4891634790046184, 0.4862464559855065, 0.49194841426625274, 0.5009154276965105, 0.47905043218910043, 0.4919557409117808, 0.48420758518721396, 0.48788049484993457, 0.489256363605462, 0.4896988703110313, 0.492666547360473, 0.4822113471132707, 0.4960669566951165, 0.4862364205981836, 0.4854598242313891, 0.4847588115649779, 0.4880737550990455, 0.4931181119081186, 0.47807539708497665, 0.48759214530935774, 0.5001290231418973, 0.4880602645822624, 0.49064724772672585, 0.4831901842955227, 0.49802528468761087, 0.4887227515599006, 0.49222102972450554, 0.48163301129898156, 0.4904410059104394, 0.4931752900384334, 0.5023161451852536, 0.5044832697475174, 0.48377118088809656, 0.48687969389596997, 0.49162656038967206, 0.49223322024705035, 0.5035859069299929, 0.48747112754670685, 0.48938857540542074, 0.48106922680719616, 0.4884071006875761, 0.4818596309904593, 0.48922414179332646, 0.48104612588700557, 0.49562719962498186, 0.4846863077861495, 0.49409425529831374, 0.4911212745447005, 0.4831984051225887, 0.4869177766788519, 0.488117893299457, 0.4806790227984351, 0.4909888448174366, 0.49460368954741113, 0.49460135001567956, 0.48184655539754095, 0.49128950776139957, 0.4848890686882692, 0.5034291573357432, 0.4901296341674923, 0.49947791557783844, 0.4880821374045961, 0.47914271268761194, 0.4885062467328437, 0.4970379114672406, 0.486471216001045, 0.48949611133597803, 0.4959517979808793, 0.4955383450624188, 0.4936118599395477, 0.48790885429522934, 0.49672710013139687, 0.47304238743097304, 0.48234393693565764, 0.4920978040054005, 0.48546830754378417, 0.4904974614436278, 0.4783342891065041, 0.4874179354583174, 0.4872830972278809, 0.4824717282585393, 0.4853810565117947, 0.4942942121966199, 0.49106540254331893, 0.47402269869854774, 0.4824918582371764, 0.500474992496511, 0.48996990708048277, 0.4936350940125879, 0.4863101375715338, 0.4754690991151485, 0.49278101444422345, 0.48802915771368915, 0.4864942022090188, 0.493692380585933, 0.5047280115507359, 0.48105394963302534, 0.4913669001725511, 0.48308123588984186, 0.49245982046549, 0.48760778377757036, 0.4859248890836555, 0.4682799885749289, 0.48423977476515023, 0.47591535124791534, 0.49338123424759955, 0.4945533524847177, 0.5009513001753894, 0.4932127799775436, 0.4880880707450235, 0.4867848564089586, 0.4927206615720935, 0.4872492988222405, 0.48339155638129433, 0.5003973237660603, 0.4972625123518361, 0.49809489008337277, 0.47708701311799584, 0.4865952801826167, 0.4926503567705698, 0.48585173398385473, 0.48650523881880037, 0.491792833878967, 0.4888609485007255, 0.4801554182218816, 0.4985106462805765, 0.4884717275527595, 0.4855737733600373, 0.48013389248649074, 0.4935123873184487, 0.49364268878207923, 0.4800060310350579, 0.4884331464709869, 0.48406308882119825, 0.48955880364661647, 0.49238073587404657, 0.4910750647617246, 0.48932405786849387, 0.4900103950580714, 0.4919472531441173, 0.4750515365548727, 0.48147341822556994, 0.48604446880077945, 0.490172818021673, 0.4899728192144625, 0.48953488324840155, 0.48231881724093817, 0.48495787614209973, 0.4866736430860048, 0.492910349339939, 0.48626258103575737, 0.47856567691304125, 0.48182243082659504, 0.4974116873029689, 0.5003089869804379, 0.4832708556715128, 0.4805789885966019, 0.4672772623317585, 0.48231763456090415, 0.49643002419425275, 0.4926053241784841, 0.48298641124263886, 0.4863098084348328, 0.4986958887611792, 0.47779036668144054, 0.4910967177295033, 0.49091851975308193, 0.48048674216006215, 0.4962553461802971, 0.4808547139685679, 0.48659891084627954, 0.490540111287777, 0.48527556288901674, 0.4899903591873039, 0.4871970860736718, 0.4906111973092907, 0.4823494260536198, 0.47649679097465575, 0.4909341428710214, 0.4811376335277939, 0.4917924924262285, 0.4801093552136849, 0.4737601938507225, 0.49411248986900513, 0.501318520387656, 0.49194887565651824, 0.494901324130985, 0.49899867360165295, 0.494213088699661, 0.4746220714252453, 0.4854603154739101, 0.4827374644056686, 0.48453667006918316, 0.481706963889891, 0.5032722869927786, 0.48724734802339426, 0.49837120375212185, 0.4953494456557048, 0.4883247426373362, 0.49797403408463226, 0.48661374866351753, 0.49326241775922264, 0.4963610890169036, 0.4954699999265938, 0.4898163233478154, 0.4875870621213089, 0.4910677981994204, 0.48103071733307595, 0.4907361410062141, 0.4913960507992375, 0.48570204533634687, 0.5006314034206641, 0.48060050613658684, 0.48338372781874955, 0.48787704209814187, 0.48447966062238296, 0.48961308627440187, 0.4880261821109325, 0.48917051593088795, 0.4879559484940583, 0.4941207816486443, 0.5012033215021228, 0.49109850529519045, 0.49095233971828545, 0.48653607998524784, 0.4977746226991403, 0.49756224185530773, 0.4780637265108677, 0.4920080162403767, 0.4976266588477355, 0.5060897949331972, 0.4838888036370059, 0.4889278545318517, 0.4936136247035825, 0.4946391264651442, 0.48917448670503233, 0.47925982680900586, 0.5001913395779782, 0.47712473053878074, 0.4871004267895501, 0.4958958317468594, 0.490411440763507, 0.48794851179233123, 0.4971934928958397, 0.4968801268831648, 0.475896536699368, 0.48314015572990976, 0.4806112720109417, 0.4884910395355986, 0.4838866127168108, 0.49291113294720246, 0.48404092717703073, 0.4927453826573065, 0.4817342941655505, 0.4924815610716961, 0.48453343451210845, 0.492953848938839, 0.49003645514127236, 0.4826489942925425, 0.4793439968181121, 0.48455208382829795, 0.48389156052345417, 0.4733583385864189, 0.4799906686503187, 0.4978213093805872, 0.4990088429317293, 0.48669460822036337, 0.4815041560389931, 0.48857363862468906, 0.4938119082571708, 0.48438259334758693, 0.49256857702801393, 0.4892358619170841, 0.4705681281843499, 0.4936366448840736, 0.48403116170013755, 0.497327397682823, 0.48939663601167466, 0.4953247065196445, 0.489635480930004, 0.47529308692209976, 0.49016647672263836, 0.4857815201564689, 0.49061498208951626, 0.5013584111908908, 0.5075912730468457, 0.47681030380218425, 0.4902154159554505, 0.49703927101056194, 0.4845511879862001, 0.49116645029602485, 0.4817290858258374, 0.47929390999325355, 0.4786644377721009, 0.489013701777487, 0.4768925377926501, 0.48806432146777967, 0.4825927041892673, 0.4848873085505861, 0.4865588358855376, 0.4796622769316184, 0.48664951361293923, 0.4842978251929883, 0.4902258358556609, 0.47595741463939784, 0.4874423962480749, 0.48107785638363504, 0.48292984365983455, 0.5104483326006201, 0.488320355004802, 0.4888659151969272, 0.4879830835661162, 0.5019218394412922, 0.4913709526738495, 0.48902377848463596, 0.4924593399057247, 0.5023834052033918, 0.47870973369156344, 0.480678979341872, 0.48440204959409405, 0.4850302999411006, 0.49650014929044645, 0.4917125746966727, 0.49731937095199363, 0.49336534289750145, 0.4826581975459017, 0.48364172722138227, 0.4883608898425729, 0.49495235586931685, 0.4889025880769013, 0.4994623710574854, 0.48247582273373424, 0.4909951250154298, 0.4810528399478933, 0.4842136416769388, 0.48324186187613677, 0.4831536431034282, 0.4881712926408407, 0.4742690369368422, 0.486135667957302, 0.4936200241595512, 0.48588078901375253, 0.47930879689276723, 0.49152699223270735, 0.47935424909369506, 0.4856366808031914, 0.4857870273275273, 0.492499376769667, 0.484710378110361, 0.4958431157331424, 0.48890897893984353, 0.48845038334765994, 0.48057953028676065, 0.4854565062188355, 0.4729196443466197, 0.48701902299833827, 0.48340179688420115, 0.4880661512547735, 0.4782429765830503, 0.4888735771205424, 0.48150190610212396, 0.4765160340470978, 0.4917828172907334, 0.48659789276468035, 0.4844835566429712, 0.4846487069812531, 0.4862344322356247, 0.495690191036637, 0.4916240344411064, 0.4999309406442524, 0.48895193238190926, 0.4791277455214803, 0.4943917258386432, 0.48480779772930926, 0.4967019210789659, 0.47984919408458077, 0.48390532392445473, 0.4827956686020751, 0.49291587761466993, 0.5011793516399519, 0.48369455729822464, 0.48625333243696517, 0.4834946129386464, 0.47925319784778814, 0.48016117740537595, 0.49386848458192295, 0.4796964252354529, 0.49714355838962393, 0.4871679045499422, 0.480707045129625, 0.48530208778606687, 0.4854532743117198, 0.48517156811734913, 0.48710241830558465, 0.48883643841207386, 0.4989133019614832, 0.48558526483825976, 0.47844404161246334, 0.4769401843946612, 0.48838283323437276, 0.49420071081002376, 0.47956665756380784, 0.47406641576568287, 0.4867982487242452, 0.4871780016178255, 0.4861541044619648, 0.4980087117083601, 0.49791016741125216, 0.48611398371770537, 0.4918690805554708, 0.48502985764854345, 0.4901074289866241, 0.48507589420781944, 0.48473620195160466, 0.4833809955833821, 0.4896790439569223, 0.48788834561046573, 0.48769002197209715, 0.48138271305959135, 0.4961669566757033, 0.48606409709126513, 0.48392445423490366, 0.49077500081315656, 0.4945138864816682, 0.4810594052849853, 0.48633052113581643, 0.4803799244193167, 0.47637522169208746, 0.4865571641437915, 0.4781793432741625, 0.47923560305696083, 0.49551049604201663, 0.4875995500074292, 0.5003290207262285, 0.4893347004335659, 0.49319204701348257, 0.49599653546532346, 0.49863504248706264, 0.49362032499207836, 0.4850860449949376, 0.49143123897895913, 0.48421280533593214, 0.4961150548331717, 0.47313442383271953, 0.48820062483438675, 0.49386831657322444, 0.4970980082452407, 0.4844939258756694, 0.4877305246326151, 0.49748730952112313, 0.482953265317028, 0.4851995752187617, 0.49765268662041084, 0.4793846346393521, 0.4786787377194101, 0.4918068585108008, 0.4913825670562326, 0.48046342983363716, 0.4760442068154469, 0.48909767400994847, 0.48696610453345757, 0.481555295783911, 0.48447114742813785, 0.4910767293116527, 0.4884344508163345, 0.4952083302777509, 0.494648767942186, 0.47789787939665407, 0.4875367943276484, 0.4879183816222615, 0.48977899073316994, 0.4909462682336646, 0.4857659948230314, 0.47983624615265325, 0.4755022287066926, 0.47872177541318195, 0.47702865541308986, 0.48001592119480685, 0.49658478703271874, 0.4935577123938807, 0.5015494848128826, 0.48050207809546586, 0.495612133567772, 0.47923141922218343, 0.48251429242951366, 0.4873883112884653, 0.48160608254289883, 0.4979814516209545, 0.4848573903848432, 0.4888227676419822, 0.4881502141371119, 0.49290857108225267, 0.48349451488133116, 0.47787793938113204, 0.48769558264012974, 0.47928630363279434, 0.47192782209314094, 0.4830665402878234, 0.49230961554790076, 0.4899397991076181, 0.4874982265159897, 0.48520823500638205, 0.4842117942636864, 0.4897512426148777, 0.4769438224002561, 0.4849357257153357, 0.4928490562273785, 0.481721077169155, 0.4949047309817151, 0.48337495166978683, 0.48823317516907094, 0.4956104303793717, 0.49474334043013146, 0.4768147442166047, 0.48754929711380934, 0.49197981113727246, 0.48437034598048634, 0.4896670129456545, 0.4712485623443281, 0.4937230776308885, 0.4877181392679457, 0.48596193905500007, 0.4914601643610367, 0.4763688268211465, 0.4810249985944069, 0.4865946731911092, 0.49532696832033396, 0.4941070995964571, 0.4764273594227145, 0.4866345894332131, 0.48021506866402236, 0.4908639122116177, 0.4808449955725173, 0.48755438571702614, 0.4798957077984609, 0.48871732205817653, 0.487023816710917, 0.4840414207930195, 0.4819716219571285, 0.4867101861196434, 0.48932211441407547, 0.4897552962869017, 0.4755626332704908, 0.4904595866720384, 0.48723214788044145, 0.4802872488742452, 0.4845404154820447, 0.4840481000156631, 0.4920970763102652, 0.4936356788983818, 0.49074981688035707, 0.4848592913263795, 0.4883948577259822, 0.4879256502215243, 0.4914496950995734, 0.4832873511287637, 0.49657263792516126, 0.4895109828801214, 0.49185192063529454, 0.48760413554601245, 0.48350641757071877, 0.4829348606171832, 0.4773469115203904, 0.49230489475520967, 0.49090964502621215, 0.47716341373620746, 0.5027349861679669, 0.4854108548635746, 0.48360130053996786, 0.4961734303457047, 0.48346046501098827, 0.47811311961559283, 0.4980478929070089, 0.4830128405133055, 0.4914663018726124, 0.4908695345307503, 0.49376300748311924, 0.4881932482399403, 0.48460218580149617, 0.49765400679709426, 0.488023111620055, 0.48165303965368045, 0.4708509124590249, 0.48386432945797314, 0.4924464584584022, 0.503798973677108, 0.490066075168596, 0.47740360159037215, 0.493909500274824, 0.4989808031696322, 0.4799204061368407, 0.47979333113110595, 0.4899229088706959, 0.48507979457676553, 0.4808849083682221, 0.47808847545419203, 0.49078120252656304, 0.4815844917845182, 0.4916535117204621, 0.48064993840486536, 0.4914939200736556, 0.4920813125900453, 0.4890865003938764, 0.48530720920947673, 0.4954972838770996, 0.48570075491815945, 0.49794670002165153, 0.4841268292531959, 0.49332072703718677, 0.4902625136687448, 0.49245761353910017, 0.4846796914573719, 0.47830552249296887, 0.4870716964685428, 0.4817358409831895, 0.49954288080548676, 0.4856850251939672, 0.48279304439489457, 0.4888755946877652, 0.4975266606216787, 0.48543317955034015, 0.48827625753969295, 0.4891450597909769, 0.4895400055892936, 0.48941607297124134, 0.48214703685217675, 0.4975523275032739, 0.4916878689224973, 0.483683249936959, 0.48531328466976914, 0.4835842366856364, 0.4918037499916638, 0.4860584356900985, 0.5000754562357548, 0.4922820582546255, 0.4888609559379883, 0.4772825469280024, 0.4884659497186976, 0.4818481907956385, 0.4885915967314183, 0.49076551074936714, 0.4877278140362707, 0.48975861898753215, 0.4926813311994152, 0.4887827881339664, 0.4893617913695362, 0.4948645471965239, 0.49286368672114755, 0.489599106768899, 0.47674730283219846, 0.49004231656108066, 0.48847251814414244, 0.49738131285291787, 0.4883941702129714, 0.4959811460625611, 0.4873380399710611, 0.485636013165681, 0.4807276729088093, 0.4900894600897599, 0.4946801371343368, 0.48795864134861333, 0.49064187789976615, 0.49196044325150234, 0.4861036817829072, 0.48898274718788803, 0.4781277577291812, 0.4810632759863009, 0.48480670254779135, 0.49075684712559653, 0.4890703164404199, 0.48969849577287367, 0.49647842437106765, 0.4876024918608094, 0.4946263129541787, 0.4877098067518205, 0.47609488251347254, 0.48790156152322506, 0.4947454587504058, 0.5003522923402899, 0.49217476281749023, 0.47786524663583596, 0.4941756292502423, 0.49303748078842574, 0.4981258603396277, 0.48876685798263714, 0.48042109034181496, 0.48821294657453607, 0.4870178539241596, 0.48363997177533735, 0.48481262465142744, 0.48979321160411293, 0.495217324137568, 0.48887577167045015, 0.4917172259717155, 0.4792401821254011, 0.4812317275032133, 0.49539718954157963, 0.47369932676343673, 0.4909950369478717, 0.4889089730877198, 0.48899383550362274, 0.47796050005571994, 0.49173053601879035, 0.48802485915760296, 0.4695869480667864, 0.49241583220178403, 0.4920434849040811, 0.4904974717457927, 0.4780321267470036, 0.4939836285219152, 0.4935638741770556, 0.49371292073756295, 0.5000540405445035, 0.4851762838637176, 0.48701027955673404, 0.49515159946533355, 0.47576406572958857, 0.49494497870310994, 0.4846818772100169, 0.4924823631860459, 0.47511683037885294, 0.47963780350289154, 0.49683257883934306, 0.4913333449481313, 0.48725564186563797, 0.49480256881708945, 0.4925142978719218, 0.4873132772531132, 0.48784152860849533, 0.4842605292354989, 0.4865962934139529, 0.48251324204615603, 0.4817425183938141, 0.4908527215915488, 0.49087470864420035, 0.4835654408591676, 0.4893282412180875, 0.4920963787176077, 0.4921324787381427, 0.5005944556308198, 0.4880176364135262, 0.4943565889020442, 0.4884004309248315, 0.4845340308461705, 0.4858174744203059, 0.4861682037405839, 0.4799143990836895, 0.47221493869493114, 0.48401349726096615, 0.4936029070468097, 0.48931065999985285, 0.48684310261254304, 0.49284099409509013, 0.4843120299942343, 0.4827925825862234, 0.48012891045346145, 0.4906886477408767, 0.4966917305455803, 0.4863476554509913, 0.4806956937752148, 0.4925235198458284, 0.4901847969043652, 0.4850783950890719, 0.49442963606676127, 0.48552769397953904, 0.49507040384507744, 0.5010740558999034, 0.4867973914175738, 0.48758368089578175, 0.49112533301140854, 0.4898316006381404, 0.48034696408189975, 0.49314303885850513, 0.47081526581251953, 0.4901452391080266, 0.482185012190146, 0.49196655293715374, 0.48804786853815463, 0.4900111801944645, 0.48779789650898403, 0.4769427480475182, 0.4947903010028249, 0.4938649606184688, 0.4838960275092552, 0.49036910161097147, 0.4926106552154938, 0.48831017740031224, 0.48983366936556866, 0.483049723803271, 0.48380080745361953, 0.49072709355163274, 0.4932279076595288, 0.49354660147012386, 0.47541733472524483, 0.49341520933145927, 0.4845140639811841, 0.49280607357390105, 0.490254799241911, 0.48620053605478813, 0.4727858157823588, 0.4865450341515105, 0.479570436076843, 0.4910240661417353, 0.4932559640403461, 0.4891212424614433, 0.4823712114178157, 0.5004014297151637, 0.485807562655, 0.4914544544432243, 0.4844413376603423, 0.4857016807162656, 0.48369973843846686, 0.48754141761057956, 0.5034807872032233, 0.47006310386675876, 0.49325863118438096, 0.4920020469226857, 0.4911220890155465, 0.48690769047765753, 0.47277432928285906, 0.4892736018544828, 0.4836465124079269, 0.4863232045182646, 0.4716616119578822, 0.49725227808598116, 0.47050206854447185, 0.4906969334760997, 0.4845328500986522, 0.4891439554971527, 0.4862783387869774, 0.4952301993825685, 0.5008975914445475, 0.4994546004370064, 0.49231207973968455, 0.4970315951989079, 0.48259024345705626, 0.49412519559864043, 0.4867757406853203, 0.5045721493068894, 0.4815370167290241, 0.48477418925027016, 0.48382658995527067, 0.4834787564136202, 0.486009548732991, 0.48976911974261844, 0.48770018098928564, 0.4825590298247205, 0.48022181048523443, 0.4994637933887616, 0.487303530962335, 0.49770191758271376, 0.49140792524721977, 0.4752573087111601, 0.49539964835602707, 0.4760300596294408, 0.4837832732987511, 0.4816580736548546, 0.49028373067708975, 0.480745257708487, 0.4864538910406029, 0.48348636185457744, 0.49009042814641723, 0.4803762833260383, 0.4798566730225171, 0.4974156345977485, 0.4862729732592402, 0.4913532790301277, 0.4727880990645836, 0.4842330165700553, 0.4959137135265929, 0.49233984936800645, 0.48879755265111696, 0.5062282037306146, 0.48925157437039307, 0.4929050803930845, 0.49273423780665976, 0.4893241516974283, 0.4854672886485877, 0.48288528194468655, 0.47985493910418553, 0.49437927671119786, 0.4936941475339169, 0.49171995309460115, 0.49626748845852237, 0.48192124664927394, 0.4976041584217398, 0.4974386779835401, 0.48156778668855693, 0.4766149951722293, 0.48638394529436224, 0.4879031033113423, 0.503546152639799, 0.4896561657791887, 0.4969828443345031, 0.5043107392826475, 0.4882803136254389, 0.49505002172063206, 0.5025580515496074, 0.48290900856709906, 0.479278118717841, 0.4852054976354808, 0.48037005959772133, 0.4933694971044495, 0.49691079757354206, 0.4785283050294973, 0.4969925593445648, 0.484985869624723, 0.4854077087340296, 0.48429929189714616, 0.48870269881477824, 0.4822939261199924, 0.4791641834430726, 0.4899749650742034, 0.47120898258945243, 0.48191822402981865, 0.4773479698444221, 0.49136795748327805, 0.48617276369845536, 0.48050291542144763, 0.4879283250528294, 0.48787653051414487, 0.5013975419336082, 0.488942032683722, 0.49868224116376825, 0.4870643885304414, 0.4844173242298725, 0.48688332497614734, 0.49008019777983713, 0.4899196522430394, 0.49733667295344114, 0.49283152891942156, 0.4923103373966269, 0.48629599319725314, 0.473697917325509, 0.4939563544848764, 0.4908362428611994, 0.48344973082209786, 0.49914671489005313, 0.4893418895877919, 0.47517438813883445, 0.487024743631708, 0.48972779672367917, 0.47771408343699423, 0.487146757065898, 0.4872857174344277, 0.48838712838128345, 0.4821327556034032, 0.48148817766305857, 0.4743697804020274, 0.49153881154764834, 0.48198160980173255, 0.48764777480763094, 0.4917517841964779, 0.4851355984047767, 0.49018079219815375, 0.48794223690641175, 0.4943279057052141, 0.4887801511598122, 0.4823422938885081, 0.4795543563907771, 0.4918169270562794, 0.48249204349346914, 0.48291098200319893, 0.48824742591561593, 0.49140953912529656, 0.49172659346858727, 0.5023225290935406, 0.4929003284748189, 0.48328885939074295, 0.4868515855333095, 0.5075513655602719, 0.4868797393401782, 0.47967182401940817, 0.4808629435158841, 0.4822856693567044, 0.48964919106342536, 0.4893267990600226, 0.5024307562463148, 0.4932693520412584, 0.4752319621953489, 0.4875528771402984, 0.47162397647611826, 0.4889572523674032, 0.48711223551063615, 0.478353593242474, 0.50141811644579, 0.49383950864284204, 0.48682891510573556, 0.4901575880089474, 0.4711350691697176, 0.4875055442847081, 0.5030902846206349, 0.49408197890075467, 0.48499910164159615, 0.5025781816822797, 0.49812120989848324, 0.4874187237576415, 0.5001355077731309, 0.48968051604990814, 0.481754637625861, 0.48550490103543, 0.48573829017836917, 0.48966591011245647, 0.48733828155314424, 0.4810548961235765, 0.48843277407629315, 0.49260833524801567, 0.4964104228015133, 0.47871782761791515, 0.4750122872959934, 0.4866011417496512, 0.48295905126234673, 0.49115940472933833, 0.4853655056966093, 0.48801227303216815, 0.49411478522138297, 0.4846995506445381, 0.49719856535990226, 0.48743205108999826, 0.4905645207812259, 0.4859792514558863, 0.49321695766322693, 0.4792409804732797, 0.48900089096843563, 0.4888641359807678, 0.4913401970110493, 0.4721301420741145, 0.48925340748899, 0.5033269024102213, 0.4897623457162865, 0.4812513040656449, 0.48856370275293115, 0.4919319618971513, 0.4918740578199276, 0.48664911657463733, 0.48943531968639153, 0.48070680037458186, 0.49223820495586673, 0.48198020919647766, 0.4863373627231785, 0.4953416291321301, 0.48446016645157836, 0.47975257731319865, 0.4806697181553036, 0.4845071899080448, 0.48575046417874573, 0.4884686731680557, 0.48430142533064496, 0.5014625941929524, 0.4822699901371679, 0.48422891523731393, 0.4937333981096734, 0.4774294800987461, 0.48925173622998797, 0.4924946521345994, 0.4845460404395198, 0.4877202151419875, 0.48076372648648596, 0.49220271236207835, 0.48371899956157677, 0.48137297240341753, 0.48754844676975134, 0.49972272666067474, 0.4933416373397598, 0.486950013947819, 0.5002742868330646, 0.4852072232843529, 0.4998080611197554, 0.4815696951278185, 0.4994579312669422, 0.4828854259594917, 0.488289123693687, 0.48887491849423437, 0.4857451721458148, 0.49968304569736316, 0.5052054288145423, 0.4970592168270983, 0.48116107672302305, 0.48134180348115824, 0.47802120048514823, 0.49599777695084396, 0.49705969089348395, 0.4878963453749699, 0.4989022382958478, 0.5016318258927901, 0.4870189646510419, 0.49339877973235047, 0.4928811091388898, 0.49044909323151187, 0.5047745130428961, 0.48343772644814975, 0.4995808878194989, 0.4829421519944849, 0.48199299726313194, 0.4747197422189838, 0.48947491459233616, 0.47669002803001587, 0.4791765504731557, 0.48610935287868734, 0.48291684772350446, 0.4780691003777886, 0.4886734104476868, 0.5018057850358384, 0.4846552712197657, 0.49632808008188956, 0.48916783368671074, 0.4841315010210145, 0.49822020178267246, 0.4783520624211434, 0.48683268582624534, 0.4921301417854883, 0.48496283450018773, 0.4929029305325749, 0.4847919710329957, 0.4750103050331819, 0.49301068664673287, 0.49323745870586183, 0.4758387365447551, 0.47785570405289046, 0.48539267367208727, 0.493796412499335, 0.47727903341468075, 0.48083524406987294, 0.48519143672745446, 0.4983942171497887, 0.49289322288211573, 0.4918875919781191, 0.47295891859653993, 0.4932021690700888, 0.4988023259377396, 0.48267606090554477, 0.48904447421795305, 0.4864021195060245, 0.481944753635104, 0.4951551173779175, 0.48959043360015764, 0.47241319495861733, 0.48935459884222027, 0.4944498479141372, 0.49101665729558497, 0.47873736821968527, 0.48902645703361486, 0.47681290931867865, 0.4820951083119765, 0.49788341667145336, 0.4864760329326504, 0.4839584867788751, 0.47145067635994214, 0.48483315047723663, 0.4864352902441567, 0.49399236254793505, 0.4968158298674405, 0.49567206622216053, 0.4838165976429114, 0.48366771609516745, 0.4869728473827068, 0.501697744504959, 0.475885938666741, 0.4798765261663545, 0.4758022547812084, 0.48744044542133175, 0.5032487103301985, 0.4706040157075253, 0.4985202582299486, 0.4860967984006859, 0.48219902470761583, 0.4710189532408693, 0.4795203737540293, 0.48977698543227516, 0.49381475436031896, 0.4886375025180029, 0.4846782828811012, 0.4788639363032019, 0.48236586452364005, 0.4983199299448335, 0.49856471802088176, 0.4827529014698814, 0.49102545284288424, 0.4909947519061794, 0.4899332083957162, 0.48975722058346305, 0.4892401337373853, 0.49609861614316964, 0.4765387609850612, 0.49287401648235674, 0.5035141887416033, 0.4945042567032941, 0.4812748937993999, 0.48659063996049684, 0.488965218710006, 0.4946414781165195, 0.4886822089258503, 0.48072306857427477, 0.48496299675541205, 0.4966830444649604, 0.48822563478288505, 0.4945674563344915, 0.49503675031214883, 0.4806689284365503, 0.4889093825192633, 0.4877140539202049, 0.49414529607813407, 0.4860783768659304, 0.5007772290785141, 0.48857368777012117, 0.483141816764272, 0.49102114273786734, 0.4751211385860421, 0.48378184630139237, 0.49035763352699335, 0.47892900275297046, 0.4757056901499815, 0.48985394584488035, 0.48401657643679813, 0.49658263423498944, 0.49655299035312034, 0.48954457370807, 0.4872358677638936, 0.4895257646439584, 0.49137583251061945, 0.49299668459504997, 0.4884111513716739, 0.48084788172305515, 0.4900451355697455, 0.4820665776699479, 0.4835373920109046, 0.48354882675390853, 0.4867941260697851, 0.488446622149857, 0.4893331977499322, 0.47952246365479856, 0.4985977758103382, 0.4878806850696886, 0.47689457272679436, 0.4773980303332905, 0.49908492028397716, 0.5002034650626406, 0.48541619067629055, 0.49705769448739895, 0.4876467774376116, 0.4893693688381614, 0.4825451767142399, 0.48703492082903627, 0.4867772769846496, 0.5010876446142284, 0.49203887791828166, 0.48470151618638696, 0.4811466381843171, 0.4957309825750788, 0.4886491273463629, 0.4907257455506358, 0.49004495387183855, 0.4830670124732702, 0.4848283530960953, 0.4919606297936084, 0.4854347970737461, 0.4925342619959052, 0.490756017421363, 0.4820910051229583, 0.49006210145209506, 0.48829137797803635, 0.4907965563700332, 0.48870866633610477, 0.49057559439392673, 0.48927985876581, 0.4897979876256903, 0.5052897328298429, 0.4819076796784105, 0.48771535403274013, 0.49848723109085524, 0.492102513164018, 0.4852584531115516, 0.47696496281480344, 0.4833815165604525, 0.4926774648077138, 0.498419565257026, 0.5016254649794394, 0.4832372680546139, 0.48967561951467187, 0.48686235257275606, 0.49799014753052495, 0.4919554672029111, 0.4902743043059128, 0.48676215642693244, 0.5096089479791196, 0.47447231982289867, 0.4945924815358438, 0.48303136457055507, 0.4886332110538988, 0.49422478012209026, 0.485264055615835, 0.4991731735248398, 0.48069472657623175, 0.48551537917415494, 0.48379390153636886, 0.5038952744173639, 0.48873287140779137, 0.48752652349518794, 0.4880654364454344, 0.4927826161059224, 0.4877568080418782, 0.47956360928776814, 0.4947427767945665, 0.49358751336715284, 0.4751164276032803, 0.4995701674006148, 0.4913591862667314, 0.48241427921618163, 0.4856025843227638, 0.491003001005191, 0.4970547101170692, 0.4943557060764158, 0.49207128303191733, 0.5040732480744521, 0.48995366115348965, 0.48929465198148736, 0.4912484540241571, 0.4959330952599215, 0.48100814177117834, 0.4867818620247205, 0.48992498190179345, 0.4874250147824031, 0.49850989382185357, 0.4954042471620653, 0.48479136677407114, 0.48158887178424475, 0.4917550154240761, 0.48960219867900306, 0.4937410156713779, 0.4950881983416791, 0.49938294564359476, 0.4834883950877836, 0.4967183759452101, 0.4884905373085728, 0.4924802243431012, 0.4817363021272385, 0.49870901895323055, 0.5003670245380272, 0.48686464687595193, 0.4913080862323549, 0.48824742485284706, 0.47787700414112116, 0.4897699258827914, 0.49760516966414536, 0.4880196424276093, 0.48877597067307127, 0.48486124144279263, 0.492765765688949, 0.4904837283837361, 0.49210168189798936, 0.4878017633426148, 0.47911667090676374, 0.4809436970962633, 0.48309244769898746, 0.49029032231584635, 0.48745228859181594, 0.48921934319561805, 0.4814936278622117, 0.4747938411135368, 0.47517786559661, 0.4949960463150489, 0.4893850864788141, 0.48749599277066324, 0.4868328137646037, 0.48801136212639074, 0.48281840274265875, 0.486238044935067, 0.490446843548782, 0.495380627859309, 0.4963505792638571, 0.48639355321292865, 0.49110976284819574, 0.4752934810042259, 0.48192498035399434, 0.48637402327096946, 0.49693270731418837, 0.482100573574577, 0.48600038419102837, 0.49025722908021024, 0.4875131119490572, 0.4971814392835014, 0.48872205715582456, 0.48762524603074225, 0.4867747398922322, 0.4785120975308855, 0.4941823688477004, 0.4997816962353829, 0.48650021478779537, 0.478493454418395, 0.4779426119760611, 0.4798358255279539, 0.49787106823708843, 0.482793943737139, 0.4835629327894203, 0.4903719102304844, 0.48453669275250777, 0.4852729495769689, 0.4963383966380398, 0.49240458232548884, 0.4917810158851969, 0.4763382421312072, 0.4911083120056065, 0.4863481832590159, 0.48351048133959484, 0.48674808728551233, 0.4982531867767421, 0.48234468109323947, 0.49097125666823094, 0.4824212781734909, 0.5025463344101888, 0.49630484080598525, 0.4912165238032705, 0.47689859383361094, 0.4777900744717284, 0.48291062644493254, 0.49379651541600267, 0.5008199719149969, 0.49167757823985475, 0.49323900008548666, 0.48149545083519874, 0.48337451937190756, 0.4874387473715314, 0.5017959280163069, 0.4985561602109169, 0.5039822782306078, 0.4885974977191008, 0.49570357806876286, 0.4891496724823808, 0.4835513010692976, 0.48074518804129296, 0.4964047550048539, 0.4881419339993453, 0.4970791670856385, 0.4789694059677899, 0.4854431676519324, 0.4745310998491467, 0.5066808375871413, 0.4830535392277113, 0.48786690080605566, 0.48321187942294025, 0.4852050747624459, 0.49880102805462245, 0.4876413631503148, 0.49605517669540555, 0.482655387785423, 0.4946973627367385, 0.4879140309532991, 0.48141804036026253, 0.48676371937438767, 0.49402489757907103, 0.4901405316251461, 0.4843793271163218, 0.47448205144238076, 0.48609856441824495, 0.48373125628754615, 0.4805260323786346, 0.4716863918958736, 0.47777528465408237, 0.49618209109719563, 0.48081860793195197, 0.4752092125145138, 0.479613252848466, 0.499567641858458, 0.4986946294012301, 0.47721186247661773, 0.49351842972771004, 0.4843220930682398, 0.4816260506553494, 0.4813244871528908, 0.4907241199092922, 0.4903962918899394, 0.4766730262419423, 0.4851120680188956, 0.5005469692521195, 0.48958251626098265, 0.48448910054851524, 0.4958022335486231, 0.479640043470404, 0.4817359397194004, 0.49265417898329356, 0.4774489921018913, 0.4867631811372279, 0.49477023011328625, 0.4862813655765873, 0.4865143579260206, 0.48933731854078566, 0.4960741234101208, 0.48991503318594726, 0.4825009904259496, 0.4780541252233035, 0.4956542935879507, 0.48003009658478824, 0.49018609467265506, 0.4842523229032811, 0.4837533887579727, 0.48315549567955424, 0.48412958939856005, 0.5023441590894308, 0.4827011045833673, 0.4959301478589779, 0.47944499955880204, 0.48987852216732636, 0.49115771942291586, 0.47495443490847716, 0.4901900472164299, 0.4917338402442098, 0.48956497613831923, 0.48177422729866826, 0.4839851448595636, 0.4960533395605005, 0.4903402901410968, 0.4895274145232651, 0.47349271689829586, 0.48203866879639345, 0.48862061069100654, 0.4918364310732417, 0.4876671473486752, 0.4776781035333431, 0.48498849047511555, 0.48332014802042794, 0.4791096640874048, 0.49263646634127045, 0.49016904320172916, 0.4871420545157157, 0.46754234594368577, 0.48711945604550194, 0.49110104071081256, 0.4871116678456298, 0.4853532175519584, 0.4908348924508098, 0.48907955563902195, 0.4927494154217367, 0.4906043823565013, 0.48924479803331605, 0.49296799073895836, 0.4846308046051302, 0.48630409115769374, 0.4712120376264517, 0.48886441602029873, 0.4876642937927434, 0.508054946653085, 0.4913636102926141, 0.48749937990237363, 0.48379938032686864, 0.4948314034671371, 0.48186765169842716, 0.48715035781091015, 0.49123787593940593, 0.4862983653305653, 0.494229262907911, 0.48638298976136407, 0.5035617680390874, 0.49050264533261645, 0.49114727150921844, 0.4854427046916634, 0.49077050870593847, 0.48889850875161633, 0.48494971570783124, 0.48928341232091416, 0.49639287698713375, 0.48567929367415846, 0.497464822013881, 0.48957071487895004, 0.4757476771671634, 0.49476152190460626, 0.49001992738445427, 0.49038992878254, 0.49323161410966126, 0.49216513818579205, 0.49485966040812374, 0.4815795053343923, 0.4785722263080279, 0.4956286917058205, 0.47675052729364886, 0.48621862737196436, 0.4929306635553329, 0.491106990721382, 0.487645215623966, 0.4873768943081072, 0.4913472199282967, 0.48770802429708865, 0.4831738676080077, 0.4842982287426174, 0.49062846197557125, 0.48848252247524065, 0.501812011057431, 0.4774826531945889, 0.4945889488375038, 0.4991650003075168, 0.48264784749955336, 0.4785428741522, 0.4832214558219909, 0.48867645659012127, 0.4947114606847405, 0.5021963898893532, 0.4838955231797732, 0.4929735905711214, 0.47840307326894055, 0.4837647035045905, 0.4821446281857509, 0.48095965963788395, 0.48492307989864836, 0.48475408752617155, 0.48636436227789975, 0.4858243855679888, 0.4843406689360778, 0.48629129257283843, 0.48965384334059286, 0.485889599364784, 0.4874669274433281, 0.47950520746903624, 0.4920746027778538, 0.48813855165035364, 0.4907591573279617, 0.48430024333094934, 0.49488922337252633, 0.48498468801824945, 0.4835641326799518, 0.4815400116692383, 0.495703602513665, 0.4853209559774976, 0.49175496101376265, 0.48692926495914657, 0.49213157099198795, 0.4836231663687975, 0.4883718514293332, 0.489053188571442, 0.48047356171848, 0.48856814487396183, 0.4891047059243548, 0.4916442914898599, 0.4878247269689241, 0.47920735312107465, 0.49432406760408737, 0.49015323251766374, 0.5048371472651407, 0.474093859262261, 0.4861765865372941, 0.4786186299101218, 0.49460355753526863, 0.5057140833972859, 0.4856037011255025, 0.47991916020173286, 0.4881913331419595, 0.47793972072476393, 0.49373090134881115, 0.48683164295207326, 0.4753564734987201, 0.4927091868672215, 0.4894914584316795, 0.48473207226402276, 0.49089568237264114, 0.4940750730212587, 0.48506641985565535, 0.5044774721440174, 0.48727548232731305, 0.4763328174514556, 0.5031031562811163, 0.4907702516534022, 0.4901918846618986, 0.4928918957880069, 0.48061653247232555, 0.48738011649816465, 0.4870816135534265, 0.47978426604566804, 0.4901998384666437, 0.48733012090678335, 0.48912791068118844, 0.49274242688348086, 0.5015135698029114, 0.4855953321037949, 0.49276415828290204, 0.4740378872007603, 0.5052160728383754, 0.49657974249739006, 0.48235437142521137, 0.4782681878032687, 0.4889250439855114, 0.49479104941254326, 0.48553689969022673, 0.4839635287612838, 0.4902494304221319, 0.4906202126143913, 0.49264179350853016, 0.49070820481220495, 0.4852730021827416, 0.48731702431914925, 0.4946662440161134, 0.4855843209176409, 0.4890305431953054, 0.49341380055317374, 0.48002334280161674, 0.4955389146645502, 0.4937487932437215, 0.49211911920098567, 0.4851901713603474, 0.48727748848312324, 0.4813679087101926, 0.48263660102756667, 0.4985500329268356, 0.4938970844400298, 0.49770486160911453, 0.4944363073080545, 0.5034135827801257, 0.4932732614124225, 0.4814974235190155, 0.4727310525321867, 0.4858116876709154, 0.48850918526701065, 0.4887406072244761, 0.49927118317687874, 0.4791851957671719, 0.488987635285229, 0.4806171519726022, 0.49547266153224606, 0.4912387327430221, 0.49199180208872173, 0.4910726781210037, 0.4879904565915411, 0.4889820660544856, 0.4911218286806243, 0.49790612600943085, 0.48925551333674305, 0.4983171664210597, 0.481557495335145, 0.4759182571992776, 0.4882495580292858, 0.4988020328413214, 0.4842039406663564, 0.49288830296595887, 0.494395887877697, 0.4915771086318549, 0.47895386677230944, 0.4787485818889479, 0.48232585891517765, 0.49085702431952327, 0.4879954788130629, 0.4938628010980695, 0.5045192349391252, 0.503017367906818, 0.5005318376045662, 0.48943194627384695, 0.48819987901396156, 0.4916790486323207, 0.4908349579552183, 0.49257133885549537, 0.5035968785251401, 0.48667176960212305, 0.5091175850972568, 0.4920229271910083, 0.4795617881922695, 0.48292392371216414, 0.484803578099105, 0.4910479606647257, 0.4897205892123577, 0.49799460084461183, 0.4946948636144608, 0.47952269392238894, 0.48884759215221, 0.4873022906177472, 0.49418435499444835, 0.4889680921158841, 0.5032944190446635, 0.49904579563913126, 0.4739836125412305, 0.48346488298968776, 0.4897421233930949, 0.48901017108317485, 0.4921414332193083, 0.4818597355294374, 0.4884538761063257, 0.4851036759489489, 0.4774034833751393, 0.4862082668463764, 0.48243627599466377, 0.48934353793431473, 0.4785595336224775, 0.4802785991811693, 0.48691528558267433, 0.4745461195528178, 0.484055243226971, 0.4974285305384594, 0.48849625953118814, 0.49195315031297154, 0.4832485905118327, 0.48541951914174225, 0.4856526791505423, 0.49138372222091703, 0.48714410521186347, 0.496325715643515, 0.4902002052283327, 0.5008674772750887, 0.4862185086113815, 0.4768111867401561, 0.4938887566932207, 0.48341445982249404, 0.4872685229542292, 0.4807790300975986, 0.48200297525567776, 0.4883453887943173, 0.48278892550976693, 0.4869248259487136, 0.4952825188730705, 0.48609844459539453, 0.49063343901065065, 0.48795984148190363, 0.49192664355931265, 0.49270529982270345, 0.4975217102966654, 0.49038275867339437, 0.47965241287630833, 0.48308699325227955, 0.483764411594993, 0.4894862851982496, 0.47798021900887716, 0.4967069088592081, 0.48033708026783783, 0.5042049750254852, 0.48708843603299595, 0.4854380629136243, 0.4964227328054731, 0.4925632595375918, 0.4835139259476109, 0.4845846294413121, 0.48888435727252294, 0.4947484734404563, 0.49170983177330607, 0.495052473731233, 0.4804125688178104, 0.4933264290370226, 0.4776590256850171, 0.4898225780498194, 0.49189258545246456, 0.49587315880040433, 0.49512707049681787, 0.48865203629382137, 0.5124054982753212, 0.49122652364372954, 0.4822076631364171, 0.49023010062185496, 0.4904887460915915, 0.4882432236659751, 0.48788212136707065, 0.48173229805644663, 0.4896608406344074, 0.49816884251158106, 0.4883624689397122, 0.4884772752810641, 0.4858164927302775, 0.4944341689257988, 0.4897899732044403, 0.48377128781224293, 0.49458138346809033, 0.4974676983342616, 0.4688526177596755, 0.49563448958698736, 0.4893556933757386, 0.484879163193347, 0.49997814082928027, 0.48388418695475816, 0.4896373670558978, 0.49135024298792684, 0.4884016210112679, 0.4766957761882185, 0.4738747988863089, 0.48830360105529175, 0.4871181451642381, 0.5035989219479414, 0.4853767156549524, 0.4744362637306696, 0.4992805481805264, 0.4878002613552847, 0.4831709575663857, 0.48749770690877303, 0.4862773364129268, 0.49955248805786695, 0.48082634144839864, 0.49887818813941315, 0.4892755580089041, 0.4816224100812006, 0.481280466152708, 0.484533271349247, 0.49592845588536766, 0.4895028331234074, 0.4961323901195542, 0.48756660924463235, 0.49583234111176555, 0.48708452363039545, 0.4817390841385914, 0.4979936365813277, 0.4886312295917688, 0.4856788236754601, 0.4830167362834764, 0.4934181213270836, 0.4790856213192493, 0.48803450062182663, 0.4762608316857906, 0.4852793321745598, 0.48536439094513406, 0.4715165248723321, 0.4809813471796837, 0.4909004996095555, 0.4904061003393568, 0.4926328377641582, 0.48878118852508157, 0.48786551133403805, 0.4728760634204471, 0.4754633760135669, 0.4944619564570273, 0.4895663206744632, 0.48105613956174254, 0.506607456468418, 0.4830708091089046, 0.48900815294408, 0.4781762059588913, 0.48691397163959177, 0.4990165752952415, 0.5032885703333307, 0.49058376694796185, 0.4875930526504186, 0.4722254417561312, 0.4913501880445832, 0.4972031164234322, 0.4817779774391846, 0.5019366088747258, 0.48958060238676665, 0.48086556968763544, 0.5001552493267397, 0.49071313875137396, 0.48373967518199085, 0.4793026776283465, 0.492179176156657, 0.4888184791739348, 0.49420773282894187, 0.49353867330241297, 0.49548524773894526, 0.48512186391476914, 0.48908291007178883, 0.4978754230258204, 0.48307601920825693, 0.4877176565543751, 0.49406843405400214, 0.49036864543561826, 0.49001719027613166, 0.4862109019289485, 0.49323591715452314, 0.48225191811691326, 0.4870093015674689, 0.4758471721466938, 0.4906032830763601, 0.4976492311621601, 0.4727667774908167, 0.4816898877656578, 0.486757953593344, 0.4815859467538784, 0.48681342877098854, 0.4969853839378831, 0.4891385966640939, 0.48720184809587314, 0.4834045052720057, 0.48254561287151887, 0.4962567179933405, 0.4969595845715161, 0.4995427095246808, 0.4987034555027563, 0.48019374405997883, 0.48391078516198, 0.49305594146571674, 0.4803214324454558, 0.4810828720373779, 0.492364515403322, 0.48920885159322086, 0.4888426281219097, 0.4959358604467882, 0.4902307988705766, 0.5001532009309011, 0.47749247383865884, 0.49193545237769915, 0.4845219205665971, 0.498557442532353, 0.4871364842178979, 0.48936901431341984, 0.4867049259857247, 0.48094658683731495, 0.49543881630996406, 0.4789144714878509, 0.47677570027290667, 0.4753117085817177, 0.5050942398339585, 0.4841924562246738, 0.4928549962795169, 0.49711167173262394, 0.4921434047140367, 0.4843523408353699, 0.4859062324555707, 0.4892090355026873, 0.4954409918227538, 0.49401076075835143, 0.4870140865967192, 0.4803404252914777, 0.4780731946471537, 0.47548127685973374, 0.48697288553946194, 0.48876737968313977, 0.47707481606197455, 0.4921447670027837, 0.49267578125713696, 0.4966167953183343, 0.4846758228338197, 0.49048685022726884, 0.48856802509358843, 0.4861210080494602, 0.4713132060501226, 0.5017702937023756, 0.4905370906802567, 0.48472087150006865, 0.4861377402271901, 0.4864410094414828, 0.4929276036942653, 0.5022077407522632, 0.49634992703717584, 0.4918141157232031, 0.486417480937049, 0.49683072733543215, 0.4866374976478033, 0.48155282577653663, 0.4726489179960729, 0.4896203807058144, 0.48944447067153507, 0.5018724548771575, 0.48197824243484827, 0.4906978395537277, 0.4814750800932453, 0.4865175054007283, 0.4844662986644581, 0.4787863186288192, 0.4864531878377403, 0.4958082654747534, 0.48999336797260407, 0.48427193480792163, 0.4789357959442806, 0.4919867298758888, 0.47921264611213804, 0.49152965237743834, 0.4950173924272072, 0.48643083800029635, 0.4941095076503758, 0.49364902875711647, 0.48816277743052117, 0.48382001808267283, 0.4775122077328593, 0.4840228435831101, 0.4883250751699385, 0.4851427897062991, 0.49057442690580055, 0.4848447297339696, 0.46894973731394335, 0.49375537858005136, 0.49022640870359546, 0.49756841918929934, 0.483113029604353, 0.4841037230819362, 0.4887204381796877, 0.4852577918153681, 0.4879728960074853, 0.48702895819653524, 0.4925420829196438, 0.48452063863737616, 0.4907470670366712, 0.5035176404188906, 0.49971892562601666, 0.47784993376069324, 0.4862604994838308, 0.4819866719137912, 0.48932995713859084, 0.48768546657525397, 0.4985668439342028, 0.49909150534885416, 0.4759148197355107, 0.484014981415552, 0.48892397969348284, 0.4844058006187001, 0.488850271802144, 0.48455396102854303, 0.4907391568735513, 0.4921274433001132, 0.48569194436025864, 0.4867317613188465, 0.47354703607193066, 0.495357612072354, 0.4770763493726955, 0.489449800604393, 0.4916324977352841, 0.4905362089003401, 0.48364533263350307, 0.4916656636191521, 0.4855503465122849, 0.4809545575600355, 0.48001158583164183, 0.5008052589301907, 0.48615575839620684, 0.4818011292531913, 0.49648016254548294, 0.4877324057222467, 0.49619979121530566, 0.4875463443631414, 0.49086351669630285, 0.49103479594813526, 0.4752835985288861, 0.5042903749908961, 0.4790704525562436, 0.4879140035540556, 0.4967940349271745, 0.4752564092972649, 0.4932831166755922, 0.49014659449254183, 0.49435776725555053, 0.49185689777284397, 0.4802957500589513, 0.490376494253289, 0.49062543242869355, 0.4822065882393352, 0.4890236914076484, 0.4886485659118946, 0.5049038489893231, 0.4795768512139038, 0.48702105829474657, 0.4925439917136623, 0.478944650214773, 0.4953573249828959, 0.48346594051704495, 0.4778687270574403, 0.4836852314588078, 0.4832822571480178, 0.49005714440894343, 0.48833624343231385, 0.48880175541824683, 0.4923785441326252, 0.48350630207103895, 0.49019673130753216, 0.49416185019302394, 0.4905753997985734, 0.49727383245820334, 0.4995348551899492, 0.490164976793641, 0.49638443532908777, 0.4841816961484977, 0.48930136487582165, 0.4879491938302709, 0.4886643106175189, 0.4904517119350222, 0.47945185356233466, 0.48373307444895225, 0.48224071267621865, 0.485850794165793, 0.4784701708768392, 0.49019100154570105, 0.49833649813669195, 0.48417634927823, 0.49156554879682507, 0.4848436883162394, 0.48286603407208856, 0.49535784788445375, 0.4759241050263549, 0.4904225601789567, 0.4837067363814309, 0.49191553329744026, 0.493777363583104, 0.483752940026837, 0.4865159871778642, 0.46785707948623867, 0.48046947958480657, 0.4879990202100696, 0.47519072402749607, 0.48495618165021925, 0.4897902336720676, 0.4883196934281512, 0.48485036581444013, 0.48675313383854196, 0.48406120726768764, 0.49476316302191636, 0.49778354680932024, 0.49004105684239, 0.4838715832341706, 0.4693331446585917, 0.48504355988890807, 0.4819633709705859, 0.49004391601475583, 0.4681231981424064, 0.4777989648191871, 0.49467091696012766, 0.48584174318891166, 0.48784055296055806, 0.48363165746175313, 0.5038245236821917, 0.49062311544372444, 0.4761110203371913, 0.48616102727778593, 0.48382361468846247, 0.49715898312270396, 0.5017692129597725, 0.48026985793015864, 0.4863662459423365, 0.4935238499726188, 0.4795234537725066, 0.4851508660006664, 0.4856211409550703, 0.4928690934898555, 0.49044297323916836, 0.4823802960058557, 0.49709329963476406, 0.4937351034067488, 0.4947241556907742, 0.4883062845289598, 0.49141137597909834, 0.4915185641301013, 0.48117000910584634, 0.4825965550618778, 0.48777288524924994, 0.5046951288085426, 0.4990950941802505, 0.48775314188094665, 0.48285207836019844, 0.49102689701132196, 0.47667503523537635, 0.48468195363497507, 0.47410024437463966, 0.4897825667752878, 0.4877018637242829, 0.4825943376780916, 0.49259439089985907, 0.49896732673778743, 0.47556368650793246, 0.4767663336873294, 0.49589143591483076, 0.4894837705663906, 0.4822669059759213, 0.47303670296698247, 0.49474169559997844, 0.48036844613033686, 0.49353608136111116, 0.49212605077041527, 0.47588658184641713, 0.48987703409266636, 0.4909172830627536, 0.48432258424775876, 0.4954499958196084, 0.4964040892447909, 0.48446369897298536, 0.4858457480340869, 0.4838821637643268, 0.48720164473700056, 0.49781171158838516, 0.48745158666977834, 0.4973945794880609, 0.5017240956701299, 0.48663542402480775, 0.4776107035216055, 0.49202050485925747, 0.49151411514000215, 0.4763917860676704, 0.4832655079669563, 0.4870640208517282, 0.5089773567826462, 0.48593571123856444, 0.4910463929465337, 0.48244746254944276, 0.4876630416742056, 0.4948728013403043, 0.48420920625707353, 0.5031579053154016, 0.47653814003467415, 0.4886587259619342, 0.49665402717019835, 0.4821222874794902, 0.4902169131275292, 0.48461056785903045, 0.49859355166289715, 0.4744422338458949, 0.4955033446922188, 0.47675674414044666, 0.4844329456024926, 0.4756970157403447, 0.49332512366441966, 0.48706652782164295, 0.4857347247950747, 0.49421073802905785, 0.4863920432920003, 0.49204536841774327, 0.49354506094996525, 0.4879094347157043, 0.4887683966122498, 0.4858510911229223, 0.4776200790429729, 0.48523104053893856, 0.48119048626683375, 0.48732393003997515, 0.4818282538386464, 0.472258457629719, 0.4870905129084479, 0.4826197870675817, 0.4838673441940929, 0.48599772603755065, 0.48080848052309755, 0.49294924547071184, 0.49768272901098876, 0.47545268194624873, 0.4918737029249993, 0.49402973171374065, 0.4787438996202993, 0.5002213099629688, 0.4945717507154167, 0.49215503605663924, 0.482058274308465, 0.49010660659851724, 0.4812409457277085, 0.4860181582813516, 0.4844896922827021, 0.4833282778789775, 0.4863841521645958, 0.4784788520090279, 0.4842331109599884, 0.4762176292461561, 0.4988047726534276, 0.4859675639762086, 0.48239907660067116, 0.4879013887983994, 0.49592537548866666, 0.48746510591628583, 0.47420394185361603, 0.4823181986799673, 0.49133018536739204, 0.5089480138723195, 0.4831370870264546, 0.4845143514992531, 0.489194714660621, 0.49222738703473207, 0.4762416149137421, 0.495498174008405, 0.491802819020379, 0.497706556144863, 0.495216903282453, 0.4834494383352637, 0.49223261696723686, 0.49391550758897673, 0.5074171550215838, 0.4871821414380575, 0.48583568295902063, 0.4816088578352138, 0.5022997661823061, 0.4923003655264152, 0.4963338165884164, 0.49670624777003636, 0.4807440898085618, 0.49452139701389636, 0.4975236111452482, 0.4911549619556304, 0.476693592349977, 0.47778696942840415, 0.5045282364673191, 0.4972254404705396, 0.4914864208783248, 0.4886026680921153, 0.48084396391955647, 0.49030929602518536, 0.4932865593583025, 0.4898403824102348, 0.49072709722533236, 0.49332304003560523, 0.49390942798051113, 0.4668664471593661, 0.4703145311871331, 0.48706705315647947, 0.49296999293417193, 0.4818269358941331, 0.48490111596191693, 0.4931529452756698, 0.48922418842987964, 0.4967082674030342, 0.48672575974157223, 0.496309054792797, 0.48837199315845264, 0.4903154572794538, 0.49200252576690917, 0.4936410576689451, 0.5070623591897228, 0.4934429260864385, 0.47835210076013335, 0.4858185579499629, 0.4917338837839713, 0.48214508541376183, 0.48556377911777115, 0.4888578005363818, 0.49066772076455334, 0.48541339399365885, 0.4843554209403163, 0.48476833071059117, 0.4851113684569875, 0.47922619710211556, 0.49164192249011246, 0.48632970228459754, 0.4877163359369143, 0.49936130301333465, 0.4891577854779457, 0.4945223313334685, 0.4908714551811425, 0.48983356733501876, 0.4878226821096027, 0.5032959207098864, 0.49718026636661067, 0.4803956901122555, 0.4791831267167107, 0.48581947629671185, 0.48800742685518117, 0.4846260459116055, 0.47978618469855616, 0.4884502562507379, 0.49273099733037834, 0.4898932669508454, 0.48210097380531497, 0.4922760847874009, 0.4713945515501349, 0.4866035767355475, 0.5046845534973091, 0.4837641073202368, 0.4806555269786793, 0.4841910348514092, 0.4875163032201321, 0.48180008673597136, 0.4891700952449407, 0.4992076841752938, 0.49347829196901466, 0.48436539752933133, 0.481536518441248, 0.49399323388768474, 0.4960788296251629, 0.485093547588184, 0.4737450439340402, 0.4889280589340125, 0.4928207080137919, 0.4874674602293751, 0.49638165004358975, 0.4878565144049912, 0.49292596534040956, 0.4911873076308774, 0.4852837424194949, 0.4790308415707139, 0.5003655964302606, 0.47506191275521176, 0.4863666981815841, 0.48007102916452643, 0.5042120902799045, 0.48454027509033126, 0.49088116811844784, 0.49668409234335165, 0.49417852288063296, 0.4782161655893524, 0.483696273011565, 0.49206058950553794, 0.48288269881663576, 0.4922656801029143, 0.4928204003878326, 0.4774681940250406, 0.4908672252191689, 0.4802634264701087, 0.4902202830481632, 0.49021071850289993, 0.4931533840780098, 0.49252622616603975, 0.4918282966184301, 0.4842465009116919, 0.48232003218555664, 0.49070010631943983, 0.4900351635471905, 0.4923516129113194, 0.4935486691205959, 0.49311525503905, 0.4936119487540748, 0.47931824273617335, 0.4929651301827059, 0.4988419134573712, 0.4784320597956528, 0.48654813478345127, 0.49467950188919163, 0.4840444259530853, 0.47017801608397286, 0.47631779747084685, 0.4907152618992386, 0.48929674395225875, 0.4995283171374913, 0.48644322049089533, 0.4877100432723575, 0.5051014486643917, 0.48221929002047587, 0.4823745683610804, 0.4870332761621237, 0.4911686239738338, 0.49971158314820935, 0.493083432289916, 0.4987045865758, 0.4825235383586748, 0.4858888634421733, 0.4893091675766441, 0.481589171300488, 0.49563032010232855, 0.4918915526361736, 0.49086387177371177, 0.48788553047192695, 0.48877774257864604, 0.4852727098754009, 0.500604747974762, 0.4948111141673669, 0.48074432022744296, 0.48594880271092084, 0.48511207998442957, 0.48536603436851805, 0.49060320874083246, 0.49795676774958264, 0.4879012625371774, 0.4876059098442923, 0.494931558630838, 0.4803900707385454, 0.484883042749104, 0.483577878398351, 0.4947711350800312, 0.4869141657119984, 0.48901038670928965, 0.49011299890263604, 0.48258765777523976, 0.49875043164088173, 0.49572135629833425, 0.4871701963031198, 0.47897954676407, 0.4748488453716707, 0.48946058820744454, 0.4824092718439347, 0.5004043971419899, 0.4836574646452962, 0.49352694044952805, 0.4901815451750519, 0.4950984627322301, 0.48694017502469134, 0.4788625986620496, 0.4898863502582098, 0.48893438281872187, 0.47443207029045614, 0.4990997067824419, 0.4818134128412354, 0.4883091085139616, 0.48314023091728914, 0.4989194380150181, 0.4886366877296479, 0.48266262277287725, 0.47937172320385035, 0.5014964622553895, 0.4805245689714533, 0.48624418133146763, 0.48575677226974884, 0.48527586250492, 0.49668026085541617, 0.4961961754560599, 0.4877411043220608, 0.47888058955468743, 0.48788997203979856, 0.48187887985561945, 0.4745654818727096, 0.48917049002151225, 0.4921768070070446, 0.48868584265941917, 0.469645014469995, 0.4874138023145189, 0.49220204975239673, 0.49377192641671164, 0.48740131062075265, 0.48035119357680134, 0.4828933663169881, 0.4863658629386143, 0.48561283882669215, 0.4918923181133058, 0.4839474399186688, 0.4766390356732104, 0.4927291512764576, 0.4783129967224674, 0.4780512425574664, 0.48918315164220044, 0.49145958319534844, 0.4891958421996502, 0.4697650096137774, 0.49485785369545515, 0.4847967616533529, 0.4878963303676261, 0.47543555195248516, 0.4874631125746871, 0.4911084350201659, 0.4904641020189053, 0.47628091562745767, 0.48259026429777435, 0.4869393204301502, 0.49715230373298047, 0.4731189994026477, 0.48648689704535847, 0.49980614032991866, 0.49031081269559995, 0.48341007633983224, 0.491760802486429, 0.4842576375544859, 0.48638140232271776, 0.48894368244459324, 0.48690938561373703, 0.4868417199202914, 0.49193285747769044, 0.4914111834074212, 0.48459013483024294, 0.48930756946539655, 0.4933715242703455, 0.48059192566530623, 0.49647193897333836, 0.4827115874733196, 0.48951484466277906, 0.4842390399099517, 0.4918181787374876, 0.48606199666355565, 0.486882537042633, 0.49602397652026364, 0.4965468321988703, 0.48687597467203925, 0.495599632115325, 0.4920628636427917, 0.48264909404392337, 0.48927447431332105, 0.49789092369700544, 0.49087320112222255, 0.48837557859991687, 0.4918863747398671, 0.4957858216321097, 0.48729799744950236, 0.4760041099842839, 0.48870665791093654, 0.48942533979885844, 0.4922735610920169, 0.4842966915648312, 0.4942874322256666, 0.49131343381094217, 0.49040022559580215, 0.49076054894113047, 0.49524124003075654, 0.47878064927547154, 0.48952256380296594, 0.4877868565468772, 0.4927922251398501, 0.4806258183750511, 0.4919301055546658, 0.4815830760240045, 0.47998269655034526, 0.48235501518820595, 0.4808572400160605, 0.47123075994326824, 0.4962344577577129, 0.4815157065689809, 0.4967352560345156, 0.490782680085976, 0.48279934851880896, 0.49096160935657973, 0.4850639988225389, 0.4724529087995673, 0.48586963603423167, 0.494884445365996, 0.48030937857329026, 0.48686540478473533, 0.4918585993390712, 0.48595803457172226, 0.4852708765964765, 0.4958667958295403, 0.490733814078137, 0.48132825121462874, 0.47832393579566473, 0.496980747032924, 0.4905025289552879, 0.4769605049805337, 0.48569864428799514, 0.49241691972035706, 0.470698149154035, 0.49638123544885676, 0.48431910199144484, 0.48872193542596304, 0.48613834046800314, 0.4857993405679046, 0.4839656709905059, 0.488390051913804, 0.49999510807973996, 0.4799924891291908, 0.49844379618427415, 0.4803813208821585, 0.49135891624413675, 0.4856909024318144, 0.4806351604599359, 0.4839193588681155, 0.48436792007297375, 0.49749626988970197, 0.4856175876535029, 0.49399923705893295, 0.4910289804516007, 0.5013675582881938, 0.4928560000274791, 0.4944404174505554, 0.4980459115068947, 0.48289441791193927, 0.4882188802865008, 0.5014218381217748, 0.4799841589446831, 0.4855264132550093, 0.48661098277522963, 0.4924308962611182, 0.48939694820843516, 0.47849109940781837, 0.4899694425512276, 0.4772036740334299, 0.48142093959117593, 0.48621713212904744, 0.48352491808359177, 0.49632433875950693, 0.491295677512695, 0.5025141157066394, 0.4936663330553302, 0.4910456734789103, 0.486445451116343, 0.5046920048142104, 0.4869682626344584, 0.4802228288849262, 0.4940190244854201, 0.49894145236707393, 0.4949215593717353, 0.4958479856022468, 0.4869444303475674, 0.4874229138986353, 0.4991195242523961, 0.4889767619040893, 0.4900832088346961, 0.47408822413190366, 0.5025525883374198, 0.47829806852732576, 0.48991645062995953, 0.4984693403733006, 0.4856603867325509, 0.4893616451240912, 0.4869019960202933, 0.4820242033295767, 0.4932391788046379, 0.48924593408546146, 0.49238629218596014, 0.4747525589713591, 0.4854280907741754, 0.48600445387032504, 0.49080484180083844, 0.49190154634881206, 0.4928294006289779, 0.5011395470974834, 0.48297011948586266, 0.4930833726413858, 0.48303695308096944, 0.46575941376910773, 0.48887070540643834, 0.4964397143783951, 0.4920518290033344, 0.48568943382187474, 0.49052396111209046, 0.4765332938011687, 0.4912861271076185, 0.47747266729882126, 0.4836097337619337, 0.48832122225150226, 0.48747839716652486, 0.4912001058435265, 0.4781373132026676, 0.48372491727588823, 0.4828274752572283, 0.494230461914477, 0.4967493361577395, 0.4964623877800666, 0.5042926740459708, 0.48423211694253704, 0.49554397147395135, 0.4906643798214636, 0.48995348294233093, 0.47834381509847235, 0.48183216818677654, 0.4766828566515368, 0.48678853001127315, 0.4856991988307157, 0.4941974923579436, 0.4865519650669643, 0.4782061013733917, 0.48099033634084387, 0.48309526751969384, 0.4792842312814201, 0.482506486032462, 0.473249319515846, 0.48029654580212583, 0.4781102746645585, 0.48665915416804256, 0.48222500692582904, 0.48664519796719863, 0.4796457938740227, 0.47953944538210036, 0.48451362343568405, 0.4902400280141965, 0.47709774998468746, 0.4808189241248605, 0.48842010635421884, 0.47891364683953497, 0.48707844841278625, 0.47442591163162706, 0.49389534416037617, 0.48817100802721225, 0.4787718145923707, 0.481624635468833, 0.4887589273223534, 0.4913939147946365, 0.4917543370097245, 0.48137941270695617, 0.48087057739614564, 0.4895971889942015, 0.4961254116815609, 0.48349159994693425, 0.486363896203281, 0.48288612900260286, 0.47280665027751717, 0.48028609203600076, 0.49006365051239414, 0.5013699276444765, 0.479114993519374, 0.485965511122776, 0.4684291167523786, 0.4880216614998269, 0.48702596947129934, 0.48062407259733136, 0.4887930304461536, 0.4904031660194865, 0.48477555077621254, 0.4823805459093692, 0.48541609570725375, 0.49293283521362197, 0.49091663203592184, 0.49065717940858344, 0.49630656799252404, 0.48783807843963745, 0.4815398306584196, 0.49431839722823556, 0.4871556477035032, 0.4832655313396609, 0.4815519352059459, 0.4886062879883769, 0.4836045863551296, 0.49237417931190364, 0.495234737776154, 0.4977200211199199, 0.4800047895712624, 0.4811062450160015, 0.4944669653806534, 0.48077793798600965, 0.4856990637917262, 0.4939626350447329, 0.48965532004105583, 0.4958473370298272, 0.4891969603338889, 0.4792521586031994, 0.48330265135028433, 0.4861847332547461, 0.49354116930658615, 0.49750002980153346, 0.4950779924002605, 0.4870912466478104, 0.5016123947963772, 0.484779753247939, 0.4776296220469609, 0.4916523849562742, 0.493514267536409, 0.4880273448904178, 0.5001199594420571, 0.5019440227543368, 0.4797078086594015, 0.49642513616045153, 0.48258540258903393, 0.4859447832687121, 0.49208372890841173, 0.47268782696167394, 0.4949727543792979, 0.48687319457954414, 0.48013597850084366, 0.4918571550356906, 0.501397256791265, 0.477837303275357, 0.4893931679599199, 0.4929701907636584, 0.49267242910736003, 0.48151114594735617, 0.49523060720173284, 0.5006390107087574, 0.49684820646901084, 0.4829852357746643, 0.48308126295531506, 0.49277888301147393, 0.4927716749525113, 0.4882236202335722, 0.4901947120381405, 0.49988382941065373, 0.48645924976835153, 0.48841236789310327, 0.48399924404813943, 0.4934493667187203, 0.48930731154479856, 0.4843510223590434, 0.4958101107866717, 0.4858175878509152, 0.49731674006637405, 0.48770752796050854, 0.49435951737035283, 0.47915387959477357, 0.4897536883010107, 0.49602799550128907, 0.5006205374932514, 0.4793260668501631, 0.48984842570773274, 0.48050558902023055, 0.4880478122369581, 0.4885594722065253, 0.49021467021119947, 0.4855958356159861, 0.499117238730678, 0.4844871737508328, 0.477386276416152, 0.47582643995168944, 0.4841965664492666, 0.4757737095236791, 0.4948684795015061, 0.4865979592099369, 0.49182563629267606, 0.4927534726467946, 0.47729737039185716, 0.48533820685718293, 0.5008318027279337, 0.48529625187125747, 0.4638528826756837, 0.46953462564616316, 0.4876109428065789, 0.4897056339369602, 0.49619812837006166, 0.4968226103342326, 0.48335632890578256, 0.48591278533885807, 0.5044725275794945, 0.48773088387623936, 0.4905704761255587, 0.49232200271701343, 0.48164034815004697, 0.48794048171000787, 0.4839695912275151, 0.4807838038585582, 0.4875966554460785, 0.4917050659812544, 0.5020162478255322, 0.4782627435145159, 0.48689825066381315, 0.49102544638580564, 0.4848723156902174, 0.4665074729913685, 0.486280988465415, 0.49141584556440737, 0.49367383500807877, 0.5031790287784603, 0.47502339376004854, 0.4840611469440432, 0.47746678635721584, 0.49758423660282225, 0.48366844875134984, 0.4968717209300244, 0.47995828776452415, 0.48434994043393453, 0.4770123498882289, 0.4784208374589424, 0.48293616950307444, 0.4863317767156665, 0.4858973858346325, 0.49149238603674666, 0.5049101319102365, 0.4883063220084811, 0.48987877831471155, 0.5000066489345814, 0.49421094743081384, 0.47761043639000483, 0.48966612328765574, 0.488826866347924, 0.4862703019651799, 0.48548020832694155, 0.5043125035948945, 0.49216677126603375, 0.48152618370177164, 0.49132112715083204, 0.4942217957421401, 0.49048730857547146, 0.4995497472826959, 0.49000557739590717, 0.4896816919451144, 0.49161415056606683, 0.48669736675244163, 0.48660535372606095, 0.4884980733145586, 0.4827518963084281, 0.4847575171342571, 0.4927703428321861, 0.48492783741764484, 0.48164417395369885, 0.4939216410786179, 0.47755398138254446, 0.4938607770235364, 0.48567381377557234, 0.49185420285431736, 0.4903494057841897, 0.48541758428073295, 0.4944094217323725, 0.4890792861898634, 0.4770474520082013, 0.5020053332322845, 0.4889227011061279, 0.501326697863064, 0.48998418879797967, 0.48169724727610197, 0.48343623331241925, 0.4855096503994826, 0.4875182195836911, 0.4823788721265598, 0.49291914825683264, 0.48337627549500806, 0.4830933476335819, 0.4730770777428139, 0.495967632068834, 0.4886398743300227, 0.48642340639065884, 0.48600420560406354, 0.4899062338906807, 0.4877327828270544, 0.49328531103496875, 0.49203098399585243, 0.48946659793884195, 0.4931562555282183, 0.4880191708990485, 0.4925989097772057, 0.48858215717557635, 0.4875038325260642, 0.4728197927638303, 0.48180026870257203, 0.49094268277756614, 0.4856238821748451, 0.4802416391742305, 0.48806218572029586, 0.48983656604246056, 0.4961184167655795, 0.48838104429949347, 0.48506715597753575, 0.48578404731644026, 0.4891792843850005, 0.4909327670842406, 0.4832600145494494, 0.47333431114298347, 0.4966744844048701, 0.48276728593183127, 0.4938522262471253, 0.4713555037538995, 0.46721864363964777, 0.4833088850454157, 0.4863809663650129, 0.48244082442067576, 0.49510716690503015, 0.48356481733762574, 0.49138381792924596, 0.49150760191726467, 0.4831097310545853, 0.49292051393771746, 0.48706264783281655, 0.4981002464935082, 0.49009510424905883, 0.49418296842869974, 0.5014617556549295, 0.4931394979463882, 0.48575583707696524, 0.48773673254224925, 0.4867238974765602, 0.4941424322096413, 0.48078145476176426, 0.4865054846510868, 0.4920063521118993, 0.49802471955146266, 0.48739815062768904, 0.4848738246016962, 0.4794550316711634, 0.48402996604043047, 0.48805200454159997, 0.4861369045857768, 0.48976602218035414, 0.478628072245661, 0.48237972663459383, 0.4803130586441218, 0.4885923989314528, 0.49034617580353923, 0.4945807065097984, 0.48505288918172473, 0.4828724260449079, 0.48336262587824597, 0.48683586900917464, 0.5018416120975696, 0.4967227349588931, 0.4911838613396572, 0.47761041073722715, 0.4898975855684829, 0.4795057939706494, 0.4969073150763324, 0.4811464996866129, 0.47806923139866325, 0.49085924483873555, 0.4879359837404578, 0.5001545564366504, 0.4911979546849556, 0.4802697365841226, 0.4862387329439671, 0.4817365393005864, 0.49252861691774635, 0.48621101604185774, 0.49128401954257844, 0.48060649123430504, 0.49007267394971765, 0.4763070659389598, 0.48971972196104446, 0.4842457850242543, 0.49592981058466784, 0.4986287307256095, 0.4978564473521745, 0.4965928034604316, 0.47485307220260237, 0.4753402973823243, 0.4837968019368587, 0.49499714913228055, 0.49122975347660314, 0.475756255054603, 0.4982532144768362, 0.48444520271064323, 0.49018368869725554, 0.4846613742615458, 0.48784816786408647, 0.4874875977406776, 0.4868007419838052, 0.4872293290496694, 0.4874493335491504, 0.4970871771983829, 0.501392588237322, 0.4940954138403075, 0.49057713877513087, 0.4797725906037108, 0.4896824220518857, 0.47604775713149833, 0.48848442754776744, 0.48082724029861784, 0.48423905515638455, 0.48797474766338383, 0.4875935661790986, 0.47193416765221796, 0.47515995258905247, 0.49394612954928263, 0.49169468979441383, 0.4867869085208238, 0.4859370766370397, 0.48112373083393983, 0.490445095142993, 0.4857284823073047, 0.482931879138724, 0.47679501774633803, 0.4760600423592294, 0.48827460355879987, 0.48831981613565995, 0.4924937902315961, 0.47809601984485145, 0.4986572998220922, 0.471070326885847, 0.488089521618598, 0.489456719992349, 0.4949243492851955, 0.49406018826491727, 0.49119247893464335, 0.48241936694802817, 0.49467191933269494, 0.47848603302393916, 0.49671130086312276, 0.49768123940936265, 0.48510118817191855, 0.4854080739912467, 0.4846346616944884, 0.4843722691350995, 0.4875746518400025, 0.4896807957339253, 0.48632642886883215, 0.4973175382511876, 0.48404697371695243, 0.4754768804341688, 0.4946194410980275, 0.4793632949200816, 0.4964263340541904, 0.49292459988269227, 0.48541132922701913, 0.4842740488535699, 0.49789013776662794, 0.48600795331167174, 0.4857263627367549, 0.4911440922476917, 0.48819756257120356, 0.494902924358328, 0.479340577336836, 0.4840999295714881, 0.4826980263229388, 0.49163424622634416, 0.49669710058803157, 0.49638091916609656, 0.4959399573737092, 0.48093930628582227, 0.48871391800670744, 0.479167472282282, 0.49517015139679854, 0.4811848595690914, 0.48692617097518054, 0.49541118834671594, 0.4770803097992591, 0.4961372539729374, 0.48896560653439325, 0.49858379731363145, 0.48466962424786086, 0.4836838285327543, 0.48271965693535435, 0.4909071582454952, 0.4802428384623378, 0.4886859591106241, 0.4862136281371138, 0.4888137289014, 0.49550530967084555, 0.4876657848186701, 0.49452628938786347, 0.49254358724847747, 0.50041854979871, 0.48427522083526403, 0.49383227953195313, 0.4757440707572199, 0.48141014931333537, 0.5001991756534672, 0.48902828944382676, 0.486010045977275, 0.4906512702672583, 0.4805575832124106, 0.4832025375217005, 0.5020269584743945, 0.4726951991490577, 0.4761414980943236, 0.4803637152293045, 0.49444030354738844, 0.4884993128417756, 0.4981290620928556, 0.4855916770229925, 0.49638760726602127, 0.48472783904532346, 0.49520694732049053, 0.4905625646638614, 0.49209584833215464, 0.49596843044930133, 0.4672380491934647, 0.4890840239224356, 0.49592144856289655, 0.4942519291691688, 0.4909997063673689, 0.48579576887777826, 0.47528008545186523, 0.49362283866283607, 0.499421099649356, 0.48322640683174456, 0.47816268981227467, 0.4885116415562131, 0.48757072134489254, 0.48189818044587296, 0.48534370663698734, 0.4923225705174614, 0.48437024406146967, 0.49948985425839143, 0.4748853342360028, 0.4762469649024631, 0.48707857005804733, 0.4964401059058788, 0.48042447467741817, 0.4755924149188089, 0.47711354188247107, 0.49688054060766407, 0.4943962739926658, 0.4854954321696275, 0.4861110945920975, 0.49453004278407625, 0.4759518009955228, 0.4875428268021733, 0.4986387881407198, 0.48614391033091586, 0.49848313032680763, 0.48518416740230336, 0.4855354378986839, 0.4713810975054115, 0.4907876660404469, 0.48644136688361134, 0.49584250282253184, 0.4938608937200949, 0.4892712259024406, 0.4815402528809493, 0.49260292489440016, 0.4833784494290296, 0.48031629617280286, 0.4882938009708821, 0.4817297699441821, 0.4895210057316586, 0.4907293207637488, 0.4888895232537519, 0.4954528608041791, 0.4920370587655028, 0.4859346596651859, 0.4820804810137433, 0.49544001045788616, 0.4951610160811052, 0.5011523416739755, 0.4845669767155937, 0.48111178822579004, 0.4807554014076044, 0.4934006507866513, 0.4898112398995359, 0.4815200363727627, 0.4994079978656849, 0.4808142677341239, 0.48375628564224227, 0.4842649625017182, 0.5009357674610119, 0.47336889536990046, 0.486041471582834, 0.4876275299099688, 0.49369497800554574, 0.49128654107612035, 0.47634278901388444, 0.49250931824498934, 0.49577238061010664, 0.48803936364621775, 0.46933089538217476, 0.4839210823361088, 0.4976077182351567, 0.4978495474391632, 0.48792030632911315, 0.48702032378625415, 0.4915033246848356, 0.47994390565681644, 0.48795778244826515, 0.4940013544044707, 0.48673366218586156, 0.4976718603989757, 0.5084132715105881, 0.4871561682022687, 0.4795885951479602, 0.49568456748501827, 0.4778635595585703, 0.48845211192173604, 0.49858169138755865, 0.4881366150193251, 0.4842521957601417, 0.49907696485200964, 0.49835289831555274, 0.4933888064676583, 0.4800301752849065, 0.4818704507125062, 0.49258631848452405, 0.49531607367743014, 0.4936320979119789, 0.47998020561553273, 0.48046954941612013, 0.49850679261293895, 0.4860582054752499, 0.48157556631011006, 0.492047345312271, 0.49485128599996875, 0.5017663121339463, 0.5025374150794701, 0.4888081794357986, 0.4937951486077859, 0.4898546693966065, 0.48644198632674907, 0.49402556235016537, 0.4910862138261391, 0.48536663026107024, 0.4921741481555711, 0.4780664301763361, 0.48657982609575134, 0.49917149114893083, 0.4720585668858594, 0.4787034787859444, 0.49317984335651016, 0.49915772145935045, 0.48471156859403475, 0.48627365239919185, 0.48699069493425684, 0.48346024907907115, 0.4816649963701712, 0.4733887186069271, 0.49003281604362026, 0.4949160785106555, 0.4827063710659458, 0.4807549480314975, 0.49568412197163936, 0.5015018041725501, 0.49958461704624363, 0.48187738271977565, 0.48943719719579576, 0.492451013627166, 0.48160194115169824, 0.4895161159821664, 0.4947378857153154, 0.4886263273903517, 0.49160486424250643, 0.478826659014545, 0.49608978101356227, 0.49659880688605773, 0.4915805326394444, 0.4945820905790544, 0.4828879513407664, 0.48755163177810495, 0.47789993199443864, 0.491095157507924, 0.4865395826640911, 0.4896043574350734, 0.4981453559050877, 0.4912632746352213, 0.49039543743440617, 0.48771736569684676, 0.4842654239482392, 0.4841647343442421, 0.48317344744694474, 0.48292519392175925, 0.48204398187755415, 0.4995179294564618, 0.48923497234933017, 0.48275069062261694, 0.49037110719105, 0.4833792349373121, 0.48714833556976206, 0.4759358557343029, 0.4843271553581842, 0.47818023695248063, 0.49045306637893504, 0.48329862119618683, 0.4777600489467918, 0.4743402744125249, 0.47637924019647077, 0.501880317857602, 0.48150454078006727, 0.4899237066327234, 0.48381795357141855, 0.48553213712000537, 0.48935356797869545, 0.5006220439377431, 0.4912483573077772, 0.4857453768641882, 0.4867480942007008, 0.4920901327128254, 0.5037225249160573, 0.48836845673381235, 0.4814048169279816, 0.49693260336837203, 0.48585491325078717, 0.485801986429624, 0.4790519823358392, 0.4973386998013805, 0.4893586340339462, 0.491658323001702, 0.4903427874613456, 0.4997226014440103, 0.5024565506158063, 0.48718280667670466, 0.4836138347439324, 0.46815829402683273, 0.485596284386615, 0.49555779114065235, 0.4781848850001369, 0.49442903676164224, 0.4797451212951395, 0.49422399436829084, 0.48696698343797107, 0.4925030483459951, 0.49290651838188865, 0.4849428681144925, 0.48358839171499773, 0.48704927592351926, 0.48071658006943824, 0.49196190872010165, 0.4813416672224947, 0.4888208402243764, 0.4938001966722443, 0.482732933424891, 0.49344137439131974, 0.4920291086683042, 0.4797778202632259, 0.4997225380611537, 0.4903948083561512, 0.4978929092576154, 0.4919589490106048, 0.48498264451122014, 0.47960730520455014, 0.4758142168006403, 0.4952369921790707, 0.4800169088259627, 0.4874097310723239, 0.47864393057939614, 0.475203061082671, 0.4847402999325744, 0.49155992118312036, 0.473432818979438, 0.48799792948274584, 0.48954112907776537, 0.48987085180461887, 0.48878358061155613, 0.4811785757230039, 0.4791096807382264, 0.4860755817366576, 0.48669180146924423, 0.4791030548976127, 0.4981331542412086, 0.4892685278523227, 0.4950625089530713, 0.48508040691382787, 0.49482038279628004, 0.483331747497151, 0.49594778613278695, 0.4826544879885179, 0.4835910545672179, 0.492986024477374, 0.484255557855945, 0.4798565337594088, 0.4978257967714628, 0.49370837626399244, 0.48950155445660815, 0.48110313475950556, 0.4937782757363105, 0.4850103899103834, 0.49434823275395806, 0.48582055158640153, 0.4877053831073705, 0.4765080490103541, 0.4993633860872748, 0.4881895298401061, 0.4975384868486557, 0.4916165161242802, 0.49238154696562864, 0.4917054144240277, 0.49872137962269186, 0.49863374156629725, 0.4686853054485124, 0.4760232375910841, 0.4913547523888987, 0.4894573465091201, 0.4965490507465647, 0.48673428688129783, 0.48266151586915945, 0.47842790142730535, 0.4895809588533224, 0.4901410824485806, 0.4781624190634135, 0.5010547372169043, 0.4844315484054103, 0.4835007672156054, 0.49224129968056324, 0.4863629401799257, 0.48396660381780815, 0.5035867430627704, 0.4918714253781777, 0.48221980896090555, 0.5026938698563002, 0.4931938757009472, 0.48853627624591145, 0.48821571430402144, 0.4790330134367583, 0.48092529262786216, 0.48700240004666784, 0.49541597645904134, 0.4980296629361478, 0.4822740309048816, 0.48937570598328034, 0.4867876322169596, 0.48471751298983146, 0.48730804481772194, 0.48966224997481045, 0.4848529182995523, 0.4898932264259502, 0.5027348934271952, 0.48145571044332647, 0.4856054357666246, 0.4903729712133188, 0.48332226394637473, 0.48283403221212545, 0.49263384612163946, 0.4779311317274623, 0.4817184424289171, 0.4897236351732354, 0.49361805168436684, 0.4920304452817648, 0.5034742877218419, 0.48118963941622167, 0.4867039787155837, 0.49344438441421157, 0.489531723054574, 0.4767824817902731, 0.47673729990648184, 0.49456788333242624, 0.4854865973575315, 0.49117211359287244, 0.4758752922651369, 0.48536755711686996, 0.4869307012461315, 0.49236903086381434, 0.48924633697169995, 0.4946387868665952, 0.4852285982264664, 0.4835426102931314, 0.4836327181250631, 0.49430204752825885, 0.48708133635196343, 0.4794657417473617, 0.47685445735389714, 0.4797879216812077, 0.4931440424341213, 0.49049733590891914, 0.49918369860407485, 0.4795450180383208, 0.4903523149237579, 0.4877902160149843, 0.4922974136478941, 0.4814761854455538, 0.4877092778643326, 0.49028376586519673, 0.49192019053810926, 0.4898030635035888, 0.4899833744144613, 0.4785644374381018, 0.4844004528913176, 0.4893423655276464, 0.4904992197772106, 0.4904936062840486, 0.4891739745473969, 0.492702152380291, 0.49791844364156357, 0.48549600779320734, 0.47936367436838967, 0.4847080083168762, 0.4910539934210699, 0.49501781903172076, 0.4901315230817609, 0.4851631300344791, 0.48922172600558306, 0.5074465368527714, 0.4796824390377222, 0.47587645041944254, 0.48829386054774826, 0.4904719302715929, 0.496125836729911, 0.4946324000717834, 0.4891745579379463, 0.4923928345417297, 0.48127759793201896, 0.4908685420292623, 0.4875549786563136, 0.4846063645605987, 0.47920035284022466, 0.4858317561518141, 0.4840210392675663, 0.4811646883460994, 0.48872905996125515, 0.4945403750805388, 0.506397708919819, 0.48832096317901663, 0.49337548786491414, 0.49146551300234076, 0.483500953000621, 0.4940714701515691, 0.48568856018271006, 0.4876125302493247, 0.4904940042389828, 0.4966987256737298, 0.5035605879652143, 0.474946205649408, 0.4854368827897315, 0.4859032483850178, 0.4923798246895742, 0.48157557935014, 0.4937093241438469, 0.4863269984506635, 0.4859419349029354, 0.48405776689934016, 0.49323503007819464, 0.48236870691993095, 0.47983789908322344, 0.4842659590733925, 0.4846234515013623, 0.48778661192889944, 0.487940082761639, 0.48772892852547123, 0.4759978637295117, 0.4885427316924197, 0.4899721719611141, 0.4760798732860799, 0.4900535763440189, 0.48769342635519336, 0.4758260371695146, 0.4886866610990772, 0.49812784603987514, 0.4911531336976933, 0.4915931562089017, 0.4871183930158083, 0.4978805613514804, 0.4813425362605787, 0.48884364614234777, 0.4832196253719813, 0.485559701130801, 0.47673073352457435, 0.49894477553638983, 0.4897279020562156, 0.490875400431141, 0.48373587118213585, 0.4947014595766636, 0.49982005669876706, 0.4841632374473681, 0.4711267116647094, 0.488480483200108, 0.49724665451782, 0.49275690072293526, 0.4980233555917206, 0.48148532089706003, 0.49687017101878145, 0.48578900519524293, 0.4938360697323183, 0.4896641723176752, 0.48607247048776403, 0.48980322303259244, 0.49069902180602626, 0.4907227862860504, 0.48601949564238267, 0.4872942878883738, 0.4984514992603578, 0.48645472936140055, 0.49308241884143644, 0.48927662202499966, 0.4727041134012214, 0.4852213836211461, 0.4864751523591031, 0.5014495875945245, 0.4996291117317485, 0.48693718156923843, 0.4925166583413717, 0.48908699178113296, 0.47892139336635864, 0.4936639071725907, 0.4884074218312829, 0.4842671355466781, 0.4934110221521612, 0.4889277466540874, 0.47677740439576866, 0.48236031870401574, 0.4965572527970703, 0.48016278347612024, 0.4917710736808581, 0.48637322938339145, 0.4824103321496239, 0.4808436993531172, 0.4956359975833944, 0.49513525494442673, 0.5010530203135678, 0.49725244686155146, 0.4854747635991285, 0.48580490490081857, 0.4925730937136025, 0.4793861833992741, 0.48155561531858526, 0.47898625522152133, 0.4949142499069766, 0.4837462203053905, 0.49314338911017747, 0.4860514916729253, 0.4829358887041344, 0.5009172461824661, 0.48093646048229693, 0.485739073688895, 0.49299752409789305, 0.47899538324003316, 0.5044328074814872, 0.48818048481272325, 0.4732030855534856, 0.4805612213124507, 0.4928478426842162, 0.48905690391858725, 0.47717577015885704, 0.4935078624290237, 0.4854456442210393, 0.47669953951571703, 0.4924184428861719, 0.5026287030510548, 0.4872925249969777, 0.48611094873687977, 0.48616947123133447, 0.47954047562772606, 0.4740365091369235, 0.4905071035479825, 0.5013909556213075, 0.49142629585925685, 0.4923427353925753, 0.4760842638249489, 0.48711308358160915, 0.4870111780550251, 0.474769515267481, 0.4806815591227692, 0.48214485720580647, 0.48541463759768366, 0.4962348293103747, 0.49243552434486876, 0.4748906646697742, 0.4818125794709587, 0.4803410356809886, 0.4924095578144873, 0.48632501410261575, 0.4873244776054354, 0.4909349020868774, 0.48824372032956914, 0.4913760651728356, 0.46930065746094207, 0.4691341211392313, 0.4888070227347006, 0.49271870152950464, 0.4836447635564484, 0.48691634340945755, 0.4899395281834235, 0.48205574824735525, 0.4799726647199431, 0.48368768992579053, 0.4768686727564999, 0.4958041632580838, 0.4985214172395257, 0.48305211448761703, 0.4737766452276751, 0.4875241997414839, 0.48966444112022045, 0.47743806528014204, 0.4916458426276917, 0.49790646657677845, 0.4908666055351686, 0.5016366546662585, 0.4926491931084392, 0.47584878019593463, 0.4901983898178986, 0.4854134518074116, 0.4855498155549496, 0.48880357447957, 0.48134865173692265, 0.4876783795869589, 0.4836117657722497, 0.4868279533699981, 0.5003965356187133, 0.4963950328569307, 0.49499992385544683, 0.4867431334469557, 0.4750951486418411, 0.48549414609555475, 0.48738799946682854, 0.4966394220413402, 0.48241274966154096, 0.48456233697198464, 0.4837667150476561, 0.4854050397453046, 0.4879537767728041, 0.48992428075521804, 0.4855881869820681, 0.485366761087862, 0.4941194147767241, 0.4788738555735457, 0.4927783907090575, 0.4921418391312433, 0.49386252644140965, 0.4965169484887735, 0.4946235136170214, 0.49177225377756506, 0.48640665331578625, 0.4919751706635292, 0.4897115828597687, 0.48234418460822337, 0.4776305502557603, 0.4899603291535261, 0.4799461228432994, 0.4839959024904598, 0.497914006107393, 0.49672579028391484, 0.4951976145256529, 0.4859222691755765, 0.49022400118550424, 0.48313414837302193, 0.4805785040405271, 0.4923208665690702, 0.4946110441716925, 0.4867876780350255, 0.48881552376977716, 0.4917790475821786, 0.4777944494917946, 0.4895719933592828, 0.4854944458156407, 0.4994062525996026, 0.48865218157162005, 0.49190565972413236, 0.494852913200452, 0.4899326939446369, 0.47930944698179756, 0.5027606259084437, 0.48817232222259077, 0.48061200210772237, 0.4884944557491435, 0.4814451013041516, 0.4834946514705686, 0.46942967692299925, 0.48239651790114557, 0.49073161463120435, 0.4859269558862087, 0.48775320747436096, 0.48240808558506926, 0.4828264837696277, 0.4891206247341365, 0.5027362690465172, 0.47774477044825664, 0.49273549226447333, 0.4841132394199562, 0.4884762075500244, 0.4973880609545997, 0.4958601661512917, 0.4861388595216409, 0.48704293137071536, 0.4865754831277137, 0.49012139378530084, 0.48779433332057964, 0.4817149726905826, 0.492254825054602, 0.5061709232118206, 0.49475260876454596, 0.48005899429734544, 0.4951875586586985, 0.49019444476888996, 0.4880643614580736, 0.482677287071105, 0.4880997846288212, 0.48225769606369556, 0.5003966795725341, 0.4849314658788364, 0.497551332237422, 0.49361570114320086, 0.4803807817178202, 0.4875199185223302, 0.4879450918473099, 0.4751654373057916, 0.4903994358793565, 0.4834556413786041, 0.4912524934735437, 0.48427609082210965, 0.494598809543909, 0.49022362156406707, 0.49729534961781, 0.4717289208583281, 0.5055255973509365, 0.48640592310882785, 0.49296988366259936, 0.47256970623534444, 0.49227946261234656, 0.49462197273627206, 0.4749133899350314, 0.4834679362324779, 0.49032731020098125, 0.47846535504306215, 0.49147305426640026, 0.501840614645724, 0.5018515405221985, 0.4842958650983722, 0.4866720289057301, 0.4858651976453715, 0.487947308346174, 0.48166991661121333, 0.47325258316868346, 0.4866881812476462, 0.4925480688595913, 0.489833300355008, 0.49043726202209253, 0.4889805876638427, 0.4907497550429676, 0.4834710281940147, 0.4815956411391374, 0.48543653540006326, 0.4889134107136644, 0.485900002242323, 0.4754636799191456, 0.4820386475169217, 0.49055242149107375, 0.4885450142795743, 0.4793376677192983, 0.4791676161533006, 0.478533236154491, 0.4930906016651028, 0.4988730007434618, 0.47732845676373664, 0.491052945616871, 0.48802927732008455, 0.47702551087928896, 0.4824840931835064, 0.49308351281611806, 0.4747117664708372, 0.490453409953715, 0.4848325254839962, 0.486223847025507, 0.48118010408454476, 0.5001928819501816, 0.48660449669903333, 0.4826299598962077, 0.4767713756922544, 0.4805965295780479, 0.47822000027926764, 0.4880772309779208, 0.4926853425026164, 0.4868919200388609, 0.49053981894476845, 0.5016545197519757, 0.498320884783444, 0.4858897099877353, 0.4881385750377357, 0.47611437659152056, 0.4962747775351381, 0.48693087968516524, 0.48657270183989004, 0.4874915639758497, 0.4801658432681741, 0.4839298362030959, 0.493444274932608, 0.4926971947822878, 0.4899535922980575, 0.4891426430665514, 0.4811886620460993, 0.49299637288558984, 0.490317729158789, 0.4758989164740426, 0.48388028744855144, 0.4940479158618835, 0.4883366281651839, 0.4815853515017449, 0.481017454197703, 0.47877402976364325, 0.47880209565281145, 0.48232434530265433, 0.49678733532816854, 0.49406311767956007, 0.4972355988531974, 0.48139816913338546, 0.48614553175410463, 0.48485033193363875, 0.49489065234742274, 0.49113581485944696, 0.48823731079267807, 0.48593232015102067, 0.5022826931647384, 0.4893812228467129, 0.49173373040635215, 0.48173895514233683, 0.4979639857663692, 0.48644625163253785, 0.48285134049343226, 0.4845676861757063, 0.48104424074152863, 0.48954648087808045, 0.48791424436904224, 0.48672464428586987, 0.48279240140783836, 0.4900141972622587, 0.4845134149241649, 0.4880147998390657, 0.4913100145983531, 0.48326982123866297, 0.47979620646420207, 0.4795386192656866, 0.4844546649989601, 0.4818005960424457, 0.4858032930905937, 0.489617378156591, 0.4894951404726119, 0.48752936605547414, 0.49968488693221036, 0.48664255890006003, 0.49115024902947846, 0.4726689131163588, 0.4840866235712913, 0.4943199418253992, 0.488712934565152, 0.4849956707092936, 0.48058075754551527, 0.5010967557724929, 0.49038951014590226, 0.4897529071641628, 0.49263504106616335, 0.48264541105986, 0.4753160493117922, 0.4793236296048898, 0.48764456698724074, 0.49446901371725943, 0.48860082924605547, 0.4844233471873925, 0.495495962208016, 0.4839384232227629, 0.4931923075076471, 0.4783591927552293, 0.48953934664811877, 0.4935712158832933, 0.4853519269987418, 0.48270097871176204, 0.4886927085132473, 0.4789142164490126, 0.49353311329082433, 0.48936050104389345, 0.49517699352857647, 0.47739538036090345, 0.47565706522447315, 0.4997158991800127, 0.4820206237849661, 0.48547813710293064, 0.48227012944321146, 0.48558183479986655, 0.4829597215342514, 0.4877532901104671, 0.4947934232191209, 0.49349166877545175, 0.49005365362516556, 0.4866187280011945, 0.48530795415703126, 0.4760440127672939, 0.48506774204132236, 0.48758267705098174, 0.4877637632153986, 0.4882128911115062, 0.4828635829414407, 0.4899539151432052, 0.4776921089504648, 0.4824433262833301, 0.48486131821835865, 0.49344731400908365, 0.4881790970863365, 0.48980063866790297, 0.48265595725981364, 0.49144335303840964, 0.494513786207592, 0.5025435010063726, 0.4844227425778402, 0.4794727989480966, 0.48623725224774605, 0.48836612374237726, 0.4944023640956345, 0.47844273840579643, 0.49483169493161544, 0.48798781057268725, 0.4833440840023989, 0.48360510559584985, 0.49749427973586635, 0.4919262248657068, 0.48671883174682856, 0.501143100611395, 0.47571587469458043, 0.4957902376801426, 0.4750231926995749, 0.4880778203642369, 0.4783950641071174, 0.4881845567988768, 0.48592291390557374, 0.47825566252536644, 0.48315179212533743, 0.4767065523452317, 0.4849436372875772, 0.494312008130297, 0.4973517403972198, 0.4967830915856039, 0.4888243017074188, 0.4839935633425039, 0.4928535253602604, 0.4911818778642048, 0.4963774954380579, 0.48303539251154026, 0.4919190672375138, 0.4852886157615359, 0.48624211100613257, 0.48631035004103684, 0.5115127556078632, 0.47909879784133214, 0.49374404962900015, 0.47455344582673015, 0.4940393299786079, 0.49418751052541754, 0.4886350857247805, 0.48504492725840503, 0.4886266629621192, 0.49171447407314917, 0.4959276986006586, 0.5069611528948856, 0.4962121703635431, 0.48903844677497565, 0.4856647344114525, 0.4925085724145974, 0.4846242784567143, 0.48824390398278666, 0.49431765353112544, 0.47501328369682144, 0.48092207761630434, 0.4860081270036528, 0.485308978870731, 0.48319493292038207, 0.49677118482677163, 0.4843563540823057, 0.5008948533769606, 0.496776177934346, 0.49324864563255655, 0.495212943337663, 0.4803320219753761, 0.4789189363155971, 0.49395904189430445, 0.48521968036427265, 0.49526830406858247, 0.48570741101901255, 0.4921908362016256, 0.4792644259578653, 0.4737614636330799, 0.49769860747428724, 0.48910290446228105, 0.49265265759677074, 0.49066145188098825, 0.48502505210265884, 0.4925102902231803, 0.49714224335670903, 0.49809444002040304, 0.4940396826903878, 0.4844460905776353, 0.4950380520503189, 0.4832527489225364, 0.4785819320912446, 0.4840166021255233, 0.4851522566621979, 0.49010963978552685, 0.4864473618431475, 0.4878529944197926, 0.48616419966017393, 0.5040301229043799, 0.48515479201237083, 0.4825734958530711, 0.48956672758298153, 0.47490870043102074, 0.48157195467314284, 0.49071883891400686, 0.4937129327524765, 0.4914892812419986, 0.4994422846755804, 0.4956147986585317, 0.4881786046239224, 0.4948581022549022, 0.48831107200590396, 0.49580104166612726, 0.47906867463492103, 0.4902156413076041, 0.4890095082159129, 0.48975293256371977, 0.48814151745704976, 0.5000919834118894, 0.4702390892159845, 0.4839666737913078, 0.5047298920275116, 0.4968301299610287, 0.49199516866731785, 0.4886641488539239, 0.47681849991113145, 0.4817044243755443, 0.46919682538827556, 0.4798039906538847, 0.48326872966780166, 0.48668755577344514, 0.47501923347629094, 0.48274625826748957, 0.4718908232641019, 0.49389507432656715, 0.4736114548705313, 0.4821214559478188, 0.4932915108218474, 0.4713732756011203, 0.4864574587332848, 0.4963781189045318, 0.487420393827982, 0.4963281586697752, 0.4786455134721546, 0.48527358901127515, 0.4883085168901177, 0.498461199432917, 0.47796699310225627, 0.4956282191271559, 0.48193242573347556, 0.48927017808498724, 0.4920154120121696, 0.496213901755407, 0.485993775403825, 0.49218474595223854, 0.48722901340178465, 0.5059964099404686, 0.49940790037755667, 0.48076442778631684, 0.48338629133607125, 0.49226009880886046, 0.49052023244753107, 0.4830898096509552, 0.49431183168203324, 0.4895787633367264, 0.502213514340766, 0.4913015578229724, 0.4856200778163377, 0.4789710884068793, 0.4843083268278893, 0.49235097656928917, 0.4901118621047478, 0.482594975754963, 0.4800673337948047, 0.47925174649794583, 0.4797671862694682, 0.49080542392461546, 0.48707472019011444, 0.4840306707934548, 0.48151812314753506, 0.494079349988821, 0.48699886886312815, 0.49159623079182896, 0.4903309752251394, 0.48883586324162515, 0.4927594180696012, 0.4863667072473621, 0.4785202128729865, 0.48023728397255094, 0.4964798049260935, 0.4918798916314943, 0.48106044569291656, 0.48852569630517745, 0.47722276942387437, 0.48495402040938046, 0.49653974000561807, 0.48579116434330993, 0.47979684030628994, 0.48167702111771604, 0.4743314590503253, 0.4815048676269342, 0.4936291944895545, 0.487204849646149, 0.49210888927208296, 0.4982185194269155, 0.48234381050857233, 0.49368261414216996, 0.4870738357047666, 0.4784249238644876, 0.4845657705275314, 0.4947005751785662, 0.4956763347611418, 0.49191265333622713, 0.48618943653053315, 0.4792364796777346, 0.48873905099140774, 0.4956178823869747, 0.49876687022676075, 0.49037804864742085, 0.4970185992251193, 0.49219218990175817, 0.48572126775049124, 0.49308155630387507, 0.49285924573994566, 0.4831771295611248, 0.4828588358546418, 0.46986352659636993, 0.486296513718018, 0.4822292096605364, 0.4897673299166528, 0.4952480391521942, 0.4886993230095026, 0.49872340133450854, 0.48193456160537085, 0.4940863754332776, 0.48792165571920604, 0.4877966270974143, 0.47815384852757475, 0.5000978297892688, 0.4790623487349587, 0.48621849996904576, 0.4898664199512026, 0.4872421879625453, 0.48971461225298574, 0.49361867706090984, 0.48635747558644254, 0.4799053510297467, 0.4961234579549534, 0.48866833257496683, 0.49352978087629334, 0.4943456956381656, 0.48283844629125133, 0.4806280523140353, 0.5041304786363463, 0.47787344486857297, 0.49771084822555683, 0.4902077414168739, 0.49045317905810426, 0.49038697772091283, 0.47697018358052684, 0.4906460716466622, 0.48740490341003323, 0.49118441569398824, 0.48817154731622814, 0.48768563487706884, 0.47758500582713764, 0.49594964577816014, 0.48324852095829474, 0.502462140573087, 0.48891685764909004, 0.4756035944174374, 0.47006545067111494, 0.4858956371518698, 0.49642557638371587, 0.49431545594064785, 0.48728039880432705, 0.4901203705221915, 0.48741903985843205, 0.4888507516899258, 0.49257266796518756, 0.489354782889988, 0.4922101822885855, 0.4884667372233691, 0.48123561875121856, 0.4872853817906203, 0.49875773507161364, 0.48853216320960907, 0.48270909717932114, 0.4928166639631057, 0.4886566690895436, 0.48200916261945703, 0.4907646888362028, 0.47548443350960473, 0.48617429012982566, 0.4880176007371879, 0.4985965995362139, 0.4862733386497799, 0.48218337529692545, 0.48861241870469213, 0.4849788683590478, 0.4890835001145736, 0.49240844476292533, 0.49926753829108, 0.4928665973480015, 0.4824522479795376, 0.49250017820078984, 0.48640503400377755, 0.48820682699029216, 0.4906189156715183, 0.5013103059437901, 0.49319574371841307, 0.48446944367137107, 0.4922013909378524, 0.4974932292439701, 0.4905093067917654, 0.4895981584273635, 0.48027767714168895, 0.48725423398753687, 0.49121888107956374, 0.49166644861398195, 0.4822230264569129, 0.48685382576132175, 0.4900239976381321, 0.47637600337724023, 0.4900321417198553, 0.49964770671326264, 0.4910913460251857, 0.48171593691346903, 0.49462899359602586, 0.4894240354474955, 0.4822888432407961, 0.49511690960511606, 0.49310366779120707, 0.4838129334681578, 0.4928867432572205, 0.4928599192570985, 0.5009211883236987, 0.4944583454523747, 0.48315499516988225, 0.49478682121592477, 0.48884129955122013, 0.4807823642707687, 0.49192843284564525, 0.4923678597606529, 0.49462344387246404, 0.48093238692103224, 0.48082749457496515, 0.49383491099903454, 0.5032058158390487, 0.4917702098147247, 0.4898558187279616, 0.4883608485294651, 0.47274868395628294, 0.48185739829641755, 0.48364404341768163, 0.4887480126260663, 0.4967364011212629, 0.5033510512733874, 0.49566680194973767, 0.4839728735669309, 0.493672050544871, 0.47694285700337413, 0.4944870209945435, 0.48562805975149465, 0.5015829364616881, 0.4782405639668096, 0.4943764527244035, 0.48794680143662683, 0.4800310936069509, 0.4838187878459096, 0.5124151849518407, 0.4862200642398884, 0.49305625457173424, 0.5007744030686901, 0.49141390731141976, 0.48350097088805527, 0.49184937642483845, 0.48203080876200677, 0.5168705019570149, 0.4726125677126667, 0.48345891053256146, 0.48732623675524156, 0.4900071161759303, 0.4931417071810531, 0.48351813334822474, 0.5004762417768192, 0.48637198037124973, 0.4943629159344581, 0.4924059381497933, 0.49157772971039027, 0.48584685348639595, 0.49423978290530424, 0.49294168399302707, 0.5036469417110565, 0.5019921184568075, 0.48318585035033806, 0.4967012757850378, 0.49456778055585077, 0.4742381558600255, 0.48881690158829144, 0.4928286405753387, 0.4806765600083942, 0.4915740311082524, 0.4850891627662861, 0.4933697444161343, 0.48364994877649664, 0.4904337222207732, 0.500485547246441, 0.4812938048501517, 0.5010117409895014, 0.48093323184857395, 0.4878732908098498, 0.48591434750895823, 0.48189366018596946, 0.48926329618472125, 0.490243208220031, 0.49269574948512374, 0.48344313533483485, 0.48715256095172266, 0.4801519573784061, 0.49654298490841975, 0.4853767730779404, 0.4873759219092033, 0.4923139338321817, 0.5035186412960331, 0.4883184237536051, 0.4791377515485157, 0.4909657432575348, 0.4708238278608595, 0.48892809905065165, 0.4819761661697255, 0.48451956082334674, 0.4855095702187749, 0.4953438563252058, 0.48826027563595925, 0.48809088993232225, 0.4845139658524652, 0.48923749410810896, 0.4798146231063565, 0.48845781082423406, 0.4894353530962527, 0.4877467628518165, 0.4961452207358193, 0.48355214310138117, 0.4968345810673199, 0.49527514709336784, 0.48691126450528754, 0.48811400936463095, 0.4888079400170657, 0.4910031002276446, 0.4833404109920293, 0.47670519175078063, 0.4932784779661449, 0.49068489397198384, 0.4933206875739983, 0.49750812211493095, 0.4860071245525789, 0.48889484294515967, 0.47898577711214096, 0.485689331023853, 0.4711224758129987, 0.46907520840523853, 0.49757747390606677, 0.4944524369351819, 0.48459237070904954, 0.48800885682702533, 0.49293982534537506, 0.48898350468251134, 0.4967405303586075, 0.4917283154311317, 0.487611931437433, 0.4885582982003632, 0.4944696513980414, 0.47798876805937496, 0.4831692472039796, 0.4835378440817502, 0.4812348060010427, 0.4872446532293054, 0.48228499350574405, 0.4908389981785952, 0.4846380517318901, 0.48583209061012617, 0.49871409137095973, 0.50019913065423, 0.481949810863335, 0.4927029894249755, 0.48545978437959153, 0.4967853542219171, 0.5008264650610503, 0.4913221344515045, 0.4898659698943082, 0.48085355272562674, 0.48631161632745523, 0.48967368877122897, 0.4855816418493082, 0.4793000127656097, 0.49127248069569474, 0.47931129128521366, 0.4882524786276481, 0.4866521666786809, 0.4862675811992685, 0.5052943064928758, 0.48860261920018966, 0.49014171781684496, 0.49205596629753307, 0.48440307867359356, 0.5022604436742373, 0.4851452807945785, 0.4946990965352931, 0.4875190614359737, 0.47975677148628376, 0.4968799787509075, 0.481302460796489, 0.48605840527940514, 0.4959898799073837, 0.4804371981193952, 0.49931948454106106, 0.4769818107069814, 0.49530125625018867, 0.5006187630226975, 0.49701953635665735, 0.48838980676208915, 0.49032674017403416, 0.4795507624523618, 0.49574605935680216, 0.5060261064975433, 0.4788438847854463, 0.48529699926862224, 0.4838376675397146, 0.4949899706521418, 0.4836599440875494, 0.48164147550223585, 0.49185070393909375, 0.49445546203790725, 0.49232570733156733, 0.4906182918455819, 0.48426676841464056, 0.48628062262893446, 0.4858615382035543, 0.49750743443208056, 0.4844492370850578, 0.4887631687214264, 0.4909541685645196, 0.48728999163733183, 0.4862960086827454, 0.48367909040781026, 0.4792987625561698, 0.4989618223749639, 0.48843456212438435, 0.4842898062345587, 0.49243558953421296, 0.49340850917864454, 0.4989965314703057, 0.4928939708220834, 0.4803319351741535, 0.47867975674988156, 0.482200663459339, 0.4908266495274836, 0.47935756850906314, 0.4851147770872778, 0.48133457323700896, 0.4905674889708561, 0.48566414448064993, 0.4932956894291992, 0.5014645456399506, 0.49625909006234414, 0.48536146673725034, 0.5004245354876122, 0.47625348241379156, 0.490917330023534, 0.4884738390268387, 0.4896061282700233, 0.48953220458230956, 0.47529777975957255, 0.511057938267853, 0.49240612278875984, 0.4795050967142692, 0.48754623493708493, 0.49148261559977613, 0.49533846777421775, 0.4826733165071964, 0.49541750312103283, 0.5015624057368735, 0.48419042528883005, 0.4861191126924553, 0.4842935535321773, 0.48243299506848053, 0.47792665483415203, 0.489612045809113, 0.48098997059512866, 0.4887393085980524, 0.4894287792625417, 0.4959498920758732, 0.48508101890215244, 0.49089513613714303, 0.4852708498106486, 0.47726918829258397, 0.48481396089097706, 0.4849483090824582, 0.494552728509494, 0.4932649802980945, 0.5050145159978185, 0.475272561672612, 0.4947566483942578, 0.48514811532329577, 0.482417860563967, 0.5007900597628347, 0.4874406474166154, 0.48486582013603896, 0.4816405387193546, 0.49338893149596447, 0.49237127228598393, 0.4943234472960312, 0.4965401805706759, 0.49420963852353733, 0.47713279424063043, 0.48150182802189134, 0.4838921544078871, 0.48789188246000115, 0.488618448261885, 0.4745506928179793, 0.49853843657644986, 0.4753244524509269, 0.4887819321612122, 0.4892904542340999, 0.5026949227787755, 0.49141996278160516, 0.4821585555295473, 0.4872102659995456, 0.49112181834815455, 0.4966329191349251, 0.48406497093293555, 0.4838351194963931, 0.4996989549260602, 0.4832947433945384, 0.4882880469147258, 0.4931629991479551, 0.4928122434472987, 0.49060844225118055, 0.49125142747805833, 0.4963932003847291, 0.49277069563086495, 0.4828440356036583, 0.4879501297030669, 0.49238088786343004, 0.4798716933000362, 0.49934476366614766, 0.4875855316967032, 0.4843133325845435, 0.48763717138634527, 0.48017520980294637, 0.49042558095052313, 0.4756624065338808, 0.4889970197884362, 0.48917459489679327, 0.4778982525596497, 0.4895593985498467, 0.4837052300140853, 0.49832028960385333, 0.48728336660865523, 0.4847679699345836, 0.49552582067551, 0.48600823151101685, 0.4761411699445123, 0.48520420755951993, 0.4893606286057737, 0.4820064291984678, 0.4925456065304012, 0.48911093746102513, 0.48228279461011714, 0.48121002032421134, 0.48815148487180177, 0.4951116031216472, 0.4933959579914844, 0.4880259374491786, 0.4940463853835757, 0.49981730947125264, 0.48120382963277636, 0.4836626955151575, 0.4816578672803735, 0.5037707944850891, 0.49873364305392726, 0.4832625655451634, 0.4875615994584726, 0.48191126053972166, 0.493816805918583, 0.49650997592608115, 0.48405472065053434, 0.48530279114477326, 0.4884238637521033, 0.49082352937531737, 0.48327891288825614, 0.4983447508680378, 0.4976450868576199, 0.4784630613239252, 0.4889929133992395, 0.4770636674596693, 0.48393381770461524, 0.4870573781639769, 0.4897715052987834, 0.48694254351562116, 0.49083621996638704, 0.4796182589028842, 0.4978884473987488, 0.48028228482593915, 0.49588556391425326, 0.48893779829314254, 0.4860621654932369, 0.4785018078929475, 0.4833680574283278, 0.47991128456480797, 0.47326875067828006, 0.4846003874574767, 0.4945260086444629, 0.4911312285585887, 0.49077277970795213, 0.49501297544920786, 0.4859214189885516, 0.47663275024438934, 0.48641641284760095, 0.48627725448386105, 0.4917085742000415, 0.48514908133644286, 0.48277525158440243, 0.48860554727666494, 0.4701820890521236, 0.4953054519389327, 0.49221514042138925, 0.4933287294951844, 0.5032049149961035, 0.4855534692703899, 0.4933406145453437, 0.5023751473937637, 0.4937916035494778, 0.48970140633071796, 0.49993835132564046, 0.4862097369163848, 0.48258882704351713, 0.4905464686559405, 0.48766670396309486, 0.48189612926996017, 0.4982335563841754, 0.49188880010080493, 0.4831773679006238, 0.4883748049014718, 0.48254061656408265, 0.4865522766546103, 0.47389689384076733, 0.47928983238569456, 0.4907714553546163, 0.48527292065047606, 0.4834952271086114, 0.49036957480611987, 0.4848944165786857, 0.494329621620751, 0.48894936470986333, 0.5017002532256482, 0.48609004902559044, 0.4887782155617671, 0.4854070843943303, 0.5000661205639484, 0.48267604300004197, 0.504036712024062, 0.48334342592807034, 0.4936537603277893, 0.49026046974447185, 0.4855876124140006, 0.48486108910396986, 0.484092951478418, 0.4881582070260962, 0.48846103825417564, 0.48264282096957917, 0.48956684929538447, 0.48433517635302903, 0.49359779932727205, 0.49671377169855835, 0.48427962738172364, 0.48557061704241655, 0.486228180405918, 0.49930733745636224, 0.490284886181785, 0.4852474842555872, 0.483414825323435, 0.4962409186662453, 0.48362928384835674, 0.48083788834383545, 0.467840713015088, 0.46494784712312953, 0.4944635943519973, 0.4881028604454546, 0.4797487826632131, 0.47582128732063234, 0.49035127599738354, 0.4873862530007252, 0.4805774778122011, 0.4822141138772366, 0.48636421542280267, 0.4804010056056394, 0.48259738767723454, 0.4770273223050583, 0.48498172220653996, 0.4978873464401713, 0.480342043239647, 0.49342738203362035, 0.4904500215108464, 0.4875761772582598, 0.48486884764123206, 0.4880178277582499, 0.49718378252663475, 0.4940701248569723, 0.4851041139778307, 0.49596525704310246, 0.47650094515389824, 0.4884151011713505, 0.4820403619294709, 0.48113206071081993, 0.4850768880581997, 0.492884353889706, 0.4933419051094942, 0.48275683433120403, 0.4911196140291404, 0.48390888736415144, 0.48636030371076333, 0.48381662266924497, 0.48212454705561425, 0.4896113584001467, 0.4997242996500981, 0.48057181840616797, 0.4814283507149479, 0.4910782853034077, 0.48692807106601504, 0.4724929369673927, 0.4862703575647626, 0.48171181128602086, 0.4980385020820865, 0.49336735353740635, 0.4808452627045609, 0.5016628944381137, 0.48638810979553204, 0.48429461450748285, 0.48304561134521407, 0.489651338669944, 0.48471472782704006, 0.4710309084978167, 0.4837418457399059, 0.49094144264111783, 0.47286702572297756, 0.4910374484859781, 0.4818476609092254, 0.4922307812511937, 0.4993135135001389, 0.49497436958291435, 0.4971153791086062, 0.48265596091955154, 0.48881127333702284, 0.4906461913411602, 0.4914901386536731, 0.4849546473915372, 0.490623479063354, 0.4989775378801034, 0.48842190073985875, 0.4877730646973576, 0.493632329578763, 0.4900991411319787, 0.4907332479930227, 0.48921958135263105, 0.4857570275314263, 0.48489105109961494, 0.48075443732257805, 0.4749959837630038, 0.48312537858871546, 0.47468688056209385, 0.4757679218350465, 0.47957539027238416, 0.487887857432912, 0.4911339599365465, 0.4884303554540409, 0.5096954922331933, 0.5053568952101621, 0.4888573980426908, 0.5014069580759307, 0.4839726035067829, 0.48901949782613013, 0.4808358584550089, 0.49818190937155404, 0.4811792516773192, 0.48626908267286756, 0.4892857398337006, 0.4743110444875024, 0.48809237175163334, 0.49017963530700276, 0.4815649780768957, 0.49928211093445324, 0.47911979019639345, 0.4878813431324138, 0.48368574631837624, 0.4822356273325719, 0.47674250035545923, 0.48440859483256554, 0.49624075747386337, 0.49369938602985974, 0.489737611739362, 0.4853494970616228, 0.4779857719718174, 0.4871840427114935, 0.4942012671439141, 0.4929619004551786, 0.4806811268097547, 0.4888350343650535, 0.47951406153565773, 0.488476775167653, 0.4868163146171711, 0.48736442342401215, 0.4822014431707854, 0.4862914043242746, 0.4997944842630309, 0.49224924229334943, 0.5071813110181612, 0.4796243532459056, 0.5002994756513405, 0.48784288935864367, 0.4885336382274502, 0.48769141152876455, 0.47019227730915203, 0.4900268556854682, 0.49045966033746674, 0.4974580415711328, 0.4946233969589745, 0.4889905868752575, 0.49487523477791145, 0.49647516912277667, 0.4903216937063193, 0.50204053644168, 0.4708058059938726, 0.47427465029589183, 0.48620917193036906, 0.4867855065784682, 0.483647172040362, 0.488217467744247, 0.4976121154059921, 0.49323174913674406, 0.4890514772020931, 0.4712962408316464, 0.4834496332856242, 0.4943935163696013, 0.501134897659454, 0.4994351783526518, 0.48846574468220766, 0.4816577525958326, 0.4932402479524579, 0.4851177498076718, 0.48780225320700343, 0.48516625591499557, 0.4917520901904497, 0.4980863687812755, 0.4937763217966949, 0.49536583248808336, 0.49131562365408005, 0.4973661104273871, 0.4902945287510475, 0.4898545993181628, 0.4838763850029843, 0.4881358849886025, 0.488326303216398, 0.4862828223038961, 0.485321702938369, 0.48543867723221623, 0.48274471694550397, 0.4871090335871463, 0.48152535247778816, 0.4868986067057991, 0.4784366678415187, 0.47390590166447016, 0.48208249200736786, 0.48716767030150954, 0.4874663803968476, 0.4873505960213437, 0.4878863853042294, 0.48803087022030395, 0.4858653196156221, 0.48438644221003485, 0.500885250161693, 0.4867457498303055, 0.4887479213698076, 0.47869153826825533, 0.4840960201825269, 0.4935852642242572, 0.4946567437991669, 0.4959485302769514, 0.4822254185192279, 0.47891729608847905, 0.49115604613715197, 0.4818279292333911, 0.49008200038152855, 0.49364651635555706, 0.4847898859934156, 0.4814205739531908, 0.5010817407854391, 0.4839334180192738, 0.48052248546442433, 0.47486022672362793, 0.49359514690874734, 0.4798087338589721, 0.4912358420940218, 0.48104788317129515, 0.48578105660938725, 0.47692826175914005, 0.47476347190558926, 0.48920624516102296, 0.49059913443590114, 0.48418184738774855, 0.48521015161434417, 0.49380003621830265, 0.485999090710163, 0.4964912353889311, 0.49632827910208616, 0.4886358842856769, 0.48415345217933614, 0.48507827453960217, 0.49435243538296014, 0.48338576305766273, 0.4871014537036848, 0.4940067541087588, 0.5045727484078834, 0.4943996714201944, 0.4904812453049203, 0.4891647915399162, 0.5013845379272522, 0.4910504714952199, 0.49993025283897324, 0.47914858729347914, 0.49229705375351307, 0.49398851828927215, 0.489429598429969, 0.48818759318210236, 0.49232270011540963, 0.48220807488186046, 0.4941970264141343, 0.4815354307891931, 0.49418728002246715, 0.4943423774538238, 0.4866782162382218, 0.4901456251868193, 0.4871400443021504, 0.48866447413031666, 0.49955504368750225, 0.4716846629299273, 0.4978770988574958, 0.4864572458369035, 0.4926622408828952, 0.4859061870399612, 0.4888860633903806, 0.491362361861675, 0.49265623655266766, 0.4919653977519178, 0.493749491588944, 0.4867942428397925, 0.4911375456992344, 0.48417562584048796, 0.47978007124551847, 0.4795251267261268, 0.4830504341422858, 0.4903550453397782, 0.4846554406644457, 0.49533578750725676, 0.47525491931068786, 0.49291535341856024, 0.4905391047697207, 0.47943852383909263, 0.49329415544458693, 0.4710072875691256, 0.48054895970703815, 0.49819759918682854, 0.4771956353234499, 0.505834087777541, 0.4892001160999112, 0.48191985883125504, 0.5003112246633653, 0.503659746166133, 0.4885136446591934, 0.4911037725029833, 0.47595965802885415, 0.48420083680191467, 0.49158081817457894, 0.4884311066644416, 0.489081081812831, 0.48180094942564805, 0.48672431012421297, 0.4836501360945381, 0.48740998994509394, 0.4879379676938438, 0.470622969729307, 0.4882827278829258, 0.48941747014173087, 0.49257913829149313, 0.49170998920238435, 0.48645543071643016, 0.47731373127057714, 0.482919923925686, 0.48554661089501744, 0.475760381356315, 0.48300310340319913, 0.48817308788608027, 0.4859551805982319, 0.5051623908644521, 0.5088551518601723, 0.4814786266519723, 0.48772314210198614, 0.48210897586281043, 0.48495249815317965, 0.48528340873281195, 0.4856980943307818, 0.47630731341859206, 0.48958777608403237, 0.4864381257216577, 0.4856751025567776, 0.4888857458864685, 0.4834351696237005, 0.48564263951170544, 0.4910920662872724, 0.5002407494651373, 0.49063408306761996, 0.49187665159938815, 0.4880333251961787, 0.4996006795554129, 0.4919981685919354, 0.49033449870640006, 0.47721738432777266, 0.5036338748184728, 0.5008616230322671, 0.48496257088138084, 0.48996648863958653, 0.47747054506589737, 0.49311037735432595, 0.4906209579474708, 0.4829144850258426, 0.4880613289461668, 0.4749570608225158, 0.49108920765679703, 0.4845407675910589, 0.494069353184986, 0.4890945594620413, 0.4808478331904833, 0.4769487010657092, 0.4926859576505423, 0.4967436310036141, 0.49624867625808083, 0.49062756221570336, 0.4904423152498474, 0.49080907104947247, 0.4857742301038562, 0.49210497488201194, 0.4981593523473177, 0.49286723798004467, 0.4839964718333459, 0.4948119649895481, 0.4872110527360888, 0.48649400368260903, 0.4804790763311723, 0.48477356031198504, 0.4846079895859245, 0.49275998809032767, 0.47419592812854744, 0.49397320670313055, 0.47691199256731887, 0.48206163064122953, 0.4766395381573171, 0.49809081142076217, 0.49094331287131326, 0.49650647680403737, 0.48896482580966827, 0.4920262984833407, 0.4988374047342802, 0.4729102453466601, 0.4831152355574484, 0.48392825367345466, 0.49244982818534005, 0.4835784162010108, 0.486827639891067, 0.5004012648859587, 0.4772216340930835, 0.48315574388083427, 0.48308307110174303, 0.5010022526668052, 0.4824494983027508, 0.49834032951033563, 0.49142799741446697, 0.4966152477218004, 0.47984467700353517, 0.48266515287842154, 0.4867642678447989, 0.48271982852570183, 0.5010866895884388, 0.4804796012707504, 0.4912681292079805, 0.49395761444524044, 0.48132910550202257, 0.4838767728911869, 0.4813087624888239, 0.47730964113146507, 0.49347058811120104, 0.487335508613591, 0.4969591077155534, 0.49007536199991136, 0.4868878138108662, 0.49099351169988875, 0.49666250090634173, 0.5032061612291322, 0.49206626305894346, 0.47538420253535313, 0.49621429385877686, 0.4795873873100959, 0.49190538849293797, 0.500368941447448, 0.4924628702334445, 0.4916957927910959, 0.4685908914947401, 0.49422493690715535, 0.4952176299408308, 0.4946042211965601, 0.5007623463022622, 0.4793542102157299, 0.4789706591702503, 0.488453900727633, 0.4914297422296862, 0.4897650460573848, 0.4929422657778489, 0.4858317950078838, 0.4767382600158572, 0.4929294117619666, 0.4930546212113942, 0.4921702433685973, 0.5064082434404806, 0.49100254114926045, 0.48475914240959816, 0.4866227940245703, 0.49501543554905447, 0.497084449589928, 0.47980597343055026, 0.48948989056819797, 0.48414089040852476, 0.47710271937100535, 0.49471972662170255, 0.48454116395497454, 0.4878822069868154, 0.4841951732700964, 0.4817393497986304, 0.49198600046155283, 0.4988504622410062, 0.4816265040158709, 0.49360960080713107, 0.48974122233201434, 0.4883420820837107, 0.49992532019228414, 0.4880641334030358, 0.4929709006892165, 0.4873820036169636, 0.48473942070482345, 0.5057595271848266, 0.4836112397498612, 0.488620830680275, 0.4873418330477574, 0.49223768553285907, 0.48573279573130307, 0.48656678452455226, 0.4885799505345823, 0.4817186145917289, 0.4930556999741881, 0.4791759813857948, 0.48353428789122405, 0.49069267122250176, 0.4938353409430222, 0.4836476772270708, 0.4961363590230374, 0.49067489498121286, 0.5001537630973272, 0.4787404878378009, 0.49086083079201065, 0.48356266547160454, 0.4904132742283469, 0.4903423843797394, 0.4958828409505558, 0.4697311110659251, 0.49544439572677645, 0.48780182368592895, 0.49191083032431016, 0.48840545831180737, 0.4813505451312243, 0.4875969676960559, 0.48743093769604867, 0.5022370164667748, 0.4855245648572969, 0.4919172036005372, 0.48182812245148043, 0.4915881986148688, 0.4935123983229283, 0.47977900320551625, 0.492700943863863, 0.49707841572362826, 0.4944004667010556, 0.483896911692489, 0.49232383025027965, 0.4775911102846717, 0.48502081964763194, 0.49684971143383816, 0.4844708138816219, 0.49960684760663193, 0.48642735488076866, 0.48211430530008703, 0.47985155713860606, 0.48907076565301294, 0.48129619850612154, 0.48886444048620675, 0.5106696655805433, 0.4802537195012074, 0.49384777511724304, 0.4785232964617754, 0.4944290183524412, 0.5019492142082401, 0.4866528193297475, 0.4936873620531889, 0.49913418927389835, 0.478462026986951, 0.4868811804901193, 0.4982509043099159, 0.5007166864765548, 0.48949222994746655, 0.5037375354497242, 0.49249098897150506, 0.48814649876766614, 0.49484189810210366, 0.5028851176923779, 0.4978266763753561, 0.491190881452404, 0.4871071297621905, 0.4857330550193114, 0.4902085932117905, 0.49117210511881093, 0.4866549586439486, 0.5045548740307463, 0.4850138686527079, 0.4940858213206352, 0.4932657223002646, 0.4979843171774129, 0.4867452391409073, 0.4829255695577178, 0.4926557351297803, 0.4826002130378412, 0.4852222230256143, 0.4949126449562404, 0.4849481298633724, 0.48460849240724146, 0.46770773790308773, 0.4891014626560801, 0.5049183923132079, 0.4802466786895207, 0.4875880613660053, 0.4832139492651873, 0.4986769219576985, 0.49897378123387826, 0.49587144902904823, 0.48199898060390206, 0.4973544377009416, 0.49830750583605843, 0.4813278985482299, 0.4815361553885509, 0.4854050172004481, 0.48423311823102383, 0.4929525100683929, 0.49055136469497806, 0.48103705360230076, 0.48267011264649146, 0.492094327538497, 0.47540214277999565, 0.483268998947183, 0.47661512191488564, 0.4869454689853213, 0.48334256001917736, 0.4888678662413123, 0.49455833375310465, 0.48541472175886746, 0.487330965374674, 0.49794599025164793, 0.48548569016298027, 0.4828932958463638, 0.4852654962151385, 0.48925979327738683, 0.4878029212815047, 0.49720141644354027, 0.4972773185929708, 0.48829290955618676, 0.4923004188806202, 0.4781609967149607, 0.4939480061648279, 0.49205590743188465, 0.4838028409901387, 0.4873001154578434, 0.47745337292871454, 0.48478155629712627, 0.483068716461641, 0.488969021138662, 0.4923513447539142, 0.4918810243807913, 0.48757629013586995, 0.492788016757381, 0.4707382970937924, 0.48575672713503915, 0.4923725040688091, 0.48524072210460023, 0.5014869506055473, 0.48870910521783506, 0.4915552592728944, 0.48616328745718057, 0.4899742125325364, 0.48192360609587936, 0.49011079432025406, 0.4875865426766247, 0.483502263240418, 0.47385681681645636, 0.4852081007158889, 0.5014121486967221, 0.48237241594158387, 0.48233055919644213, 0.49738615108009093, 0.4896265994045455, 0.48566708216410603, 0.491362569044871, 0.4892060172489352, 0.4965656415417841, 0.4821355785441342, 0.48667825022373895, 0.48622173311232003, 0.48789373182748946, 0.48206057387035106, 0.4889600647727918, 0.49458006251291464, 0.4743254157377864, 0.49378948177207027, 0.48488192067200625, 0.5027143264249608, 0.49136727614791714, 0.48612235058317566, 0.4942582353249057, 0.49349390260966647, 0.48564631927415214, 0.4912170432548636, 0.4790507713323915, 0.4831186099113572, 0.4935807767517544, 0.496148959909427, 0.4906222091518151, 0.48021297629463594, 0.48876700660077055, 0.4852445650336858, 0.4798375384837032, 0.48763819083832927, 0.48741786960722405, 0.49137859716166876, 0.4829001692820423, 0.4828151401212643, 0.48862928994469945, 0.4847957899675565, 0.4895144026809064, 0.49525580960851134, 0.4874202072404621, 0.49601583177634095, 0.49289765852353656, 0.48973294529782463, 0.4867986981625449, 0.4762631530293246, 0.4806890089555287, 0.4858616947275148, 0.4885800592358459, 0.4805604928530632, 0.4912178390842847, 0.48712622694259516, 0.49264296412730496, 0.48691481568069983, 0.47750930030115013, 0.49617922084868643, 0.4971265941038544, 0.4785764144844038, 0.4848402175686525, 0.49315844577734386, 0.48263700577512, 0.4796854409760389, 0.47712644593249093, 0.48390664794532473, 0.48499964479377095, 0.5009433421115179, 0.4832300317092055, 0.4862266440362904, 0.4895668717033158, 0.4791468931589811, 0.48683349487384914, 0.4935336409577982, 0.49144919469909665, 0.4800325316586515, 0.48228769132501226, 0.48534133165745985, 0.49857668705443603, 0.5019392604136923, 0.48082500215552526, 0.48253261521793084, 0.48371390488361754, 0.48319595236111146, 0.47691397888620446, 0.4952133236895118, 0.48756099795517704, 0.4791242769023664, 0.4717336517276013, 0.4892942078872575, 0.48544666658302754, 0.48376400221706156, 0.47920717844459376, 0.48666332396052925, 0.47271202520637773, 0.4761389444144074, 0.4947188996768708, 0.4975868991088404, 0.48119448285384075, 0.4910700795840109, 0.4746062859110407, 0.49061725681210766, 0.4777998860334833, 0.48595819441590987, 0.4727649677422762, 0.4694981515226276, 0.4918115449906529, 0.4833786574801965, 0.4798635549942309, 0.48044720214939196, 0.4845642101136095, 0.49673123392592683, 0.4994272698638309, 0.49304227938567585, 0.4861581918733985, 0.49977300659112106, 0.4944697509593395, 0.49352719778796816, 0.497581077644228, 0.4814612991226226, 0.49074291890702293, 0.48843829185618354, 0.4941589651725231, 0.47796565310983297, 0.48447636998847976, 0.4913721975347494, 0.4895912520609763, 0.4784382415136256, 0.4922031527150119, 0.4871430890161289, 0.48653269281732403, 0.48658740787492516, 0.491541802086155, 0.4888803961599419, 0.49644420114239374, 0.48342255247173743, 0.482238636419435, 0.475219663674587, 0.482480603455787, 0.4780224957838213, 0.47349296883031056, 0.49454805502650295, 0.4935989301752996, 0.4903890667735477, 0.4914206134752155, 0.49374223612757884, 0.4901197176555827, 0.4849022510435913, 0.4826463344688388, 0.48489372577878, 0.48947156970244565, 0.4744431459916848, 0.49346356018704257, 0.47933409099485563, 0.4901562519908605, 0.48277012541155395, 0.4916096570723983, 0.4862778778165394, 0.4820690525432338, 0.4819627050151148, 0.4948550358007139, 0.48479474748530177, 0.4873304724820894, 0.47874124374911486, 0.484073146069928, 0.49082554387035254, 0.4845083504964758, 0.5032712526071786, 0.47511501572298104, 0.4926297829152008, 0.4871115251248021, 0.4873537135710774, 0.484044175897194, 0.490319972097342, 0.49520871343607376, 0.4866210331103162, 0.4826429435614726, 0.49476820609655775, 0.49848446461212337, 0.4917884642073182, 0.4905882574043949, 0.49168064879915163, 0.4819612827148239, 0.49190314925520967, 0.49363173524769244, 0.49003183427577285, 0.48798223631967663, 0.48608779406167507, 0.4846630944448023, 0.49228713585090683, 0.4784079483118924, 0.5016797236386058, 0.4878924474980342, 0.48758552104742353, 0.4831178752918407, 0.4972199176511213, 0.4958949850880929, 0.4896511502572843, 0.4761154469233538, 0.48044913299221115, 0.48001645221514844, 0.48790128208188993, 0.4820140304833159, 0.48941846265337013, 0.48287030535730063, 0.48462830279741903, 0.4913370541233442, 0.49197987447012326, 0.4996344290263414, 0.49680207739913834, 0.48456787501686666, 0.4899073187746434, 0.49368439571298806, 0.4845833869881693, 0.47485884738713713, 0.482744178530716, 0.4844530237974526, 0.4781096462338102, 0.47528386165374975, 0.48759858462267464, 0.502457126923187, 0.4952060752631471, 0.4902582704305293, 0.47634668715140893, 0.49651294886025626, 0.4824317904674312, 0.48469565167922946, 0.47821117133710017, 0.4855456891138288, 0.4875971642263611, 0.4916949545972745, 0.4850258967516948, 0.5017774969956411, 0.48191598260103563, 0.4901339797802506, 0.4965118789512404, 0.47744826524622275, 0.49375906557153504, 0.484907000940605, 0.4870283131692012, 0.4819728354947666, 0.46009781379711767, 0.4893401170308628, 0.4860914424306925, 0.4896463696447199, 0.4841083966195764, 0.48653397808202126, 0.4819898222091489, 0.4999098858333387, 0.5030251275263059, 0.4851833670035141, 0.48341959606939966, 0.4884296126236684, 0.49190083548110947, 0.4838358279091803, 0.49461002645144814, 0.4917043027521917, 0.4890891096903186, 0.48079497433861706, 0.48719904112769347, 0.48436227975603685, 0.49903460633452223, 0.49518123552283416, 0.49608057931537536, 0.4940495175277185, 0.4941015943217093, 0.49227911946057573, 0.49840787111100676, 0.48480069189922365, 0.48795592770674273, 0.48985112898286476, 0.4849140638958565, 0.47578524930226385, 0.49857930882234536, 0.48289116019659895, 0.5006835853355752, 0.48210466813229547, 0.48249956531279886, 0.4717346347302125, 0.4852785477174619, 0.4949820634149474, 0.48503837654080434, 0.4729092615366668, 0.4927099980575826, 0.4948010737303099, 0.4901568067280825, 0.4824238927856014, 0.4857683092403206, 0.4857630149409558, 0.48579347038815973, 0.4770210621774033, 0.4874896454805584, 0.4916655185516045, 0.49269014448021303, 0.488267293766106, 0.47711617951599794, 0.46872096717154554, 0.48473161376114227, 0.4830921856718987, 0.4930218577375886, 0.4855754282208087, 0.5042822105251094, 0.48331701274017974, 0.4830892556717881, 0.48497181002051354, 0.48718218162368593, 0.48486845989870936, 0.48106485070258126, 0.48238782393862917, 0.486248572248063, 0.49425416632071484, 0.499544462792972, 0.4935880950131396, 0.5012552298730754, 0.4905470608644904, 0.49788809395203737, 0.48681170045128963, 0.47704312986740655, 0.49496324575572737, 0.47574413434186374, 0.4747481790001811, 0.47990929197980325, 0.48695370044523945, 0.48307551456385955, 0.48833683849773646, 0.4865855488736225, 0.48383467171076117, 0.49428522616802023, 0.4877373594194256, 0.49218056966656065, 0.4878073675331151, 0.4908953402360361, 0.4859060504593071, 0.47807130171123746, 0.49311789708575365, 0.4824665951696905, 0.49030752297810454, 0.49388957208326495, 0.4787295380819352, 0.49064793876697105, 0.4832031273063642, 0.48421284921515556, 0.4935150197727213, 0.48222981489639, 0.48147221467282275, 0.4921400942528276, 0.48504060685615036, 0.49190941792219345, 0.4748459455472492, 0.4747588378261461, 0.49709444336538555, 0.49028070269882096, 0.4837786875646631, 0.485429034150984, 0.5050324695021031, 0.4932361378473696, 0.4848287304941451, 0.4806188538581862, 0.49286244395216555, 0.4742572032862639, 0.4944990246048868, 0.49422817119556134, 0.4691925739089553, 0.4991379624764048, 0.49603412877805725, 0.4931978497064785, 0.49062991566592273, 0.5031485282903329, 0.49458483241613777, 0.48632596173716164, 0.4922547907130981, 0.48646308758047196, 0.4996382690702725, 0.4778043709679708, 0.4962600057157793, 0.48523670371945593, 0.48641377701663563, 0.4880490858334339, 0.4845152229380145, 0.48996225961034606, 0.4968780244669728, 0.48978791288456514, 0.49108766050375147, 0.4899992659411538, 0.4850199223202696, 0.4782795970613594, 0.4926865008439879, 0.4969781116036946, 0.4926509265790732, 0.5014910835042222, 0.49462208632019605, 0.49432051779733416, 0.4917909175844633, 0.4767345133457282, 0.4932816919098872, 0.49527565025839865, 0.483117538342704, 0.4924545173346342, 0.4763296637659875, 0.4900317333636367, 0.4911761933879168, 0.48132892406519506, 0.48124261703342147, 0.49847505813788084, 0.48914996949037604, 0.4851221810354899, 0.49322432084225726, 0.4885800442966063, 0.4915171279806445, 0.49089426840506173, 0.48197460229357064, 0.48992683087409433, 0.49511780356275054, 0.49142318443075045, 0.48025043741794715, 0.4866315025214173, 0.49549832978625374, 0.4805960046076435, 0.4960969327390281, 0.48998772599616525, 0.486526035902297, 0.4901795636913519, 0.49170233941888275, 0.4989158404647748, 0.47434929562453565, 0.4853328559116987, 0.4755267907970935, 0.4892658390920339, 0.5022182909145573, 0.47685908491515516, 0.4774313221984606, 0.4951694183147141, 0.49355595321357215, 0.49199278088744847, 0.4792287135326135, 0.49068621377637084, 0.49175807783908343, 0.487072846040281, 0.49450216865456426, 0.504630650239652, 0.4871341591514392, 0.4847103286105594, 0.484685582064988, 0.4879580044765332, 0.4928414614183791, 0.495496437248579, 0.49046793045171594, 0.48859481223836687, 0.4781861222863457, 0.49513453987223355, 0.49482870700935966, 0.4956250349934066, 0.4904776756466624, 0.4828041927151959, 0.49124352240050356, 0.4917017224073311, 0.4916951498295195, 0.48770264328840807, 0.4859598668099997, 0.4949427823632017, 0.4870096284180477, 0.49340803272973455, 0.4882861976398999, 0.49369673284570226, 0.49121171388209667, 0.4923709327257636, 0.48754968085696715, 0.48781266140517304, 0.4949018584994715, 0.4983718049241549, 0.49339545241995514, 0.49037847082993224, 0.48710359018052907, 0.48992638988081044, 0.4833910541710428, 0.4891663150645684, 0.4777148792495621, 0.4846502259277029, 0.48013101451316287, 0.4931997810268793, 0.4844050733885031, 0.48086709559656043, 0.49773255229435315, 0.4836871583846154, 0.507689054090188, 0.48464157444305334, 0.4877404547012848, 0.4887041093829068, 0.48682264503759026, 0.4850643325084271, 0.48137511140663836, 0.4934841630147158, 0.4835006864657922, 0.49255651974366227, 0.4813701408016644, 0.4794216699258826, 0.4891290675649842, 0.4841021232279742, 0.48170451602037734, 0.49166036687695386, 0.4884694194402612, 0.48297255642931275, 0.48627524315733356, 0.4785087305009563, 0.4845411929242978, 0.49104825184328543, 0.4859747681320408, 0.49858460659317916, 0.484636303633019, 0.4931604449518779, 0.4736205268235363, 0.49447802437444743, 0.48623821479518753, 0.4841071756500576, 0.4892350568344978, 0.4904919525916138, 0.4844185847769326, 0.4757717500176102, 0.4861879953805654, 0.5002310826525824, 0.4872183277093538, 0.48038094103323065, 0.49141718921458477, 0.4851094858306699, 0.5030622967594299, 0.48475760958798225, 0.48433988255058125, 0.49869439260439063, 0.492418291251675, 0.4723634092793909, 0.48865563114243743, 0.490766828029847, 0.4959074556936316, 0.49220044291776316, 0.4735786908530921, 0.48461262048305104, 0.49210496277500265, 0.49350590096349684, 0.4930986923744053, 0.48358256155769014, 0.4845560315437701, 0.4827091708019589, 0.5090951562690458, 0.4875757600215736, 0.48647543104103075, 0.4844467745181912, 0.48823873376836013, 0.49233698415454935, 0.49837255953045523, 0.49004302299526314, 0.48841245145396783, 0.49239244143137245, 0.49036413665104367, 0.4910249173867368, 0.48169586247835283, 0.4852480195109915, 0.47897344564467736, 0.4882164543605992, 0.4960076965992433, 0.4846040042419993, 0.4717712117693607, 0.49227937413654005, 0.4811404849156583, 0.5002784983168531, 0.4863539835919816, 0.4751366426632801, 0.4881087048628305, 0.4768511278320909, 0.4722204045798874, 0.48760551397941665, 0.4950735611186312, 0.49576910594284945, 0.481264497715625, 0.48281618386265407, 0.4881647789411235, 0.4798836799586089, 0.4964901085065071, 0.48878160070105037, 0.4826390786362661, 0.4919401792002356, 0.4848548729870937, 0.4982628765492067, 0.48821633144901816, 0.47910135025961237, 0.4802181865483315, 0.48737097115706257, 0.48482861830399365, 0.48663901475205573, 0.4813219759466781, 0.48126653340119385, 0.4968136355458813, 0.5002151662874981, 0.47808485605892187, 0.4836205006207903, 0.47283130467835904, 0.4855702903758603, 0.5003097683440503, 0.4833156967591146, 0.48403095085593717, 0.4892768403061453, 0.4877202823351851, 0.4912223318774039, 0.4949850677477586, 0.4806466714202212, 0.48120598705343814, 0.49879047646472086, 0.48976516754051086, 0.48294765111322485, 0.48553110636862845, 0.49063917787693356, 0.4915373604804733, 0.4784670670972781, 0.49084698681585937, 0.49018463099931614, 0.4931547544452725, 0.4976785580681723, 0.4747205148187572, 0.48181462019772847, 0.49080947250155504, 0.4819474699937912, 0.4809359041619842, 0.48032439137744787, 0.4894392483828624, 0.482340016652191, 0.4775715714782314, 0.4865530138859081, 0.4914045801565918, 0.503594194736509, 0.47549467997481765, 0.4803573793290656, 0.4880080914368593, 0.48930277531010397, 0.4833582121737003, 0.47149449016773065, 0.4912376454942706, 0.4722100564361276, 0.48435961730808996, 0.495523604422763, 0.4895833135721477, 0.4962909022450537, 0.48684234086701855, 0.48098640046031893, 0.5051009147561806, 0.47782916757315275, 0.46205760323807055, 0.48024477324743325, 0.500400209911845, 0.4876201189396006, 0.49484132612443177, 0.47817974601868596, 0.49558460437615676, 0.4921866461330852, 0.4809407826866685, 0.47554736674425035, 0.47776910748044876, 0.4920905164839644, 0.4884483939020176, 0.47815570852443007, 0.47612290506263266, 0.49408652920802154, 0.48965969461322734, 0.4902895323267227, 0.4793534847731864, 0.49021498591840373, 0.4930339925800286, 0.4766033377643276, 0.4672727465563648, 0.4826811718110446, 0.4889619617906186, 0.4904206770617329, 0.4801817104954311, 0.48385788668965274, 0.5007521266229248, 0.4938382697927245, 0.48157500815225934, 0.4950614178100895, 0.49540229288306165, 0.48454043886571946, 0.4801273566731609, 0.4933845350113313, 0.49411746666528117, 0.48991196545462934, 0.48542863543467324, 0.48916893974606507, 0.4855407249715705, 0.49611515880838614, 0.49174729345870793, 0.48786806941173044, 0.48671040285109285, 0.49436143370409247, 0.48101201923772097, 0.4851412656565413, 0.4990999549780622, 0.4968109860123499, 0.47945211769079604, 0.49908661297838003, 0.47571061503641066, 0.474670814127881, 0.49519885995428614, 0.47780456813972566, 0.4860348511844057, 0.4942970375865174, 0.49018757274461033, 0.488204113993891, 0.4896526097367571, 0.48677417162977155, 0.5109519536516793, 0.4780101617659192, 0.4776434679016789, 0.4872818237078895, 0.48544025988171513, 0.4903476993055423, 0.49655339945968674, 0.4931479146601357, 0.4936257554004701, 0.48560659928157845, 0.4868507294657982, 0.49524105581614764, 0.47539968061689547, 0.4886395380507138, 0.49357709666875754, 0.4858914171803673, 0.48639730669328946, 0.4835766615457256, 0.4909978574134582, 0.4856836636575501, 0.4954648831724307, 0.48419748729530715, 0.48845964032026956, 0.47651364786981903, 0.47731608576743206, 0.48771529806334535, 0.4834974484773088, 0.4933853258946185, 0.4838574034100844, 0.4881470219812193, 0.49096976912657103, 0.48154247393338007, 0.4797095531239542, 0.4902753134633363, 0.48541251926046614, 0.4902917743627702, 0.47726434281748314, 0.48633274886895256, 0.4888948382017809, 0.4945556779687856, 0.4895980007213377, 0.48641909135734057, 0.48062616122239643, 0.48307813692988316, 0.47625927402398166, 0.4981226273713111, 0.49981982188339213, 0.49018305909722115, 0.4813073544869648, 0.501370442907543, 0.4897859916739561, 0.49141788883364856, 0.4945380422596792, 0.48325599067087, 0.49518123969281563, 0.4915089523082643, 0.48899345702918057, 0.503808968953987, 0.483520199707113, 0.4788616477770683, 0.4909128569759394, 0.48163728935782524, 0.48903891555710904, 0.48947001944298496, 0.48537478977439474, 0.4737949716234496, 0.4849342923240102, 0.4839846868472618, 0.4887593089353258, 0.49288611727538867, 0.4882302821703175, 0.4854425973790316, 0.5002705009511136, 0.5009610275077303, 0.48420859290888196, 0.4844940385496725, 0.478653655095933, 0.47331080976218054, 0.49868310325292914, 0.491145621099289, 0.4837142073872949, 0.47730146081735425, 0.4784480191410651, 0.49708553767865604, 0.48756136496229885, 0.4890036898420572, 0.47870043922882954, 0.48793999143025574, 0.47581099346697564, 0.49465145487938383, 0.4995903770130622, 0.49272758763964863, 0.4912128209066737, 0.4995914424927028, 0.487767215600431, 0.48165633753484594, 0.4909618026398067, 0.4869170817070509, 0.49291148092538123, 0.4889265891783989, 0.485262041358897, 0.4940681217295805, 0.48323843734516375, 0.4818672159497076, 0.493374895083589, 0.4799128927169154, 0.4771660412857048, 0.484135352580348, 0.4892376094311289, 0.49845564871269304, 0.4876922353370947, 0.4790677272114576, 0.48722614923577173, 0.49217080061681945, 0.4894336823473006, 0.48451086181901887, 0.49326271501999897, 0.48500702332907536, 0.49339584542313875, 0.4886493064083095, 0.49425138493135545, 0.48867635730595854, 0.49820913822128005, 0.4863638412244754, 0.4708657522651501, 0.48652022443336096, 0.48840065180195036, 0.4937413406132016, 0.48734138077891215, 0.492502356572353, 0.48026926117825963, 0.4823596545845079, 0.4994668850291236, 0.4918991975648979, 0.4892644468753385, 0.48711482408226753, 0.48218350833182194, 0.4839003892706846, 0.49490392414998724, 0.4790274992303489, 0.4747910906932505, 0.49892641245101316, 0.4944461835035363, 0.49638294894211976, 0.5000759554143399, 0.4957927039419556, 0.49474738629677245, 0.4777403079611587, 0.48492424481334717, 0.5022833550898487, 0.48241458092292777, 0.48717704530495676, 0.49817760058870314, 0.4894699252030492, 0.4794119686175316, 0.49684683467720375, 0.4869651384125882, 0.48644925994137456, 0.482299992153996, 0.48322062792369974, 0.4924971582528306, 0.48507435065279436, 0.48092565665148834, 0.4928024758002643, 0.4870574305912648, 0.48380460388044044, 0.49777250608184936, 0.48532684463094067, 0.49279384938516185, 0.4930459516320445, 0.49362155126060403, 0.4799485572443564, 0.49632750773407924, 0.49996617326479204, 0.49150321985120327, 0.49431257097775444, 0.49519921502862935, 0.48428125616337575, 0.486639762810251, 0.4834353592607922, 0.4911355550546383, 0.4912423281498204, 0.49101657014990907, 0.49506620316142663, 0.4853817126674342, 0.4920982772133449, 0.4946001445669332, 0.49051466398261595, 0.4858985049368672, 0.48258605545779876, 0.484342591328834, 0.4994717309061185, 0.4819130646882711, 0.49591556414713667, 0.49246603453656446, 0.48963056905152946, 0.4735805670108568, 0.4862568440279854, 0.47665992988164474, 0.4885345979604941, 0.49594435508027235, 0.49334913915436573, 0.4914543519329907, 0.4834050538296338, 0.5024243954372176, 0.4813998457180618, 0.4923141239147608, 0.4816009157687474, 0.4936986499242882, 0.49477278165070915, 0.5064766206672578, 0.5042667522669394, 0.49081586381547326, 0.4937134044595078, 0.4807384636060489, 0.4890562403770432, 0.49397289344649375, 0.48775752833699393, 0.4987194521989442, 0.4779832921081106, 0.47792532768839013, 0.4776299391559046, 0.4955904169573806, 0.48489253158311457, 0.48444282683482415, 0.4811176598903405, 0.48382200825000093, 0.47393824397733375, 0.48934973651893493, 0.4940628899706407, 0.4849166947276742, 0.4879360501175449, 0.4903765246667152, 0.4773257224301551, 0.48067609375715126, 0.4971600620258334, 0.4768448701081979, 0.49095282143100916, 0.4789379622809464, 0.4833691351039949, 0.48503365168921136, 0.4826750020912579, 0.4816013385077677, 0.48840426368891166, 0.4829360177046636, 0.4949811088874288, 0.47237285408100604, 0.48970827594063915, 0.4817502430392319, 0.4824387155130666, 0.4884658380595178, 0.5078856314615341, 0.48420797298785856, 0.49243064237250866, 0.48529727203984646, 0.4901977056963931, 0.4901127578155423, 0.49592556733980075, 0.4816481884732467, 0.47075534958816007, 0.4953248321028707, 0.48753188981148776, 0.48858928414256186, 0.48919793523120886, 0.4953347095374957, 0.49113519913549414, 0.4893047771830435, 0.4923521667406731, 0.49201751438464814, 0.4942798292393211, 0.4982949482870098, 0.4762301335075846, 0.4861058612935678, 0.4911754155741593, 0.4816010818665933, 0.48679329578383385, 0.49336177592700214, 0.4959523754922822, 0.5032334456848975, 0.4945781224993094, 0.47610600698483624, 0.4880824170780369, 0.49391217123924386, 0.4903616038746934, 0.5012970523330875, 0.5002807875093587, 0.478064412948908, 0.4902127791462378, 0.4904582971912992, 0.4760556163450978, 0.4875374622467279, 0.49886313337013427, 0.4897394051323856, 0.48259992166978966, 0.49035601454973543, 0.4828629572897471, 0.48936303615072557, 0.4990261170324611, 0.492310861862604, 0.49128948091136876, 0.4863234182647166, 0.48954499343865854, 0.47826098820585683, 0.5009087893324773, 0.4817497291708628, 0.48457155068998864, 0.48294627075627117, 0.482123335742439, 0.4927713018427485, 0.5022553393922663, 0.4770377553636832, 0.4925580349379659, 0.48607745979362943, 0.4910638144482702, 0.49514480920690873, 0.49508052048816137, 0.49571750234919965, 0.48148809854592667, 0.4853089952562403, 0.49678279591247726, 0.4757814128905768, 0.4910530385786887, 0.47746696232431163, 0.4910859394305287, 0.4796023891922761, 0.47782862188389247, 0.48851149373453207, 0.49277121743637164, 0.4968547552594258, 0.49060992823659927, 0.4937340601952834, 0.4824951753145218, 0.4813790976794273, 0.4862208725691593, 0.4881768989653374, 0.4897426698538387, 0.4853378956117605, 0.4978079740411527, 0.4844782683499416, 0.48211154919831073, 0.4896485574082212, 0.4832443661510735, 0.4799221398269998, 0.49161114651999815, 0.4908292722789147, 0.48880494534451774, 0.5003490751252894, 0.48903929505067206, 0.4888650729499197, 0.49625282442808233, 0.49862779516580297, 0.46814762220973644, 0.5105253375498555, 0.4933915150733777, 0.4875427624535919, 0.4918963524587066, 0.4743287288395125, 0.4872163507476515, 0.4963625075392426, 0.4875169608356794, 0.49221343365706743, 0.4985902952262105, 0.49039431402368056, 0.4884635886166544, 0.4885517974994578, 0.4848041694512516, 0.4800276174253291, 0.4806107666141342, 0.490275885932076, 0.4780383178795145, 0.4812583868528823, 0.49292429378123037, 0.49128550617175476, 0.4776862835453649, 0.4838896503072532, 0.4966109170227663, 0.4785580788197307, 0.5008079694768065, 0.4811749845446351, 0.48658793009333506, 0.4990496341699516, 0.4875622238876519, 0.502515644104817, 0.4847703078555197, 0.48112746070289153, 0.5001280899631052, 0.49875574699342096, 0.4942650151661469, 0.48663855989015364, 0.498242631338735, 0.49232416929058037, 0.4872529904827358, 0.4816853415867902, 0.48584954460908003, 0.4910351985986342, 0.4747646311517271, 0.49267065101630997, 0.48726070670703997, 0.4873846851978835, 0.4838317042385796, 0.48849502632311725, 0.5047895117000266, 0.4806742578846818, 0.48458176540453535, 0.47712838864573553, 0.48039923378678673, 0.4840406972845209, 0.47130132725904406, 0.49387784844341254, 0.4809022699684899, 0.4852623320491915, 0.477799912623586, 0.47999970615080684, 0.48121068855488636, 0.48539819116861754, 0.48664639913634045, 0.4793005561602306, 0.48335197687113485, 0.4886250419623863, 0.4838871229237989, 0.4797116514185874, 0.488894834413466, 0.48652521147704725, 0.5064428337686679, 0.48679213788656106, 0.4834613703886843, 0.49101874421250513, 0.48501035513457524, 0.4814465276069891, 0.49003396374957003, 0.48099035987388866, 0.4931163215977134, 0.483204426774115, 0.48156889652328055, 0.5020505956187993, 0.48891201993825584, 0.47444896271451614, 0.4967125343158453, 0.49090257132618575, 0.49791795553243395, 0.47547019759448444, 0.492302976025722, 0.4942139996519137, 0.4897159114084021, 0.4898614179750664, 0.4886400826198855, 0.48542475760536874, 0.4841936123533636, 0.4848159211311896, 0.490319822671677, 0.49996491408942867, 0.4906865120184276, 0.48601401352548806, 0.4854981263490312, 0.4949331068328564, 0.4819453686805384, 0.4822053798562416, 0.4949836653488694, 0.48923024630487727, 0.4960679778727406, 0.4848633869892213, 0.4875104648555945, 0.47880028894113047, 0.499621737147583, 0.4876142150274216, 0.4893994052319151, 0.4868484115127444, 0.4977200363664782, 0.4923247259845151, 0.4894611472642011, 0.4837444272949147, 0.4942091706041875, 0.4901845109052423, 0.4887078531143426, 0.48570980404414027, 0.4973188236789588, 0.48562184443794937, 0.5003252405800312, 0.48879196811316994, 0.48539235260112423, 0.4819644914349432, 0.4808538158671743, 0.4874530371640079, 0.49187032247752327, 0.49718352880775735, 0.4805538942973094, 0.47939001232761147, 0.48486748516453915, 0.49456245155114636, 0.48858739985792327, 0.4926905070769174, 0.49107938025297576, 0.49154277894937476, 0.48968242563871933, 0.48703285240644056, 0.4860080079989331, 0.47989772006758435, 0.4830702664465719, 0.49312519561170093, 0.48510012278435466, 0.4850511248379995, 0.49041525800717645, 0.4809208265736076, 0.49396600286335884, 0.4872735462957822, 0.478517342936993, 0.4927080799692122, 0.49589677402944016, 0.48519296787476296, 0.4876420349161341, 0.475967384136645, 0.47829995429941347, 0.4896251761336976, 0.489557636848187, 0.4732451671321663, 0.49107107029129915, 0.49111528362082096, 0.5013671074062123, 0.48744505124045023, 0.4892383758515489, 0.4838861514421946, 0.4959021316116544, 0.48418494382857463, 0.48165337807360514, 0.49109059788166404, 0.4928463189420253, 0.4909374243554421, 0.4941070749443733, 0.4764735088411817, 0.47219649320578294, 0.48809092054621006, 0.4907689212450403, 0.5036015422793078, 0.47549721699320535, 0.48271793444327604, 0.48544674235409613, 0.48898297855093303, 0.49768046845185, 0.4878843373006892, 0.4767726493456646, 0.47651363267860203, 0.4910620470219878, 0.48109522821631007, 0.4745259592890898, 0.49220736174443097, 0.48104559409569564, 0.5001374300749314, 0.4943254428808922, 0.4976505727373214, 0.4867578063583542, 0.48564319653540355, 0.4862448835947905, 0.4876717073453605, 0.4857607794795403, 0.47900870161313747, 0.49107185609730125, 0.4827936690364083, 0.4957346168361952, 0.49128739843043323, 0.48221511900999847, 0.4805443004586265, 0.49123207434148264, 0.4798602779119433, 0.4836671093890628, 0.479207599404678, 0.4935767246223403, 0.49402267822809653, 0.4874055274944726, 0.47092568514457483, 0.5055387511238807, 0.48947192564505754, 0.4825701583004826, 0.48225265426817304, 0.4852077875377852, 0.4952589411509954, 0.48914754871815735, 0.4806552698724715, 0.4865497139069512, 0.49012138643726066, 0.49031197871689053, 0.48244776215599805, 0.48300131735333035, 0.4976384789702836, 0.4827246234645636, 0.48792126647803435, 0.5038282535268607, 0.48965363652680527, 0.48950600082116935, 0.48081391523156813, 0.4919243525505604, 0.49186921898434977, 0.48957799793741064, 0.4886047299297753, 0.4774223195675793, 0.47399547137091363, 0.49275679949846535, 0.49542756781390057, 0.4870356963350696, 0.48299141213291125, 0.49439597453515055, 0.49223543109925416, 0.49019179664090623, 0.4920490271228907, 0.4920992782491647, 0.4995056867121178, 0.4855629840133299, 0.4883445209012165, 0.48908908118331773, 0.4866086024826664, 0.4787887527621099, 0.48281452898441923, 0.4825383541644178, 0.4890664779982943, 0.4867419954997946, 0.4836417465549047, 0.4908207375082213, 0.4857370867135374, 0.49280897513676547, 0.4923174404137165, 0.476492010407992, 0.49874944509258706, 0.4825387931134559, 0.49062087194332926, 0.4994584351205589, 0.5051566311717115, 0.48134119367646205, 0.4834048294198125, 0.48846824465593913, 0.48405930747849346, 0.4843631391447678, 0.478366796408889, 0.48002365835434413, 0.4772876932558185, 0.4853436425202181, 0.48632374621936025, 0.4850237783438172, 0.4804280005058741, 0.4919701300090989, 0.4934339263957724, 0.47988043963690974, 0.4800539302921779, 0.49614801503783545, 0.4930906490995811, 0.48755982851614643, 0.4910938526659459, 0.4908068124588476, 0.47057347633129604, 0.48621254169464234, 0.5009245177103796, 0.4836952655197777, 0.49222883240592996, 0.49872316717910814, 0.4894472493906789, 0.46560086909556186, 0.47933476248978646, 0.48496657484363836, 0.47723582558112615, 0.48680294122248713, 0.4845474704688436, 0.49505548875159666, 0.48179946756772624, 0.5010644523755636, 0.5036260877645954, 0.4954788494817767, 0.4933276013493308, 0.486098018242482, 0.4847886342189227, 0.49157254265281614, 0.492789354502323, 0.4777467989017131, 0.4829843186698423, 0.49552154561312745, 0.48507331755691485, 0.48383268705676324, 0.4991393618543736, 0.48002446274707394, 0.4884819589967424, 0.4803390908780941, 0.4892024382006679, 0.4902348294565115, 0.4860109306135105, 0.4800770481032815, 0.4807301491302074, 0.49756849015790305, 0.4933737819163794, 0.4940990483983781, 0.47367672081674034, 0.48979483473589414, 0.4727652067454703, 0.48886730732996686, 0.4947319087903285, 0.4856560981509109, 0.48603646754853785, 0.4885007832061199, 0.496624617686879, 0.48332640900801394, 0.48743929249445256, 0.49931237308800763, 0.4885630030679201, 0.49356358834792063, 0.4886502817812659, 0.4905499773328671, 0.47291526992716304, 0.4834677875219512, 0.4864029363376968, 0.47618003229761346, 0.48034857928261243, 0.49316492381072913, 0.48247042194894757, 0.4984430499620089, 0.4806179484575451, 0.49839483505270893, 0.4843806529358511, 0.48187730036771864, 0.47824905513630117, 0.4840820706518386]
mean_norm_big_MLS = 0.4870540489001939
sd_norms_big_MLS = np.std(norms_big_MLS)
z_statistic_big_MLS = mean_norm_big_MLS / (sd_norms_big_MLS / m)
print(z_statistic_big_MLS)

### PCMCI

In [ ]:
norms_PCMCI = [0.2563439001290615, 0.2114813586513302, 0.22888315728228203, 0.24855241021744812, 0.21259570979146883, 0.21831865095975828, 0.22624415410998489, 0.20638230819283057, 0.2725032700393174, 0.26236712314163785, 0.2199432298439248, 0.17748082345040028, 0.18523915918584086, 0.23833952620043236, 0.2556792659840321, 0.20884948419891017, 0.20535859967954906, 0.20843973906825622, 0.20829165850551395, 0.20891443880093508, 0.20691585045356825, 0.23332382199397683, 0.25642462188488824, 0.22019542773824427, 0.2565722521346254, 0.21318800355515344, 0.26230314463166593, 0.20932531980866984, 0.22562254316697258, 0.2427649190067676, 0.21766702071860397, 0.2645585181907174, 0.22727511358574523, 0.20879098312891364, 0.23696801949211638, 0.2886923820692884, 0.238913835978008, 0.22222055079549205, 0.24487455841196226, 0.23280832901502976, 0.21546084938843696, 0.24679239029681652, 0.23837722089488386, 0.23141497810614375, 0.22461900586605715, 0.22391579843915596, 0.20179690762458916, 0.25203337335152537, 0.24547376735875406, 0.2525424927380947, 0.21427467180984186, 0.2305467456794558, 0.20961940893297368, 0.2378202385217434, 0.23325156360995233, 0.23295204417261697, 0.2689246158569093, 0.22307304363833855, 0.19493591490641665, 0.2499977057150848, 0.2357743607729027, 0.25557269670235316, 0.2431277089859349, 0.24561784356625144, 0.20909693125863296, 0.24992530504545266, 0.18632077902496602, 0.23011565977698045, 0.21725536497788106, 0.2425796458828032, 0.24147295771063662, 0.21570404826512318, 0.25613163095904234, 0.2827070333235195, 0.18136460072922797, 0.21611724955965403, 0.2292681980150247, 0.2620878209171658, 0.21015281662769134, 0.23930338746030005, 0.2386153603087823, 0.26319965734040446, 0.24013022427723443, 0.2449905461745342, 0.22932872297918108, 0.22326726204377179, 0.19861571720618146, 0.2539560658034756, 0.1888641985132356, 0.2477221193618402, 0.23930112688419125, 0.2236400753054834, 0.22631414097851066, 0.28065834075480744, 0.2536685853961798, 0.22156349884727106, 0.23434357240151465, 0.280250871383223, 0.24588311422297773, 0.20510025913579505, 0.20887027800792954, 0.2463771558519707, 0.19575079671784784, 0.24570806518032237, 0.22967030397270358, 0.20798237121955165, 0.21650586770965669, 0.23705751947641754, 0.25569482770574287, 0.24366938007739028, 0.2895977988249696, 0.20563820654555032, 0.19451286111315286, 0.2385821798693061, 0.2316093174343768, 0.19678442164735646, 0.24524767339907674, 0.23267847186739757, 0.2876818339201158, 0.22520732332215643, 0.20729970594570638, 0.2372457000542752, 0.21734269544357074, 0.2236450694023979, 0.28777791361066857, 0.22772341523477768, 0.25680166359504664, 0.26254318515852965, 0.23500248130736326, 0.21079976062821665, 0.22655180334169833, 0.2069671394864144, 0.19651944561991339, 0.23830249045942817, 0.21203994251059957, 0.2323500078559558, 0.1941072329528299, 0.23305752341277092, 0.218125410968671, 0.2499483906833701, 0.2697542434437187, 0.2493724809541909, 0.24850490629956629, 0.21060727485779526, 0.2636956312829775, 0.2659960239095121, 0.2741728660998377, 0.21049331207496005, 0.21524245428349895, 0.21690930374154965, 0.2665768572526771, 0.2090537510657057, 0.23443335799513498, 0.18765526276367156, 0.26808064874612175, 0.24356126443481851, 0.19801903506575796, 0.2588766325080664, 0.1909205807184635, 0.24374499398175523, 0.22825558852729175, 0.21446753544586328, 0.19867557028785243, 0.206390636075021, 0.20862413367693258, 0.26563795470302265, 0.2256667550704851, 0.26598103092110853, 0.2654799786741832, 0.2249147720461281, 0.24582911468337115, 0.22495098238231787, 0.19904020578042608, 0.2851686842965545, 0.22942436498567653, 0.23188115386302788, 0.2784038207287066, 0.23022574393913778, 0.2129567606774964, 0.2300611634200332, 0.2250417895176701, 0.21964981225039978, 0.2504947106605685, 0.2465600560098554, 0.24533332789035567, 0.2609010656224423, 0.24169092162431674, 0.23541089558702152, 0.2327584232702512, 0.25402344368482294, 0.19693228782505137, 0.2118147152522337, 0.24018258754301178, 0.2392461389449098, 0.24675328745642475, 0.26367437001043414, 0.21369342819013587, 0.27986830576861804, 0.2201181099718132, 0.27041877961235816, 0.27671712082731836, 0.18707581744415447, 0.23312947995559785, 0.23866255160893066, 0.24033552230175498, 0.26671562864958653, 0.22348149456585922, 0.2215020198116451, 0.22627248337297745, 0.18900138832549304, 0.2233450419973239, 0.2584022739073298, 0.24937340042654127, 0.2317794772990474, 0.23317627315789702, 0.22459334447675455, 0.2378213719835775, 0.26917641884491816, 0.21097075780979718, 0.24272608117761277, 0.2152935605362452, 0.26202057543872165, 0.2554736945131915, 0.2203117520345959, 0.2445169803307727, 0.1746802923631303, 0.21224563908794086, 0.21610742065230235, 0.23450147442760966, 0.2851827495218866, 0.28052596879169955, 0.2075040019039305, 0.2294279829597915, 0.20502578268012067, 0.197848252607845, 0.24394017576488872, 0.2427114492995306, 0.2602322889034952, 0.19005946441210064, 0.22294980737441988, 0.1944556977456033, 0.20555142040358726, 0.23804617097405817, 0.24304413255260277, 0.2350956026956956, 0.2199364838646691, 0.2315690042366703, 0.21504568185724893, 0.24461111223613646, 0.22907643754183585, 0.24868469599920062, 0.21863613694867515, 0.2414705738247637, 0.24149899644377462, 0.2409302818438596, 0.23477203638774485, 0.22759413970938544, 0.2746855892771408, 0.24808384554382107, 0.2211306882356833, 0.26299261498244947, 0.25566628552601606, 0.21975557583339114, 0.21672785524840354, 0.20072797725581018, 0.2552126626315088, 0.22587549709300708, 0.2738972114268505, 0.27357394270049834, 0.2534728429474588, 0.25978282480284814, 0.2570739879452511, 0.23676257958273844, 0.2199319463076954, 0.2698725951332637, 0.2217270990412881, 0.23835097697238739, 0.2007633227770579, 0.24377743469098592, 0.2560350595796026, 0.20728118288080247, 0.2556393581758862, 0.20111518884947976, 0.22574639435844074, 0.2258657720407313, 0.2764132264655425, 0.25475127940380654, 0.2429479976987394, 0.2441792596397804, 0.22034588944338845, 0.19893565307124272, 0.24515013541990954, 0.23009422526041975, 0.22341961992562281, 0.2240543138215872, 0.22925678369517288, 0.21889618423552185, 0.24479277043060882, 0.28902667761535467, 0.24790579898510912, 0.20082520708307833, 0.24181471167814353, 0.22763310682063864, 0.25517874603336865, 0.2357679855712561, 0.22748151982016424, 0.2341103979896907, 0.23329750493898996, 0.2096047213707368, 0.22767729381778237, 0.20959674647940935, 0.2555331442622347, 0.2343476923674494, 0.19065412844031526, 0.26160029040812727, 0.23832386282103424, 0.24382623048161917, 0.20627608588936638, 0.2347309385202014, 0.21109416351765997, 0.24573541358924658, 0.16907562702776044, 0.2619584504214292, 0.24330680907396302, 0.2450599964209389, 0.23818155251075104, 0.2201054770732125, 0.23403422092397794, 0.18646941476621823, 0.2575610707930782, 0.22191811441512146, 0.19826833054926574, 0.2389673579507055, 0.3077968074294435, 0.2781632428226255, 0.2561063218204735, 0.1738538755686952, 0.23202572535426091, 0.23732300214631746, 0.21995297549825946, 0.2329362283279968, 0.22458186820198675, 0.2845463408587735, 0.22379133133163534, 0.23039983105145967, 0.21094063495887716, 0.24501059440483702, 0.26109908627369693, 0.20138653076422908, 0.26783359606009854, 0.2445717231854206, 0.23244147261204665, 0.2220767745632723, 0.22596167531793893, 0.2689944399817187, 0.2641083739052082, 0.18252970511010622, 0.2118726540956554, 0.26545386114612046, 0.25557682258652525, 0.2750467960163747, 0.2575344226015074, 0.2429101717328228, 0.23227678436154287, 0.23104274425496757, 0.25770422950384675, 0.23689381429969014, 0.1967617023040404, 0.2055279111048468, 0.2639324631630269, 0.24569682104535653, 0.2475249559235899, 0.2170158820804198, 0.23252089973722995, 0.2350034971831561, 0.2680668681642477, 0.2182874008902132, 0.2049296541582764, 0.23129871479906614, 0.22927373826132458, 0.23038677790721962, 0.2517485813866245, 0.23834098750895666, 0.1972413503831646, 0.22761281089800225, 0.21555145004354512, 0.21379665274128568, 0.23015033236406512, 0.2665892888336942, 0.2247391692512113, 0.2091034914919019, 0.25021479991918694, 0.22970216138057142, 0.2695082742152781, 0.228585889729791, 0.28287988634902816, 0.22531264798084427, 0.21223494894775222, 0.29059748174517613, 0.23355105533842624, 0.2745139531307847, 0.24482469802636228, 0.23601692958575385, 0.24288105973036253, 0.2519331298321469, 0.24996619401645648, 0.26275082292560836, 0.236892276775685, 0.262640034844876, 0.26827982726772054, 0.2557277562231681, 0.23366707613366886, 0.215223736571693, 0.2348450686551783, 0.2224520798488996, 0.21978168153900526, 0.24603073939028397, 0.24071299026076146, 0.2590023532262035, 0.2187371302509496, 0.23970532370448497, 0.2970896213026947, 0.21518014226490786, 0.2445972071623622, 0.21352345819901322, 0.2368306207824725, 0.22979354042434036, 0.27563033631420547, 0.17250327465964888, 0.2924523438441496, 0.27062750838629074, 0.20361315386734552, 0.2348588563762873, 0.24813173693863844, 0.2367437367105245, 0.22755004726517272, 0.2062312336871751, 0.22538632528996239, 0.21027855684723762, 0.2159865955207389, 0.2424954369702373, 0.20415643867785466, 0.21115632384482774, 0.22647641497314644, 0.19179362011751344, 0.22608194507246598, 0.23187195994642684, 0.23489768213980589, 0.2421956732432907, 0.22824575248967746, 0.23499975474344917, 0.253956322100389, 0.23221901275965506, 0.20676726293819805, 0.3005099000614017, 0.20718230975407115, 0.22829660764580006, 0.1911637008444527, 0.24462982954865703, 0.24755898702062681, 0.2400308131784696, 0.2081347021679141, 0.25299426620435017, 0.20589101070550725, 0.25489848419060196, 0.20859084254400578, 0.25184864863148804, 0.23445339957566935, 0.22514907173928372, 0.19699239876009111, 0.25031982039138984, 0.21743470879805885, 0.29361073442860913, 0.1887758432108531, 0.23294108522520166, 0.2756282916013807, 0.2649784940541992, 0.21751511270817508, 0.26169930279805304, 0.2539799595394429, 0.22399327787783022, 0.2639794948926195, 0.2043735919404046, 0.19977342715463736, 0.2525077327422321, 0.25260779569595804, 0.24445133379767967, 0.23305138287810948, 0.2436476270037884, 0.219140322325358, 0.23116367052668282, 0.236922086336297, 0.26836999100669895, 0.25565105257727067, 0.2308976167266917, 0.21228878089204928, 0.302582503329165, 0.2299656431294984, 0.18876618380476845, 0.2709316095953072, 0.28056657692529835, 0.2745706872108732, 0.2136610952232624, 0.19513745277801148, 0.21929376966376848, 0.18136371718333244, 0.22041736739231477, 0.24795005908066534, 0.2687192762563053, 0.2533982118362993, 0.2307571910547829, 0.26301294858074276, 0.218119353075714, 0.25320832891476414, 0.267822573231541, 0.211471687231451, 0.24346096740519757, 0.26968553149657115, 0.23338110145989893, 0.24373910211440264, 0.18494624707947124, 0.21211999782513533, 0.19217510622568432, 0.1934336866295887, 0.2563488201323529, 0.2283405814981294, 0.243344922633356, 0.2482401858103395, 0.23425756484956536, 0.21225938656762064, 0.22346422436347524, 0.21124536503878363, 0.2597229999316211, 0.23990899362955123, 0.23909973751658162, 0.24473062914056373, 0.1975082249085892, 0.2622850071834694, 0.2776476192087254, 0.20335536198279325, 0.24156927965971456, 0.22060074549966222, 0.2166594275764006, 0.2296002975425029, 0.24327539166291448, 0.23761445658276117, 0.1978270347217023, 0.20052128176544176, 0.24883055690683864, 0.2583603634718232, 0.22147221584900162, 0.21396572455083218, 0.25037875304602036, 0.27035894936915855, 0.25649233739414073, 0.229133881167109, 0.2533092207270293, 0.20641663113536696, 0.24656421293946185, 0.2295729492814995, 0.2899649681606284, 0.2639748519406349, 0.24821406810568597, 0.23849924043751047, 0.17622840531032138, 0.22732484933084884, 0.2547327938329672, 0.2532321138451469, 0.2176110201401905, 0.2185939112757685, 0.20710878184688092, 0.25101745653437835, 0.17998854107305648, 0.2338801198546455, 0.23620494860758434, 0.2412325914412458, 0.22666212268396735, 0.22851403298329825, 0.1959800572718387, 0.21111044238371188, 0.18375198654486044, 0.2909154961058891, 0.21786483182207078, 0.2409170062769585, 0.27665072056970796, 0.24318769773014956, 0.2565331737578778, 0.22774666128407114, 0.22450446858522957, 0.19828329007692053, 0.23152760810535095, 0.2647211796791846, 0.2501941746387293, 0.24688982223202263, 0.2044606492498296, 0.22667265434187028, 0.22995894252101973, 0.21519323884624622, 0.2635343564503414, 0.23745009637055636, 0.21494066656399669, 0.23874079045713617, 0.22451840697812073, 0.209254714511316, 0.20910089526624598, 0.2655761836165247, 0.234426874244277, 0.2230473537986504, 0.22674972994705947, 0.24666867547524088, 0.2561054871630761, 0.27331807783136314, 0.2505333780958351, 0.24018776426790867, 0.24806617452610497, 0.2582886637057773, 0.24954323916364776, 0.24384791027242725, 0.2196289234152034, 0.20041572821332346, 0.23514390407215174, 0.24932375138541754, 0.20192154001913465, 0.26729194971577025, 0.2552210515651362, 0.213900821473369, 0.2314394526039166, 0.20344120821450196, 0.2725177469714247, 0.23048547262383076, 0.23342402487595815, 0.22896702756471965, 0.25895261259389907, 0.22186743285792337, 0.23486005905021476, 0.22585369458385848, 0.26234713734074633, 0.2808700241552004, 0.25035705069653896, 0.22882923093489033, 0.2455234744153705, 0.21462466004992842, 0.2556144107640952, 0.2215078127067782, 0.19698549427186457, 0.2359783796656156, 0.20582842546093114, 0.2515771789885954, 0.23194442141937502, 0.25508974426118775, 0.25876547918682435, 0.2429293109952715, 0.2482529166652922, 0.27469452801458755, 0.1922974911408511, 0.2762201087412446, 0.2764562845780709, 0.2048501153684946, 0.19753458018184214, 0.20489032201390858, 0.18317590248511958, 0.21258693161741224, 0.19500490200418485, 0.2451726094731502, 0.22581633386284036, 0.24515314195376148, 0.25523237364164897, 0.25136004980205945, 0.21348739388249094, 0.23416456419852122, 0.1973027020519812, 0.21681085330501565, 0.2047360191291619, 0.19643965847196318, 0.18442861801955535, 0.18748320521741468, 0.20409658377292667, 0.25551673585959717, 0.2550709766604165, 0.23146621779453402, 0.20467991728476886, 0.26581905832164227, 0.20947094655658227, 0.24295826475993298, 0.22386093613403363, 0.23002705895019238, 0.26290028480694694, 0.2504107644015112, 0.21856049236356223, 0.2312825466042788, 0.19053737546446867, 0.2448904604361467, 0.22836251469135949, 0.3162060677781978, 0.2290097401864696, 0.22162075399586953, 0.21113581888539293, 0.21046400570354296, 0.2273504029787762, 0.24725171473961044, 0.24633828095015803, 0.21874488742893547, 0.2106163440188201, 0.23903026621800683, 0.2106844444328724, 0.23949236415465566, 0.17349531825239176, 0.19846516384394014, 0.19868116142736506, 0.20629744705358286, 0.2623488806100254, 0.26533694776527433, 0.2460415614533878, 0.2463354938223515, 0.2508197285893372, 0.21205820581621762, 0.2810803105792872, 0.21310127699814202, 0.2645223552126468, 0.22561217603492023, 0.2365014920852229, 0.24087245360396806, 0.21035514084384926, 0.20940701769047387, 0.23305466427381105, 0.24467545241759786, 0.27289919504488314, 0.18778239501776095, 0.2248011887996404, 0.25311526103059034, 0.2237304264730398, 0.21042520172866105, 0.24364080204630925, 0.23836833331293794, 0.2826787724783765, 0.24300142941980088, 0.2417678007874373, 0.25536904232528335, 0.25742441474714306, 0.24239579517090717, 0.2323852666359394, 0.2379726719672455, 0.20147796993811415, 0.24162496991191304, 0.23901985396356895, 0.23641861246139909, 0.2283854033987471, 0.22015620734158242, 0.23086739408904175, 0.19566570863780175, 0.2625589631255514, 0.2143670113508622, 0.27081217052628836, 0.1975663458830811, 0.22357447538821792, 0.1962392054030906, 0.228385996305383, 0.2401158348089115, 0.21783444624734663, 0.22514280395861158, 0.18462251608413854, 0.2707368058641331, 0.23161375112463864, 0.2270723390709841, 0.23191753993536252, 0.22403545936578753, 0.25697150435887084, 0.24057111715631857, 0.21894406663322755, 0.22799321377749565, 0.2645494654005967, 0.26765769743591794, 0.21424008886800675, 0.3009668265650667, 0.30376240617911276, 0.23082642208816603, 0.2719800120441437, 0.25765125105791564, 0.21825258225453253, 0.252394456463756, 0.2248392321720514, 0.2602301549421762, 0.2567311895315565, 0.2419448994456107, 0.21696661814408333, 0.21633303859080563, 0.23313709971863686, 0.1974671797406833, 0.2538857113821046, 0.24655162146864984, 0.22614027564992367, 0.2426006272751718, 0.2069164042842693, 0.23049361909482047, 0.22280613981794306, 0.24305391734300022, 0.21338561414855275, 0.2391945859451379, 0.2290057837183581, 0.22740573818340146, 0.2526352361877666, 0.22852480200819086, 0.24966922564179014, 0.24518850731839273, 0.23444111537172102, 0.24055537346101447, 0.2079188619442473, 0.2134609600618865, 0.2646232464667798, 0.24654880723478648, 0.23629672730432352, 0.21221449292985878, 0.2448774779529905, 0.2641062601272326, 0.21786176319529635, 0.23983939505839486, 0.24975050881541636, 0.2422325026034318, 0.26257254652032225, 0.23169637565076498, 0.25298453521529574, 0.24823281100701733, 0.23770299536760264, 0.2082217490522387, 0.22613476769087631, 0.23986761625354194, 0.2882702211886071, 0.2161165973627213, 0.23117686180937624, 0.23066482200493837, 0.19837200659062007, 0.22302637125149963, 0.2583197092183712, 0.22078999436122315, 0.23227034356566154, 0.2579419262829985, 0.2639556303660338, 0.19060668773558717, 0.2419225230940673, 0.2616557789338924, 0.2405759435315667, 0.21234472154770884, 0.23745168477155462, 0.20206420053481783, 0.20417096824755132, 0.22710617057001725, 0.2253420114520022, 0.21416074598785625, 0.2348462939488335, 0.2025574729404181, 0.23893792480158907, 0.23884558032711395, 0.19807479390522864, 0.22494680344085588, 0.26064052471862825, 0.2447006809257686, 0.206822164823434, 0.2487809043754597, 0.22848021927119572, 0.17764798702994802, 0.23044021854032476, 0.2608872938099658, 0.24793436079497155, 0.20480408891460655, 0.24534642360830872, 0.27169720133259356, 0.2230988738391345, 0.205540752094061, 0.2976036164907345, 0.23706713730043458, 0.23796035623603773, 0.2084865399824047, 0.20615013915304328, 0.2266306830722156, 0.29726911173465437, 0.2534191459521037, 0.2888434420395761, 0.21083835451364394, 0.2943534904538568, 0.26657043392825536, 0.24929857586230944, 0.24824822226944723, 0.23289529497504222, 0.24601953077825733, 0.22547846404365962, 0.24279119988509443, 0.27221791256045935, 0.23309717480168474, 0.2818704773032486, 0.2229045283131247, 0.2781569371522217, 0.23728540089907527, 0.22129984227553784, 0.24372401526920695, 0.28513911930339614, 0.22126054000814088, 0.22371015596628857, 0.24140883152372442, 0.2164821058958518, 0.21770615586312728, 0.23986218046690788, 0.24723006227300126, 0.27150580418408626, 0.268110966386856, 0.19679513022455547, 0.23506510919132, 0.2726288996492261, 0.24905162370148248, 0.20818454685600343, 0.23039486562500905, 0.2488197839097397, 0.20425838454517575, 0.20119911549469877, 0.23674018058653315, 0.21216199229133986, 0.22801108825386732, 0.24091549023738007, 0.23776971694028992, 0.25980118225500526, 0.22502040415369948, 0.22385393367574413, 0.236122869395751, 0.2613527788075538, 0.24507260025227617, 0.2445657287564014, 0.22551104298143082, 0.2357986946697186, 0.21812923548987495, 0.23903776763037116, 0.20490931201134668, 0.2259760166330418, 0.2348551328740777, 0.2609354554198559, 0.2068026122849219, 0.2343295612926633, 0.2215529670834469, 0.24691164265910662, 0.22538352291890293, 0.20061842815626493, 0.2567080904038204, 0.20942494421084468, 0.18353923332640548, 0.24467662785187602, 0.21901584583044073, 0.21441428878206986, 0.1669863387680756, 0.22423263418303546, 0.24076262960844252, 0.28526670576774543, 0.22911172188897264, 0.23550796228218626, 0.2532046214333386, 0.25192565716706816, 0.24103268415144663, 0.24838713957338154, 0.24109642692525804, 0.23883241855537554, 0.22550676859539032, 0.27601571922965246, 0.2701281699592187, 0.24038173504712712, 0.19987303185528446, 0.2187728589478884, 0.21365111963982059, 0.23674629852762627, 0.23864386804404944, 0.24561943228273686, 0.2551711658190562, 0.26932099072831495, 0.2818992818270249, 0.2205997309770981, 0.2550964115080807, 0.2132784843141739, 0.2089489210346256, 0.20877159485782906, 0.21133791241690397, 0.2168913904001585, 0.2564658660285868, 0.22610505525715838, 0.26623122806492405, 0.26222264376462534, 0.22053666714994136, 0.22314766978468994, 0.22916714066864768, 0.25810171987464564, 0.20573697258843207, 0.23142735057353517, 0.2634424924967065, 0.19499948378677462, 0.27098727854055255, 0.24106088681028848, 0.22496204179609985, 0.2660763076199502, 0.2698241372243434, 0.2629936398627837, 0.23959754902835875, 0.25085613192798384, 0.2556043535669387, 0.22939033381167062, 0.256948261568027, 0.2673223552653509, 0.22560640751181277, 0.279440389134749, 0.21105505017174161, 0.23572881997359282, 0.24322464639944133, 0.25852333385675647, 0.2808369854488069, 0.23079876476984917, 0.26547437246341266, 0.19447694491625334, 0.20328510792274232, 0.19700856889449528, 0.2076501011997053, 0.2408323409883105, 0.25967568333825625, 0.2242973354425915, 0.22079982157700015, 0.2551285660205288, 0.2061123080070874, 0.22748614605033055, 0.25243282712215803, 0.26842190268364374, 0.2510238016585632, 0.25597171771425886, 0.1909672229208243, 0.22303264731750339, 0.20715395839806874, 0.2605724950228666, 0.22707004642321216, 0.2534966091879151, 0.21753189003735068, 0.2154820668420541, 0.2110547559210487, 0.2592245399106423, 0.23777831218916598, 0.22527188345788982, 0.28715101060842774, 0.2760821583298087, 0.23511794840070704, 0.24568026190781514, 0.21958172863337655, 0.24273540327990217, 0.21266365509456064, 0.2375677074355691, 0.22124147300740543, 0.20941280354076633, 0.27727809527216246, 0.2474597130355245, 0.23816444572408302, 0.2301122391831664, 0.2302522388365944, 0.18542348289382501, 0.24723389104931132, 0.24006495641332998, 0.29426581747633557, 0.21308426454372764, 0.23925855361254583, 0.25937057524827944, 0.21527634774209092, 0.235619271441595, 0.23865776908541536, 0.23907123237328773, 0.2484230122759222, 0.22007882602101847, 0.22747624914929956, 0.26839751570916415, 0.21354570002029233, 0.25201726483076764, 0.24926619559344798, 0.2370262116296629, 0.2678467064652004, 0.264489282666019, 0.2150886779593111, 0.2434668019521945, 0.2123943872090394, 0.2534616174752971, 0.29293275222492327, 0.24098632887868804, 0.2343239758119376, 0.2073031212512557, 0.16418733818231832, 0.21268033149239268, 0.2908815109697766, 0.25621512142700303, 0.22603608354960741, 0.22245543163619458, 0.23693277384044087, 0.2396588256886667, 0.23706980809536615, 0.18712166534141664, 0.20059247645625605, 0.27523782339796116, 0.21963417249494738, 0.2698874849678965, 0.2865110621719833, 0.2036275517849016, 0.24382973117264756, 0.2603846268912599, 0.22328619129297778, 0.2428624183038516, 0.23990827365327327, 0.21894533593325616, 0.20389125740833847, 0.268101259292472, 0.2524214980489013, 0.19917857404012762, 0.23034926945618417, 0.25664070843554065, 0.25038771720948283, 0.21907080265906562, 0.25177720791346236, 0.2899943618339327, 0.21590980831627768, 0.2845414749055014, 0.2444401372462012, 0.21077372206956388, 0.19994119651937303, 0.21675116384438228, 0.22443013195672837, 0.26025258005336027, 0.22974306008919793, 0.21652157048926873, 0.20425882387426286, 0.23606404317933585, 0.21121814058876334, 0.2630000017571093, 0.21562782434722338, 0.25092802677170106, 0.2190031820263201, 0.2910672934569109, 0.22206183132349472, 0.1922461015424266, 0.2515588622635515, 0.22936329814858325, 0.2702776065521207, 0.2357491353480484, 0.22340370495944037, 0.2577008081484518, 0.287711417513119, 0.2181274710870087, 0.25492449481787754, 0.18758518448778227, 0.2293818508591195, 0.23153815864808125, 0.21771044922521454, 0.26021172831048583, 0.236808438386372, 0.19691908196711766, 0.24139702600790497, 0.21301282765125532, 0.23501292931351933, 0.24775046213985913, 0.2671546293581046, 0.24886803340082064, 0.25905031716892285, 0.22906743688323514, 0.225057990664511, 0.25951625908872594, 0.20894443720217964, 0.21654611284000272, 0.2766245933182827, 0.27800629476241384, 0.24900817554909266, 0.21131039710970778, 0.2583482668965021, 0.23787033611904274, 0.21664065020571785, 0.23178575725424633, 0.23407423093528965, 0.2041604451623389, 0.2586406644103701, 0.23130878824638756, 0.23609867313303956, 0.2506361923755089, 0.2702312383473052, 0.21529006630448685, 0.21554500189443898, 0.23183251297077692, 0.26366354288378685, 0.20719848728752757, 0.2754391182294689, 0.2536025808228257, 0.2326888134966716, 0.23136196871256517, 0.27204934009954823, 0.2660484399467115, 0.3057265282819752, 0.24959191099667377, 0.21333008423451122, 0.2081385732463205, 0.1915194838572551, 0.2277919282790708, 0.25173605299768365, 0.23725569540253286, 0.23072912399889212, 0.23104705633292932, 0.23033479269674098, 0.21121804899838492, 0.21722639515679348, 0.2678141737337035, 0.23064438614062574, 0.22765108526937816, 0.23850683739511708, 0.21258372271031814, 0.21780689314949903, 0.2257412201302346, 0.2302142085040551, 0.22532954093364965, 0.23999082443539854, 0.2087058140738824, 0.23358508078610413, 0.21713353990127432, 0.21256579388206062, 0.22493493207935156, 0.24363370258274877, 0.2442064283753229, 0.22439078057258469, 0.26507762285545766, 0.23722088064657207, 0.20816051244230474, 0.23318459731599356, 0.23360582988956172, 0.25705236953214217, 0.25837097587916413, 0.18791418886431463, 0.21540724564572644, 0.21673692421755303, 0.2354777663035877, 0.20618107291278512, 0.2535130872489978, 0.24416866058394868, 0.2655771396715048, 0.2664156947080293, 0.19743062620153137, 0.23720172265802922, 0.22627601388409482, 0.22242289918564365, 0.2171318298312815, 0.250206850122325, 0.2044849283134917, 0.2458297065730747, 0.22519719460377113, 0.27835712254640665, 0.24891334009827218, 0.24093279504371465, 0.25911035380673025, 0.19200637019716013, 0.22730814828946744, 0.2623158405251394, 0.23812549451433296, 0.2554421477142078, 0.22547281226028443, 0.22238117361414114, 0.2627750979692597, 0.23183403674939046, 0.24187869604816922, 0.22077334189790843, 0.23076047968228522, 0.21125300805997682, 0.25669539910939093, 0.2063369206982872, 0.2483961172374003, 0.2054261041843999, 0.21723101696131433, 0.2400844021279211, 0.2502664072267562, 0.24004284854847474, 0.21643220410901726, 0.24864889639201873, 0.19121097202670023, 0.22870532126227128, 0.2602806621618285, 0.243612877843403, 0.25899542892986266, 0.2262759338980255, 0.2606150845582763, 0.22889691969978992, 0.2533411738868596, 0.21849229251318916, 0.20911760793746087, 0.2848537022842993, 0.22554964266230987, 0.20376690771508257, 0.2160964767552621, 0.24165464170462078, 0.2545195892957462, 0.23333822792981096, 0.23780341894760912, 0.2294928912665172, 0.18599636082687232, 0.23055051426753778, 0.21178835313880645, 0.23763560436094863, 0.20469186009883367, 0.2295016338748589, 0.250521767179698, 0.22760331244072898, 0.21909914216054668, 0.2232893153913481, 0.2657703730143829, 0.22350027060485766, 0.2301820729151246, 0.24113801951188674, 0.28279071325154087, 0.2607487180296675, 0.293428395155451, 0.26144258676150095, 0.2747370029027921, 0.20267836809345793, 0.21393986123513833, 0.2620555129605391, 0.2439697811028986, 0.20833920678868625, 0.24648288449154673, 0.2249494506209501, 0.21327944353414047, 0.2542963290101241, 0.20865551914645217, 0.19642773473460134, 0.24119986103624158, 0.2600599667493264, 0.17611345464788628, 0.20957020320393474, 0.2372201335036081, 0.22521547198820369, 0.26910172771449026, 0.26660437356863625, 0.2527469183704698, 0.21944099209769827, 0.218809935222695, 0.2326887151079027, 0.22942854640179142, 0.2429669315149754, 0.2661559445851118, 0.2617075846393565, 0.24558625219911576, 0.21840970368147597, 0.22081944569565465, 0.23388250989133225, 0.20657357833511616, 0.2279298064201653, 0.19730877530199226, 0.2738768658657989, 0.24555221342872952, 0.25742897785502, 0.24869799195636919, 0.2354589429179878, 0.21446745620486882, 0.20182980656965066, 0.22378305552463282, 0.26076348968419555, 0.22595302164477166, 0.2220852029930201, 0.23037665616990835, 0.24204224017062387, 0.21460994133110858, 0.2073317892748973, 0.2598307093785892, 0.2603741834972926, 0.19868890336344783, 0.22969794736530988, 0.23489795664232155, 0.24948336901515555, 0.2288712343846612, 0.23463726741410693, 0.20501865506596845, 0.2532523857163846, 0.18007412721971472, 0.23468438847985076, 0.2316252212766985, 0.20867479573107364, 0.22918039633080264, 0.2502219567309497, 0.24704260777056936, 0.23218944667294464, 0.22856342021226136, 0.2205535520056714, 0.24404053791007307, 0.22628340268523542, 0.22887909431678108, 0.1972499149749007, 0.2194963673763554, 0.24400289508639228, 0.21775171725646242, 0.28838433126674484, 0.25316489270453024, 0.21861389302060974, 0.24636593663767126, 0.19599288385750763, 0.23626136122882968, 0.25098414186865925, 0.2199836982588776, 0.2091958850046747, 0.2676651169616394, 0.2513020571356899, 0.209084704643339, 0.18107939772950266, 0.22057300820668868, 0.3167421887372812, 0.22128443594026787, 0.2700653349599245, 0.26268797156331813, 0.20674468033594587, 0.2518110936907167, 0.2592616007256654, 0.2505989640291221, 0.22939221817582223, 0.21849897798355525, 0.23024536282530947, 0.20496104099071888, 0.2629381999686537, 0.2522426815628229, 0.2085139227143026, 0.21783862837470488, 0.22652218004059582, 0.23574070731146876, 0.25318825779838183, 0.2854102080710429, 0.2777771115409161, 0.23711720732927385, 0.2595852023208931, 0.23109228471414936, 0.2259485130912588, 0.22366972933968154, 0.206091520123628, 0.20235558648785715, 0.23517068184817314, 0.25658531761362335, 0.28015209421252696, 0.24172730497904044, 0.21586107748209638, 0.26021504156952796, 0.26077420893333386, 0.24776332910529913, 0.2842162731119026, 0.2133707103865821, 0.2911741660182498, 0.23622222558870845, 0.24608204156321065, 0.26649700438013674, 0.23648163293482005, 0.24869744155445753, 0.23891611486766676, 0.21230691155124157, 0.26884981687561116, 0.24758680575299202, 0.21470215979825671, 0.21995801704021548, 0.26094964221625955, 0.2594400734883476, 0.22512698612953883, 0.21105183559711144, 0.2366974392703022, 0.19135116613250186, 0.28504448413515543, 0.2405550571236902, 0.22524080343956607, 0.22889134866651514, 0.2293836082771749, 0.20384948304598705, 0.2074928131291563, 0.27402149188163966, 0.24906693439162336, 0.2122848587211894, 0.1881324132467387, 0.2349324867313295, 0.21334740623258822, 0.2397453926181161, 0.23313465472448128, 0.20650014688621215, 0.2261526805020901, 0.2239777050733197, 0.21641847239732798, 0.23864080031935697, 0.23282443503446126, 0.23067315136573496, 0.21968842534547117, 0.2221587423518898, 0.2355003034183661, 0.24098341565534687, 0.26460825298722174, 0.20967151547216614, 0.22703035682405093, 0.21418996330234502, 0.2557147933200603, 0.21554658084199405, 0.20023515205435327, 0.2235019518768282, 0.23706306898602378, 0.3036917605107967, 0.22113364204323774, 0.2262920165110636, 0.2375622075199301, 0.2175636937354593, 0.21008099240759762, 0.23233384236375493, 0.27501236766824655, 0.25943659134617797, 0.24980149498637375, 0.21734081571646938, 0.28849030117178986, 0.2286065152783803, 0.257890283789415, 0.2782745056909462, 0.2958435723686156, 0.2265653822878606, 0.17184904195345962, 0.23486057863502177, 0.25966187859174983, 0.23084022965858123, 0.2159497398011512, 0.27769952503164635, 0.20394328212937368, 0.23725836093965608, 0.2918976980950515, 0.2383158720059075, 0.24555618211917984, 0.24531327787884039, 0.22825001838460335, 0.25502689598233014, 0.2312913117764771, 0.26854479896653416, 0.2733483700134192, 0.26013268539500245, 0.2288305165009413, 0.27776553886087785, 0.2170292337229592, 0.18397833009057576, 0.2945752338871674, 0.2045605952771906, 0.2307533363148831, 0.256968536841497, 0.24263115795070106, 0.2649951671464533, 0.21445639448068363, 0.23047583148124062, 0.20143340259978623, 0.21551618724603347, 0.21241243589754372, 0.24630761245032856, 0.2658254457070373, 0.21385817344169622, 0.257596656502115, 0.2539138307867077, 0.2741374596991549, 0.26824116244731494, 0.2491836638686212, 0.29359701211416045, 0.2376026323662341, 0.24156120571801595, 0.25283085941373223, 0.2222112476371662, 0.3016946202071256, 0.21136205580886822, 0.20978852190602773, 0.2446918461407984, 0.24578658624147923, 0.27786346270692824, 0.28600682043661935, 0.25109522160238595, 0.2188688173508151, 0.19456672625280994, 0.24672232477130834, 0.23686193738108757, 0.25029063421459535, 0.22074716518614065, 0.23968782692691964, 0.26874826113145067, 0.2379941018987478, 0.2657068209651102, 0.23762000827236684, 0.2706618481856789, 0.24071101887427704, 0.21666727657222348, 0.2252173644006993, 0.2367005455053793, 0.21795846338795655, 0.24977461233644016, 0.18723237451359298, 0.2373356757106158, 0.21221878703128927, 0.2372641880950332, 0.23034450277479593, 0.21851101448085058, 0.24134136712465204, 0.24930246392928465, 0.20826987188248638, 0.24652562921785012, 0.2233796284738982, 0.26705528263075123, 0.27403478841585843, 0.2031498223052441, 0.25067311749320303, 0.20760099300814797, 0.2597853242775143, 0.23069291860192717, 0.25937154675988094, 0.25480803517809814, 0.2863564239996627, 0.22671315338022874, 0.2329612181465647, 0.21087508249390488, 0.2090448784458906, 0.19059526274817334, 0.21586509071720114, 0.26203554898379544, 0.28353631418056807, 0.233868845405175, 0.28270807017424654, 0.27422186794391745, 0.20907602123015262, 0.21852315447646756, 0.23204138045765715, 0.30547054189882167, 0.2580585180537729, 0.19358978138626287, 0.213293412695739, 0.2856569376717267, 0.2602564791502358, 0.21439323613663006, 0.19269894534663154, 0.2397220591493798, 0.2328909862202104, 0.2464671211274197, 0.21797352122922323, 0.2717781693428112, 0.2630103438135635, 0.21628573379472668, 0.27983019041860197, 0.22094920497662596, 0.2531419679967519, 0.22442466745759457, 0.2579326510296259, 0.2288998708812092, 0.220386072378645, 0.25510006369895727, 0.24537480855935545, 0.24491707604514423, 0.24113502453095562, 0.21461954747165188, 0.2760824515525474, 0.24703833560163652, 0.20431574104226802, 0.22589311461030698, 0.18578624719993933, 0.2383831751259895, 0.25148909480699094, 0.23725875789988612, 0.26354250795290607, 0.2161451457573069, 0.2570229583710447, 0.21501188194306814, 0.24099080263825995, 0.24511159499779545, 0.2545886629772083, 0.24554540149805884, 0.23157695561977865, 0.28727095214414916, 0.2787840307983454, 0.21110962724567267, 0.24840514408497175, 0.24292583387704383, 0.25598473074627337, 0.2699440603829054, 0.2135307332569133, 0.22636463372160143, 0.23843313995183096, 0.23267734579977145, 0.2419150309678638, 0.2278136104727529, 0.24454633560438485, 0.24396614875540285, 0.25808546610924943, 0.2720839279748894, 0.214025916491324, 0.2321721984880231, 0.25494548957294116, 0.25204417134308404, 0.245702182812638, 0.19272371940812472, 0.20132626436079404, 0.2544085419382531, 0.2645648463826303, 0.22660795385114432, 0.2344135872519642, 0.22363040581815027, 0.21747305937478803, 0.24856610631146517, 0.22267575908212614, 0.19216754806127387, 0.21627928668363064, 0.21398480536126288, 0.2430548331264192, 0.2503086152247377, 0.23385067510069882, 0.20956937217629804, 0.2519114332016212, 0.19350297386874454, 0.22502553273889217, 0.21284294832292525, 0.2322207434756688, 0.21357643613899335, 0.265030373389828, 0.20795938536981506, 0.2267828840213985, 0.26928117197557877, 0.21603878016592729, 0.18723476115109497, 0.21886178182481317, 0.254416283690541, 0.21602713631703707, 0.22962837678542175, 0.2575858143416664, 0.23717811078708959, 0.2578470646175565, 0.24921401144439462, 0.21353825058963305, 0.2179191236640797, 0.20883044355715916, 0.2580024778255588, 0.24591236333590166, 0.24965479085128953, 0.2367373832841219, 0.2606967995060894, 0.28542552965007906, 0.2222341966474739, 0.2405006424487987, 0.2419972376345527, 0.25438653583165277, 0.24773109464896786, 0.24260582963531888, 0.23576629588557366, 0.23278994398338032, 0.22075888433645058, 0.1886254343446658, 0.24958160761327, 0.19208508423134849, 0.22198710878332195, 0.20731017319575445, 0.2384425740775635, 0.25535908391318446, 0.23718132006247034, 0.26048268475914554, 0.2542069649094935, 0.1996857356480038, 0.23247518255702512, 0.2855887489091575, 0.21308666452764624, 0.23972457435397973, 0.23023778052353727, 0.22321614642939186, 0.21677552647244522, 0.18732365501898735, 0.24827853268946973, 0.23538469508268242, 0.27688102610316406, 0.26808298832713967, 0.2225337212474783, 0.2072179672015686, 0.22193853316110815, 0.16387775474405583, 0.26933034307610165, 0.22349486427931398, 0.2662863893440795, 0.22306144808228986, 0.21007890875453722, 0.21340084569115705, 0.25448599839374986, 0.2578977559200105, 0.21421831345667514, 0.285765972852814, 0.20122403982959589, 0.2285905893611279, 0.20447366270811057, 0.24249300338713878, 0.2159960103071305, 0.22870270734221843, 0.2624898302696523, 0.24173149193466098, 0.19474380104877176, 0.20459305001020686, 0.22121990446680784, 0.2469413023773296, 0.25289366112851885, 0.2088261194285717, 0.2230229520856252, 0.2323080868348352, 0.24903015200421688, 0.2386350543089003, 0.24442127717196152, 0.2324178351825235, 0.21374033333745981, 0.21305372282964485, 0.26856618314087016, 0.21372850995234902, 0.2727140176690431, 0.24799466766954903, 0.22240093146632314, 0.24891760871132232, 0.21771593752054974, 0.20300254778711083, 0.24102431193611976, 0.2644847580518415, 0.24269332008729586, 0.2423191341609454, 0.2239487011331734, 0.2259686551829761, 0.22711725333101654, 0.21405157065522865, 0.23655409916667788, 0.2908225009592127, 0.26638770771142223, 0.1761175601615117, 0.22052214477774196, 0.26225516734456394, 0.21967397873493613, 0.2204857479217793, 0.22656937845741185, 0.2786901345330704, 0.22434116880154192, 0.25875209027320817, 0.27433955730835957, 0.22305570495262558, 0.20616929391128408, 0.23359751150858543, 0.21385449143640697, 0.24905871533809967, 0.20580296378441804, 0.2359752322529508, 0.23765558889718794, 0.23684980942866862, 0.2434414538895326, 0.21198334642075303, 0.2303663646405246, 0.27128656252876654, 0.2812967120759153, 0.2515653805958604, 0.24300916850952112, 0.26853988564577685, 0.22488351927514447, 0.21944711660278501, 0.22704260767123385, 0.25020939201955805, 0.17892177298169565, 0.21083502949588448, 0.22983721882132777, 0.2247401627039157, 0.2973407079674882, 0.27442299930677244, 0.2513238833616417, 0.24503464572696193, 0.2404547303251715, 0.2240899971896178, 0.26945687432064347, 0.21286479977155828, 0.2778848155178862, 0.216473894055748, 0.21326256393663495, 0.1954569348504102, 0.2167627018590901, 0.19590776236963356, 0.23491381944653517, 0.26093175316810374, 0.17962322011726192, 0.24109558560732167, 0.21111574491402924, 0.27463060350342583, 0.23952989846771924, 0.26387962833226447, 0.22157140963512703, 0.21794452226324743, 0.22413665136603805, 0.26360316629340114, 0.24263349988679078, 0.20228507041092336, 0.2616847381285575, 0.20316377483691309, 0.23981613016954878, 0.2479854861035577, 0.2139671366672325, 0.2399666282552152, 0.26098317547395533, 0.20133166302330532, 0.2522889467566282, 0.20331438301138224, 0.25169385388833804, 0.21080548454121725, 0.21901065368612707, 0.19586228335812042, 0.20565437888505278, 0.23535566952598239, 0.2457747325276794, 0.2481829926948212, 0.22412354401184653, 0.20693165743327765, 0.22026350783301318, 0.22555632762971303, 0.25028573027160356, 0.23536232957683018, 0.21127072565422836, 0.2626549970303918, 0.21918194001708202, 0.26079661005652144, 0.24180649963808543, 0.25297785643079973, 0.27331585608767317, 0.21634245206825028, 0.2241066883833242, 0.28427728309264616, 0.20723701313012236, 0.21949705616347712, 0.21551274809015264, 0.21749937952121728, 0.2534718536309732, 0.2684196436748779, 0.2309183627335762, 0.23551685146203458, 0.21557933808495466, 0.2900510204284849, 0.22581137257024958, 0.22928829689157854, 0.27099511783484653, 0.23155531525916806, 0.276409405538442, 0.25058987300216645, 0.21359340121238515, 0.23259110416151382, 0.22824407985887182, 0.24955212925415846, 0.24849006568834878, 0.2273074916125688, 0.27022915129074365, 0.1695313839016502, 0.19357164464225132, 0.23509637540674574, 0.20116859368551482, 0.24292280187243365, 0.2233669026129464, 0.2087743811785976, 0.2405643540238039, 0.24998620093112298, 0.23455072326325252, 0.24276464954849486, 0.22895557777117567, 0.24046668173521238, 0.223580785572762, 0.24739797909383204, 0.22721139822258876, 0.2522967344683664, 0.19911548090870215, 0.2697947614910886, 0.22533515294467144, 0.24816095053092088, 0.27402180648076413, 0.20594286239291, 0.22889728271267665, 0.25027830905042725, 0.22850816624403125, 0.263668531806765, 0.23524614486534295, 0.27604656077287054, 0.23130945822265378, 0.21538732937807561, 0.21307324355685467, 0.2500664480688207, 0.23801942361201325, 0.28632161600120204, 0.17720197412865396, 0.2600870908925032, 0.2163978835499951, 0.20478792292657824, 0.26025514430506735, 0.25746258497008334, 0.20059132474877975, 0.19616353412965307, 0.28088007882508137, 0.20326350032421006, 0.2775003836595202, 0.26010432942998435, 0.20934255267603233, 0.22161562579254404, 0.24993681052443598, 0.2413914058922199, 0.23526873303578721, 0.23961576692684336, 0.2127305987184337, 0.2580304421168487, 0.23158029829200222, 0.24214783227639014, 0.27119582897706085, 0.2608234349846627, 0.26845972551562736, 0.2821467000482472, 0.22563077181616129, 0.26683673832504035, 0.239800000640136, 0.2500202004493123, 0.24423418344987788, 0.22299513613966748, 0.21353690790013713, 0.21853573527163203, 0.22969078866351772, 0.2812789856817973, 0.21760382958215532, 0.2306929967862861, 0.24865341853976705, 0.23222098214745918, 0.27276729734163546, 0.22084380038093832, 0.25112755966491523, 0.22407519912246088, 0.23037480776873562, 0.23529945068977362, 0.2337380815685748, 0.23994265632396522, 0.26559562998236724, 0.2297609902110221, 0.2063905550128535, 0.23099432086541669, 0.22509375767434653, 0.2376168210552793, 0.272835715791201, 0.20353379451080647, 0.30104234698001425, 0.2443650688501725, 0.24607501935219722, 0.22157712755708925, 0.24832423174029342, 0.2634625442821246, 0.21651803733321823, 0.26345454896250553, 0.2100281757277444, 0.24482452866135254, 0.27301612606008757, 0.2572766398773259, 0.1882452950384877, 0.2548084652290015, 0.23397226651419983, 0.2490793264408392, 0.2696153716915663, 0.23149020473116036, 0.20580066187166576, 0.24056055743972518, 0.29725946903626044, 0.25402026030071473, 0.23051080095711438, 0.23615860629812477, 0.21720303557418502, 0.2036392940851212, 0.25955121910695705, 0.24054958537828577, 0.23166021476325332, 0.23055898980256834, 0.23747553102215144, 0.22498597344145393, 0.22703488030736788, 0.23750665682491015, 0.22873286881845126, 0.2645712296000456, 0.22283339441389066, 0.21941102471784862, 0.28926978771533896, 0.2420925856591107, 0.23845208142005547, 0.29959645304825855, 0.26001612136100544, 0.2066997657076593, 0.22568970935904087, 0.2353312907993289, 0.23454685336090647, 0.2362251246624904, 0.23152545213691972, 0.25319277634250714, 0.21819798241496177, 0.20741544418028948, 0.2372515067713139, 0.2345468853811458, 0.2191395902570782, 0.21532087957359608, 0.2572602064120361, 0.2316003209657139, 0.2084010150368381, 0.17610867260151297, 0.1976246763747575, 0.2267215538917536, 0.22615389055270468, 0.2270727229495534, 0.24919081631256246, 0.21509288837690468, 0.2026544395450175, 0.20632696370760298, 0.25197514996812603, 0.23330443649216853, 0.2249363486367302, 0.2605303776117354, 0.27982365140130444, 0.22213209365535383, 0.2593668363613138, 0.22899782378102532, 0.2434830440556591, 0.19131199767533572, 0.16917389332433808, 0.2171177043581595, 0.22775596498721865, 0.23479675486717852, 0.221207521153931, 0.23294675370281492, 0.23393370806730462, 0.18438457596946578, 0.24215644470706293, 0.20670890849560888, 0.22546743612475223, 0.22904054192309695, 0.23580772537379252, 0.23760741262789534, 0.18511123989721956, 0.255299860311189, 0.2685167968933705, 0.22639059613082727, 0.2547533098568802, 0.2836019151520105, 0.24419093503775816, 0.24409696888136245, 0.2446935861943676, 0.23108198200528732, 0.24046346820411788, 0.23066303745979913, 0.27276452988809036, 0.223470271184592, 0.21211433376574085, 0.258373233541848, 0.24030666266039072, 0.2521064073836197, 0.23173525468481268, 0.23093510850406432, 0.2589515652773543, 0.26923793530361156, 0.24453312045896228, 0.2269113753386043, 0.2467075797971648, 0.21341807465255402, 0.22616581244831416, 0.20789599099733794, 0.23801992919719822, 0.2323260358439361, 0.23773031531359312, 0.18444747806903639, 0.26463816795624767, 0.2388841237345463, 0.2250001570790079, 0.21718461510982348, 0.2597362009296101, 0.2879208415414578, 0.25868652182914265, 0.17966110958757628, 0.2532145846511648, 0.21866121564164454, 0.23296388963372963, 0.22603475028301398, 0.22127208549298, 0.22239111752050778, 0.2973584492887491, 0.2731713658026435, 0.20729323244863185, 0.26374353174586485, 0.2394214430351578, 0.20759477351129235, 0.20712120154104915, 0.22644905672268056, 0.25087191203160564, 0.2656839990764226, 0.24775550781096023, 0.221034733267214, 0.21192942657495953, 0.24908475670489585, 0.18648965340288995, 0.2624834294907477, 0.1965834048116251, 0.25865088569544337, 0.20692999098684842, 0.24600829663926313, 0.27980523390874196, 0.2217914833839276, 0.2094092859234109, 0.24725433611167918, 0.23287571979186222, 0.2057651021234668, 0.20720641400454504, 0.2754224766164434, 0.2369146763632924, 0.21569233952819852, 0.22059987272351494, 0.23502688835439947, 0.18789581054031237, 0.217466922389553, 0.21112059313268786, 0.20421117421525617, 0.2084780979276778, 0.24093991207740734, 0.26682411415637675, 0.2687603167302092, 0.23980728792619296, 0.20675890183367313, 0.24878065803184063, 0.2591370039371211, 0.2158429320728625, 0.24116249407552992, 0.23417821732970842, 0.22502428057213816, 0.24182923235942222, 0.22029074731166087, 0.23093180054984944, 0.2111019253227211, 0.2689847529017836, 0.24147305885381898, 0.22158010275027964, 0.2100368899303179, 0.20479402659028098, 0.19067285046112414, 0.27392653593090527, 0.2089518417161629, 0.23276018634054207, 0.21841357440419845, 0.27492715020268466, 0.2675133000214917, 0.22167097406952152, 0.2609038304755628, 0.26933311269585214, 0.21371537908997462, 0.2352038567096113, 0.21941306123198565, 0.22263721443066742, 0.20055336371233887, 0.23953540731654863, 0.18642104625415243, 0.23013369271912693, 0.27428183820526403, 0.26239689488411705, 0.2249761085515152, 0.22628510739028135, 0.18812799875019012, 0.2335214139035889, 0.2340247064289686, 0.25798414790769336, 0.21691643056205992, 0.2076849537757255, 0.23881320701887734, 0.23871763144894712, 0.2852896693077045, 0.2560124953545042, 0.25212969799347923, 0.2674068818449425, 0.27319568200862127, 0.250216962582883, 0.22544153236288697, 0.20684058069448816, 0.27428729739593544, 0.2023575478069612, 0.19255196526758966, 0.2462278497286482, 0.2263469467643531, 0.2560253146993682, 0.27270061664784145, 0.24211336688721627, 0.22516018819271266, 0.19783816438788684, 0.19322895053366546, 0.28264188226162656, 0.28547516229191283, 0.23045292024187336, 0.2940556470938555, 0.2112174684169582, 0.2190632448573856, 0.23119862891613532, 0.23049882785104062, 0.23057946523072462, 0.21745927021117334, 0.23699516798132636, 0.260636265831087, 0.2443091370636545, 0.20974002225570157, 0.2609689954051934, 0.24218198191627036, 0.2574769331356081, 0.2547682531294927, 0.268263783991766, 0.2508739295288674, 0.209745136652602, 0.23626833652614876, 0.21217654557663398, 0.27650272034301154, 0.22392575862693237, 0.24315198496151771, 0.24369106308119792, 0.19243959425679513, 0.22258646248161199, 0.22034993879224257, 0.2424844183578911, 0.26300498769576014, 0.23715582501683255, 0.24470840583853987, 0.28299240378057244, 0.2302160816706384, 0.1898529071721099, 0.2230871750905207, 0.22045806433924156, 0.1752587747049298, 0.22193334895780448, 0.23016012692504798, 0.22605009423887318, 0.2300035828797028, 0.2065820804980076, 0.21302559822383435, 0.24299421585161896, 0.19865068795144672, 0.27436726765566083, 0.23824892749406898, 0.25077082176158405, 0.2516363465945196, 0.2211191829278086, 0.2537230521983218, 0.2647693570573189, 0.24216004719607695, 0.25564833548283095, 0.22753968641822162, 0.190908601866266, 0.22793738355423646, 0.2668050588247245, 0.25270035487448983, 0.24207647023930653, 0.204111351732788, 0.22673130678949022, 0.2780805809923352, 0.27501002789425066, 0.23289031638401267, 0.2475149006261623, 0.23745158711173953, 0.20942260215856512, 0.2504016046551299, 0.2202902963919775, 0.24467404232810236, 0.22753574015275196, 0.19762215173165487, 0.20837109926982442, 0.23687843022027533, 0.18395960415351403, 0.21332336954302997, 0.2805339466030317, 0.2452815363655628, 0.2540283950862569, 0.25339424286106915, 0.1967342368841388, 0.2342142386295428, 0.2609561425187112, 0.23469838508486643, 0.2302138717002059, 0.22955779319638733, 0.22775316595384137, 0.2801371952860779, 0.19549580690891746, 0.2256468583929362, 0.23778725156892763, 0.22787877030461942, 0.19000949965671488, 0.28669632171928766, 0.2559728741220121, 0.2742528841013127, 0.2363518397067256, 0.31379109906981173, 0.2806557917326892, 0.20469500925739467, 0.2196187269469223, 0.2362723579266159, 0.2168262264542279, 0.22948419366192233, 0.19834681492671485, 0.22171841353370578, 0.19019799027549902, 0.2482228980447305, 0.24544152149225842, 0.26454127048033055, 0.2630021729713843, 0.24896132716573877, 0.21403597451264644, 0.19948525178926207, 0.2549350312810508, 0.20428738108233047, 0.18929634695018083, 0.24712828056524272, 0.20998707256202848, 0.2338116627620567, 0.25565731312892676, 0.2524327777682393, 0.23325388316676324, 0.26027756711692557, 0.23893223667090344, 0.23196902281195647, 0.2164227221223275, 0.22069494476420906, 0.22631380408332122, 0.234345129650321, 0.22473054668783699, 0.24632307290557418, 0.2422426364723235, 0.18833942007684554, 0.268324825006483, 0.2116358972430932, 0.2176505319722125, 0.23617803447379046, 0.22998807321194598, 0.21709851147471845, 0.20534323653853473, 0.21484093977742977, 0.22609357694089965, 0.29039434109142426, 0.26984298152416514, 0.2772552366740738, 0.27192152825148763, 0.2794965623951606, 0.18531031070856305, 0.2560583909590339, 0.23569849815314686, 0.2480195011604219, 0.22384116479515329, 0.21147157468779645, 0.28835060637505505, 0.22535259401896607, 0.2200286247329492, 0.26523491562682844, 0.2450008770419492, 0.2431336209760948, 0.24677820460276834, 0.2679122366759895, 0.25746510551081314, 0.28584062448944797, 0.2684019252876288, 0.23884970087739846, 0.22128812578290608, 0.21868562825240287, 0.2584314919950625, 0.23103273900045865, 0.2432405233613923, 0.22964853177377278, 0.22150544067442834, 0.19440034799996692, 0.21207933429527728, 0.23185738005887177, 0.21379256697344984, 0.22415385251967263, 0.23537499990048655, 0.23576483152595262, 0.2209164706137647, 0.28674784403280235, 0.18507810968277533, 0.18361450447402772, 0.255273614131548, 0.2600439080953895, 0.24396750940915365, 0.17852691184735872, 0.2484220983367113, 0.23664616953025105, 0.24310746762772958, 0.23549509125489937, 0.23971993883631268, 0.23752681978303544, 0.23105148369791315, 0.2917847453931481, 0.24319645054325104, 0.2517652509366208, 0.2529091207011235, 0.21082988526749316, 0.27908407451029243, 0.2575388329477136, 0.24584113906734664, 0.2369604866823241, 0.22191160501201204, 0.2395211960716244, 0.23460551571998106, 0.24993099654548057, 0.23488635206372838, 0.2822489210791506, 0.23493115093356265, 0.24516093244134332, 0.25820621348123884, 0.22591196961011448, 0.19858424536053468, 0.24617273219238622, 0.2598505135464523, 0.2669589349594366, 0.2772963289260269, 0.24959987284389892, 0.2348644793582821, 0.21822213578156063, 0.26782624878134087, 0.21185608950063192, 0.25556685703564147, 0.23659382660462452, 0.22987189630679963, 0.24926529976705009, 0.2916563206718334, 0.214780928009429, 0.23573019295505562, 0.24902024126612496, 0.24041781268900267, 0.22563523727446075, 0.24198657150883846, 0.24215667866790874, 0.25866990181716826, 0.22044379803044134, 0.27056090758126833, 0.19732663479039553, 0.22482974510101134, 0.22584708799376427, 0.25530541439887794, 0.22079519351498067, 0.20086973018113277, 0.21374114243028752, 0.23704690155752386, 0.23620638053160542, 0.24506454198749777, 0.18442038490081344, 0.24709574258386932, 0.23014183074127886, 0.2558227099943858, 0.2247767230099656, 0.22492680160183318, 0.2071563252355536, 0.24445000949078263, 0.22215501531475637, 0.21856441843146893, 0.23135553572657933, 0.2890520043448075, 0.26030584031140325, 0.27302654479406513, 0.2063092042113942, 0.22335301051469386, 0.25867298959697743, 0.2527684194321953, 0.22895930396512346, 0.27081762648632246, 0.17202382702611332, 0.2374481992862958, 0.1827451173627236, 0.23468057453763747, 0.24536649810727507, 0.21825554374316355, 0.20563836695582158, 0.2684731019111875, 0.2749744078481919, 0.2405593113860749, 0.2486871857813625, 0.22640764400731758, 0.24495224494631393, 0.2160351558956316, 0.22985385170850756, 0.24222866249187902, 0.21747966042579778, 0.2723161369748813, 0.27436072723120175, 0.260331319227736, 0.22961696896129238, 0.2400387632842454, 0.28151254643650003, 0.20191269277842527, 0.2137213996171911, 0.26742302975451526, 0.22887035570962538, 0.2230210876448779, 0.2063015307954065, 0.20111989156084698, 0.22442536153631293, 0.223851789998375, 0.22681391790338387, 0.24919032578666594, 0.2543536785396754, 0.27004914692275833, 0.21526886056684844, 0.28371960944822466, 0.22801823352796197, 0.21499526578339329, 0.22585799077435542, 0.2758919081755031, 0.2620878699828506, 0.18506914287135448, 0.19887385065378735, 0.20914445434820148, 0.30190664427079483, 0.23762513038277427, 0.2003086350843207, 0.24376578313463249, 0.23593358794155964, 0.2669565748712605, 0.23658947584218668, 0.261977408642341, 0.24755640737678547, 0.1898397525092413, 0.23760767351794748, 0.2051925281253956, 0.24838882470292323, 0.17867622003711617, 0.21209216376306644, 0.23013806990330596, 0.23820438155681758, 0.2709107598680812, 0.2117786658035524, 0.2643134984646396, 0.2371100804686506, 0.2235454381576563, 0.25412716427739074, 0.22340330107333395, 0.23071820471287338, 0.2591407335636254, 0.2119935561255072, 0.2382912539878464, 0.2601366086471357, 0.20309623262504015, 0.23528299790897675, 0.24227320138447067, 0.24099771117608926, 0.23842809547635557, 0.20301287242651753, 0.23150907310087648, 0.24115101267350653, 0.21863913750503144, 0.25551652137472547, 0.22112236707080948, 0.22427830043388347, 0.24373135815465896, 0.24873554313965932, 0.21944537469785158, 0.2895409895594109, 0.25327332758621335, 0.29584362093216837, 0.23058908178518636, 0.20249317370232872, 0.24926676150028648, 0.2272431414070377, 0.19026011165612924, 0.27955137137385716, 0.20945326085723737, 0.23289719246699048, 0.25136533799998567, 0.2438932153438131, 0.22603721434876803, 0.20865558502072268, 0.23248852338790768, 0.26083202606080286, 0.217672156057901, 0.2808552181011819, 0.2565320207971985, 0.23339079203520513, 0.2498939701904951, 0.25602710569726717, 0.21120506618139123, 0.20942599111396265, 0.23413597670026665, 0.259013964766407, 0.23597159432829168, 0.2778855191048952, 0.23204832345211393, 0.21181241730182604, 0.24897074877137765, 0.2716048760752657, 0.20464352277495326, 0.2563203533487823, 0.2318213304440237, 0.2710637445946076, 0.21338048552962047, 0.25824035312744587, 0.22819183660865067, 0.23512892164083335, 0.27179456054690804, 0.2350884420974618, 0.25948584411424647, 0.23331062536258657, 0.22445755028831402, 0.19515365448591618, 0.24462507699057964, 0.203327692518036, 0.2428553458097999, 0.20771033547761752, 0.2508134973452929, 0.2476270395066421, 0.2368183336558231, 0.19245768366010363, 0.26402186107656234, 0.2715500938250186, 0.24804290835163278, 0.2709708361128995, 0.20625741307406403, 0.26370431902433183, 0.26759535012230384, 0.2420667489275275, 0.1975977051435483, 0.2729663556029022, 0.2697433899017466, 0.2268966537661376, 0.25761403580741454, 0.21401087298296767, 0.2535809632724857, 0.24582553771602508, 0.19450567899820773, 0.2584457603581694, 0.24925296935347144, 0.21950679519288482, 0.2469195731843722, 0.21547055042784968, 0.2106030380151362, 0.24123647150918767, 0.253065159912648, 0.19208681403553038, 0.2507105643350766, 0.23619122237750462, 0.20300224664729682, 0.23109334268402304, 0.24447301908130878, 0.2165365377849634, 0.23744453302182153, 0.20499188454789025, 0.20762154821439896, 0.26637775145556086, 0.22944738245988874, 0.24902702591884582, 0.223958089322739, 0.25698191474883175, 0.22974449241706457, 0.24190853245678798, 0.25529418117069363, 0.22045442431170384, 0.23344465029544914, 0.24230561963188033, 0.24936190979827916, 0.22071964147256046, 0.25477321753835147, 0.2100628498652242, 0.25040655130158285, 0.20469079151418695, 0.23790192068130778, 0.227775953928522, 0.2907898439187158, 0.210088704951093, 0.22523876089045988, 0.2273505654050001, 0.2580031478343897, 0.21919237397434937, 0.2339973519174959, 0.2170800955144618, 0.24266546539660683, 0.21422566172761226, 0.20661070314624977, 0.25803581382868995, 0.21144183057637356, 0.26570214073078724, 0.22296267979871653, 0.24422981190578727, 0.24103330785387156, 0.21646395836832547, 0.23039618138035758, 0.2213876169767404, 0.2504243626841654, 0.21527250967795925, 0.20603536909403838, 0.2323372276727601, 0.2953408141512786, 0.19721276651631833, 0.2594539300382219, 0.2520642882720012, 0.21004435493823567, 0.24498827215787125, 0.24213723259167713, 0.26834866395593826, 0.19804696333097505, 0.20323884807136774, 0.2571100109710105, 0.20511465493908418, 0.24277749132074225, 0.252929930084951, 0.26103023888146903, 0.22104403267291417, 0.2829015588147681, 0.2380582758509275, 0.24923952139731778, 0.23649922293137154, 0.22817170198625472, 0.2448353353648027, 0.2729391241167603, 0.2663021125983384, 0.23594959926721773, 0.2574802120060462, 0.22739325427700008, 0.236699438341951, 0.24601315246886354, 0.19199961249849076, 0.25928683914617245, 0.24816050558610928, 0.2226390234747485, 0.2225626118426437, 0.22638767468772, 0.24323567661738862, 0.2500164365482541, 0.27394455846180277, 0.26077799975487714, 0.22675998541373027, 0.23597606564690482, 0.24582897564874434, 0.219181018914755, 0.20068849127170174, 0.26724919505872763, 0.23053993086546956, 0.2691587044371575, 0.22434967558144817, 0.216796875738033, 0.24763349218119698, 0.2139694554459872, 0.2777390804084096, 0.25934048528664266, 0.24074360068416742, 0.19021080362928672, 0.24082721702612495, 0.26574797700776304, 0.21797016491682888, 0.2068223708166288, 0.2810600670853578, 0.20505213068595676, 0.2510970849307342, 0.2505268535875691, 0.24545439794389182, 0.20418301876596334, 0.24836287378194985, 0.1936260247480904, 0.2258442650590955, 0.24134434806799943, 0.2360850239671628, 0.24534439045534384, 0.219306674938472, 0.22657173676798747, 0.22769561783678602, 0.26630550334780895, 0.2650313517643286, 0.23331734834555096, 0.20227295277124852, 0.2250780382302118, 0.2515696229952554, 0.21781255697692026, 0.2689393051661475, 0.24429678829143306, 0.21817358142741144, 0.24595653115599594, 0.22217933781785748, 0.2263501153508864, 0.2613374425935818, 0.24047924590006126, 0.2656176984359427, 0.2089491762132406, 0.23248511291094215, 0.24003658836093347, 0.2233973579828441, 0.2518104849365775, 0.25303754747805035, 0.28089471523310766, 0.26747833471874843, 0.22789746524063875, 0.22986608236922268, 0.22066407689636916, 0.20024735829563986, 0.25769386228234153, 0.21117119510090682, 0.2256789751042314, 0.22513742926621372, 0.2048772906802824, 0.273718124306011, 0.21428557888425082, 0.2281596773929421, 0.21152785597967377, 0.24195562815855404, 0.24102356971429298, 0.23124338257394142, 0.26367198648650975, 0.24317761235229535, 0.2767985297511545, 0.2681034283390474, 0.2497911453935783, 0.26122124639553934, 0.24146651523743556, 0.24033363623728626, 0.22568724024599365, 0.23202781965002792, 0.22507286425984938, 0.22961835561058197, 0.20517709866928444, 0.2721992875566665, 0.25415943356321796, 0.21642179196221173, 0.2427366507370561, 0.25546900279039186, 0.2412419816638455, 0.2266925526264154, 0.22544799869611776, 0.2693134443672618, 0.23080477161876556, 0.21255617951107086, 0.2775813618306256, 0.21070786202422453, 0.2799019039215825, 0.22196437064058527, 0.25312983419537993, 0.26072580584768185, 0.2450682641797624, 0.18863793869426604, 0.20924764660613843, 0.2526082601455447, 0.19861605700144705, 0.28840365315490124, 0.22038815644220272, 0.27892385177619616, 0.2655894614123679, 0.1982908791162435, 0.24703781354630192, 0.24997360992380085, 0.2263133844149958, 0.2480022577065256, 0.255090097605402, 0.20803697768132637, 0.2940253511133258, 0.24366098547868917, 0.26341591992903446, 0.21106277843345658, 0.21710099164259875, 0.19872208606579939, 0.2607300619178512, 0.23780678267471, 0.2217817218169049, 0.2671384097927234, 0.23082139129011905, 0.2482278390167588, 0.22451388536852548, 0.22706342010280078, 0.24770967095424068, 0.27061798593454656, 0.21425859280714848, 0.21507380613402474, 0.26124925852023945, 0.24878111575414183, 0.24449469943725802, 0.22137795807215258, 0.22531288673063074, 0.2291416032941414, 0.23740370764792165, 0.19742666791629537, 0.22038759691342452, 0.2192247273980539, 0.22649005977005135, 0.22575188127077264, 0.25233236339179416, 0.23510194610749463, 0.22887575539423552, 0.20193307448032313, 0.20426249801038304, 0.19979646340016416, 0.25698419296849057, 0.22206469192585576, 0.2782951018425587, 0.2463319970052731, 0.21363824368485848, 0.20178381732090195, 0.24403285282869294, 0.23819799169357997, 0.22114296062467356, 0.23735686808478407, 0.20637900402054773, 0.20732394781407326, 0.240070227678426, 0.23596586497267544, 0.21804606918774969, 0.2619571123163353, 0.2570566648023417, 0.23412304022947425, 0.2099001841364582, 0.2412190693573277, 0.21956034363027419, 0.2639696953916338, 0.22663593321950704, 0.2161903533767199, 0.2286199304112069, 0.22164137491777697, 0.2260126663365248, 0.23548343994809023, 0.24274868244189018, 0.20349430540808114, 0.26099684576798743, 0.2427961051202023, 0.19257155159797548, 0.22248253489432343, 0.26329901331164485, 0.20183360261334504, 0.23918061322792125, 0.29472585569120907, 0.22831984237140676, 0.2468383401523924, 0.25550010678806984, 0.24105171639292697, 0.254023471589739, 0.2530198133844769, 0.20315210525040756, 0.21258894846348245, 0.22821600805946957, 0.238227217965209, 0.2636284792873355, 0.22929999101245047, 0.23192793880307142, 0.24078697724120718, 0.23097398041566877, 0.2293769182048089, 0.2368382971095876, 0.25419660416288353, 0.24940010247299774, 0.23811778223095728, 0.23947429004926957, 0.21226826587804729, 0.21014391816396014, 0.256745324467825, 0.23353519621603558, 0.24425589654921892, 0.24302741694335492, 0.21359940769654062, 0.2709842917744102, 0.27101178118017755, 0.24672795428668498, 0.21882344833496212, 0.21604663104925292, 0.22383142179271226, 0.24033003122995603, 0.21173370209969403, 0.262671491530645, 0.25335152743382605, 0.24681530057480286, 0.2748362986012266, 0.24228222665063193, 0.2301883163620757, 0.23773472376137117, 0.2450094684303326, 0.24211401724100787, 0.19778043698917755, 0.23064781366426648, 0.20277954331222048, 0.2530036845681451, 0.28368044037576884, 0.2777194307355107, 0.23111542736092208, 0.1909601233365438, 0.24264763016215643, 0.2932800697185633, 0.21441128367231968, 0.2736642789319087, 0.29663923831956124, 0.20935697608454423, 0.22501383893767848, 0.2044708852324445, 0.22250338065163747, 0.2563209066539532, 0.22684587286231617, 0.20101621221592392, 0.20132873040338853, 0.2431518987642892, 0.20556219344267565, 0.2036545145827587, 0.21070866888679246, 0.24990545233073566, 0.22013743491250112, 0.23581460899011986, 0.17964336325024133, 0.2755376080499657, 0.2201949517895158, 0.22662312701386386, 0.2148245457801787, 0.2601173498715759, 0.2272760851908712, 0.27604940990071336, 0.25668186821428757, 0.2274004414702195, 0.2777229887886969, 0.24152437155309203, 0.2279618509265604, 0.239420554292988, 0.2990135314730757, 0.256026012829999, 0.23283050348362377, 0.24779548570519494, 0.21915161866008015, 0.25005943701789407, 0.24792883613036545, 0.22128566485195977, 0.23786434762127626, 0.1860671482468176, 0.19492714870976546, 0.24630694608468925, 0.18619167285037644, 0.2412614243574933, 0.2369562844449649, 0.22934521382706574, 0.2681782322412499, 0.24570124325788836, 0.20754761169186667, 0.20453488746298049, 0.2855759835572125, 0.2213240423225589, 0.2377416102204937, 0.24598450517439144, 0.20604313613088215, 0.24679694094949264, 0.24465103721126225, 0.21140374299277767, 0.2622333490461706, 0.2475720194828124, 0.22479373927740304, 0.23322864615417596, 0.26470038417949465, 0.22612014621412055, 0.2171745199293719, 0.21903366077893377, 0.24968949589729253, 0.24361128614308136, 0.22542659906648307, 0.20918904625591578, 0.24483887581251368, 0.2288864742018954, 0.2199632180030726, 0.24165827671682805, 0.1884697049489253, 0.2609308300275811, 0.22846981724792878, 0.24062243509377243, 0.22079044595301514, 0.24045968635011497, 0.230630531331351, 0.21570692020081061, 0.22205377168507018, 0.21673849906694334, 0.22955216227337627, 0.2724855680423582, 0.24768167942736363, 0.23698023473118546, 0.24041957972464864, 0.20739355551357436, 0.21789750833502863, 0.22172694503672663, 0.2340090527215468, 0.23747120887674988, 0.2300786034924377, 0.27544536735857755, 0.229755175088482, 0.24687598683715858, 0.24334577696607285, 0.21232584972583624, 0.2816615337582948, 0.25007521151394063, 0.219177398964157, 0.22543181653721606, 0.21826870200646278, 0.21603968821211605, 0.23999805811847544, 0.2366759588151344, 0.2465287002574673, 0.23281019064857975, 0.2614702239393017, 0.26975160067203807, 0.2145099947982097, 0.24052151927855114, 0.2494548282034706, 0.18548148901229722, 0.23037235175404078, 0.22342342053864794, 0.27896115009142614, 0.2295298177313888, 0.25093542137936276, 0.23289013802727346, 0.2781907838847947, 0.2788667575243301, 0.2676580291610607, 0.24367065742084537, 0.2949789665484065, 0.22255991059097877, 0.24742157308864435, 0.22660979122202893, 0.22076181998484562, 0.23185205716848598, 0.24265162897163192, 0.21190803223769505, 0.2434544954144915, 0.22729825605605344, 0.2219958729771155, 0.25589787475279363, 0.21581469649206408, 0.2061603297720813, 0.2338886833324678, 0.2710899284513516, 0.2569636160854773, 0.2187507837762681, 0.24352526105665812, 0.17255417614224677, 0.1723540445792351, 0.2538266126939312, 0.26513734868794203, 0.24108738990302997, 0.25458915241432334, 0.20748424381703778, 0.246413169046299, 0.22381670803715525, 0.21982728835975499, 0.202363198791622, 0.2149420433563748, 0.260192908915687, 0.21518146183872083, 0.2305211833201472, 0.21639686382351775, 0.2131194921507979, 0.19512230210159123, 0.21606079289974459, 0.23959004728463262, 0.2298076164876892, 0.26770001141355626, 0.21954743827098958, 0.23396018867753549, 0.2003657444717415, 0.2723492574025704, 0.20246784461481052, 0.22991438253013186, 0.2847335484497674, 0.2355797236930753, 0.18842506697327824, 0.2762904903723728, 0.247425698735018, 0.2362130204378133, 0.24531004633029066, 0.3028990554634171, 0.20851591999340596, 0.281704435057051, 0.26551072388431385, 0.22625890371853363, 0.22851860333991358, 0.2221268248709022, 0.215824446955894, 0.2836232527632319, 0.25550740502417474, 0.2283497578821876, 0.2576143460142747, 0.22435871133980365, 0.22519158822628685, 0.20669606279485564, 0.27913465905719753, 0.2293857057281902, 0.2320740447428404, 0.23057636541450688, 0.2369731611637916, 0.25355089868769876, 0.21314094562598104, 0.22451922290989132, 0.23955744809716656, 0.24459955083400092, 0.20929930017198284, 0.22197699069838314, 0.21842754805144618, 0.2652157229917963, 0.24361264096356558, 0.23046783107452476, 0.23222247095573928, 0.20358478004873337, 0.2701640685492943, 0.2642547663071656, 0.24474554683812746, 0.2055670927331091, 0.22856417188307412, 0.25554727642293157, 0.25767235548699974, 0.21366554592036469, 0.2900618787717932, 0.24557200262546755, 0.24588607090085457, 0.25758005974309306, 0.2532057257210547, 0.2106758338685248, 0.20417263648918357, 0.1999416514254855, 0.22224007809846696, 0.2232990537923421, 0.22813889232052956, 0.21036607573865482, 0.2428260179104467, 0.24617566687475653, 0.20213143483386645, 0.21699643605630908, 0.23501335249956198, 0.227400303114532, 0.2973430671159658, 0.267974794099719, 0.27384133330759464, 0.22671225438429846, 0.2551683285744733, 0.2893740017821983, 0.24996969824422316, 0.28264727241635695, 0.21777446589256022, 0.24319493542299375, 0.18712162666757123, 0.2447308811011628, 0.21580297486995528, 0.2700497498406016, 0.24824642119741017, 0.23145364397128504, 0.24083540727617503, 0.20695295283033238, 0.22450986579120225, 0.2036345509331184, 0.24575017984384645, 0.20062608666608958, 0.2706478332846702, 0.20909148244497625, 0.2584630270357313, 0.25816201791229615, 0.21951517278069307, 0.27463126605870297, 0.18808158551775192, 0.23381401259734816, 0.2704863345212346, 0.2535916408534499, 0.2540651106555148, 0.24339885394479022, 0.2783145122747353, 0.21833424633166643, 0.2633927345422811, 0.20742387079862667, 0.24094880589635023, 0.2266304693773127, 0.22670804289847357, 0.25161333270590724, 0.27006956471167176, 0.23736644201835388, 0.2359329336298211, 0.2925834157774716, 0.23567142422275378, 0.2375058546506939, 0.22897121862657469, 0.25990678265982686, 0.22409425444559733, 0.2548282451259379, 0.25739170370794606, 0.21960800890680465, 0.28459766806447184, 0.18622231416495919, 0.29121480557961876, 0.2486275205329476, 0.23975700369427888, 0.2103050410684085, 0.17379164397000327, 0.20965823202697226, 0.2758112824391505, 0.24145111977293646, 0.23123691060959536, 0.23893140028908616, 0.22725736839238914, 0.2599153796210692, 0.2378800868538022, 0.2214309530130393, 0.2631515180236541, 0.21091955148414226, 0.23771379983270494, 0.20677054491231092, 0.2697556130249307, 0.2501456233282863, 0.24654989671591826, 0.24755126537884184, 0.24443799848863906, 0.22025257996259726, 0.25993146216718616, 0.2436204337220929, 0.22606483696602533, 0.21220592822009265, 0.22344245043528352, 0.23438077605660554, 0.2659036413405317, 0.26296899214820096, 0.24175497824576325, 0.22706889007268233, 0.2192868859304765, 0.24320793511409616, 0.30036157341791725, 0.27714218788408196, 0.24759676159366303, 0.22792443219172984, 0.2385623064969374, 0.19491870955123733, 0.225831562309726, 0.26403629719061944, 0.23273749612758807, 0.20813036341651944, 0.2531855616286859, 0.24626899379639786, 0.25591927428024824, 0.22502997761582852, 0.22942393622316032, 0.25065798528978267, 0.2434993693822709, 0.2281912705685128, 0.24868912725567505, 0.25687649638995474, 0.22670727361993712, 0.25195651666720337, 0.20361154939255677, 0.23261355811968498, 0.2049403003843802, 0.22899619198015903, 0.23351114940910356, 0.22327384952985227, 0.2091178176656561, 0.21063026660638745, 0.2495466900325806, 0.22107132626003054, 0.23099344501881447, 0.23424172796252868, 0.25642073697360107, 0.2637963813041702, 0.2655448804228324, 0.21745008786281805, 0.18957353515115366, 0.2469308919715672, 0.23569609596030727, 0.2177361251121277, 0.22677691869016944, 0.24224056670110333, 0.2729249064627094, 0.2148587910150178, 0.21618421237519775, 0.2568907805488231, 0.24557319324853097, 0.21181329992852704, 0.22341107026219192, 0.21716346767915068, 0.24810935875707812, 0.2333421665394461, 0.21227783651306248, 0.24548645477499792, 0.19750784239387278, 0.2418576692592787, 0.26169856138548947, 0.2800945401322876, 0.23525169370696256, 0.23122735367295819, 0.26092309760173527, 0.21594666430728893, 0.2181546645781155, 0.21033959969379312, 0.21374713166013165, 0.2769118435239561, 0.2590329302346445, 0.21905977741252408, 0.2202114017075503, 0.26010177472843476, 0.23716880822629158, 0.23271330058472275, 0.20760956293171126, 0.21014027239843405, 0.2052830644040219, 0.2475101510897765, 0.22790835594007258, 0.2348188162595954, 0.2530169011432381, 0.21669972740927354, 0.21761399964264144, 0.2437788777621275, 0.2644155848941329, 0.21870143358182217, 0.2636419128974587, 0.2485232353054009, 0.2488903288320423, 0.2311963507586642, 0.1932818283171572, 0.23241080600724856, 0.21986401533989963, 0.2453452690433167, 0.22886877438533193, 0.2273489416710644, 0.25415822545740524, 0.22399868998326597, 0.2573241097822051, 0.27676430643413796, 0.23404145812215552, 0.22119391067858016, 0.21650199215890456, 0.25599052469591527, 0.2479848028084597, 0.2195287655507215, 0.22082926298165956, 0.2732987054988078, 0.1982239025456202, 0.2292093582573112, 0.24861684640718598, 0.24406782003924488, 0.22783092510187675, 0.19000648610727439, 0.26837669419923743, 0.22719076304372315, 0.25071897738751486, 0.18999370652744577, 0.1799786402644526, 0.23997379594632795, 0.23819986036376853, 0.21743241758104948, 0.21393410930679843, 0.25829796447772124, 0.24685886204916047, 0.21505351059718994, 0.24324170091005493, 0.27654824041565595, 0.2552379877957427, 0.2106606821356946, 0.21930343269811198, 0.24256078582489674, 0.2187215673425991, 0.21688830282747967, 0.27911418807096394, 0.24748168528941974, 0.23961200167285154, 0.23367465434141627, 0.23670431745681675, 0.21317592241037456, 0.26416194027416107, 0.26289638257994286, 0.22312476789771127, 0.24233855096970874, 0.2145809628481504, 0.21782210755771153, 0.2385100633886617, 0.23732134190952076, 0.2403696213645426, 0.2458850344019502, 0.22835530997626918, 0.29764235733049776, 0.2336830978204144, 0.20381054827655656, 0.27644589696958827, 0.22100464454796734, 0.3136796850786266, 0.2515495947360341, 0.2706712913059717, 0.2133205404959152, 0.23265492736251558, 0.2254988769582133, 0.24788291059386577, 0.23032234832963513, 0.22400043187288543, 0.21975772479852684, 0.2254477453712007, 0.23618396402367356, 0.23906257462683503, 0.2559558181940824, 0.2249140414032959, 0.2182134252947891, 0.21343511285792624, 0.2790009549614403, 0.26963985626439324, 0.21003226251319446, 0.24663162019220783, 0.2825608608441807, 0.20495235761389918, 0.23261569724829786, 0.2511628886232605, 0.24308175427960746, 0.2814190818878741, 0.2418691684194007, 0.2247088299066549, 0.29688283664967413, 0.27034644230437366, 0.24722814267697096, 0.22206440487962717, 0.2529683123091646, 0.23252205882682989, 0.18364283362636669, 0.25529192923272787, 0.20163226568386494, 0.22115139652228227, 0.22512944331183374, 0.23982264571533923, 0.22506236986697117, 0.23321553065327974, 0.24074534704544745, 0.23224963848933483, 0.2416343044784816, 0.2545419360608135, 0.20493997045980386, 0.23369048746606053, 0.2083343403648082, 0.21786034174763988, 0.19630532676671875, 0.2647456309280483, 0.2387398382993984, 0.21434406886815843, 0.22691017878427075, 0.20213956361873495, 0.22561235409596242, 0.25673937555892995, 0.23074070309459996, 0.24629901894483794, 0.2459962912888147, 0.25120412203458126, 0.21773432959452138, 0.23844305666566368, 0.23598631313537796, 0.20861025869558444, 0.28122941013113734, 0.21524294468824728, 0.19267028496418415, 0.24968369047140973, 0.2493000626485097, 0.22142079464859646, 0.24883025120932686, 0.23817339281866784, 0.21276555930446894, 0.2621611038769178, 0.24259061730479148, 0.237691586476235, 0.2160323735129293, 0.19028144144207906, 0.25531671084235663, 0.2977157889978764, 0.25208028203965577, 0.22983386143396667, 0.24963815033765216, 0.18838709360851721, 0.23419159884651097, 0.251156284907236, 0.24413925994265223, 0.264640099930764, 0.24650787390757178, 0.19340605548622422, 0.28337790749863656, 0.22816725859116999, 0.24010846606521757, 0.19995685328203577, 0.24218741409933942, 0.2517224652410102, 0.22992694999275237, 0.2375745804682925, 0.21146640099320102, 0.21437148297469727, 0.22766960330521976, 0.20486715826699126, 0.26304784273232285, 0.24605870683580752, 0.29347386497907285, 0.2527635622088609, 0.25994920636080965, 0.2129366901610254, 0.22037399813363645, 0.2467793683385247, 0.23914855504878976, 0.2214265198461268, 0.2321827341238989, 0.22017362781182606, 0.25371136943653355, 0.25165069083507285, 0.2525136719752272, 0.2134888037147545, 0.23452776354546562, 0.2118496807079777, 0.19565352313260567, 0.23439663323035673, 0.21616727439812194, 0.19824214786426325, 0.2423685472720926, 0.1983737950832942, 0.25127462525775857, 0.24416326538683064, 0.2483616275490588, 0.25989944721224506, 0.22542401597102668, 0.22868706815702922, 0.22601288680922416, 0.23150831063583438, 0.21822251617652091, 0.21665006378211699, 0.21558687106202984, 0.2242501168212868, 0.2576776648488344, 0.270276369742018, 0.24997005372062828, 0.27584722820146884, 0.23635050832738352, 0.2653503917258944, 0.21206690219241883, 0.22259786816352398, 0.253728631977769, 0.24890195357706302, 0.21218078157249323, 0.24271944443322505, 0.24048602876584946, 0.2501323233838063, 0.268335541945374, 0.19360273487149876, 0.2100063131097226, 0.23599126251149707, 0.26917001412286956, 0.27966462805459474, 0.23201451789850006, 0.23207932111267, 0.21168782359443597, 0.21909111254244829, 0.22714461916052417, 0.2400233856168227, 0.30766118295919853, 0.2164626008034793, 0.22746940075173172, 0.25835190284068005, 0.21889377926645953, 0.2062868042989711, 0.23736372729104052, 0.20917206914798778, 0.23421453356515168, 0.22641252698888825, 0.24541108601559194, 0.26787532852569224, 0.2199361421261494, 0.19563060343892263, 0.2535562633020909, 0.2387764156852758, 0.27060719182955945, 0.2764887340934767, 0.20176471210992897, 0.23472795802698532, 0.23158257751885236, 0.23174581641366077, 0.25041580368352023, 0.2606109128178466, 0.2077912598226037, 0.21342900978494267, 0.20584731794188058, 0.25561012565938984, 0.22211930999023802, 0.26018359319895984, 0.260972520912477, 0.2121651096632628, 0.22404263233561122, 0.20420516646362416, 0.24554043243583748, 0.2419213649579555, 0.3236742943579219, 0.2257619480487477, 0.22425547088472877, 0.2737740349286477, 0.23179261071803722, 0.21736922116787302, 0.2008882111779667, 0.20422635537558767, 0.21236624213093008, 0.21924701539471103, 0.2588944361060668, 0.28475713399709884, 0.2365226419759463, 0.21017234733635703, 0.2223611662028424, 0.2159072665143057, 0.22372963866259735, 0.24896394650804873, 0.21708362251213337, 0.2575986851400077, 0.2184033611477664, 0.25140046992226855, 0.24519383436085898, 0.2187387812291864, 0.22577507115068662, 0.26005284394884043, 0.1700288131361765, 0.205357041555324, 0.21266983354490304, 0.22765302706824753, 0.2558943230298065, 0.19986854882986307, 0.2567569219184427, 0.24709657585684328, 0.24215897025004174, 0.2551313246992548, 0.19403830483526163, 0.21026199894612257, 0.24112619718265307, 0.18823517050136884, 0.24368048133851808, 0.26527469238261403, 0.20590417016034343, 0.25491191861172746, 0.24221673088729445, 0.23801737101538029, 0.25922821902100646, 0.24035522350381933, 0.2535785763514912, 0.2531763239417891, 0.23062958309338405, 0.24413500396757373, 0.2182302333940385, 0.2728323299224042, 0.23640356315793462, 0.235839756706606, 0.2055103626045693, 0.24704588571751002, 0.27330526890082896, 0.25594191673025274, 0.17758894829327448, 0.2387163170133849, 0.23001534689251346, 0.22542257609870162, 0.22647428227411037, 0.2105669140124329, 0.20704690599896272, 0.25334734333598613, 0.210274523470299, 0.2206811152596829, 0.23919389627386023, 0.23157520695685477, 0.20010547147792715, 0.2671213864990034, 0.28418765179808575, 0.22803643002593674, 0.18373561447092962, 0.257802618105512, 0.2073450239970423, 0.1917601597981833, 0.20561272528448732, 0.25851876090415654, 0.20000184289704043, 0.2492979942100198, 0.21979632870823118, 0.23354999742766508, 0.19662339778539697, 0.22675977803393393, 0.21987359942155377, 0.24518565315210136, 0.2359857502999365, 0.2162807934300992, 0.2151380671343512, 0.19464821200718505, 0.22155321903655834, 0.298153999168827, 0.23905137190446923, 0.18184279425192912, 0.27074860352417207, 0.23914349695378442, 0.24149547394069043, 0.26767062344259696, 0.2125286149277495, 0.24315324486267947, 0.2755503778764259, 0.2542216845941158, 0.28141208596407313, 0.22957942267803264, 0.2149743090619852, 0.2772452262684878, 0.256037348979216, 0.22139474963024902, 0.25798369969921325, 0.22924626690247915, 0.21074761575973883, 0.25114513403428107, 0.25838501232897404, 0.2551907547011724, 0.26334403609858065, 0.2762603084108258, 0.21865797009318289, 0.268653225364942, 0.22141756712617375, 0.22562041111327366, 0.1919157629044124, 0.22829315657679636, 0.2703370100977553, 0.20990361614162364, 0.25824354787648496, 0.2715500286168083, 0.23704193321771777, 0.2889368594940109, 0.23450034329281605, 0.2360757561856888, 0.26875322073187685, 0.22918511641968156, 0.25431595776604404, 0.24955327207075007, 0.25228478252805736, 0.25041451319522845, 0.21032323471484837, 0.23566697151726568, 0.24496613796603472, 0.23083671995041052, 0.2511516226873664, 0.22421248148932488, 0.22175514020084447, 0.22979092287976463, 0.24097819655427227, 0.2371444136905415, 0.23694136579303776, 0.26923500232610204, 0.26769748602496124, 0.21412170021592755, 0.2600868919103229, 0.23958747577419223, 0.23003411661329637, 0.25092554992887994, 0.22972667780725675, 0.22606229529219005, 0.23502037709053727, 0.18521421800384563, 0.20420944343392852, 0.24419861994351105, 0.2713343315579002, 0.22190094316579206, 0.24430058036894917, 0.2360442094739612, 0.2613314088550093, 0.25741812887648124, 0.18371189470013152, 0.22461676756101712, 0.20307171467946755, 0.19992839373014054, 0.29492056486468254, 0.21918588978347567, 0.21775666119310708, 0.2394241790254264, 0.18540716748986358, 0.21733914163563484, 0.20441573904756413, 0.23431487972090936, 0.24560922056089926, 0.2020941438170503, 0.26693210847686194, 0.24442564094786193, 0.2598747505019819, 0.22706837955822867, 0.25724138861862883, 0.2429005377141899, 0.2834770547382338, 0.24645627144258453, 0.23081956085433653, 0.21113210827622558, 0.25218991919569483, 0.20966095979483645, 0.2518245947768214, 0.20123018036540202, 0.23970537920479876, 0.22642277299321706, 0.23388485647563229, 0.19300086854646809, 0.2691950929824941, 0.27141642282798867, 0.2612028238003456, 0.25533506278113605, 0.2591097088977373, 0.23580712772499876, 0.32701548654674656, 0.23754155990439516, 0.24348364094715552, 0.23143395188147575, 0.23059649287571313, 0.1785126578945096, 0.21871609922803967, 0.23927098840223504, 0.23099383792259184, 0.24261864059913038, 0.2515975685812188, 0.2578742887408834, 0.20735008176861863, 0.2662208833558636, 0.2514938792227394, 0.24464906392709102, 0.25360667300305306, 0.23428694899355731, 0.19852103048667372, 0.20964870799749197, 0.23227907570645653, 0.25677568691710756, 0.24110626121878395, 0.22049904838221923, 0.24402756125295277, 0.2478277585999826, 0.24179417890257662, 0.22680883510062821, 0.19191779799482797, 0.22626631953303078, 0.2444433676380847, 0.28293434283560864, 0.23747728016029995, 0.24531937836721404, 0.1873023671915245, 0.2595027166200699, 0.24496143565143005, 0.22675235935494967, 0.2634436678342284, 0.24847005198012934, 0.2190364310958519, 0.2151846838663878, 0.24865702030926687, 0.2164706598549342, 0.2622138374485285, 0.22754826223029243, 0.2349243511303716, 0.2570980952395282, 0.20718784850647634, 0.2188428936455889, 0.2553637890849775, 0.25331641358419926, 0.1581347916207937, 0.25081859775454696, 0.2447740395084372, 0.18320984806020166, 0.26156447315556125, 0.19350938962465458, 0.2640830839485393, 0.2221328928675923, 0.1938908149056078, 0.21716359587455084, 0.2592017229113903, 0.25826526523651544, 0.2585362729722228, 0.26899542953490335, 0.2271723120824504, 0.21102517386706393, 0.21500375637011132, 0.2149953496308917, 0.23209883268059334, 0.23154332891254664, 0.23282529511870811, 0.23317983202500459, 0.2225550993857491, 0.304280746226438, 0.2761470595979997, 0.26517381565708603, 0.2577397227500081, 0.18687409099098523, 0.24534358258538286, 0.24572112757673603, 0.22438328386131823, 0.2237583734095671, 0.2495995018988927, 0.262679878176739, 0.19754155899961826, 0.24876307716405177, 0.23189267859553336, 0.21418611246196623, 0.2568453820857112, 0.24101773154068395, 0.2507229640653888, 0.23826885278074303, 0.19959963173686823, 0.24224552332842772, 0.2667767582224222, 0.2619223459545058, 0.224266085804105, 0.23389184704562893, 0.2183773697260376, 0.20797811804455799, 0.20881584171646211, 0.24210320556725184, 0.23314802914271882, 0.23711924363511924, 0.2289342632091854, 0.25680324028728047, 0.23403994490877716, 0.2622457483150912, 0.2533763773330023, 0.22292341074323743, 0.20658627802731316, 0.17120868280896162, 0.23711786043250008, 0.22950031116203723, 0.25132473489943974, 0.18223518985728684, 0.23650151552564744, 0.2501454982959378, 0.23328049042642304, 0.24067366535114193, 0.23737610943767, 0.24107991705981605, 0.16279359415799283, 0.23105416650142846, 0.23848539758478648, 0.2305882415485947, 0.23826786807446235, 0.2362357705616123, 0.2504320104898641, 0.2443390354226393, 0.19818314562694012, 0.23630200429996576, 0.18351690307477544, 0.23968492336207986, 0.22165690921963258, 0.26682161984492786, 0.23743772591605122, 0.27913857544867804, 0.2542247015943878, 0.22866585188721392, 0.2134655327462575, 0.21035484446676758, 0.2594589379731078, 0.227917838950223, 0.21098998173203184, 0.2116325971166423, 0.22225078534745263, 0.2229158800993499, 0.2536514273167313, 0.24328566516968475, 0.24184534329240231, 0.26268573942578843, 0.22309501452665115, 0.21578019757447406, 0.20217078957435647, 0.250525853442935, 0.2353197707763871, 0.2436894946700055, 0.22355547126545, 0.20894052176948566, 0.2520739129897271, 0.22889042797337972, 0.21620873317246045, 0.2206694644626994, 0.25957611565440797, 0.25435640939427356, 0.22266991980631914, 0.24549408579538584, 0.19524693316949704, 0.21335959049114775, 0.24516837802456648, 0.18011094966317537, 0.2746115202650951, 0.2586224153469467, 0.2089084379860823, 0.26686576621718716, 0.23250181530157418, 0.1938482502028825, 0.19075416502769055, 0.25071227144586156, 0.19046368143681566, 0.2795213660522995, 0.23852880434043114, 0.25600907592999744, 0.2668864160693254, 0.2630565306048958, 0.2290380631196315, 0.24318730174925038, 0.25148499501862503, 0.2515887879395945, 0.2264181213456196, 0.2257534191079284, 0.20531955411823535, 0.2349357150230259, 0.2572560699560604, 0.22257581770282414, 0.20851845046475, 0.2326650838671383, 0.2151409261472029, 0.24706604394561907, 0.2293100448835747, 0.21440316680968002, 0.23304114680885585, 0.22505837383582386, 0.21869839242954514, 0.23673677160700315, 0.2158196063513816, 0.23007972504732077, 0.2461081269321294, 0.226539741511781, 0.23726828167855152, 0.24164964009535658, 0.21200455846842747, 0.20765048142589543, 0.24553811912585743, 0.17905446700476496, 0.20020361256950045, 0.21234863415911442, 0.24645190014757115, 0.2466477103894778, 0.22362477772946093, 0.245045968942124, 0.27440845592305585, 0.2699254503454449, 0.2373565196340903, 0.278537331215776, 0.23008677798337396, 0.2335452159134551, 0.24254942927948733, 0.2256609620560819, 0.224684371112707, 0.300385258765241, 0.22203307234334002, 0.24094649720526776, 0.24516439043493726, 0.20791685265946094, 0.25496411091583454, 0.23782871134944839, 0.22190057066276864, 0.21291528257750872, 0.2595583998884243, 0.21514924257695364, 0.22245897621281538, 0.2428241938678426, 0.24464978889520309, 0.25492062948473565, 0.22735864002015146, 0.23240761861027948, 0.22997921212895306, 0.25243546648260384, 0.23392977699447173, 0.2076588424589397, 0.2578333696529029, 0.2529787692505613, 0.23543992990410567, 0.25433609037380633, 0.18808499743805585, 0.2681688666550394, 0.24280390236896085, 0.232331540725842, 0.24531546395498233, 0.19706477680106838, 0.22018601640815103, 0.21293517800518377, 0.24752557752312335, 0.2122084544705902, 0.25061792270613514, 0.2370825687912226, 0.20443403458845125, 0.21336988524910994, 0.199343841072967, 0.2571900282791543, 0.2611317085026886, 0.23750746506876502, 0.2429360176237916, 0.23208308651372808, 0.20016154175127893, 0.23917847234261613, 0.22390785228285218, 0.2443632623526471, 0.25067170593818017, 0.27688678735408406, 0.26004575955214515, 0.24899396492113535, 0.24323691890285748, 0.21602272591442537, 0.2566148158835147, 0.2189737161735101, 0.2204998868902273, 0.20730107625439992, 0.23984394734192743, 0.20853504389470517, 0.22047179520035337, 0.22912567915325585, 0.19266886393218718, 0.19361877337287542, 0.2566779171408313, 0.2613202322059827, 0.24245748781480417, 0.25305492367227245, 0.19593188454472316, 0.23530159393444805, 0.19132161893007746, 0.2523573071795937, 0.25742259813226925, 0.26110373210000126, 0.22863183002448426, 0.2263981820998232, 0.2645003876871128, 0.22823354523422554, 0.26374542484185, 0.21950859132552974, 0.23845497200504265, 0.24098516127182001, 0.22656915558501417, 0.2541523288773303, 0.24791103931943578, 0.24663363290572257, 0.1910663940545591, 0.24559644899406016, 0.20694241261835147, 0.2280064123531841, 0.26183969673096524, 0.22904816956540622, 0.22548672692033395, 0.24794176520911473, 0.25016837310903983, 0.23745106873491145, 0.20086713497238232, 0.2719685562160434, 0.22600769004018298, 0.24102861768100753, 0.23008089299426335, 0.24875209028607598, 0.22146483937752048, 0.1929877221203652, 0.25721590434467445, 0.23397610133353156, 0.23525981409485627, 0.21520512227056054, 0.2814158638516389, 0.25300046697496487, 0.24842909457829493, 0.24637182371310853, 0.23852348961112965, 0.2198516681948097, 0.26167700226145574, 0.26351575449712544, 0.22200129178027714, 0.23697112417464897, 0.2610924711196395, 0.24266881510939595, 0.2804496743807477, 0.26089100854317476, 0.25845868939638994, 0.2694347816375915, 0.21901255421522348, 0.24028745841251567, 0.24826120069538563, 0.23205057955406352, 0.25142687054320484, 0.2683401371948092, 0.27112781388632795, 0.23450502858493005, 0.2516450179377809, 0.2418604002584251, 0.2402826740865988, 0.23618052576810672, 0.24013332822288713, 0.23971631271144522, 0.2838376372176314, 0.2349931970868969, 0.2580841338149774, 0.20628047479713033, 0.21582230005447192, 0.21842887815273399, 0.24585261143706932, 0.2545476238622125, 0.2639978144747752, 0.24684072883433714, 0.2206455231721965, 0.23095013401242528, 0.24761303302516946, 0.188431447741284, 0.23627627605142726, 0.23624724764273694, 0.21034116532703556, 0.25260987127358614, 0.22693160275440433, 0.25865835468422727, 0.22478395775745977, 0.2426608225981547, 0.21605424782086521, 0.26725371342019305, 0.20656116010692419, 0.24767947361548814, 0.22715588017509522, 0.21411306425274948, 0.23166999149787035, 0.24171735140745035, 0.2451799109122786, 0.2771755754908517, 0.2822211915734583, 0.2400176090921453, 0.2202058766235932, 0.23084078287463958, 0.2427878053617466, 0.23619968470888683, 0.23801542403122583, 0.20091445833654656, 0.22087066151032592, 0.27848088574783947, 0.23878752293401906, 0.23252357649126762, 0.23894282584265594, 0.22932551230166137, 0.22197634801319285, 0.21904069585718733, 0.2584500350799173, 0.18714817498697361, 0.19883067658770237, 0.20887089470417636, 0.2182466382878264, 0.22382668811925294, 0.26301438247727876, 0.24580168013758244, 0.24379119192811963, 0.23797179810740324, 0.24064180343037278, 0.22076905957070128, 0.2830714100753346, 0.22887027798828, 0.218658995457674, 0.2044926842681721, 0.24220232315974083, 0.21033092814336174, 0.2281972923485759, 0.21923804500715693, 0.2075147365806353, 0.22708593554493536, 0.2285543148986214, 0.20686726036552597, 0.17673572574384824, 0.22300193912178998, 0.22671331722912724, 0.2387588757240692, 0.19129474264502538, 0.2612511665585552, 0.23038842981490582, 0.2575951627754221, 0.24722981213510561, 0.24562483453850203, 0.22006322225754735, 0.1804335881981834, 0.2067838717306915, 0.23739776071430949, 0.2179579149340182, 0.27511473265867253, 0.24947283532576447, 0.1987821393109942, 0.19632732698294672, 0.228203186818882, 0.2761364262670647, 0.2974982799778733, 0.28278174602960765, 0.27748292411572956, 0.18720302988642892, 0.19501325636587916, 0.25124132943170796, 0.2473583307983145, 0.23135265900830795, 0.22535635727253145, 0.26543134889792325, 0.2653017499810944, 0.23053525570467528, 0.24723209910571298, 0.20629825442508914, 0.22677612146141776, 0.2312770669299694, 0.24460171433428501, 0.2139069809664949, 0.24695645117000914, 0.24456569029967593, 0.2046404411481861, 0.19293896827194937, 0.2703249586573844, 0.2402606425719341, 0.2374048834960844, 0.19495557337683006, 0.2600734160361019, 0.20922500570950373, 0.22994395326508768, 0.24296897926157168, 0.26340164899156615, 0.24035061588606627, 0.25324445352877945, 0.25563080122211435, 0.2369385925860703, 0.24957927190090093, 0.25038835056194986, 0.23451824558089052, 0.24560315032407626, 0.26868692674035893, 0.23548490117814455, 0.23238088057835118, 0.24758678895395111, 0.20127749383925592, 0.23433736481176057, 0.24659342014350163, 0.22909124187005764, 0.23688603584118512, 0.2301392221318429, 0.24001366271606842, 0.23641258573733726, 0.21031985972984238, 0.2541575429161382, 0.2548822966038602, 0.22551463125623183, 0.23628720199495887, 0.2747668292814835, 0.25615621016642465, 0.2731258669570919, 0.20615449699129743, 0.2480193889902228, 0.23343072910861257, 0.2452444380721601, 0.2203535442446463, 0.22149468223952412, 0.2557256498906771, 0.21536674855837332, 0.2867538030850488, 0.23181507772056897, 0.19324121846545905, 0.242875056178824, 0.23364685436232802, 0.24801430798532348, 0.21346403371860678, 0.2338993146601809, 0.24397724944316154, 0.24847870597118357, 0.2049829664326365, 0.23647737202352478, 0.1856757071974576, 0.2541400068040568, 0.22360356558458677, 0.2389029796027991, 0.2330936737322602, 0.24897244509685382, 0.20920205868210529, 0.2459668262256237, 0.22757100491318635, 0.2532967751561492, 0.25393234689746347, 0.24641375441539318, 0.26290986254321685, 0.22291854310974218, 0.22221616544293193, 0.25275970817296944, 0.19713604865238885, 0.22803546343258452, 0.221439232763479, 0.2577477796883009, 0.2302182358490625, 0.23968453539404927, 0.2068641986556311, 0.2596027179657248, 0.23834023203044966, 0.19296605345617873, 0.22639150685749038, 0.2387845669551937, 0.2467613405141812, 0.20033659202541296, 0.22704289078396556, 0.22802162173759802, 0.20804513744622852, 0.2737996136718595, 0.28510639873354665, 0.25332161487003957, 0.24535192288132576, 0.25924430105915097, 0.2758371430407404, 0.23583713832069217, 0.2356602941491993, 0.24480907114529457, 0.2427098718521821, 0.22387656899478345, 0.25087303266422917, 0.2609983214757343, 0.23102748195245099, 0.215563538192144, 0.23545000681882022, 0.22516124627572295, 0.21633333139722685, 0.24256107415252956, 0.21848701999028483, 0.2608459621794341, 0.2577619278374984, 0.22995907268675772, 0.24021616833760887, 0.20180821760545517, 0.23460471442914457, 0.2381136034478355, 0.2425455361495346, 0.23002908943700015, 0.25898048033859844, 0.2016560223932201, 0.28219373600765596, 0.1923881310746389, 0.22131070549833542, 0.27842938819796037, 0.21451200510916318, 0.2574628017271308, 0.2789568080718972, 0.21728679052822408, 0.22851624510611332, 0.22061142851530327, 0.270663474422129, 0.21986480441980435, 0.2204931939909753, 0.19674045805655438, 0.20335304860397369, 0.26707990300848383, 0.26573142604565286, 0.23011086262120112, 0.24180250236639686, 0.2347126482313393, 0.2636816339384914, 0.22595930554757768, 0.2583603396013509, 0.21732354848791963, 0.23521803107560305, 0.2486485425597256, 0.2668436686646859, 0.18549524850331203, 0.22320348279034816, 0.3026245572156586, 0.23807691912907458, 0.2519095327529911, 0.24766616761944152, 0.25538624167669427, 0.24516450014851757, 0.22610480329549854, 0.23586029710937673, 0.2535400340628425, 0.23128362650571077, 0.21545601549351712, 0.2534976570021676, 0.250199478197929, 0.25878634261405725, 0.243883362049972, 0.22206283003128113, 0.23146907066632563, 0.2606157083441577, 0.24364988717964253, 0.22358253686290183, 0.21249431007613864, 0.25053405758061476, 0.2536258163140244, 0.24340772532711777, 0.21085787677639056, 0.23555141565479304, 0.2311182165595618, 0.24882979110980796, 0.23723253125960791, 0.25605743195223934, 0.22841961912837816, 0.2611920558286128, 0.23709632269932612, 0.2424817153651401, 0.23132278958850921, 0.24388649526870448, 0.25069082390623415, 0.2588073143207283, 0.23757076962840687, 0.20950647084203325, 0.215197815119627, 0.19679648204838068, 0.2745392930342089, 0.255177831228871, 0.23894851779185952, 0.23066949697065026, 0.1933723767023157, 0.2456871064383559, 0.2697415151039361, 0.2106476748591908, 0.2463914919536954, 0.3065689521408181, 0.21684265018508847, 0.23892307976891214, 0.2276872599898557, 0.21611192315504293, 0.22836927702263243, 0.21056134687025782, 0.2577144166814158, 0.2435212315370033, 0.23775259897131182, 0.23000310280452424, 0.25458735869925486, 0.22805715520795208, 0.24426548195588219, 0.23620360381645011, 0.2297435779408896, 0.27780293233048453, 0.2193853049175388, 0.21138162389133894, 0.26471414019287265, 0.22318224978314954, 0.2071013547650994, 0.22304333443139915, 0.29014353621913336, 0.26000397547719195, 0.22388228621401304, 0.21625259791902862, 0.22232862158315103, 0.29887028552764416, 0.22959565173965946, 0.20344800707400884, 0.23205480694480504, 0.3055521963851606, 0.20308730616726178, 0.2053997165348699, 0.21514268377722487, 0.2631154587015342, 0.20696737941593815, 0.26327824564623725, 0.2288494959812417, 0.24899313291057576, 0.24713268170895722, 0.2052187575448184, 0.23082971635075636, 0.27087428304564926, 0.22798755438101923, 0.23180453618534938, 0.27402766857451927, 0.23976440203477753, 0.19314239887934923, 0.2215012093081668, 0.20381698791204297, 0.2461872498960309, 0.23829367308106847, 0.22324439101636503, 0.23257743954926158, 0.238986382650106, 0.22952163161550368, 0.22626049485812913, 0.1861156291631997, 0.24248047425309965, 0.2505646928089958, 0.22510815330161893, 0.24845171180129966, 0.24112906796331735, 0.21956531342919733, 0.24718041122895562, 0.261817992910282, 0.23903313869091, 0.2564170616552452, 0.24265376655762855, 0.2430189524493238, 0.23321394638954612, 0.2514641720688265, 0.20556108922651953, 0.24674221210940822, 0.24385051569342078, 0.21136370238848476, 0.28032697285403657, 0.21852554946835562, 0.24822889469720416, 0.2238224029915257, 0.273293970864027, 0.22500342559277836, 0.27564235283077243, 0.2438745077472894, 0.2436996982918335, 0.21783760719412237, 0.2137017678548882, 0.24576694405119015, 0.24321459362683653, 0.225329767682383, 0.24676943523119252, 0.22519448278476917, 0.22038393879250956, 0.2521778327048958, 0.21890389817176362, 0.3041180386468317, 0.2843015762944852, 0.23860521558582365, 0.20276296553616413, 0.24946342888812592, 0.257234288410457, 0.22559333080959143, 0.20280201546072882, 0.251627915577242, 0.23115476746953037, 0.21765556833595096, 0.16844136040914232, 0.25791488204158886, 0.23527007192476135, 0.2512868862053169, 0.25433475744053247, 0.21133693792669828, 0.2218123979396203, 0.21593285446552038, 0.25802170076767916, 0.2114773977146691, 0.23942819120550904, 0.2406665756361385, 0.2585576217156996, 0.1914417214588641, 0.25840996358679974, 0.2176169058295208, 0.2593259903558913, 0.275756970600057, 0.20153172278001413, 0.20502413010428436, 0.24235003868323599, 0.21712998965154032, 0.2446683434152427, 0.19332431496151983, 0.2312546538947499, 0.2537472542639182, 0.1748034813381356, 0.22819481284534943, 0.24566789767402356, 0.24157732226967904, 0.265600056834101, 0.23666229924958093, 0.2473180804904082, 0.2440376445876912, 0.1796602934203667, 0.24769409531215394, 0.2845188232800691, 0.2023346935184892, 0.24980146260634153, 0.2088625438462533, 0.22209511970615659, 0.22864897648334467, 0.3043253058262585, 0.2331296259882304, 0.26845147038391715, 0.21716406100060248, 0.25244728533679156, 0.2505705994084294, 0.18875565382768705, 0.26825933460840823, 0.2406848455672244, 0.25851092871889964, 0.24607176582417614, 0.20658749921804145, 0.19708543964281336, 0.2117474855920874, 0.23268851836031448, 0.26709261013250685, 0.25963654240342077, 0.24755167957157517, 0.21995533435840406, 0.25281311332099127, 0.27360460401952075, 0.24112934656362636, 0.22958140924261872, 0.25251940417785934, 0.21127005046214978, 0.20565416085315835, 0.25667044148772894, 0.2465011440469515, 0.2676534345968862, 0.21957947731607738, 0.23448488119228295, 0.22579972038177912, 0.22818783202008996, 0.23834269160425686, 0.2510792491967771, 0.2539229998270704, 0.2530292724395269, 0.2513699531016103, 0.268866358220041, 0.20313090824755214, 0.25233978090114934, 0.22083255712557892, 0.2818406554556486, 0.1850015137767912, 0.23225667538779854, 0.23160881192336225, 0.25606265323855054, 0.23571679663391254, 0.2652125140238407, 0.22525561849183465, 0.19545795314799957, 0.2510460404346225, 0.2142737610784879, 0.2077437382855915, 0.23377616368084697, 0.24085510707744476, 0.24287527147420826, 0.23045666147745744, 0.2228569842947434, 0.24691238939181984, 0.17836697816411584, 0.26605916732974744, 0.30284468452081686, 0.23253759118759953, 0.21461773028447784, 0.20328973707991246, 0.19563687808529706, 0.22298623966952302, 0.2566445091672259, 0.24130681752084682, 0.267562848116565, 0.2475536002998505, 0.25974769976583917, 0.22548278424570548, 0.26179970627422416, 0.19293516914002656, 0.1931412336158303, 0.190538638742343, 0.23357213359457077, 0.25615824303703705, 0.2161701544471769, 0.22871777075817526, 0.21500603025230794, 0.19194923141299952, 0.21644663950971765, 0.2594417204333414, 0.2529372117731541, 0.201678105835266, 0.2505832373355693, 0.22959159936649293, 0.2551810941483532, 0.22362569691644033, 0.2455452453783077, 0.2484128581648284, 0.25901719444521115, 0.21809598812898137, 0.19551132365273438, 0.2361249739727669, 0.2486896127835086, 0.28604761097633313, 0.22177873888459507, 0.23646583069087054, 0.24273113307876784, 0.23741804827024052, 0.22778864398602833, 0.25061327658753935, 0.2951819138613356, 0.2609273198027975, 0.20201246873140874, 0.24241591035933505, 0.22701643358666077, 0.19833250378722167, 0.2633148756339845, 0.2264759639873944, 0.23265475523459755, 0.24013308110439685, 0.19735723547174502, 0.20341180603071857, 0.27786826623824185, 0.2560396830759066, 0.21186433059471513, 0.1835452447154735, 0.2443967607155566, 0.2843146835975863, 0.22769360469121805, 0.2916606762475666, 0.21970356314771095, 0.24174745261922045, 0.2178381714761152, 0.19144688923592434, 0.16964145312265985, 0.2450775495528962, 0.21992387660006274, 0.21429192396650298, 0.24547339681816832, 0.23676340438825116, 0.21162721172116492, 0.229414318280168, 0.2211057222395658, 0.24760492424838076, 0.21464202256972975, 0.20357349978740602, 0.2028747781410834, 0.25315667098501765, 0.22360701369585256, 0.21893609143383658, 0.20155952228533286, 0.23801531171562523, 0.24484945564132504, 0.2300093498799908, 0.2257288143350425, 0.24201303335033342, 0.27609851837008, 0.2445063559438787, 0.20671122753485857, 0.24483655809310922, 0.2740507409551202, 0.18791136784640267, 0.2502321295112073, 0.24935430679825035, 0.22644751027647111, 0.24969975453663626, 0.2426268520715062, 0.22843460605836086, 0.2776193861459204, 0.16850520910657527, 0.22535910469589154, 0.24921749773805837, 0.2205947700606679, 0.23658468365796143, 0.2721465912054674, 0.2534532037404213, 0.24840462541470903, 0.21083961764768153, 0.22943127937083055, 0.2360032075460407, 0.2077320951182697, 0.23995950985628872, 0.24849993669017564, 0.24728653209351437, 0.2341749224404816, 0.2044058772449559, 0.23390296447813622, 0.25887169246645647, 0.2235397406816288, 0.19753522712549149, 0.20515334291949847, 0.22378324721699078, 0.21076587824485912, 0.20272556739766856, 0.29737625075046825, 0.25090440271146813, 0.27348314928438583, 0.21804283307716954, 0.20893023240969213, 0.23171137347592724, 0.2562846468306595, 0.21528509091478656, 0.25613848729591165, 0.23807741602936433, 0.2078661369640312, 0.2210527412536479, 0.20880210273956884, 0.24395460137402614, 0.19950895502528848, 0.24329302729916064, 0.1922845964655967, 0.23809438386412932, 0.23454403981738106, 0.22928829880500343, 0.25241271473591215, 0.211556615060389, 0.2324877859435945, 0.21587879919626665, 0.24791858660861282, 0.2676862300486949, 0.23574473796411635, 0.22014799627997553, 0.24475162531574376, 0.2384327920352399, 0.2669841263876584, 0.2775643799243076, 0.23929785911647644, 0.29103354020719585, 0.22213372000147183, 0.2666728217639615, 0.22491954822067084, 0.21948480475561027, 0.26321355868714164, 0.23209926458711297, 0.23761759773138694, 0.1797461209295793, 0.19209856495557087, 0.25112751948781376, 0.24471886294408082, 0.2505799444349306, 0.24154242534920836, 0.2478007752653119, 0.21270364685473078, 0.2381983036196859, 0.25570833660475273, 0.2320540233565932, 0.19106131129437162, 0.2575312440763525, 0.2718643056694087, 0.25333890569231554, 0.24421073198706178, 0.24418610059040385, 0.24477244731489425, 0.23372170909709025, 0.2551797930642057, 0.21562861400311165, 0.2632101131177596, 0.21799870080668673, 0.15912914667045103, 0.20502331756555653, 0.2240213121219624, 0.22498421281421271, 0.22761383109399597, 0.25802650138422584, 0.21908240549493865, 0.2460165113796016, 0.20922135970090228, 0.20397515082510123, 0.24102138557633548, 0.22153190138704038, 0.25883215088346545, 0.27342004101189893, 0.21471947216692017, 0.18011698393100156, 0.21524588437822514, 0.23960159848568335, 0.2279145128242662, 0.2625190033146378, 0.1915324595565932, 0.2470345101229553, 0.21519291157014475, 0.24478854442017364, 0.21090673072120572, 0.27432727266779855, 0.24026882799858465, 0.24781201785009757, 0.22078105158531655, 0.28688774829920816, 0.21335484943241947, 0.26542701841893157, 0.21462250155674814, 0.2324128482003517, 0.23262249380193176, 0.28390396306603416, 0.23900916899047192, 0.23768046951025606, 0.22416272401540638, 0.2014855486806438, 0.2323148583273408, 0.26541356975048874, 0.22383516004772094, 0.2443630986731325, 0.2339600404156912, 0.2429831013993614, 0.22484727141439684, 0.2580349138144611, 0.22301028229393624, 0.2141938291252142, 0.24414949163865693, 0.23029195912915174, 0.20410868285056166, 0.2414231988439437, 0.22983366832800506, 0.24700342960887475, 0.24063708182366403, 0.2267773297275875, 0.23557250923276066, 0.26665633580731085, 0.22429453097071142, 0.26072764971496226, 0.2187425847952432, 0.20483801802574358, 0.28028289813428225, 0.21774862035901013, 0.18828537888245817, 0.21886043624141818, 0.19974022357710808, 0.25720525677594613, 0.22110754393811283, 0.20973467839065738, 0.26397335921619586, 0.2595061171517093, 0.27422485465046376, 0.23609738841665528, 0.23351024436573564, 0.2227930691502503, 0.20470550132891518, 0.23415282631330409, 0.2592590868323622, 0.22018802872313067, 0.24483922445905704, 0.23680594112405937, 0.23682714206003333, 0.2249709905938839, 0.21836702218965895, 0.2597938639652553, 0.22864656674437597, 0.2626758285144199, 0.2614523269635586, 0.2443695861428335, 0.2138027133385978, 0.2633225990780237, 0.22254719901808717, 0.2605575338247674, 0.2502011733820049, 0.20956011241131497, 0.2374567096301169, 0.25160205293661647, 0.21137569740522763, 0.2543654444039226, 0.23361633319862934, 0.25108176753008116, 0.24117078207569434, 0.22505143286208035, 0.21322909978652246, 0.20896245211917125, 0.2492809375565168, 0.2492504097313145, 0.22408173899672565, 0.20754865242292295, 0.27558138041512775, 0.2510957647598102, 0.17215507119874807, 0.21169460548041227, 0.2324069717195312, 0.2678362295290059, 0.2478135458358793, 0.20440245828820935, 0.2351568813590129, 0.22696642471105777, 0.23692035359544553, 0.2463611025590063, 0.2340521358623203, 0.23793914890152823, 0.2833055677850011, 0.23491605293096285, 0.23143451283156516, 0.2108997719720142, 0.24273652967016235, 0.27087493396064344, 0.2744065734347124, 0.20243895032762702, 0.2318005373920636, 0.2428624323045399, 0.20710091833471136, 0.20553587070690182, 0.22744247089745223, 0.23055184650850913, 0.23105633347654592, 0.2565690485569658, 0.25888113181045164, 0.23991785124299586, 0.22223375500522194, 0.21360333336881906, 0.2397142310562063, 0.21258887635828663, 0.21471085572300946, 0.2322323666553926, 0.2309352386686356, 0.2654864682122848, 0.23596608874504768, 0.23603400843531436, 0.23828226505653963, 0.24198464670123335, 0.25550310523429887, 0.26286189408769933, 0.2160123830182891, 0.20594669587190215, 0.26388482188327117, 0.2551142176028271, 0.19645000083155237, 0.2664309151696573, 0.2228274581929678, 0.23749215578363, 0.23602289572084487, 0.25931414959854204, 0.24682504519980022, 0.25499634235327673, 0.22952903100451233, 0.2195697972328043, 0.20678813036266325, 0.3334014546159908, 0.20761987490172643, 0.21787904744117015, 0.24066326155759882, 0.26630704676132183, 0.24713248786895542, 0.2401828743410516, 0.23599790692009331, 0.18496351597549543, 0.2472338563396354, 0.30215536165201656, 0.21195387031298615, 0.238087008116757, 0.2644078028963688, 0.2333902611774074, 0.2265935091996708, 0.251570541981073, 0.23621163272266302, 0.2719696438160966, 0.2675180717331036, 0.2565610645997721, 0.22976550515001326, 0.19668951870797813, 0.2415903425434002, 0.27981187170978417, 0.20585058122023123, 0.2262638169936835, 0.2519853142153262, 0.22073313833281843, 0.22229902504966625, 0.19547920527085735, 0.20879436421572, 0.25789792798879424, 0.2282062194317701, 0.23785411738313814, 0.21104658373497667, 0.23175276880118073, 0.24658667723225072, 0.3102378937160678, 0.23951019989429134, 0.24971542538671357, 0.22491788139716015, 0.2245450992387083, 0.26390713937136995, 0.263352164606859, 0.2166286979190311, 0.24234517026280247, 0.28826872295661954, 0.2075592245489618, 0.2505121700575744, 0.22521938266857922, 0.2963188990017518, 0.2466664007859286, 0.19259581854268187, 0.24350597314498681, 0.2817929232932387, 0.23817396169901586, 0.2071031139036969, 0.2845240015313351, 0.20755827818569764, 0.22066385716217024, 0.19242932102533886, 0.26328672474828857, 0.2657509876888319, 0.23001418213192507, 0.23894640120491364, 0.26444192660188165, 0.27054626134517057, 0.21334586927857757, 0.26284538328700163, 0.2195115218440234, 0.24203159174652356, 0.2604999306337297, 0.2847921877415098, 0.23588634098267183, 0.23767504590456476, 0.2184574907894548, 0.2204498045002254, 0.2294967676494242, 0.2403809688803011, 0.2502604164219934, 0.2381674741924638, 0.2675848262017433, 0.25250514957236436, 0.21122002215280805, 0.25356193594936177, 0.20898848789049068, 0.22331374438485463, 0.2185393130244466, 0.19682117564648102, 0.25122222452907744, 0.25193916221525564, 0.22201796389684725, 0.25266549942813255, 0.1775238802233762, 0.2561271927006331, 0.22243154273958027, 0.22828154577295148, 0.20962597100887093, 0.25866193235216073, 0.27477839810796184, 0.26579902449825066, 0.26582844201093636, 0.26159274030221974, 0.22362801694877854, 0.1918774409113014, 0.2245201469675133, 0.19104137965161372, 0.22792753135415758, 0.24035166208644876, 0.23694003369611127, 0.23353699711319567, 0.24013381597148784, 0.1968740762097239, 0.20584000732045238, 0.2280974800729774, 0.25849982734057775, 0.24436235228401143, 0.22680279931719988, 0.2545271392118734, 0.2505340558910818, 0.2280411658383178, 0.18427962002748208, 0.24391857580489573, 0.252905690321197, 0.26595854117637413, 0.2565330356362006, 0.22697238540052492, 0.21287609962898538, 0.2226367592402992, 0.2933925601760065, 0.24401358771473775, 0.2305780513875225, 0.21959442677858254, 0.2559733039483207, 0.24278780906858213, 0.23074795867362194, 0.2375922308532506, 0.23124192225695195, 0.23748325607654996, 0.2526784662101042, 0.2377917447070481, 0.2333732565206869, 0.25632818411082003, 0.2309458003857725, 0.20965721678444926, 0.2646678432839315, 0.1949846904853185, 0.22737937890074056, 0.20298539725847076, 0.2618775702198459, 0.2307942848850528, 0.2604810430618018, 0.23012852511423026, 0.2551736470360559, 0.21734979130729154, 0.2341793022842397, 0.29873562319942215, 0.2248190809522937, 0.270336586992506, 0.26856882126422704, 0.2603858549039038, 0.2215975131962229, 0.25602835717257966, 0.21941958888766852, 0.2012601329146903, 0.25635175641821906, 0.2042986330176894, 0.2793603216293954, 0.2661364618020197, 0.20732817052533306, 0.24860684773397193, 0.1992569963856683, 0.2440984395010258, 0.20603987161212994, 0.24107928946950116, 0.23685560132658967, 0.256300141908534, 0.19840762831999148, 0.23726149026522517, 0.281017564695608, 0.22338880936464228, 0.3279241983619068, 0.20172007765040492, 0.22880053236027317, 0.2187873777639319, 0.23570608992419737, 0.25739154220474847, 0.27174552959143766, 0.25229752398924404, 0.22565432751682765, 0.21656983266722918, 0.23152643621407165, 0.24465982991516813, 0.27433091196072606, 0.23656532362171107, 0.22608583140814237, 0.28658416506825896, 0.2467951004963267, 0.21310924208575402, 0.20754061548193478, 0.26112407476351407, 0.22403222132147674, 0.23217922212799313, 0.20825012468471155, 0.21692717456873573, 0.25635249167231094, 0.21724149875994486, 0.2549944976561162, 0.24080304982427653, 0.22617804089624016, 0.24318717783566451, 0.27021996074417076, 0.2606373771698697, 0.2478318427520951, 0.20968833862889003, 0.24337308210211636, 0.22461156207996577, 0.27427030272439673, 0.24479955988057597, 0.25042928590610514, 0.21873650530408645, 0.2189541453782654, 0.2828787855885986, 0.2774439661680998, 0.21749561718784963, 0.17673910505628895, 0.27803499030811524, 0.24623824797020516, 0.21920370686571536, 0.23069967888369475, 0.2364216140805944, 0.22976258531128704, 0.21675956156268134, 0.2479001641600549, 0.17445482786287925, 0.25870429182949467, 0.1955735814746787, 0.2170471752029365, 0.2598610189621468, 0.2669853150949, 0.25178933003883486, 0.3028696999459594, 0.27704162066131066, 0.23335122347907455, 0.24868541807791958, 0.24554910135723415, 0.2361061295459765, 0.22307821276078166, 0.22447195928886826, 0.21441430457895694, 0.2396884608936465, 0.28416484659389557, 0.26640899869730605, 0.22997447424797582, 0.21286512198219107, 0.228881683390362, 0.26306712238205454, 0.2542309109535666, 0.2607628531668277, 0.2574362754115562, 0.24320675916250056, 0.2671810808161598, 0.24819991504928182, 0.19909125815424056, 0.25685557732857806, 0.2636545142948861, 0.2403819824446169, 0.20437605940099782, 0.23444705536276808, 0.2300809584996682, 0.21013883441851988, 0.24311725615664276, 0.2603850425388133, 0.27201724006985567, 0.2755711039672774, 0.24765544119235408, 0.22109046953677436, 0.2598960535801271, 0.2543191504118277, 0.22742365488643435, 0.2270612356707995, 0.25851758391258384, 0.183295845797414, 0.2188165548482707, 0.2294323562192443, 0.20778317653319675, 0.19808757612507338, 0.243155801682956, 0.21580248955799589, 0.23560935469047273, 0.24268165091936797, 0.24697029820315125, 0.19927892677066955, 0.19657989824613975, 0.23792161978867346, 0.26229906263937275, 0.25473935798441333, 0.2189264961563777, 0.25384291572605067, 0.25290304128864727, 0.21613878658178978, 0.21615439631559435, 0.26972532993748277, 0.21993812162378307, 0.23978634547975378, 0.2519235174758514, 0.25757149225519027, 0.2162258355068717, 0.2120564068195219, 0.22578404683930453, 0.20019662650753503, 0.26475542963566456, 0.22804421279359086, 0.24731679149505312, 0.21438011851989117, 0.24324708346167226, 0.22737485070214877, 0.2496127044303777, 0.24112909492620108, 0.19040579984273492, 0.2433746468002899, 0.21216645615677204, 0.22590545027667222, 0.25600961597424354, 0.2125495882006884, 0.2371671182611216, 0.22720926922436438, 0.2315045130331067, 0.248479577541668, 0.21153834907812288, 0.20814422788627351, 0.24884188368665366, 0.19564510082152214, 0.2371741619075537, 0.2767870355711838, 0.22830570329950875, 0.22938804205989696, 0.2667308967092063, 0.25969613932805674, 0.2525857677994558, 0.23500753451859294, 0.2516614646462066, 0.2518989006406257, 0.2256100333849026, 0.23094570912173304, 0.262538429735706, 0.26246417199117006, 0.2434547323816236, 0.23848681367913005, 0.21336764925149126, 0.2481082328190929, 0.21553424234443155, 0.28207717101182544, 0.27185993549799337, 0.22514514994065482, 0.22167794133809443, 0.22957322272756991, 0.21795873325726034, 0.23271084183342458, 0.25270532067260015, 0.24498752160025664, 0.19434474750561614, 0.20952518139157844, 0.2525912787735354, 0.2094345852483793, 0.21928478028991336, 0.2118558136984795, 0.22235160536962145, 0.21350500599705477, 0.26813637410514785, 0.21632150087487464, 0.22756086123716449, 0.240745569526004, 0.23719837427050902, 0.24774814357728248, 0.2851729811508237, 0.20949089990398098, 0.2238460192296917, 0.2332885098417816, 0.24540347100172547, 0.2173131725096222, 0.21291735010868654, 0.25436012832348154, 0.22856789178807035, 0.2618870085898644, 0.22230760570780722, 0.21487229919886372, 0.25321312913822025, 0.258117965988009, 0.16993156705406753, 0.22681624179824855, 0.19063412512859285, 0.2590481902132491, 0.21027081227742464, 0.1732423966148431, 0.2353597173140327, 0.2754223257618869, 0.20691085563051478, 0.24061684639161252, 0.22268321122895765, 0.21877341708103168, 0.19367383950461864, 0.21919013322310432, 0.20310301738034164, 0.29022955841306297, 0.2312839665205163, 0.24249629356942748, 0.2313254118689682, 0.22615110698062604, 0.23102340716633848, 0.2376907834675932, 0.23704102893251344, 0.24486205156043855, 0.262884259283992, 0.23298355378486113, 0.22052866795043677, 0.2455707082332704, 0.2053323161891069, 0.23647480862161874, 0.21086711818379633, 0.22575523877625026, 0.23192002083885316, 0.22910254992305265, 0.24669045818589738, 0.21980268144394366, 0.2831399361736597, 0.23561544349634753, 0.18026357279649566, 0.24139695478243883, 0.2135704787580762, 0.21591677379735233, 0.2514922847663773, 0.22718388044607835, 0.24484304545960953, 0.2295779699386606, 0.22449695939948136, 0.20401225767570416, 0.2561054177127339, 0.23700183195496835, 0.21998127505423998, 0.2364097023097676, 0.21950397171784344, 0.2280545801067522, 0.2470639515708216, 0.22631040355491103, 0.2694446092764876, 0.2382301506378377, 0.24471315085020867, 0.24650101688093326, 0.22024512207481287, 0.23573246572039747, 0.21977081289384884, 0.22805971352908655, 0.23964451565718137, 0.1866134193090567, 0.1980192084442608, 0.23327661691820997, 0.23030809998083904, 0.19330360546879446, 0.22720893530377106, 0.19689468218304126, 0.21195388700621529, 0.2300357184073849, 0.21901512125402864, 0.24042230165800435, 0.25878188743879343, 0.2340318897860075, 0.2657974156962326, 0.22759635734048764, 0.23341416200208293, 0.24602209345771595, 0.24947321339962933, 0.2404387735402126, 0.20419886376586285, 0.27858384783667367, 0.22800671237914694, 0.21360314380659187, 0.2359458378992932, 0.20395252490948895, 0.232672747368338, 0.24771941515920196, 0.2656384692282614, 0.2452721250719509, 0.23812141112657279, 0.209103469671896, 0.2775518543169643, 0.2224961493705176, 0.2097821947893238, 0.2522998569097224, 0.2632371830659845, 0.2367136962051436, 0.21315437177558005, 0.2281981278730212, 0.2027559641267018, 0.23437919910462543, 0.24272602078407246, 0.246053468685058, 0.22543919971220203, 0.22083153569589056, 0.23345615063397762, 0.2194836305157181, 0.26706951286274494, 0.21284289715179447, 0.26833899395221256, 0.26404011023029195, 0.23995814865988568, 0.2474519264083382, 0.2370487816062725, 0.2547573745299108, 0.2383207363128012, 0.23977674776227234, 0.22776875395530655, 0.29576948517663276, 0.22835075404144636, 0.20970296698882673, 0.24649863045818682, 0.21802087553546032, 0.22604937628592112, 0.23998564167390318, 0.2575903384637358, 0.22578509572186273, 0.2507899871584998, 0.2494461798732834, 0.20551982551898149, 0.23352643419046773, 0.26944425060868304, 0.21195558182688218, 0.23835672880925088, 0.24281334897726858, 0.21397550529288326, 0.23186353474317095, 0.21993182943053388, 0.2338851650136396, 0.22015652439742675, 0.18763581205103205, 0.2663854158048165, 0.25945465471201395, 0.2112158263559676, 0.1920398242373428, 0.21990305704018817, 0.27790957579871306, 0.264467150816266, 0.22283781835018956, 0.2706335984539369, 0.2659560614052228, 0.20132676223125567, 0.22141331486455554, 0.23564669008149752, 0.24330522004824215, 0.23784536304209253, 0.22322077382198313, 0.24688292647365362, 0.23926668704606566, 0.20045605922058402, 0.2312224705941409, 0.2259632830462322, 0.26754436195470893, 0.2753815643738211, 0.2149806357841088, 0.22772577681084308, 0.1976289027075166, 0.28096949478009287, 0.22538410210252177, 0.2084994529492128, 0.2669299356516476, 0.2468167625715252, 0.23639861848037977, 0.2574489795090679, 0.26072781245138604, 0.23432909136817046, 0.2459897806360338, 0.27420834556531654, 0.2191908514832953, 0.20782785547354035, 0.17409252598547786, 0.24244085724686645, 0.25820932606824126, 0.19637019725273058, 0.2707005766726483, 0.23347567748759926, 0.2550393194088497, 0.22347125677827964, 0.2026591324160548, 0.22738462339249266, 0.1871128881347155, 0.27078429048688407, 0.2523133410025302, 0.23170802360985515, 0.25208089129506567, 0.25731792756612293, 0.23659519320051745, 0.23162251494297634, 0.2573267876126151, 0.2569642559349597, 0.23089230485336362, 0.24491181599701978, 0.2542274520433611, 0.22001680218794037, 0.1977598882121328, 0.27202297469105347, 0.27734303014661393, 0.17320084013207762, 0.21706459360128924, 0.21714798056246085, 0.23518164091416865, 0.21859870349292712, 0.2156018059443591, 0.26293557608343615, 0.25502911153519653, 0.2323956431768695, 0.2231843894819948, 0.25207037634746665, 0.23310792128864646, 0.23842206998479742, 0.2666612840799859, 0.28186689420346234, 0.20504590178453866, 0.23598114627620914, 0.2502814890575406, 0.23956937547169782, 0.2605721809810426, 0.2417841996059536, 0.2069187478217529, 0.25649169049805254, 0.25088574181055323, 0.1995327514749802, 0.2245101202852105, 0.26607253640159384, 0.22327452844472612, 0.23923917547654316, 0.22742020863920015, 0.22517586926406594, 0.24285745380369356, 0.27685300575673977, 0.2586287619804326, 0.259335717793446, 0.27827681354247374, 0.26681837575924344, 0.26305465435803677, 0.24307119848941502, 0.2365764352678466, 0.24441630219068133, 0.23157102064475213, 0.21307319130908592, 0.20821148926499627, 0.23545663265664038, 0.2907681350326413, 0.2095283573251999, 0.23295905710332682, 0.24324817656443123, 0.2559425663075636, 0.26929143575842734, 0.2336107727989386, 0.20085604888806313, 0.22776495586564818, 0.24921527916623623, 0.24043623804393247, 0.2647853361355258, 0.22495845896515404, 0.2622753687258507, 0.22973471697630407, 0.23637977146650585, 0.2407970527524053, 0.27311111563043927, 0.18234244161331653, 0.2560588616681772, 0.23189858316626283, 0.1951239144122706, 0.2377592843676571, 0.2061503646128508, 0.1971388196702022, 0.23313614263495874, 0.20043535487214595, 0.22607891032779964, 0.24329638557513766, 0.21065442238277138, 0.2634955733369474, 0.24818775137831287, 0.20523629206102503, 0.2176953139682332, 0.1942086180435018, 0.19867025771442634, 0.19755903832276364, 0.24195282880889227, 0.22466571297332308, 0.2448214616170291, 0.22465757099584707, 0.21819439586431272, 0.24115619877343417, 0.2923148870184906, 0.21101789339322685, 0.2315694950036256, 0.21208767684422225, 0.2627105551845536, 0.21520794972085242, 0.24833558234720662, 0.22634437308243494, 0.23658115784512265, 0.24298809912652794, 0.21853507189494584, 0.2627579379578237, 0.22896695375337225, 0.27484151219125375, 0.23062542894269253, 0.24041102157976446, 0.2384042703285192, 0.21730644145304429, 0.20370429711969804, 0.2611957668012233, 0.22582234408931673, 0.26064993773160877, 0.2539172820631325, 0.23577022420402863, 0.22052696381309708, 0.2521466139966177, 0.207904495457711, 0.2535214905585252, 0.22053244939148114, 0.19769997537707987, 0.24883037913072484, 0.23286647072771643, 0.29561848213811054, 0.2770656599436276, 0.23874782669558695, 0.27008964072491903, 0.18124400734270543, 0.1999344400268398, 0.2222589850347002, 0.2575387285770401, 0.2233888382743841, 0.21710442883193978, 0.2795354971399095, 0.27085718057829195, 0.24559228843761666, 0.23987181733201215, 0.2349558188654405, 0.22665941232681922, 0.21974267089063565, 0.21413897868873732, 0.2493111574065519, 0.17615749803647446, 0.23598684144314888, 0.2342820218092929, 0.22831372370410405, 0.19162958203764854, 0.2760248085569944, 0.23003435342760808, 0.20613205651460728, 0.24263709231930866, 0.2439480553063338, 0.2555632817655176, 0.2202528319232695, 0.22637206558021397, 0.24585497839314893, 0.2350405719376886, 0.19300454040996987, 0.23447367343351624, 0.24611521151899235, 0.20458595685270178, 0.21832527636860827, 0.2248768794620389, 0.24421464716063102, 0.2336421813219202, 0.1779099196179112, 0.21800506033164616, 0.2273066484159211, 0.20474290811261525, 0.27638105598712065, 0.2515660557634892, 0.21998108026507499, 0.21645275278265189, 0.23622015258311535, 0.23820997970708535, 0.2212015441159477, 0.23689032094629198, 0.2889512037120128, 0.23360819615105208, 0.1821928558282382, 0.2783723482418974, 0.2762762334343484, 0.1917633659473325, 0.22597200752777719, 0.2235496066146279, 0.22277042409133246, 0.24595838973718354, 0.2568877372805035, 0.2670803416992889, 0.2688935224350788, 0.25191989147907634, 0.21963933638181646, 0.24933662994080127, 0.29820629754113476, 0.22882941488380043, 0.2505172544022537, 0.24546688389145885, 0.23646547132321855, 0.27225882403804774, 0.2633185442947513, 0.21571901590517362, 0.2812421786437434, 0.20440933763579694, 0.2464118941302938, 0.23750405530561194, 0.2306825052921523, 0.2421275440745748, 0.20970057112560592, 0.22787057036743869, 0.26136463109506036, 0.24284094040438156, 0.19990094136463057, 0.25540014758341095, 0.2067154622166299, 0.21758162077001, 0.20872386250200722, 0.23388020684291874, 0.2969159577004443, 0.2226310208830035, 0.22817065190760105, 0.22691937737622628, 0.2279363050127171, 0.24142920061496068, 0.23771207327161972, 0.2207746182651093, 0.23431469069482805, 0.22850562681114092, 0.2217650034969064, 0.24507927009155747, 0.25951242623200577, 0.22296410037409772, 0.23655570493580477, 0.24742815447403035, 0.2239350880620598, 0.23586602890104555, 0.26450944954337147, 0.2311922668761656, 0.24458845942487614, 0.21781235140361307, 0.22885790634906808, 0.24111265136875706, 0.27002686082730754, 0.2806239256045135, 0.20979899325188678, 0.26524448568842235, 0.24261115715698045, 0.22063041371572914, 0.25694433531959926, 0.2628180377418188, 0.22504422534493218, 0.27092327248548137, 0.2394627839612121, 0.2605277390319335, 0.23365109886352484, 0.2395476077831149, 0.23431539071716798, 0.2269561912706734, 0.24782261586321927, 0.21617787950963258, 0.2580007968905944, 0.2126558058478971, 0.2655235723166004, 0.2508133542961233, 0.25177098153568644, 0.2723542284759146, 0.23986847369857134, 0.22046466333595371, 0.20551722200561442, 0.2009474172444211, 0.24365199197721613, 0.2308232005524684, 0.20621105078142088, 0.21235571346022877, 0.25404642086541024, 0.2825996429396722, 0.2217600930583708, 0.18171579596530027, 0.23123602768738039, 0.2104410122792817, 0.2611174750619725, 0.20993530171315, 0.18607646445360584, 0.22853392386963145, 0.2191648562889479, 0.24019542325329146, 0.20242638500489135, 0.2561484672415158, 0.2745189243783253, 0.24395373424627417, 0.22511629330242147, 0.2510769625819004, 0.2501250192356686, 0.21068965382274032, 0.2184767736349559, 0.1975468238040504, 0.23705525670003705, 0.18100886650164497, 0.21335532503914545, 0.23523842742806128, 0.202845926175934, 0.2130559547846591, 0.22202764892141175, 0.21899483913746814, 0.2474495397968159, 0.26300497832571385, 0.28545327149034955, 0.26599570611921747, 0.21716228824453132, 0.21206859703141445, 0.17560828755956934, 0.23954069763230185, 0.2608625919359731, 0.25176611537001803, 0.25875690242335136, 0.20977517931950299, 0.20725681766031823, 0.2323240625541951, 0.2688791862200427, 0.24957751451679286, 0.25486713505536523, 0.21458288222894234, 0.20673777137407412, 0.2168224215541201, 0.23558217446918156, 0.3040306264129256, 0.19488899713795924, 0.24107803461188806, 0.2303752575235122, 0.22140961753486393, 0.2842603180090822, 0.23537377156559375, 0.27283646503768944, 0.24420386490241106, 0.3070587540712893, 0.27872166904554196, 0.22984564473482713, 0.23303484347625122, 0.24938646701452774, 0.25874649611247513, 0.25606949565118514, 0.21738864832858296, 0.22718953949252643, 0.24221828509243443, 0.23260624543913289, 0.23256993299491863, 0.2564581989442919, 0.25970960498163864, 0.27741779051356036, 0.2359259876301029, 0.2390903619927745, 0.23488656224574173, 0.2494150265181974, 0.21549824794734238, 0.27831008025068577, 0.2646189008905303, 0.20476000540064984, 0.21798662440978103, 0.21115489070004123, 0.24633397412263092, 0.23844970270981244, 0.2543392282212004, 0.23518065940303556, 0.24929938018715725, 0.26248828531278007, 0.22718457564838662, 0.22242952237566438, 0.22107281155762162, 0.25621015867997426, 0.2039620674186474, 0.21722246319192748, 0.2615146626619099, 0.17576494258271233, 0.21140562437733118, 0.2718392131544627, 0.31088769886614087, 0.24584311258601269, 0.22906968026512325, 0.213742409968305, 0.2162729001320519, 0.21029482348525805, 0.22856853105838973, 0.2267276358396071, 0.25698876084720745, 0.1997239166398553, 0.2005569028337811, 0.2615917064672132, 0.24322130990854504, 0.2571807872038813, 0.2380625956606516, 0.2335413358699942, 0.2852155516806085, 0.23842794711017168, 0.22122594572067036, 0.192947530358944, 0.23397619269675188, 0.22047020640479661, 0.20898849750384918, 0.2655528262127433, 0.26458446292596893, 0.2741411008076247, 0.2269870728287383, 0.23417143683457842, 0.2576183149491143, 0.21964626431062098, 0.23568936109765398, 0.24932657053033444, 0.2280527724477308, 0.23454965178161255, 0.24128059134279042, 0.24827563723042892, 0.28002796029621685, 0.21058796775895347, 0.2103065101527546, 0.24774397766107845, 0.22676393345017387, 0.23162972123314618, 0.2067993405716689, 0.2604405069635468, 0.17418716634697184, 0.24249844306430407, 0.19901772912242596, 0.2289997368190491, 0.20098145065059866, 0.23995286714892256, 0.24881621746211915, 0.19215300192091364, 0.24408773762740985, 0.19472130854706826, 0.2034020498815642, 0.19992960085212502, 0.25084389863323436, 0.19090967264394892, 0.2585177629908868, 0.26635560418476323, 0.20966072953819592, 0.2591645696654813, 0.26486645963301164, 0.20770949460233118, 0.2243393720106046, 0.22374445765855713, 0.2441264499751253, 0.23655621985829603, 0.2043624646566122, 0.2639474197806552, 0.255848161630961, 0.2690150424928189, 0.2263112250521261, 0.2347137106615694, 0.2271773164188303, 0.28237365485461613, 0.22668338491222753, 0.23755685407378188, 0.2790194838335196, 0.23206736641503153, 0.2309357887517053, 0.22319278963926198, 0.20391638117307623, 0.26543851926776707, 0.24462045981100225, 0.22686609816520314, 0.24380482903919626, 0.22446467998609837, 0.23201333691517143, 0.24956147502383547, 0.24511340308410035, 0.23216270811657994, 0.2170388514923695, 0.301952794431412, 0.24470152874468648, 0.2854354520309679, 0.21693457457750573, 0.22643194614229997, 0.24435318040486884, 0.1900912851125906, 0.2667318083964593, 0.25531775742407753, 0.22327920201836024, 0.21513396512887126, 0.22113212961481987, 0.21276571689204085, 0.28083931450393995, 0.22519963641854956, 0.2866164723377389, 0.24797750981197136, 0.22186857359603832, 0.24859422680906196, 0.24140906027857661, 0.26539600436858984, 0.23293572951605182, 0.2204530939830735, 0.26390935347750566, 0.23740083352419392, 0.26037858030717703, 0.2418280438626535, 0.25715902486546677, 0.24102871160804973, 0.25933102782961853, 0.1975704654633415, 0.22005462326959727, 0.2408714384428213, 0.23066873328657553, 0.2440932839933631, 0.2301275171154511, 0.26451691108127573, 0.2333566988051409, 0.2296417737865137, 0.2696683877737109, 0.2545865197981762, 0.24304277673639513, 0.250577646208388, 0.24979696036410243, 0.22401744428382186, 0.2387401968304353, 0.2853861491930869, 0.2366073009706085, 0.27623352474676577, 0.24423747873056748, 0.20399128765468957, 0.2622408735188447, 0.21620100307711695, 0.23315996994804508, 0.22271462317843568, 0.22042092345017122, 0.25908927609297433, 0.24903891359622765, 0.2608417811611727, 0.2447576583815794, 0.18679755023618988, 0.2553004698728128, 0.22383452229152484, 0.19088856455340042, 0.19044743386453822, 0.2789086949548723, 0.25513686173678685, 0.26784830921891556, 0.24622647831009495, 0.19568793442606514, 0.26484632007845765, 0.25924472055977804, 0.22348239830096756, 0.23479442071451795, 0.21376607457234983, 0.23712742203601192, 0.2689812096147359, 0.2303262076658985, 0.2518543898165926, 0.2381478382876163, 0.22815348071159802, 0.2384524839430192, 0.2202411798874707, 0.2129650943928372, 0.1888081836311642, 0.23978181271766122, 0.22834448202565352, 0.2378103369261702, 0.2383971105342279, 0.2389235580600442, 0.2613582498321489, 0.2676588654347902, 0.2562759346141785, 0.21970726745814947, 0.2973835030394898, 0.21544782507406854, 0.21024197016742746, 0.21942540473698235, 0.23819176851026153, 0.2509282085917558, 0.2589553295030544, 0.20622697517107985, 0.2217825305036332, 0.21412868103403973, 0.2196205211512671, 0.20305983931421334, 0.23300484836472335, 0.235587855634277, 0.23560896463149203, 0.24184311791137675, 0.22765097781184648, 0.2212740787146132, 0.2539566201723242, 0.23352913378636642, 0.23000026851734004, 0.2484908823431842, 0.2538403080438958, 0.26323882290664125, 0.2396796334119609, 0.23050375433797987, 0.25216004943847703, 0.19469021271760067, 0.2817288240406124, 0.2780012158326521, 0.21279292679009051, 0.2174059911157424, 0.22165826748757403, 0.22435010159958801, 0.23512210034657907, 0.22220904538910663, 0.22984095699690732, 0.23946992506833573, 0.2873323788837745, 0.22191329566676915, 0.2074132316117762, 0.27319044338946, 0.2751523685501098, 0.23609712101557417, 0.20253246103424263, 0.21519894011847288, 0.2311419184181017, 0.2029170026977095, 0.24053401200724925, 0.24245271116620612, 0.22054337838797597, 0.24079161140870117, 0.1920736501926961, 0.2679212438895892, 0.21271562636241653, 0.23210078917228114, 0.2450490544506466, 0.28684003516931744, 0.2699029208156007, 0.1891342763592401, 0.21990852886548606, 0.2190473332118596, 0.22619265306485162, 0.2320365867286962, 0.2679818310801912, 0.25644214543446275, 0.2475204091142365, 0.212494472816875, 0.23278853778102754, 0.21443846841124692, 0.2540341445655329, 0.2624075911287153, 0.21799511997012364, 0.2276119844123065, 0.22455536231478113, 0.23013449026516045, 0.2752623987777726, 0.2267705437472887, 0.20160091894234572, 0.21838222864934906, 0.20597567578225134, 0.21851174364306655, 0.2346076897347957, 0.2528374860070056, 0.24108976332838924, 0.23361769246252284, 0.20302316417616678, 0.23344955801617354, 0.24190833167343548, 0.1836958239040564, 0.2975039215353478, 0.22897734855013593, 0.22495471618469784, 0.2246956775493159, 0.23861180195300824, 0.19852663979937452, 0.19180652520479682, 0.2139787647296076, 0.2829016060557522, 0.24379637106911528, 0.251255680084118, 0.24222960633436993, 0.25654539059817905, 0.27410263611046587, 0.2267565015422609, 0.2979908197111771, 0.19947577493778335, 0.22824339436015798, 0.24928734496027788, 0.27343891523306046, 0.23289925158605457, 0.19995231422463358, 0.21666383407577916, 0.23969165728376804, 0.19993381024260495, 0.21374464697416462, 0.2602033055912268, 0.2424623938361266, 0.23721981313385615, 0.22434891024973397, 0.23591172431592472, 0.26412998239530416, 0.2220029848923834, 0.2204531326997565, 0.2456017184402975, 0.23617841786893734, 0.22454030468996913, 0.22075379190705674, 0.21651450366256655, 0.2457863465081984, 0.2456909304388718, 0.25161407021944504, 0.22390355042241047, 0.22442099898991966, 0.25490243782773747, 0.2158862109812726, 0.2328702200882254, 0.18679487288275634, 0.23251193763122688, 0.26600886559609493, 0.19021274559616166, 0.216225851501041, 0.23995764597392047, 0.19793740592172385, 0.2314627970257491, 0.28603847282910966, 0.2545822757213498, 0.2683751045441088, 0.22984295155524984, 0.20640420099469076, 0.19825102957961155, 0.25647295902216577, 0.20244653290298656, 0.23139775313797004, 0.2102724084812587, 0.25702000803825176, 0.26862732729409233, 0.24823050059625215, 0.25151199206113556, 0.2143401491801466, 0.23420998523225217, 0.2624449438548378, 0.2341576893536423, 0.23553358836785535, 0.20861052479837663, 0.23105327118604393, 0.25580153419149254, 0.27148940430792134, 0.24265044084145354, 0.2586660302593466, 0.21683844586368986, 0.223398917861353, 0.2727336931825627, 0.25340905185738327, 0.23188538401673658, 0.26291430831674856, 0.2542923081483914, 0.23384430579097426, 0.2632294034149068, 0.23003983188503616, 0.22706093903068228, 0.23569803647711218, 0.2200169426292038, 0.2123623146437223, 0.2158111269790518, 0.19250592760330135, 0.2224087891599318, 0.2223912665830969, 0.25040893291524013, 0.26299396126159635, 0.17945585645522866, 0.24068660549493773, 0.19805976307930515, 0.2246555943154224, 0.22761742359842052, 0.22488892239897704, 0.22083983211095579, 0.21653778681388286, 0.2546413708590308, 0.22554269894087892, 0.208048993283976, 0.23985445513383225, 0.20470615347899876, 0.17340094093609626, 0.23049426645256074, 0.23609464980476721, 0.23790982438164512, 0.22546260443450578, 0.23562972362242698, 0.2017334477916852, 0.2417944993630205, 0.2753297363200923, 0.23521264373953796, 0.23974014020041917, 0.23234391749420533, 0.2339538711551874, 0.22057492176086296, 0.23602500932829723, 0.23833395231347607, 0.2331083836183566, 0.223547081422264, 0.23147513682108956, 0.24264306140632058, 0.23596758519184993, 0.27185931825978127, 0.2171707849746126, 0.20599979368691415, 0.25646880463071153, 0.22227041266995276, 0.2243750757017714, 0.19205269693957372, 0.23039937716984016, 0.21150013587801075, 0.2637930819343757, 0.23198164765680881, 0.1844421335178134, 0.23625458280161674, 0.1899846667735639, 0.25013312794973724, 0.18737705925962705, 0.24396198420558113, 0.256380596077029, 0.21043556676009414, 0.2323005210382466, 0.20810337442118068, 0.23812457673861503, 0.2463515287821048, 0.253375021406689, 0.20927688453744037, 0.2159614651126467, 0.25017088205754007, 0.19974501762544086, 0.26039206296118694, 0.24176407829987737, 0.23399270701684546, 0.24143452785024308, 0.2187987553104384, 0.17637838051778612, 0.23849928501362544, 0.23935040509476524, 0.21807810699293678, 0.24978552056015038, 0.23200666272632095, 0.24637413862944618, 0.24110325083568807, 0.22221254973236912, 0.2698168075522039, 0.21789532072545906, 0.17263206694545968, 0.21162836939283636, 0.279622384215773, 0.22905258069497042, 0.25322826393028003, 0.24173943242756848, 0.22601872107333565, 0.22888339802282454, 0.23624754376523235, 0.22842432108618796, 0.23492210012297898, 0.2584764430153817, 0.2535766542391167, 0.2278901775922534, 0.18382979310257697, 0.25169218910193597, 0.20636378421605464, 0.23203961842601575, 0.21928476552471754, 0.24668245849821188, 0.22761764813085147, 0.2467265146490838, 0.21221783647882075, 0.26322741809163236, 0.22711321280804656, 0.1992249604122545, 0.2891859282448982, 0.2174820598048682, 0.25345723506029183, 0.24626012578785308, 0.2210947534810007, 0.29743775358931956, 0.2214818290024566, 0.24875727772295048, 0.3022141591548581, 0.2648529549624793, 0.26546026557365465, 0.24404278577662877, 0.22380130528911585, 0.20901164547441256, 0.2563592026642982, 0.25068822425147175, 0.21956938873607312, 0.22815879617515739, 0.20190062945583884, 0.22554404872200662, 0.2191954148301452, 0.20463753990848882, 0.2071439876522331, 0.21499767180575768, 0.2129692746445417, 0.25548247507479777, 0.21661847676999854, 0.2309194922502755, 0.23438789158754633, 0.2572346213463574, 0.20319172858422388, 0.21683388691867275, 0.22915694267830033, 0.24778319257405412, 0.2208907307055561, 0.2118497337546065, 0.2222948541975355, 0.2092754393286513, 0.20352806230857032, 0.26406122391188264, 0.22734313677635537, 0.2623033317894525, 0.23827004707787414, 0.21550681126235752, 0.22413650322694537, 0.25701783005258366, 0.208015718737851, 0.20636987260872175, 0.27016849318120195, 0.2448968892148168, 0.22596325009851542, 0.23567052259766902, 0.26826571950415795, 0.248261872350971, 0.2315949857320756, 0.20073893347423968, 0.18181200020596153, 0.21640509147290338, 0.20108819024979677, 0.23039545640420078, 0.2736019358226616, 0.25159998386183285, 0.2503783474788334, 0.1993684767124697, 0.21304826521854064, 0.22730560699252206, 0.21736741908858023, 0.25136423221801074, 0.23232636513499205, 0.2352888225160493, 0.22329769247524867, 0.20736789618991158, 0.2589664361179736, 0.23680291444230273, 0.19396818279360653, 0.22283177688049455, 0.2521697578918782, 0.24929089419910752, 0.25692852694176144, 0.24600157796421498, 0.2517219363417869, 0.2630379979110619, 0.2355289786187371, 0.22639651224739077, 0.19904140735527592, 0.24998726363304, 0.23705745708137485, 0.23582930587598358, 0.23249771090012264, 0.24762459105178625, 0.25433763478976135, 0.22704343183304584, 0.27395202265303653, 0.2261426937437429, 0.23366744961918734, 0.1891763503901764, 0.2192773871708786, 0.19341239682984857, 0.20787835747056296, 0.24052314217384985, 0.21779398854003074, 0.24135557321795764, 0.2374003660661467, 0.23565088867164624, 0.2107425112079488, 0.22068365002971016, 0.22567498974455216, 0.26546411425640754, 0.21954099295098445, 0.2763456370837375, 0.2542800243701352, 0.2390845334244545, 0.2767318006778746, 0.27115158532991673, 0.1975861783921398, 0.27255708400224954, 0.23934484995724695, 0.22607323736395601, 0.21524531831558852, 0.2235375411942186, 0.2020044705079002, 0.2235049146716482, 0.2722545074409049, 0.2555145437244843, 0.21948953948990363, 0.22000090281786677, 0.22937980227747376, 0.26215564821696347, 0.25907636054428834, 0.261172792977437, 0.26693137866265926, 0.2514976122227208, 0.27614066953264826, 0.1808505097255286, 0.22922811111168695, 0.27635840885798635, 0.2415389472021977, 0.18998226141530225, 0.1913965419955027, 0.251761007495123, 0.22790908045173044, 0.23264124119134474, 0.25506093736473384, 0.2126527556353756, 0.23069924522103222, 0.24235274446664296, 0.22324749755483103, 0.24209609804767462, 0.21173679326856612, 0.21863177327606462, 0.20516059530138628, 0.23058879967420984, 0.17559541594286665, 0.28304460866387154, 0.21414581112943595, 0.20222660044559163, 0.2774010977635623, 0.21505398018615335, 0.25447380779157486, 0.27464186862636863, 0.24491488896267535, 0.2248600683050392, 0.27050046403106137, 0.24436190502570507, 0.23514673418121054, 0.2444507453940963, 0.26441998526879884, 0.21883272708558543, 0.20103636449771217, 0.19742184244971478, 0.20878320709111173, 0.20839981776679173, 0.24702803074248036, 0.2690148539921511, 0.25906309296448093, 0.20878728943768293, 0.21166930053117952, 0.2533836731853179, 0.2320566689042005, 0.2097017041546476, 0.22773435686295398, 0.21341971917755873, 0.2524034326627968, 0.18315438368392592, 0.1952825812415744, 0.24183949974089883, 0.2640368811953232, 0.22607371190049066, 0.19376301565172174, 0.24760870072253507, 0.2458790272958752, 0.23999319382572046, 0.19895497694161307, 0.23492669005552003, 0.2482880712497662, 0.24092022282082823, 0.233322571893403, 0.23352620769558755, 0.2398748590931549, 0.23681963065220232, 0.21897677870387666, 0.24934812088892624, 0.231567461408614, 0.19293125131932953, 0.2516873044780439, 0.23817903398841572, 0.20753974065837552, 0.23957598698096588, 0.22792237915956937, 0.259555068640699, 0.23353526050427656, 0.22767935393258967, 0.1763055702427414, 0.24041649583362096, 0.23757477478296518, 0.2121173145565454, 0.21077060239128595, 0.20346913722278606, 0.23736687110488464, 0.17834638144823967, 0.24820360481233625, 0.24465102934821048, 0.2334327613006718, 0.2528422014037658, 0.23287579696028857, 0.28681604698683094, 0.2291033043615502, 0.24705001532603074, 0.23667441083741805, 0.2029090274678533, 0.23320697850553906, 0.2516393729306056, 0.2509173599446112, 0.22761037200527554, 0.19953702885124686, 0.22424985638459421, 0.2663288499137842, 0.21489996632244143, 0.21861408421078107, 0.2165712202396177, 0.2479495030074727, 0.2682148268946177, 0.20525423514448696, 0.19458159756361143, 0.23605177024071886, 0.26875362610689374, 0.2591668615279661, 0.2555294010422572, 0.21903933522431063, 0.26755134269445524, 0.28164216598302483, 0.24101212643112768, 0.25272225545331495, 0.1980338689751402, 0.2846556118292916, 0.24380881915270827, 0.2619945902174872, 0.23860278915972183, 0.29955171257715785, 0.23980281192030672, 0.23360171236733895, 0.23497147702144364, 0.22872333431399158, 0.25226845031080997, 0.23563145760093068, 0.22536327468473258, 0.2360713127051762, 0.19247215115804375, 0.22238922184939644, 0.2351312573238963, 0.24159302038060082, 0.2167817247226671, 0.25019991649824264, 0.26362533967657575, 0.2390163361276684, 0.21928337148726865, 0.24661846292708905, 0.24179967286594906, 0.2133471288482531, 0.2253719331295677, 0.2340569965887608, 0.23434169866510887, 0.20175583925546234, 0.25938303706696775, 0.18973270314022903, 0.22588944581433867, 0.21649181888137825, 0.2348898045989175, 0.2658663851632121, 0.2142901573445924, 0.24646835507985196, 0.24082474429424816, 0.3065781756816413, 0.22947628681953938, 0.21642662621683542, 0.25105901318418355, 0.26866343391971703, 0.227092792256866, 0.2681481455463018, 0.24330546105656925, 0.24838023497561607, 0.19899743285928712, 0.22401783621545618, 0.3011934451273828, 0.27496749521472463, 0.1991604611274661, 0.24797701248731627, 0.25651470150862377, 0.26983807009167615, 0.20038681089801946, 0.2480707425759296, 0.2545829015395988, 0.27814627251993906, 0.22113265570299978, 0.24475424240323768, 0.24047298235677955, 0.23296242170762524, 0.21985165906904733, 0.22024448682594033, 0.23022249341676346, 0.23580599158468724, 0.22166453295612948, 0.21126645455968152, 0.24999480155517237, 0.23799852978432923, 0.28582493342046156, 0.23441147008222601, 0.23530980271516183, 0.2434947727543359, 0.22766116307793532, 0.24931626216954986, 0.25537650363417397, 0.24629351664093072, 0.23863241055940432, 0.2559592596282663, 0.22284090690723532, 0.23283159267138534, 0.20613620879971495, 0.23310703683793754, 0.23438437416939975, 0.24647129744842222, 0.219385703980707, 0.24770267667413387, 0.24584090463673272, 0.2161580110412015, 0.2496231303420274, 0.24107391531900713, 0.23982069225963518, 0.24355331969771862, 0.23655594394488436, 0.24865299522832016, 0.23472344465262784, 0.22503173985986674, 0.22747878078760192, 0.23061834509763604, 0.29052244756743234, 0.24814646930568565, 0.2581302021229782, 0.24317468938168132, 0.21560319735761085, 0.21895071677674266, 0.20021506761050517, 0.19010534150645056, 0.22992115819673087, 0.236865635370711, 0.24373684702631582, 0.2120242250236649, 0.20716573193550164, 0.24467666810869568, 0.24850852796633932, 0.2503849690217716, 0.24920586046652016, 0.22229087941291578, 0.2712654424903516, 0.24675404394091577, 0.22534160898375782, 0.2685895484626209, 0.2463647958034663, 0.21847002223971754, 0.21223197840480962, 0.2224721592293445, 0.23011208650211182, 0.2292015212657501, 0.25153402558881455, 0.26332949987742277, 0.26197358292979156, 0.17388788225172608, 0.20673976601647118, 0.23299007475559014, 0.23148760483727412, 0.23860666418108434, 0.24455027843922847, 0.2645837912752813, 0.2727893120022849, 0.22995418737949225, 0.26067674559142534, 0.2707787954453322, 0.1961275784819451, 0.24489728580929065, 0.2716422375569024, 0.22241505545965604, 0.26446198669612114, 0.25207192886922414, 0.21079851086146914, 0.19793540283034686, 0.24644711571692682, 0.28190488786052137, 0.270187698225005, 0.25963292230860274, 0.2627672162239474, 0.23485645349730433, 0.24969458010390144, 0.2432672487625135, 0.23115049312293956, 0.23074145764364593, 0.2365413509644647, 0.26369602937763337, 0.17291428535584044, 0.2124962401839803, 0.2188737265916497, 0.20948463797082714, 0.21827312686542508, 0.2518768520871994, 0.2563292394769284, 0.21828353328171343, 0.23236046495786952, 0.22338250844277308, 0.251738377989461, 0.2698768571540169, 0.19376913239874857, 0.2153895236130022, 0.20607486276059375, 0.2237815955748003, 0.27007226231211995, 0.24888683319125188, 0.25831214735998703, 0.22973165089790906, 0.2375281132643672, 0.21679605633857735, 0.219856271699708, 0.22704905964800395, 0.19103295085752794, 0.23312010542462525, 0.21645499938808993, 0.2253408290989563, 0.2443207756670585, 0.25396415208948303, 0.23214185572047336, 0.21469383112794138, 0.30776343371308085, 0.24828211518219942, 0.27218877715285517, 0.20895794178182642, 0.21489288847180624, 0.257407769190876, 0.26015401531005966, 0.21819069651817147, 0.2688176536123029, 0.1898353902744602, 0.22904393327596237, 0.26592633453060105, 0.2206985622148983, 0.24638602247535693, 0.2741614104908387, 0.24531595934346942, 0.2257162722623424, 0.23623024032863085, 0.24012157721459707, 0.2398292202675167, 0.2784779110340855, 0.2601010958987813, 0.19675923414940297, 0.23393707317910592, 0.24618279607119695, 0.21269081672952558, 0.20253678794960064, 0.2254707427179963, 0.24514628514423104, 0.26151887321163114, 0.24266069313307131, 0.24142944209744363, 0.241001587464421, 0.2122845775397275, 0.230832696337728, 0.24021928154031563, 0.24340618555344654, 0.2156492537197622, 0.2001050314182972, 0.23005382936152277, 0.23600430515620335, 0.26101736597918995, 0.25015551158974353, 0.27621129736514327, 0.20026680423611112, 0.24891879156643126, 0.25360609204582996, 0.2558566887141222, 0.2509704001515333, 0.21063344045954321, 0.24234448521258775, 0.23312891000003882, 0.23486503875405307, 0.22669819931545582, 0.23010266896961973, 0.1930294252999482, 0.21422359600192487, 0.2576516308027706, 0.2337236867480124, 0.20965503571668556, 0.21070737604544146, 0.21010665717973373, 0.24410046207813846, 0.1940162030490027, 0.19189975596432868, 0.21401649710739518, 0.24978901454176755, 0.2691692815440171, 0.20283408445867998, 0.2216223425639818, 0.22336446881993924, 0.20218458323499172, 0.2753548979658579, 0.2685716174793786, 0.24699257854748913, 0.23358676546261142, 0.24030935927302177, 0.2237364660139123, 0.23688715606919325, 0.26329298348332786, 0.22541835873758204, 0.24359270810654254, 0.25064638102436426, 0.2173561511205209, 0.24025646103488343, 0.24109479014727975, 0.23159425206225034, 0.19832405884175275, 0.22802285080427442, 0.2369431705517196, 0.21809722618209343, 0.23835005407064142, 0.2219345634610426, 0.2359817746868086, 0.23269097076678055, 0.24904101785564647, 0.2313496602157513, 0.27624881245022676, 0.24055809083451082, 0.22108723458564072, 0.23410951164712027, 0.2629205122457502, 0.26120745492891284, 0.23651329172206537, 0.2970641876513798, 0.21883159980587247, 0.2726628536754249, 0.23164101611848092, 0.20542929231725116, 0.23686339184120264, 0.2161649374067205, 0.2238000373295512, 0.22865489830936586, 0.2558346162814378, 0.20667252127211289, 0.24287088350580685, 0.2526753013113993, 0.2124393549486059, 0.21542610478694801, 0.21895667149125236, 0.23733894518541676, 0.23639531598379035, 0.23401414367046264, 0.2403884290164647, 0.25526817030730786, 0.26543136469517, 0.2266225232373343, 0.2567433350095534, 0.23849969221339015, 0.26334677887285896, 0.22360468400197694, 0.2238448744826559, 0.24984472010381753, 0.21455315192753002, 0.23559062445274176, 0.2550860806210651, 0.2843042824805132, 0.26505917418254243, 0.2421503617070556, 0.19959074540679897, 0.24667059658512505, 0.2514348372019836, 0.23082416910036668, 0.23853571547129065, 0.2382851962100162, 0.23737578992769567, 0.231253212889836, 0.2693280662216059, 0.22959910911230658, 0.24575018921264702, 0.23489053164985552, 0.2330204996639701, 0.24024879717145742, 0.22587528568136345, 0.2765188894649851, 0.20531742708806688, 0.2402972636461162, 0.1802635054184656, 0.27314492086729264, 0.2579081034782562, 0.265675662300174, 0.2934979849263604, 0.2661055574553726, 0.2705202845402136, 0.22748989353908286, 0.23552019747488406, 0.3145660087952563, 0.2236049321694593, 0.2584427859005216, 0.2364699403661916, 0.2813464046542138, 0.2891513282788871, 0.21762406325314548, 0.2511227664901211, 0.2238854790635003, 0.22405082849726546, 0.26087576206422625, 0.22284307641146178, 0.22708754663374012, 0.22756030167364638, 0.2212063319578479, 0.25590433589062134, 0.22986982306562925, 0.2547720360352083, 0.26652197631904045, 0.2099692487908452, 0.26579768643845386, 0.23508623035375714, 0.24785443400943463, 0.2228716253869044, 0.2814293945353166, 0.25934557889994814, 0.2503177327293925, 0.28446262362896324, 0.1938886743950391, 0.2872983591229865, 0.2452265943882292, 0.26510675612390355, 0.23140396208255806, 0.24028601040830289, 0.2299466158500903, 0.24472325356200722, 0.22562948882223155, 0.2925453546132903, 0.19956033995151587, 0.21904987375746204, 0.19665907581610356, 0.20342648081304746, 0.2689096115650178, 0.2579743679292504, 0.237935477730834, 0.23056387603232234, 0.22743192170617493, 0.22631353420471675, 0.2335423036896208, 0.2569679130585193, 0.2633233411673806, 0.22315197546150267, 0.26650919918703075, 0.24756041110741006, 0.2203744227354074, 0.21996097702586148, 0.24536702169991528, 0.2664159136233912, 0.19268878192052474, 0.25708169667713654, 0.22357673197876138, 0.233646723576054, 0.20662696220957938, 0.2680907865624721, 0.21075515279625812, 0.24883147093233132, 0.2451193180215226, 0.26028109497294594, 0.25147268873488887, 0.27402154013251684, 0.22680048630223126, 0.1952709854701846, 0.27387534589431084, 0.2515672738327043, 0.22210127602439747, 0.1950087738335898, 0.2036239398284775, 0.21487380878905474, 0.22479391388821687, 0.1996147343553816, 0.26026631139473916, 0.2682969264460118, 0.2615025836295751, 0.22049154047980815, 0.24576831803102117, 0.28793730627576236, 0.20092145358242564, 0.26497968813316225, 0.19975495839826163, 0.20843298761156998, 0.25015106564739475, 0.23961065700623205, 0.22138613607220461, 0.23223887578173683, 0.2400891894672664, 0.22588467562123993, 0.2446092321036169, 0.24345984510107532, 0.24873831891804174, 0.22257012796219833, 0.20404573488081934, 0.19380547360741496, 0.21331751837301663, 0.2584462697459341, 0.2458879533099794, 0.24468320694149384, 0.23427230567883012, 0.23243641529965228, 0.2146637011283545, 0.241023028894057, 0.22074526472304867, 0.22303729062285735, 0.200149122951953, 0.21972502670872762, 0.23592889511839063, 0.24465497403093855, 0.26207860539214145, 0.17385512395563207, 0.1979191482370578, 0.23125050627281035, 0.19501556775155157, 0.26211278481544853, 0.29113738052185006, 0.25823590153673587, 0.26177812597941325, 0.23181203344634282, 0.19196773105026038, 0.21643490017441944, 0.27065692311747164, 0.27764514207707125, 0.23104678223643807, 0.24107664557622924, 0.22143649395279885, 0.24870161344439, 0.20606659230789745, 0.17581357319189786, 0.2568786976954368, 0.24859154552759116, 0.2531754871679896, 0.18976796027420512, 0.2109687070753259, 0.28036545069291363, 0.16776366188623534, 0.2183227236698839, 0.23897222332978335, 0.2369383301557474, 0.2429217578583728, 0.23895660207698743, 0.25364444135852915, 0.25425378609357546, 0.23927456292374175, 0.2513943927069304, 0.23206071526308047, 0.2633705043448366, 0.22704512613062774, 0.20402494963128345, 0.23580761070610712, 0.24854021783485963, 0.29133914965235563, 0.24454910188670342, 0.23607329630511525, 0.28771802607635777, 0.19830467061333665, 0.1976100467945619, 0.26324156453930214, 0.244863691979299, 0.24754529690522384, 0.2153859636688371, 0.2497457982634463, 0.22842736287578608, 0.21515654089787753, 0.21423691460836772, 0.27838841619777577, 0.18053922270860365, 0.2314335319102419, 0.24867053094126718, 0.24806551280134886, 0.23515210399186162, 0.18527264776936242, 0.24405766907313733, 0.22605018846378191, 0.2610302142892778, 0.2576815610365335, 0.23881215745050688, 0.2354971976807985, 0.2372097179355879, 0.25431402487288196, 0.2370203152204957, 0.23760527785725588, 0.26041281676080774, 0.2310138600965671, 0.21680458439305553, 0.2396911732739625, 0.23195032968764825, 0.21315479611729943, 0.26754344530073126, 0.21222333714751104, 0.2344615340525987, 0.2562831730338566, 0.25426068020922354, 0.2677851312291072, 0.21997976658332388, 0.22007048224731102, 0.21652088099139116, 0.22151606068279941, 0.23731535273707896, 0.20731862772092696, 0.25749037092266214, 0.22820164799348228, 0.18666094396951052, 0.2587394215062353, 0.19004392858175045, 0.1977644444756208, 0.2410148798789238, 0.28102359144341693, 0.255239200055775, 0.26093267077564986, 0.24001664813495607, 0.2346700470013819, 0.22528235935822852, 0.19757324294468187, 0.2073211165185184, 0.2519692352745292, 0.2652772820795547, 0.24993010195669238, 0.21977232330839624, 0.2326269822477572, 0.23597195805484267, 0.2344613509910286, 0.23934727652495238, 0.26902347372434177, 0.25600437388488495, 0.23596579580314195, 0.22852302298314778, 0.22872715295063617, 0.23568419931371945, 0.2723922966429057, 0.20813282874496583, 0.24072611748009276, 0.28393035358323343, 0.251361686321447, 0.191365363850837, 0.261748028134882, 0.242440165186753, 0.20223809125692033, 0.21903355420653225, 0.25923916599440117, 0.25691932666235334, 0.2183901887341311, 0.29461561834976313, 0.2711636964072209, 0.2640165285164281, 0.23320040650374302, 0.2510011375555246, 0.18990148017933184, 0.24146064368678874, 0.22799867286293096, 0.2441222355727316, 0.2269257662397508, 0.21551469794725514, 0.27301199162960893, 0.2479581994692481, 0.25275058982789417, 0.2698654589096779, 0.18221868452744652, 0.21873737145521113, 0.23820046598373398, 0.25481614813259423, 0.2168674808586552, 0.19930739471003778, 0.24005131807383537, 0.21560814946149995, 0.2537582680732367, 0.2476787072929738, 0.24392810007110646, 0.2586839912332915, 0.2533208990687395, 0.23358382900253627, 0.23453820895775968, 0.23518427191184313, 0.26071084186265375, 0.2737287336973817, 0.25561594876908694, 0.2625686053628404, 0.23665385248715415, 0.2428088902474657, 0.26643144647787076, 0.24342548168088018, 0.2598867758305317, 0.23858158088988393, 0.23103319004905673, 0.2314207364914047, 0.19690029674467696, 0.2300684022445118, 0.2175372495998746, 0.22544918902718988, 0.24128481580417527, 0.20825001788508157, 0.22820720585676732, 0.2518641904070453, 0.23282679113952148, 0.24195316067302683, 0.2710420789609488, 0.2355241261030642, 0.22401593132925116, 0.23795680254373147, 0.23287307412665378, 0.3078377649067669, 0.27197516257251647, 0.24713623490917921, 0.26040338200645274, 0.205090091761983, 0.277796299842951, 0.261465481703012, 0.22266715973327486, 0.2741471102909383, 0.23378458446349806, 0.26832254062905736, 0.26958809169882675, 0.24620828497020014, 0.2331856060108823, 0.2723338568151453, 0.22167267977198843, 0.2778152543694981, 0.269103918412856, 0.2490879488149867, 0.20960373740336172, 0.154179939260119, 0.2008431107313156, 0.2213833326135215, 0.2313236881645371, 0.2549783753839083, 0.19819875359784406, 0.272191750532084, 0.23065302172474858, 0.17561754716927572, 0.2375924084704291, 0.1956407285700777, 0.22319612516947648, 0.2625789431092105, 0.26656831245804286, 0.21603792531384208, 0.27159244402265464, 0.22302504528638123, 0.25572163595313957, 0.22265404557707308, 0.25559883528777044, 0.30944525779584997, 0.25444387960810944, 0.2303981124308247, 0.2234890045953387, 0.24750280675346928, 0.25955014473761023, 0.17505708189740715, 0.24328150922348063, 0.24313267005930828, 0.22562880533458443, 0.22697311313749988, 0.22469434443143924, 0.2293889449665832, 0.24511970752284862, 0.24833345346510916, 0.24006609407164803, 0.19128213042804496, 0.23473136477076073, 0.23874213547064532, 0.2514344989411559, 0.24295622830322341, 0.26171499073021515, 0.2406089686634732, 0.220673858720879, 0.18276436189623826, 0.24530998200479068, 0.25969045330891166, 0.2302873692909319, 0.21205653919462522, 0.2458452258672183, 0.2347702880862277, 0.24027653667820134, 0.21927658064220645, 0.24092949425224866, 0.2271391060122093, 0.2625059305463666, 0.24451617043479387, 0.2544755639463168, 0.254398264417137, 0.2533395486390629, 0.2049921672944196, 0.22472840307170486, 0.2122988915671881, 0.26624417707210984, 0.19311050767752458, 0.2252582731721812, 0.2236178780810435, 0.23771255163508556, 0.2207330096440125, 0.2592556866077012, 0.20979300906076145, 0.22702598956758324, 0.23807030903523757, 0.22107903468036716, 0.20661987603501067, 0.2357594115275901, 0.19979767807397414, 0.24162139970697616, 0.24071890580346295, 0.2067259520846762, 0.24103907723304802, 0.24921489415244588, 0.2501373002867888, 0.20966755482940308, 0.26498866145337, 0.19877443729325822, 0.20985035947547384, 0.24263799707300007, 0.2308862455296193, 0.2002392189017083, 0.2447636009208536, 0.19763387137441768, 0.2265037665751678, 0.22403564261706901, 0.24961648477808387, 0.2386194596351986, 0.22301469324374046, 0.25121694697407093, 0.2711900986128552, 0.27633078770242137, 0.24759746685387227, 0.23124512493021665, 0.2120753568184778, 0.2328365715300921, 0.22194089033852718, 0.2507146550007919, 0.24412647266736862, 0.24673119791356396, 0.25567098834031204, 0.25533189452347316, 0.22420485379282334, 0.2008059805890718, 0.20240318634976373, 0.24925110106480594, 0.23796895453674966, 0.2247251744426899, 0.2359206316852326, 0.23999720245626605, 0.24385316091420373, 0.23978409946723447, 0.2482547572461961, 0.2364472595196621, 0.27765922119319475, 0.20957114685810693, 0.23261155178296175, 0.28333993114543776, 0.20734616614990203, 0.2090630150236713, 0.28800444189796326, 0.2501965318341403, 0.21792546889972936, 0.24160543475267357, 0.22643312991775152, 0.23215687391412482, 0.20170788808646392, 0.24570606359322275, 0.24739682996966436, 0.2829578552647371, 0.2634188120310566, 0.22985022135611405, 0.22204749028919638, 0.20050137478304547, 0.20484999007351912, 0.2683557205274829, 0.20868577191636126, 0.24545797743663936, 0.2216368272198242, 0.26335653700619666, 0.23015842756658933, 0.23253139183083352, 0.20733123345684976, 0.20356636062509503, 0.23753732892865573, 0.2568190434661952, 0.23566743949797567, 0.19148842762355073, 0.2032213011613705, 0.19990637364959643, 0.24745769214092392, 0.25117786295842043, 0.19691834839274616, 0.2376738632179488, 0.27414290754529475, 0.2729732813509985, 0.20961704940014567, 0.22355653821137203, 0.25003760987835294, 0.24033206729944007, 0.27224552204250496, 0.2674826875000474, 0.2521023555719678, 0.26436757938510796, 0.24857474805700353, 0.2595458685439091, 0.25094024838194134, 0.25002060815864313, 0.23374456662632545, 0.2919822777213651, 0.2687072165093261, 0.24026464855895335, 0.23022209032046984, 0.24569553064241997, 0.2568498644509879, 0.24526450536888492, 0.24493828152607786, 0.2612040784705554, 0.22952843480252574, 0.22986441486968717, 0.2385776978325013, 0.24380559703414023, 0.22504455950989927, 0.2589390571335603, 0.22296770115318348, 0.2554446712661458, 0.22306586740235662, 0.2102182862014963, 0.22086085188816956, 0.22800362631037893, 0.22828464199721907, 0.229274496827727, 0.22980290333660508, 0.20668083520075903, 0.22371089808176814, 0.2479461289248188, 0.244960588845362, 0.227910730363135, 0.2148258244754996, 0.2260495559171472, 0.2475002091818711, 0.22070704700919688, 0.24274655572462858, 0.21361039109657623, 0.2219301170619014, 0.1920462521255769, 0.2005319883058042, 0.23357505521607902, 0.194992176524255, 0.26186084936412485, 0.23427186063552416, 0.2663252103472409, 0.2557310012782677, 0.23314924230544498, 0.19844472897024604, 0.2286462036732564, 0.20279537575157738, 0.24997247326285177, 0.22809361738199294, 0.27112627560141794, 0.26097431328877485, 0.2113520149391904, 0.21904060322679955, 0.24554202432015992, 0.23997957728639052, 0.2259582012669249, 0.24499463931428336, 0.22763860338742548, 0.25135705204009284, 0.27271582266489813, 0.2538759458637596, 0.22657450603712773, 0.25889091277349874, 0.25080936543596033, 0.2504103181566451, 0.20877673681400025, 0.19575887892005112, 0.2636469970850415, 0.27019985294252385, 0.23459182169559648, 0.21990095857427025, 0.25807152618541207, 0.269066753592587, 0.242635950991969, 0.22323213353477409, 0.25338129912964724, 0.20807271258825105, 0.26611703494161704, 0.25184618078662374, 0.26475795855679396, 0.2339142254608153, 0.22090443339922278, 0.18582397908910123, 0.2638460492019625, 0.19116884949701043, 0.18918684020907361, 0.22611414805913632, 0.22571764404877875, 0.21257847705071092, 0.22973502946601868, 0.23704426452218547, 0.22838555395902577, 0.21265215996109213, 0.24016757051280052, 0.22914444863715225, 0.19834065941968707, 0.2333690914366932, 0.2156296491293362, 0.24384339562641602, 0.2549803990211947, 0.2789348489168297, 0.16824086571771676, 0.20997495140830338, 0.18998171403286332, 0.19159155283622728, 0.24927560580816224, 0.2180797099687211, 0.2361766516794189, 0.22146271209665505, 0.22069213279639543, 0.28724008583177857, 0.24476639712695344, 0.2139449595000349, 0.2556475604567016, 0.204845216018003, 0.2351419886464287, 0.22574490668903432, 0.26381611878580075, 0.24372687288529338, 0.2561258122585428, 0.2009924938095194, 0.25746433336650815, 0.19851061501726505, 0.22046986752050973, 0.22983866123676155, 0.23699810167347254, 0.24353298127501524, 0.24341848848043346, 0.19219527967776473, 0.2067190747780796, 0.26777998877497805, 0.26589656108699516, 0.25996106224346766, 0.2086630408162842, 0.2422120007639105, 0.2439060522321459, 0.21987642711616467, 0.20014204575483885, 0.2411531619066711, 0.18662813529588462, 0.28651392111138096, 0.2576665518862335, 0.22573435603244849, 0.28890516588770515, 0.25306345255573354, 0.21416950433178186, 0.22666894811417715, 0.2173428049401606, 0.2024258632943602, 0.22315977836239317, 0.2656951962501748, 0.2584486695196217, 0.2323030603809445, 0.20199617989699145, 0.21250740225403159, 0.2536711596403191, 0.23235191409100572, 0.23672367184925242, 0.24777001054379277, 0.21760181100643372, 0.234670473752731, 0.2303608273713053, 0.2390549059752549, 0.2331633843410867, 0.21236771981802288, 0.267479497961596, 0.22998901438785596, 0.25302943648045545, 0.22809207852801497, 0.20266787059842398, 0.23885149282061574, 0.23183679918235545, 0.24792709288442155, 0.26294920201866695, 0.253648267145734, 0.25073140820537626, 0.207573081307801, 0.2661007274123773, 0.23643738016660715, 0.2628663992457423, 0.2719729340572322, 0.22051885360001558, 0.2373135011319794, 0.22617421436815327, 0.2679012081848519, 0.23552886178067997, 0.24002605529472276, 0.23922515248532628, 0.2564783449273607, 0.2731258602338539, 0.2319319415246598, 0.21757130275750525, 0.2068816516375864, 0.21737086548813364, 0.27938583071244827, 0.23779007764403054, 0.23454068773165812, 0.23894687514069754, 0.24232537569495155, 0.19669907817398097, 0.29067268841190386, 0.2450769455574748, 0.2290211843824624, 0.27575883074362045, 0.2156364980295678, 0.25295497680352413, 0.2742555033123624, 0.26965826893156003, 0.22403516884459918, 0.2552958746692648, 0.24386707969586222, 0.25971871855762013, 0.22378728945144463, 0.23823787797461202, 0.24872564966505536, 0.29057550751904093, 0.2703635896917773, 0.22922548267945086, 0.24956980187475383, 0.20738705480851796, 0.28322320405039597, 0.24259621566196696, 0.23408945019687175, 0.20424817499691073, 0.2592936201363752, 0.23811736962667443, 0.255987122765566, 0.24385726506157054, 0.1799331487391379, 0.24560502323134187, 0.2528490504737351, 0.23062957753826466, 0.21616693807032897, 0.21799737053936566, 0.22810870638947628, 0.2361000219903227, 0.1759504935275913, 0.1735231812889641, 0.23048489782819614, 0.20396400927374692, 0.17850940130325751, 0.19554264432102345, 0.23735391958556304, 0.25493852468389144, 0.22625601908702278, 0.23797510722001722, 0.24822748351007803, 0.24013141117746942, 0.21164792481894662, 0.2386002198914539, 0.2177590758546232, 0.23071447306318146, 0.19197725420485104, 0.21196787857162255, 0.23564718898437656, 0.2362220707425459, 0.21272874100328895, 0.22161488429143605, 0.23924904849269857, 0.20113393355824025, 0.2833932604978587, 0.2495377994431362, 0.24369668608003273, 0.24503856294446852, 0.243658840210893, 0.2550088501736538, 0.2780794015675167, 0.23808290979489038, 0.20428360580125687, 0.2634529704027568, 0.24835730973953343, 0.2173347596608971, 0.20087790806250963, 0.24756745183073325, 0.2539966232511089, 0.20718391090909458, 0.27837918808768225, 0.21225342640318104, 0.2784295012382115, 0.21387504761683365, 0.1914444667132692, 0.24459717216017818, 0.24962332156612482, 0.1901439554428179, 0.2710306548304521, 0.2450592270717037, 0.2231176513439196, 0.20218226577403667, 0.2224544174869543, 0.2457116039148651, 0.22650036865453482, 0.1885848052762718, 0.21005265777511758, 0.2671734684414172, 0.21970666181573137, 0.22920316995340487, 0.24517979150145322, 0.2133311069375678, 0.2514239502645167, 0.25990730078476604, 0.24923580986485963, 0.27702466159771055, 0.26738171689574997, 0.23023151675003564, 0.23860433605577605, 0.22753501286452135, 0.2523552978145775, 0.24164152088681579, 0.2555450065097182, 0.2259773854999469, 0.27630923586164313, 0.2413407603583451, 0.21600725180646127, 0.22764428024392316, 0.21512388700367954, 0.22233013554840428, 0.22523127913827864, 0.22274634339078433, 0.26380388795341747, 0.26820012616985833, 0.22982626081316654, 0.19333053296330308, 0.2135667241509674, 0.25922437288923394, 0.2926382089504481, 0.24362474718017507, 0.25122683434627463, 0.21558121243977002, 0.23642133136707894, 0.22370202374319548, 0.2152233632163689, 0.21763335716000393, 0.2253122569124833, 0.2540140907886597, 0.2123950794456057, 0.24194305372372094, 0.2436140608216006, 0.24686353466219557, 0.2584567370397802, 0.21917270652289686, 0.22390260284983973, 0.2273083608312105, 0.27287771120930965, 0.24029990482466265, 0.2643663810656534, 0.21169483473292947, 0.24531306692701865, 0.2555724701167741, 0.24642908980963812, 0.21780091690120112, 0.20871681875025436, 0.21045292044876426, 0.2275958246577426, 0.21020003622586794, 0.22291865840555636, 0.25173088623956846, 0.2608384810805428, 0.23328726896100627, 0.232302082915747, 0.22812063699993998, 0.22095026277723542, 0.27449610135050784, 0.2558991847786424, 0.2163157359663058, 0.24295647926943778, 0.24017474192570643, 0.26476778978717513, 0.2267964505067593, 0.25240880640312857, 0.292875972271155, 0.20138289437272613, 0.24651843225164005, 0.21217762925592598, 0.26034725951971, 0.25983233057610144, 0.2655630933137161, 0.21000809318989944, 0.25048087638315925, 0.22509255872072842, 0.2669737880188327, 0.19697887313908216, 0.23902313627589192, 0.21007139455208393, 0.22648846132971445, 0.2775751804135743, 0.25664444944053705, 0.21434161042581992, 0.24520380330834338, 0.20326604312915275, 0.22108868894964748, 0.23457200699638364, 0.19495338324486597, 0.17991904028689193, 0.2060885858361958, 0.23781375412570066, 0.22692381613709503, 0.27450882904415547, 0.22519416890401414, 0.2557541554327273, 0.24294365159091566, 0.25954951543446697, 0.23115277938034165, 0.24908648547562962, 0.23022692812271056, 0.23346641795712816, 0.2198707095370615, 0.21358884826594227, 0.2304976554368566, 0.21932235605514175, 0.21965464971055865, 0.212909974473805, 0.2769630305643971, 0.2544219970264057, 0.22805320275265548, 0.2224998798220827, 0.21991585548281076, 0.24764673432777476, 0.20674482828918994, 0.22560258880485473, 0.189067059333298, 0.22754382494230366, 0.2349264052596777, 0.19262690528696008, 0.25898268511575934, 0.22977902822489102, 0.24159534448106573, 0.22157258912299638, 0.2150079802969858, 0.2649383529694403, 0.21390998504437145, 0.23353450700386877, 0.2759528338571511, 0.27653687583385006, 0.23818674626465847, 0.22269688022630416, 0.24741490069397135, 0.22689451269148136, 0.24107122545725043, 0.24570821243252108, 0.24359529820246215, 0.2538372182884751, 0.22227549771206168, 0.19499984685086333, 0.21200706331599659, 0.2526923923232821, 0.24536504573927714, 0.23883527878573954, 0.2255682669023032, 0.2272449283536104, 0.23627752371378816, 0.22053380840536035, 0.2255926433099903, 0.2284836539480248, 0.23022995614642183, 0.25604967920372007, 0.2220651828043323, 0.28378310750466035, 0.2939441347082251, 0.2549568336348778, 0.2506201728842049, 0.22705259157536506, 0.24045243027965615, 0.1985071041948323, 0.23428531481042056, 0.26375508700389116, 0.25416655304318164, 0.2591885197734037, 0.2417153776978474, 0.2598431841505605, 0.22469242865892766, 0.23513919315581874, 0.21875169127994662, 0.20989510857234747, 0.24896263829324897, 0.3213088529783489, 0.17014363718091163, 0.23920578467666048, 0.2958776269960345, 0.25314616934115014, 0.2428389187131181, 0.2579690602632771, 0.27980274319400683, 0.25144723743590497, 0.310455264922513, 0.25470539691812516, 0.23255557821922684, 0.2692462589333118, 0.2651996234981102, 0.22852957222221337, 0.2524923720909073, 0.19670743285244735, 0.2055185150759195, 0.26205595656846253, 0.19017044515814813, 0.28487403189918337, 0.20798293783218627, 0.28833108894715775, 0.23419620214863515, 0.197511195926444, 0.24061982906552515, 0.2612257333292671, 0.2359930589377135, 0.2446980189323207, 0.2287972609953039, 0.2379644986048591, 0.24006666563086299, 0.19587127476354504, 0.2245012240828668, 0.20193187601714876, 0.23930710018371065, 0.22898961033958498, 0.28394820007302823, 0.26057548281371146, 0.21149033351950244, 0.21932101667064874, 0.25577779562267405, 0.17129138541649735, 0.23820320781015056, 0.24299728733259598, 0.24771108977514494, 0.24820222723193375, 0.2644648403795798, 0.2301518297794545, 0.2457931102183379, 0.2307019504460744, 0.1863659805610406, 0.2305768550613358, 0.2654794198932017, 0.1985362960235204, 0.25028323051347906, 0.2210345673183481, 0.24282607776586826, 0.20668462915277303, 0.2547293122142451, 0.2059612581154013, 0.22829992538999622, 0.2501995915699853, 0.23839279218942624, 0.2308951684673968, 0.23716801408099134, 0.27636680563975136, 0.2629212065764435, 0.19211078500058557, 0.2142631085408906, 0.2877595692776317, 0.24564148148407988, 0.27752959231320157, 0.21048930969992768, 0.2519614026204695, 0.2272712854560723, 0.2727503526737189, 0.2985805686102033, 0.24785488556001134, 0.2266035048751668, 0.24115454902172834, 0.23927662103287473, 0.24296274414358146, 0.20097027751039595, 0.2719539948772625, 0.19260397828670367, 0.2208158542659519, 0.21762840718153062, 0.2588882268084408, 0.24344289911056785, 0.19661004157513676, 0.266098061679135, 0.2975517353211395, 0.22872656503743277, 0.246014455246378, 0.2670516235096551, 0.23181583498770175, 0.2661184753891306, 0.2416024933038237, 0.2912372698245017, 0.24701242107431715, 0.257885983716707, 0.2036308455826167, 0.28047459499253025, 0.2398796008633795, 0.22568729111899555, 0.2662231780061008, 0.22663131211783194, 0.2152878237418522, 0.2248201295309825, 0.227711824560863, 0.27436526943222117, 0.25729654889743203, 0.23798710808291235, 0.26666167568962384, 0.21250081235046897, 0.24346433455978542, 0.2244701268515162, 0.2914134649398521, 0.19130721722850721, 0.2468370977132909, 0.23639285658088027, 0.2181803678369464, 0.2308592487670513, 0.19657122334496183, 0.21755343429111537, 0.2016934345510291, 0.29380631079459274, 0.23115538908697914, 0.20503452351165324, 0.23541497431042457, 0.21509232440525416, 0.2426282094025152, 0.19739669556388956, 0.2692051307378407, 0.22957363497216227, 0.21597816876275572, 0.2755104453369746, 0.2447524853167932, 0.2543411185671789, 0.1868048598900594, 0.22541492073955016, 0.24250292968129045, 0.21018487695088572, 0.26460897779853954, 0.25037253924710035, 0.2491235140959914, 0.30639694926944233, 0.22034793178451706, 0.24154285468675882, 0.21741929682245004, 0.2012405323439769, 0.23426805721881458, 0.267774032805907, 0.2638469289975381, 0.24026541874521354, 0.23466838029189835, 0.21362586981632067, 0.24957830843015946, 0.2223085387948399, 0.2174672242244797, 0.20749485606084436, 0.2539122401023744, 0.22796954847966505, 0.2568643225277759, 0.2663194780956091, 0.22688492351612535, 0.2731960928444566, 0.24174092144030282, 0.232716063844319, 0.24198120507849594, 0.23379479094872235, 0.23253823178928437, 0.22561205999504144, 0.21542481348716544, 0.20127436226325118, 0.25667217339174325, 0.20961123350430572, 0.2746577435636761, 0.23080336824259526, 0.2089738806669917, 0.2318032892773264, 0.2975375451457257, 0.27922890894122027, 0.25991755247875165, 0.25800647409825644, 0.2022160482944974, 0.21090531997862114, 0.22896313624686807, 0.23738855664154834, 0.21294302932987355, 0.21489013184646444, 0.20289125145743628, 0.21921027749515784, 0.23518963002380383, 0.25600106854793664, 0.24429094094064832, 0.24016292923219104, 0.23531559084713524, 0.21336435118212227, 0.239727558214438, 0.2505296991048532, 0.22195259167602266, 0.18618252244462885, 0.2682599435998504, 0.22775320959259268, 0.25978554225193595, 0.3018579739755263, 0.25204149919946356, 0.23697878074303025, 0.2389404962101307, 0.2035778627504108, 0.26592542986711215, 0.20843575698252587, 0.2470654585928319, 0.2144480453741353, 0.22188357519916463, 0.19436349009858545, 0.23164260770766146, 0.2339163659404917, 0.26643868685892363, 0.2389322063626062, 0.20873550778599126, 0.26762562999182554, 0.23627746393368265, 0.2051067536666982, 0.23160303805404303, 0.26135328333562735, 0.22917967063313577, 0.258564755451463, 0.22029715405678327, 0.2928976964215184, 0.18835105399206784, 0.22241766444949534, 0.21577219146099014, 0.21463699393837904, 0.21341518562099238, 0.2297499323599331, 0.19490601787401202, 0.2484202379226949, 0.27041853251906056, 0.23138594915244315, 0.22141378441748763, 0.2301379089816222, 0.2618558574666591, 0.22888902339167108, 0.22548232091158874, 0.23302966269226, 0.22917208455239133, 0.22993641390801503, 0.21926911757623488, 0.2844736261817686, 0.2514990360473933, 0.21960521724076862, 0.2556691162185227, 0.24948231810358273, 0.24615867277823886, 0.24634970507538376, 0.2944130574135824, 0.27536087952009636, 0.2850315140898619, 0.2295243132415493, 0.2625317119240112, 0.1982100982924312, 0.2231522184478225, 0.22351994012061963, 0.22234790214903868, 0.21882259575670931, 0.24630990775188133, 0.2508918139400417, 0.24852991843667024, 0.2620231482454153, 0.2266442020936826, 0.2585413387975689, 0.20520342512645381, 0.2654400089813671, 0.25165603069439774, 0.2429019492858784, 0.23921483431188673, 0.24851524726425328, 0.24300108138913457, 0.20672790999489862, 0.2673465825857069, 0.22800353339013485, 0.26824086891077864, 0.22881705369438723, 0.23462570218406706, 0.2712761634637991, 0.23305850299345898, 0.21444177360197358, 0.25872169409172674, 0.3216832184324498, 0.2575194011998489, 0.20609963346779797, 0.24339583423882688, 0.23898386025403176, 0.21630366952484328, 0.22602797145448222, 0.21121600051316683, 0.18012849411232223, 0.23810252869061602, 0.17569002926428376, 0.2277433130235054, 0.2579939347406827, 0.23747960190366132, 0.24512848602578138, 0.25530904651738234, 0.23519828073997268, 0.20340883295257314, 0.2893043646294058, 0.23239769434558935, 0.21434498186437756, 0.27057135401078136, 0.25343517263850396, 0.25675067162781223, 0.2781598554156186, 0.26123672139584114, 0.22356783344504755, 0.24730963904521555, 0.22352222995945978, 0.23029658322055924, 0.23245627229339014, 0.26520348415776973, 0.24014973947576781, 0.22000434959485288, 0.25646245292280434, 0.24612893034782965, 0.25107737582646256, 0.21994691674940153, 0.22036576441092745, 0.24499068301110138, 0.2670060789154708, 0.23097287701563346, 0.2325367582951621, 0.20973270583797815, 0.2097854266826488, 0.21203346731080513, 0.259697573400272, 0.28006915425018664, 0.22065081541287046, 0.21955081501729107, 0.29927905838288993, 0.21170571515878947, 0.20956923212728215, 0.22911675352851119, 0.2611129866191147, 0.24532933812610436, 0.2795548947627069, 0.26886503607780665, 0.23142718656606664, 0.1775645324786207, 0.2891189898137079, 0.24409223046637665, 0.20197231169248434, 0.20728705680985784, 0.21629520174473318, 0.24549801016124212, 0.2108344665668191, 0.25079791296811255, 0.30001942096562156, 0.21929107138223478, 0.20171549096713387, 0.23556688299282652, 0.2760121965788909, 0.23811488885476273, 0.26039617808522675, 0.252447487278829, 0.3072841833866211, 0.22829397480310468, 0.25868434222221665, 0.27274760312914614, 0.2273882483008835, 0.2481166049336254, 0.21275632956503754, 0.2167608347472503, 0.20161748360485282, 0.22359559639044832, 0.18966645499385157, 0.2620182603489327, 0.20292878460493644, 0.2619433890034866, 0.2387269758202165, 0.25171698831645667, 0.22941936429293808, 0.2681185189653787, 0.22107442173356767, 0.270600097035276, 0.2593824781873684, 0.22867196632300799, 0.24970364133357303, 0.2300723662099413, 0.2394064709781496, 0.24847411512568032, 0.24198049776338357, 0.21911167887492014, 0.2617415804676425, 0.26351635919405036, 0.20801047204966464, 0.21302014590260357, 0.23278853520315007, 0.222208528617654, 0.21111292657216366, 0.24364664670053957, 0.22712663967087549, 0.22202162406685, 0.23741557506063163, 0.23435871719042234, 0.21804810851104006, 0.2736467943249455, 0.2612821252957867, 0.2290449111357054, 0.21610674237079452, 0.24007394238801225, 0.22508779667357443, 0.23455786002017126, 0.24734729117644386, 0.22810888918836955, 0.24911311657463578, 0.27961471064632254, 0.2770951600297621, 0.22687962698992026, 0.22476805144642623, 0.23224155944773583, 0.22260046321379387, 0.2648112484433738, 0.21964846757805168, 0.23513116474964296, 0.22160583182579954, 0.2237960642342031, 0.22076100859273162, 0.25890711320665116, 0.23916710988365694, 0.2521535920447641, 0.2267229067628215, 0.21346258879353233, 0.2501422117381749, 0.2203477236950426, 0.20977119736653985, 0.229665182837282, 0.2367233688370888, 0.26248588986481663, 0.25247858723757594, 0.19372765712079007, 0.19313434745219843, 0.2353002050587599, 0.2588507932617996, 0.25186164727140387, 0.24310275420722288, 0.24668497782990303, 0.21837157584035746, 0.20856567286257063, 0.24460145522585186, 0.27280574501901644, 0.2610515149620961, 0.21962109749048866, 0.24724229468586076, 0.24061104393287086, 0.22089940904476843, 0.2473167847488074, 0.25241248648220055, 0.26923670718621884, 0.22230740000721969, 0.24106441668568823, 0.2313842651856235, 0.24418226548101005, 0.2236778138340734, 0.2611713171079963, 0.19455183620010288, 0.22347886367772465, 0.21652036915137635, 0.1995749690295811, 0.26619899229010746, 0.2590570588584284, 0.22529340184237998, 0.257109868151589, 0.21735841590071703, 0.2489447495103059, 0.20384269630819832, 0.19667192164171782, 0.2534787416449048, 0.24187613906925265, 0.23629258953892007, 0.2532800024806235, 0.23811795488419812, 0.20554301692021726, 0.2517523002864339, 0.22172011408231898, 0.22220405258215267, 0.20707591621690952, 0.24864272217155342, 0.25081184626738995, 0.2293590990753976, 0.2633393572659695, 0.24344400514133893, 0.23492301483431077, 0.21964613077340145, 0.25999867847427904, 0.22844124992015485, 0.20810890919506142, 0.21659837221211015, 0.24642105069162687, 0.24160440705158698, 0.2591346659778617, 0.2575410349929323, 0.24708145087591388, 0.23362925002575016, 0.21812949484260863, 0.323463374403702, 0.25138165844682525, 0.24359116302873005, 0.2613991383127748, 0.26019115034957274, 0.25676108363457784, 0.23309322651709982, 0.20760848697235937, 0.23323015125403324, 0.2438169484367901, 0.23207856465441276, 0.2154948475414016, 0.22117660124614136, 0.2527189650651988, 0.19672957849926792, 0.21040253092009142, 0.2584222245776722, 0.2897353460059908, 0.2763353371865448, 0.20549450274824918, 0.2325751054893669, 0.24308876130582924, 0.26612222648863426, 0.21339098675361043, 0.25158565826549306, 0.22605776562738705, 0.2015210794112107, 0.22799478181694977, 0.2462261874337358, 0.2331985414687475, 0.2601794220538184, 0.19652148939362565, 0.22880969058766432, 0.1796935944700822, 0.1781757813572584, 0.24229742017097822, 0.20723467553474784, 0.19595798169979656, 0.23244958988816447, 0.26477781759195623, 0.1856862110580883, 0.23117816868241697, 0.27580218022166186, 0.252661694461034, 0.22264697458845317, 0.25886321089606074, 0.2344181146208867, 0.21258674305105618, 0.20069865951927082, 0.2335092027135278, 0.29553082964455013, 0.23370508434359996, 0.22018235122303803, 0.2400353700992873, 0.22968348964560814, 0.2448361499770839, 0.220279502161147, 0.2534880539118681, 0.23156143444321614, 0.2652189983241683, 0.22410657539526666, 0.19694189794773073, 0.18532501983571803, 0.19090955880922034, 0.21940865749133895, 0.22239948861184713, 0.2380011463242748, 0.23599343181893803, 0.26019224267798974, 0.22509180196047532, 0.23977021577162505, 0.2254209077692004, 0.2441827071579974, 0.2273480887828007, 0.20191466588098578, 0.21639560720387196, 0.22240859674474361, 0.24995588019744377, 0.24105056712600686, 0.20813702712986162, 0.23066930328050245, 0.2509253289510461, 0.2558136987467545, 0.23995644696489082, 0.23621493700872476, 0.18951997085874592, 0.2407381558622959, 0.18339236535998046, 0.2594561278556166, 0.23894684144223108, 0.24493938647943933, 0.2615421121398091, 0.2644884269643522, 0.22645132401055765, 0.25562672549641857, 0.31787342157187853, 0.24103450230739595, 0.24715825477574177, 0.20106539725604458, 0.26644714591852586, 0.272642055763897, 0.2765920275608514, 0.22833977591915244, 0.2121683199523226, 0.2171916461729196, 0.20284170790984166, 0.2724421887556581, 0.26887883299554727, 0.21970692459349667, 0.2406119114058135, 0.22393410195297245, 0.2849989474354545, 0.2175937665089919, 0.25471773924885543, 0.24097109325610572, 0.2389781703034658, 0.2198251230748623, 0.2326071744913226, 0.19937690605683372, 0.23022412735781922, 0.268547144274211, 0.2648706026009669, 0.2503493142064604, 0.2201647435831357, 0.26918022176908907, 0.23545679675474937, 0.219331407679349, 0.24679797867020087, 0.25118177190473573, 0.22836198449225178, 0.2362907768700123, 0.25551190537594265, 0.28711396698577457, 0.2380614948992884, 0.22729622686111503, 0.26027984281703354, 0.2620905605193305, 0.22502684020736474, 0.24751106247416474, 0.21075920690913116, 0.27181316099085034, 0.25832381045267194, 0.24186771840319385, 0.25575625195978385, 0.21116035999747684, 0.25832925778697036, 0.2574434660454034, 0.22254369810453725, 0.26834253275535797, 0.2964047785680253, 0.22155378799294254, 0.25137936383801446, 0.24566643773789348, 0.2088208986752546, 0.25937362481905846, 0.21276857841022564, 0.2765604956877802, 0.19420839084575423, 0.23885036822627465, 0.25695565205301196, 0.2210071054967348, 0.22416629755006165, 0.22942580448762892, 0.27483405247984705, 0.2220357581740196, 0.21930776704085306, 0.2813989910019133, 0.2521926958787132, 0.2537087093819078, 0.2053102377441943, 0.30431754147870177, 0.246074082298262, 0.26162905858807767, 0.2433103137314481, 0.1944088716387681, 0.18574653245622957, 0.24165427488246238, 0.2645067573425851, 0.2399143774532212, 0.22530383845926635, 0.22569555502770502, 0.2640355969779692, 0.2439989107222264, 0.20996776201494619, 0.21109565081634662, 0.2442832618374823, 0.26624649575831993, 0.25762524241003865, 0.2122466162670154, 0.23874997557884864, 0.2782840855754483, 0.23934682194952062, 0.22579883441002885, 0.1994798494643637, 0.2154564928468104, 0.21836950386152235, 0.20177053064844166, 0.2172755063387026, 0.2610644624686284, 0.25160811104091013, 0.22481509604625813, 0.24011377108003296, 0.21866139933427903, 0.1874355463366779, 0.21935651493321673, 0.2646100205926192, 0.25155320394821545, 0.2341215245773134, 0.23709055422930655, 0.23287564088916007, 0.21558448503797187, 0.24357428312325624, 0.26032404490432753, 0.2415509783184367, 0.2271128339207743, 0.24958482450351824, 0.24505866266753995, 0.17834842285467967, 0.24051892662699037, 0.27700088872462375, 0.24445922779035587, 0.21530009568506742, 0.23995791992927892, 0.2480248134662477, 0.2748651191527511, 0.21464674274866796, 0.23091793659916693, 0.2567887495664317, 0.25712572830107, 0.2083527149435704, 0.23181732907252736, 0.23096792824311158, 0.2587461835812401, 0.258150446193083, 0.17391139017550694, 0.24201794852421896, 0.24374544974964446, 0.216920125806377, 0.2537977742026941, 0.25500825949490147, 0.24362092571582006, 0.2494155168051933, 0.23349187250035766, 0.24382126307788968, 0.23890415472319845, 0.24293543198510917, 0.25020169370647916, 0.2657192450659156, 0.2744465102952253, 0.2132413948998097, 0.2215642035511249, 0.24631333067247885, 0.23268215461823977, 0.23151647023200256, 0.2483550754238494, 0.25574757135784537, 0.24548512854845717, 0.2579516238010077, 0.25296529453706557, 0.19902023783288855, 0.23746589547580718, 0.24861503688186615, 0.2050341110299815, 0.21885021243265054, 0.20882852929210072, 0.2550925789208922, 0.20990480447053678, 0.20332910109050878, 0.17662389660262934, 0.1632517631611088, 0.2366722663774821, 0.19592778563935753, 0.21631353399836584, 0.2587285248883113, 0.22527811716998714, 0.2837971078634434, 0.20859857551937272, 0.21354231617723698, 0.2308835272956608, 0.24116285967944429, 0.20615464881565385, 0.2538995477815863, 0.23693104853870225, 0.25340707350428193, 0.22545283062758809, 0.24400482096401285, 0.19747529690127227, 0.18723249826803784, 0.23200665429788653, 0.19620633665129716, 0.25676996201107016, 0.21872723275330744, 0.23313723276207318, 0.22877537631446285, 0.2197925937532916, 0.21634386447188197, 0.2515785940362688, 0.2597578791409704, 0.26825317492954964, 0.19956849481603625, 0.2700075658518375, 0.22690089652095385, 0.251421022338338, 0.24350103326228748, 0.22918184717146106, 0.2111258836293473, 0.27878739621558474, 0.1909292697948199, 0.24256928442967463, 0.22474019060539935, 0.22335984096063166, 0.2233794776171087, 0.24167886425844715, 0.22744274943813808, 0.221226018523417, 0.20221920640106222, 0.20299717851645424, 0.29331043618375235, 0.21062672432816182, 0.24120690970992764, 0.22682015013923126, 0.1835992471820895, 0.23490111919974963, 0.2644355596686975, 0.23911310963435625, 0.22503234100155758, 0.21139851799713874, 0.21610567147483145, 0.2517133456803753, 0.2096809441116232, 0.21064011579992734, 0.22422697844808234, 0.2305916422456338, 0.1949497146194676, 0.23458999904480352, 0.22440336488423404, 0.22202139559470227, 0.2592567944590349, 0.20373403411459917, 0.22362541427800667, 0.21854238293143907, 0.26129402873171076, 0.2247313721232376, 0.2359618254901503, 0.24346904218739632, 0.24797974834800462, 0.2513639233690051, 0.20601857326207873, 0.22786206547960458, 0.22825300677980978, 0.22224867720939312, 0.21848524312016593, 0.2590885964312579, 0.22134180973677925, 0.2367449376534925, 0.22941346705789592, 0.23325158942107044, 0.18172087230842143, 0.21522052849633816, 0.2584417574982005, 0.22723607869289877, 0.23194512241976498, 0.26418642883237664, 0.20142979647645448, 0.267310715808408, 0.24749252083941362, 0.2644681185054029, 0.26208978586654086, 0.22047498227007092, 0.17994748641566854, 0.2819632747831687, 0.2759286727242393, 0.23477777791848226, 0.20390268628468405, 0.28540432881586925, 0.25068686535918244, 0.22204384024613918, 0.2443249210699682, 0.2863322446306792, 0.22009776115066387, 0.2402356846193803, 0.23106046513480194, 0.22787072068579187, 0.22399080476251548, 0.23853147789733467, 0.22215762421631632, 0.28353003999255794, 0.24834157214598912, 0.2442802172783227, 0.2534386751493658, 0.25609409535594113, 0.20203989834708935, 0.2562813683609587, 0.22445740459673053, 0.26594624930486055, 0.33594851770248557, 0.2255639714235508, 0.2628355460711426, 0.22529069000966478, 0.24964426098082632, 0.2308274191260964, 0.28039688095388726, 0.2350838990447793, 0.23201928362023547, 0.23548308295866882, 0.22639921135889693, 0.20699260087556284, 0.20912940942280306, 0.19973891886916845, 0.2038439946872062, 0.19012622398437923, 0.2535088810070719, 0.21904671893498975, 0.23956781780698347, 0.2647854766251655, 0.28159828991705477, 0.2621692788698506, 0.23123092684082833, 0.20126209002145068, 0.23205081316414172, 0.27168257901345977, 0.2365501338463167, 0.27317947047310115, 0.2334879422147035, 0.22246549301145052, 0.2348153047636203, 0.25308765718389, 0.19782346748042148, 0.2235233913975642, 0.21544325522567892, 0.2880978334320328, 0.22989642743494662, 0.3078519972865484, 0.20666838800092507, 0.24448558803872253, 0.2420673107980782, 0.272832763388252, 0.24331888281480274, 0.24605119344200732, 0.2631752843883332, 0.2143907888333067, 0.23696794566844884, 0.2582352297451492, 0.2547787577967721, 0.2692536143572381, 0.19736313488131582, 0.20073997482942693, 0.2539054281369943, 0.22561501850875398, 0.24488244379857887, 0.22336441096475024, 0.26460906571037857, 0.3021108800745531, 0.21450134295892329, 0.25034336071755176, 0.24228869983279044, 0.2435254925176621, 0.22968569963784746, 0.2454188986011032, 0.23185075203276131, 0.26221620989204447, 0.26133648660434955, 0.22008428944235048, 0.19629245644156784, 0.2983152595589115, 0.2065890942881227, 0.2471082281960797, 0.22512326545263553, 0.2248088421198335, 0.27731013140468874, 0.23740963409156263, 0.19959234539032136, 0.23190030824095786, 0.22300600171928842, 0.1940489584829279, 0.2239142664833259, 0.1838150405609063, 0.21099569254510717, 0.20414685025742227, 0.20381698683406316, 0.23387789168861106, 0.2227774387081548, 0.21330188374881934, 0.23945858323610847, 0.23093743188995425, 0.20804148947741705, 0.22905579619994437, 0.23961865117756884, 0.2304069704108204, 0.2754258835856284, 0.22450401628246738, 0.21615111784808372, 0.2012724570382331, 0.21509859855813424, 0.25298122860503075, 0.2566783884949044, 0.2393636122677489, 0.24605145278029816, 0.20405933605507537, 0.22825351779901049, 0.1829357739490052, 0.22089650798706026, 0.24118110589368283, 0.2210880088559133, 0.22162871517211438, 0.21020980823516802, 0.2240557887764255, 0.3030650359551469, 0.2605935202793727, 0.19932092699720164, 0.24864659857063684, 0.239151336672472, 0.27940546596438576, 0.2325193215044264, 0.2286801146732275, 0.19858844765225042, 0.22004787119144023, 0.2726007739348147, 0.30207014542529026, 0.2330966581893667, 0.22618069322256407, 0.2479913479751908, 0.2502828831475095, 0.23851376623532153, 0.22186403475708028, 0.2342880457495797, 0.2589644197982013, 0.24517979696023753, 0.22676217503716772, 0.21054500442206947, 0.22027893748567062, 0.2887165292084408, 0.2114135451508751, 0.24330187939611359, 0.2683777221733074, 0.25000584767105327, 0.2334114813404616, 0.23365936466723722, 0.2233286391116164, 0.22585810639938514, 0.24862681548043716, 0.23307958463104403, 0.25974272930134523, 0.23268423392325766, 0.17744722313705757, 0.20544446735380284, 0.26619510231869364, 0.22904328134513957, 0.24552918824221767, 0.23243453361012764, 0.25095066289174806, 0.27276376275689557, 0.2709073867390665, 0.23883867170548642, 0.2633630985176926, 0.2685657828941754, 0.2561389238159621, 0.29116349608089404, 0.26816291827841143, 0.20961292773452966, 0.24030982433155126, 0.19845704843266113, 0.23904217319443244, 0.27225975496462285, 0.2577265805687865, 0.2477137963167318, 0.24772558617190715, 0.219962825482905, 0.22193151875775516, 0.23827817920288086, 0.1875285106703627, 0.2431881325504618, 0.1878777001516544, 0.19032608727345557, 0.22218482821166669, 0.290081186898016, 0.24282735730013624, 0.22142344363861538, 0.21705141475020215, 0.2586798464628654, 0.2755401787691145, 0.23978664560096724, 0.23950453505142696, 0.2199004213452043, 0.2650409923992225, 0.19514221409759547, 0.18995602062837413, 0.2134060166677073, 0.24805377233408463, 0.2693009426636618, 0.27169307748570504, 0.25811779063911017, 0.2251111963630593, 0.23444495993086095, 0.20401395379000145, 0.21921283185979173, 0.22664760586847602, 0.25937466807312715, 0.18742221360760808, 0.1958982580746551, 0.25658889164126003, 0.20266643880949842, 0.20122083347634193, 0.22741577441437397, 0.23325019581743373, 0.262642502694252, 0.2813406137879587, 0.24407726763713922, 0.22884579885723916, 0.21109258693795496, 0.26262740291268283, 0.2097782533946076, 0.21068894013186507, 0.3060674318852499, 0.2426716062458021, 0.23660385106989382, 0.23056099431572855, 0.2590236771196969, 0.2472077268504547, 0.2168071243176141, 0.24285816023460569, 0.25016532578074635, 0.32410130323828645, 0.24355936438721837, 0.21299727440074404, 0.2841095990745878, 0.22646356390105568, 0.20932911484294947, 0.24709492154656537, 0.2243830338549834, 0.2321984069661206, 0.23374906780202054, 0.2018903475826649, 0.24966006211097702, 0.2423898063411765, 0.2572813659710788, 0.22281230968873336, 0.2365649331698122, 0.20559549873539548, 0.2451121810533166, 0.2228823431410984, 0.2773229335457353, 0.2583829617079791, 0.23795371257922956, 0.23610424301881225, 0.22690385929233242, 0.21119922224510643, 0.2742728926834147, 0.22434793522862376, 0.21975527564409306, 0.22790222150135112, 0.24211514956530408, 0.24612772592642612, 0.23202200337269188, 0.2722082934867216, 0.2562468416464922, 0.24511691383746523, 0.23261028260963437, 0.21236324614632252, 0.203737391652488, 0.2362174769110458, 0.24220010885625326, 0.24601519157481502, 0.1822869428799052, 0.22855068530894254, 0.23214032381659766, 0.2558778144641019, 0.2151440308752664, 0.22895618652983848, 0.19940253527491986, 0.26537495121627824, 0.2526138036533276, 0.24768012329012004, 0.22995842347773868, 0.2580439885325401, 0.25396098665593086, 0.2682352730569071, 0.23522224830420424, 0.2443161915460351, 0.2548478766155576, 0.2416020532963709, 0.2265343495737721, 0.20659059754698, 0.2169277893698757, 0.26855080731757947, 0.22375698166688493, 0.225372309170116, 0.22355708593078705, 0.22681731290298804, 0.23904376147399733, 0.2184452075788826, 0.2657772468415208, 0.24756357571760643, 0.2118768122289658, 0.2344709389351954, 0.23297300564448709, 0.20215880210890275, 0.2596738681027483, 0.2576291988627713, 0.25003876277510134, 0.26844722766379975, 0.2610598925231457, 0.21139645731662246, 0.24455628026749845, 0.23018332819730591, 0.20512133398003282, 0.23157869229691236, 0.24064078925965968, 0.22582033785699884, 0.26680498745867764, 0.2090289438428392, 0.22531450224999602, 0.23457657828264813, 0.21765347426902332, 0.25086123146305117, 0.2635271913989119, 0.26751994546068775, 0.2862088866609513, 0.21663107086245828, 0.23953606897800234, 0.23136849669058385, 0.20257796983048815, 0.24915760466016684, 0.21434576076415707, 0.21913958276600806, 0.24438740245189325, 0.27505051203446784, 0.25512276868533545, 0.21296527907876742, 0.2664605590582069, 0.21231823633794566, 0.25310412495998014, 0.25153391023822147, 0.21592711128021205, 0.28899527553502263, 0.2310273157514575, 0.22580563712084195, 0.2340862609889467, 0.24347545320724395, 0.2561031599422516, 0.20407585625046573, 0.23168085878433978, 0.22776141106653616, 0.22297609846181551, 0.249836254617169, 0.2044274181064156, 0.2392317259762151, 0.2521542912564399, 0.24109118658062362, 0.22790811024477356, 0.20904550547441822, 0.20471476558772547, 0.22511916794254402, 0.2369546031801432, 0.25038781807826194, 0.2719614306509429, 0.2799285388638035, 0.2820057417245425, 0.1888886217168856, 0.22314895671217422, 0.23783054017064878, 0.2519198216343959, 0.21914929286816162, 0.20980467300886846, 0.2351293855002927, 0.23684178343116302, 0.23949099272341465, 0.22228806987614447, 0.2368652222449658, 0.23579093068020604, 0.22910655361361562, 0.23706514639024093, 0.23393725934199738, 0.2257641642786538, 0.29684736569469017, 0.24408729379583188, 0.22651883285112934, 0.22605783135744117, 0.281530400530458, 0.2216433216495693, 0.20830538913067634, 0.24347692998204246, 0.28166841748197086, 0.23461901542083985, 0.19269689788247532, 0.239218618283402, 0.24671631598776894, 0.2884904586845265, 0.24620650503386332, 0.24363496873854068, 0.22217346812856795, 0.21182536496947985, 0.26480841735702343, 0.24847022243830452, 0.2507960629815626, 0.23524005474579054, 0.24040036841556678, 0.2641818318499763, 0.20834369017396506, 0.2227130273065532, 0.2129281544479894, 0.22331885692617426, 0.24969190131833083, 0.19707531428925038, 0.27210232663955497, 0.26422349762100567, 0.23213309441152133, 0.18995038150179583, 0.23571480086463492, 0.20731430329527373, 0.22514283603181398, 0.2236622114850078, 0.24596024357898857, 0.27981719634543034, 0.2310772423978815, 0.23119498976122962, 0.24873262216339861, 0.20155647279509753, 0.23272077002043884, 0.26774020875245264, 0.22895729573197818, 0.23960545616860604, 0.211488451566915, 0.28250016394415844, 0.23400732884340417, 0.21666091195332285, 0.22657818907779909, 0.20879791213906246, 0.21423369183982416, 0.22262340456219076, 0.2423855604675892, 0.2464421889064146, 0.22648381627957664, 0.23929112920032916, 0.2965370831138317, 0.2364572892427355, 0.2132475622631673, 0.22968061525378328, 0.19002311977432434, 0.1943146445090588, 0.20582430639698904, 0.22274852375497575, 0.22872062145713173, 0.2513539345578146, 0.20082284407953135, 0.2757648469651654, 0.23416863120814163, 0.2645548029099317, 0.24678222581314158, 0.21158883970749, 0.20211541715307338, 0.24290536772521848, 0.23446841811392277, 0.20861714610086568, 0.2755640126485875, 0.2591168786508837, 0.2174936871089316, 0.21012230379725885, 0.19769913627000674, 0.2385831848401947, 0.2518655769753164, 0.2834464384711596, 0.2693259410070227, 0.20416853659534245, 0.23541379314842809, 0.22672684960858352, 0.2298565253685529, 0.2083555791187831, 0.2447144101372372, 0.26564416788014933, 0.2319598610978534, 0.228008907306938, 0.2539219489813845, 0.22481620863450558, 0.2686409634910378, 0.24523224480184488, 0.2880815272844408, 0.26065565831575904, 0.23370919023951797, 0.24031552051465885, 0.22887147477585448, 0.24076574514263063, 0.20679919556853935, 0.2607683948887451, 0.25688682294268306, 0.24415440363767849, 0.2683146788178567, 0.21066673326544116, 0.2417687621218029, 0.2097157957749261, 0.2462801896096613, 0.18903866619687162, 0.2672232165414962, 0.21551236726474746, 0.20095319811354764, 0.25720360371153744, 0.24741587585930425, 0.2468034270696272, 0.24323472236398042, 0.20512763061483316, 0.19416530672580043, 0.27081498278129, 0.252202489433869, 0.1865115423438337, 0.2567464860794968, 0.22882891196160166, 0.17925615664687417, 0.2487113920972445, 0.28138035951113216, 0.1965944419410683, 0.2687614512164739, 0.2255692710252089, 0.26256672869289144, 0.23505970134306045, 0.22648262503703848, 0.2681540054178838, 0.2320111436213013, 0.23098324106275603, 0.2121042351892609, 0.22406345282738005, 0.25165538275153493, 0.23852801304072488, 0.20250019612568418, 0.26462649012394496, 0.22285399891115348, 0.22723904825000163, 0.274272323067661, 0.25005381171857893, 0.24990747355296405, 0.2497518462984915, 0.24719391498944204, 0.23670868538324868, 0.20201645652548006, 0.20578553088964602, 0.2348614502148546, 0.2572691752045679, 0.20565288406409443, 0.22284017175260748, 0.22956919263116485, 0.23206809548268656, 0.23728263109035186, 0.25785888953206476, 0.2410427198604239, 0.21715267580181322, 0.20921762606029407, 0.2420238314497093, 0.22260183513508372, 0.23284030026420816, 0.23152952916098482, 0.1843895768230805, 0.23774467342469568, 0.2362997682749236, 0.27571107886440155, 0.23797925568417228, 0.2376356865873811, 0.22705749199254913, 0.27032407586539975, 0.21350176899339451, 0.23223026762673768, 0.2044609333778736, 0.2846493503502555, 0.2529229100657752, 0.2349541607859682, 0.23593608612071407, 0.23968844893994726, 0.24963805073628387, 0.2398947397962939, 0.22821946158165482, 0.28041329013532856, 0.2567947782962904, 0.19952223081898213, 0.22165963045827025, 0.23203162464557614, 0.22532194986077114, 0.2573849248917059, 0.22160070695154688, 0.2231257152547495, 0.16602851647061675, 0.23056836439636014, 0.23495109599114283, 0.24713873180985452, 0.24797837032628597, 0.2253923580788232, 0.21143132715913734, 0.26072782822415597, 0.2377929041638489, 0.2540706317021182, 0.20923542261522168, 0.19358605511669993, 0.26159496428795576, 0.3109560707526483, 0.21419300204607725, 0.24268149741324269, 0.2146500132069491, 0.23622662924643797, 0.2816643764704174, 0.2376238753034071, 0.3155890923500028, 0.2627655332858888, 0.2178045483341727, 0.21049512287080538, 0.2503487367797051, 0.2508224218888106, 0.2652256539483027, 0.24406910416994446, 0.23552573951529093, 0.21636402036576557, 0.23482039214715375, 0.2247047560763531, 0.20435218790670132, 0.252294101673664, 0.2022662053661764, 0.2404450998802204, 0.24830158572690555, 0.22505898239540215, 0.21856239617107295, 0.23945983858578393, 0.2781575146140239, 0.2363291989904998, 0.2462966592553715, 0.18894822395088226, 0.18882581415629376, 0.18910189918311987, 0.22896874630557676, 0.2233716505352556, 0.21528213507542193, 0.26460973034518703, 0.2705829493296976, 0.26614382723865265, 0.20302095528648018, 0.22596650038461344, 0.1930848326819814, 0.23012440878829518, 0.2519207004727507, 0.24033632765435384, 0.25624662175676544, 0.1744734915476789, 0.19844206536597564, 0.2771573303573483, 0.23306711842241273, 0.21290218851794585, 0.2193814181020867, 0.22617723466010028, 0.25178981037179543, 0.26403492103353027, 0.2471652794913755, 0.17355097282604415, 0.22619580259806032, 0.2272546771996941, 0.22650040810014085, 0.19183260983310052, 0.2632518140661434, 0.2868127642756287, 0.2268567379198905, 0.2137895335421148, 0.21578671569135754, 0.21260014212802214, 0.254069805787301, 0.2396245521044093, 0.25687156548334045, 0.20924166425903262, 0.254575706213444, 0.23766172318420975, 0.25857357870439357, 0.2454503771171185, 0.21568441647742156, 0.27889063866819747, 0.24912994560982493, 0.2442660411310419, 0.2272461797593231, 0.2617027812304163, 0.23847412269832322, 0.22552250314978264, 0.25537954765187004, 0.19487522919617084, 0.24119494011455805, 0.21355789534850483, 0.17985080685785523, 0.2721812956146746, 0.27315783297776747, 0.2642613550535653, 0.2546279982766858, 0.23379000355080864, 0.2099702683985505, 0.21758080625577172, 0.23775602172749527, 0.26674839190771865, 0.24406380740192013, 0.2664613379995232, 0.2510675142621902, 0.26191255280114045, 0.24055996575184396, 0.271998462443722, 0.24498411208985904, 0.18605070725245015, 0.24084917062871966, 0.23178806224893825, 0.22074043147483685, 0.24466496676635924, 0.2267867266173176, 0.25071885377984304, 0.2095052913866444, 0.21342109384415422, 0.21151050833416274, 0.2212198124682838, 0.2963295073363555, 0.24274349977916118, 0.22935010883120455, 0.2053851877372305, 0.1889770935311231, 0.2588489881549912, 0.20006660207050883, 0.2504262617538863, 0.21457655511089338, 0.2571191098563592, 0.24059964834351172, 0.2146978551109736, 0.2468879642551266, 0.20590372559243003, 0.20377681554104868, 0.23228433587720893, 0.22786371334038874, 0.21292976727657914, 0.2605411437531453, 0.24914879720305144, 0.18808914843017152, 0.2522545590067978, 0.21451811591212822, 0.22345669880378627, 0.1905623700241049, 0.29404093153636285, 0.22950661364135086, 0.2005376468292923, 0.23044808535467112, 0.22198113741237854, 0.23909815761418549, 0.22173305044426794, 0.2910166458925814, 0.2642939034319877, 0.2108846172992328, 0.2155559124648521, 0.1989436791733708, 0.22043414409763978, 0.20345881925592565, 0.24850323916086414, 0.21122710633947234, 0.21265368782629243, 0.23777605108942182, 0.24506556386068426, 0.27950156924363395, 0.23333540539544012, 0.29556450775426707, 0.2086605557879749, 0.22712812883432298, 0.2121941436826773, 0.2549470904384562, 0.23033717746603763, 0.2737360942674771, 0.2942103151718856, 0.23899233632838116, 0.22867326299552787, 0.2584554785582738, 0.19140966848486596, 0.21951172414747405, 0.2387291632523368, 0.21948233740360526, 0.2912497994697108, 0.2839353585955267, 0.22453841340141953, 0.2694178382480576, 0.23953241427766012, 0.2222714174635235, 0.2199812584362035, 0.21137580311277662, 0.2548050832767974, 0.2606985456772207, 0.2207043263366686, 0.22284184483810718, 0.24553375209117814, 0.2332820197400544, 0.195618630663361, 0.2078765999372056, 0.2496682474692625, 0.2603022873824742, 0.22478478291763068, 0.21974921873177528, 0.2163214854614632, 0.24888835130552586, 0.24460193683108278, 0.2051037960763896, 0.26307008038366425, 0.2776613287949262, 0.28868365595746415, 0.23949205092721146, 0.23463425725128226, 0.23169925832889668, 0.262987402019635, 0.2697645414645194, 0.3101084129988674, 0.22825772547570075, 0.2165657936693331, 0.23218404919075114, 0.23962467880833657, 0.2943670114318987, 0.21571290266753607, 0.2409385035691501, 0.21986027628643856, 0.23737126327313318, 0.25459482107799036, 0.2381080731386753, 0.2408571419430327, 0.2356791696503005, 0.2602054707805536, 0.21001151074110896, 0.2527167150916769, 0.2040058282460698, 0.21780737356306906, 0.24194180093017376, 0.24506963679385668, 0.23419655351565785, 0.23685867300312566, 0.21730157938528266, 0.2590482359442509, 0.2654856336843042, 0.23233282955467885, 0.25104952863972885, 0.2194585289363517]
mean_norm_PCMCI = 0.18631502705536102 # norm_try_PCMCI
sd_norms_PCMCI = np.std(norms_PCMCI)

In [ ]:
z_statistic_PCMCI = mean_norm_PCMCI / (sd_norms_PCMCI / m)
print(z_statistic_PCMCI)

In [ ]:
z_statistic_PCMCI_432_600 = (0.1903673093120785  - 0.18788567936502548) / (sd_norms_PCMCI / m)
print(z_statistic_PCMCI_432_600)
z_statistic_PCMCI_600_864 = ( 0.18788567936502548 - 0.18631502705536102) / (sd_norms_PCMCI / m)
print(z_statistic_PCMCI_600_864)
z_statistic_PCMCI_864_20000 = (0.18631502705536102 - 0.18439702079318868) / (sd_norms_PCMCI / m)
print(z_statistic_PCMCI_864_20000)
z_statistic_PCMCI_20000_10000 = (0.18439702079318868 - 0.18304493098429017) / (sd_norms_PCMCI / m)
print(z_statistic_PCMCI_20000_10000)

In [ ]:
# Z score for T=10,000 as test
norms_big_PCMCI = [0.18558514836683163, 0.19751152196237817, 0.1826880562825586, 0.1954658574302998, 0.1893522436847457, 0.18692646112888572, 0.20439260595292544, 0.19097894256167486, 0.1854486404171097, 0.1889029979898218, 0.18027726166457014, 0.18632964635316798, 0.18745429380490666, 0.19717580160789613, 0.18887473062118262, 0.17493351792811385, 0.18160405162835322, 0.1915375225948053, 0.18013098187697155, 0.19269034024415507, 0.19113540643745175, 0.19370518402687611, 0.19392612377442323, 0.18760860907092458, 0.18557141212565226, 0.18963094692759647, 0.1897529190747908, 0.17265018512181135, 0.18660645865509978, 0.18979998620484553, 0.18795002827421134, 0.1960364867672985, 0.18780117090341064, 0.1903200726867382, 0.19674456828198825, 0.1754982637678565, 0.19402245265373474, 0.19456813472556378, 0.18819445573190696, 0.19512775178323777, 0.18359092376400318, 0.17899357061609295, 0.18627453737665173, 0.19306517132112194, 0.1799623617982962, 0.1899275394433948, 0.17728927219389704, 0.19048270653006974, 0.18704673635106683, 0.19345112050642213, 0.1943654686828987, 0.19373577379995344, 0.19070878299577382, 0.18899317300579466, 0.19822332425527062, 0.18351868313739084, 0.18765280065330103, 0.19570807953485608, 0.18433311420455784, 0.19158316067505896, 0.19607121008254358, 0.18808357917227678, 0.17895959335217648, 0.19439010549726912, 0.18057503559281063, 0.18133854672199853, 0.19202444872254357, 0.18085755494442735, 0.17638182860143814, 0.18348322962329827, 0.1837954911361871, 0.1832723730871549, 0.18837652357146853, 0.18264023544803437, 0.18693081509892157, 0.19007283155615312, 0.18811209313837265, 0.1831265266468667, 0.19238522753891876, 0.17762703308474942, 0.19223605073465327, 0.19073630113534884, 0.18514866139222996, 0.18899994520064273, 0.1760469344768838, 0.18615661368093792, 0.17972072524360092, 0.18306463694460212, 0.18477566482142072, 0.18222515882771698, 0.18393750454123037, 0.18724101099068238, 0.20318546542501978, 0.16487475175305416, 0.18624439748745286, 0.18110679794250506, 0.1818082529629289, 0.18541946296696538, 0.19252420161468994, 0.1941747107992591, 0.19157858445718712, 0.18133441159600763, 0.19171809322283398, 0.1871671044683798, 0.19251217635421783, 0.18753733628445185, 0.17522992331048617, 0.18554757880097542, 0.17406193717651594, 0.18137335984281877, 0.19281615550195824, 0.183141398272675, 0.19676065673910287, 0.18232868724070536, 0.18440232413510263, 0.1894893718062645, 0.18725045537642615, 0.18558290297684216, 0.18541124466235456, 0.1964545878754928, 0.1851499545779322, 0.18401586886055246, 0.19652046749801047, 0.20216518307050171, 0.18837845975120432, 0.19295637031029714, 0.19216169804111383, 0.18261510537593686, 0.182659146817437, 0.17567719436788126, 0.18928395977606585, 0.19520730787293067, 0.18697341815601576, 0.18699396840825697, 0.1874989611286799, 0.19438001016055764, 0.19183192721403208, 0.17612891731349536, 0.19049343494396193, 0.19485860134657407, 0.18259304095787202, 0.17877976856041947, 0.19009451734301483, 0.19002613038122407, 0.18284719470159924, 0.19527991197963307, 0.195729754391106, 0.18861190587878776, 0.1729797349885267, 0.19109001040160792, 0.19113488257707692, 0.18999992457516163, 0.18035902465117207, 0.19613754998594557, 0.19157206521765738, 0.1883325818157026, 0.1888681827855177, 0.18890376357739605, 0.19121577780739238, 0.17394362171290756, 0.1957562216946175, 0.19020472554184772, 0.18387001344262335, 0.19565256108016288, 0.19211649319079274, 0.195035194899956, 0.18957013040459542, 0.18009375387195103, 0.18747403307350602, 0.19701760031958018, 0.19006975187290584, 0.18972997239441458, 0.20084815749602916, 0.18918959841229763, 0.17965454612249768, 0.18402979951779966, 0.18067703027882465, 0.18318759375268678, 0.17926123802380237, 0.18327268912996203, 0.18897158782222373, 0.1951480701557319, 0.19582030251642477, 0.1803215167991227, 0.18538922338690889, 0.19113307501479726, 0.19554216607266625, 0.18940497902221298, 0.20237688633217732, 0.1911027226178281, 0.1922232666710827, 0.18732071608026044, 0.19026719993020588, 0.1818683640611364, 0.18350905258283237, 0.18318476909625525, 0.1932330748090721, 0.19250852613935443, 0.1903014110424027, 0.1892618865665531, 0.18502520990706847, 0.1893175570880022, 0.19088289334254152, 0.17807444762225966, 0.1855249106874913, 0.1828800350867853, 0.19813121065877626, 0.19978101726292025, 0.1794671990085169, 0.20157848255936073, 0.1938482957736148, 0.19597839157285254, 0.18010278259496876, 0.18564970245447787, 0.19087758123858092, 0.186427262780544, 0.19357135302544273, 0.20279601468004338, 0.19408240831780574, 0.17848123812129538, 0.1874212381270033, 0.19434348856173966, 0.18429792796646724, 0.19375856163835795, 0.19257200627012933, 0.18963820714359764, 0.18147943173901968, 0.1902697998235097, 0.19178110449376956, 0.18390300840638854, 0.18097702015743528, 0.19019494380160448, 0.18504038068964848, 0.18198020646042656, 0.19376778797827218, 0.19068161458569519, 0.19207711006866543, 0.19016323851376513, 0.18694701939445937, 0.2006689450785218, 0.19086366388546763, 0.17270494145675203, 0.1895433361196975, 0.1817152974145958, 0.1901107332235925, 0.18616260715351818, 0.18128378937672657, 0.18843993723385574, 0.18030031241705666, 0.17052826885396566, 0.20176319059751385, 0.18720641488156578, 0.19211365238016181, 0.19696032789075132, 0.18634713644452322, 0.17761686566807877, 0.19267622744981894, 0.17575560918896196, 0.1865304916004289, 0.18811988159980794, 0.1739687612982674, 0.1966596298180653, 0.19370094726156772, 0.19149141477588325, 0.1747890903562823, 0.18481596157785043, 0.1881736358677528, 0.1782928985748862, 0.18272160942473284, 0.17759006874122477, 0.1858321071050106, 0.19954581316601464, 0.18523441844045538, 0.18802218367621734, 0.19658896288881159, 0.20064027811779578, 0.18468523667778713, 0.18507777170488487, 0.19594545670971092, 0.18701336079714626, 0.1898554734639777, 0.195267952786056, 0.18134833201340614, 0.19176949322417536, 0.18174829291975492, 0.19263387622364106, 0.1854891844795998, 0.1873182210483189, 0.18652481463640014, 0.18889661789476134, 0.19081842345966274, 0.20107350961868614, 0.18568839617392913, 0.19424243115236842, 0.18588784582213547, 0.18664905921455877, 0.18012053512284756, 0.18855931802441742, 0.18512814409152545, 0.18777935664344417, 0.1890461380681939, 0.18413272740208958, 0.19480084191537286, 0.18566112047883238, 0.18452971521560604, 0.1772319295349384, 0.17518117888666232, 0.18135321150637698, 0.17738765103340964, 0.18346642360131657, 0.18750665865408658, 0.1942326679986407, 0.18165567825512657, 0.1995542848414886, 0.1881131296204122, 0.18493081219696547, 0.20115521644039563, 0.18773822729201403, 0.1717173824702065, 0.18983145841266258, 0.18468386694402777, 0.1898468818770816, 0.1927802470787816, 0.19432758256321933, 0.20254974580318072, 0.19001316106793745, 0.19014871555226345, 0.19507744572029007, 0.1964504248459933, 0.18345004880498555, 0.18632269901861265, 0.17691028012247095, 0.17662232395090066, 0.17775025531052652, 0.17696187297523214, 0.18912342614355868, 0.18919070430973264, 0.18971540682728086, 0.1917579918070227, 0.1826264426354336, 0.18146135043259923, 0.18700997077911657, 0.18791826682853405, 0.19684817537423754, 0.1867617783935297, 0.1892257734802182, 0.18872526627298525, 0.18779563696149584, 0.1906687784738838, 0.1821537378491817, 0.1782866429553208, 0.18582033420787197, 0.18223791124052766, 0.18409832943035107, 0.17955169714380062, 0.19149431807885778, 0.18099153385647393, 0.18237740770566377, 0.18334250271461025, 0.17860292278725365, 0.16830932709541502, 0.19165381529484715, 0.1698687432595523, 0.19864094663141962, 0.18785191507284432, 0.17840057322512046, 0.1841461540633777, 0.18950227448898632, 0.19989037515323035, 0.1971625738739721, 0.18743800784408252, 0.1911691660710346, 0.1973227619664078, 0.18546258748666697, 0.19109589979960337, 0.1877177081630391, 0.1909200852480533, 0.1801458481912271, 0.18492379395398534, 0.18826588804089026, 0.19137056671620273, 0.1898252212322214, 0.19418281618012473, 0.18872903836164034, 0.1916831229328114, 0.16799701914075085, 0.18325291453058434, 0.17909095253180557, 0.18448430794257936, 0.18244666959817604, 0.18514093060462591, 0.18691541996398195, 0.1926353228109534, 0.19393299718971796, 0.1848507180359342, 0.18617256792084, 0.1820590281296563, 0.1857139746126733, 0.1983321371807086, 0.19567058533518736, 0.19100638323105543, 0.1877764390210001, 0.19356453883908836, 0.19171381026149323, 0.1888577834840829, 0.19805527999916564, 0.18632968218178214, 0.18206642866697964, 0.18560397336783166, 0.18945220798038206, 0.1910501345516951, 0.1912400291092985, 0.19629077908308537, 0.19028025480988928, 0.183366547817634, 0.1850597471858072, 0.1846924493782727, 0.19185167052248084, 0.20543671687477225, 0.1888395848033636, 0.18524329351025406, 0.19499476281748523, 0.2019883313318368, 0.18921811863498003, 0.1838090866288534, 0.19612344804087636, 0.20049718857805207, 0.1815723664673751, 0.18912657317590992, 0.1874227080390363, 0.19170875092518863, 0.19446225138787182, 0.18603490663722572, 0.19197081279219202, 0.1964350553944098, 0.19945602888460934, 0.18698837579536448, 0.17647159446424637, 0.1881745414514923, 0.18885840185756983, 0.1978615830410869, 0.19067774930364514, 0.18473098226329596, 0.19086450282108972, 0.18300537561044627, 0.17916324886050192, 0.1856489475871296, 0.18561555498803256, 0.184274324430064, 0.19468774659714488, 0.1822512907927649, 0.18274601601445353, 0.1870669349121263, 0.1960822390322853, 0.19095367523114423, 0.18784122406507175, 0.1898498147464811, 0.1823803732296313, 0.19175830508280906, 0.19085346111617152, 0.19551215688939783, 0.18849823535749066, 0.18818943920717562, 0.1833904904702077, 0.19468175242996102, 0.19756854989531888, 0.1986434974873477, 0.19905958460561793, 0.18422230182656527, 0.19754058155647497, 0.1791168084541427, 0.19381867279181303, 0.186427260993079, 0.19352699825206657, 0.19533506726462485, 0.19293247459997498, 0.19126622498862828, 0.192346238613021, 0.1932040543940385, 0.1774498802601476, 0.17843033975954092, 0.1825081786729383, 0.18301868978324007, 0.19061613229812369, 0.189215660442645, 0.19584460325947745, 0.1869794860292629, 0.19128910546742806, 0.1979990145230662, 0.19657075752991976, 0.18804394600472102, 0.1880699491939846, 0.18394027661153572, 0.1909859794678181, 0.19126194867489354, 0.19021818902535953, 0.1861557929056964, 0.19823861455347694, 0.19410358178630843, 0.18545184278311644, 0.19320252296940482, 0.18211797344442826, 0.1887802154075878, 0.188531137515057, 0.18449550417073268, 0.17957709346426667, 0.18710239805927958, 0.1899327341205803, 0.1792748911005364, 0.19812025766024238, 0.18759975377297552, 0.1962356829371027, 0.1963517236950538, 0.179711842493808, 0.1935910359286532, 0.1901721750483983, 0.17861019845686413, 0.181386417661066, 0.18820048012142374, 0.1952110106416915, 0.1809231263552224, 0.19071170244838076, 0.19076550464596312, 0.1855442113520661, 0.18596549044961105, 0.19224264969321306, 0.19512549989453631, 0.17793172008548785, 0.1821269257908347, 0.18142330236747162, 0.18481834645198866, 0.19884336125936428, 0.18568762581929282, 0.18062862003979052, 0.1904855927027614, 0.1917340128905415, 0.1896717589034437, 0.1866852972184883, 0.1854092534247067, 0.1935255670390777, 0.18333343857870524, 0.19024879463641073, 0.1784821159768212, 0.18751966424049712, 0.17486336976340314, 0.18841683883254035, 0.1834256076586989, 0.17903417976855515, 0.18766279201482017, 0.177069884096166, 0.18381870219503293, 0.1809056647109195, 0.192844786931831, 0.20023245289049685, 0.20380982584788604, 0.18820956444959872, 0.18813315195866293, 0.1762521432286207, 0.18498272747936423, 0.19629162878360404, 0.17923900313166355, 0.1850792498202112, 0.18886166463816706, 0.18678132519224103, 0.18073596038836248, 0.1918040876257424, 0.1783921917358604, 0.18066661927090796, 0.19091831394982092, 0.19061209139523105, 0.1850578270065692, 0.19817542077417852, 0.18282108347169226, 0.1830456499561272, 0.19044743257866434, 0.19211676216991452, 0.18673094315955815, 0.18276452819120909, 0.18590538025370745, 0.19196226467310137, 0.189992746595908, 0.17746850858147128, 0.18802760061383245, 0.188609324056646, 0.20599542167577867, 0.18515636751970715, 0.17958875132870586, 0.20689984626183786, 0.19676907130017524, 0.18978233367178834, 0.1946285851246106, 0.1892579151868629, 0.18728845051250376, 0.1961735782823206, 0.17837939266989503, 0.19759846362321176, 0.19524545613303726, 0.1841012215014433, 0.17569934169961765, 0.1795526242843314, 0.19245162245664924, 0.188071439836739, 0.18932603485517113, 0.1986745918806873, 0.20728750934036458, 0.18228023054493572, 0.18892952067959412, 0.18086163467620048, 0.18373187406839522, 0.18937027447740595, 0.19568667159396283, 0.18292249999198415, 0.1876050942858056, 0.19508869710557414, 0.18713347532776226, 0.19056117559786548, 0.19516549058401084, 0.18512313541994227, 0.1884679204020779, 0.18309065319673554, 0.1814548070739201, 0.1922406232862394, 0.18555894333536987, 0.19559148779396024, 0.1797080206019001, 0.18839089877842016, 0.20465948440273837, 0.20241039591892973, 0.2019169417775071, 0.1807894779001378, 0.18608629331777002, 0.1796844605471461, 0.19141032317810291, 0.18705463014194307, 0.1836696471713938, 0.1763350533673909, 0.18110192300916608, 0.1826481552021334, 0.18469468222186272, 0.18046148257217112, 0.1825480483614624, 0.18764049506258595, 0.18168938354389405, 0.18292084231532846, 0.18933704460546677, 0.17932407249875149, 0.1866955751351921, 0.1922771599059257, 0.18874932223697852, 0.2033121426676159, 0.1916530841432287, 0.18256349714489922, 0.19155041958109711, 0.1915774596677173, 0.18372737171758483, 0.1813154166438275, 0.17384158496140079, 0.1847727673794853, 0.18611001051749712, 0.1952058063399221, 0.1803417255271268, 0.19430872664608645, 0.17756900325427585, 0.18361538278946046, 0.18832450049534583, 0.18916347378181805, 0.19397348219150365, 0.18397537504495778, 0.19519663479648647, 0.19741359458964303, 0.19154521327760227, 0.19349426795514754, 0.19711749096029776, 0.18910831390452104, 0.19957337524523194, 0.19490529824766775, 0.181168491202604, 0.18555343772310512, 0.19213000879842998, 0.18494763622190655, 0.19803152541516358, 0.17998119748953104, 0.1797665047153088, 0.19044134087555523, 0.19274580382139775, 0.195496988474116, 0.1862606194100379, 0.1868743492948005, 0.1871175457236535, 0.18203452411443583, 0.19438429793917192, 0.19411954468818135, 0.19677184195733552, 0.18904923482592056, 0.18655288661174904, 0.19037326391893447, 0.18400199752247598, 0.19198983917466655, 0.19709536816335693, 0.1861418876756597, 0.1838011198486399, 0.19590386581921587, 0.20113245337804248, 0.1789137288395906, 0.18069088430882863, 0.19281322313253685, 0.18563896217698045, 0.18293374192852263, 0.1879419540123256, 0.1935643900114433, 0.19382489800162192, 0.19682753910350595, 0.1956857562599794, 0.18743858504432326, 0.1834948541882702, 0.20460021659375982, 0.18402781743058363, 0.19659350614548948, 0.19173917025659004, 0.17852936766424254, 0.20419433277744797, 0.1855019806467318, 0.19689706725342312, 0.19045717811830443, 0.19057934923879447, 0.1902341637016722, 0.1729555073932203, 0.19152680731470134, 0.18348682966540653, 0.1925852760380223, 0.17393868226507975, 0.1815173459174158, 0.1808838731486451, 0.1803464701517759, 0.1898761893356608, 0.18828418091275653, 0.1852913321969902, 0.18644104661528524, 0.19480165679354644, 0.1927556145585368, 0.18626404099564114, 0.18514584529121536, 0.1735435704471341, 0.181385230192796, 0.19142801864088524, 0.1884720046461699, 0.19739551310050688, 0.19027199678537485, 0.1796708723495069, 0.18738737517285886, 0.20330591982725593, 0.19312982188453998, 0.18399333675127516, 0.18257253996366302, 0.1902658597820606, 0.18784219395065885, 0.20403402497057022, 0.17919484759750023, 0.18917541081794392, 0.17689642241340522, 0.19342441388628664, 0.1890654365615767, 0.18748176307194733, 0.19369530041765648, 0.1973240118660804, 0.18815347980046968, 0.19400926462223686, 0.19402458196054825, 0.18469931214953983, 0.19967085410324895, 0.17865659111774407, 0.18165248398256262, 0.18939315249894736, 0.1868590896017148, 0.1949896722594298, 0.19094064161646468, 0.17687967697411225, 0.1928196220124629, 0.19446205960358512, 0.18121029778095965, 0.19118277810038173, 0.19105179401561861, 0.17952501148810607, 0.18963484674107478, 0.1969547436877168, 0.1816981251331706, 0.1810811665301049, 0.19187003260537952, 0.1937167225194481, 0.20078562010230355, 0.1788168762757644, 0.19270874741724944, 0.19294566340066746, 0.18874770363650728, 0.18404695849723576, 0.1926777143580988, 0.19507879869189101, 0.1931839025990938, 0.19303467025530174, 0.1833873659901105, 0.18547811479634807, 0.1882887095603668, 0.19382104204418427, 0.18091180332008633, 0.1863154190259098, 0.17573410602562406, 0.17088141902653833, 0.18287836434392823, 0.18758903198084023, 0.17950309376638993, 0.18143099164306206, 0.19421512503818458, 0.186088718473143, 0.19827404144309624, 0.18033876221561354, 0.18290754332632303, 0.1749264685111865, 0.1861479500947618, 0.19751313362300615, 0.19174925332471981, 0.19473719963142916, 0.18484008259319742, 0.18748410693006795, 0.18921604236876277, 0.18379117204947462, 0.18525448318449486, 0.19746972497014478, 0.18826397889523097, 0.184340749160098, 0.19526064598446366, 0.18098946473269364, 0.19247473666194523, 0.19018692560209846, 0.18228545681091224, 0.1886337563113352, 0.18316258518217532, 0.1764292259746691, 0.18128254127030757, 0.1860539414109105, 0.19136833897558206, 0.17500446065350547, 0.17757621301089127, 0.19261418331093946, 0.17989507375257102, 0.1865641576648516, 0.19633406503261625, 0.18154426938188462, 0.19718203852805685, 0.197114095129728, 0.18500582012009828, 0.18712585031205833, 0.1935092226439041, 0.18727152883185352, 0.20141302771825056, 0.18111624423673017, 0.19307001384730435, 0.19352353667757458, 0.1931182835247676, 0.18779290795434703, 0.18877902269682775, 0.1978230911418208, 0.18822107375177533, 0.18281822629320776, 0.18758196804966312, 0.18924823178029423, 0.18762960545569468, 0.1928401858678747, 0.19533167503793858, 0.18216551455943047, 0.18469683187626063, 0.18464148581320014, 0.18194959695684224, 0.19358214953543768, 0.19863304143053007, 0.17675117543286903, 0.17813470925615896, 0.18660302830468353, 0.1787823800591319, 0.1924264487887758, 0.17501678871968496, 0.19359083573426006, 0.20439488137676165, 0.1817771131191369, 0.1838374387452189, 0.188200754543749, 0.18133322855582815, 0.17310061280726205, 0.19010608400804505, 0.20003585715526867, 0.18123917645453874, 0.17519920512081216, 0.19393327830651497, 0.18799486374710614, 0.2029050493848269, 0.1961472944608825, 0.18826502077006005, 0.18337007086914672, 0.1823264067335168, 0.18015054887852675, 0.18378347166567588, 0.1856100412002374, 0.19599654036590003, 0.19135214528488675, 0.1905644135272953, 0.19233135180925728, 0.19732412912189418, 0.18867110201019896, 0.18753073187840436, 0.18594739954552256, 0.18581873703818969, 0.1845922191697245, 0.18780806233712752, 0.17634185676702643, 0.1942780567228538, 0.17440243101274877, 0.1912976721373225, 0.18274672207996137, 0.1777345382714393, 0.18305156133088227, 0.18473616869401893, 0.18817864236574414, 0.17403315921138546, 0.19747398278137007, 0.19017426006019095, 0.19089681726282753, 0.17981585521482812, 0.18961804641731134, 0.187094692801964, 0.18487444811448453, 0.18388573997633537, 0.19819069658333163, 0.1903819123700938, 0.1897531507396034, 0.19746797843598415, 0.19021430051058597, 0.19246785047604098, 0.18742702357704896, 0.19059663934280402, 0.18281123975101296, 0.17745557180577945, 0.19070146022706413, 0.19592514769511923, 0.18270388882396482, 0.16843980612909165, 0.19115876292365738, 0.18239499163488623, 0.1756030346579234, 0.1830945703253062, 0.1739646773751127, 0.19853532655231854, 0.18414390292782723, 0.19257347913699768, 0.1957250530225097, 0.1942550140280224, 0.19369306785504237, 0.18250667598317788, 0.1830044028969629, 0.17668527699192502, 0.19033381988028192, 0.17972456907082773, 0.1954419377118164, 0.17649140041513386, 0.1918695564760732, 0.18655300938271788, 0.1911823393124383, 0.18913932860858093, 0.18090658011339453, 0.17840616407748872, 0.19326533415896208, 0.18326351492478682, 0.19091956961381235, 0.19332210469167188, 0.19638242054760735, 0.19344883865006948, 0.18084544672533023, 0.19501628123092316, 0.1931176925776957, 0.19360280372115168, 0.18709896335696832, 0.1887820351070307, 0.17607329326213234, 0.18779631404301958, 0.19275875676741702, 0.187694520485158, 0.1853609001748647, 0.19722875172842663, 0.19264491716692142, 0.18691943075586334, 0.199454095743858, 0.18071728283377286, 0.19003909732093793, 0.18981854049232838, 0.1874726910016835, 0.18348657719927125, 0.1917143837694314, 0.1836127563334487, 0.2002971805704983, 0.1804565533036996, 0.19624765692285132, 0.1913057674263282, 0.1795719272059116, 0.1776901780381403, 0.19028588664053175, 0.19086546004455116, 0.18430951951021407, 0.19080255925426598, 0.18789616989914437, 0.188169466868436, 0.18157803354400254, 0.19193167188055202, 0.18449403954540383, 0.187944063721438, 0.20003973387758972, 0.18570624685263318, 0.19109246722942885, 0.18851056687162274, 0.19308343385246848, 0.1912410549061984, 0.17819575065439666, 0.1913435218871018, 0.18411121934410013, 0.17866619760053093, 0.1851397812367079, 0.18123294247983718, 0.1919558801012141, 0.1901245130340236, 0.1861520852162752, 0.1839992723116033, 0.18840459090154216, 0.17141300502547327, 0.19350539702266017, 0.18996576689086533, 0.17812135230965565, 0.18681606080871535, 0.18727859010615125, 0.17880321221007295, 0.1870281615253754, 0.19629293708689305, 0.1910710190982192, 0.18208294803339178, 0.1864695814948601, 0.19510279477742032, 0.18507558999572046, 0.18656285838846265, 0.1801779686517439, 0.18971020204766537, 0.19675184466033813, 0.1897896660153776, 0.17401787927950715, 0.18925737126441988, 0.18417889125313877, 0.16824467524689113, 0.1988673603493078, 0.18805554565272278, 0.1849048843850563, 0.18312027514300622, 0.17568409509691174, 0.18601813420474167, 0.19098676748362417, 0.19004511816586717, 0.1882479285487134, 0.18064259215265538, 0.17857911225477377, 0.18586156907090756, 0.1827177339349546, 0.1816251159008882, 0.18523135309400754, 0.1912241370766653, 0.19628673603037608, 0.18146441251307852, 0.2023732352840684, 0.19186310714571236, 0.18325492247250583, 0.18553702231047886, 0.18535088356753138, 0.18840916444158115, 0.18621541124964988, 0.1883570248268644, 0.18388233369499632, 0.19459659807935867, 0.19033506330448421, 0.18753769610024848, 0.19113117629385995, 0.18417882936249155, 0.19102149121473733, 0.19932836119913597, 0.19196610133680994, 0.18666309861176125, 0.18883398512986307, 0.1819530397125688, 0.18451386195699987, 0.20064611486649514, 0.19423717892115502, 0.19245123353217067, 0.1939187071171996, 0.1823638972976565, 0.1870113706676939, 0.18069774329252025, 0.19465517137731203, 0.19056115680033908, 0.18023426544339882, 0.184829946763868, 0.16810875270481024, 0.1782637072229236, 0.17636196613791652, 0.18306866547448114, 0.18573784091202278, 0.18364196150667808, 0.1903014773554716, 0.1840659380213577, 0.1790151088305801, 0.17960331209922906, 0.19073805069145244, 0.19186338284624044, 0.18106176918874242, 0.19632749632877827, 0.18229318335941902, 0.18956579113214242, 0.1936469827458717, 0.18188866087010613, 0.18200266475131385, 0.18283495518457607, 0.18323801755701416, 0.17890815586981798, 0.19363325630065786, 0.1831692641068762, 0.1939675342943768, 0.19839321663433862, 0.1895080346839223, 0.1919153727872545, 0.18664112646323275, 0.18897651463311996, 0.1877533336359026, 0.18690255467959366, 0.18134472098406526, 0.18047109932899955, 0.1877988370061106, 0.18444647148392898, 0.189133823931853, 0.18578760568465763, 0.1869743715254876, 0.19140880846066224, 0.18793011801087545, 0.1853159561935638, 0.18709384650622568, 0.1875698018251501, 0.1918076511062144, 0.1825712846258756, 0.20119259832055125, 0.1833044888979885, 0.1796030026272262, 0.20289363425944248, 0.17947227103264757, 0.19899759284629168, 0.19214880944337487, 0.18993048437211713, 0.1953319463636259, 0.1854674229996175, 0.18487987917962817, 0.1817945531473425, 0.1819943613790487, 0.19436894800790622, 0.18413572634269743, 0.18572268155810154, 0.18264141338367368, 0.19163361717524494, 0.184674303706695, 0.18969293055494496, 0.18022260743880156, 0.18788027334253907, 0.1916313543569172, 0.1773813449588205, 0.20367960234425428, 0.18158153855060455, 0.1727238116944479, 0.1903787450957203, 0.179942450061601, 0.1940984265389178, 0.1813733235108536, 0.18944602824023718, 0.18008271930813574, 0.17643643281255458, 0.18620806195663056, 0.18844880781711637, 0.18305653967397917, 0.2077209006912881, 0.18376848308502652, 0.18432693019837088, 0.18790735058969146, 0.18936954033511127, 0.1803922553966734, 0.1958211950343354, 0.19969746247082054, 0.18095077844094562, 0.19479743100079383, 0.1856559230299392, 0.17968808380864867, 0.18174064015207478, 0.18824375593393908, 0.20220294181207354, 0.18778028673944372, 0.19448314642928288, 0.1931165667989879, 0.20067200315777933, 0.18495544354838392, 0.1907742449460175, 0.19842029377801151, 0.1849246298435293, 0.19494586979659548, 0.19209000916246216, 0.17846327313615362, 0.18734472774034272, 0.1785071875651924, 0.18060875416584232, 0.18335894749275516, 0.188358811156357, 0.19089225485430042, 0.19322943245845403, 0.19149259662620402, 0.18123461167019103, 0.1890030410808107, 0.17994503880458984, 0.198492339920056, 0.19426545436968756, 0.18845842230351104, 0.18940886050343478, 0.19077123755162245, 0.18696723114788086, 0.1885586073233894, 0.18412427622340863, 0.19054534199525636, 0.19015824145027643, 0.17758663445901873, 0.19966149814661482, 0.1904099528822916, 0.18419953237863013, 0.18983186013542078, 0.20070057021632298, 0.18084589273795854, 0.18296553510981406, 0.18407263629844672, 0.18217543520181614, 0.20395091998530673, 0.1887041534421855, 0.1805489616613154, 0.1929975717286664, 0.19382143068461138, 0.18790593714366846, 0.19510290126446123, 0.18204764056230485, 0.1881724770219686, 0.18839153374681641, 0.1917042894229297, 0.1808905035631451, 0.18368814486099788, 0.19322955106332998, 0.19036497462207094, 0.19598873894648344, 0.18803361656950737, 0.18743425528927105, 0.19662617774201044, 0.19466024286637518, 0.17715274027373007, 0.18300464875884914, 0.196884362928762, 0.19066989323593203, 0.18520054493810473, 0.18540761111130472, 0.19526162507372827, 0.18290789706767818, 0.18744791046461806, 0.18088863052243423, 0.19320850253734112, 0.1875556442501768, 0.18721821000739752, 0.1917213499936172, 0.18413447498002522, 0.181166525084672, 0.19009898004573977, 0.2032986210129949, 0.19290441055394847, 0.19400792968969818, 0.19105656323618175, 0.1837008205540186, 0.18945361939245686, 0.18439474132397563, 0.18395157271215323, 0.18092512550248943, 0.19936159594247788, 0.1777459786627741, 0.18116008510577747, 0.19354917142504563, 0.18219042361783053, 0.19925057815493785, 0.1787373926557998, 0.18330028130758275, 0.19619152423489863, 0.19760090662273955, 0.18989618487961346, 0.18854536365998759, 0.1924829284984018, 0.19732450097971665, 0.1910607131709059, 0.19103009753598738, 0.19390196002038043, 0.19283423949833486, 0.19100086241947578, 0.19517919378536458, 0.19343306508238717, 0.1875334936925201, 0.18297917648729484, 0.19192174204632734, 0.19184691671799572, 0.19446406668725855, 0.19494703325004317, 0.19524263011755474, 0.18100711288666285, 0.19871055185788744, 0.2017324137398372, 0.1909548557661746, 0.18250825622777797, 0.18099575379700614, 0.18801547910501026, 0.18518711389032091, 0.18184037446708987, 0.19015344290469785, 0.1887790472779768, 0.18916273794158442, 0.18768714292247018, 0.18753088304091162, 0.19294571476932296, 0.19245745323905672, 0.18329643648923596, 0.18423621896315942, 0.19125795213636274, 0.1795615738124251, 0.18423200929433053, 0.17555226366856058, 0.1974777796445249, 0.19289628962062255, 0.20178884289979626, 0.18869156480885121, 0.19753806600279716, 0.1864945154140608, 0.17908725368437267, 0.18750070752513828, 0.18917752772373786, 0.18373927107836832, 0.1846215693366113, 0.1967257999876829, 0.175709458037866, 0.1802144289000347, 0.19525553575529767, 0.19836458690941058, 0.18975927000785253, 0.19428842175529465, 0.18520062922616484, 0.17896116564784223, 0.1788530935129569, 0.18938671992535433, 0.1892585378483166, 0.19307124601339565, 0.19144999083440145, 0.18534975090148212, 0.19813425398344098, 0.19737063742687364, 0.1820987763366073, 0.19473800847389605, 0.17911238352730638, 0.18344083707638148, 0.1835021020899289, 0.19295918869841902, 0.19414764564878007, 0.18922772125976056, 0.1830000750514008, 0.17437298075699345, 0.19116483225780356, 0.17913773787262222, 0.18940557545236514, 0.20562066338235377, 0.18247577810284066, 0.17735887896583608, 0.19387051348041973, 0.1965672300072944, 0.19716332529740205, 0.18671521578519062, 0.19635338788549248, 0.18747338804433883, 0.18595369557850494, 0.18363432575045657, 0.18728592298183636, 0.19566884552828928, 0.17421676574440634, 0.17943191761881325, 0.1903288672441136, 0.19479677989770908, 0.19514641058465004, 0.17965920258690066, 0.18537587999645996, 0.19411022717434012, 0.19352566094658283, 0.19170364305323087, 0.1861109175072297, 0.1904277597535361, 0.19040683227835156, 0.19856197813778795, 0.18218708578185958, 0.193877397379399, 0.19146724966786421, 0.17628531020608668, 0.18976205946102276, 0.18644241303601644, 0.19911799825520285, 0.19269170330734645, 0.1825401296254024, 0.17251784727028988, 0.19085060811005794, 0.17893801343216004, 0.19258080328721283, 0.18916076238289048, 0.19387154461149997, 0.18438512745223803, 0.17750394608885287, 0.17976124484931666, 0.18516687826870612, 0.19506777907051467, 0.18177821848164472, 0.1880819602721398, 0.18814274419000007, 0.17467697311762845, 0.18297418053731296, 0.1972834408418385, 0.19078845385534937, 0.1804370201216547, 0.18126409847094963, 0.18030104759360072, 0.19616149953621848, 0.18240600114982305, 0.18572956509248575, 0.17627707065348258, 0.18946044596052072, 0.18825141560149877, 0.196589558666869, 0.1909171164615966, 0.1867359434839687, 0.18567603876188207, 0.1911755296378315, 0.20496098751766362, 0.19127150079777896, 0.1953445624138498, 0.1877789473400829, 0.183754131943563, 0.19019779101733786, 0.18477895022832744, 0.19064422743523826, 0.19870033364360948, 0.18329784213057537, 0.18316649565539841, 0.18819115026909503, 0.18081984470408924, 0.18800256803758328, 0.19057784908688857, 0.18571105134249236, 0.19817604686257398, 0.2002545565287348, 0.18407911069488422, 0.19370688690464374, 0.18961746783548858, 0.17924181128426503, 0.19609816296685884, 0.17754624604835703, 0.1905398041666691, 0.19677166775670518, 0.18201313793264978, 0.18444238308997774, 0.18502502131944176, 0.18316977196712103, 0.17772160338300974, 0.18567030552905156, 0.1809301966907508, 0.20185496751236373, 0.18417957061225676, 0.19233857081392858, 0.18278454087110693, 0.18220982872866806, 0.19290613167788032, 0.1990699941137629, 0.17698513398180904, 0.1789238073361685, 0.1881905882078442, 0.19286692969743707, 0.18529403776430683, 0.18850849692010924, 0.20129663532584474, 0.19124439959760933, 0.18629249888047397, 0.20943950101047476, 0.18953084761296246, 0.17694509640931594, 0.19708791556955893, 0.1793867705809449, 0.1877952820834093, 0.19212917384815412, 0.18487592199229164, 0.19329092319075641, 0.20134203409548587, 0.179351547742517, 0.1911027040974579, 0.18069709899236602, 0.19536040776383415, 0.18948803984316767, 0.20217618935846007, 0.19387623594044434, 0.17972276597217599, 0.18805923589935103, 0.19486417148068264, 0.17684912682559895, 0.187612896444659, 0.19270717119174513, 0.1870348507410131, 0.20479645029822702, 0.18773182284693982, 0.19080197962515932, 0.183011837220794, 0.18766866070680807, 0.19270523969773645, 0.18849448996823437, 0.19834304321069687, 0.18754128412412976, 0.1885953487795353, 0.18270072786896455, 0.19090715645277911, 0.18920241931118933, 0.17632300201743867, 0.1948098015670161, 0.1870164357561403, 0.18380012222794448, 0.19084899282345544, 0.2007628297305943, 0.1770753719863258, 0.19347640046859788, 0.18157412132332118, 0.1907084733505104, 0.180960802043508, 0.18541049737735596, 0.1853385182755181, 0.18947666095422788, 0.19829480519890988, 0.18828376543402361, 0.18111695830339336, 0.18288186687485652, 0.18129232511978902, 0.18306083149764915, 0.1939032617311505, 0.17715425374998175, 0.18608325078773175, 0.18854472894174246, 0.18450852838055143, 0.1963535863729328, 0.18747847352634228, 0.19110004682678702, 0.19175607945901374, 0.18288553589503923, 0.17293893215056694, 0.18397895441197518, 0.1839707335103968, 0.19570048834550177, 0.19143470735391616, 0.17161058039860994, 0.19711486200908024, 0.19341717699124536, 0.1921935206899294, 0.18631834900763428, 0.18919003569992598, 0.17433983135236264, 0.17830178210110867, 0.18831714658377346, 0.18826184634433607, 0.1920522290251972, 0.18803739886167403, 0.17655249434039086, 0.19267589823600204, 0.1956620800386012, 0.1828361619552594, 0.1890695013583641, 0.19184924065180275, 0.1905867831511211, 0.18872636196673678, 0.18844940175633265, 0.18055319995848637, 0.1915492704426538, 0.1829685991855387, 0.18786423220330464, 0.18592831613473004, 0.20524258751748234, 0.18389356125738343, 0.1946372345762762, 0.18583548217358548, 0.18655079215177905, 0.19673942435850864, 0.19912873042934406, 0.1849962366248781, 0.18822833065947417, 0.18602460567922555, 0.1845898645739412, 0.18341099040269898, 0.18437964655666206, 0.19042078304621488, 0.17715407867650942, 0.18634078981592028, 0.19464624505177347, 0.1880622655808919, 0.18568579744841332, 0.18502976090972467, 0.1798868942570215, 0.19104015150333262, 0.18634310498531056, 0.1754265271369206, 0.1920595546659163, 0.20584919522196551, 0.1833599735678488, 0.16975418702064776, 0.1971996602213112, 0.19017617304349319, 0.18752959944416256, 0.17939239823934525, 0.19745510728880922, 0.18543474314170322, 0.19152102578301886, 0.18171673510663688, 0.18260872613008425, 0.19453559550557198, 0.1979759792075748, 0.18404683838575267, 0.18151979990072967, 0.17857511414745275, 0.19158641337662613, 0.19577446813461738, 0.19151527810571164, 0.1864420979619501, 0.18266766303395088, 0.19639944251887564, 0.18919555421032744, 0.18790853338820396, 0.19466616350429422, 0.18746925155312444, 0.18382034476124468, 0.1757925066492677, 0.19278590539867277, 0.1918673834189575, 0.19074308655955363, 0.18060732408523694, 0.1900042921080684, 0.18747265376379083, 0.1893445312697536, 0.19326316450820497, 0.18453640757548812, 0.19190874216068332, 0.18831154098793138, 0.18705853438240036, 0.19381975031937748, 0.18764561355288192, 0.17752489636058855, 0.1819957648669945, 0.19018783860369237, 0.18465977721593213, 0.1975825930399344, 0.18785579687330375, 0.19136806380259663, 0.18476796639485013, 0.1866093118059023, 0.1914151660696139, 0.18943345507105383, 0.1925222430730344, 0.17947634155212122, 0.19223641377380665, 0.1680230997042224, 0.1920220841143254, 0.18735108753573085, 0.18753963908451596, 0.19654149557043504, 0.17107694832296796, 0.18750170123590287, 0.1839435760651191, 0.18002916484491663, 0.18651778911030314, 0.18167206213554468, 0.16685266871322119, 0.18564162920973112, 0.1837922342974214, 0.1731833915776941, 0.17665149298425992, 0.19591894406651925, 0.18866452007627665, 0.18939215812093893, 0.1938859322918275, 0.18305322986423866, 0.18497359603250974, 0.18343780893196207, 0.1854857487881013, 0.1878763564931971, 0.1828669478700603, 0.18723838815593094, 0.19210696344967454, 0.19308771159560284, 0.18807972354488986, 0.19350734905466282, 0.20267944581520622, 0.19144511440313697, 0.1951948734579029, 0.17974563125450538, 0.19724063649095228, 0.19665001631535745, 0.19148383040814768, 0.1998107230115999, 0.18702711781003004, 0.19012459562116066, 0.18903033823556997, 0.19122921714357258, 0.18684806000239038, 0.18501571063175337, 0.18580705383304844, 0.18813958532740113, 0.18796011905398186, 0.19381671455430885, 0.19460076693545547, 0.18824389973433614, 0.1906608086904179, 0.1903160413189602, 0.18830503115768438, 0.19320554839912626, 0.18633304548613164, 0.17855923883206967, 0.19257843066319882, 0.19099809288576805, 0.19460582631292592, 0.18028267453824487, 0.194664593467971, 0.20358926456241347, 0.18092881470559724, 0.19679676997499673, 0.17558569248837047, 0.19240031953618045, 0.1872399570017812, 0.19278373540883323, 0.19301452424529364, 0.17656544537655383, 0.17766376409167714, 0.2058964944103425, 0.18407846261553626, 0.19177465875217956, 0.18633146986079313, 0.18862080623024402, 0.19136872124061477, 0.19019685222169508, 0.18115311481577587, 0.18979580579009947, 0.19675293252940884, 0.19685590293117225, 0.18707160278557175, 0.17793213901623398, 0.1822209877720796, 0.18228820629288423, 0.19978384561180623, 0.18847664634231776, 0.19581276306152143, 0.16989448202955182, 0.17787229731727572, 0.1858229333493684, 0.18495034836344373, 0.18942523872532901, 0.19352102057390616, 0.18501336366366566, 0.18982757241365805, 0.20433991491094736, 0.18990729970676315, 0.1990613006634759, 0.18839620142000202, 0.18290432898297262, 0.19653069411606422, 0.19114610492385178, 0.18973537466394794, 0.18601426910014934, 0.19835080173718073, 0.19054256423697644, 0.18248069280717358, 0.17869047843283548, 0.18416334000495227, 0.18272175656978623, 0.18652748988776083, 0.19745754099328125, 0.19097748747889823, 0.1799055146336219, 0.1955543552182135, 0.19832039764199041, 0.17228962491539304, 0.18471138579608393, 0.19243402637085247, 0.18740796975572244, 0.1909370157904783, 0.18208197769233334, 0.1860337899690196, 0.19163800730750674, 0.19729318195255907, 0.18841586364689047, 0.18943130313501472, 0.17830579754542275, 0.19720511604492758, 0.1904994420112941, 0.1959458829672228, 0.1908069325319369, 0.1923504719971752, 0.18078501695810756, 0.18256828846697343, 0.1910242288044881, 0.19717200844885494, 0.18968189260169682, 0.18615316679981175, 0.18909933376421922, 0.17091573612894484, 0.18984986376236698, 0.17885215499285598, 0.19040831529375313, 0.20487403124026368, 0.18990288573553085, 0.18745174538321085, 0.19843553732691677, 0.18366942441291048, 0.19394807851391713, 0.18517522071828973, 0.196200213654407, 0.18261249502689939, 0.19150103243365887, 0.18747627859884644, 0.17968371929821575, 0.18575694729996275, 0.20108449478999663, 0.17518842819337455, 0.17320691460454227, 0.1737805128487079, 0.18553230617675281, 0.19699524196305726, 0.18402506716466754, 0.1890274484732923, 0.19911699550711043, 0.19976239890656677, 0.20026314930818856, 0.19255161039191604, 0.18765962376086662, 0.19603927543928965, 0.19647873250749334, 0.1828472597729334, 0.1789084246735773, 0.1933005516422769, 0.1800975829801172, 0.1933449261412544, 0.19356505143667707, 0.18864563430822592, 0.1810461517382705, 0.20052990881015834, 0.18494455019000985, 0.19620102121439717, 0.1885602866662067, 0.17741526309077, 0.19124411241534311, 0.20706931218858277, 0.18213625479232243, 0.18774705516590182, 0.17975342064466315, 0.19355936197016774, 0.1877678285608748, 0.1858760606163482, 0.18378620271125476, 0.18877017953186415, 0.20608412199007164, 0.1976869928562959, 0.18086856436251877, 0.18734720242211717, 0.18321999862191793, 0.17207923613358136, 0.172850336696697, 0.18236345951826793, 0.18246079765771978, 0.18080783932240665, 0.1898122635373879, 0.18890287470261394, 0.1983924308008064, 0.17972634900114665, 0.20118357689163957, 0.18166930078964738, 0.19929953182213878, 0.1874523652648333, 0.1840008473859789, 0.1905147616435916, 0.17847487210830815, 0.19286282698102666, 0.17544158745343705, 0.18053389156970764, 0.1790161287430022, 0.1979196396848372, 0.1827958833227631, 0.19526887513372645, 0.1887089830182393, 0.18870680681630203, 0.1860888705332747, 0.19601615746516432, 0.18699775235139215, 0.18045140562609713, 0.17954544122436086, 0.18909565500805534, 0.18848040329634475, 0.19730251999089335, 0.19129706025691603, 0.1849422184093188, 0.1919954934010991, 0.19365735426481276, 0.19451176875336307, 0.1908775848454232, 0.19077691711994732, 0.17750434324308775, 0.18963470897221396, 0.1842102748827446, 0.1872026139805647, 0.18876033093129146, 0.18420188040831917, 0.18974089971527314, 0.1926275658243981, 0.19314707463344752, 0.19102907287625673, 0.18419222371321306, 0.18614620714390112, 0.1862271762730068, 0.1752818126114254, 0.1959300091408105, 0.18972333473189942, 0.19283488601303828, 0.18470863834827128, 0.1911105223168216, 0.18417873338743493, 0.18705559780901848, 0.1802688520476405, 0.18381143752657456, 0.1962170286179115, 0.19319649720310322, 0.18778115383708696, 0.1954502630128506, 0.18507152442201089, 0.19966684215103975, 0.18944963962639688, 0.19148795487965883, 0.19181233666061878, 0.19115284250378298, 0.1931967913692989, 0.18178726747464635, 0.18978269137002074, 0.19153387717346984, 0.1875110912164726, 0.18833995269283055, 0.18980513091195972, 0.19458923758887905, 0.20042380583517957, 0.18994367791211186, 0.19206434418660875, 0.19674532873231068, 0.1894065822615944, 0.1829480400451802, 0.20348245255762998, 0.18748968129840765, 0.18952420867397038, 0.18825671198349772, 0.19301241968435262, 0.18122029476290916, 0.18596731685292875, 0.18233710809859635, 0.18627184997405594, 0.19012675369414125, 0.20399839756643293, 0.17853082401036038, 0.19720480037052338, 0.1847007688324957, 0.20887649726693153, 0.17782690186883268, 0.20324338974902167, 0.19984799083426885, 0.18224555065465387, 0.19056025590672981, 0.17397134171265669, 0.19351562021531724, 0.18088401633191134, 0.19071161923124702, 0.1790474593347804, 0.19486132107131687, 0.19857375547335776, 0.1823564347928444, 0.19077416240232903, 0.1941032388505976, 0.1874557176036373, 0.203357719910531, 0.19431743778777227, 0.183875230426202, 0.18016849804885118, 0.18651961924080143, 0.19273866524302796, 0.17318129745198424, 0.19461539998749863, 0.17026977014218533, 0.17280328394424413, 0.19009840819792587, 0.1857565750902353, 0.19001459605201076, 0.18808010099260108, 0.19210208269940932, 0.18486593356691022, 0.1907841408940168, 0.19246831349209917, 0.18902906679072784, 0.19125726318816533, 0.18805790465139785, 0.18955920720572147, 0.18767347608977053, 0.1881518164903897, 0.17556652306831025, 0.1898551007370301, 0.17552817378792632, 0.19185579642755307, 0.1722967150411961, 0.19230870533001854, 0.18851764897785167, 0.18516345023576514, 0.1973489125509778, 0.19024146717214807, 0.18831448311851967, 0.1922726354786864, 0.17726125734335385, 0.18552953348454376, 0.19431480950734656, 0.18559791634589615, 0.19576076218893854, 0.17918447485446268, 0.18528258438692152, 0.1868797492864316, 0.19716307580243325, 0.1885592468076261, 0.19692957852913315, 0.19743752496307385, 0.18115552539120025, 0.18639437782468235, 0.18911185362987187, 0.18983342642018156, 0.19181807240857154, 0.19373267389633356, 0.19653352172679678, 0.19426167147553475, 0.1890185519237318, 0.19736279892442932, 0.178926895725632, 0.1862686528899714, 0.18896850956982172, 0.18947762057259884, 0.189710009320052, 0.19775673980914324, 0.18526496894598796, 0.17744163765651008, 0.18376755617023766, 0.18639880794967806, 0.1987395750385087, 0.1896258784689731, 0.19010690902645477, 0.18236953565680009, 0.17872088535919808, 0.19050390247048643, 0.1938982911373877, 0.19617110705077945, 0.18994324079243988, 0.1888710011925179, 0.20050686898481246, 0.18513578257414023, 0.1984478898906807, 0.19631390052553443, 0.19353344160768385, 0.1820884726239685, 0.1943013852257728, 0.19573436249125728, 0.18386282121924627, 0.1887298598957422, 0.19260552457651037, 0.182547951778943, 0.20557568276591043, 0.19365512574867402, 0.18568672814936057, 0.1857215654097759, 0.16809857803254877, 0.17567917248783435, 0.18631400212535057, 0.18372665445255207, 0.17685673606367003, 0.1898687387592026, 0.1982031613167966, 0.18850949115689397, 0.1734977111907886, 0.18864796778682558, 0.17861003597140182, 0.19033425875705992, 0.18861375448004192, 0.1901106227853322, 0.1778048482239784, 0.18714075902114716, 0.18376514254348436, 0.17770929630654184, 0.17993977797399394, 0.18236771107380007, 0.19083323767149157, 0.18828581831636512, 0.1851923986258357, 0.1898961709079073, 0.18833359474153188, 0.2026980619078937, 0.19149434607621243, 0.1962908571446346, 0.18299866515541, 0.18464751763874254, 0.1868548934878747, 0.17609012579209699, 0.18986843940399256, 0.18346617777035792, 0.1937396309298693, 0.18225172002379345, 0.1905775232981945, 0.1912702061409295, 0.1904071956687744, 0.19286725867433835, 0.183399328775691, 0.1834084397128856, 0.1918104337070685, 0.1841405954298734, 0.1890109872225505, 0.18831938107814564, 0.18791362537381356, 0.1987178236003437, 0.1810050397883872, 0.19336890855742508, 0.19203181178622739, 0.19673567863741573, 0.1925873966134063, 0.19403106891897842, 0.1982710910136793, 0.17969649410400285, 0.18934350194766175, 0.18915106748128502, 0.18274381732054507, 0.19148994394071142, 0.1742600162741191, 0.2027208506401493, 0.17628501395874036, 0.190660733496953, 0.19073385150892572, 0.18465372193964225, 0.19206196812513301, 0.1760498465830454, 0.18285244612360063, 0.18890465866883419, 0.18740708335203776, 0.1798473395047388, 0.19238294326787467, 0.17787741901515675, 0.18964059013665144, 0.1920735346842585, 0.1956331794151676, 0.1834386426720401, 0.1832679614790875, 0.19739243820750105, 0.1875155414250841, 0.17730747669076788, 0.18312946769139177, 0.17852948088385773, 0.19786432869206078, 0.17742077374864262, 0.19345343225945522, 0.18533948198554773, 0.18976344029569542, 0.18956477354649917, 0.17484311640556574, 0.1736677475009403, 0.19140048042894908, 0.19091450264829138, 0.1839545323913248, 0.199793146864422, 0.18114955679709835, 0.18604620570358474, 0.1796710393815593, 0.19294580376727133, 0.17936863914669698, 0.18419203893662892, 0.18552703043011856, 0.18914786635681535, 0.18161431444692797, 0.17762363150434066, 0.17936414398488476, 0.18918073935295118, 0.18791826357346536, 0.18721921173831474, 0.18310657102703146, 0.19124759207089437, 0.18521268906683946, 0.19360967588803824, 0.19269836355578365, 0.1881116565212757, 0.18492057455850738, 0.1887751439639757, 0.19080418515652325, 0.18958315495757153, 0.1873471636660143, 0.19175492230795416, 0.18125891816537504, 0.20158085801455083, 0.1838484609604371, 0.1860168404007004, 0.18536812940366962, 0.186070403458463, 0.19629597762103712, 0.1835013505841337, 0.1939454264785559, 0.18536798900069423, 0.19479909828546704, 0.187011823499571, 0.19878236601921542, 0.1745124932450654, 0.19000646034164356, 0.19308547260256517, 0.1782283198827552, 0.18199905465263025, 0.178072773817967, 0.19310557897176114, 0.19134948631129559, 0.18664280328920185, 0.1883201271103853, 0.19278518351821392, 0.18163506622170367, 0.19334210891563267, 0.18956190494704933, 0.19396349180073547, 0.18247447932736435, 0.17844132632617438, 0.18268975669257073, 0.19064069085779273, 0.17817183498948763, 0.18974661114018262, 0.19546288563261013, 0.186162566155724, 0.19333748980239882, 0.18440761709413456, 0.19128193139716584, 0.1844455135251049, 0.1890173846026296, 0.1875883160327229, 0.18704093509467273, 0.17468733024660352, 0.18277489475376207, 0.18118142793081543, 0.17270818078299396, 0.18810446598413974, 0.18422337018899726, 0.19188480484254894, 0.18454597783621896, 0.18425076010535993, 0.1976660024776938, 0.19538511727030536, 0.1964616588585089, 0.18115986412056678, 0.1851393987157316, 0.18575883884416935, 0.18236579944298031, 0.19087706512938013, 0.19900268642423274, 0.17978249426638102, 0.18256169140172349, 0.17760376193888883, 0.17178061666502642, 0.17691995174500058, 0.19180380630294172, 0.17565023085192116, 0.19274441920182256, 0.18971661352955474, 0.1894513757653829, 0.1797173229851549, 0.1839933864379713, 0.18778812888837293, 0.18899652674638473, 0.1873470221417732, 0.1854639975165398, 0.18112978488144174, 0.1885891080176207, 0.19830953014020405, 0.1980899958632328, 0.19376150907485573, 0.1856382753973946, 0.191424351129488, 0.1873367338041457, 0.18371737493120455, 0.19118772353851576, 0.1889646535488692, 0.19360104012444088, 0.20090820759518904, 0.18624344449649513, 0.1949422923122962, 0.18098022929663135, 0.17917176091273812, 0.1971762365167195, 0.1807081694590556, 0.1922202263172704, 0.2029176345485068, 0.18579164035226053, 0.19380148733742006, 0.19086909214761172, 0.18899037407371552, 0.19182358643520492, 0.1920841961218533, 0.1789880497363856, 0.18483456859489708, 0.19821994022697045, 0.19448185958309036, 0.18926947381047102, 0.17841370774472567, 0.19366149763907026, 0.1894431532771819, 0.1994516527571661, 0.191325283185321, 0.18781964188344494, 0.18120787844536698, 0.18151674096283318, 0.18807780522996215, 0.1837118936133934, 0.1786299299805941, 0.18284024263593654, 0.18377111855170322, 0.1903181426199228, 0.20367725209634366, 0.18379317107295404, 0.19837090679523492, 0.18748211664846925, 0.19032592459759226, 0.1848036036839795, 0.1787188439505213, 0.19049607772519764, 0.18658751269794568, 0.19215095375208577, 0.18270328568172725, 0.1874634921024908, 0.18847803721983855, 0.18500565235852542, 0.19064220257900832, 0.18672006306629552, 0.17504698496919244, 0.2016184703687511, 0.19007652529675947, 0.18798714526452315, 0.18509891898961253, 0.18035717122135161, 0.18468890204088786, 0.19286445426016233, 0.18858774175122192, 0.19340855603368826, 0.19008358064763348, 0.18161714005521576, 0.19052738823131382, 0.19217381143026027, 0.1868713311531347, 0.18959852284979178, 0.18341637837487892, 0.18564523308643144, 0.19058602638442093, 0.1877503691973599, 0.19210670971195, 0.19170308558700724, 0.18690602823345698, 0.18138164057788744, 0.19021187304528608, 0.19222642922014252, 0.18472611363081376, 0.1853925106327852, 0.18870289804492668, 0.19169282256860667, 0.1798188071579607, 0.1877455274430913, 0.1954657803717294, 0.18893444171779816, 0.1963141350451525, 0.18006032563478563, 0.19477342518191462, 0.18523629303191627, 0.1933943275093406, 0.18367798193525334, 0.1760610728909948, 0.18600085054402452, 0.18361059045393047, 0.19168884203782313, 0.17891410533907864, 0.17956102016196965, 0.18202619177291543, 0.1839862259841836, 0.1908331121637395, 0.18512202574758682, 0.17990738005242543, 0.18341154302996493, 0.18812756144770634, 0.1849990774266113, 0.19271845992689665, 0.19451889086013924, 0.1989372469004963, 0.18528230899371162, 0.20059706527975968, 0.17827754006832489, 0.18783291385321518, 0.18588639934023493, 0.1908292179205108, 0.18016393810566386, 0.19608576399901428, 0.18577576507824575, 0.188320100294904, 0.18966617579532208, 0.17732380161510894, 0.20169215569136933, 0.19541292150362896, 0.1817643865658085, 0.19161828131489891, 0.18897968488042896, 0.19556480549901487, 0.18691907383120138, 0.17602623719369856, 0.1958012388943677, 0.2015144395398077, 0.19998737014333146, 0.18214033468022667, 0.1860896655906561, 0.18159006166023625, 0.19232204275173698, 0.18555981766895338, 0.20027909588928464, 0.18425253003185224, 0.19623404478624187, 0.18170698890082354, 0.19034127625408687, 0.19326802488093967, 0.19344783333173746, 0.19081667555309434, 0.20274311113337493, 0.19596250237417803, 0.20263674492961006, 0.19198518476523468, 0.19282597322725137, 0.1873002965672207, 0.1903011638387781, 0.1987411953485981, 0.19911953118066247, 0.1905212698529661, 0.18636191220813408, 0.18870853277720032, 0.18255191392556191, 0.1886612108948326, 0.19888585521184546, 0.19242680053003638, 0.18224574849788727, 0.19031865126885994, 0.20310401600544992, 0.17820812464226832, 0.18775628241526182, 0.19488725217925193, 0.1965721068480666, 0.18389774654043742, 0.1861857001467049, 0.19175712665316133, 0.18523263444185534, 0.19210338739904598, 0.18447938849280693, 0.1802294262910827, 0.18684527803890635, 0.18435556024664662, 0.18099214309155015, 0.18115409050844902, 0.19083267414464436, 0.19120638171885024, 0.1746082031673974, 0.1865600813132825, 0.19433355750894715, 0.19685548397721556, 0.18305202901591477, 0.19204037194281975, 0.19188019243015952, 0.19217800038498967, 0.18639499605835969, 0.17890536660203343, 0.1776491465041592, 0.18469595664872393, 0.18043940822660262, 0.19044359903021088, 0.1889829461588238, 0.18101383518861758, 0.19273279666515247, 0.1909830239288959, 0.20248118634241608, 0.18476660236612358, 0.17944947417121887, 0.1832129291624462, 0.2027883098827569, 0.18447035146414686, 0.1837417803369975, 0.18417281190713936, 0.18614956363442073, 0.18349382789642307, 0.19693612433966856, 0.1864474381744377, 0.19165590334705623, 0.1927769325337951, 0.18595455649393364, 0.1888314416629508, 0.1956659293281558, 0.20118875031810865, 0.194961079920267, 0.18498773723546463, 0.19534488603271494, 0.18628849541135784, 0.18466637376789577, 0.1867571359851582, 0.19170024772223504, 0.18147400501249486, 0.1945426585332109, 0.1984113427578448, 0.20541031719400688, 0.1931832194459585, 0.19404695473710204, 0.18673160087883978, 0.19069279696439592, 0.19029811729896504, 0.1886109945320505, 0.18274798543158616, 0.1876736062308651, 0.19613778878800314, 0.19072853049201646, 0.18417315676949128, 0.18664346678754004, 0.1821873779325219, 0.18467707555797658, 0.1792609549204997, 0.1745222489679626, 0.1940564386990832, 0.19388944492803045, 0.19772119613053793, 0.18947524137844243, 0.19153890542586086, 0.19156744142842094, 0.19582195498767974, 0.1728039141420108, 0.19152228929064374, 0.19578765394437841, 0.1873711072675275, 0.17968646030226246, 0.18836056354454198, 0.19278583179545453, 0.2016675293335443, 0.192211769592713, 0.18842244996739418, 0.18600175982255546, 0.18410048711914895, 0.19233073087665756, 0.17938593333075292, 0.17921637659952333, 0.1834374799354793, 0.19299854220939502, 0.19483847580623082, 0.1904868100410508, 0.19815608061875364, 0.19595804596986244, 0.18446414495760358, 0.17589168623141996, 0.19639333977231044, 0.19283682743871178, 0.1925071967409224, 0.18368531702192056, 0.1755910924507505, 0.19092494864980622, 0.1862027938039677, 0.17973097402557908, 0.18130398199964973, 0.19704027297136106, 0.1876307464729191, 0.18002786346301222, 0.19694573606059143, 0.17713088612207045, 0.19127433966362992, 0.18807767928093763, 0.18466137797199514, 0.18666533811961739, 0.1939090665493969, 0.19210003002071097, 0.1926424523598776, 0.18269647032217046, 0.1861849619696949, 0.18335203446100226, 0.19095242831517267, 0.18062139907028493, 0.19544534239457892, 0.18744857964843606, 0.18745402546158138, 0.19014515010733873, 0.184820934543242, 0.17392123631366743, 0.18793691937502685, 0.19819135771593263, 0.1937341437072164, 0.19321717417087075, 0.2024886038578955, 0.1981186090131753, 0.18470797082864093, 0.17540905601669224, 0.17896876625503405, 0.19134304513164088, 0.19095644748845164, 0.18175467668536666, 0.1893797058190501, 0.19293391006612193, 0.1922527585221268, 0.18419329618293356, 0.1895540425685772, 0.1986941384015385, 0.19018159091989403, 0.19399695998839667, 0.17873131881234502, 0.18170559780671958, 0.18586277367079604, 0.18000312855755923, 0.19477607706878225, 0.19568844685056333, 0.1939430704650864, 0.19156128718778628, 0.1830663753311572, 0.20499030786491282, 0.1940675476572441, 0.18710631554215013, 0.193812629957015, 0.19719866034295463, 0.1913754745734618, 0.18795343810982015, 0.17681067550522672, 0.18245594701723727, 0.1865939531283481, 0.1832384531487682, 0.1825042582038919, 0.18510670505506, 0.18756045004577093, 0.1886555261878585, 0.18812892761741684, 0.18820410549597166, 0.18828681535698605, 0.18734904296925434, 0.1913458805940602, 0.19455992969950078, 0.1971757259893665, 0.19131453991481398, 0.1840541750628325, 0.189219834020185, 0.19002431766810746, 0.1765131774129119, 0.17638485418636082, 0.18648183408754362, 0.17700294033472916, 0.19393365837899443, 0.18163355665234446, 0.18744835935346119, 0.18684369174446958, 0.19059674730467976, 0.18255482816123444, 0.19769446278572084, 0.18544205218917673, 0.1977786176518348, 0.1839052855559853, 0.18972932085539254, 0.19118782083494953, 0.18361710448012736, 0.1948937803401849, 0.1840671164464047, 0.17480915020931326, 0.1861895777563453, 0.18696983125355873, 0.18376450173724365, 0.18716199342874718, 0.1844540155800316, 0.19596130341391058, 0.18561894865860146, 0.17913359233159717, 0.18570024991716344, 0.19910884279291863, 0.18415984440684993, 0.19598530642816428, 0.19225288043219038, 0.1843274528883236, 0.19919049847387973, 0.19034520483198725, 0.1810719073869963, 0.19319402158254048, 0.16947025618940662, 0.17994526012142992, 0.18685612015955808, 0.19087091941698742, 0.18670263628020928, 0.18561464944880005, 0.18934024803140106, 0.18647198110315824, 0.18468301118557556, 0.19655769632729855, 0.18679578612540373, 0.18414595764465172, 0.1728087804055482, 0.18069439944139, 0.18514667220199027, 0.18450663503433085, 0.20056137285576306, 0.18473880701809228, 0.19298333976987905, 0.18769516692801444, 0.1907052317432533, 0.19247439297923705, 0.18412646427872556, 0.18618604625437465, 0.19047437769302833, 0.20161367623749385, 0.18410074120924136, 0.19154252530704538, 0.1882660484968068, 0.19858981087196637, 0.17860278974071853, 0.17763259633201828, 0.18850689270862783, 0.185546561710784, 0.1846969260655494, 0.1826446565561256, 0.18464513871146762, 0.1990093337327832, 0.1909716657671317, 0.19070836982762715, 0.19514358750348826, 0.18659256148960784, 0.1837059507811849, 0.19188002581233007, 0.1802711972047824, 0.18900903412687692, 0.18062193482401975, 0.17726647183188762, 0.17572470011461158, 0.18551388315319947, 0.18789925919263972, 0.18701608240910933, 0.18903616947557894, 0.20349151773654317, 0.19395862727323607, 0.18827290268203678, 0.18789843724731523, 0.1940342277949026, 0.18807225612133524, 0.194098935957179, 0.18907170981690724, 0.1965991882952069, 0.1861301660012969, 0.19173886387578845, 0.1898116607561184, 0.18324061264208788, 0.18437216690294905, 0.17942253729439286, 0.20287042454875395, 0.1813265226780775, 0.18948061372821087, 0.17808443453139985, 0.18434913450433124, 0.1806454648948221, 0.18724789717591253, 0.1830090274429224, 0.19473034229095174, 0.1705204841185279, 0.20281947764336958, 0.18829384721446069, 0.197675119802914, 0.17952440909392076, 0.1962902286246238, 0.19073609774608563, 0.18948172290787604, 0.18506310329948447, 0.1851474499726379, 0.19694057030190756, 0.18903957518343828, 0.18320772131416357, 0.18652945656907652, 0.19151255594373462, 0.1865988287713702, 0.19434260217600247, 0.18420964642387228, 0.18926469663904205, 0.18739983289220116, 0.18469136712031403, 0.1853501362121719, 0.18068411880633234, 0.1901820754432363, 0.1983418951760581, 0.18953760891470933, 0.2030211329622117, 0.1918613406101465, 0.19089566820525847, 0.19068323172825732, 0.19013749309555658, 0.19693892206858332, 0.1856334722246744, 0.20536759528416307, 0.1798397935435381, 0.1700050635638156, 0.18159889674385837, 0.1916177077470229, 0.192913015496151, 0.17638463673439697, 0.19285124834501113, 0.18937508477432563, 0.19937614308338125, 0.17673050864376558, 0.1876157350554689, 0.19204595293888102, 0.18724508306188464, 0.18808580253478188, 0.18829260641622048, 0.190693588290708, 0.1926895914590214, 0.18879960163618742, 0.18199373708580208, 0.17472933210191077, 0.1783714502565591, 0.17603541090300304, 0.18392984675170507, 0.19077642745677276, 0.18117344593533036, 0.19457562186517863, 0.18580666453063796, 0.18856530173152458, 0.19748890512031717, 0.1851519398235823, 0.18550365412295594, 0.18369203582925753, 0.18027518908773926, 0.19295238107753152, 0.18533059549027547, 0.1877077278314689, 0.1725138674848894, 0.17999311984587296, 0.17897397121834016, 0.18604069899543207, 0.17816635183579876, 0.18394984858202484, 0.18782583715221207, 0.1846517909081109, 0.17853221560223195, 0.18111454477005123, 0.1921884453218447, 0.1854707951951373, 0.19594072724233066, 0.18830855575285796, 0.18681306299608946, 0.18736560548360773, 0.19683506670045847, 0.18322073526248323, 0.19842753796821772, 0.18951975604477977, 0.18715612049189634, 0.17535642710944113, 0.18302163617182576, 0.1852311831649166, 0.19163254901830806, 0.19015491809838686, 0.18682993433171022, 0.1860129634146193, 0.17890463862298378, 0.19661844042503202, 0.1959036891999365, 0.19903255683945764, 0.18172409726521654, 0.1873273940370575, 0.1917110655570826, 0.19600009039985947, 0.18627785277693318, 0.19313302856006598, 0.1842414182782928, 0.18285726873677358, 0.18508415939505854, 0.20347184039972963, 0.1844948848501053, 0.19950372012370904, 0.1966135794189917, 0.18426569757413838, 0.19129710326754473, 0.18869462889435093, 0.19366812210910042, 0.17655537704035124, 0.18006722313074752, 0.19056080022477925, 0.1809138682088782, 0.20130164573261639, 0.1862021775437913, 0.18220710950714206, 0.1996667657399694, 0.188320144757237, 0.1984841659173693, 0.18691101822520592, 0.1896617730185552, 0.19514128544944934, 0.1889862536130864, 0.18896443705371652, 0.19307257316147186, 0.17894599038355397, 0.19211772350846126, 0.18521980375783556, 0.17763616893937403, 0.18827465540575558, 0.18675534427998294, 0.19408531148219316, 0.17723521273882054, 0.17936851862951264, 0.18359316058196395, 0.1939472949968606, 0.18593556593702834, 0.19883813535192385, 0.18991517316313528, 0.18384645689668686, 0.19407482486513486, 0.18836951660430923, 0.1871411751120334, 0.1863259186559378, 0.18223194415796404, 0.19819220592535558, 0.18913157581943787, 0.1825964380959399, 0.19200563655611977, 0.2025440923651637, 0.17995869383739163, 0.18538084876253474, 0.19898778513870924, 0.18477726256423854, 0.1815471593487123, 0.18126928053509644, 0.1941461780930825, 0.18993287559314714, 0.19099785663093163, 0.17627981150606384, 0.19420118461471206, 0.1730784462936109, 0.1916110348125684, 0.18946011464943674, 0.18635914550510996, 0.18961494838003246, 0.18186579374580789, 0.1805387676004502, 0.1840728554565428, 0.19557940598547724, 0.17970015038148335, 0.18490307582842347, 0.1793095696239282, 0.18359443335602582, 0.1932253281417998, 0.18514340948602165, 0.19384464370315818, 0.19559246427008367, 0.20265916320465863, 0.19219790576299148, 0.18656687230354696, 0.18241307676414414, 0.18659380780631094, 0.18271325236890815, 0.1930672953819856, 0.1853800279201331, 0.18441740381124383, 0.18708234429749537, 0.18701947487301337, 0.19425151473476424, 0.18852953542143638, 0.19145045536713967, 0.1866584299610208, 0.18340542340618377, 0.1895722390879049, 0.1875727149212066, 0.19099160654123978, 0.18654002735328135, 0.18142823897333543, 0.17690458013062374, 0.17793824751268225, 0.18208908568405996, 0.1799428175858224, 0.19016080601600555, 0.17944009186284499, 0.18224516846083938, 0.18373323910359796, 0.1921224316666709, 0.18411671176852837, 0.18448503955016804, 0.1911534764023063, 0.18859131736658274, 0.19605727171441287, 0.19153517853396979, 0.20364143373171856, 0.19096085006006164, 0.19044929725599283, 0.18241935062034587, 0.18076574130715284, 0.1751879635538869, 0.19186381558423968, 0.18600617733586794, 0.19172271589306408, 0.1868982308595609, 0.19010148034492155, 0.19101288025739552, 0.17740791569970568, 0.18137861280315598, 0.1930287181635264, 0.18649318898012415, 0.1882813564480276, 0.1872117456620603, 0.18770618607020106, 0.19480015010788634, 0.19416751227667137, 0.1914347579770278, 0.17811909226251843, 0.18703907150163812, 0.19235659071394345, 0.1951865817734466, 0.18294029938936732, 0.1701812897769534, 0.18772881468970895, 0.19003205667002906, 0.18023096493202084, 0.19547774660075218, 0.18861768543171084, 0.18600403562142032, 0.19689507719643673, 0.19654296779965844, 0.18662203068491176, 0.18864087332587212, 0.1987995537800041, 0.19156326945824773, 0.19406530740574748, 0.19182564584886572, 0.1890173450262252, 0.1957475721927765, 0.19263094274556405, 0.18910330325937494, 0.1753733687111594, 0.19083680330268557, 0.1890708140129637, 0.17915310665510023, 0.18508324351096445, 0.18956906376195726, 0.18446174433282142, 0.1901774128199026, 0.19044231170223466, 0.19230830367234886, 0.18445810363725057, 0.17952863786758524, 0.18550078194492486, 0.2036709332594549, 0.18126727097854503, 0.19397631136931273, 0.19074180850264696, 0.18995473438329735, 0.19548932623378737, 0.19118634412893598, 0.17732268442080001, 0.1885911789452066, 0.18824460514155536, 0.1740903414518823, 0.18496661262412856, 0.19024738822458234, 0.19267481759042915, 0.18380583353596197, 0.18811954898047323, 0.19223244318532107, 0.17901877277672784, 0.18579199235774493, 0.19121073509799516, 0.1982692757779077, 0.20322527088052322, 0.19431068106415028, 0.18954856456096542, 0.18735852200400815, 0.17802709868713967, 0.19622672170601507, 0.21055714123796074, 0.1832170291557794, 0.18931748426464554, 0.18920483459499488, 0.1775217719596027, 0.1869980130156654, 0.17926102928280713, 0.18990775141874924, 0.18226703401832972, 0.18272706400265423, 0.18775407464987284, 0.19196790281808584, 0.19232694527265523, 0.18843073754274772, 0.1858056795914268, 0.20278917734291657, 0.18575707708541586, 0.17632427638856216, 0.1839916355627587, 0.1867223051561904, 0.18538669624016837, 0.19667176720514956, 0.1822657273143703, 0.1842699984565618, 0.19640942623584237, 0.19552531348932714, 0.19113462450531615, 0.18602593531064243, 0.19576278341731046, 0.17767865923322898, 0.2028790527778439, 0.1873383325033761, 0.18925865870691172, 0.19031990029019893, 0.18406346326277773, 0.19480802360162894, 0.1844122411965435, 0.18554625471622244, 0.18557105044728836, 0.19743021924341436, 0.20405666258091162, 0.18956765827111324, 0.1832851065634593, 0.1937999515580286, 0.1823725651487352, 0.1890365067606233, 0.1982261018514197, 0.19923979149631127, 0.19533372727434148, 0.18806700839050736, 0.18572027641678956, 0.19529379294215538, 0.19048470806250592, 0.19016001954511746, 0.1958079277231857, 0.18984266057400492, 0.18142032518391768, 0.18275567216895724, 0.17482967651876793, 0.19106102957242235, 0.1918827447931056, 0.19422496953933513, 0.18180279286851406, 0.21016459291328626, 0.18176863666506002, 0.18473213811445335, 0.18919214206795681, 0.18694894920472427, 0.1860342927971495, 0.19850628973607207, 0.18504740071437653, 0.17848977780877395, 0.18375072948246976, 0.1913570762201969, 0.17900595865468258, 0.18637129969689203, 0.18748760705723938, 0.17459167946962587, 0.1793788721011794, 0.1858948898331125, 0.20054958487014807, 0.19961130483174946, 0.19371572413525515, 0.20135365950406398, 0.19452701022519578, 0.18256171576583596, 0.1870696708795207, 0.18630583720242225, 0.19281094751643496, 0.18612847095902874, 0.17102750047693702, 0.18897328049474418, 0.18606932644245003, 0.17915415492812947, 0.18351213484002568, 0.1961803726353711, 0.18863790857242987, 0.1964790460386216, 0.19210790464412453, 0.18246213530672417, 0.18718277988397689, 0.18308893541992027, 0.1891333293705249, 0.18689861146595052, 0.1863534431938802, 0.19337633638851254, 0.18340278563300705, 0.18621037616672462, 0.19599023417301123, 0.1827275648334162, 0.1971068184314203, 0.1832405317281998, 0.1868739620123511, 0.18625156222738717, 0.19106843696049475, 0.17735255364335015, 0.1850151524105311, 0.17987829627984556, 0.18088998516180363, 0.18381280829758748, 0.18815730390833924, 0.19304396342470345, 0.19226264077152488, 0.18203316705853076, 0.18251252396728615, 0.18763219165688633, 0.19024360397007503, 0.20240767504194718, 0.18576828750879606, 0.1758759437769742, 0.19919959356211808, 0.18749438154685885, 0.18939799103268345, 0.17900962829073622, 0.19077126883533535, 0.1985270301327754, 0.188921671301262, 0.18814812816588625, 0.18632228269001766, 0.18624660559827116, 0.18341087273018503, 0.18352474287845777, 0.19861145408031408, 0.18505205968329494, 0.1875774459781452, 0.17888180912707524, 0.18445911878903545, 0.18580630985471452, 0.18138837879916178, 0.1846421730755343, 0.20234123890212177, 0.18356837825348968, 0.1977220339397183, 0.1913665451286184, 0.18849068261435714, 0.18481284178412946, 0.18377489582702386, 0.1884214284510881, 0.1840313476755211, 0.18303523125154758, 0.1894088468676284, 0.18517858985890842, 0.18717275886612544, 0.19506542245657893, 0.18795256030780744, 0.18139314855224173, 0.1922727093961751, 0.18935111689440903, 0.18925579334085516, 0.17176498029831516, 0.18205865844457145, 0.1812449064360892, 0.19121861111378077, 0.18395399150667308, 0.19254320970186495, 0.18129643794821337, 0.18790817285987274, 0.18253932000152767, 0.18179003931976961, 0.194440711527554, 0.20676522246941578, 0.1656762864340874, 0.20228300492426166, 0.19594618586501705, 0.2024344662896116, 0.1818751374559326, 0.18507376201354112, 0.1846555151738775, 0.17624951579209835, 0.18586309220657296, 0.19281400143363725, 0.1836941911349362, 0.1939363117041863, 0.19222781308053508, 0.19437606754707534, 0.18152468219876705, 0.19388009373805964, 0.18792367981717312, 0.17603862532624653, 0.19126561068433476, 0.18305861385394653, 0.18757359722696024, 0.18220496890024196, 0.1931662756602262, 0.19042152771732623, 0.18423916532462994, 0.18961852295413947, 0.2001072393298634, 0.19589030000571378, 0.18183847000983838, 0.1856241290397197, 0.19314849346677354, 0.1897043879931842, 0.18515839606131937, 0.19224414764425582, 0.19378804996584884, 0.18903849812158985, 0.18965359793989534, 0.18232520648749004, 0.19246755437799823, 0.18465807890449695, 0.177375625167106, 0.1818047919677511, 0.19553061173484074, 0.1883293667602551, 0.18282782794491131, 0.18384124335231075, 0.179855993232277, 0.19146293856128477, 0.18501667542536915, 0.1873791778294976, 0.17774605350878728, 0.19352344253041756, 0.18865533142150323, 0.19433980905309486, 0.18018759424800107, 0.18168327751495983, 0.19327504665568568, 0.19203433991900853, 0.18679341732189353, 0.19378796630109127, 0.1948347421111699, 0.18451024372433122, 0.1800611602872227, 0.18912564795654327, 0.19247438829988966, 0.18773721393159568, 0.18732738365195054, 0.1804569869113226, 0.1857654596839255, 0.18181918890997048, 0.18601340521995724, 0.18606572686566286, 0.18692853924359534, 0.1844800397441884, 0.18341681289543407, 0.17548063090286184, 0.19108697593516044, 0.188146611139938, 0.18930225497124206, 0.18798989164366894, 0.19613039525393342, 0.1946984556250506, 0.18360055387811586, 0.18360578114589782, 0.18466884876263373, 0.19273832518572606, 0.19608321016715566, 0.19802969318059507, 0.18050635830296816, 0.19369818178867204, 0.1744711482376713, 0.18148166763036158, 0.19323448235259397, 0.17663868053477078, 0.19699569856693958, 0.18955694712178459, 0.18773068434694357, 0.19622670362953543, 0.18013765360622566, 0.17440560013671327, 0.19542337942297697, 0.19534454222111378, 0.18567847574970625, 0.18116086638429946, 0.19205684801876205, 0.19140595064004678, 0.17888333635454617, 0.1886101264182163, 0.19646146957770644, 0.1803460049048897, 0.18019762174589823, 0.18961044870809554, 0.19004653276583314, 0.1852995110794329, 0.1863812972737322, 0.19247026804149936, 0.18401653195510334, 0.18667441780966945, 0.1922389450063908, 0.2141742643185728, 0.19000978570785113, 0.18198983494446844, 0.1882464419425603, 0.19429227273310487, 0.19271274907476496, 0.183691077411555, 0.18415685219970704, 0.17838890151036763, 0.17926762746560249, 0.17736849117905668, 0.19051676621587407, 0.17306006168879323, 0.1872692857805376, 0.18066597660921133, 0.19025369830568553, 0.18843128384607422, 0.20307365623463344, 0.1861278369121596, 0.18504930549804097, 0.18639208841029165, 0.19219956558774326, 0.17327248094910816, 0.19591820008420954, 0.18723884847504466, 0.17344402141286952, 0.1786473613414728, 0.19574611038477097, 0.1998953928095465, 0.1923406632313895, 0.17666931520583465, 0.18448213581739925, 0.19209185811012025, 0.18907541233219988, 0.18368826455369616, 0.19158782951128392, 0.18729930349698523, 0.18869681864353713, 0.1958121902515531, 0.19371787133599194, 0.1829529555550473, 0.18895662105845637, 0.18430579653804952, 0.19501432935327342, 0.18734482297379498, 0.18736322283383472, 0.18541594437480668, 0.18439384751613824, 0.1896125365635316, 0.19247693067960248, 0.17630152459453527, 0.18930936936784953, 0.18553559008685813, 0.19221234403703377, 0.18871455630531064, 0.18855566488779316, 0.19926780176602327, 0.18078053501450164, 0.1842490860986004, 0.18142856025460827, 0.20128669481533143, 0.18533342094489466, 0.18844408394761913, 0.1860850718014052, 0.1793320073771113, 0.1870718748056807, 0.19176768604630676, 0.18960643352215903, 0.19550663466598087, 0.18707849298935314, 0.18398809767126723, 0.17676610027250286, 0.1797334489660588, 0.20308913568602305, 0.1872465346003471, 0.18473603498396274, 0.1876090713295534, 0.1819289867222315, 0.18304237153184919, 0.18473794925200554, 0.17303475113949396, 0.19144164835860802, 0.19377496134114172, 0.19828686721828107, 0.18425718651103934, 0.1836631962851999, 0.1793445800236318, 0.2010656417438209, 0.17662091661041776, 0.19845292509433435, 0.1893415253960503, 0.1875154840254148, 0.18887689091683288, 0.18021001172048168, 0.18503232477157797, 0.19595476987365534, 0.18767790683271396, 0.1846308729982419, 0.17979415760798814, 0.18395159927021154, 0.19261786828103133, 0.19578980667016682, 0.18068290768376172, 0.18559313766460417, 0.19529903117025593, 0.20237583105655194, 0.1830394849108498, 0.19045265517091592, 0.18682633235256413, 0.1971883238030198, 0.18519224185192928, 0.20274037057416683, 0.18880102626494358, 0.19430922842420453, 0.18482201316735983, 0.18044920994972571, 0.18942444999028107, 0.17815226374595663, 0.1801855541399258, 0.18744717742906009, 0.18545036666119025, 0.18879575981997967, 0.1794205655444362, 0.19680131166271017, 0.17250466157280672, 0.1931016397198981, 0.18213859750822223, 0.20120151952542079, 0.18679768541172698, 0.1917154394030805, 0.19057075879760368, 0.18673205155590836, 0.18881411938506584, 0.1962612948690742, 0.19070345323389118, 0.19731234238373463, 0.2003796868696103, 0.1940070925333239, 0.1994653814625211, 0.186789021554476, 0.1955168866262445, 0.18462466156561497, 0.191572821963583, 0.18760227078256012, 0.19853033847456794, 0.18840386586225766, 0.1916841339764747, 0.17994944492891582, 0.18970059992774846, 0.18439658638168677, 0.1877432557433028, 0.17818172074131183, 0.19455437787796515, 0.18560293543687817, 0.17486310939004984, 0.18745147382489968, 0.17742710190599556, 0.1756305975560153, 0.19159048595377107, 0.17951431729022954, 0.1952370869941965, 0.19966174927208777, 0.1856729500980105, 0.18184449014939066, 0.18838851857443506, 0.18321937748082284, 0.18991105711637843, 0.1888737839604917, 0.18001847155352751, 0.18943699936333233, 0.18835592772182524, 0.18839340968942267, 0.1874729792783274, 0.1930256482405863, 0.18497028795707668, 0.1925205051448796, 0.1874450028454996, 0.19367372625079513, 0.1887956462465283, 0.18767121573000015, 0.1749651687767137, 0.1818077772480537, 0.19190584697574037, 0.19315738532011478, 0.1855845192824172, 0.17997377654372304, 0.18921873282636822, 0.1851649345978512, 0.19185789353381752, 0.19095567374521888, 0.18789960017235768, 0.1778204865856049, 0.18530635158150374, 0.18774385088474665, 0.18281839880048958, 0.2013769256905874, 0.18714144575993685, 0.1865608879855724, 0.19085087746901178, 0.18869991466078365, 0.1889738964290263, 0.18585798268902412, 0.18324511911788247, 0.189014300382972, 0.17270144673483934, 0.1816289752024506, 0.19078764962201145, 0.19733617870163167, 0.1875362602003852, 0.18187472073775998, 0.1834946346442403, 0.19095596742873697, 0.18967475772092685, 0.2031745720591842, 0.19183600792000247, 0.17743277910341165, 0.1955833263623911, 0.1819257630366938, 0.19655676715498088, 0.1998609355624738, 0.1907565882035679, 0.18020967469504562, 0.17660572414103426, 0.18782426576887534, 0.19919542383249284, 0.1942336035651431, 0.19108704330959367, 0.18567457947139412, 0.20400838945620603, 0.1814086915241767, 0.18717967972455654, 0.19965754137332248, 0.17954301902841577, 0.1873694672617358, 0.17964670261409244, 0.1918410005185762, 0.20126740831357134, 0.1925899442171085, 0.18832114958579935, 0.1828973735914322, 0.18606812598708877, 0.1733852376402435, 0.18466840131866813, 0.1863258411498078, 0.18447436185719363, 0.18508846338839866, 0.195393445880817, 0.18498556885802264, 0.1781122298757274, 0.17772719643676277, 0.18751999242258358, 0.19018102004860307, 0.18160033982328805, 0.1826138382726925, 0.17988863048255527, 0.1923019757765561, 0.17922016329443094, 0.18533164656884704, 0.1897373108831086, 0.1859464660539729, 0.1930579283159684, 0.18325949431808342, 0.20919346750226933, 0.17655494568992863, 0.17757506904211995, 0.19326969624912999, 0.19301583095149047, 0.1862292381907931, 0.1906376440698535, 0.19015336504350866, 0.18412946899943955, 0.1883690181559763, 0.19572935655913692, 0.18505269845039407, 0.18323699410445415, 0.18161313982936736, 0.1872955689839551, 0.17077691356436428, 0.18914356019639775, 0.18529320228267795, 0.19223320525407026, 0.18726668573947636, 0.19240196888535177, 0.18647201748051506, 0.18778788936048096, 0.1991102607224552, 0.1833818901307069, 0.19335145679201435, 0.1850429556038543, 0.1932256682813717, 0.1781423753375887, 0.2004732659995122, 0.19323817206819785, 0.19085964592815377, 0.1896384984287854, 0.17272652587185167, 0.20113581017510604, 0.1822887859826386, 0.1984005003883416, 0.17290024494627965, 0.19347122818037998, 0.18859785382876393, 0.18283181786696856, 0.18731858441741323, 0.19230804525968384, 0.19123336052171394, 0.1948472602726015, 0.16878729986729238, 0.18898495665884976, 0.1922597560180326, 0.19076723719390856, 0.17883364348229064, 0.19063732072543196, 0.18280528536200585, 0.19923095040353614, 0.19282695792434423, 0.19155871608266936, 0.19120519324123236, 0.1858740263905554, 0.1870588184545218, 0.18493758539115898, 0.1805697258990036, 0.18518938633097132, 0.18525984925383113, 0.19527907926710186, 0.18595347562340248, 0.19496669036855832, 0.18819932259002423, 0.18495171074645492, 0.18583833761712462, 0.19157906025101384, 0.19404602231816226, 0.18665235435332897, 0.18957514626078167, 0.18785514903867917, 0.19380610547119342, 0.19197554072982867, 0.19243650204936003, 0.18602481684311653, 0.1813343455939891, 0.18854801214642472, 0.18774190785649475, 0.17577555648158297, 0.18871990999013044, 0.1832512955651953, 0.18492404601196097, 0.19333745410867761, 0.18511579341037385, 0.2148339022051768, 0.18926332460745815, 0.18795743321834185, 0.19144785862651167, 0.1768725848579974, 0.1853266962066013, 0.19649778442480118, 0.18029355269113884, 0.18695040172798427, 0.17826399343717222, 0.1802238236073398, 0.19499561041838429, 0.19610494325569733, 0.190779807846408, 0.17361165207185517, 0.18948256653283793, 0.18146207178836743, 0.17078939016890043, 0.1965768791368809, 0.18612427262732925, 0.19134724310994294, 0.18192263107306797, 0.18607534868064526, 0.19184969991285802, 0.19790721870996655, 0.18325118724103764, 0.19831617800028462, 0.1917018720711826, 0.1820287476239464, 0.18940811582289502, 0.19826521836406494, 0.18788710712718218, 0.1876363615564665, 0.19854761531524237, 0.18941040171583196, 0.2040549997871237, 0.17968475242535437, 0.1864484499139329, 0.19323960327233078, 0.18318019057605367, 0.1815394514998739, 0.18590416511873653, 0.20550256151848506, 0.19647653406218246, 0.1956765073368673, 0.1809657618486304, 0.19147010526875166, 0.18529923539155127, 0.1936471501536572, 0.1954449282263229, 0.19209683620495926, 0.19139711515874078, 0.18810992497591736, 0.18880768311710627, 0.17985250683915718, 0.1898962311009271, 0.190047230816619, 0.18595866562352883, 0.17958019679752, 0.1866321415085115, 0.17995698774722915, 0.19103617017278457, 0.18625344434831861, 0.17703442904756955, 0.1897775269052464, 0.18485580885950853, 0.18474441629647054, 0.176877265019899, 0.19584566296855516, 0.1763433300480252, 0.18155229804969367, 0.19286673597266926, 0.19823476539862167, 0.18067987876414893, 0.18622815561229136, 0.18908298888218408, 0.20324779098902618, 0.18110872725309346, 0.18902817449051154, 0.18799856790048886, 0.1818515948695964, 0.18700525994436845, 0.1927943225884064, 0.19609344548682048, 0.17985153845937327, 0.18110696353732691, 0.18999611168771877, 0.1918259060847283, 0.1884186346909942, 0.1870058213363954, 0.18949205360921706, 0.18366137330727106, 0.18507555195207012, 0.19070637288870054, 0.1860369435889826, 0.18835744494353948, 0.18830511154875096, 0.1928053074490786, 0.1788081379520306, 0.18320187642579475, 0.18432946583463153, 0.1990354465835797, 0.18842950081022886, 0.18048274555417412, 0.1871659551065689, 0.1884913292230896, 0.17431776908377683, 0.1880588226070352, 0.19788158940055084, 0.19140110368590715, 0.19496760580462, 0.17596875176152552, 0.18297001176778444, 0.18261227159184226, 0.1921139503986649, 0.1921723388841342, 0.18659975814988283, 0.1880331951262746, 0.18829742711525907, 0.19933187095557006, 0.1879159576584395, 0.19181000519485528, 0.19054741829477484, 0.17283545633781908, 0.1754859504017668, 0.189179157157773, 0.17952408404493472, 0.18405789733618294, 0.1861334159414474, 0.19586502300033465, 0.18382954459371528, 0.1873735559918543, 0.1895888032537656, 0.18950911935241785, 0.18806836839355354, 0.18886141833192147, 0.18014511060440733, 0.18966210857239313, 0.19175811591180766, 0.19550875272636717, 0.18759092234986774, 0.1864884258912991, 0.17570207178695582, 0.18643932049648965, 0.18663581379656088, 0.18505070830923906, 0.17473696857092394, 0.1880836221892709, 0.19113998049510594, 0.18894101759166684, 0.17821948904981028, 0.20313890580712932, 0.185265811980672, 0.18373996574016868, 0.18530106806488592, 0.18930190023007687, 0.19039132885001936, 0.19506041554022754, 0.18506894637379528, 0.18560444773907986, 0.18229767025231688, 0.18830533569495989, 0.1905170573677172, 0.18543810585066647, 0.19438884357157915, 0.18967845579557527, 0.18329242004647261, 0.18712455578292445, 0.19197005846930731, 0.17866880868694499, 0.1964763599758152, 0.1848086074618704, 0.1984737804807471, 0.19113215221352584, 0.17918016841286522, 0.1774540535599536, 0.17978608941274837, 0.18991662803262707, 0.1815071687170456, 0.17453713119425157, 0.19157587965142664, 0.18723379682587254, 0.18291018061649067, 0.17920414249546682, 0.1840865588763711, 0.18618831778195016, 0.19657169390736362, 0.1893052024014645, 0.193293806781925, 0.18063906708234181, 0.18537091344698833, 0.16414146119414383, 0.1804787204773252, 0.1791963258988445, 0.19608391857332252, 0.2031456607711586, 0.18463101311739782, 0.19428386878671708, 0.19133895412116037, 0.18888121913067815, 0.20040024126075737, 0.17371381920703366, 0.17772251820048757, 0.19134523348894208, 0.20934700182257063, 0.19752397511608538, 0.1890137093623965, 0.17595932266274206, 0.17997788466361667, 0.18131677684568076, 0.1815078758002618, 0.20029938841115769, 0.18934142843832788, 0.1891382421143399, 0.19526007460011205, 0.17956173208916473, 0.19149924471282326, 0.1841953415337987, 0.1878011347253942, 0.19320246818869063, 0.2023695008532127, 0.19213047034302067, 0.1902758830311274, 0.18505200060786073, 0.18731190573965825, 0.1807464766273632, 0.1897011191281299, 0.1834311322011674, 0.1803390670600213, 0.18198333756097237, 0.19373078731728288, 0.19126677649491494, 0.1893802968331611, 0.18422875941445657, 0.1763462464473647, 0.18890175585708505, 0.1904462851044673, 0.18804632634079432, 0.17985677599411617, 0.1828406236631551, 0.18823022261233638, 0.19907215971891276, 0.18060433280896407, 0.18575487286483236, 0.18511170986707373, 0.1924613613315361, 0.18253247913870072, 0.1816741489110475, 0.1917192267643693, 0.1915267371010465, 0.1925883496513267, 0.18180560047863603, 0.1843637080842364, 0.19051892182170393, 0.19052801442055073, 0.17847455471736232, 0.19162810663778562, 0.19163465889261272, 0.18599678491991617, 0.17824115485601166, 0.1987943942451392, 0.1833386849548164, 0.18858682549084493, 0.19105228736579022, 0.19108618197839092, 0.18192416518172308, 0.1960003835975284, 0.19595159867860243, 0.18539824777777034, 0.20079315868237588, 0.18647182684175082, 0.17706195549172313, 0.18358855152501993, 0.19677203801600823, 0.18255311621805034, 0.19916958900839826, 0.18561536258322556, 0.1792506968425699, 0.18800424197348609, 0.17894722083097309, 0.19185690509943557, 0.17579582306567057, 0.19852654825515603, 0.16890765775433844, 0.1912268261192532, 0.19122050034581548, 0.19434752230667598, 0.19733382956523388, 0.19171961166594473, 0.17683235806554698, 0.18777397559869916, 0.190523819489015, 0.18593855855709968, 0.1795036080317778, 0.19221542828277496, 0.18878932836368886, 0.1916398425878976, 0.19029567763719574, 0.1893324877643067, 0.18700172965363535, 0.18258815692336247, 0.19841259268743258, 0.18648460245794843, 0.18509523508046294, 0.19411971221035637, 0.18195440894989512, 0.1911104238371734, 0.19376276837205914, 0.19039845280709594, 0.1935145866213707, 0.1914125386566665, 0.18572799814484625, 0.17458257924246467, 0.1927317650848668, 0.18142600315410237, 0.18249822621327505, 0.1874655210294608, 0.18770279536959475, 0.19474888532805812, 0.1999762394965698, 0.18285644988010694, 0.19759885356830068, 0.18356546314511638, 0.18111955944721447, 0.1811661552425516, 0.18983714320458034, 0.17274722236900242, 0.18893671699556602, 0.18837134301954472, 0.19746806153959834, 0.1961471532209762, 0.18747476769869864, 0.1905561318385785, 0.19152219447266794, 0.1847225101594975, 0.19169130353700695, 0.18449552111320958, 0.19233108744939706, 0.1915729309291815, 0.18396186754739544, 0.19187946543103346, 0.18012714944494448, 0.18708313528834325, 0.1915686459572142, 0.1786984304245978, 0.19442216835381143, 0.18637877444809806, 0.194943785001034, 0.18697777948049976, 0.1869346449351544, 0.19071068782169195, 0.19055737145685134, 0.17563261347182957, 0.1820193247945733, 0.18222425717378832, 0.1796868031948822, 0.19616858740144394, 0.18381779388190855, 0.19744292797779658, 0.20435734482000337, 0.18884478371765478, 0.17951997437792164, 0.1888863485533808, 0.17178015900284843, 0.18245293313678246, 0.1791515759972443, 0.19027850654903034, 0.17841522147598818, 0.1799851330285396, 0.1908252889493688, 0.18849613574393415, 0.18408806820889545, 0.18428133428781923, 0.19010218730126877, 0.18717917470621656, 0.19673416642600386, 0.19498705495299726, 0.1872411672941132, 0.18217158922032964, 0.17902487646082832, 0.1839729526890686, 0.1770012934812789, 0.19145709181299034, 0.18052663976175087, 0.18770853506905164, 0.19413986611788064, 0.18577333528169598, 0.18362450162923136, 0.20447855652903615, 0.17792257870884493, 0.18612540189978052, 0.19647932078730676, 0.1897565797916794, 0.19303103464222945, 0.17881489632732783, 0.18228966849854, 0.1976157842523529, 0.17883322894888193, 0.19755886348020407, 0.18240884410870692, 0.18406120612919746, 0.19379785549544343, 0.1929622774919265, 0.18880666436426777, 0.1827278855211986, 0.1858642188034544, 0.18658934440931063, 0.19073614001438016, 0.1866457087241967, 0.18862530317702797, 0.1806041222819338, 0.18342391179812653, 0.19868579557930968, 0.19491887866055368, 0.18011319862447694, 0.19380295724390345, 0.18469756343078908, 0.18577167267815162, 0.1840680357865674, 0.18307484812722294, 0.19202925679951663, 0.1787622006191318, 0.18214657554156397, 0.18425396250266163, 0.1844438189824737, 0.19066616333574576, 0.17808504709395584, 0.17874085386876915, 0.1794600708802857, 0.18811174335428166, 0.18574732281955011, 0.181500979206536, 0.1955463910055177, 0.18001087154463238, 0.19965597516749983, 0.18366017802771425, 0.19726002214469626, 0.18048229173466077, 0.19385259395918295, 0.19028621175285768, 0.19249405257103433, 0.1823307121576405, 0.18913691168949556, 0.17798367860329545, 0.20095007713870505, 0.17755173516001643, 0.1938742160089129, 0.1832130678425517, 0.18782846574358505, 0.17641411932725654, 0.1920570847864078, 0.19008862849105113, 0.18334720493034795, 0.18500190392757646, 0.18132490883938296, 0.18170045384164546, 0.18189809504442372, 0.18819702606097258, 0.1806643041477815, 0.18949971489298925, 0.18430087597756548, 0.17857548492841654, 0.18305594364591263, 0.19121824101940207, 0.19156573963488926, 0.18194421938110344, 0.20532843910836862, 0.19141996991933113, 0.1910010009704233, 0.18660011696157686, 0.18390388712538835, 0.19990540729716352, 0.1844026793961173, 0.18311587269227483, 0.18652257232792988, 0.18800618703752392, 0.1876492199713211, 0.17784171381842753, 0.18811496014396115, 0.19322747038948085, 0.1869158080302264, 0.18503577078741645, 0.19112586299280016, 0.19394160914129588, 0.1970092072476405, 0.18333396098398388, 0.18822822630711394, 0.18885753857659512, 0.19998974758076904, 0.2009642466151037, 0.1905914583518561, 0.18946593633421568, 0.18264188387457875, 0.2026232268990103, 0.18066209186400417, 0.19224871431370144, 0.19869695974631704, 0.1886491029408525, 0.17477481011756255, 0.18760909369754006, 0.18898810246834385, 0.1997106024917634, 0.18969683539559204, 0.18794440272194296, 0.18432676866691564, 0.18740681148622376, 0.20239328113571928, 0.18368819001244052, 0.18788553469995103, 0.20214925139483544, 0.18949956834338497, 0.17998041595023945, 0.17820835479028774, 0.19154149653784752, 0.17785436220486886, 0.18313745155687014, 0.17777595159766674, 0.19297132404704412, 0.18377752548640133, 0.1808737539986114, 0.17937988535612837, 0.18883945968045313, 0.1782118432300884, 0.18311927894886498, 0.18455566921290556, 0.18804398671290373, 0.20109751604978265, 0.18485789334468003, 0.18773841991790105, 0.1875749887539963, 0.18749214240641718, 0.19057519155751101, 0.19417184732025486, 0.1817843223981756, 0.18650758747190066, 0.17687715564390738, 0.18393215558673184, 0.18365179521921107, 0.18860751009409446, 0.19017851109042286, 0.19218633146050226, 0.18363278366713198, 0.19653907134109733, 0.1717841535736332, 0.181480630530012, 0.1852989565582417, 0.1938617566875219, 0.1779013264411865, 0.19249985421751242, 0.20011935113276144, 0.19515346314535678, 0.17611033732333073, 0.19006005918848606, 0.1848389193794118, 0.1999897763525033, 0.1859843207116699, 0.19663820896579545, 0.19474174117920867, 0.18544944648081357, 0.18293106338205176, 0.1915367210942446, 0.1936114931370123, 0.18404154448033488, 0.19224441949118637, 0.19056429175072415, 0.1763894807699675, 0.19654847502874492, 0.193358483115329, 0.19348738509988708, 0.1985432762189913, 0.18774371853172617, 0.19005048209359138, 0.1800663923516394, 0.1830023911384762, 0.17609395320358817, 0.18188638793213752, 0.18884235000833904, 0.18414462344016344, 0.19417145661802687, 0.19359065217656268, 0.19020540564116886, 0.18849231615569542, 0.1908757125520536, 0.19173619317468577, 0.18606798519730386, 0.20063374082723984, 0.1900960033936142, 0.1982562838618473, 0.1886584872826067, 0.19166045602024673, 0.1903629112606752, 0.19665371720834984, 0.18824338174901376, 0.18741822040504635, 0.1887722746340438, 0.18564258953314586, 0.17803476547520578, 0.18674624260143025, 0.19747868673508084, 0.18607023669613657, 0.1817825400789238, 0.1748605632268964, 0.19158177017814199, 0.18254611144899388, 0.18445773419217773, 0.1800123261099379, 0.19890606654369036, 0.18932672202085096, 0.18992078221224362, 0.18995970274003787, 0.19062453534586415, 0.1928552045043628, 0.20565239860539977, 0.1843960696832514, 0.19278800906486132, 0.18350103607030804, 0.20077640257223203, 0.18321470930306585, 0.17872942718798943, 0.1868095562356771, 0.1907791754004001, 0.1857237700363315, 0.17119061316465906, 0.18665357961110166, 0.2011830688713406, 0.18999661914208682, 0.19965182641907578, 0.19817704813416337, 0.18631054341709677, 0.19760987775483707, 0.18656197667250113, 0.19097732120147295, 0.1853785594853813, 0.19433846494691256, 0.18931106451219978, 0.18249351356717847, 0.19087081458380128, 0.18774545301655443, 0.1863215949758981, 0.1939713582868747, 0.1822393561120478, 0.17910503608577483, 0.1861796066653312, 0.18164714834141252, 0.17788130169275523, 0.19397379357939049, 0.18068623073331413, 0.19112036567196736, 0.19605545116416742, 0.18885441382612142, 0.18852516764691823, 0.1925344570237476, 0.18711089042675222, 0.18608755632102814, 0.18407945090895705, 0.18713753522612447, 0.19605194666858325, 0.19042704877067457, 0.18239207825468812, 0.1949271391429938, 0.19050050992774395, 0.18727281135414503, 0.180767273394149, 0.18193177271340014, 0.195893751161696, 0.1937663790915017, 0.17819273718845438, 0.19446855945117872, 0.18525016925569235, 0.20183070342245615, 0.18520300599525827, 0.17854939947937182, 0.19320311666871873, 0.19397416261082653, 0.1874763630080168, 0.19197877488411613, 0.1989309854056954, 0.18525313572585742, 0.1865041433786721, 0.1997015132628761, 0.1785456712507668, 0.18403872389758005, 0.1859075749049966, 0.1853570775017451, 0.1838856866475043, 0.186666729644615, 0.18821502791646288, 0.18524117641232363, 0.1814576492696366, 0.1930521113815833, 0.19578832687467912, 0.20100018466311892, 0.18946792701878618, 0.18969675922264073, 0.19820510110879852, 0.19001358173112176, 0.1797720162540098, 0.19285505639615555, 0.1751133331827183, 0.19737474574453573, 0.18601745475536438, 0.18579278477746883, 0.1919670658279933, 0.19559740522393163, 0.19672282282847786, 0.19316593068743693, 0.1809051517707942, 0.18866185910492592, 0.18519763887546045, 0.18831450495984595, 0.1829716578134288, 0.17785136569385573, 0.17843681860012522, 0.18725523224555357, 0.1759462019110937, 0.19863597254031962, 0.19245999141940964, 0.18901750035612486, 0.18772872006004318, 0.1899836746147049, 0.19204773663503907, 0.1824625614777374, 0.17626581947178668, 0.1990672377862759, 0.18549759338985655, 0.18605937657378394, 0.18500813868763563, 0.18357472227473606, 0.17389000654201095, 0.19750427454055453, 0.17884423665166904, 0.18854318073023352, 0.18509117595845745, 0.18786540316767217, 0.18886024652169575, 0.18763244784257266, 0.1822166633869054, 0.1741652092973387, 0.19110202445995095, 0.1968985051466991, 0.1911080498957696, 0.18248803301535776, 0.18999462979021786, 0.19694621351268643, 0.1849259251354245, 0.18395594067844243, 0.1787582131710988, 0.18785968262529984, 0.1845062559854913, 0.18583436076810822, 0.18965948248781858, 0.19480687256970824, 0.1901275891219739, 0.20169271299061373, 0.19414324375501948, 0.19110287698124856, 0.19489871806972286, 0.19263563769728387, 0.20216775254223632, 0.18160065947760937, 0.19708531729033282, 0.20048025692858168, 0.19831948398227414, 0.18309895289839173, 0.1850519148372847, 0.1913005118782356, 0.1884930422815506, 0.18009845843889055, 0.18397262409246903, 0.19271630305248275, 0.19399253480247614, 0.1969352722578068, 0.17921745216907092, 0.17950674507888198, 0.18634981786535146, 0.19354579384576873, 0.18931453442781745, 0.1716383978160445, 0.18035852440195269, 0.1893884689150324, 0.1860259326743323, 0.19298721660846244, 0.18919341420542168, 0.18742418294175517, 0.18367512499251124, 0.19994980056977116, 0.194660201258371, 0.18724502411684535, 0.19346172419694097, 0.18776616061538962, 0.19498948395345042, 0.19114009028320056, 0.1885927390550527, 0.18459640477522019, 0.18178011584782716, 0.1951004342289104, 0.18566137256400506, 0.18910578645825546, 0.17869174758371256, 0.19082905044430998, 0.1941524339979556, 0.18918086128255296, 0.18708899764636336, 0.19203288364720494, 0.19521940402095408, 0.2084154478014113, 0.1867238823884557, 0.1932248805037475, 0.1825549154467906, 0.20063372068049568, 0.1859652585886141, 0.19292014827354087, 0.1848853674383018, 0.18892114204367944, 0.17585455707098982, 0.18058693269108156, 0.18913702300914073, 0.18235197913465498, 0.19211270627866178, 0.17996180386302577, 0.17848095501160266, 0.19476660427222195, 0.18578436637308807, 0.18489566333662158, 0.18540160295224584, 0.18594411192995003, 0.18958299687511002, 0.1873357489903803, 0.18889170269813302, 0.1843599299688062, 0.19768549218577045, 0.19121501319149878, 0.18440414553543186, 0.19421788630144413, 0.18645881058545394, 0.19023732784988373, 0.19773658885581358, 0.1877991832131104, 0.18704052386471853, 0.18962969028310406, 0.17989845017782133, 0.1899471534650225, 0.1983237535627571, 0.18476942468291613, 0.198871529420747, 0.1763276778065769, 0.1870314024539869, 0.188247922630147, 0.18866173598380045, 0.19765491690206044, 0.20042469051713158, 0.18554456063583494, 0.17941986826270767, 0.18203909197966264, 0.1818123738205841, 0.1927869414661689, 0.1961656142590448, 0.181942985281869, 0.18768664997136575, 0.1836619416932839, 0.185705415873971, 0.18005541381016013, 0.19477830446865735, 0.1775931320275971, 0.19104314915950013, 0.18666777650928945, 0.19307290204959643, 0.18865363437493993, 0.18638236243752151, 0.17996663964340864, 0.18843697943205534, 0.19477126159399974, 0.18110929202328732, 0.19172612148963097, 0.20124578426111617, 0.19676886618886266, 0.19034718619261368, 0.17859927031912315, 0.18805931917200858, 0.18356219056626283, 0.16916697335021225, 0.19745328148579397, 0.18687759443152818, 0.18119260714238172, 0.18937035863671212, 0.19186863731846307, 0.19459001163493986, 0.18774015273755257, 0.17698991670198294, 0.18535763935990315, 0.17849681338177753, 0.1806526301701782, 0.18680805016795282, 0.1854780498715567, 0.19322756421515558, 0.17722643710746921, 0.1754007134378077, 0.191768292544972, 0.19679447060691585, 0.18431097394998705, 0.18242098117983957, 0.18374861286295868, 0.1936201989768825, 0.19027591838846153, 0.1876847515841727, 0.17785034998484775, 0.17530898870428874, 0.18650427481107149, 0.195742649409985, 0.18799318108190094, 0.1938799689028632, 0.19007951253355887, 0.19735480675339792, 0.18370778484255287, 0.19494006087254212, 0.18949540342933432, 0.18939348878453002, 0.1875654777008522, 0.20250177595865118, 0.18379404366432708, 0.1883278184267824, 0.18781453325712796, 0.1878308212209164, 0.1863311908356606, 0.1869974373533982, 0.19183041904023668, 0.18825713750969586, 0.18398453901131712, 0.18756052359874317, 0.1764634313041093, 0.19274830053527464, 0.18359889510531055, 0.18252663025435623, 0.19216836130336298, 0.18918021909604157, 0.19554264282996264, 0.1855530006613409, 0.18929541812113154, 0.1772643563951099, 0.1900407746872438, 0.19416192746528446, 0.18342927722679558, 0.19333610000910065, 0.1874971727716876, 0.18608348353338142, 0.179829677424833, 0.19207649077852978, 0.1932669573458637, 0.18812297009969633, 0.17972657348347837, 0.18959579950295796, 0.1834289595769381, 0.17869754975566352, 0.17765260139806363, 0.18242127268497269, 0.19030213038914737, 0.18303748690358806, 0.1968975411182864, 0.17551911052918223, 0.18918722014161984, 0.18251941787927176, 0.18609246212904118, 0.1803479464140697, 0.1958816059152062, 0.1859987982713894, 0.18015530286753348, 0.18313893360197403, 0.2022435473734542, 0.1833313245403364, 0.1934656667423928, 0.18555063336125, 0.18615763854814293, 0.17793130023111575, 0.18446217152193176, 0.19274512083879297, 0.19043050260325284, 0.19526022906247684, 0.19322363984260243, 0.2008180620866432, 0.18713180269010066, 0.19293546742406492, 0.19415512873438054, 0.1831026930476127, 0.18675826159534945, 0.16677734028647676, 0.1856489943564559, 0.19118449913647853, 0.1800931269977008, 0.1865293954635771, 0.18167208748437155, 0.1930493244347661, 0.17526654461506638, 0.18856291789619145, 0.18586408593076298, 0.18195313905069657, 0.17949388908707917, 0.20293224555315467, 0.1938173480787285, 0.18047990186810103, 0.18604993712257303, 0.17773838920178323, 0.19808082152042655, 0.19680327514433235, 0.18439114213656854, 0.19231539222370983, 0.18839563632145714, 0.18907227679217647, 0.17631750414723973, 0.20396963377909738, 0.17946216981335197, 0.18992395503052117, 0.19093448260288245, 0.19092829582040527, 0.1825749999268309, 0.19419225874467122, 0.1792562893169995, 0.17848532864778266, 0.17669913072000187, 0.18165154229001804, 0.18074773776151645, 0.1852313894049233, 0.18901759925638123, 0.18967151040593913, 0.18688787320740494, 0.19034073902310622, 0.18362873081276654, 0.18596876809005708, 0.1854537161648962, 0.17504368916020957, 0.18683152694027225, 0.1925431079327069, 0.19287633527428466, 0.17858135329713157, 0.1831489567734583, 0.19437012405435464, 0.18525126764767888, 0.18981975625748926, 0.19807959924216942, 0.20053476666577208, 0.18293526491650924, 0.17630256232014135, 0.18180274445337602, 0.18709858560204026, 0.20328636035181294, 0.1875397162392856, 0.19068462173318645, 0.17649711512118801, 0.18754549110087002, 0.18742235057988219, 0.18805177873349707, 0.17908278916033762, 0.18692611551010793, 0.19438934613416686, 0.20275994217699514, 0.1891224851855926, 0.17360604399086527, 0.19566928599209252, 0.19027920039055948, 0.19797339109748419, 0.19161098633836252, 0.1728445563064489, 0.18522180234756852, 0.18020959735603256, 0.19520126041474592, 0.19192003901392196, 0.192901526259477, 0.18322716582408, 0.19081521390209755, 0.18720227749143786, 0.1824871377790324, 0.18898259779474566, 0.18216892915484217, 0.1891917627100121, 0.18846826252823032, 0.19659194882263484, 0.18052604813212245, 0.19344756268609234, 0.18751514127465443, 0.1908728967085417, 0.1768428232546776, 0.1825907729318384, 0.18772989918304298, 0.18280473370212752, 0.19295454339244691, 0.18610584035594227, 0.17670984381941127, 0.18716860568862836, 0.19144208954760664, 0.20056885273503813, 0.1868446468229371, 0.19010015161234828, 0.19105470318637194, 0.20224810338435048, 0.1892772957940287, 0.20222282508641548, 0.18994094665084738, 0.18170311998979574, 0.18597123008781483, 0.17898838139946152, 0.19818679476789136, 0.18555625196652037, 0.19021581606550592, 0.19005861165601504, 0.17852788915465048, 0.18149753578209074, 0.18079374154935574, 0.19118809798217445, 0.17352843908432983, 0.1869412236256034, 0.17505461226952437, 0.19820226739408936, 0.1852041025433724, 0.20781819824962314, 0.18396801166516888, 0.19888290146457227, 0.18453810815247176, 0.18031438150422271, 0.20940134678765088, 0.18219803508712828, 0.1850789820274163, 0.18449250066089387, 0.1838586848853549, 0.18568938858037318, 0.19727748921206598, 0.1885335504237506, 0.18916005494853438, 0.19540881103531413, 0.18214631688804364, 0.17404189340160758, 0.18470372397363344, 0.19264927824565653, 0.1853614735731704, 0.1921624046904812, 0.20309365730808107, 0.18731652478853364, 0.1880516724702714, 0.18485192975590123, 0.18540101192828898, 0.1886255654626533, 0.18312755276003712, 0.1673580434510285, 0.18348029697674162, 0.20849898314209978, 0.18841822792590582, 0.19280403798025797, 0.1883023960030758, 0.1860042460502483, 0.19677220009055751, 0.19196739720523026, 0.18703200143252935, 0.19335924958813366, 0.18979173188365728, 0.1971479884147995, 0.18112154938738068, 0.19407170971989285, 0.18629871291621278, 0.1795814700923908, 0.19108263999465883, 0.18669823212362469, 0.1934636511411916, 0.18495701803490613, 0.17818767409489822, 0.18713257424972912, 0.19255303482962527, 0.19274362689788063, 0.19058807581718318, 0.18230823861658046, 0.18256396963585658, 0.18478387334658947, 0.18755099669723804, 0.19426882442157062, 0.19164652624216125, 0.19445540281646145, 0.1903785281362507, 0.18822342472796422, 0.186344725963304, 0.18280941943518797, 0.18035950482375612, 0.17646462640611366, 0.1803511007901183, 0.18300136194919936, 0.18121409103119165, 0.18552546412584214, 0.18074523598399447, 0.18447751531369688, 0.1873758682616801, 0.1880640555631562, 0.19093057922974277, 0.18274532359442347, 0.18984238967199926, 0.19297286486839182, 0.18448646189896892, 0.18752020545524492, 0.18752235421350752, 0.19902933653459032, 0.18716588846100035, 0.18361452566762002, 0.18017787029859378, 0.1883601648346209, 0.19058758664340228, 0.18218880170211282, 0.1930580685649429, 0.1857749986869648, 0.19143620604562034, 0.17212730604760149, 0.17759621520393037, 0.18054683665157645, 0.18667801293927996, 0.1880387720890739, 0.19052183453562452, 0.19342306061405715, 0.1793568476406958, 0.18066602045295313, 0.1856310769822307, 0.1819479462369754, 0.1869836349718527, 0.19018279487249387, 0.18557183776873626, 0.1871259426467745, 0.18807170015303645, 0.18574545436815282, 0.1940140531675305, 0.18209089028642902, 0.19458769328071884, 0.19570552882272554, 0.1764213190484214, 0.17715298372774627, 0.19532973110682353, 0.19394535602895727, 0.18809465971271966, 0.19243366375579382, 0.18089213775256457, 0.18181563365180076, 0.18458359584255493, 0.18840934515865415, 0.1800325097448609, 0.18065905488312997, 0.18720833751942534, 0.1903527715764178, 0.1899070603922922, 0.18994519996630885, 0.19756571936287087, 0.1813189672663996, 0.18790926388914697, 0.19844140819410566, 0.191642584698819, 0.178214585731184, 0.18042578418010904, 0.17297879129483146, 0.19027401773032868, 0.1834790782492767, 0.1936738897645201, 0.1828280018187032, 0.18369945334994994, 0.19753465166706544, 0.1741332320452964, 0.18756129735211224, 0.1924565877692608, 0.1922178873823533, 0.1934960054597529, 0.18814184412340715, 0.18413063138099425, 0.19316858892671535, 0.19215382394249592, 0.19212585809879465, 0.17739696214677664, 0.18484621751067598, 0.18061767690364822, 0.19184474002837776, 0.19094261324622236, 0.19164735791085233, 0.18687681488568358, 0.19137182575670983, 0.1776640156740707, 0.19985230374837462, 0.19212475241350102, 0.19034295073741433, 0.19565979931159497, 0.19511339388002813, 0.19400128862859017, 0.18770177967056992, 0.1854715694215372, 0.2001557544997552, 0.18561829252509432, 0.18622116963190594, 0.17089122162178158, 0.185002095662895, 0.190519233506914, 0.18203143663479956, 0.18582298813518294, 0.18944197950647063, 0.18430397337922172, 0.1843077449179085, 0.18283667951255209, 0.1829931732451949, 0.19250905571058383, 0.1803420034429116, 0.1960873120865143, 0.18340102776416942, 0.18779979120491685, 0.18853653082995872, 0.18447609180028599, 0.18096101176659246, 0.19705531931238446, 0.18972748032116826, 0.1825650244083734, 0.19377182647439808, 0.180286950958011, 0.1858263512524862, 0.18541223430640097, 0.1824177047115286, 0.18765787410720677, 0.1804706567044378, 0.19887490743749084, 0.17248744020545057, 0.18471705165812738, 0.18649536486619575, 0.19971062366252512, 0.18397870040288639, 0.19122085485941157, 0.18862567353406023, 0.19408798750773015, 0.18543404264341637, 0.20221112202810773, 0.18549123992513802, 0.192922678088655, 0.18281365054156104, 0.1911210553618362, 0.19209787888504146, 0.18518577815924334, 0.19353216475958102, 0.1658549352323603, 0.1861111897584827, 0.18919192369284193, 0.19071652599286207, 0.19201351641934125, 0.1867774836668539, 0.17945919950230102, 0.1766012421740965, 0.18879720739889677, 0.18712454561772499, 0.18427673724136603, 0.19466823911064868, 0.17882783776520025, 0.18925243204532685, 0.187174244948178, 0.1973555217370244, 0.18437405670699186, 0.1832825167116719, 0.17876899757512463, 0.18376008536042981, 0.18037599174209484, 0.18625338145529666, 0.17626012398222768, 0.18322130673283107, 0.20336688421700605, 0.19425621702129944, 0.20141127749046095, 0.18408436656215826, 0.17806955094755272, 0.1883115841304992, 0.18835029758396027, 0.1868686171151592, 0.18254399709302546, 0.18326520221597048, 0.1870892984843591, 0.1731528614926512, 0.19759255267263653, 0.18482170543484347, 0.19927962294541374, 0.19263524371261204, 0.18964391749116946, 0.1817405916250737, 0.18551560795380626, 0.1973199764232934, 0.1897673461563692, 0.1902815135127957, 0.18560486056177766, 0.1858559031556586, 0.17907980782434887, 0.19501719542638538, 0.19760691980228123, 0.174745953300238, 0.17853853511055412, 0.19309702171467896, 0.18706657450903658, 0.1799762272392586, 0.18488340452596208, 0.18475451509598867, 0.18453109730518039, 0.19118348413210265, 0.18375505097434175, 0.18939791250284974, 0.18824662407204204, 0.19410630681436766, 0.18274696630217807, 0.19702636161965026, 0.17148693383958388, 0.19067049708577044, 0.19774041843601595, 0.1874834353639212, 0.18922906646412, 0.18948871550019095, 0.1804731249888136, 0.19997335184511547, 0.19226972060434566, 0.19152374929830887, 0.18790905507543923, 0.17700555188181638, 0.18716534130353196, 0.18720894672586713, 0.18135462598336757, 0.18388517222262898, 0.184004307715539, 0.18871688264104053, 0.18692777325530896, 0.19989539781753107, 0.18317287623269513, 0.188192764230577, 0.18832631515357187, 0.18614876101145625, 0.18926910585571352, 0.19067994777501382, 0.19859863984929846, 0.19224180602323826, 0.1962247783537446, 0.1872737775266064, 0.18577870700476087, 0.18394565215818476, 0.19044495010081164, 0.18081870692900195, 0.20163746289855927, 0.1866712897099751, 0.18163024106133244, 0.18156318924103276, 0.17418829635028327, 0.18006384424865957, 0.18857569487327652, 0.19285958959164431, 0.19612548832456544, 0.19402661568695648, 0.1912335324473804, 0.1838344077510816, 0.17225082207007364, 0.18514209791650185, 0.18870387911100422, 0.18846807680375055, 0.19001917916911093, 0.18713876052511835, 0.19150375951039003, 0.19093905126779231, 0.19114980012157065, 0.20008216086584954, 0.17285800889874245, 0.18666419790789754, 0.19898817871719857, 0.19273354642980295, 0.18910325573319123, 0.18985923615505226, 0.18470924631519242, 0.17988347883840022, 0.19133902777629946, 0.18916169345275538, 0.19749559729111904, 0.1823158474193265, 0.1815713118772351, 0.1716132841653469, 0.19645981535750445, 0.191283054688563, 0.1833821485691543, 0.19067340440810282, 0.18062805752246877, 0.19652185697119753, 0.19187748806866364, 0.18987140515644393, 0.18406283464656556, 0.1775418141183215, 0.19350354791621166, 0.18656656003585262, 0.18555868941298448, 0.1927938793988561, 0.1932860441954578, 0.17860416803827747, 0.18217364886709153, 0.18491865013733064, 0.20004975536160494, 0.18299443823889178, 0.18237146770050247, 0.17241412487815488, 0.18156057058795602, 0.18934297369244726, 0.18935103982928783, 0.18049650514329948, 0.1739944280862407, 0.19501856845028417, 0.19207835096696113, 0.18708551283419664, 0.17855166986131127, 0.19010668252740773, 0.19740183067877384, 0.1771757005861298, 0.1909854330602662, 0.18798263153645886, 0.18687124892273985, 0.18551364372094817, 0.19978632549776307, 0.18734019800070353, 0.18421973862997892, 0.18555789448046542, 0.18277008916786852, 0.17949869609687868, 0.18500551884662483, 0.19693575992863335, 0.1820978140379934, 0.1936640077688197, 0.18814371493094217, 0.18890827116468598, 0.20151966685108977, 0.1916999665264715, 0.177675061362449, 0.18952716242425527, 0.1903580454672438, 0.1842621950565586, 0.18483872400502382, 0.17949995645575093, 0.1895611623886579, 0.17722968491719723, 0.19664344407577047, 0.1922506841687761, 0.18502020192364677, 0.18496318684027255, 0.18279238246208804, 0.18709031245102906, 0.1978631178511015, 0.18433318017462477, 0.19303538303024645, 0.18530894978868223, 0.18134737150253488, 0.191854446685244, 0.1900508225554641, 0.19046470750905037, 0.18289173727547378, 0.17467002503258242, 0.1845934704886148, 0.18930650680710753, 0.19094465247389206, 0.1741771599079124, 0.19127214092537786, 0.18268338975229, 0.18400772168165178, 0.18760971933615175, 0.18878626040100957, 0.19187636285454276, 0.18025796127273555, 0.17849960183274405, 0.19333127649084708, 0.1947205930840653, 0.18402938146475983, 0.17762262222861014, 0.1888839183341512, 0.1934189026931508, 0.19163254524036882, 0.18665541647643794, 0.19146926129749822, 0.1869891997549044, 0.2002938945119208, 0.18000358215578652, 0.186583317349545, 0.18434405942664925, 0.19770810554027718, 0.18521262257853904, 0.18573272529255846, 0.18478530552986655, 0.18832637836234886, 0.19587303628433925, 0.19217738936511253, 0.18220076166070273, 0.19047132008097045, 0.18319027008778205, 0.1924829192733379, 0.18077890227170365, 0.20256035388165633, 0.18474032265714097, 0.19135294808872713, 0.1885745456927284, 0.17800634458240353, 0.19613249204334923, 0.18926532589659323, 0.1839750931804821, 0.19565548161508098, 0.18180420303756195, 0.1915257323878037, 0.18150857056108546, 0.1838312800185906, 0.19525499409982688, 0.1931251409013638, 0.18888576062392357, 0.18951455742133202, 0.18559463521464908, 0.1914989166534891, 0.20021392265523208, 0.1892741667098667, 0.1939326642807237, 0.19312555715497687, 0.1904007458007584, 0.18478057303418494, 0.1870112644176258, 0.18686298636687382, 0.18259226432929654, 0.19201428048336597, 0.1898785595888777, 0.1861633876746663, 0.19011571472256872, 0.19300796062822242, 0.19872421038128313, 0.18493414420144139, 0.19073891231954362, 0.19457809813438695, 0.19238861469501498, 0.17296626807825474, 0.19158069438311803, 0.1841340782360629, 0.19111170891783855, 0.17789230833120168, 0.20069976631436307, 0.1794900899787965, 0.2010407858095013, 0.17284711556577897, 0.18841332479201803, 0.1840185792163721, 0.1788982212307397, 0.18014166527364953, 0.20805148370610832, 0.1876306431964342, 0.17914358204589223, 0.1965600449748564, 0.19248022591141678, 0.18265678887298364, 0.18893391916893762, 0.1875256279737506, 0.18675902374515554, 0.19465067127386756, 0.18439061291655653, 0.18338779377403283, 0.18699676609768565, 0.1857272933335654, 0.1846765829291968, 0.19916452645231453, 0.18928520058597778, 0.18729149673707346, 0.1913527553069204, 0.1953062352003449, 0.19115469225899076, 0.19204644090714287, 0.1778335211067906, 0.18058841456439303, 0.18797041944760126, 0.18177717022549988, 0.1947462116530922, 0.19476905694795738, 0.18435102322485206, 0.19036745753129214, 0.19268883854559204, 0.18860943506020775, 0.18023976613215098, 0.1934452047191824, 0.19330757162785753, 0.1938264206754676, 0.18451397714523782, 0.193513101829241, 0.1936672218878418, 0.1696340406332588, 0.19100733543876366, 0.19016326140553902, 0.1864830522508807, 0.1947542961415785, 0.18718095923998043, 0.19132873289371286, 0.1923147485375163, 0.18987122007221954, 0.17725342392547877, 0.1904863934016855, 0.195345605777402, 0.1848504415427325, 0.18640984514294973, 0.17944220002051295, 0.18398005784665658, 0.19110486358951215, 0.19364661106754724, 0.19067030888534034, 0.1812961927578259, 0.19445142722570774, 0.17956826357581598, 0.18833769782737125, 0.18805607822872877, 0.18792918800651706, 0.1904511157115318, 0.1956324236650005, 0.19091872385040873, 0.19244187716654695, 0.19381163328223666, 0.17976228925174564, 0.19586551635956276, 0.17950828828479212, 0.19500050549463027, 0.19028750826891613, 0.18205803627928324, 0.1875275039210987, 0.18252566140961982, 0.17920038971860142, 0.19423657783792336, 0.1830347800542477, 0.19125934716308884, 0.18484629790757492, 0.18916886100406818, 0.19202236291728358, 0.1835819742276975, 0.17369999174711223, 0.17366086933727792, 0.19111344176246975, 0.1818718271947669, 0.16845384727515567, 0.174722106544647, 0.18857260485832383, 0.1863873156820244, 0.187379641466538, 0.18936560557670917, 0.18050965316844353, 0.17958926913626794, 0.1956903723215514, 0.1846086402728807, 0.19639502993681926, 0.1865952269118071, 0.19423668134386113, 0.17148978882010493, 0.19517917431795184, 0.19459755066940929, 0.18045307917286726, 0.1930874984728144, 0.18830951980047267, 0.19083056139943821, 0.18334634991797516, 0.19495676362945422, 0.18225239529998685, 0.1989858550032658, 0.20053613815757845, 0.19044793424378514, 0.18029001719054163, 0.18359340107908856, 0.19389859837993706, 0.18732985212739967, 0.1851071062123743, 0.19049910561078298, 0.17897367499901531, 0.18371757250905407, 0.18342797576142933, 0.20532891365965641, 0.19669608174198674, 0.18622170990254644, 0.17832861560337146, 0.19467466040430062, 0.17142449644161228, 0.18460058351672073, 0.17933306032639104, 0.18454600975877883, 0.18382934180248542, 0.1818964247789114, 0.19901915020420363, 0.18215961246403875, 0.19659965640548438, 0.20128152121403736, 0.18865724916628848, 0.19593896434689687, 0.19841873238874957, 0.18380380236077737, 0.1971808384079869, 0.19823992628450207, 0.1693322248697831, 0.19600922853343175, 0.18645862458747925, 0.18953473487472558, 0.19267328034425943, 0.18115722212765545, 0.18968869160028143, 0.18473818245225565, 0.18829861894772915, 0.18906721945932212, 0.17982366302756198, 0.194627651441691, 0.18545424680949377, 0.19634936421508362, 0.18639176514599498, 0.18680652185557506, 0.17941078098668195, 0.18931464572296916, 0.19002764610123776, 0.18565507987754154, 0.189756979983037, 0.18658070533614637, 0.18760748343202055, 0.18403563469701, 0.18568530440651246, 0.19198108454611673, 0.18442494114649272, 0.18600738791280977, 0.18764771663785304, 0.1810956506766487, 0.18928801724497635, 0.1913282337537507, 0.18967923183033056, 0.19415488590817845, 0.19110957814977556, 0.18296887209900947, 0.16730796604462359, 0.19152812523096013, 0.18073742957526667, 0.1882525936064541, 0.19482841564819453, 0.17919884485437798, 0.18399390988310674, 0.1842689778923861, 0.19775369329702416, 0.19297593773491623, 0.18995648883965738, 0.1677839486035748, 0.19367721860054557, 0.17913008845175196, 0.18653064216286297, 0.1889774765956558, 0.1993705604084364, 0.18449106847146274, 0.19741251141575333, 0.18951964119262538, 0.20414039658573435, 0.1930405564636696, 0.18544139365451548, 0.1946783348174875, 0.18064710099779688, 0.1863719696314395, 0.18670706913125756, 0.18025906514793807, 0.18193262090901893, 0.18425279731432667, 0.187466744081994, 0.1894868559636311, 0.1834576965344129, 0.18668043430997472, 0.18522925292110526, 0.18623787099057124, 0.1950854550316281, 0.17774138039268902, 0.18842474843726048, 0.19200614566070753, 0.18348265697470556, 0.18042498382636996, 0.1927090195578821, 0.1883954983101399, 0.19305600667289358, 0.18511195825824647, 0.1886528928036196, 0.177557361259675, 0.19411709525808546, 0.19427886459716995, 0.18904795680700268, 0.18489306990052085, 0.1952650204034966, 0.1857818676357553, 0.17855907131332668, 0.18411868484466634, 0.18369628425419016, 0.19797113943931927, 0.1777402420600002, 0.19699966687509723, 0.17875139252649414, 0.18074249786674268, 0.19204776268216645, 0.16412561352837354, 0.19515533051229597, 0.17956317452626616, 0.176662252676498, 0.19579752820004698, 0.18449004572665945, 0.1869314623400741, 0.19967168152160267, 0.18336864903251385, 0.1968823946793151, 0.18202866670874868, 0.19184207796300715, 0.19144626538127046, 0.18338771021930547, 0.17338649290370592, 0.18199292503548659, 0.18708621920385776, 0.18129866527797178, 0.18974998026131515, 0.17900278075235385, 0.1830624112758886, 0.19021375350132294, 0.193692947067639, 0.18805058993003554, 0.17004663740262274, 0.19007158292284826, 0.19405992720035994, 0.18673755629312197, 0.19397594957435493, 0.1802483673039477, 0.18025699181548932, 0.1830728985490866, 0.17248292431200932, 0.1964390736986255, 0.17902302324089958, 0.1849501519607119, 0.18751494432283755, 0.18822759579765713, 0.19001686118784322, 0.19722024069821276, 0.19625124933112278, 0.18602110608563177, 0.18239477162545728, 0.1857580091667585, 0.19026882448037447, 0.19580641093959625, 0.19585786610234704, 0.19014902548335696, 0.18113024103020506, 0.18551431787346073, 0.18819346252337343, 0.1774922300903401, 0.19351215457476925, 0.18122600773281441, 0.19118035734984834, 0.1845157452157176, 0.18758603288697873, 0.17784230792936445, 0.18365213948707138, 0.18287926870743554, 0.1956403690473091, 0.18830759096392252, 0.18927145406318643, 0.1869190079018779, 0.1896453787517855, 0.18991914780950292, 0.18305708890553224, 0.197773888898081, 0.19583883092787194, 0.18245677238421268, 0.1975497022477678, 0.16911374429497783, 0.18400454384873302, 0.19692948280865585, 0.18084939858245133, 0.17412027443093742, 0.19124956845208185, 0.19482965313598122, 0.18156580738118652, 0.18617171344840158, 0.1784707207287549, 0.17798784420089395, 0.19277060720863395, 0.18780744909017005, 0.18485333054052294, 0.18812075534262943, 0.1959046661083194, 0.17988191383685698, 0.1903411308217947, 0.18612078151463746, 0.19896772079400363, 0.1948980871524736, 0.18483036512142312, 0.20019663325247836, 0.19405166784466962, 0.18070199484721655, 0.19041486811728792, 0.17967735872098778, 0.18593035682034617, 0.17933138668091936, 0.19502759697256916, 0.17750144377203492, 0.1844546482583808, 0.18226060986049872, 0.18318519912092607, 0.1898850165128661, 0.17835046174107017, 0.19646560324786874, 0.18983386810280953, 0.1897075742222166, 0.19554030783767176, 0.18598744953717136, 0.18141132701891624, 0.1895662530258521, 0.18480139590440775, 0.18530457815470253, 0.17343706413731189, 0.17794981387290057, 0.1839089257683713, 0.18691303063545942, 0.19141571295799784, 0.1829287637577185, 0.1790138464805569, 0.18355526737382186, 0.1850623934494685, 0.18740736139052816, 0.18308856176184796, 0.18974774056661559, 0.19185629579507302, 0.19738444525064033, 0.18641588686216518, 0.1972527670135409, 0.1863831852417929, 0.18770938913397867, 0.1845668319707753, 0.1764642754307228, 0.18283241309473544, 0.1753753445916548, 0.20014811012839157, 0.19032881861134585, 0.19290954477810782, 0.17435101543493564, 0.17965274682006965, 0.17881911348113064, 0.18579791842052237, 0.18964322826500954, 0.18651963832254168, 0.18137888385922676, 0.1926338042565999, 0.19202129331430606, 0.1844051337703562, 0.18371023608344386, 0.1955591307191702, 0.1969659568204103, 0.19590842333631261, 0.17522441036876926, 0.18693281453811347, 0.1904247944726239, 0.18241082297024006, 0.18740815770745317, 0.1969998696194894, 0.17653055391865072, 0.18595029268716176, 0.18446974128285945, 0.18530107434860152, 0.19398925163288838, 0.18317627394698777, 0.18407505617910233, 0.18414998145436062, 0.19909886815706646, 0.1950374981397677, 0.17751879175921328, 0.19079526706491803, 0.20071206177005632, 0.19023122659307723, 0.18568641141156392, 0.18415813727218824, 0.1938817770441428, 0.18625584251768387, 0.20376171739871307, 0.1933277789982258, 0.18628459498715902, 0.1852547735936736, 0.19333353548790544, 0.18652205398036062, 0.1911221400592048, 0.191569858313335, 0.1747476750284579, 0.1926594464880291, 0.19418433374589944, 0.18044781563411308, 0.19254622325275417, 0.1975103672731679, 0.19278059433029088, 0.18703250090432408, 0.18318176807435307, 0.18670216800903378, 0.18741681149103664, 0.18792608372157638, 0.1866216281590722, 0.18721857537671677, 0.18823705277950556, 0.19134357985529596, 0.18982055831283223, 0.19584335709495077, 0.1893104845103314, 0.20004177435867143, 0.19032765983050653, 0.19181468221643494, 0.2047603971485542, 0.18715908540913068, 0.17706553618469786, 0.1968217513463045, 0.19303245962525906, 0.1850452081515264, 0.1807530837550558, 0.19029508274198473, 0.187497592205006, 0.184153184820547, 0.19104404474353945, 0.20728937595042074, 0.18772154019469842, 0.1683037506150388, 0.19404130208708883, 0.1981819815762003, 0.19672832296582324, 0.18944596260296748, 0.1940567481254359, 0.17903674773145084, 0.18928183676803256, 0.17649488594347956, 0.18581663795116984, 0.18543603077239307, 0.18460682401236087, 0.19391251493028577, 0.1817879959440647, 0.18672820311369082, 0.183629027648821, 0.18879178204948577, 0.18310679956739115, 0.18864372483069725, 0.18801801301945925, 0.1757601569338588, 0.19546816381754767, 0.20027092331164353, 0.18050667976334683, 0.19486011122315836, 0.18709512860469996, 0.19143308247869525, 0.1866640231566739, 0.18608899076121488, 0.19034792523217958, 0.19580000909246914, 0.1940742881331159, 0.19382783675902238, 0.17710417875265932, 0.18608576368314358, 0.18809654311213256, 0.1887496699146338, 0.18202482379834622, 0.18548348969112324, 0.19060706360198232, 0.19657406077278936, 0.1816591075277175, 0.18855522683664097, 0.18821261973423237, 0.19094875594051838, 0.18513421085744436, 0.19733660909551282, 0.18216144935358838, 0.19361824210098874, 0.1842162951140983, 0.17708774079484077, 0.19844121183958893, 0.18318474344164762, 0.1751610005544645, 0.18048955000209804, 0.18576596196672202, 0.19512927271141697, 0.17119960855364333, 0.19037396165907933, 0.2031763665885883, 0.1883510215958141, 0.19609098900187616, 0.1790685135493672, 0.19436635197829943, 0.19817126500339388, 0.18216685790883475, 0.18373369833791936, 0.17537963871943674, 0.18895728478134838, 0.19179048699286375, 0.17828574831695443, 0.1810332561548427, 0.18801244540899556, 0.18632742333271826, 0.19083777098497706, 0.1861803058807754, 0.18552064364938767, 0.18099839185191238, 0.19041564049885462, 0.1874758460714361, 0.17862165742553796, 0.1900753702954067, 0.1791260297197332, 0.18034033666430727, 0.17879983118458304, 0.18931165048184387, 0.193320346828275, 0.19994765757623342, 0.18965180238127535, 0.18609617558016298, 0.195180069193372, 0.1869299436592792, 0.19449234755857311, 0.1820916591340208, 0.18306194699711315, 0.18910984263040093, 0.17988996362548812, 0.1952690898434665, 0.18219817299037866, 0.180757258632461, 0.19674828486103782, 0.18854682614891513, 0.1810494923452806, 0.1848660468334844, 0.19044504747425448, 0.18614777826032283, 0.19892821582547704, 0.19126109935292937, 0.19643725036373685, 0.20402576166761008, 0.18863294746745182, 0.19090619629189762, 0.17690381567761962, 0.18714899201549287, 0.18466142125768553, 0.19927962375401556, 0.18259032163836417, 0.18961676773803526, 0.19073876566636144, 0.18603554769372033, 0.18132881248857893, 0.18309333226447608, 0.1837917105000075, 0.1879620766623643, 0.20188370902515554, 0.18148581895754, 0.18956604380851813, 0.1900590425494551, 0.18763810056021563, 0.16733609768311547, 0.17782688811169126, 0.18479159758773195, 0.18619480651990772, 0.19278421298884532, 0.18190862620134404, 0.18311058263320304, 0.19132795452855889, 0.1719670995122606, 0.1806394446302603, 0.18133428979817415, 0.18888974015383345, 0.18770163244595126, 0.18354730790020277, 0.19377303674385807, 0.19063758046093157, 0.1922437034599768, 0.19217857904072186, 0.19516147239822026, 0.1965097355945268, 0.20197745223938535, 0.20342157123264315, 0.1926180177361106, 0.193985383924179, 0.1974608591125152, 0.1914326814300522, 0.1896169077741984, 0.19399306886340176, 0.1879950421702302, 0.17774484008211555, 0.1940318748214734, 0.1959611209651418, 0.18144286789230193, 0.20441121996776732, 0.19108250872280175, 0.19282982448451794, 0.18594933561042906, 0.1982567618487427, 0.1795649361429346, 0.18879493395143349, 0.183296913869241, 0.19298040705161074, 0.18532883015725882, 0.18292469337217143, 0.18667772650484754, 0.19282225029249672, 0.1866949268717375, 0.19565923645034666, 0.1812448259982484, 0.18457999601869304, 0.182606597187995, 0.19883967048028459, 0.1857466624112269, 0.19870020244075726, 0.18579441805567806, 0.19808061935208707, 0.1727635420058561, 0.18640177284361406, 0.18655512962785642, 0.19287473168713226, 0.19107929710522423, 0.18574834453752348, 0.1874029723174032, 0.18312187022605028, 0.19455839347932333, 0.19068524555884855, 0.18556187841319624, 0.1860688964805167, 0.18434116983661702, 0.18367810831647122, 0.18684681143414605, 0.1977023755260215, 0.1783744513305635, 0.19639089381972363, 0.19674550416263098, 0.1855276691761479, 0.18584763659085657, 0.19398434365383446, 0.19671861934861387, 0.18895483605857272, 0.1964791241714962, 0.19040002561811345, 0.198172891985501, 0.1951013707443811, 0.18249630909006803, 0.1864949580197132, 0.1935378319879579, 0.18422648513919887, 0.19430165937911972, 0.19301376833733008, 0.18345135587421632, 0.19147637301687703, 0.19760623276612668, 0.18855819113219427, 0.1839600475179466, 0.18374868337095374, 0.1879181752546189, 0.19471411461820512, 0.1828808632665803, 0.18721656602489042, 0.18171908612073773, 0.1755686078572665, 0.1852647840865558, 0.18714607473826878, 0.1926623688357448, 0.18756530792316575, 0.19430402017927056, 0.1769299893035779, 0.19162509841058611, 0.19583213362888902, 0.1912356551157598, 0.18700636241989887, 0.18498320772639615, 0.18538799539409923, 0.18669948450016632, 0.18017366704003238, 0.1883499162905324, 0.19440436952573675, 0.1963100305483429, 0.1878183211074497, 0.18400694557249478, 0.18714526530321768, 0.1881165073400615, 0.1910164116365003, 0.19495801955318698, 0.18197823378238778, 0.18054626675487187, 0.1938175725935412, 0.19320299689845258, 0.19936149453758895, 0.1891532359886634, 0.18757990640344646, 0.19403524612117332, 0.19948994786937074, 0.1872240611630651, 0.1904295342689229, 0.19175233344523263, 0.19885446744016638, 0.17814036120227447, 0.19150629102765887, 0.18949694747193338, 0.17843878942473404, 0.18984976127536293, 0.1844365947914909, 0.1860221870113553, 0.18863606403552324, 0.19798022136878726, 0.19514101728943664, 0.17926767437873894, 0.18949173414780968, 0.19581122833189127, 0.18940598646834167, 0.1888565566814704, 0.19896010895548016, 0.1862278504656665, 0.190844644691557, 0.17883089664202034, 0.18121205413332273, 0.18335905516914092, 0.19882082027023237, 0.1919574169000251, 0.1897787709459774, 0.19153866760190513, 0.17739623364146537, 0.19056698186718182, 0.18428437275310225, 0.1906553160844006, 0.1831752897490608, 0.18974733906916408, 0.1767764126767215, 0.18552098682679757, 0.19643206881218492, 0.18139426430685904, 0.19423057835236596, 0.18995987754537041, 0.18070029695379342, 0.18795235241056446, 0.20432743581951843, 0.18529554966373915, 0.1894157534236678, 0.19725476286993313, 0.1917593740123112, 0.1916559429208474, 0.1906136102144, 0.19198707522255481, 0.1850600767466066, 0.19507732904981626, 0.18803264401156608, 0.18752637772910358, 0.18972536212390173, 0.18910797748187677, 0.18831631818200736, 0.18820673154489098, 0.1923202221480005, 0.17429333724739346, 0.19844972282225887, 0.18297831352564245, 0.19092495069835116, 0.18768306720296174, 0.18392413199767993, 0.18768385634595192, 0.18859969185057757, 0.19478669974938903, 0.18833689879202214, 0.19513321552278948, 0.18397760337672292, 0.19197727924026048, 0.1871015093877069, 0.19778018005761072, 0.18933899022481426, 0.18656805070224458, 0.19429323395600473, 0.19118012320091246, 0.1910446840283235, 0.19655113714396313, 0.1865029253910952, 0.18652974514929058, 0.19545246357970264, 0.18797620512330845, 0.19600246965398752, 0.1823689737957268, 0.19050813200634412, 0.1926736382066558, 0.18595892049732807, 0.18706633203071885, 0.1938028497085131, 0.18773038054352206, 0.18680751590230135, 0.17979922763377107, 0.18097566690671443, 0.18677070243728158, 0.19453106170759804, 0.20071787438377936, 0.18775199428961536, 0.18635199273311548, 0.19077264165139024, 0.18913107050591976, 0.18690771207649526, 0.1821260217764036, 0.17865835581153183, 0.17419236632926394, 0.1876305101073824, 0.1841074759352134, 0.18180941706918297, 0.18208296799687218, 0.19332728436909447, 0.1793621530482803, 0.19371175637513474, 0.19745024197724897, 0.18850079472569747, 0.18549291589423353, 0.18562043892323035, 0.20181532124748972, 0.18066837154970894, 0.18932270321478145, 0.18689583636500956, 0.1887331892342495, 0.1863610060373177, 0.19517963533529703, 0.1754478676556595, 0.19191023156128612, 0.19243557549594642, 0.1891710208244159, 0.18855666960744707, 0.18463873654324486, 0.19209557917682452, 0.19891179442522972, 0.18420679186782024, 0.18335979967160138, 0.19361853586213063, 0.18672176192033582, 0.18647668722932476, 0.19270699704139008, 0.1958658284323875, 0.19089343975034373, 0.17792420229986747, 0.1932487054043948, 0.17867584676072223, 0.1858991908712524, 0.19535835551734954, 0.18431813811974035, 0.19083173693966068, 0.17683264183621306, 0.18630220400873623, 0.18545265231972058, 0.18035741755025886, 0.16767900304856703, 0.1924219880927001, 0.198864924908489, 0.19383338960577323, 0.18586199016733487, 0.19974270452459822, 0.19080415280928056, 0.2057417911503464, 0.18597128082864295, 0.18666803715639876, 0.20409429008437083, 0.18186614144848579, 0.17427642573731422, 0.181610837191221, 0.19904294289913818, 0.18260609098511452, 0.18807425372604603, 0.20577823332509373, 0.17909827028316502, 0.19494139245699216, 0.18254510805758173, 0.19597156795222426, 0.18028769663078525, 0.19172119931759513, 0.1766869974459697, 0.18904643633033677, 0.191988109478486, 0.173034805348061, 0.19036145045505634, 0.18943841698602554, 0.1807124644096475, 0.19666928589651977, 0.17158952658251547, 0.1948617485087828, 0.19250252925464922, 0.18907963126356886, 0.18887251138228528, 0.19542115990187617, 0.1693173505297268, 0.1761285751420091, 0.18598955318094154, 0.19032885502071936, 0.19166820982250904, 0.19091518685314776, 0.17238613949677054, 0.18792517510857423, 0.1910064737616559, 0.18236814597729684, 0.18143473242498243, 0.17313316648251925, 0.19180141750869584, 0.18753049682638243, 0.1760830272662163, 0.1753341113034452, 0.19565382336787512, 0.1851564917314105, 0.18809684177285954, 0.17688748525058193, 0.19662783646123516, 0.19656477148312237, 0.1934060230198817, 0.18669821292945782, 0.19157969768537758, 0.19359849529507492, 0.19206436761375315, 0.19356203017525425, 0.18774766817780916, 0.19023486804657924, 0.18328991018391907, 0.19482298070786158, 0.19652321975276288, 0.2030027818915749, 0.18459961697363153, 0.18512076650666814, 0.19397515366375243, 0.18898038022827032, 0.17526759976475145, 0.18843450720479107, 0.18972253581294798, 0.1875033631611206, 0.18853793323959944, 0.17863579128727325, 0.18481055684789963, 0.18924802279288502, 0.1835384227108958, 0.1877884547239595, 0.1839586075270546, 0.17951980335290255, 0.19052813209198907, 0.18266763245354312, 0.18511441177476418, 0.18456920576636163, 0.19240707041690608, 0.19589716573759985, 0.18081420292314562, 0.18133309199800815, 0.18039144430931944, 0.1761246702274745, 0.2004264168302206, 0.18972123659395693, 0.17929820422542883, 0.18260131968644427, 0.189290771380206, 0.18175775501104585, 0.19698715070854705, 0.18902927224811425, 0.19056239012604584, 0.19895927742549654, 0.194112685941672, 0.18715777937098077, 0.18084162183278382, 0.1883194344442495, 0.18767801444292073, 0.18740316259729808, 0.18795488538949703, 0.20084667976087314, 0.1870586843006115, 0.18076999246647474, 0.18238392870289116, 0.1937029704249662, 0.19517242327972528, 0.17819620852733511, 0.18382883890016372, 0.18063028217477586, 0.18731911627051592, 0.18815133176830864, 0.1811099826272464, 0.17704966818797113, 0.19811058887193064, 0.18200235285769967, 0.18727645812004032, 0.20043743323040752, 0.19874704098831225, 0.19774442599532188, 0.18894376087022519, 0.18452125531332045, 0.18567052045209156, 0.19140202475750473, 0.18412641990017561, 0.18157176372400513, 0.17966401636203475, 0.19161637024720904, 0.17887859880707827, 0.18322291198975763, 0.18082223706331246, 0.1746049199683437, 0.18005887969855688, 0.17940214150803144, 0.2002987397838196, 0.1902029159495089, 0.18725940232091867, 0.1809270893126559, 0.18382756509275128, 0.19758819942541833, 0.19110442564527144, 0.1928363373959712, 0.18528552176785865, 0.18475943304256456, 0.17810884714572145, 0.19252550227854065, 0.19165488798623154, 0.17812702760774937, 0.1839552780633005, 0.1847306559406542, 0.17094144319694377, 0.18280200972493627, 0.1776067203875146, 0.18586887789846618, 0.19139416454378377, 0.18761701505692452, 0.1921002298034351, 0.1823930653235113, 0.2005747713852798, 0.18651517294750844, 0.1744973382213647, 0.20039030575843034, 0.18248867238428726, 0.19894019713595862, 0.1882993757428589, 0.18266474133928606, 0.18685186432296405, 0.19366852422410288, 0.1741110193462836, 0.19447116986478394, 0.18840007835679323, 0.1837926688287242, 0.1897057986521103, 0.18117262501053716, 0.18274633560487763, 0.18865940703133072, 0.20497536307050215, 0.19169658293339903, 0.18608454417976286, 0.185797155288119, 0.18085783439471062, 0.1889221655851587, 0.18358103774033324, 0.1887492922605002, 0.1832292983410297, 0.18619344032283344, 0.1906777671225341, 0.187985173399008, 0.17903852804897966, 0.181636500026269, 0.19204777029492337, 0.18134853198848228, 0.192036517391654, 0.18434346302083465, 0.19250948765388107, 0.18908886225643512, 0.1856454610230662, 0.1729556272915803, 0.18816579924559682, 0.1875616974364128, 0.19832480653030496, 0.18832490253026132, 0.18926378094763058, 0.18992195712190327, 0.18809193863892676, 0.1952793543853661, 0.19457967197757933, 0.18254182408746, 0.19312458880041303, 0.18460099009552713, 0.18523090739832576, 0.18531362165674686, 0.18662302726906224, 0.1899204704508033, 0.1877714565587712, 0.18849642104825368, 0.18402060639036816, 0.18959310681597544, 0.175564034204087, 0.18664798991822115, 0.1884119687682107, 0.1867595649349222, 0.18811954378998175, 0.1938560166079192, 0.19301118046993873, 0.17888349530187783, 0.1765354588793569, 0.19702222331747565, 0.18664712245128462, 0.18446615424039384, 0.1880316057648926, 0.18892215006016974, 0.18941681173943364, 0.1962552398003771, 0.19148146874698188, 0.17996387826577967, 0.18762522782660618, 0.18716417839522811, 0.1864109764025671, 0.17492470631568494, 0.19334116364126813, 0.18818153574312288, 0.1992812562015619, 0.1762131578778022, 0.18679944876573204, 0.16791126385017843, 0.18006109867765793, 0.1832989825798109, 0.1916849816413975, 0.19168799930130637, 0.19503554782864876, 0.18917291343315248, 0.1890113355729547, 0.18846789863841776, 0.18431160152156242, 0.19363742905006126, 0.17831372350101107, 0.19166769956244156, 0.19324788907810414, 0.19558569590659397, 0.187805985618329, 0.18817291562743135, 0.19327758244083093, 0.17873279507887974, 0.17462557937619327, 0.1896311870231816, 0.1841803556058142, 0.18505926926584632, 0.18685007711344956, 0.2022790977475153, 0.18554729936998762, 0.1809734915567129, 0.19347762804390364, 0.19733136389897835, 0.17982779061865173, 0.18059480834788172, 0.17960675571396625, 0.19406493955217652, 0.18999945334328724, 0.19407065801614204, 0.18630536450195714, 0.18401943403649576, 0.19896833718985582, 0.18952102570912743, 0.17706852817211013, 0.18777174675945904, 0.1935508829989476, 0.17868714491440754, 0.19555233488410556, 0.19292453974363055, 0.17801248281277704, 0.17999651742431613, 0.1876834595319789, 0.1862856060837786, 0.19423809626690822, 0.1923399139847329, 0.1868329155651739, 0.18186569930925234, 0.1792503649718925, 0.1803253197856068, 0.19392867161279634, 0.19702041247592206, 0.1846781518261523, 0.18297263267502503, 0.19889033582816526, 0.19124245244261556, 0.18785844720944805, 0.19297212520352394, 0.19646989883474486, 0.18845624165650907, 0.19895879266822292, 0.18533318019119688, 0.18176895970171095, 0.183133524651437, 0.17898950620190418, 0.17560439529607594, 0.18673103751510825, 0.1892852466659803, 0.1848120960205781, 0.19251601717715158, 0.18917036552515956, 0.19472526076835278, 0.18540304762043297, 0.19156653757597775, 0.18802855368871021, 0.1914873429446506, 0.17707973779535477, 0.18211918778609687, 0.18673737715284322, 0.1883524516877953, 0.19004386458403508, 0.19496931834050774, 0.1945227356637944, 0.18480746879505985, 0.2002083499139347, 0.18274057258808263, 0.20137161628861816, 0.18790847279710443, 0.1918436088680538, 0.20021787599898458, 0.1796451732425379, 0.18655758597148717, 0.1932730128386185, 0.18119867005809182, 0.18106895433005657, 0.17487575786427736, 0.17635273256658324, 0.19211029688279824, 0.19352599240363177, 0.19586107449770085, 0.1904389844146912, 0.18720847041039262, 0.1934420193712061, 0.18189823228588103, 0.1984090327882984, 0.18483686756827958, 0.18088396833738743, 0.1934572898436707, 0.18836920900551068, 0.1826028571268228, 0.18583151038598905, 0.17348719110202993, 0.1804184226564615, 0.1892456881199765, 0.18590897581257204, 0.19055184732758348, 0.18623770376648605, 0.18714821676673357, 0.18709957749538442, 0.19727564265581565, 0.1937650539820185, 0.19175545151221443, 0.17929139104267222, 0.1878348135463746, 0.18748443706011014, 0.19115335626364596, 0.1908583398245425, 0.19005385268468106, 0.2047564204351268, 0.17653968423666355, 0.187769105137138, 0.19590156689151236, 0.1916640046172272, 0.18825287799877594, 0.1850791745798018, 0.17829874417351047, 0.18266109600172822, 0.1854659998811057, 0.18734057318696581, 0.18849831389696525, 0.17738919413827275, 0.1845142816803782, 0.1952192776961055, 0.1920141730158848, 0.1858633478340173, 0.18996191732553483, 0.19010371293873618, 0.20010345740937224, 0.19045277195909815, 0.1786661976566647, 0.17995929341769326, 0.18756869809297436, 0.18919699719178876, 0.18382431198915827, 0.17667666867437565, 0.18389486214541073, 0.18430548494020704, 0.17768541148138456, 0.1882052841074529, 0.1866862466328426, 0.18587554723022268, 0.1953439832005724, 0.2024188776832814, 0.18987527773402918, 0.18265740777354192, 0.1831328139610901, 0.19122108951292957, 0.18793724881555096, 0.1818163337391599, 0.19041859210231846, 0.1885758827788874, 0.17551964604620934, 0.19230876623544574, 0.19218486407169277, 0.19020381083118898, 0.17771783459161936, 0.18537379590469055, 0.19439845501430908, 0.18905091026385715, 0.18408913912583763, 0.18866217259537685, 0.18287985790513459, 0.17272960046706728, 0.1871525218790185, 0.17375398226531166, 0.18468694742445477, 0.19730511604639897, 0.18674742964917831, 0.18184035950591632, 0.18522481837602936, 0.19098284045071637, 0.1898181613692723, 0.17803332571849517, 0.17869104101534075, 0.1783039067698732, 0.19581120866366877, 0.186698258624417, 0.1853396591826741, 0.18090659252649327, 0.17764967967215745, 0.1819207711711596, 0.18601938823253858, 0.17919990993116727, 0.19135801635017166, 0.19176362438292077, 0.1817109117195805, 0.1874399577947304, 0.19035866190239484, 0.1894319271273754, 0.18338650771617965, 0.18956585675001736, 0.19214117867472089, 0.1984833196924151, 0.20082509063264944, 0.18239743724867818, 0.18183975707584776, 0.19674429656568335, 0.18576625424306395, 0.18125847000963485, 0.18550002010321692, 0.189309284016041, 0.18532976403592, 0.16344296875685915, 0.17806678758167502, 0.17991208517516694, 0.1876181090061649, 0.18196576468199338, 0.19235444213814046, 0.18200567580462496, 0.17401832624398877, 0.1994079571973461, 0.1899201859177516, 0.18509369086673325, 0.18849727208526873, 0.18990979374873887, 0.1878660228082865, 0.20181865316694966, 0.18658300546061427, 0.1964088149684247, 0.1852894741368483, 0.19475004038025356, 0.19115812854215114, 0.19060240420422234, 0.1834110240404262, 0.18189443491225057, 0.18660862618839508, 0.19636478040459585, 0.19331684098268154, 0.1971842634170444, 0.18705779827010316, 0.19538298679580848, 0.18497075298781934, 0.18686679841958645, 0.2000285744452749, 0.1849366645965869, 0.1846616286242602, 0.18013004103553729, 0.18152890069839872, 0.18755114338384182, 0.18990706189835221, 0.1953420922267699, 0.18643584064688684, 0.18006100591524465, 0.18284468575695279, 0.19076318258686767, 0.1958153297200128, 0.18751705748967099, 0.18881317919934199, 0.19141183352937302, 0.18045173285428764, 0.18231137495428348, 0.18607012100656564, 0.189572685747152, 0.19117323480298795, 0.19745847268608016, 0.19055832501412717, 0.18783364302269362, 0.18494685558595744, 0.17905093194077307, 0.19073804578254572, 0.19027841541697757, 0.19059901709677152, 0.188104143529151, 0.1709650777680409, 0.17926612441576212, 0.1922729775771479, 0.19996063713681078, 0.19131640694815538, 0.18909250322889543, 0.1938105195295313, 0.18304276829801763, 0.18263483463626798, 0.19548451308041268, 0.18653953736228018, 0.1882995784250097, 0.18586161753810773, 0.18950073011022844, 0.18793794035281974, 0.18332771890464863, 0.189568945110182, 0.17817283743499956, 0.18354649060568012, 0.189396991744675, 0.18768958645375813, 0.18177742476378753, 0.17531546075754678, 0.19778096191957842, 0.18655066546283072, 0.17819469825338252, 0.18659784212459798, 0.18933399038016696, 0.19304301654534417, 0.1825613729144153, 0.19328861047823134, 0.19055086698021695, 0.1890886412400324, 0.18220786457673266, 0.1921565103464285, 0.18835555767840872, 0.19212990462286839, 0.193101463542882, 0.19486942358896917, 0.18587260242527073, 0.1846618658504865, 0.1973109508046243, 0.1938288133844644, 0.1911645016612526, 0.20062631651879206, 0.17703836334413578, 0.18930218906151644, 0.18932806863085733, 0.1902752773713102, 0.20474921414620503, 0.1853385524267634, 0.18443630073329, 0.18995562253134562, 0.18549308156422736, 0.17884752921736444, 0.1918231688046736, 0.1845959000678632, 0.1961181961977202, 0.18235612766197973, 0.18977908143376854, 0.1829306355374759, 0.18221527917888558, 0.19714029245067602, 0.19032960669429186, 0.18917327675259318, 0.18778674076172125, 0.17954020452098787, 0.17836394270099243, 0.18797593103617144, 0.1794673011505149, 0.18747876473139638, 0.1858812939046065, 0.18630106867087767, 0.18784148671204245, 0.19957770669988514, 0.17962863556001957, 0.19051445643334705, 0.18905194996069832, 0.1939833253556796, 0.1857122593104282, 0.1804200435397026, 0.18685526713301495, 0.17587471698951865, 0.18075579714944826, 0.17455721675620675, 0.1834857713865305, 0.20327404918535377, 0.18914438301938968, 0.18777615070443562, 0.18165001041181156, 0.18104776364215255, 0.18754754725550277, 0.2028956217314944, 0.1915955958469578, 0.19350570902323794, 0.19735777523515208, 0.186004686160443, 0.1778821001924148, 0.18854767909208997, 0.1784962204564243, 0.19055172006354498, 0.1868900706147879, 0.1808215927672345, 0.1848718757715765, 0.18295164101746023, 0.18630350649701655, 0.17834293609472057, 0.18450197071891566, 0.19313714078956243, 0.1898444974859235, 0.18680606721938223, 0.19241922056125788, 0.18569247806888675, 0.18401796845789725, 0.18365474668791684, 0.19231345431101055, 0.18345412750941037, 0.18270935535401875, 0.19711352202356744, 0.17883311537039095, 0.1868563268310634, 0.185804533816023, 0.18596308962723362, 0.19099448091215335, 0.20394257935481497, 0.19965560915032968, 0.1963295926410435, 0.18417014844176285, 0.17929820904982047, 0.18807339219681804, 0.1729111320809096, 0.18875861672341387, 0.1964367453969279, 0.19004370593922343, 0.19228482032161667, 0.18795937308903857, 0.19707650635952734, 0.1965693404460311, 0.1836617114797611, 0.17384397494183804, 0.1874023388535873, 0.1836750124777091, 0.18940158081409575, 0.19558337736865627, 0.18208057980826756, 0.19172131119469266, 0.19112125405382818, 0.18935872323202174, 0.1903286301262835, 0.19781870459146927, 0.18888404289225494, 0.19880416311222185, 0.18851078051050862, 0.18455425162925393, 0.1985905685051536, 0.183333997216769, 0.1913519149213825, 0.1808067201621517, 0.19375228400024175, 0.18297218570446622, 0.17849450856286428, 0.1790024997700021, 0.18335016017514189, 0.18791871423370768, 0.19055931870378295, 0.18884478614411976, 0.19013346482176552, 0.1862755450128677, 0.17640701118277577, 0.18997798914305392, 0.18739483301834076, 0.1894554410367183, 0.20219800625203493, 0.18976644242808288, 0.189949288887464, 0.19260686809488514, 0.18498878840536137, 0.1933050150471557, 0.18829745580783397, 0.17698478141721938, 0.1707405633224743, 0.18848064270423595, 0.1870392627931298, 0.19136863158552211, 0.19799171330650847, 0.1861651385440659, 0.18588098848051532, 0.18012431330699064, 0.1906877616629729, 0.18966150763541764, 0.19034155797495383, 0.18323525417354822, 0.190557624179647, 0.17797852603201061, 0.1820936878731308, 0.19010452473661402, 0.19283215697654615, 0.18852996617190973, 0.18160162972705443, 0.1942085764041924, 0.19230538404429415, 0.18523768486038716, 0.17730458080600128, 0.19083954440461315, 0.1876879747996312, 0.18470051709196647, 0.1824198948674746, 0.18541822815971054, 0.18638788456722302, 0.18531996731851985, 0.1866125329430845, 0.1804242681927728, 0.1909311221021051, 0.19549347026952468, 0.18455505732234095, 0.18825246806503634, 0.1850822961405714, 0.1857716036269178, 0.19390939070400862, 0.18285993852362945, 0.18553400176767199, 0.19012000557480477, 0.19556991877768273, 0.18660851474897538, 0.186261359015285, 0.18918101935160161, 0.1844224850666407, 0.18531277939533639, 0.18911666009897724, 0.18752833386086315, 0.1914178399713625, 0.1917240157283378, 0.1886301921582876, 0.18143253912062607, 0.18770834693931135, 0.1854135253482745, 0.18329915899147145, 0.19530638428404, 0.18008926666140215, 0.18979504372996395, 0.18848590222664385, 0.18962567524088286, 0.18948519197350738, 0.18902203266884837, 0.19546104399145586, 0.18995694030540514, 0.19170579934489523, 0.1854819027969772, 0.18832991627332266, 0.19152571157872259, 0.18553930116578674, 0.19975277446391101, 0.18896950984509406, 0.18828315026045253, 0.19271276624999556, 0.190051147504802, 0.18353254633141475, 0.18774163284339496, 0.19618277592666267, 0.18850901615709706, 0.18421537950908845, 0.18682843967145704, 0.1973397564487448, 0.19273076311455623, 0.1902670643222876, 0.18976929482103266, 0.17930859921298456, 0.18294395060693014, 0.18174435355373222, 0.19100664550675406, 0.1915949788933772, 0.18014957741418308, 0.1888352675531243, 0.2043690964454845, 0.18996449529554119, 0.186148221054505, 0.19290612038972377, 0.19244875962730326, 0.18475237508940245, 0.1833520383815197, 0.19049371912164662, 0.18789474663190847, 0.18782042138479463, 0.1887572492491436, 0.19393662473346576, 0.17580079194606787, 0.18627974603710842, 0.1892153636627436, 0.179001522470942, 0.19192293871756858, 0.19179152044214406, 0.18266733077213113, 0.1924111156574321, 0.1854449275572704, 0.18285202092180788, 0.1761584843768717, 0.20854479973241938, 0.18318446672695668, 0.18104583115672182, 0.18386418329307616, 0.18927214234526957, 0.18115566713395298, 0.18704068415026076, 0.197300657663091, 0.17928514192021008, 0.20550367136695558, 0.18822250758287395, 0.1972466026031785, 0.17933041353004076, 0.18137199720383737, 0.19371853585614973, 0.18524180420064712, 0.18421172710540798, 0.1993508163372665, 0.19147632618781152, 0.19003581269695038, 0.18408527489858073, 0.17953632838529282, 0.18490160500106573, 0.18577588574646237, 0.18502457714467704, 0.18743473772541772, 0.18611629460452353, 0.19270252721058348, 0.17617079753387338, 0.18923330849766612, 0.18498863907246005, 0.19548052447720346, 0.1824298961390383, 0.20177988659751545, 0.1808966658147021, 0.18261456459596184, 0.18600688975243693, 0.1860935763615111, 0.19708052358617917, 0.17585217823481497, 0.1737507050009779, 0.1843710061854705, 0.1879054327948328, 0.190973203099974, 0.1866039441103985, 0.19607486446171926, 0.18531621516140429, 0.18330112637349916, 0.19809020598534413, 0.18853157794321845, 0.18436916991998553, 0.180655042258958, 0.1873813112669606, 0.18206577838305613, 0.1915421953920417, 0.18631614935011745, 0.19061254979025152, 0.18401103205125277, 0.18149783755897203, 0.18660651919819593, 0.17740977493407178, 0.18880453531723024, 0.18328271388582362, 0.19654383376325557, 0.18394327971933291, 0.19167055430133936, 0.1797448795168401, 0.17992988551796454, 0.18222332789395107, 0.19535845799116525, 0.2002091257317222, 0.19281047142534416, 0.17866363209576358, 0.193317722196871, 0.19808131749588434, 0.184799833295327, 0.19047145719646358, 0.1873371499170675, 0.19046494660539254, 0.19437482113689472, 0.17936966047486705, 0.18450484642172812, 0.19226592090104774, 0.18544907346566383, 0.19947090588657915, 0.1868518675515523, 0.1788644220808904, 0.18413909795924255, 0.1936304683266024, 0.19750429978143105, 0.18286463324109217, 0.1847119548013151, 0.1869666181146805, 0.1828774051991447, 0.18211166925999303, 0.1843037010453311, 0.17682798394223725, 0.19660380569624053, 0.18880360727728102, 0.18960305838507646, 0.178347849830901, 0.1898165111128128, 0.17397156540487096, 0.19661342560024328, 0.18886702987530488, 0.19576139458397895, 0.196849289269863, 0.1810753850073931, 0.17922931477861648, 0.20152055137328193, 0.18963864013208281, 0.176507055848513, 0.18498757236281563, 0.1877350966106927, 0.18176453223477904, 0.1827993019287736, 0.18329616838174134, 0.18554165204830436, 0.17969491773544458, 0.2002706008499001, 0.18069826799118494, 0.19535704501961362, 0.20012442170950173, 0.18061334425432457, 0.19818321309375658, 0.1842699315512409, 0.18459902937160977, 0.19254892349074726, 0.1915307346109314, 0.1743212129859774, 0.1935610992095216, 0.19010085979762187, 0.1825418952762976, 0.1920250260153599, 0.17426732764442976, 0.17981028549449604, 0.18728473747437685, 0.1893397790780618, 0.18012780274490914, 0.18745387961683813, 0.18666553396707772, 0.19068036459491894, 0.18137506905096606, 0.19097868317438782, 0.18744964530067174, 0.1884671641469166, 0.1979753728851023, 0.18964974468112794, 0.189589986573678, 0.1829314689325457, 0.17263207567787972, 0.19146176235523804, 0.184785158119206, 0.1957934090769565, 0.1865326599455355, 0.19377884925394392, 0.19512858382805412, 0.19273338436610968, 0.18667371341162806, 0.18634083531653287, 0.17806459415720075, 0.18584936316801812, 0.1853758183952609, 0.17985715121711932, 0.19647170846669132, 0.20256981203193228, 0.18946386541149618, 0.19566680355159308, 0.19118646502035422, 0.19526624708998233, 0.1847851832523525, 0.18649836282555585, 0.18795474079340424, 0.19380432652687168, 0.18437936412795214, 0.17104633242816775, 0.18663961073668278, 0.18445545521324247, 0.1863451127109199, 0.19982658319158236, 0.18627619874691984, 0.1914886318425353, 0.18783748310036127, 0.19660627786882043, 0.18982312639056081, 0.18725566811308117, 0.19035159005236207, 0.18684793659469662, 0.19254934799871729, 0.1829986886963638, 0.1972645074433883, 0.18974032839401317, 0.18846088241088543, 0.1954035218660536, 0.18374310438948036, 0.18433418080803218, 0.1853413516259536, 0.18209474505580056, 0.17938959336779428, 0.18241007701381862, 0.18399119652460444, 0.19804068844493855, 0.19682379393954105, 0.18161660197138296, 0.19883773789305503, 0.19339227066385767, 0.19979322548796213, 0.19044885782770954, 0.19495232512070262, 0.17044493609048966, 0.19156476162031394, 0.19413524372439692, 0.18710880120167667, 0.1844849554517494, 0.18151173376087643, 0.19387243463649864, 0.19260243783145325, 0.1791574731445836, 0.18781716986707891, 0.18913481644918612, 0.19013102538165097, 0.17734191139475658, 0.1890136963748851, 0.18799750266791626, 0.18279306956565802, 0.19119955871654556, 0.19125166233580695, 0.18477207128711456, 0.18581870951936288, 0.18519591329228238, 0.18295736379355856, 0.20038278965788417, 0.19332896563408025, 0.19518942060970298, 0.18890983170575396, 0.19092998506171666, 0.1977947305492497, 0.186581490852439, 0.1782193321343363, 0.1901731008050835, 0.18408073999381816, 0.2024261364953154, 0.18403673735835424, 0.18601989221698495, 0.18176195920193622, 0.18974519970250633, 0.18381862246246922, 0.17625332962781057, 0.18764224554750078, 0.17772766943476356, 0.1937567486809619, 0.17905323310815394, 0.1901817658188628, 0.194565565107127, 0.18594869772970316, 0.1952694104004562, 0.18674025755202792, 0.18731008947766578, 0.18118534378142162, 0.19131281989486973, 0.18610844045655597, 0.18121736819133188, 0.18748094029739704, 0.18655108731422082, 0.19004504800624797, 0.19732856600743048, 0.18715291881519855, 0.18647470962104823, 0.18351693212171588, 0.1797791866617131, 0.18103896162119007, 0.1781236663442199, 0.1741612237301721, 0.18943575843217134, 0.18440802766348724, 0.1921660942593716, 0.18109690180252805, 0.18674038736315712, 0.1956311473473424, 0.2011602267378639, 0.1830455206631647, 0.17823505429976708, 0.18776297548272552, 0.18623638065283896, 0.1806102481079813, 0.18737591502208029, 0.17680196551484334, 0.18938506283390782, 0.18772294817220225, 0.1837984715285972, 0.17538269950854138, 0.17628942222157665, 0.18978466545231884, 0.1848513323059435, 0.18327290331197266, 0.18340576736142364, 0.18012562619793004, 0.19165157420841297, 0.18744614300127554, 0.18938294768894096, 0.19404439438195661, 0.18462389458845588, 0.19619116775525683, 0.18888378689220003, 0.18279681632143974, 0.188555998863907, 0.18208476912967475, 0.1743792504220058, 0.18360842374005912, 0.18442143998901844, 0.1946590688302027, 0.18328573729812286, 0.19713663941469878, 0.17760768129181195, 0.18501637102805504, 0.1854224863842738, 0.19193460550481858, 0.17839948596622981, 0.19897159898276376, 0.19589128296272096, 0.19068875761025048, 0.18926793070199482, 0.1877397412343101, 0.19200460559603758, 0.17966631884273812, 0.18986717150908158, 0.19775358110818897, 0.18052160461435224, 0.18636647411713908, 0.19441604366460458, 0.18863046339216005, 0.19473574853205047, 0.17765923575119955, 0.1912636072283376, 0.1835970038317794, 0.19340995716799775, 0.18499051479611153, 0.1887635087916126, 0.18487445955337295, 0.18671964316015024, 0.1942014418455935, 0.18318681739813714, 0.18223315126688414, 0.1839371382828282, 0.18859245692904134, 0.19088535382423405, 0.1957261447741364, 0.1832028628145803, 0.19186399911048885, 0.19198730324531618, 0.18668366410634452, 0.18786728584836965, 0.184411112600612, 0.18705067680776097, 0.1903285379415954, 0.1905010672050048, 0.1909730854644227, 0.18754529935000763, 0.18836391730322571, 0.17512800437717957, 0.1945223605585085, 0.17342671143817148, 0.19089816221769795, 0.1899823989619371, 0.1978669373612614, 0.19204137972714744, 0.18792607900099448, 0.18751243543216958, 0.18676690179859684, 0.20088828341965823, 0.19805705881806554, 0.17335629201799407, 0.19546619783317395, 0.1887435485705693, 0.17710581222818267, 0.18942935031729013, 0.18297904946880716, 0.17837082731196008, 0.1742818386938328, 0.19066533429562232, 0.19133440876521202, 0.18866917817908185, 0.1784819455125392, 0.17941541577001172, 0.195082985940166, 0.18188138949843202, 0.19382742252583102, 0.19189892383902163, 0.17856619967843243, 0.18499508486561758, 0.1895552810195678, 0.18813757931089706, 0.1838377688567517, 0.17792015459433597, 0.18890599290503535, 0.18729132885904923, 0.1884957801426749, 0.18744187512678168, 0.19163977959610876, 0.18581303654053624, 0.18702938290662569, 0.18439511492733543, 0.19838196569155542, 0.20361974268625496, 0.18997923459717114, 0.1798264835249092, 0.1858715224982286, 0.1887449588338206, 0.18455323531349935, 0.18525546429299003, 0.18896267447248466, 0.19627417827177085, 0.1982485213114103, 0.18585202813373988, 0.19280141308852802, 0.19012179584855401, 0.18662827600431206, 0.1816742802004613, 0.19575701190255707, 0.1980331979714243, 0.18499413758719474, 0.1984728407466396, 0.18203886085181897, 0.18507457103441294, 0.19288036604153377, 0.18435157335248828, 0.1848094079059381, 0.18789101608798356, 0.19085461114739619, 0.19323095634559007, 0.18516014721309826, 0.19310224765548642, 0.18474498597146125, 0.18939347592588784, 0.19464132165344256, 0.1794946881722464, 0.19648076491163427, 0.18647195493711138, 0.18296623327545009, 0.18626488584171386, 0.184646525923373, 0.19446106509066755, 0.1876389986694271, 0.18163248250254127, 0.1931685714023051, 0.17761168627944557, 0.17524050280472045, 0.20529913994585855, 0.18360798427919162, 0.18663128342805133, 0.19437049364602524, 0.18743809089761992, 0.179778433255162, 0.18180326544248063, 0.1950241831201823, 0.193786757489343, 0.18420829754099408, 0.1867850935104537, 0.18314789202493595, 0.19900219716066955, 0.1786519353651754, 0.18360926856080168, 0.19163511136827935, 0.18161207660097473, 0.1863972897167609, 0.1803591642371318, 0.1721546927260178, 0.17958504505932882, 0.19330100144469753, 0.20175490222691608, 0.18932222784126856, 0.1811260417659311, 0.19621781471544372, 0.18352234028520606, 0.1989248057385631, 0.1784303612980716, 0.18851230616377793, 0.17448565265740393, 0.18404291984579133, 0.18847420655587135, 0.19399685484878695, 0.18082509302195596, 0.19591626544232613, 0.19376395135981297, 0.19299231561832558, 0.19666861798467059, 0.19402811844920473, 0.186512567423416, 0.18783798886866424, 0.19169627519499136, 0.1851282817859909, 0.17681033813186312, 0.17830847209053452, 0.1954758744209002, 0.18049267980882325, 0.18261951374920285, 0.20910506562334155, 0.19519949204211984, 0.18675387949445318, 0.17537687980653666, 0.18990925282465818, 0.19394844861809749, 0.1877071195913212, 0.18039975133362246, 0.19771766048209344, 0.18042088293028372, 0.2022270455594023, 0.1841021743971543, 0.17446501813263302, 0.17863154660734248, 0.18814588688147557, 0.19331649919401014, 0.18209219171355595, 0.18434049257513827, 0.20188051722610273, 0.17037013671541118, 0.1857816773073967, 0.18671346560528437, 0.18236955164571517, 0.18545464365532235, 0.1911456323713899, 0.18207822357416867, 0.18254706228297993, 0.20267922245710143, 0.18944167085663313, 0.1768081434458147, 0.17861143936329033, 0.18247015598739935, 0.18971568886060064, 0.18421806569944213, 0.19473447607684996, 0.1906092636957869, 0.1837053818477749, 0.18128359686529066, 0.1905155027091031, 0.18492925324309722, 0.18307245662415192, 0.17758973349253881, 0.1813865681076988, 0.1918636424959058, 0.19403496751206775, 0.189292262953663, 0.18708425524094846, 0.17884231902970396, 0.18860857424466315, 0.1780163936670052, 0.19428686254147806, 0.19208459033644731, 0.18992054854834062, 0.18458692062604742, 0.19790775288087303, 0.19161511072321263, 0.18444473576637563, 0.18839834164868513, 0.1899898373556493, 0.19510127017049908, 0.18970301305980952, 0.1872326876967699, 0.19036943922434935, 0.18277179709245991, 0.1947030593489906, 0.1829317945726748, 0.18235116246582497, 0.1836488582441155, 0.18389711546350687, 0.19057864018054674, 0.18823541015642567, 0.1896111671560735, 0.18508067834836553, 0.19054063312192254, 0.18652398701636977, 0.19025796521005753, 0.1828406557055227, 0.17929182893268444, 0.19941492670927916, 0.1962717308760268, 0.18029387537113611, 0.1914980460335587, 0.1816696958953025, 0.1856423682668287, 0.1896288868694663, 0.1883403198977967, 0.1769547187324759, 0.1869737792166022, 0.1870291947021098, 0.17929728704331535, 0.19175303564372917, 0.18652780387552242, 0.18942684554337902, 0.18611045118237107, 0.18728096204765976, 0.18029971547597942, 0.19359648485193023, 0.19005523038413805, 0.18458490524235688, 0.17908053573943658, 0.1793196773954443, 0.18901190880341162, 0.19898784195297342, 0.19297959831533854, 0.18196456477251338, 0.17608628888507988, 0.19744402373268244, 0.17104229569580284, 0.19248402289393296, 0.1858304249587583, 0.1884892885559903, 0.18788023236636653, 0.1893060364878606, 0.19089190683204785, 0.1879185981328222, 0.18052617866963364, 0.19670045786524867, 0.20264674033617797, 0.18561829688672934, 0.1736465567342757, 0.18560439511370733, 0.18154727818779923, 0.18248032252245322, 0.1813615429930729, 0.1868256365121904, 0.18228032626467763, 0.19061185865439717, 0.18358514864483255, 0.1841862765251179, 0.1921322237006591, 0.1802858866737933, 0.18024075763810282, 0.18515435715467968, 0.19160256681463778, 0.1830630038200007, 0.18293580675821153, 0.19806399405862865, 0.18302399533686864, 0.18047736972584114, 0.19599258228261757, 0.19201650278738952, 0.1929439470752589, 0.19373932958868034, 0.1855061562044505, 0.19169828247512996, 0.19348218735326417, 0.19787925512026341, 0.19613282198205223, 0.18765245847903295, 0.19902539892959475, 0.1924843878163659, 0.18463173010473288, 0.19273219192202426, 0.19014196560956964, 0.18716107533712723, 0.19261126967431988, 0.2033043896188736, 0.18021559281822047, 0.19633289750397676, 0.1849488558660824, 0.18539320305114398, 0.19312820852574372, 0.17697084690341505, 0.18811823048389179, 0.1811127246549603, 0.19266076015818043, 0.19845059352499578, 0.19595506235797452, 0.1814902229515713, 0.19163718158863774, 0.18313381007872215, 0.1875950005592849, 0.18854369214640207, 0.1941760619670192, 0.1782514721579932, 0.18772888614320457, 0.19505435056160636, 0.18307247032023763, 0.19372371313962297, 0.18715542880794947, 0.1931380173120641, 0.1911679344042551, 0.19440394532429256, 0.19159322679290575, 0.18648207986715828, 0.1895172395836799, 0.19107815019901173, 0.1854282479857267, 0.18773279155221673, 0.18956222961335403, 0.18677741913802368, 0.18084665537454803, 0.17908574297481408, 0.1781239797819226, 0.18330259850750016, 0.18487749268149684, 0.1860703854624628, 0.197014021977258, 0.19473309022434288, 0.19316456784711217, 0.19427180680277478, 0.19096736515174637, 0.1742663875776192, 0.1906764137923515, 0.20112740036810356, 0.1821657253393824, 0.18755730579125562, 0.18461867189443049, 0.1964640697569664, 0.18303891703991895, 0.18722252027034567, 0.19537644756565795, 0.18383679300385647, 0.18379572819983947, 0.1938156743688774, 0.19437020346766024, 0.1895413037551166, 0.1837034063010978, 0.17791704308560727, 0.1910110119014262, 0.18555153089405252, 0.19536529411495357, 0.18761720537332374, 0.18891772005057536, 0.19637890271124167, 0.1830216886204402, 0.19089338333689437, 0.18780548430292365, 0.18460126390095985, 0.18888168509636685, 0.19617193177478853, 0.18223456999483548, 0.18602584033397346, 0.18492477757409603, 0.18411945638476115, 0.18318534079178836, 0.18387722377307308, 0.1939476808485947, 0.19033409592850534, 0.18582627441907482, 0.17725751927284114, 0.18667699115348343, 0.1891004749155724, 0.17872901589214082, 0.18030632893617907, 0.18963648341769984, 0.1854465247310289, 0.18282743030978327, 0.1897155053570516, 0.19186190604231826, 0.18655451447478796, 0.1821074717457552, 0.19646874015963495, 0.19439988547927456, 0.1891320881283643, 0.19918906769337694, 0.20954346652418684, 0.19884288362605307, 0.19477927215001897, 0.19033890467979128, 0.19245960126355852, 0.17774035240008768, 0.19323578826706714, 0.18069643013128336, 0.18547845847933256, 0.19555883273763508, 0.1879049753360102, 0.20031373541613226, 0.17548254801777363, 0.17649275401686587, 0.18268217237441592, 0.19326594624147456, 0.1855808433947777, 0.1976609635714767, 0.19013959626213175, 0.18457189306287852, 0.19904510602210934, 0.18385844593694758, 0.18971457142252296, 0.1862240667754465, 0.19640141103549774, 0.18710701022593787, 0.18800975450196158, 0.18793781562316753, 0.1846328462159171, 0.18512012709479236, 0.18033557077278678, 0.19224275697636803, 0.1763529840564464, 0.19891718943462233, 0.18619812158748975, 0.1933403816173404, 0.1845583360819181, 0.18027991617956637, 0.18089924619966014, 0.18410881631761022, 0.1797034571475931, 0.18735766288916442, 0.18063437984393363, 0.18681600259911052, 0.18325966797594306, 0.18557727615029582, 0.19232205102179156, 0.18555498481421145, 0.18168268424625275, 0.1950024806769901, 0.19617672258041446, 0.19132147895955476, 0.17985914330116146, 0.1818113088851816, 0.187779947944731, 0.18761270023340657, 0.1948819652742145, 0.18982220781591408, 0.1762538703452874, 0.18402297346521956, 0.1900806295828679, 0.18193902674675286, 0.1801997972234248, 0.1920231625956286, 0.20244761238044687, 0.1881157874953324, 0.19118238780264596, 0.17510039744573233, 0.19387990642974368, 0.1856465717253567, 0.17477303123928645, 0.18490846660919208, 0.19106434481550533, 0.18857235647789297, 0.18769415824429234, 0.18615534728703373, 0.19184772053899055, 0.1839269322990682, 0.19133559247320495, 0.1943021889436388, 0.18358921713051404, 0.19158133802346686, 0.17807400154137218, 0.1869846501784365, 0.19094677598462617, 0.19560103959321387, 0.18751483654098444, 0.18739827210063575, 0.1966039436895438, 0.19755510099001775, 0.1910499565557122, 0.18764533894652796, 0.1909839923062413, 0.18814240675673757, 0.17867913048506978, 0.1861390378948476, 0.18912230800130275, 0.19433368163982517, 0.19343808788477754, 0.18122485221786727, 0.19843433132252106, 0.1956505492891481, 0.1892015330339019, 0.1907793230273297, 0.19782007119741102, 0.18388220965580537, 0.18999187100898485, 0.19047787606436403, 0.188352952789307, 0.17812228371815972, 0.19143139182130298, 0.18474419997634825, 0.18319919489339656, 0.18879600692924842, 0.19114509187412299, 0.19113134607533874, 0.17481699405649437, 0.19229100232452526, 0.18342932042133564, 0.19028784108030003, 0.1888053019612621, 0.18844250361709028, 0.17628698684650057, 0.1881534233957198, 0.1923550682502609, 0.18382821787019965, 0.19019288244928753, 0.18834023336297026, 0.1832595840150014, 0.1895022452756863, 0.1972940407133224, 0.19267229275813405, 0.20155198361099697, 0.18626955919213697, 0.19034345742516118, 0.1856405063901874, 0.19896619805326757, 0.183924063390043, 0.19252820035101906, 0.17737983001604407, 0.18799646185639454, 0.18586555265891389, 0.18540108210114864, 0.2014711448712104, 0.2073755013611968, 0.19787591410278813, 0.18728942024482712, 0.18313370912435725, 0.1818225294686402, 0.1891939619233754, 0.18607666225814146, 0.20162775409887723, 0.1915356714241459, 0.18821097424628438, 0.19243203402726475, 0.1933552825765245, 0.1840580957015483, 0.19100259563498415, 0.18179901310399482, 0.20092232312178274, 0.17851111459004193, 0.1831585537859936, 0.18950773236813598, 0.17968114333899374, 0.18185454423730243, 0.18340252516246877, 0.1975299238738418, 0.18749007319167518, 0.19174691506245245, 0.19081549493968122, 0.19688504218517242, 0.17839766659877382, 0.1941335887650953, 0.18985060880806381, 0.1935568398747914, 0.18865297187041047, 0.203767672215673, 0.1874598370163205, 0.18255576862040307, 0.18430975879020262, 0.18485384437759178, 0.189119863998971, 0.19293816756688942, 0.17997111478474428, 0.17821737215311728, 0.1894534918532147, 0.18862143153123823, 0.19521491221473392, 0.1761822770666778, 0.1976640905077306, 0.1830201602642535, 0.18851640047583798, 0.18869299682490687, 0.18790166388330334, 0.20067851731475125, 0.18003161782353572, 0.2004601226122929, 0.1908257266259963, 0.1899658584500717, 0.18509094927961525, 0.18293807718811075, 0.18794098649199928, 0.18703430282885752, 0.18209411606251258, 0.18999920751623167, 0.17657864798238795, 0.18098970748129314, 0.19175121160152062, 0.18011344927130557, 0.19364086154915244, 0.1889217296888923, 0.1909750062377568, 0.18677931439135986, 0.1900559138892389, 0.187528658408152, 0.18181241736359954, 0.1918839342676049, 0.19809551337289757, 0.18291812064975596, 0.1860466309985344, 0.19017160259624558, 0.19694011308996254, 0.18173767291398213, 0.1864548437763892, 0.20297513827533997, 0.1932916077378181, 0.19234569422721856, 0.19555549674191602, 0.1978177523397999, 0.1972557094676356, 0.18649925237273873, 0.18071663562347842, 0.183604565234865, 0.19417973953263662, 0.1955589896685459, 0.19675098052148715, 0.17823716513355445, 0.18182286492546212, 0.18913849057869855, 0.1826741117665208, 0.19164327236339493, 0.19366316442170875, 0.18514611303008785, 0.1890694565531229, 0.18850146876503873, 0.19271727777488806, 0.16940984885682817, 0.2006548895464348, 0.191677898204329, 0.17726675435623168, 0.18656853736914655, 0.18908368733672556, 0.19068376601184203, 0.18817089393418435, 0.18317964960906666, 0.18360441907074562, 0.19024409843222873, 0.17896573359618786, 0.18191646344268633, 0.18656297057028437, 0.1826366676823024, 0.19001352961855417, 0.19254045572319886, 0.19353586462328248, 0.1927253879022078, 0.19199306567596552, 0.19070640657488366, 0.19116937481904098, 0.172248725746353, 0.18370430101040988, 0.18742167531629567, 0.17295340606902665, 0.17169332690473468, 0.19131474059722872, 0.19137623842048904, 0.17942801843595113, 0.18485215414756898, 0.1982869788641007, 0.19633794976885466, 0.19893232152230086, 0.1880033549588104, 0.188767889925023, 0.18310445215764418, 0.1831423462455749, 0.19063396971460606, 0.1851411129924875, 0.1843102507010768, 0.18405456517772467, 0.18981986772623075, 0.19896807528071914, 0.19561309155604076, 0.1769285491243972, 0.18424873055932786, 0.1916046613835601, 0.18524120878771047, 0.20056223269614415, 0.1922380219449768, 0.191064425674577, 0.19795335780812198, 0.18112704444137062, 0.18864128073425998, 0.1876997741567119, 0.18875703324342333, 0.18498190877634332, 0.18736221288698585, 0.1836909943340658, 0.17855647474759911, 0.18260034507205858, 0.19654118504600404, 0.18762556632661911, 0.18118011616816057, 0.19477244966937424, 0.1851039751520759, 0.18877878862191305, 0.18738249003381766, 0.18469087318730623, 0.18967582261383167, 0.18365788664224236, 0.1825322337013257, 0.18840379027769072, 0.19815194713444953, 0.1866039026546715, 0.18882818000859036, 0.186821508427416, 0.18963247924179738, 0.18130466223789174, 0.19261054924527465, 0.19303126839741927, 0.18945467872217106, 0.20067378851379836, 0.18594246761620786, 0.19411755087185406, 0.17640129475822205, 0.17754877855765228, 0.1811697577464599, 0.18726041972593194, 0.18132257568436733, 0.18328193763671982, 0.18286372345027072, 0.18963646916434504, 0.190431473738949, 0.18517539440822542, 0.18130993842095003, 0.1795273854706826, 0.17611296946075528, 0.1901156811310918, 0.19357432060503393, 0.18831219704398705, 0.195706059817965, 0.17761111249970632, 0.19177446573726675, 0.19673275933328843, 0.18092944735236233, 0.17712885593618244, 0.18776606393070044, 0.19816974175062027, 0.18255160252615044, 0.18434536298601192, 0.19246731504284684, 0.19076812357702175, 0.18368005919715044, 0.18785929623072767, 0.18299778118289753, 0.18515558987733105, 0.18545565504803593, 0.18366899448637203, 0.19183657263785023, 0.18159848092274153, 0.18211854907617234, 0.19556886039280816, 0.19521716767331998, 0.20059485038820987, 0.19150922418051114, 0.18802334835842568, 0.19437348902566348, 0.18411941763501835, 0.17399946557218687, 0.185509058859725, 0.19575641650748749, 0.18772727747387077, 0.19994410136114105, 0.18700968392191722, 0.1679002201043342, 0.1911627923851359, 0.1783114620370756, 0.18337602766650454, 0.1932112916312892, 0.19013140737983442, 0.19793689842475162, 0.19120739030203837, 0.19018492450758587, 0.1948368294996576, 0.19637500774649452, 0.18879635827092922, 0.19075194712390958, 0.19043423750810784, 0.1881388060740282, 0.19563889853612443, 0.18267667991741507, 0.19373344015168398, 0.1828332293238129, 0.19138193911613505, 0.19177060547097716, 0.19807939637148061, 0.17215954169338615, 0.1856634528537886, 0.18816727625362153, 0.19168944662959964, 0.19435481788325112, 0.18996209047797627, 0.19356954529542816, 0.18303215230709377, 0.1784250696553438, 0.18041395818811695, 0.19157449778771846, 0.19186034692097767, 0.1838537093604572, 0.18171059823091912, 0.18918042210103536, 0.1861427021759696, 0.183022025565899, 0.18013643181962122, 0.19576806010441977, 0.197922035455419, 0.19852836725107434, 0.19441556561631346, 0.18607391227963116, 0.1795170312038812, 0.1875698226022211, 0.18550526960741048, 0.186151709740819, 0.19065905407801764, 0.18996618746607577, 0.19991442815396887, 0.18928083060591278, 0.1858402452292332, 0.1821644609688463, 0.18598306839805143, 0.18957324921151553, 0.1811808448669553, 0.18550166083996164, 0.188946760305426, 0.18453195482404033, 0.19228643547734375, 0.18411951453021944, 0.1888238073366428, 0.19513872076173497, 0.19138126722018406, 0.19551008165148326, 0.1825499167808816, 0.18346780512039684, 0.18865441816695763, 0.184497759249001, 0.20323284382796739, 0.18336654296141283, 0.18410857346100754, 0.1941969052653234, 0.1784375583216895, 0.19240639756407363, 0.1825879173622104, 0.19088671702304522, 0.18671657483400697, 0.19406961132434222, 0.20559335600426387, 0.19194410034462012, 0.18378281095616153, 0.1950411569207652, 0.18440539250479013, 0.1964772913908633, 0.19144042453756183, 0.19287909716795093, 0.18871507946169752, 0.1876195325948378, 0.18726953506888439, 0.1810581603838772, 0.18792068248549942, 0.17579824699969787, 0.18706544116939777, 0.18667320932264236, 0.19247241551794333, 0.18883055761346276, 0.18116509687995053, 0.1912948361165054, 0.18491902438030233, 0.18307683708523723, 0.1884458554850202, 0.1908407758664488, 0.18141810467533007, 0.192922624690695, 0.18571274176527056, 0.17584045916929555, 0.18066839697609283, 0.183129849543093, 0.17924074432683007, 0.1884326409974054, 0.197366078546403, 0.1852627756925592, 0.18108386657934952, 0.18398427903027412, 0.1972494248549566, 0.1814340631953058, 0.1896751239995857, 0.1956200284089778, 0.18790407443271448, 0.17748289311046822, 0.18693417795216566, 0.19494474446214194, 0.18647519476533128, 0.18090981306268428, 0.184103267079531, 0.20402391621262347, 0.18427962268474776, 0.18241076713792148, 0.19616050006549235, 0.17710099774625288, 0.1912820365088756, 0.18596751776530482, 0.1870126694109243, 0.18728791761942418, 0.19058399651726324, 0.1851303843805591, 0.17101253962033997, 0.19304681946608046, 0.1849150544648143, 0.18411566189875542, 0.19011132123901064, 0.19493741909872994, 0.18307208328071023, 0.1879565752844227, 0.17879218110125952, 0.1860468993277363, 0.1681362705492976, 0.18858682888527298, 0.19643276357010164, 0.1911926929967815, 0.1877008780037989, 0.17543580501559974, 0.16667759129366916, 0.1951892672182407, 0.19412752938510558, 0.20367507862120746, 0.1871014868501177, 0.19309399744787353, 0.1887895636381962, 0.18589529741178776, 0.17592792028386417, 0.19490766157339318, 0.18884540650005788, 0.1778355109633615, 0.18963783711076895, 0.18456649644862766, 0.18461362621459107, 0.19332738597140467, 0.1921401060706381, 0.18760555006852184, 0.1790497256929146, 0.1835032188539426, 0.19733059090386093, 0.18616943137560293, 0.19479498804783688, 0.1838928993645779, 0.19213167139082724, 0.18245734580557119, 0.18795097744984185, 0.1895922490796469, 0.187068150143838, 0.18827026843657857, 0.17794355560806016, 0.18628239189316248, 0.18467400512302334, 0.19786556665824517, 0.19245045184987605, 0.18826719981513634, 0.17706697718246092, 0.18692374502605, 0.195307454007323, 0.18243929723447336, 0.19209968278793862, 0.17743697374483794, 0.19105659570282896, 0.18299657864106747, 0.18236159511385988, 0.18532629574238935, 0.18040902609199924, 0.18037224894678558, 0.18198476736596636, 0.1814351341576421, 0.17302611367614407, 0.191394435518613, 0.1894697655209601, 0.1881891043942633, 0.19246143160883375, 0.18957314444038156, 0.18669130996692382, 0.18399333325744424, 0.180625222368482, 0.18217140954589423, 0.18252419245629886, 0.19352412934240182, 0.18341468173949463, 0.19295476353777538, 0.19107014416953771, 0.1802340322051037, 0.19626432824615234, 0.18957532338404362, 0.19315549171391988, 0.18972962552579203, 0.18197981088782905, 0.1972833791426356, 0.1867488737141195, 0.18605476421269912, 0.1921373099687346, 0.19918390362533123, 0.19414880322904343, 0.19750407532924882, 0.18669566503459695, 0.18673906032859897, 0.19042263004014917, 0.19937479130567728, 0.1906058944151686, 0.20097679096522886, 0.18573011798374325, 0.19099880310682424, 0.19305041561098552, 0.18916846678802343, 0.19030916615840587, 0.18615777139448209, 0.18244559646780215, 0.18601695303245797, 0.1769865509360432, 0.18150036467169053, 0.1960019917466715, 0.184812979623025, 0.19417368707027785, 0.18213804852870233, 0.18210579557880746, 0.18658667722015163, 0.19509940645571297, 0.18934780108290758, 0.19296189150678297, 0.19208518342247022, 0.1799873936428998, 0.1873489859538125, 0.18113288089107316, 0.19086275215222687, 0.19366260101275387, 0.18064866935687665, 0.1904581731138071, 0.1971068409675525, 0.1908643834133572, 0.1900848393309055, 0.18822475023324725, 0.19241051590154995, 0.18350517190495488, 0.1877734303802261, 0.1730516153226801, 0.18766714127951614, 0.1842368019909856, 0.19216320375838933, 0.1732519921850612, 0.1852515973879522, 0.18640159821642105, 0.18317297166316088, 0.19007631175600292, 0.20122517337960294, 0.19341452377387439, 0.177699290516468, 0.16792383752522158, 0.18611192645188196, 0.18169310893333848, 0.18468244032761466, 0.20053710998633745, 0.18922597515116177, 0.18960438151782946, 0.17127449896695013, 0.18881932408279123, 0.1851668484607741, 0.19625122001729117, 0.18809549641666304, 0.18268611839860324, 0.19033745931854806, 0.18923012802751132, 0.18777861335022047, 0.1886848414642579, 0.19778187364010386, 0.18543519401459524, 0.1822803369787096, 0.18827039222715414, 0.18320777651918496, 0.18379872658381924, 0.2055216172030419, 0.18204688049982273, 0.18773770265285533, 0.1918261074193097, 0.18342437506592155, 0.17800747286164237, 0.1855767974310185, 0.19226450129428901, 0.1866599352511171, 0.18622571879310085, 0.17435056354773656, 0.18430074948567338, 0.1910596668430303, 0.1887845459069819, 0.19632379652677096, 0.1826798000578051, 0.17808309901476482, 0.1948719659176477, 0.19017743393960987, 0.19477578715653243, 0.1765438626124089, 0.18425297148542152, 0.1929826011132373, 0.18920585641618518, 0.18733414941455437, 0.1853209954482095, 0.1855181939019902, 0.19058709520481495, 0.19338503792613504, 0.18418290038289536, 0.1812734482139289, 0.180974231778549, 0.20846679320874043, 0.19329782440285012, 0.18476700538363475, 0.18430063391969717, 0.18196041165085747, 0.1787626738514068, 0.18800267954105107, 0.1811627403446042, 0.1885381378872655, 0.19640252658797783, 0.1823465792851386, 0.19099792620839723, 0.18155413089031477, 0.1764954803996536, 0.17501529638808316, 0.1847431580805208, 0.1814579817401692, 0.18433309489697527, 0.18233902318424094, 0.1823947178549605, 0.17890119373827978, 0.18510100345735966, 0.1858567336731227, 0.198388089251763, 0.18592209536642826, 0.1893540597175501, 0.18491746775978715, 0.20250742429662555, 0.20097195800612436, 0.18383668815212093, 0.17676694040641458, 0.1883396657684615, 0.1860982003763949, 0.19056190355690994, 0.19192332854530594, 0.19299772809957969, 0.18324686826512568, 0.18057436860285445, 0.1867815093343774, 0.18329855532104844, 0.17874123758125376, 0.1792198527201428, 0.17998522059538988, 0.19344800704832796, 0.19281888316532686, 0.18436409259579392, 0.19104922393134, 0.1891431827737288, 0.18253660855322343, 0.18857339453784144, 0.1826312899802042, 0.1914266468485473, 0.18562175345251888, 0.182636915353488, 0.18803764553954133, 0.18824264455996378, 0.188203870398608, 0.20225240707431977, 0.188280324804999, 0.18023785519088228, 0.1674871442981528, 0.18370829439092298, 0.1770472131032801, 0.1881219453041663, 0.1947171761128537, 0.18588988786310912, 0.1931463580842939, 0.1814086436796658, 0.18652956488806138, 0.19302502040186342, 0.1925863016446721, 0.18926935803414635, 0.19415512872070964, 0.18386115905257489, 0.19112502496498493, 0.18231384569807443, 0.18505153571103786, 0.191090143539375, 0.1829094206286533, 0.19164621069771987, 0.1826340978548297, 0.18574129502182474, 0.1880556982294196, 0.177150102226159, 0.18523403510642716, 0.17912227335596712, 0.1864036368892002, 0.18603272496259557, 0.18560246638339808, 0.18954769736009494, 0.19323033500892708, 0.19043523125787432, 0.19333567604889282, 0.18993784818707124, 0.19142731634031343, 0.20092210869049826, 0.19195890542957328, 0.19009077105992844, 0.18223072615801486, 0.1862150035615902, 0.18183795151119342, 0.17811981615652012, 0.19163437602409542, 0.1831351317806796, 0.18853511628898756, 0.18477052016158033, 0.18689743053890553, 0.1908761908925213, 0.18745294081711558, 0.18377420026936037, 0.1842147753037189, 0.18531949436192496, 0.19104731584565254, 0.18604073916876698, 0.17852933145511413, 0.18974303127375133, 0.18511880917963616, 0.192379538012191, 0.19334525195515317, 0.19327937639071718, 0.18300210362433014, 0.19416370418937834, 0.18407884000402525, 0.1825683900165466, 0.18141825869663394, 0.1981569626180096, 0.18406753040388227, 0.1844857847151141, 0.1795826686480896, 0.18716350284752323, 0.18589207164323343, 0.17086239168509104, 0.18519769159323138, 0.19520641206263967, 0.18887642209863553, 0.18141733369751487, 0.17702588984155398, 0.19164902938736683, 0.17855328936613532, 0.19064928254891977, 0.18576253599082782, 0.17953152884633936, 0.18498179268448262, 0.19717864691742862, 0.20027450092802454, 0.19305945077423395, 0.18941919269167679, 0.19675971654424237, 0.1983627784184158, 0.19189614206918154, 0.18331892646905176, 0.18779159784554272, 0.18341394790388257, 0.18459486009793888, 0.1821856395994593, 0.19581802239202536, 0.19225194299744852, 0.18454319896089502, 0.18878127854336557, 0.19005558639399797, 0.1965027534632321, 0.19221090970320898, 0.1905950521472551, 0.1836743893090144, 0.18987993340187168, 0.19851240877728127, 0.1931371501908433, 0.1879305458512475, 0.1792196773890187, 0.18129781682396867, 0.18434056749198952, 0.19691570220252783, 0.19075868814, 0.18980403985756814, 0.1922628554104874, 0.1878227368953622, 0.18946797471180496, 0.17861507981321387, 0.18444371190937514, 0.1860015758624785, 0.18611826636617634, 0.17952346575644218, 0.189289514665747, 0.18457970843859367, 0.19040171725109295, 0.19659977447027374, 0.19629830254643282, 0.184239351756938, 0.19601874151409698, 0.17082374203545647, 0.18851507155415875, 0.19424522821366308, 0.17985887623637073, 0.19169947016272243, 0.19307630456079217, 0.18942205257785796, 0.19409244586330124, 0.18991965471937944, 0.1883462537561039, 0.188924897660875, 0.17683415018453602, 0.18391581456181355, 0.19826508475594118, 0.19399885105969503, 0.19395669479023608, 0.20574772354977452, 0.18793706549595077, 0.1961643778791713, 0.18531125107884536, 0.17741287092953842, 0.1858214210020577, 0.18766346103971407, 0.18333411571990307, 0.18543376499890266, 0.19170244470825665, 0.17795236370962042, 0.18458258531638205, 0.18761339222590648, 0.1873168375816599, 0.19144148744936185, 0.1884656186558896, 0.19065195277906524, 0.17276341195935363, 0.18620838081158764, 0.18578117869184604, 0.18710635292912636, 0.19024729008318306, 0.18990632256890566, 0.1828227027106599, 0.18388626317810633, 0.1842506373656444, 0.19713391518984832, 0.19925175088085548, 0.1903292595614093, 0.18070544924891052, 0.19275966307086387, 0.17414826567626793, 0.18998610043673964, 0.18623645539618108, 0.18668046686838574, 0.18942860289201516, 0.19314095619103608, 0.18769363736467473, 0.18733830877241703, 0.1959981306783294, 0.19911222442450632, 0.17486353411631275, 0.17856010002212197, 0.19538633347170747, 0.18610492280184956, 0.2052120681304709, 0.1959458094018916, 0.19775411970765888, 0.18334695428673523, 0.20128209742250866, 0.19706046756341022, 0.18013188851957662, 0.1761048926688532, 0.1875416511540297, 0.1943428096032592, 0.18903922010880472, 0.19330995663670486, 0.19310134868973716, 0.18330197946939306, 0.1892992219397895, 0.19156640267637917, 0.1918505237444846, 0.20283218648880053, 0.19313416995322727, 0.18946318578675508, 0.19879342702880132, 0.19030050814282212, 0.20188021551879637, 0.19505879952426408, 0.1842941037323667, 0.17883421500500515, 0.1939650236692069, 0.1816074913146522, 0.19458178026824277, 0.19345360891709773, 0.19332309946799336, 0.17343356089221526, 0.1914879980827226, 0.18200816698278632, 0.183598251628402, 0.18355879014229431, 0.19001517085207437, 0.1897462227436324, 0.19253919367299635, 0.19212277067544598, 0.1831199147526339, 0.1958878314681255, 0.18477123243608734, 0.18317320103695856, 0.1941457382484312, 0.19307319565146996, 0.1908932027596175, 0.2118562422743172, 0.17998069971861405, 0.1965614111115984, 0.18835423744638644, 0.18360352513508485, 0.18929662290014396, 0.19568815571592912, 0.18543550824836746, 0.19146599338450696, 0.17803006638942542, 0.1835230287943246, 0.17950736504323903, 0.18384278453738476, 0.18404990446348574, 0.192526632003698, 0.1938436677968272, 0.18829525813427048, 0.19824086527112264, 0.192599901769838, 0.19243033766175374, 0.1848188148434683, 0.1855441401253715, 0.1937588952239101, 0.1896246355990235, 0.18484700558101078, 0.1925280516411864, 0.18623613208880543, 0.1953572512662435, 0.1794470997470357, 0.1885114089877409, 0.17988933502447238, 0.19730509232846694, 0.19508213909299746, 0.1768315363626062, 0.18015172976255428, 0.17386822486368164, 0.18678499270117607, 0.18565877257762012, 0.19242650113065687, 0.19959240813701942, 0.18260212145134072, 0.18386765455109078, 0.1938728057523839, 0.1881178373370239, 0.1834577052931243, 0.1868278296474411, 0.19152087891672542, 0.18638294470009198, 0.193364907389462, 0.19675102970590938, 0.18128638335679287, 0.18970430024931742, 0.17923843287706498, 0.1958239150666928, 0.18932047270158625, 0.195439572193925, 0.19785670511256817, 0.18311871780583436, 0.1948243605895976, 0.1929275539776201, 0.1818554364365496, 0.18731988964290686, 0.1967918731776818, 0.18195634919081283, 0.18863137855915724, 0.18580507091444762, 0.189775514215952, 0.18231785486363497, 0.18572505069903636, 0.19291631517619404, 0.19959149849929134, 0.18791236681322773, 0.19003198814506306, 0.18400042106075237, 0.1774609114265329, 0.18821614423539615, 0.17793338234111497, 0.18906927284494016, 0.18437476178477266, 0.18151427162144806, 0.19166983111438757, 0.19096242866111504, 0.19261149173628622, 0.18952763499552303, 0.20305506853082844, 0.18620865165049422, 0.18281423850938086, 0.18508118739853588, 0.17594217146191013, 0.18695024733750085, 0.18730428798223736, 0.17653707077328618, 0.18800702160359367, 0.18878244770624553, 0.18238635970263575, 0.18349802115598607, 0.18752413938672582, 0.19128047425564615, 0.18272660313151048, 0.18530680347264797, 0.18462497149750634, 0.18582259052182432, 0.18676896332027781, 0.18790677130268132, 0.20209043300592727, 0.18350650590644485, 0.1805145282849491, 0.19016165874269572, 0.19477057758316457, 0.17390512109852085, 0.1934071357981076, 0.18658778398994869, 0.18864538405577636, 0.19516138081823345, 0.1824119528104401, 0.18960403288332295, 0.18983943099351813, 0.19282387285920524, 0.18631783123825724, 0.18033449191977466, 0.1863132897209714, 0.197487095094289, 0.1901081069678915, 0.18356612666323463, 0.18959022530741768, 0.18027814798015895, 0.18936875503762293, 0.18255792971080428, 0.18671845387454533, 0.18213021350607855, 0.1954618988193488, 0.18433402745185065, 0.1876887049938098, 0.17812411771402512, 0.17976667803243285, 0.18405228757549405, 0.19401137142352265, 0.18511507447384382, 0.19442630593414054, 0.18452485319733475, 0.1977427339503575, 0.182013256183292, 0.171527990499572, 0.1803780604575854, 0.18666171766124903, 0.18784391050046054, 0.18607145254606813, 0.17844501256104311, 0.18745842622304656, 0.17560721363501589, 0.1941386532951133, 0.19180983679150257, 0.18418906809068228, 0.1870997162968002, 0.18406121661865119, 0.18713163529201537, 0.18477509780340304, 0.18737568882298955, 0.18724164793216946, 0.18329654412974822, 0.17659299880054322, 0.18986519966611043, 0.1853428289390192, 0.19087789034906608, 0.18301056063937052, 0.19238538576055106, 0.18316411311873806, 0.1844924752363118, 0.18320701514505905, 0.1918645961811667, 0.19434680109960856, 0.1882758930361305, 0.1896557955277378, 0.18873039514525994, 0.17851771491668522, 0.18479139512615658, 0.18878889713460592, 0.17989806371656591, 0.18373662548539538, 0.18145626257257058, 0.1966131885897075, 0.19812330985166532, 0.1987891695326233, 0.18854288265158617, 0.18560937317299026, 0.1820591082191625, 0.1948971916951855, 0.19824201580811032, 0.191601080318336, 0.19040431861869744, 0.17399352309921143, 0.19456035632066945, 0.18877078363908947, 0.18060585683318828, 0.18388013784862747, 0.18049559349189886, 0.19712532330473498, 0.19756823498634343, 0.19573539315260532, 0.18422962460788522, 0.189809785478214, 0.18657878105716497, 0.18430493162889472, 0.18464991004280862, 0.1975768325070446, 0.18570183074037314, 0.19056769794396775, 0.1816797364378345, 0.18137576739917333, 0.185087716922281, 0.1870136356318805, 0.1843776162387587, 0.1957775686502285, 0.1986021892036412, 0.1991969585052814, 0.1790192864121941, 0.19356871981194917, 0.19012010227942502, 0.19462742318919207, 0.19393691267378563, 0.18449428777024177, 0.1977654355987156, 0.18618992636430837, 0.20498395818840506, 0.18389688216031203, 0.1842947614684293, 0.17892045020271305, 0.1935361250798162, 0.19416036273341467, 0.18545615059209594, 0.18468537210832742, 0.1932039407784783, 0.20457299035073945, 0.18417681687734722, 0.17383082810861575, 0.19450130731826235, 0.18785735808514634, 0.1767033450736263, 0.18098849626254168, 0.17797473583360726, 0.18008032139209432, 0.18195465047177348, 0.19158509621300931, 0.19094547222839267, 0.1896786406183354, 0.19755643513274365, 0.20256976660373227, 0.18103751900283865, 0.19185554411720415, 0.19460681075933406, 0.18689461431161572, 0.19577262100020262, 0.18386064075023556, 0.18926726554070605, 0.19205334727139856, 0.1862982027235415, 0.19021952788715807, 0.19843008306746146, 0.1906806991170564, 0.18885612284962405, 0.19165336847179865, 0.18989111946229076, 0.18757963071881076, 0.18359581147670545, 0.19315628724749412, 0.18764302474618247, 0.19554783021692304, 0.19472051079234007, 0.18766731403147308, 0.19780870296841888, 0.18542306129980132, 0.1950637778484058, 0.17984202877428868, 0.17153409615523255, 0.18165419274804426, 0.18495715485080005, 0.18797388175318525, 0.17434024365898745, 0.18261530600451248, 0.17726119179668784, 0.18239916618692875, 0.18635297589901495, 0.18298960407598489, 0.18432143331976045, 0.1847557377576977, 0.1818321005764196, 0.18935087000810216, 0.18412773746766592, 0.19258854012791748, 0.1745920897694434, 0.18246515954580403, 0.17702662703429675, 0.1884461100649027, 0.1899676273802436, 0.18675946043198932, 0.1789631728857171, 0.18371674578033106, 0.19630285389730998, 0.18960395570799457, 0.18802759395119212, 0.18413801972923485, 0.18669132820416973, 0.18162205060921566, 0.18815013399135683, 0.18201197970298857, 0.19374018665740292, 0.18137827849142077, 0.1899983091622263, 0.1966741427597282, 0.19334719351852825, 0.19117380091645095, 0.16819415606014881, 0.19002028233083976, 0.19474031004510978, 0.18799442336274289, 0.1916779900418936, 0.1882105845952285, 0.19131680127941747, 0.18524224507933312, 0.18742697782263107, 0.1797131212823049, 0.19585865650797973, 0.1935367903560157, 0.1811382576699932, 0.1924308964397457, 0.19119152285604082, 0.1831135602252721, 0.19512460548884422, 0.17800902577691582, 0.19586022764750105, 0.19001445023432142, 0.18434249591913798, 0.18813071748752272, 0.20155879442944136, 0.1884628484735767, 0.18434471936004063, 0.18348872391101123, 0.19457696504492963, 0.18977263055897517, 0.17669751301848305, 0.18227740403901777, 0.1954697254921414, 0.19166213515691774, 0.19634760402448057, 0.19445293365960398, 0.1910104174018245, 0.1834593366843337, 0.1944839004311761, 0.19395700076439523, 0.19178276188270604, 0.17868542971221268, 0.19081730349544057, 0.18384145089411227, 0.18898195823116234, 0.19179010254437148, 0.18199530506265485, 0.19899247272282658, 0.19636645430808009, 0.18878444584231863, 0.1984573978281304, 0.17785063559976985, 0.1866562295176474, 0.17783580538188265, 0.19572393560054627, 0.19524937324022687, 0.19741872315708994, 0.1885561979045614, 0.1834252054125502, 0.19058672524366865, 0.18425191327990217, 0.17216165612621753, 0.19316101596602828, 0.20132904248772635, 0.1891696646991734, 0.18026145491441914, 0.17536031974183103, 0.19956653238990876, 0.17265367515149954, 0.1901933035514627, 0.1752783304639509, 0.19386599789392497, 0.18409154524805965, 0.17988966425051392, 0.18995693431409216, 0.18536722163244881, 0.18632910189947716, 0.18571066105855377, 0.18644859326953783, 0.19018871777901059, 0.18889642527056988, 0.18976713703650408, 0.1667010386691206, 0.18513459643084393, 0.1878363292819356, 0.18910527646214717, 0.19407547292601823, 0.18734463261691778, 0.1845329575768385, 0.19396235014822194, 0.18865496277360017, 0.18170194258029918, 0.19577659734225566, 0.1871486299246449, 0.187237924773957, 0.18625150235766816, 0.19067141407312924, 0.18827511993314497, 0.18428718193055096, 0.17597662444699452, 0.19042566724695317, 0.18744329613241012, 0.1855950199237589, 0.17908152650270684, 0.1790532900020875, 0.19260590075050762, 0.19030439584894115, 0.18252679131616223, 0.18576890263436868, 0.18580655830806195, 0.20118965592443736, 0.17703734592133288, 0.1954822786445392, 0.181546859909048, 0.19576005869035515, 0.17873211062270528, 0.18259143667619832, 0.18332724832379815, 0.19009578283020168, 0.19117969768094747, 0.187093578141649, 0.18581685725537295, 0.19938173603006545, 0.1847259916949744, 0.18833791269185718, 0.19077643502117508, 0.18751312081652916, 0.1947882470474629, 0.18996346227868674, 0.197531873468663, 0.1896167709696005, 0.18543988367305, 0.1944828344979285, 0.20005256597436347, 0.1875042403091217, 0.19288602184117393, 0.19309744809310952, 0.18264795889855645, 0.1929201907193858, 0.19564928754963834, 0.2000071361114183, 0.18538164638511076, 0.18495630572017882, 0.18072134498651785, 0.19270231333512644, 0.18999569777370057, 0.18992311515069968, 0.19119886233940683, 0.19052565325457976, 0.18923712815590218, 0.184172132823959, 0.1885955774457865, 0.18436606240598571, 0.18660918812325847, 0.1790740642569249, 0.1682536287734106, 0.19606438914487806, 0.192562306287532, 0.18455048955076225, 0.1682830252089474, 0.1927316657006324, 0.17473906686079366, 0.17999904919778234, 0.19803186081860405, 0.18691256426198155, 0.18714921448365854, 0.1851231283478233, 0.1788656966250708, 0.1962640592328511, 0.1899000669895887, 0.1912273477421621, 0.19467558574320426, 0.17435159306006623, 0.18453528295598398, 0.17507816044690913, 0.18738956186001943, 0.18835318670018938, 0.18996308182620444, 0.18663683986700857, 0.1944247489824452, 0.1768465313459987, 0.18403742441755705, 0.1851773067337868, 0.19634170106384186, 0.1938364783925204, 0.1872207209467178, 0.19427902642880007, 0.19099143807146968, 0.17559763653933028, 0.18466448901992655, 0.1920609250128147, 0.19135468178261597, 0.19765775811153782, 0.18645077358569218, 0.19125220715088842, 0.19275129644416275, 0.17938923037639268, 0.1927611152490376, 0.18251682191883606, 0.18051498462304685, 0.18902870466063282, 0.18504045359163218, 0.19441586962116586, 0.19312231528940402, 0.1852566472442718, 0.18184212288802748, 0.18345454092050287, 0.19840381379563365, 0.17443937394465564, 0.18128535607717125, 0.17281986583119913, 0.18487883939306574, 0.17686223272684573, 0.19292006432018924, 0.16647841493436621, 0.2040080912234437, 0.19234394159614637, 0.18977348531741414, 0.17842894701654294, 0.18799567692832236, 0.18404560740619735, 0.19120678048291945, 0.18922279564760727, 0.18689865279118748, 0.18234780267640438, 0.19882890562172192, 0.18733918844437622, 0.18694765810307662, 0.187026550421387, 0.18858210660375818, 0.1909985867000761, 0.18343395212195895, 0.19629614155950625, 0.18707641919210652, 0.1844718767651478, 0.18479238995160302, 0.1908994810700445, 0.19090362095421953, 0.1770847901099508, 0.18574500361157112, 0.18955397809957442, 0.1937476018499714, 0.18421558219109452, 0.19105956575950284, 0.1855198292427368, 0.19291089021317234, 0.1917951494887364, 0.18181422009734965, 0.1996981460857941, 0.18422163175455677, 0.18943056563927632, 0.19311712315405202, 0.19268244916118765, 0.18710672015273463, 0.2008942889038185, 0.18608176581735475, 0.18644302093094517, 0.18304922550890826, 0.19217786273323798, 0.18404788993749643, 0.18769653490802726, 0.18512051679948258, 0.18605209351311502, 0.19422580594127523, 0.19676921908011552, 0.18811682967860702, 0.19528293047424305, 0.19244503454563278, 0.18197985626416677, 0.1935153630963372, 0.1900267251045471, 0.19886257531961835, 0.19751663618976362, 0.1904618527434218, 0.18191261003690298, 0.19094088646466625, 0.1794528225741232, 0.18513015839963032, 0.19117828826018388, 0.20007185691494303, 0.1985080552811166, 0.17989075558106457, 0.1893674071265303, 0.19179555963867392, 0.18732294780520362, 0.19423212204047308, 0.1934589628270777, 0.18653738826063263, 0.19383388115553432, 0.18327597469680823, 0.19591939707577527, 0.17702521830480153, 0.1791283056171001, 0.17446227115903643, 0.1860753399762531, 0.19774051194143055, 0.188148136832513, 0.1956435954365853, 0.1909060917425969, 0.180677298080834, 0.19491219539485033, 0.19051916562463636, 0.19401242594092485, 0.18993105213733644, 0.18391905280570897, 0.17881892155948384, 0.19110636259600525, 0.18498950964902736, 0.184143993571847, 0.18562786381652369, 0.18994310105585244, 0.1969710187111618, 0.19413185660518412, 0.1834005486141037, 0.18500844056150895, 0.19144779114500632, 0.18445510062861875, 0.18832777125017197, 0.18926761869927491, 0.19213490212971773, 0.18440297795853294, 0.189594803506409, 0.19371016670270597, 0.1894985815252777, 0.18609062541639942, 0.1896903245734399, 0.19086292724259657, 0.19904440599438483, 0.20181276184667324, 0.1825615329153043, 0.17290211093261043, 0.18279358011355626, 0.18203696434062458, 0.20561042313661648, 0.1943279926422533, 0.17875509521497934, 0.1887068921550306, 0.1734328192641116, 0.18867111787566707, 0.19159866143692642, 0.20395322003811817, 0.19023145669681038, 0.1824195651420075, 0.1933359049545804, 0.197513700790282, 0.18509458103041604, 0.18665469481853655, 0.18268961000030529, 0.17868643028753958, 0.18816176570812518, 0.19604172066571074, 0.19450453086771105, 0.18651831115767165, 0.19029190248317476, 0.18960638802951663, 0.18871277278150705, 0.20031360788034253, 0.19200141236983298, 0.19906011547487404, 0.19069724159336884, 0.1959779782532884, 0.1863144448533323, 0.18953364558431107, 0.17415204082622882, 0.18399711488090886, 0.19233139104664093, 0.1873973714074914, 0.18558994140689436, 0.20242438226190027, 0.1784207227251953, 0.18753890539531642, 0.18049932382082995, 0.17793561545660566, 0.1807232385817694, 0.17826167240713306, 0.19357539028175516, 0.18662311825505892, 0.18068776242576057, 0.18705811625879012, 0.18105777938065554, 0.18915796029144422, 0.19179076807479692, 0.19076570967376805, 0.1826609871580458, 0.17407123468539693, 0.189938087721405, 0.18138053616636982, 0.19555404922524192, 0.19995217760183961, 0.1880060125039859, 0.17512125016191318, 0.188653228239546, 0.1904120526561677, 0.1863221405118226, 0.19388070852548314, 0.1926285686140992, 0.18002494983488673, 0.19680955268356526, 0.18207749965039297, 0.19956353697771206, 0.1940602066194371, 0.19387588804824404, 0.1853921319803312, 0.18383172451706417, 0.18378521020261765, 0.18103439973306254, 0.18217080508855973, 0.17816360594049918, 0.19050477560250997, 0.17672757773528994, 0.17440098491153086, 0.19019682470721774, 0.18578527820546986, 0.18538851061049283, 0.18789838576559806, 0.19098494864416496, 0.18852349918781744, 0.18095892037806063, 0.1849635702582986, 0.185128400069281, 0.18444000936683866, 0.18686616262168854, 0.18124551476860062, 0.19432195032563607, 0.1898150700600549, 0.1835079747334621, 0.1778843583495651, 0.1930203371138593, 0.19297668102188262, 0.18921088217680498, 0.19434814154355468, 0.1970739788080138, 0.19906262025219196, 0.18457962111062798, 0.18835545834312012, 0.1884137903854449, 0.1925689164300539, 0.17529537800987227, 0.18406926742011523, 0.1877712272828309, 0.19708074623481203, 0.17621709747964254, 0.193957275936137, 0.17621889760606818, 0.19562788258342106, 0.188576643652421, 0.19730605272052218, 0.1847048569678974, 0.19131550151878132, 0.20133790741672486, 0.18746595702405605, 0.1960080173833678, 0.18589651974703422, 0.20226934903561564, 0.17663820132596625, 0.1887800657216356, 0.18800654603928896, 0.1873336734255, 0.18482088913157543, 0.187427024598055, 0.1874511921345551, 0.194634119755419, 0.18524128002126622, 0.18810670039628846, 0.18871058534634488, 0.1848614041195557, 0.18965705373178413, 0.1893306992479322, 0.18122503554121852, 0.18362816271360086, 0.16992834036113177, 0.18758239410657024, 0.18998909665923927, 0.18536306260185056, 0.19777833900450603, 0.18765809905811812, 0.18700699174540944, 0.2001488365667528, 0.18444100224372312, 0.19763177294787335, 0.1871051911844477, 0.19227471335117124, 0.188025295622383, 0.1762897281215174, 0.18842460214734222, 0.18363514154488791, 0.17933370535048596, 0.19302456323948533, 0.18896108804151523, 0.18956575525682717, 0.177210722887914, 0.20087153035621133, 0.18690513934513742, 0.19365986325060197, 0.1879183137370895, 0.18358435421057973, 0.19028565241224935, 0.19172223739767258, 0.18858288508643264, 0.18882977061625555, 0.1929978376938251, 0.1810470935373468, 0.1795415881672016, 0.1796031752692745, 0.1916929228044984, 0.17373460451043046, 0.18298080878746226, 0.1884393970204451, 0.18442751452760142, 0.193105141234425, 0.17321868702726692, 0.18311060368602472, 0.1886717796867513, 0.18481328179858933, 0.19773958253359736, 0.19007234022533573, 0.19815075057985262, 0.17256825859975555, 0.18891769263588012, 0.19362090108213295, 0.19411961281170834, 0.19743888744476298, 0.18204622242064694, 0.18398441958229042, 0.18495757015671602, 0.18520306853368387, 0.1829876288637836, 0.19155745337791788, 0.19274395464690633, 0.18728899535515217, 0.19290425422886234, 0.17721004555283465, 0.19147037695691552, 0.1782676135032557, 0.1914405369551792, 0.1952651537986224, 0.18150209184263694, 0.18410032424299644, 0.1827590870839508, 0.18006561179166944, 0.17782213973368732, 0.17980788900485162, 0.19905400867053194, 0.18531879644969373, 0.19220708448679494, 0.18027221192879536, 0.18639711514354607, 0.19417834578553408, 0.19243639251206468, 0.17161081864321134, 0.1884189338838462, 0.19123340315121135, 0.18301511921819305, 0.18170306388533083, 0.1869612733164095, 0.19362108305895456, 0.1890680064655734, 0.1887207635837927, 0.18401770828689587, 0.1920051634244922, 0.18031545340880933, 0.19054397897395586, 0.1936858461466425, 0.1841466786855747, 0.20297667669544364, 0.17883920305602694, 0.19336252395676537, 0.19176789343117964, 0.19050101746069334, 0.18364215161584624, 0.20309130493238758, 0.19321888828508604, 0.18170820457955902, 0.18384341947935104, 0.18801321060546966, 0.19046841212552099, 0.18149699408666112, 0.17999628872630835, 0.18373491215280946, 0.18579635811516962, 0.16874300376695345, 0.1863941822569518, 0.18671917034755656, 0.18410035391859736, 0.1817844705017427, 0.1761792516533003, 0.18965216887599595, 0.17531020589696938, 0.18085625467232086, 0.1917719209713466, 0.1966693638965583, 0.1779528004250168, 0.19298761623073066, 0.17865777918166373, 0.1991674675590628, 0.19684212938620305, 0.18176590650141192, 0.19505045226776735, 0.19212387813294007, 0.18412460995546473, 0.19220841000168037, 0.18884558345975125, 0.18810442149118076, 0.18116620860491336, 0.19028269975199616, 0.19294756250515446, 0.18684404882860675, 0.19800097160086885, 0.18473561239186656, 0.18055995437100159, 0.1857570801407554, 0.19519414278101005, 0.19837739107665758, 0.19067089248338337, 0.1951528495436594, 0.1909045036202135, 0.17743592980568718, 0.18652378995368984, 0.18718450639656842, 0.18185217372808218, 0.18591816006144413, 0.1974902002245948, 0.18784686063166023, 0.189338650480404, 0.19907627694924776, 0.18339889234063597, 0.1998228786800957, 0.1976954527874728, 0.18600423325865742, 0.18269395342489195, 0.17909827320741475, 0.1931332485662469, 0.20307445896095824, 0.19079129428476854, 0.20371683620007802, 0.19268753940565467, 0.1976852223174776, 0.190920425030863, 0.18643048108541344, 0.18847428885196496, 0.191170620289338, 0.19288863996779618, 0.18710450370922396, 0.19444626131047268, 0.19478043605341536, 0.19948867589567537, 0.1847150225064444, 0.18813115614673376, 0.19155290006318118, 0.19460625055509967, 0.19354382206899523, 0.19401255889867117, 0.18774667922275778, 0.18788317839469534, 0.1942569241986586, 0.19211170253465704, 0.17838522554578173, 0.19321152488080442, 0.18976594289519852, 0.18122342281251722, 0.19310497459959827, 0.18581985762659117, 0.18427293423548297, 0.18641816512466425, 0.1859192142934387, 0.18250252904552733, 0.1854751287030263, 0.18936046456548428, 0.1834062650857153, 0.18556075263743516, 0.19309117268687742, 0.19131609986941234, 0.18719718947401476, 0.18694023661421919, 0.18195973573740254, 0.18524776662951506, 0.17374163887933225, 0.18414290107997897, 0.19469945925120458, 0.19499005786646936, 0.18462890861019915, 0.18952704002788281, 0.18913909290922148, 0.19636469998801762, 0.18814995559463543, 0.18937790456454318, 0.18372387219354774, 0.18934347300248897, 0.20069820113610334, 0.1812809054719484, 0.19149465267756255, 0.17993823294911682, 0.18277685480072006, 0.17863966056743397, 0.19110906361028207, 0.18941448841716818, 0.18343316161344284, 0.19701319689734145, 0.1933123008319114, 0.18873674848125882, 0.1917911067801904, 0.18915537998488507, 0.195429965998675, 0.18929113028679564, 0.19058189688048843, 0.18496669358243667, 0.19060314369543804, 0.18351386638106512, 0.18944976620342133, 0.19252071589263064, 0.18372727353576745, 0.1905309505203189, 0.18779869905496463, 0.18851631051006246, 0.17764184695683852, 0.19263233529434934, 0.18921626043225862, 0.1758280833503379, 0.17850299120479132, 0.18539053502108954, 0.1898993173690221, 0.18040005628548383, 0.18518522611094446, 0.19633011221282648, 0.18149570415117725, 0.1839215961496558, 0.18242279689476837, 0.19602371809963426, 0.18773952639794125, 0.18358021038973596, 0.19825399185666265, 0.19824400646242823, 0.19922494597471227, 0.18026230519457978, 0.18607701989120842, 0.19564980814205996, 0.19805442631623424, 0.17578709032572487, 0.19266021755882398, 0.18788072210549048, 0.19570589404613362, 0.1890472656110014, 0.19069972534740143, 0.17997543396253787, 0.18949201457432482, 0.18580565923851594, 0.18604339697718206, 0.19852125562058262, 0.18932312951452734, 0.1887630502822124, 0.19043727386540554, 0.18554152178098626, 0.18500856091997805, 0.19218713436510876, 0.19288544695588583, 0.19615473087067634, 0.19484883398951813, 0.20398244440420177, 0.18313151560706922, 0.19222062680575855, 0.17746119654400505, 0.1791783316148247, 0.1884669047951295, 0.18972865387934038, 0.19286094654521094, 0.1815983027253637, 0.1884196633694794, 0.18551160562552707, 0.18048557683995875, 0.17725302167224988, 0.17638484067837662, 0.17803569808294264, 0.17873271369491672, 0.1976699369122284, 0.1875730091843789, 0.19006288812297298, 0.18415991432010093, 0.19414006949957757, 0.18684248882701615, 0.19347319602796592, 0.19113649374332412, 0.18449601623890446, 0.18998343368451448, 0.17953441754832494, 0.17471327725139868, 0.19521124435388004, 0.18976837382648035, 0.18626522474997442, 0.18569185602244115, 0.1896391708483504, 0.19435018430403653, 0.19165557031196018, 0.19382461182377428, 0.19253779255366257, 0.189511448034351, 0.18802512357946774, 0.17696391775954534, 0.19473130972389177, 0.19347849204451031, 0.18957070756307567, 0.18931428504158224, 0.19025071382936168, 0.17891735384902457, 0.1842148223130539, 0.18391210550155224, 0.18949383442748907, 0.2027433458167142, 0.18342362250595418, 0.193184075408536, 0.17820887078034647, 0.1958977142332473, 0.18895727955336103, 0.19305469898340616, 0.19375483694426685, 0.19496778984568378, 0.18800812347531057, 0.18914360122018478, 0.1903600818062711, 0.1874204054834224, 0.19156023092407126, 0.18976442448659772, 0.19121099153614243, 0.19102641432130726, 0.17874942037863956, 0.1933711361035114, 0.1874850375743526, 0.18587432643457735, 0.1878413851427208, 0.18267242156549385, 0.17811105762483045]
mean_norm_big_PCMCI = 0.18304493098429017
sd_norms_big_PCMCI = np.std(norms_big_PCMCI)
z_statistic_big_PCMCI = mean_norm_big_PCMCI / (sd_norms_big_PCMCI / m)
print(z_statistic_big_PCMCI)

### Bagging

In [ ]:
M2 = 1000
m2 = np.sqrt(M2)
print(m2)

In [ ]:
norms_bagging = [0.2287423641308614, 0.2389589852412949, 0.2256259926576523, 0.2523423483144557, 0.19138881261142868, 0.2217896798794777, 0.22216171078935262, 0.2639171423548674, 0.27671239097768296, 0.2282771441848567, 0.28497625179035035, 0.2701536944837547, 0.2226186483391151, 0.22769936214591044, 0.2351430203515426, 0.27539997820605877, 0.2633169651909091, 0.2671894777284162, 0.23127982444536155, 0.24361615933160766, 0.27192917502088565, 0.26355962270518507, 0.28353400302794857, 0.24429978716342304, 0.2457064237804403, 0.22232587452536853, 0.2300503570991007, 0.1969399590324254, 0.21164171869336673, 0.2655040869021218, 0.26302110489894304, 0.2952421640383075, 0.23909586567753094, 0.2559436051187518, 0.2909974188134704, 0.19678555867871628, 0.22924929177838158, 0.2753431567419801, 0.2587607888751356, 0.2071339768451249, 0.20996955169734868, 0.28662368594330806, 0.2284974545729123, 0.27653188388793043, 0.22037532937737273, 0.25160596627771453, 0.21288101318831373, 0.21508277489905844, 0.2677255663278015, 0.25143755912811505, 0.24492466298764354, 0.2206548526963708, 0.2523032296840952, 0.27958001924514775, 0.22612865193921658, 0.25152530407884816, 0.2813390334019879, 0.26687931426190675, 0.2197674984680958, 0.26070016180974775, 0.2750734693441118, 0.24346269511198557, 0.2719569605516932, 0.25540759588233836, 0.2430020229470448, 0.26471866521851534, 0.24871386884143837, 0.2744761938988142, 0.25388559138866135, 0.27118001254897534, 0.2520653220297935, 0.23156125413858386, 0.2319406595767402, 0.21854646643644335, 0.25491811604232645, 0.22724802878647787, 0.27043830017593723, 0.26177173957879724, 0.20168587533430515, 0.2614845424670903, 0.2422603937277579, 0.21808920765655052, 0.24788121549600456, 0.21745681142151269, 0.2659347876175338, 0.20731707734405402, 0.24974194085557694, 0.24012842835587128, 0.23823714012582217, 0.22451278389039975, 0.22896132539455857, 0.22142891930136702, 0.2505870305241145, 0.24910579153259244, 0.25219695313898743, 0.20663895912030122, 0.2434106163853543, 0.2023939617632906, 0.23497048040386775, 0.23738039980786713, 0.24401826091862075, 0.24131470915548264, 0.2300911516128702, 0.23834573544570048, 0.24765442535662224, 0.2387669297315547, 0.2506192424336393, 0.19440985967664212, 0.2601769909119613, 0.27204444374677234, 0.26238334616378994, 0.24226597171467706, 0.2767782672189652, 0.22328570069195325, 0.2565802414205311, 0.27028031196773233, 0.21678701959850624, 0.277810678979504, 0.24433445702604228, 0.24935612254727554, 0.2757324611423578, 0.22847527314217198, 0.25027636871198716, 0.22516063095510114, 0.27243811587681294, 0.24723334447401069, 0.23935610605804217, 0.25325716052788305, 0.22704483701590153, 0.2462080493593284, 0.26433651956333865, 0.21438292344213564, 0.2645412644727594, 0.299985467644651, 0.2383659129130944, 0.27865518702211856, 0.2793052388678837, 0.28512386094671865, 0.22661044217384563, 0.19359081594222363, 0.21705669301135005, 0.23980121980275093, 0.22280176507844174, 0.25415049702317893, 0.28020716619950026, 0.2557015524455319, 0.2486786924142546, 0.26682978935872514, 0.2829693145327332, 0.25549228723907025, 0.2711782826825118, 0.2816880733523043, 0.229054316131869, 0.23810617435155707, 0.2454856010716398, 0.2345293498316602, 0.22945602881644667, 0.2188494411675432, 0.23497252821320108, 0.2383834006895861, 0.2260880844756241, 0.25588006019924087, 0.27402876321233893, 0.2816594300232315, 0.20957604860050275, 0.23662845296304816, 0.20272795112542505, 0.2725268788888305, 0.26903289357038634, 0.2309678427369462, 0.29437521347024365, 0.23497569136158952, 0.25886529199564584, 0.2562489816284373, 0.24502776795631445, 0.19915990077373721, 0.2395045398463871, 0.22059960971957496, 0.22220002488979748, 0.25753193509586814, 0.2503761682974674, 0.2433355441752317, 0.2510780466958063, 0.2429902918211206, 0.2286545358867597, 0.24858811752312052, 0.22974843741325973, 0.2229882224991946, 0.23743123778316538, 0.2947219353245034, 0.24697282435651985, 0.22901569965598695, 0.2486126302934495, 0.22807232524615473, 0.23287586057725249, 0.19486297656739932, 0.28080968938790174, 0.20350290328215084, 0.21338466505633058, 0.21129423785940046, 0.2219611825410754, 0.2408233249619186, 0.2401443706162485, 0.29151141305674233, 0.24962885893977824, 0.24784797467519315, 0.2597114976592476, 0.2759507620246158, 0.28306950737866576, 0.2221329039731978, 0.2786851856395719, 0.23423584150489504, 0.2698970971748811, 0.20414492490309424, 0.19909278435166677, 0.3281110540396279, 0.20432805572949106, 0.2761715500259473, 0.22340138356589126, 0.22916589963314768, 0.2823344647342192, 0.21036984987191482, 0.25198781344403814, 0.23042539616050722, 0.2623171422153971, 0.20710479177910981, 0.2694094619299332, 0.2406077106759541, 0.2815390496577242, 0.1842398636964341, 0.2516297190537725, 0.23428986186248, 0.23308153776882698, 0.2549757011362802, 0.265855457497032, 0.2650413934643193, 0.2193421198712695, 0.2609905733078021, 0.19930333814581297, 0.2902625779365008, 0.2561786109846334, 0.2668692995950664, 0.2740332014246032, 0.25360096475315486, 0.20979643039559318, 0.24631965849334927, 0.2556875559575528, 0.2570055727821363, 0.21930782193316553, 0.23599806105409477, 0.28583722482657115, 0.22378143085157645, 0.25227400381090564, 0.2777812099065024, 0.2907996564691604, 0.2740324727952283, 0.25993738460201565, 0.28681088900638824, 0.27870918799019934, 0.26293293272010676, 0.2640230708264614, 0.19124678933044414, 0.26677149801025163, 0.26269652042220737, 0.22785935224315926, 0.2395444021996236, 0.23340712361888755, 0.2797920951467826, 0.23845772284294156, 0.2660183913894356, 0.23147717545597463, 0.27058322807483065, 0.28023339457964647, 0.19796506927899607, 0.25569131713725923, 0.24076887784132278, 0.2233609685540444, 0.2412736122730561, 0.24862895306876473, 0.25606582127438393, 0.23724440918396564, 0.26454887614586636, 0.2565019171311058, 0.2601825541155596, 0.2868914831809368, 0.2888299535747464, 0.2689476304363216, 0.2631833300879409, 0.2195517497866801, 0.2168861456686476, 0.21570982179013193, 0.23290683096506243, 0.24163629730450456, 0.2744387811489182, 0.24836267296437675, 0.25331754390319156, 0.24333146915009338, 0.21878836362974435, 0.2696308103124377, 0.24294765380635536, 0.2278927142090314, 0.22606911180144587, 0.26213414254159545, 0.23091641155366024, 0.2631769402946088, 0.26214469085256487, 0.2220341842269555, 0.2704156934417235, 0.24682438534497858, 0.21637196827133187, 0.24879197194449149, 0.23949600273895702, 0.2502684054316417, 0.28813005865948044, 0.23055937304872604, 0.2454448118797921, 0.27108185015616454, 0.2103867215092379, 0.27863636223122396, 0.23736340683502005, 0.21694968072181847, 0.25195453813684177, 0.3054313101402566, 0.24510902613794086, 0.2609850179064759, 0.22670553042132544, 0.2500325557867328, 0.23333418227336009, 0.20518469953078536, 0.2688062992409455, 0.23217768275448983, 0.22698454127192844, 0.2524796911499412, 0.2287724832565019, 0.2796489963733408, 0.2287550461536539, 0.23736248937375837, 0.21619146012450366, 0.26418764395446065, 0.28056201930554464, 0.30738440992307203, 0.26674434037594585, 0.2252903713698465, 0.23573852039470933, 0.26812375166471786, 0.2761389995655603, 0.20226755486430656, 0.25744605053974384, 0.21276899257229578, 0.2426210293594751, 0.19301517072878427, 0.26455267341155636, 0.22203370380784254, 0.23678203380494656, 0.25247339928199086, 0.2606005505831722, 0.25501082888854115, 0.23901412342986872, 0.2400667998251777, 0.2254102077631116, 0.2053999758380601, 0.30334294893078195, 0.25320196877828816, 0.23193926070306872, 0.24615513170680345, 0.274464610123207, 0.2316960477497385, 0.21996718138588897, 0.2624408570906049, 0.27070184433776995, 0.2390546787204228, 0.17223166116509148, 0.2669172695165909, 0.22714783757716564, 0.23798110790090618, 0.2620365093098197, 0.2497069242121455, 0.1970024484934203, 0.22682785098930583, 0.2545661983672844, 0.1901351324841147, 0.2346984048570984, 0.2490257627502748, 0.22312330937014693, 0.22738445193360246, 0.24018590251303817, 0.28899066140119856, 0.29612582715558905, 0.22155768442355758, 0.29205783172908506, 0.2540190100226793, 0.20388397363611013, 0.23747165610551985, 0.23613924493438768, 0.22926342348158252, 0.2206690454388486, 0.23373862688765126, 0.21143409808927252, 0.24805926328551337, 0.2473084425386455, 0.2552573828299213, 0.23880963620255713, 0.24225040656639474, 0.25663513665201043, 0.26869584241465383, 0.27670139665622423, 0.24292853745428417, 0.19402306947820286, 0.3018693626713781, 0.2642522325588013, 0.22991523559309393, 0.277929871130483, 0.23289934468961224, 0.1892186993297604, 0.270918592070587, 0.23474014836030288, 0.24877032386958214, 0.31253480262311745, 0.21342566035272742, 0.22328700240330207, 0.18511334464525192, 0.29920595058184224, 0.23858378849968792, 0.21934233101289277, 0.28807045093738237, 0.30033192042552603, 0.27620834313838744, 0.20241164243892143, 0.2723598139515948, 0.22420754676591392, 0.23095963177795956, 0.28444782469398006, 0.22808730373009137, 0.2237090135817654, 0.23996666440141323, 0.25003581954952825, 0.2696284077523468, 0.24319697287768186, 0.2807135710192318, 0.21179510768120247, 0.2017340634778983, 0.21711387368504598, 0.2248408305261318, 0.24847723173698313, 0.23855066930891697, 0.2805211773997273, 0.2582321442374572, 0.21872149972252652, 0.22803015809946445, 0.20356705777548662, 0.23645278419073548, 0.2623874780570469, 0.23242868271598166, 0.17481092313315452, 0.21805786915631994, 0.3012338672834898, 0.29383461842796, 0.24618620678703318, 0.23719151722379572, 0.220236468666897, 0.24937494944361444, 0.24171949780095295, 0.23334320700419606, 0.24787473093922466, 0.21871759180469164, 0.22166540001208412, 0.24693954697805964, 0.3121262207273461, 0.20238554150569074, 0.2022805307817906, 0.2685605138895347, 0.2078604031135003, 0.22011269562563598, 0.21827407660531437, 0.269285133141463, 0.2524880250997858, 0.2562879436223859, 0.2657043544580618, 0.2658335268380267, 0.27014906417870427, 0.27630388519577553, 0.212338744229871, 0.22650823258802508, 0.27124958817354644, 0.2699415707608923, 0.2219659372061197, 0.27227984071189276, 0.2339458640341732, 0.25761524274498615, 0.2892394408092796, 0.27459640858532897, 0.26237371296132733, 0.24760820435133754, 0.2324001177246913, 0.21689897545773532, 0.2412819072636871, 0.250195434811279, 0.20112617239046904, 0.22169010023626576, 0.23891786252181502, 0.2389855515003346, 0.24557690903033563, 0.258148499895734, 0.2522667956577921, 0.2501094895792284, 0.19292316794950573, 0.2303256836539591, 0.24932210433627813, 0.23760524698069282, 0.21981793528334242, 0.22025824444436173, 0.23593647920313532, 0.22465153649340688, 0.2829038818439029, 0.26212908380678346, 0.23334819628515824, 0.23604271215354744, 0.23974093049965217, 0.20261839077306038, 0.2323197745648108, 0.21880982396511803, 0.25666800528851946, 0.2601148401030776, 0.22200345295546195, 0.27063272863566284, 0.237272600650293, 0.2534530416269493, 0.30911954975032413, 0.24961500038884596, 0.24672147003734787, 0.25340892503958773, 0.2521357012906814, 0.22146082042279663, 0.25007622645064775, 0.22509016820360908, 0.24810789504308492, 0.21874960841160687, 0.26073445015337604, 0.2844197976841148, 0.2370730858059415, 0.24762785689828756, 0.2687618980628669, 0.31390682027964706, 0.2446353699992612, 0.23689252309370307, 0.2371195935464443, 0.24343108466469146, 0.27597795315049695, 0.2663466084630018, 0.22495993815787396, 0.18058266107297938, 0.2990741606307035, 0.2571410422022393, 0.28520280769568346, 0.3057603052746333, 0.24970424069991823, 0.23304438989356466, 0.250653473932494, 0.25256266168447167, 0.1872495606881303, 0.2571780140394689, 0.2386594231804362, 0.2536900425884166, 0.21181295032859945, 0.25541725188153186, 0.21179291046940435, 0.266755638814386, 0.27828494190095904, 0.2194582428495634, 0.25919379972828593, 0.23601053770714003, 0.28078041724768615, 0.24544523178427735, 0.2683603837645708, 0.2506734797561554, 0.2539706955076178, 0.21559228209366668, 0.25050850672186725, 0.2387032701635757, 0.2521621038018584, 0.2576652347077762, 0.23547309868033128, 0.2169709041943048, 0.18410851732960387, 0.27421779085240156, 0.22671424106858062, 0.33776025890696126, 0.2721342328143526, 0.22811103502716526, 0.2703773252670148, 0.269291757919134, 0.26056179961173, 0.21016935496109368, 0.3256519651783298, 0.2783340446576049, 0.25754492959471414, 0.2332993022066708, 0.22678646485380546, 0.2521651908466505, 0.23980870744047186, 0.23515413540794902, 0.237686498245204, 0.25762607683255745, 0.2622205526280849, 0.26828967413079247, 0.2568741310408879, 0.23730813748288684, 0.2294027405781405, 0.26351027864451027, 0.18877697781601677, 0.2545525703658477, 0.24974231066833216, 0.22880674451888713, 0.2786854601005001, 0.22307189702593944, 0.25304308244194135, 0.24305701478934938, 0.2490916011927691, 0.2685599966076132, 0.21205590142454853, 0.2391572447783854, 0.284361410371076, 0.22411179366658193, 0.24314620774147339, 0.25861272139444214, 0.2584787338459206, 0.20272511291396006, 0.17055440567628552, 0.2432656305791735, 0.219110864235452, 0.17852290949206379, 0.24811751285124514, 0.2739557416918581, 0.2535313998450597, 0.2375325974726475, 0.2649417505458917, 0.27092752090241873, 0.27542598283170455, 0.27125290647903694, 0.25012219533111496, 0.21625436419061553, 0.22849009887749594, 0.24942975801648523, 0.24540283310351887, 0.23953030816529247, 0.2049207603360703, 0.2563543882709825, 0.26806686382404743, 0.2122739949957199, 0.2669045751526242, 0.2535848521234304, 0.21355504718845436, 0.26029038263550774, 0.25333470441277095, 0.2244406102148051, 0.31760623153819184, 0.250606843464792, 0.277050817556876, 0.24650614074223762, 0.2959112686756288, 0.2581581252809551, 0.2153430000731951, 0.22519891019352936, 0.20152338536500797, 0.2697544606833479, 0.2635420481392458, 0.2309704060216489, 0.2648329189906179, 0.2699755276141945, 0.2688259748702052, 0.19890574813133635, 0.2349704079925705, 0.20240175352820874, 0.26286116126416054, 0.27810506224845744, 0.24551072596037168, 0.21791581813657515, 0.22204810661954177, 0.2638703937605052, 0.28069573409893916, 0.26363563147116975, 0.29019389074615176, 0.17803138980786695, 0.26444665450609345, 0.25188568261165056, 0.27214107571862867, 0.2404619746015848, 0.25184381674277245, 0.22343476382758617, 0.2795830389367391, 0.24811877753188347, 0.24012275201946326, 0.23308457192950474, 0.26103945094314956, 0.20891955672562576, 0.2060601729922577, 0.2647329212147349, 0.274594258132692, 0.2788181191813869, 0.2356643152082851, 0.21408641713767088, 0.2227850545344007, 0.21336344621886882, 0.2503398774484241, 0.22774318197364446, 0.23192210226396173, 0.2573415445763284, 0.23515870269460745, 0.20155273524166253, 0.24350144816151414, 0.19909297584988309, 0.2548004598154213, 0.2216866395708282, 0.2404900971909016, 0.2230842691171314, 0.22441275414022707, 0.21093251716615574, 0.21646358203243077, 0.2165530715987595, 0.24768342649712213, 0.26149730842756946, 0.23101459476630729, 0.2324621658793694, 0.20942495389297205, 0.230834193677779, 0.23657785161157732, 0.24366129411522772, 0.23175144221086266, 0.19641308553322295, 0.29362274700691043, 0.23221333611773204, 0.20094900892083767, 0.2548168090720692, 0.23618948057898154, 0.2729637090808072, 0.2427663474332536, 0.24726730901201693, 0.24377114418332158, 0.26076513251143707, 0.2254410232143293, 0.29535244471165234, 0.24198944922487578, 0.23441523221712982, 0.2899387958027363, 0.22379865447777011, 0.2805803750178049, 0.22251088559440943, 0.2816230504594272, 0.25497221364044725, 0.25534604564592356, 0.21232381765491293, 0.2529745200750269, 0.27586456617496274, 0.21318955545628482, 0.2431370820966787, 0.2626544112782925, 0.19604431382773044, 0.2621763442696486, 0.2304914978827563, 0.24346003903254396, 0.26540503307053964, 0.2540802724348746, 0.2674899935859021, 0.2838900740936022, 0.21834874818504166, 0.22059398965882032, 0.33348877362261586, 0.24416444962007566, 0.2074815385055132, 0.23538434859362464, 0.22759387038305168, 0.19783269090434133, 0.24575962612575158, 0.21715701815197663, 0.2602327937154425, 0.2541727916170228, 0.24467779054618277, 0.24347757661506345, 0.2561265144725618, 0.2528748402897978, 0.25770553816987085, 0.2696329366937235, 0.23657930630079335, 0.21847586400473368, 0.22684583753041793, 0.24419755044106914, 0.2378376073842217, 0.23361538806970011, 0.26060927271422796, 0.18687728033288545, 0.232022031273543, 0.2953205628898814, 0.267795579278405, 0.2310802857819477, 0.2958070952912055, 0.2039771083129555, 0.24970780537424597, 0.25560594452168595, 0.22422198640429028, 0.24243058868455064, 0.23640785295837494, 0.27091258178134403, 0.198388991472951, 0.24553210990959304, 0.25309340820587617, 0.26093584224383776, 0.23940498805150867, 0.2799367413858313, 0.2769854869175763, 0.24766905641016582, 0.2804434603240818, 0.24261217731748144, 0.3021732617432955, 0.31270701697655173, 0.21345961509709457, 0.31183310702374867, 0.24236042208884617, 0.2899317970985269, 0.22499149020877846, 0.28118323591970196, 0.25516711586805574, 0.2753636625909163, 0.23641557534277954, 0.2633005323112082, 0.25599839493231064, 0.20637365587258744, 0.2622892282743623, 0.23787212859510687, 0.2447458535067341, 0.25575435172174943, 0.27686804581808205, 0.29002348585599036, 0.24741854500857027, 0.2803739495804608, 0.24859871479182702, 0.23571259426208258, 0.2146930479483975, 0.23500309806924655, 0.21989906183230168, 0.225734668199222, 0.2412096457822069, 0.21049085095371053, 0.2637017295559414, 0.2208181476592991, 0.2838852774056531, 0.2374464914905178, 0.210015526992005, 0.23377308413737916, 0.25706022573744114, 0.2652239117438194, 0.2715378827712792, 0.286724597305353, 0.25287437785924755, 0.2605549398681427, 0.20524813932402752, 0.1933466458559351, 0.2157352135668968, 0.30566921227799965, 0.3094709121736053, 0.20428642443225375, 0.23773297613040423, 0.24908222061027877, 0.2539276965709066, 0.2731070985668694, 0.2563482453111811, 0.22220133250170293, 0.2423786681337631, 0.19589297989499405, 0.192307612587831, 0.21111344811589133, 0.22018796089082737, 0.24060761899246325, 0.29272735942482275, 0.1958018151740321, 0.26298169312533426, 0.2221806146680211, 0.23596490499570635, 0.22557150329756256, 0.22337256939257277, 0.25880016884670093, 0.21787412242507304, 0.2171680710080853, 0.25460699019953126, 0.25368939489662823, 0.19100120176558427, 0.2197056762618772, 0.2559026147815044, 0.23066064518807589, 0.26470057274437453, 0.24848918374087245, 0.22948775206575275, 0.24157193343542743, 0.19948421257273122, 0.2546842137057981, 0.25731410680025313, 0.20884509760646902, 0.2639122806814905, 0.24436281144612026, 0.22207483757139967, 0.20567075146587996, 0.26073187756204896, 0.23060877498923807, 0.2524444727794626, 0.27770673095595305, 0.24750257884917373, 0.27050951251437116, 0.23825571939137638, 0.21494602475407862, 0.2594819328657884, 0.2344871320177609, 0.23434804502046874, 0.17192748200729066, 0.2670018607861755, 0.23714245119456873, 0.23362279475091297, 0.24362716052012498, 0.2273244750397865, 0.2140275971235535, 0.2682418215117241, 0.2968506334959275, 0.25677056322359554, 0.22620187825801075, 0.2746897048545431, 0.22307034693175332, 0.2454145471994127, 0.22848097199590622, 0.26699829364466077, 0.22936140578408137, 0.24684839053569643, 0.26106405869872407, 0.24322681187137346, 0.2335138744764977, 0.27442708397527993, 0.2666530799130467, 0.26327625311238123, 0.21105388192688815, 0.2410628389933826, 0.2623485198126069, 0.2031366145863312, 0.27045436279374435, 0.25112126437883275, 0.24843440834853553, 0.21730925956480393, 0.2456887791400407, 0.2383707489946101, 0.2606703304060947, 0.21262962272577796, 0.17485311044310814, 0.2106286364177112, 0.25037497428748384, 0.2626688884410962, 0.20725873857048557, 0.2109106720707646, 0.22742737474497415, 0.2166993768094583, 0.2317065029487305, 0.27926623064710016, 0.22927544163581112, 0.21692035716994784, 0.2401335738706459, 0.2477231266714525, 0.23532209650227875, 0.22189775258003067, 0.2661999799464829, 0.235890316676062, 0.26905989962987775, 0.2397836760292288, 0.24985143738526042, 0.22594208596671875, 0.2677266381772366, 0.23013023056249865, 0.21048838363245415, 0.28201797994683697, 0.2364702353766314, 0.24724687378952737, 0.23202612858993757, 0.21742934545710915, 0.2613894347079279, 0.25040920768906433, 0.2677934519687978, 0.21314085381757805, 0.21700666981259684, 0.2390323896759278, 0.23428344810463833, 0.24189355362859732, 0.2540332630925906, 0.28552392224709805, 0.24214627071473918, 0.23824196255152866, 0.23695894553619473, 0.23909898945052893, 0.21252788288961733, 0.25075367837251095, 0.25118716715954004, 0.2339416388290504, 0.24242387401605134, 0.25198075041464374, 0.2701076148817578, 0.21296027632501827, 0.2685319098608128]
mean_norm_bagging = 0.2011617419891221 # norm_try_bagging
sd_norms_bagging = np.std(norms_bagging)

In [ ]:
z_statistic_bagging = mean_norm_bagging / (sd_norms_bagging / m2)
print(z_statistic_bagging)

In [ ]:
z_statistic_bagging_432_600 = (0.2091991126251739  - 0.20632048461714927) / (sd_norms_bagging / m2)
print(z_statistic_bagging_432_600)
z_statistic_bagging_600_864 = ( 0.20632048461714927 - 0.2011617419891221) / (sd_norms_bagging / m2)
print(z_statistic_bagging_600_864)
z_statistic_bagging_864_20000 = (0.2011617419891221 - 0.19851195476445752) / (sd_norms_bagging / m2)
print(z_statistic_bagging_864_20000)
z_statistic_bagging_20000_10000 = (0.19851195476445752 - 0.1941689793264833) / (sd_norms_bagging / m2)
print(z_statistic_bagging_20000_10000)

In [ ]:
z_statistic_bagging_PCMCI = (0.18304493098429017 - 0.1941689793264833) / ((sd_norms_bagging+sd_norms_PCMCI)/m2)
print(z_statistic_bagging_PCMCI)

In [ ]:
# Z score for T=2,000 as test
norms_big_bagging = [0.22094150971477167, 0.20350996166734764, 0.19193608964585374, 0.19279463803152916, 0.21904041786465975, 0.23158379082866526, 0.21825396941688802, 0.23152709424849804, 0.22941222339065653, 0.23624422540341122, 0.2212688873865289, 0.2306418312785771, 0.2376717342690909, 0.23188682043458272, 0.230149921438537, 0.19857452105129914, 0.2260281807344979, 0.2470764294681207, 0.23086302361109662, 0.21520046246998945, 0.2196494828332486, 0.24083440476767418, 0.2172693196570089, 0.22666035633291182, 0.2204203912527695, 0.22469940457235804, 0.24499560167134016, 0.23025761933722516, 0.2425516891148137, 0.1996948176636468, 0.22403370367010775, 0.20910934151337016, 0.19185431116649457, 0.19168659740371305, 0.2356961589894409, 0.21670508503782168, 0.21454603225635993, 0.22516494015998068, 0.2289616390895111, 0.2104782650309989, 0.21300270873030624, 0.2294563554387714, 0.2319136653397754, 0.25583109804779613, 0.20301768990114127, 0.21517559733614566, 0.2165378192853566, 0.19873706291836343, 0.2019597449594186, 0.21216895203151195, 0.2164826953751256, 0.21767848904165488, 0.22133751641225946, 0.24679536884142017, 0.1821563570255004, 0.22960683547982616, 0.210598802624716, 0.20813829869514866, 0.2126854165400254, 0.22474677893597592, 0.21377045554145868, 0.24030050390635158, 0.25729207224423434, 0.19566080914885958, 0.20155365121816884, 0.2380294116233726, 0.2095479933591664, 0.21139778553172872, 0.2368771609519639, 0.2215635224494043, 0.22431318720516, 0.23140670835407465, 0.20688582793917934, 0.1924662363701801, 0.2360629225891753, 0.2309777861277335, 0.2170387995836294, 0.22299363051055526, 0.20750203466245842, 0.22340856810146764, 0.21750745599936364, 0.206516438332067, 0.21042821514764126, 0.1944347041498614, 0.2194577338486526, 0.22745868002493902, 0.2487780602290086, 0.21704067087890594, 0.21584160365117078, 0.22651539092415846, 0.20245225372539663, 0.20644950440603882, 0.1999209965054465, 0.21891511120110504, 0.22511336255344697, 0.23878584705098424, 0.19418996387504303, 0.23966642541239022, 0.20506188956538854, 0.21453505820239627, 0.19738705598910725, 0.21228947414352475, 0.199836699864881, 0.23246315632476267, 0.2193495776708505, 0.21127819980507528, 0.2069892371931525, 0.2062347975893051, 0.23893858381933433, 0.22524036167398204, 0.25352104127342334, 0.2099347417195962, 0.23416929923805202, 0.24933317593868318, 0.21258740255139696, 0.19950970979385238, 0.22576256815874368, 0.22246837121338175, 0.2330965764751968, 0.2084429253014037, 0.20830039677797982, 0.20889811271131511, 0.21945296678799361, 0.2089017559358021, 0.2246020709879056, 0.20449676332354388, 0.2254282540951545, 0.23039299501863492, 0.2094401513304619, 0.2227737571009574, 0.23562216409432243, 0.23337336178437112, 0.22163450279174524, 0.2126903217757637, 0.2369460309475222, 0.182725645614371, 0.21332905862978663, 0.20130060849325224, 0.20890374712465468, 0.2225971261892594, 0.23217484693957957, 0.25241470609545363, 0.21986853737581166, 0.22147670654101587, 0.19717236419453896, 0.22535266416227645, 0.2319283625234522, 0.2025731081745493, 0.21861863209706933, 0.21864172538645926, 0.22063603780922783, 0.21867928724222388, 0.2084769382462823, 0.22459543122874276, 0.23669044790883903, 0.21258971150461464, 0.2440198064567847, 0.20122149583665586, 0.20975907083123604, 0.20151255832929205, 0.19548113266199812, 0.20004966476688285, 0.2385002010339902, 0.2294850606883035, 0.23981964807397596, 0.2192558484862622, 0.24704223665730737, 0.21055272041564593, 0.2350992873147559, 0.22064903404392158, 0.20366713239702264, 0.22436351376190833, 0.19478852594363605, 0.24387511167409961, 0.1808092351308743, 0.2225805856437708, 0.20809829617765221, 0.23551411956094945, 0.2224377257921806, 0.22490384090470683, 0.23708732738682636, 0.19979961603297758, 0.19103763618463301, 0.20948964025045522, 0.25267942937107984, 0.2250728676535497, 0.2113601832305694, 0.21896790889731582, 0.19472727218396094, 0.18580657513980842, 0.23438099706019835, 0.19607804718926708, 0.23766336963901416, 0.20148391989195072, 0.2348565749462532, 0.22766533987993118, 0.2215985363906503, 0.21094380171184013, 0.18154659784885835, 0.2567937159002005, 0.2356314266905911, 0.21639387464122378, 0.2548503651410477, 0.19501925065463954, 0.21563440690765442, 0.19610362767752013, 0.22769664627589764, 0.23874853442258664, 0.22797872545168846, 0.212400408817433, 0.2187440582910866, 0.2017006979407864, 0.19321772482656374, 0.20598689352257757, 0.1981750838799817, 0.24652152671500788, 0.1974915845225947, 0.22688042153740068, 0.21281017864307025, 0.18803214595892884, 0.19946944504177597, 0.22084578822526135, 0.18365976300518191, 0.20966830606823836, 0.2172322887792649, 0.2348287868612487, 0.18973637755515854, 0.17299700639480303, 0.21129843739438062, 0.2559860680564404, 0.22043445735178105, 0.20028278939345076, 0.2257761936974677, 0.22432582765045905, 0.2033238622033372, 0.23285413606488403, 0.21914273462047237, 0.2199515026013751, 0.2380373930785839, 0.21857593986914275, 0.21992984008375513, 0.23016334230601979, 0.24224198575204722, 0.22083400347852972, 0.20326560182795636, 0.18962786069302687, 0.2151906472476742, 0.2259515261665704, 0.2199574773040084, 0.2040468917173183, 0.2338574377658224, 0.21686406127437013, 0.2616786516020207, 0.2170650204196692, 0.20350972778864101, 0.21116853005569833, 0.2095715903236982, 0.2616446463206448, 0.2369084202494601, 0.23992853237938813, 0.22494459087941632, 0.1988986274687166, 0.23140072189031724, 0.22986822019243291, 0.2364040754130178, 0.22918426853829668, 0.19495653357106357, 0.2240905029903757, 0.2279177551561206, 0.22844083305833013, 0.21595075742988834, 0.22534312921799174, 0.22515414248599294, 0.20774304207076666, 0.22748785388142248, 0.21971101540920304, 0.2398388792360294, 0.19304282526846356, 0.2524020228920115, 0.22136392733506424, 0.2466930341252103, 0.2013907406326083, 0.25745670782379765, 0.2000536144695963, 0.22425266680698303, 0.23014550251380553, 0.220092598326845, 0.2333470578992303, 0.2397850451132213, 0.19399565464742646, 0.23803109774742065, 0.22421967252668412, 0.25685211651055506, 0.252019727072377, 0.18443235480617823, 0.2352243594653112, 0.22879343402236685, 0.21209126799921676, 0.20778305360259505, 0.22574378429182548, 0.21765387987438306, 0.20680414971912345, 0.2166011374987049, 0.20271220572878582, 0.2060363068246171, 0.21441362041712347, 0.22621987646015293, 0.2348257755221279, 0.21415746226116678, 0.28177989168949885, 0.24618466934614144, 0.24238379845879346, 0.21810325898402289, 0.2072734695507949, 0.20453704576356804, 0.21890247076783192, 0.18480952983229337, 0.19724001149699655, 0.21273739259557678, 0.20727961004813744, 0.2298182676718433, 0.18740984348333656, 0.21300405755063961, 0.2032279768575969, 0.2779299757801714, 0.20559684152423163, 0.21068578741505423, 0.2329658181695377, 0.1970775102684622, 0.2039196937912763, 0.2156885850833507, 0.22315002309935278, 0.1957906262970726, 0.22191050589076214, 0.2111783744286228, 0.23937308114149686, 0.1850500338524835, 0.24302937483459297, 0.21401904302338448, 0.23090226081358728, 0.22435428391511253, 0.24550864823882204, 0.20194816671087387, 0.21189354610700706, 0.2147423670260836, 0.19717146533982968, 0.22620186039925033, 0.21309545763778107, 0.19713791715348367, 0.22968376536409824, 0.23681713590743322, 0.23736861221085268, 0.21823794390182305, 0.2520288219738873, 0.220687137992656, 0.2050746261374822, 0.24767955288591192, 0.21359122151746454, 0.21345162025763398, 0.23112091038303256, 0.22404508082429916, 0.21613670323270046, 0.21849627654319617, 0.20294861463076966, 0.2027591353655109, 0.18770516330785988, 0.19632715749276108, 0.23051169608670252, 0.24459087303688, 0.21670864802054068, 0.21808973884317367, 0.20574237198872314, 0.20691730443731254, 0.23761069645675037, 0.23056784471051844, 0.2240369759719738, 0.21260932796247714, 0.23414119077178755, 0.2282324262816369, 0.22330022748023193, 0.23039040103124944, 0.21151639297684632, 0.2197264911666033, 0.19395501502582985, 0.18769779003558798, 0.24804310274333535, 0.23122257268648355, 0.20633476224410016, 0.20838434211168555, 0.18520833396863826, 0.25368818235248725, 0.19483839308921772, 0.23610045080691877, 0.22911356924467366, 0.201301788469217, 0.23261031434256244, 0.21522336787659574, 0.23354736558218958, 0.20678726523845167, 0.20719899876749154, 0.21437456864604817, 0.2386397289046762, 0.22610192196966547, 0.21343009667268747, 0.22050904095730653, 0.18787403605028155, 0.20960199077056757, 0.220537403123026, 0.22592932027669665, 0.24016402515699914, 0.22022454523178212, 0.21120715617513203, 0.2162326361790664, 0.23081709231875963, 0.2370925314056181, 0.20877681583756347, 0.23239600088479326, 0.21493040773349115, 0.2153795538196926, 0.2344717654582246, 0.2102365744746247, 0.23328896293150558, 0.21014533147380107, 0.21652379929316218, 0.2199638731408313, 0.21818776317675367, 0.24578798275409852, 0.20694128581189597, 0.2583046713845929, 0.2349716295095989, 0.2149588311903041, 0.2184209097899601, 0.21672475127848814, 0.227951976829503, 0.21590791320082248, 0.229139766208067, 0.2399303997291103, 0.2236987253297057, 0.2235835624961756, 0.1959378819369115, 0.237445634825418, 0.2294739358674597, 0.22042433244527784, 0.23971293089643125, 0.2326681769641182, 0.2139417941612655, 0.20421806658430527, 0.2338716270086777, 0.2215621546741995, 0.2269553782228263, 0.24157168757207328, 0.2351417375710388, 0.19390648248031708, 0.22725314994231738, 0.20139286857586866, 0.20425248320065542, 0.18888505858465973, 0.23258170883459892, 0.20824525603093205, 0.2103538510053125, 0.23232242817733886, 0.22342127493525202, 0.19105382631634465, 0.2342169737305444, 0.21428265653333964, 0.24215505433043377, 0.22041037353030923, 0.22831276922779878, 0.21782106618555017, 0.20998233969504365, 0.22074925282165114, 0.21781255313514689, 0.22272964083015942, 0.22787845457921782, 0.21978092801283858, 0.21591717919157977, 0.22651873771410314, 0.2539588948923793, 0.22197946618094336, 0.19743223058977255, 0.22235942896322447, 0.21964959106069193, 0.251829065185799, 0.26069548170443585, 0.22606175804084197, 0.22795903138260287, 0.2348265431783543, 0.23533586368389992, 0.23945992309051364, 0.25260275777855906, 0.20528403474284077, 0.21196495139776475, 0.2315160690256956, 0.209652758223733, 0.22172928394206443, 0.194837039882931, 0.24378097659574569, 0.22120858621386547, 0.19773842992328258, 0.23019794626230417, 0.2168888837664674, 0.2284001239350056, 0.22832016772079358, 0.23509869244346746, 0.20056392387057073, 0.20917833554013224, 0.2168579392461326, 0.2353353646853364, 0.22570548262067416, 0.23115950086361636, 0.22799829152951198, 0.20942110052326568, 0.2012200854350201, 0.2044250796262406, 0.2202275744700749, 0.21375525002619428, 0.21543906118546308, 0.20255883027900265, 0.22915864901994998, 0.19894278355440786, 0.20810448635283868, 0.22359509299484756, 0.21094945864608847, 0.227226687991105, 0.21172117446153751, 0.202678660556657, 0.24044548365269874, 0.2070306651945933, 0.2072048852691976, 0.2212968711919958, 0.20699893896371796, 0.23635958035299862, 0.22296671300087845, 0.21369394810102532, 0.2279429774373476, 0.21656209574161353, 0.2380252370191696, 0.16504738590936743, 0.22646658600951405, 0.213999585914417, 0.25339606009584514, 0.22266543153896795, 0.24467983945494615, 0.2303984044100046, 0.2461087582058004, 0.2330765656559605, 0.2014292738485666, 0.20561843656774742, 0.21718518426758568, 0.2304135692856551, 0.2077431574734716, 0.20493869997461167, 0.23159307442604807, 0.1955926735712342, 0.1891379886089667, 0.22978711693679613, 0.2218902884239784, 0.1970415647971468, 0.21836074814851086, 0.19940430820816596, 0.23230847659675036, 0.22667831827948923, 0.18511217493584115, 0.20563242699341316, 0.1937719424506984, 0.2125436942820512, 0.21546851246570514, 0.21257146073840175, 0.17626969099504383, 0.20400509293111754, 0.23780127892531372, 0.2282581096645111, 0.207879750373144, 0.21320668961979988, 0.22063316929408872, 0.22037617241959265, 0.23245320852246432, 0.20745434024621096, 0.24641960794033288, 0.2156151844399228, 0.2346688047203553, 0.2072703171316693, 0.22089630117837256, 0.21027828393563192, 0.19383373210079452, 0.20336671216153746, 0.2182635734198279, 0.21554564827934622, 0.2030936692998735, 0.24333798869742815, 0.22471539691853368, 0.21783411798302918, 0.2329688478852065, 0.2299929379994385, 0.2507734375973864, 0.21702439931245934, 0.21379687238433198, 0.21724907706153498, 0.21981621132801102, 0.210797633862743, 0.20394056402451227, 0.2104994124020273, 0.21243432422754327, 0.19098502957596697, 0.24804924884160068, 0.2100474066813316, 0.1977701056838262, 0.24200667539058457, 0.23180599689159556, 0.22567422440865434, 0.21277383593172688, 0.2378136455166896, 0.21078240378781027, 0.2289106966562777, 0.2182406722408963, 0.24925776497357904, 0.20399662100107943, 0.2250441758259294, 0.2036563899517325, 0.23274418295976199, 0.22918034515183505, 0.2148615943655915, 0.2168278607823401, 0.20159651820006247, 0.2104792644948681, 0.2201856256435601, 0.2286154635908991, 0.21206152570407988, 0.248190463824382, 0.22037752397461713, 0.17784497793479095, 0.20676897351148252, 0.21582865173105506, 0.24446428687125665, 0.21119285100095836, 0.2215757993385138, 0.21932850183269367, 0.21960539154351458, 0.19088558377029605, 0.22448026478746885, 0.22178873155987305, 0.2166874289240229, 0.18539738316653925, 0.233460632197047, 0.21678952305582394, 0.199269472071232, 0.1910093305069282, 0.2109850562548021, 0.22469777062181595, 0.21592576095048813, 0.2464918730172274, 0.229352162281112, 0.22368863964775926, 0.2188677757427725, 0.21523796530123077, 0.18048583781141397, 0.2050320787006229, 0.24500169851991266, 0.22037084568749332, 0.2314869773386769, 0.2319580362174356, 0.23440322181768578, 0.2358744496850244, 0.20806134732235484, 0.20439064233690646, 0.22120622047538488, 0.21947722996621247, 0.18800985271417164, 0.23543531663504272, 0.2708377903043288, 0.2027780972598473, 0.20200253225316503, 0.19644655908700692, 0.206059561422495, 0.22493524418960661, 0.22008409408186272, 0.2655934493616694, 0.2056365987057042, 0.25219339104151256, 0.2112814950841695, 0.23690877893835738, 0.22603061013802378, 0.20610290335246964, 0.22181142302807444, 0.2109737262782224, 0.19550200366172504, 0.23757060295344387, 0.19440567713836687, 0.2343307021964973, 0.2132268469862263, 0.21027009305099104, 0.24730646023979985, 0.21488972618987776, 0.218942491819231, 0.20991498549372806, 0.2082205956758999, 0.20210193813851676, 0.22688448361915137, 0.2185503293086961, 0.22886850002619927, 0.2090816356864924, 0.21718368181021425, 0.23991921191507715, 0.2182887308208537, 0.211517852985377, 0.21365151643773128, 0.20102109230733112, 0.20583102723407193, 0.18814891021274238, 0.20210409435487398, 0.1910825091953054, 0.2000637036112551, 0.23980068482269454, 0.19801817205303446, 0.21601439406924414, 0.23504844747646722, 0.1958909559243279, 0.2579729172887744, 0.22009872824084695, 0.20070597623411124, 0.2274935982675693, 0.21129990979561972, 0.22199165685170172, 0.21233672828150665, 0.24287330534753, 0.25364772026694704, 0.22200323324526408, 0.2548908503106536, 0.21851531611945482, 0.22168107231719858, 0.1887374139391475, 0.21806497203357592, 0.21087324741061594, 0.19407441777912288, 0.24507024510475114, 0.19305266888775552, 0.19677634095403362, 0.23034606438275435, 0.2010440998217542, 0.21189975249140466, 0.20533209577616854, 0.22765604056866937, 0.21310985847592429, 0.22794354331414604, 0.2193409252811101, 0.22046967679906912, 0.21455715076715265, 0.21049146997174584, 0.18869095877159853, 0.22664214259696036, 0.21944073759464025, 0.18487235333202934, 0.2202620606932951, 0.20271356594375164, 0.22375362501654386, 0.2168691512681573, 0.22323750084921012, 0.22695494695449903, 0.19246631039363393, 0.22003259176111015, 0.20407061842451804, 0.22414188523095976, 0.22890652003457543, 0.2215173014583692, 0.20010710222718844, 0.240626763677673, 0.22904786212219558, 0.21104936731314505, 0.21631985829123399, 0.23847899126376756, 0.23584202423467965, 0.21508359666103016, 0.22019253851179957, 0.21952223694341436, 0.22524335044725782, 0.2034297530384983, 0.22968681520883424, 0.1989710929693651, 0.21931663055271794, 0.22650562684348358, 0.20265432629356345, 0.22931308189762067, 0.2477203406231703, 0.22027435174231474, 0.20313375476405007, 0.21038540049889123, 0.19412477916783996, 0.2114707660102353, 0.22566359942610356, 0.22118794070403328, 0.22185935928883518, 0.21526392928123517, 0.21856419621927908, 0.21834520213994416, 0.2118488638725286, 0.19334183839920524, 0.24049174606405585, 0.22210923283220715, 0.19480099519519847, 0.22020466484249457, 0.21923239044870738, 0.21602778591142335, 0.185166273400354, 0.23522552868254915, 0.22300763375130264, 0.2240150567288443, 0.21172248696003668, 0.22026148331658268, 0.18779602316159097, 0.23103154230394185, 0.19773462572576592, 0.18854901656520423, 0.16251552260381727, 0.26078223120847804, 0.21423745608539216, 0.19053253132464884, 0.22654782138351082, 0.22523828313802968, 0.21069798048302935, 0.23657625943594476, 0.25255600209898554, 0.21882545439706236, 0.20961325188642907, 0.18897204535742163, 0.227719765852365, 0.20749423367372868, 0.2592927043332379, 0.2213246817546316, 0.20826104682993662, 0.2324281311730036, 0.20634063737247757, 0.21022690932706026, 0.20606226233705247, 0.21775114753584332, 0.20671127292858915, 0.21479867398244895, 0.1992048519807852, 0.22018032955553749, 0.21147002902828285, 0.22936645804155822, 0.19425410619377664, 0.2047156758182226, 0.2326296429410786, 0.22230557276843302, 0.21588178651289996, 0.21230664910355823, 0.21640522367988202, 0.21941336965314115, 0.24246012609824788, 0.22770299941600755, 0.230987101414731, 0.20856381923693576, 0.22385683234378617, 0.21258317627474746, 0.2148300474728191, 0.24800280630455843, 0.22334717946312702, 0.206448729641661, 0.2283618439672712, 0.22571335270324702, 0.22018402442498422, 0.17869414996406896, 0.2097801009005452, 0.21626485772996773, 0.23759271466382803, 0.24022591190905354, 0.2066085748651435, 0.20883353636370125, 0.21007175242633824, 0.24827888199724818, 0.20483198414669954, 0.20130602293353173, 0.22872142261624281, 0.21706912016790184, 0.210833182024924, 0.21982957346220935, 0.22808538868641257, 0.23750662488344812, 0.19834277141061146, 0.21720957609109356, 0.2080288570609568, 0.22115745649829097, 0.2127795069366646, 0.23890866987980142, 0.24303919066850643, 0.22191401711741093, 0.21820788258943793, 0.2401645969287459, 0.23386429570558256, 0.1906279793670924, 0.2131682356901909, 0.19704111641590735, 0.2256413757158235, 0.18368367156083495, 0.2257135164568223, 0.20352906968840906, 0.21006451986818436, 0.2454093369273191, 0.26124980850088236, 0.22284860678487922, 0.20009807744815336, 0.21662028615460552, 0.21204516206412452, 0.2517588476433745, 0.19938130817539065, 0.2165456577455158, 0.21228113855973826, 0.21299533303272417, 0.2472579070654209, 0.2300811537014916, 0.21091365161686013, 0.20355396114959953, 0.23225964717233427, 0.20046794934119805, 0.20057166143165853, 0.20221434942977173, 0.21896590496480878, 0.25416417483763604, 0.19053328258302552, 0.22610637937399228, 0.19802497893194124, 0.2344558101316205, 0.21530027629138995, 0.2346314915388441, 0.2162477306506007, 0.21139841253590494, 0.24468587476321388, 0.23834784187479643, 0.20210516857749056, 0.21855995675105033, 0.19676183183797435, 0.22038941841748955, 0.20277091627737867, 0.22189981352757607, 0.2188541593530664, 0.22252410058082842, 0.19081954227389433, 0.22035959189790683, 0.2182139392604063, 0.20777494062558244, 0.20426934073061243, 0.228923714380939, 0.228179624549634, 0.217133062021013, 0.22885208837260826, 0.22140634995711334, 0.2353496808629057, 0.1891659738780266, 0.2420981814513372, 0.21723757430410928, 0.19891912544134113, 0.22601768980596804, 0.23916133173823723, 0.2274035520449086, 0.20278663224416082, 0.2316788745745573, 0.24291554285298095, 0.22533383354742728, 0.1884016328745539, 0.22618892309560054, 0.25153097476225894, 0.2265134170291209, 0.2135982421478112, 0.19609933312285197, 0.21803136002007684, 0.24332605104386437, 0.2109646033900659, 0.21116348176508717, 0.22934127512686134, 0.2609209061185721, 0.24309639113244197, 0.23611180388642902, 0.18899312704458113, 0.20380913828451747, 0.2228247747802587, 0.21280399397417155, 0.22834932102286123, 0.21007779180417163, 0.2145043536012499, 0.22619389997587203, 0.22876075782009309, 0.21428984664003284, 0.20355971965715125, 0.21443773645666175, 0.2125312555558592, 0.2430700197514813, 0.21347198194205697, 0.21145961938587715, 0.20556952429721945, 0.20142482851640334, 0.2548271828454734, 0.23881036637973535, 0.21062966662783347, 0.20954999352717774, 0.2016776983539238, 0.20957317933697395, 0.21272537211451065, 0.2535509843472934, 0.21636492536595542, 0.201035606269263]
mean_norm_big_bagging = 0.19851195476445752
sd_norms_big_bagging = np.std(norms_big_bagging)
z_statistic_big_bagging = mean_norm_big_bagging / (sd_norms_big_bagging / m2)
print(z_statistic_big_bagging)

# Testing Structural Breaks

The **Chow test** is a test of structural change. Essentially it ask the question; are the parameters the same in two subgroups of the population of data? Is the equation different in either of the subgroups? It takes a break point in the data and tests if the parameter for the binary variable that specifies before-after the break, is actually statistically significantly different from 0. That is, for a VAR(1) model, let $\tau$ be a known break date and $D_t(\tau)$ be a binary variable indicating periods before and after the break;

$Y_t = \Phi Y_{t-1} + D_t(\tau) \gamma_0 + D_t(\tau) \gamma_1 Y_{t-1} + \epsilon_t$

The null hypothesis of the Chow test of no structural break then is:

$H_0: \gamma_0 = \gamma_1 = 0$

That means that there is no significant parameter change at the break date $\tau$. However if either is not zero, then this might be an indication of the time series experencing some kind of discrete or gradual change in the coefficients. These changes are then called _breaks_. 

The **Quandt Likelihood Ratio (QLR) test** is an extension of the Chow test in which the break date $\tau$ is not known. Rather it is assumed to be in some range $[\tau_0, \tau_1]$. The QLR test uses the largest F-statistic of all from performing the Chow test for all possible break dates $\tau$ within that range. Thus this test repeats the Chow test and then takes the largest F-statistic as truth. (Schmelzer, 2024)

The **F-statistic** that is used in both tests is inherited from the Wald test.

## Chow Test

_Note_: the documentation on the Chow Test python module is not up to date. None of the sites that come up have the right scripts to import or call the function!!! Therefore, the inspection below might be needed to see how the function actually works. It took me over an hour to correctly import and call the function. Below is the description of the function:

_Def chow_test(X_series: (pd.Series, pd.DataFrame), y_series: pd.Series, last_index: int, first_index: int,
              significance: float):
    """
    This function acts as the highest level of abstraction for the chow test. The function firstly checks that the
    input arguments are of the correct type, followed by calculating the sum of squared residuals for the entire data
    series, and the two sub-sets of data, as determined by the last_index and first_index arguments. The chow test is
    then computed and assessed against the significance argument. Finally, the chow_test value and p_value are returned
    from the function.:_

    :param: X_series: the series or set of series denoting the X variable. (pd.DataFrame)
    :param: y_series: the series denoting the y variable. (pd.Series)
    :param: last_index: the final index value to be included before the data split. (int)
    :param: first_index: the first index value to be included after the data split. (int)
    :param: significance_level: the significance level against which the p-value is assessed. (float)
    :return: chow_value: the chow test output value. (float)
    :return: p_value: the associated p-value for the chow test. (float)

In [ ]:
#print(dir(chow_test))
#help(chow_test)
#import inspect
#print(inspect.getsource(chow_test))

In [ ]:
# example usage!
df = pd.DataFrame({'x': [1, 1, 2, 3, 4, 4, 5, 5, 6, 7, 7, 8, 8, 9, 10, 10,
                         11, 12, 12, 13, 14, 15, 15, 16, 17, 18, 18, 19, 20, 20],
                   'y': [3, 5, 6, 10, 13, 15, 17, 14, 20, 23, 25, 27, 30, 30, 31,
                         33, 32, 32, 30, 32, 34, 34, 37, 35, 34, 36, 34, 37, 38, 36]})

chow_value, p_value = chow_test(X_series=df['x'], y_series=df['y'], last_index=15, first_index=16, significance=0.05)

### Visual inspection

Now the Chow test can be used to perform analysis on the structure of the data. What we would like to test is if the estimates for the parameter of the model change over time. Even if such a change, aka structural break, exists in the time series, an estimator like OLS would return the mean over time. Therefore it might be better to look at the parameter in two different time periods, splitting the data into two sets like in Schultz. Start the analysis by splitting the model into two variables: the dependent variable $Y_t$ and the independent variable $Y_{t-1}$ following the VAR(1) model (or any related model like SVAR or CVAR for that matter). Since we are only looking at one time lag, this is enough. So we get a dataset representing $Y_{t-1}$ that starts at $t=1950-01-31$ and continues until $t=2021-11-30$, and one representing $Y_t$ that starts a period later at $t=1950-02-28$ and continues until the last period $t=2021-12-31$.

In [ ]:
df_extended_data_dsdt_Yt = df_extended_data_dsdt.loc['1950-02-28':'2021-12-31']  # As dependent variable
df_extended_data_dsdt_Ylag = df_extended_data_dsdt.loc['1950-01-31':'2021-11-30'] # As independent variable

In [ ]:
print(df_extended_data_dsdt_Ylag)

In [ ]:
df_centered_Yt = df_extended_data_dsdt_Yt.sub(means_climate, axis=1)
df_centered_Ylag = df_extended_data_dsdt_Ylag.sub(means_climate, axis=1)
print(df_centered_Yt)

In [ ]:
print(means_climate)

These datasets can be examined visually by a scatter plot that shows the relations between the variable at time $t$ and its lag. Visually this can give some insights. For example, if a variable goes from one value on the x-axis to another value on the y-axis, it would be logical that it would end up somewhat on the same spot everytime. If this is not the case, something might be going on, like a big error term causing fluctuations or the underlying process that has changed somewhere in time causing the variable to behave differently the next time it reaches this state.

In [ ]:
variables = df_extended_data_dsdt_Yt.columns

colors = ['lightblue', 'palevioletred', 'orange', 'darkslateblue']

fig = sp.make_subplots(rows=2, cols=2, subplot_titles=variables)

for i, (var, color) in enumerate(zip(variables, colors)):
    row = i // 2 + 1
    col = i % 2 + 1
    fig.add_trace(go.Scatter(
        x=df_extended_data_dsdt_Ylag[var], 
        y=df_extended_data_dsdt_Yt[var],
        mode='markers',
        name=var,
        marker=dict(color=color)
    ), row=row, col=col)

fig.update_layout(
    title='Scatter Plot of Climate Variables (x-axis = y_{t-1}, y-axis = y_t)',
    showlegend=False
)

#for i, var in enumerate(variables):
 #   fig['layout'][f'xaxis{i+1}']['title'] = f'Lagged {var}'
 #   fig['layout'][f'yaxis{i+1}']['title'] = f'Current {var}'

fig.show()

In [ ]:
def PlotScatter(df, df_lag):
    """
    This function is to plot the data easily and to include the time dimension in the plots by coloring the scatter dots in different shades for the time.
    """
    variables = df.columns
    lag_variables = df_lag.columns

    fig = sp.make_subplots(rows=2, cols=2, subplot_titles=variables)

    for i, var in enumerate(variables):
        row = i // 2 + 1
        col = i % 2 + 1

        # Normalize time for color scale
        time_norm = (df.index - df.index.min()) / (df.index.max() - df.index.min())

        fig.add_trace(go.Scatter(
            x=df_lag[var], 
            y=df[var],
            mode='markers',
            name=var,
            marker=dict(color=time_norm, colorscale='Sunset', showscale=(i == 0), opacity=0.6),
            text=df.index.strftime('%Y-%m-%d')  # Display date on hover
        ), row=row, col=col)

    fig.update_layout(
        title='Scatter Plot of Climate Variables',
        showlegend=False,
        height=1200,  # Adjust height to accommodate subplots
        width=1600   # Adjust width to accommodate subplots
    )

    num_ticks = 5
    tickvals = np.linspace(0, 1, num_ticks)
    ticktexts = pd.date_range(df.index.min(), df.index.max(), periods=num_ticks).strftime('%Y-%m-%d')

    fig.update_traces(marker=dict(colorbar=dict(
        title="Time",
        tickvals=tickvals,
        ticktext=ticktexts
    )))
    
    fig.show()

In [ ]:
PlotScatter(df_extended_data_dsdt_Yt, df_extended_data_dsdt_Ylag)

In [ ]:
PlotScatter(df_centered_Yt, df_centered_Ylag)

In [ ]:
def PlotScatterSeaborn(df, df_lag):
    """
    This function is to plot the data easily and to include the time dimension in the plots by coloring the scatter dots in different shades for the time.
    """
    variables = df.columns
    lag_variables = df_lag.columns

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()

    # Normalize time for color scale
    time_norm = (df.index - df.index.min()) / (df.index.max() - df.index.min())

    for i, var in enumerate(variables):
        ax = axes[i]
        sc = ax.scatter(df_lag[var], df[var], c=time_norm, cmap='Spectral', alpha=0.6)
        ax.set_title(var)
        ax.set_xlabel(f'{var} Lagged')
        ax.set_ylabel(var)

    # Colorbar
    cbar = fig.colorbar(sc, ax=axes, orientation='horizontal', fraction=0.02, pad=0.1)
    num_ticks = 5
    tickvals = np.linspace(0, 1, num_ticks)
    ticktexts = pd.date_range(df.index.min(), df.index.max(), periods=num_ticks).strftime('%Y-%m-%d')
    cbar.set_ticks(tickvals)
    cbar.set_ticklabels(ticktexts)
    cbar.set_label('Time')

    plt.suptitle('Scatter Plot of Climate Variables', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
PlotScatterSeaborn(df_extended_data_dsdt_Yt, df_extended_data_dsdt_Ylag)

In [ ]:
def PlotRegressionScatterPlots(df, df_lag):
    """
    This function plots the variables based on the regression y_t = β1 * WPAC_{t-1} + β2 * CPAC_{t-1} + β3 * EPAC_{t-1} + β4 * TW_{t-1} + ε.
    The plots include the time dimension in colors for better visualization.
    """
    variables = df.columns
    lag_variables = df_lag.columns

    fig = sp.make_subplots(rows=2, cols=2, subplot_titles=variables)

    for i, var in enumerate(variables):
        row = i // 2 + 1
        col = i % 2 + 1

        # Normalize time for color scale
        time_norm = (df.index - df.index.min()) / (df.index.max() - df.index.min())

        fig.add_trace(go.Scatter(
            x=df_lag['WPAC'], 
            y=df[var],
            mode='markers',
            name=f'{var} vs WPAC_{lag_variables[0]}',
            marker=dict(color=time_norm, colorscale='Sunset', showscale=(i == 0), opacity=0.6),
            text=df.index.strftime('%Y-%m-%d')  # Display date on hover
        ), row=row, col=col)

        fig.add_trace(go.Scatter(
            x=df_lag['CPAC'], 
            y=df[var],
            mode='markers',
            name=f'{var} vs CPAC_{lag_variables[1]}',
            marker=dict(color=time_norm, colorscale='Sunset', showscale=(i == 0), opacity=0.6),
            text=df.index.strftime('%Y-%m-%d')  # Display date on hover
        ), row=row, col=col)

        fig.add_trace(go.Scatter(
            x=df_lag['EPAC'], 
            y=df[var],
            mode='markers',
            name=f'{var} vs EPAC_{lag_variables[2]}',
            marker=dict(color=time_norm, colorscale='Sunset', showscale=(i == 0), opacity=0.6),
            text=df.index.strftime('%Y-%m-%d')  # Display date on hover
        ), row=row, col=col)

        fig.add_trace(go.Scatter(
            x=df_lag['TW'], 
            y=df[var],
            mode='markers',
            name=f'{var} vs TW_{lag_variables[3]}',
            marker=dict(color=time_norm, colorscale='Sunset', showscale=(i == 0), opacity=0.6),
            text=df.index.strftime('%Y-%m-%d')  # Display date on hover
        ), row=row, col=col)

    fig.update_layout(
        title='Scatter Plot of Variables According to Regression y_t = β1 * WPAC_{t-1} + β2 * CPAC_{t-1} + β3 * EPAC_{t-1} + β4 * TW_{t-1} + ε',
        showlegend=False,
        height=1200,  # Adjust height to accommodate subplots
        width=1600   # Adjust width to accommodate subplots
    )

    num_ticks = 5
    tickvals = np.linspace(0, 1, num_ticks)
    ticktexts = pd.date_range(df.index.min(), df.index.max(), periods=num_ticks).strftime('%Y-%m-%d')

    fig.update_traces(marker=dict(colorbar=dict(
        title="Time",
        tickvals=tickvals,
        ticktext=ticktexts
    )))
    
    fig.show()

In [ ]:
PlotRegressionScatterPlots(df_extended_data_dsdt_Yt, df_extended_data_dsdt_Ylag)

In [ ]:
def PlotScatterPlots(df, df_lag):
    """
    This function is simply to plot the data easily and to choose whether or not to include the OLS line.
    """
    variables = df.columns
    lag_variables = df_lag.columns

    colors = ['lightblue', 'palevioletred', 'orange', 'darkslateblue']
    color_count = len(colors)
    
    fig = sp.make_subplots(rows=len(variables), cols=len(lag_variables),
                           subplot_titles=[f"Current {v} vs lagged {lv}" for v in variables for lv in lag_variables]) #, vertical_spacing=0.01, horizontal_spacing=0.01

    for i, var in enumerate(variables):
        for j, lag_var in enumerate(lag_variables):
            row = i + 1
            col = j + 1
            color = colors[(i * len(lag_variables) + j) % color_count]
            fig.add_trace(go.Scatter(
                x=df_lag[lag_var], 
                y=df[var],
                mode='markers',
                name=f'Current {var} vs lagged {lag_var}',
                marker=dict(color=color)
            ), row=row, col=col)
    
    fig.update_layout(
        title='Scatter Plot of Climate Variables',
        showlegend=False,
        height=250 * len(variables),  
        width=350 * len(lag_variables)   
    )

    fig.show()

In [ ]:
def PlotScatterPlots(df, df_lag):
    """
    This function is simply to plot the data easily and to include the time dimension in the plots by coloring the scatter dots in different shades for the time.
    """
    variables = df.columns
    lag_variables = df_lag.columns

    colors = ['lightblue', 'palevioletred', 'orange', 'darkslateblue']
    color_count = len(colors)
    
    fig = sp.make_subplots(rows=len(variables), cols=len(lag_variables),
                           subplot_titles=[f"Current {v} vs lagged {lv}" for v in variables for lv in lag_variables])

    for i, var in enumerate(variables):
        for j, lag_var in enumerate(lag_variables):
            row = i + 1
            col = j + 1
            color = colors[(i * len(lag_variables) + j) % color_count]

            # Normalize time for color scale
            time_norm = (df.index - df.index.min()) / (df.index.max() - df.index.min())

            fig.add_trace(go.Scatter(
                x=df_lag[lag_var], 
                y=df[var],
                mode='markers',
                name=f'Current {var} vs lagged {lag_var}',
                marker=dict(color=time_norm, colorscale='Sunset', showscale=(i == 0 and j == 0), opacity = 0.6),
                text=df.index.strftime('%Y-%m-%d')  # Display date on hover
            ), row=row, col=col)
    
    fig.update_layout(
        title='Scatter Plot of Climate Variables',
        showlegend=False,
        height=2400, #250 * len(variables),  
        width=3200 #300 * len(lag_variables)
    )
    
    num_ticks = 5
    tickvals = np.linspace(0, 1, num_ticks)
    ticktexts = pd.date_range(df.index.min(), df.index.max(), periods=num_ticks).strftime('%Y-%m-%d')

    fig.update_traces(marker=dict(colorbar=dict(
        title="Time",
        tickvals=tickvals,
        ticktext=ticktexts
    )))
    
    fig.show()

In [ ]:
PlotScatterPlots(df_extended_data_dsdt_Yt, df_extended_data_dsdt_Ylag) 

In [ ]:
PlotScatterPlots(df_centered_Yt, df_centered_Ylag)

In [ ]:
def PlotScatterPlotsSeaborn(df, df_lag, color_map='Spectral'):
    """
    This function is simply to plot the data easily and to include the time dimension in the plots by coloring the scatter dots in different shades for the time.
    """
    variables = df.columns
    lag_variables = df_lag.columns

    colors = ['lightblue', 'palevioletred', 'orange', 'darkslateblue']
    color_count = len(colors)
    
    num_vars = len(variables)
    num_lags = len(lag_variables)
    
    fig, axes = plt.subplots(num_vars, num_lags, figsize=(7 * num_lags, 3 * num_vars))
    
    if num_vars == 1:
        axes = np.expand_dims(axes, axis=0)
    if num_lags == 1:
        axes = np.expand_dims(axes, axis=1)

    for i, var in enumerate(variables):
        for j, lag_var in enumerate(lag_variables):
            ax = axes[i, j]
            color = colors[(i * len(lag_variables) + j) % color_count]

            # Normalize time for color scale
            time_norm = (df.index - df.index.min()) / (df.index.max() - df.index.min())
            sc = ax.scatter(df_lag[lag_var], df[var], c=time_norm, cmap=color_map, alpha=0.6)
            ax.set_title(f'Current {var} vs lagged {lag_var}', fontsize=14)
            ax.set_xlabel(f'Lagged {lag_var}', fontsize=12)
            ax.set_ylabel(var, fontsize=12)

    # Colorbar
    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    cbar = fig.colorbar(sc, cax=cbar_ax, orientation='vertical')
    
    num_ticks = 5
    tickvals = np.linspace(0, 1, num_ticks)
    ticktexts = pd.date_range(df.index.min(), df.index.max(), periods=num_ticks).strftime('%Y-%m-%d')
    cbar.set_ticks(tickvals)
    cbar.set_ticklabels(ticktexts)
    cbar.set_label('Time', fontsize=12)

    plt.suptitle('Scatter Plot of Climate Variables', fontsize=16)
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    plt.show()

In [ ]:
PlotScatterPlotsSeaborn(df_extended_data_dsdt_Yt, df_extended_data_dsdt_Ylag, color_map='coolwarm') 

A second visualization can be done by subtracting the lagged $Y_{t-1}$ from the current $Y_t$. The difference should stay somewhat the same over time, assuming that the processes that influence the error terms also stay constant over time. What happens if we plot this? 

In [ ]:
df_Yt_reset = df_extended_data_dsdt_Yt.reset_index(drop=True)
df_Ylag_reset = df_extended_data_dsdt_Ylag.reset_index(drop=True)
df_extended_data_dsdt_diff = df_Yt_reset - df_Ylag_reset
df_extended_data_dsdt_diff.index = df_extended_data_dsdt_Yt.index

In [ ]:
fig = go.Figure()

for column in df_extended_data_dsdt_diff.columns:
    fig.add_trace(go.Scatter(
        x=df_extended_data_dsdt_diff.index,
        y=df_extended_data_dsdt_diff[column],
        mode='lines',
        name=column
    ))

fig.update_layout(
    title='Steps in values from one time period to another',
    xaxis_title='Time',
    yaxis_title='Difference Y_t and Y_{t-1}',
    legend_title='Variables'
)

fig.show()

### Module

Now we will use the Chow test as provided in the package to test for structural change in time periods.

In [ ]:
break_date = '1986-06-30' # The middle of the Climate Dataset
df_extended_data_dsdt.index = pd.to_datetime(df_extended_data_dsdt.index)
# Split the dataset into two parts based on the break date
first_half = df_extended_data_dsdt.loc[:break_date]
second_half = df_extended_data_dsdt.loc[break_date:]
# Define the last index of the early model and the first index of the late model
last_index = len(first_half) - 1
first_index = len(first_half)

#### 1 Test only

In [ ]:
# Applying the chow test on ALL DATA
#y_series = df_extended_data_dsdt_Yt['WPAC', 'CPAC', 'EPAC', 'TW'].values.flatten()
#X_series = df_extended_data_dsdt_Ylag['WPAC', 'CPAC', 'EPAC', 'TW'].values.flatten()
#chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
#print(f"Chow test for overall model: chow_value = {chow_value}, p_value = {p_value}")

#### 4 Seperate tests

In [ ]:
# Applying the chow test on WPAC
y_series = df_extended_data_dsdt_Yt['WPAC']
X_series = df_extended_data_dsdt_Ylag
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
y_series = df_centered_Yt['WPAC']
X_series = df_centered_Ylag
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on CPAC
y_series = df_extended_data_dsdt_Yt['CPAC']
X_series = df_extended_data_dsdt_Ylag
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on EPAC
y_series = df_extended_data_dsdt_Yt['EPAC']
X_series = df_extended_data_dsdt_Ylag
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on TW
y_series = df_extended_data_dsdt_Yt['TW']
X_series = df_extended_data_dsdt_Ylag
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

#### 16 Seperate tests

WPAC as current

In [ ]:
# Applying the chow test on WPAC 
y_series = df_extended_data_dsdt_Yt['WPAC']
X_series = df_extended_data_dsdt_Ylag['WPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on WPAC 
y_series = df_extended_data_dsdt_Yt['WPAC']
X_series = df_extended_data_dsdt_Ylag['CPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on WPAC 
y_series = df_extended_data_dsdt_Yt['WPAC']
X_series = df_extended_data_dsdt_Ylag['EPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on WPAC 
y_series = df_extended_data_dsdt_Yt['WPAC']
X_series = df_extended_data_dsdt_Ylag['TW']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

CPAC as current

In [ ]:
# Applying the chow test on CPAC 
y_series = df_extended_data_dsdt_Yt['CPAC']
X_series = df_extended_data_dsdt_Ylag['WPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on CPAC 
y_series = df_extended_data_dsdt_Yt['CPAC']
X_series = df_extended_data_dsdt_Ylag['CPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on CPAC 
y_series = df_extended_data_dsdt_Yt['CPAC']
X_series = df_extended_data_dsdt_Ylag['EPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on CPAC 
y_series = df_extended_data_dsdt_Yt['CPAC']
X_series = df_extended_data_dsdt_Ylag['TW']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

EPAC as current

In [ ]:
# Applying the chow test on EPAC 
y_series = df_extended_data_dsdt_Yt['EPAC']
X_series = df_extended_data_dsdt_Ylag['WPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on EPAC 
y_series = df_extended_data_dsdt_Yt['EPAC']
X_series = df_extended_data_dsdt_Ylag['CPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on EPAC 
y_series = df_extended_data_dsdt_Yt['EPAC']
X_series = df_extended_data_dsdt_Ylag['EPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on EPAC 
y_series = df_extended_data_dsdt_Yt['EPAC']
X_series = df_extended_data_dsdt_Ylag['TW']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

TW as current

In [ ]:
# Applying the chow test on TW 
y_series = df_extended_data_dsdt_Yt['TW']
X_series = df_extended_data_dsdt_Ylag['WPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on TW 
y_series = df_extended_data_dsdt_Yt['TW']
X_series = df_extended_data_dsdt_Ylag['CPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on TW 
y_series = df_extended_data_dsdt_Yt['TW']
X_series = df_extended_data_dsdt_Ylag['EPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on TW 
y_series = df_extended_data_dsdt_Yt['TW']
X_series = df_extended_data_dsdt_Ylag['TW']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

### Manual

The Chow test can also be manually done to ensure our understanding of it and use all the data at once. For this, use a VAR model, estimate the parameters and calculate the residuals. Take the sum of the squared residuals (SSR) and save this. This will be the SSRc, or the sum of squared residuals for the complete model. Then also split the dataset up into two parts to compute the SSRs for both parts. We ahve the option to inspect it for every variable seperately or to use the sum of all SSRs. Then the F-statistic is computed and the null hypothesis is tested.

In [ ]:
# Define the break point
T_break = '1986-06-30'  
df_extended_data_dsdt_early = df_extended_data_dsdt.loc[:T_break]
df_extended_data_dsdt_late = df_extended_data_dsdt.loc[T_break:]

In [ ]:
# Complete Model
model = VAR(df_extended_data_dsdt)
results = model.fit(maxlags=1)  # Assuming a VAR(1) model
SSRc = np.sum(results.resid**2, axis=0) # SSR for each variable seperately
SSRcsum = np.sum(SSRc)  # SSR over all variables 

In [ ]:
# Early Model
model1 = VAR(df_extended_data_dsdt_early)
results1 = model1.fit(maxlags=1)
SSR1 = np.sum(results1.resid**2, axis=0) # SSR for each variable seperately
SSR1sum = np.sum(SSR1) # SSR over all variables 

In [ ]:
# Late Model
model2 = VAR(df_extended_data_dsdt_late)
results2 = model2.fit(maxlags=1)
SSR2 = np.sum(results2.resid**2, axis=0) # SSR for each variable seperately
SSR2sum = np.sum(SSR2) # SSR over all variables 

In [ ]:
# Sum it all up
sum_SSRs = SSR2 + SSR1
sum_SSRsums = SSR2sum + SSR1sum

In [ ]:
k = len(results.params)
print(k)
n_early = len(df_extended_data_dsdt_early)
n_late = len(df_extended_data_dsdt_late)
if n_early == n_late:
    print("Evenly split :)")

# Compute the Chow test statistic
F_wpac = ((SSRc['WPAC'] - (SSR1['WPAC'] + SSR2['WPAC'])) / k) / ((SSR1['WPAC'] + SSR2['WPAC']) / (n1 + n2 - 2 * k))
F_cpac = ((SSRc['CPAC'] - (SSR1['CPAC'] + SSR2['CPAC'])) / k) / ((SSR1['CPAC'] + SSR2['CPAC']) / (n1 + n2 - 2 * k))
F_epac = ((SSRc['EPAC'] - (SSR1['EPAC'] + SSR2['EPAC'])) / k) / ((SSR1['EPAC'] + SSR2['EPAC']) / (n1 + n2 - 2 * k))
F_tw = ((SSRc['TW'] - (SSR1['TW'] + SSR2['TW'])) / k) / ((SSR1['TW'] + SSR2['TW']) / (n1 + n2 - 2 * k))
F_sums = ((SSRcsum - (SSR1sum + SSR2sum)) / k) / ((SSR1sum + SSR2sum) / (n1 + n2 - 2 * k))

# Critical value for F-distribution using the package
critical_value = f.ppf(1 - 0.05, k, n1 + n2 - 2 * k)  # <--- change 0.05 if we want a different alpha!

if F_wpac > critical_value:
    print("Reject the null hypothesis: Structural change detected in WPAC")
else:
    print("Fail to reject the null hypothesis: No structural change detected in WPAC")

if F_cpac > critical_value:
    print("Reject the null hypothesis: Structural change detected in CPAC")
else:
    print("Fail to reject the null hypothesis: No structural change detected in CPAC")
    
if F_epac > critical_value:
    print("Reject the null hypothesis: Structural change detected in EPAC")
else:
    print("Fail to reject the null hypothesis: No structural change detected in EPAC")

if F_tw > critical_value:
    print("Reject the null hypothesis: Structural change detected in TW")
else:
    print("Fail to reject the null hypothesis: No structural change detected in TW")

if F_sums > critical_value:
    print("Reject the null hypothesis: Structural change detected in general")
else:
    print("Fail to reject the null hypothesis: No structural change detected in general")

### For the simulation set

In [ ]:
df_Yt = dfY.loc['1950-02-28':'2021-12-31']  # As dependent variable
df_Ylag = dfY.loc['1950-01-31':'2021-11-30'] # As independent variable

In [ ]:
# Split the dataset into two parts based on the break date
first_half = dfY.loc[:break_date]
second_half = dfY.loc[break_date:]
# Define the last index of the early model and the first index of the late model
last_index = len(first_half) - 1
first_index = len(first_half)

In [ ]:
# Applying the chow test on WPAC 
y_series = df_Yt['WPAC']
X_series = df_Ylag['WPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on WPAC 
y_series = df_Yt['CPAC']
X_series = df_Ylag['CPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on WPAC 
y_series = df_Yt['EPAC']
X_series = df_Ylag['EPAC']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# Applying the chow test on WPAC 
y_series = df_Yt['TW']
X_series = df_Ylag['TW']
chow_value, p_value = chow_test(X_series, y_series, last_index, first_index, significance=0.05)
print(f"Chow Test Value: {chow_value}, p-value: {p_value}")

In [ ]:
# AND NOW MANUALLY
# Define the break point
dfY_early = dfY.loc[:T_break]
dfY_late = dfY.loc[T_break:]
# Complete Model
model = VAR(dfY)
results = model.fit(maxlags=1)  # Assuming a VAR(1) model
SSRc = np.sum(results.resid**2, axis=0) # SSR for each variable seperately
SSRcsum = np.sum(SSRc)  # SSR over all variables 
# Early Model
model1 = VAR(dfY_early)
results1 = model1.fit(maxlags=1)
SSR1 = np.sum(results1.resid**2, axis=0) # SSR for each variable seperately
SSR1sum = np.sum(SSR1) # SSR over all variables 
# Late Model
model2 = VAR(dfY_late)
results2 = model2.fit(maxlags=1)
SSR2 = np.sum(results2.resid**2, axis=0) # SSR for each variable seperately
SSR2sum = np.sum(SSR2) # SSR over all variables 
# Sum it all up
sum_SSRs = SSR2 + SSR1
sum_SSRsums = SSR2sum + SSR1sum
k = len(results.params)

n_early = len(dfY_early)
n_late = len(dfY_late)
if n_early == n_late:
    print("Evenly split :)")

# Compute the Chow test statistic
F_wpac = ((SSRc['WPAC'] - (SSR1['WPAC'] + SSR2['WPAC'])) / k) / ((SSR1['WPAC'] + SSR2['WPAC']) / (n1 + n2 - 2 * k))
F_cpac = ((SSRc['CPAC'] - (SSR1['CPAC'] + SSR2['CPAC'])) / k) / ((SSR1['CPAC'] + SSR2['CPAC']) / (n1 + n2 - 2 * k))
F_epac = ((SSRc['EPAC'] - (SSR1['EPAC'] + SSR2['EPAC'])) / k) / ((SSR1['EPAC'] + SSR2['EPAC']) / (n1 + n2 - 2 * k))
F_tw = ((SSRc['TW'] - (SSR1['TW'] + SSR2['TW'])) / k) / ((SSR1['TW'] + SSR2['TW']) / (n1 + n2 - 2 * k))
F_sums = ((SSRcsum - (SSR1sum + SSR2sum)) / k) / ((SSR1sum + SSR2sum) / (n1 + n2 - 2 * k))

# Critical value for F-distribution using the package
critical_value = f.ppf(1 - 0.05, k, n1 + n2 - 2 * k)  # <--- change 0.05 if we want a different alpha!

if F_wpac > critical_value:
    print(f"Reject the null hypothesis: Structural change detected in WPAC. F-statistic={F_wpac}")
else:
    print(f"Fail to reject the null hypothesis: No structural change detected in WPAC. F-statistic={F_wpac}")

if F_cpac > critical_value:
    print(f"Reject the null hypothesis: Structural change detected in CPAC. F-statistic={F_cpac}")
else:
    print(f"Fail to reject the null hypothesis: No structural change detected in CPAC. F-statistic={F_cpac}")
    
if F_epac > critical_value:
    print(f"Reject the null hypothesis: Structural change detected in EPAC. F-statistic={F_epac}")
else:
    print(f"Fail to reject the null hypothesis: No structural change detected in EPAC. F-statistic={F_epac}")

if F_tw > critical_value:
    print(f"Reject the null hypothesis: Structural change detected in TW. F-statistic={F_tw}")
else:
    print(f"Fail to reject the null hypothesis: No structural change detected in TW. F-statistic={F_tw}")

if F_sums > critical_value:
    print(f"Reject the null hypothesis: Structural change detected in general. F-statistic={F_sums}")
else:
    print(f"Fail to reject the null hypothesis: No structural change detected in general. F-statistic={F_sums}")

## Bootstrapped Chow Test

...
Lutkepohl: For the German investment/income/consumption data we have fitted a
VAR(2) model to data up to 1982.4 and we have performed a Chow test for
a break in period 1979.1. The test value is 30.5. Comparing that to 29.6,
the 10% critical value of a χ2(21) distribution, stability is rejected at the
10% level. A bootstrapped p-value based on 2000 bootstrap replications turns
out to be 0.21, however. Thus, based on the bootstrapped test, stability is not
rejected. It is typical for the test based on the asymptotic χ2-distribution that
it rejects more often in small samples than the specified nominal significance
level, even if the model is stable. This distortion is at least partly corrected
by the bootstrap.

In [ ]:
def chow_test(X, y, breakpoint):
    """
    Perform the Chow test for a structural break at a specified breakpoint.
    """
    # Convert date to index for splitting
    breakpoint_index = np.where(X.index == breakpoint)[0][0]
    
    # Split the data into two parts
    X1, X2 = X.iloc[:breakpoint_index], X.iloc[breakpoint_index:]
    y1, y2 = y.iloc[:breakpoint_index], y.iloc[breakpoint_index:]
    
    # Add constant term for the intercept
    X1 = sm.add_constant(X1)
    X2 = sm.add_constant(X2)
    X = sm.add_constant(X)
    
    # Fit the models
    model1 = sm.OLS(y1, X1).fit()
    model2 = sm.OLS(y2, X2).fit()
    full_model = sm.OLS(y, X).fit()
    
    # Calculate the Chow test statistic
    rss1 = model1.ssr
    rss2 = model2.ssr
    rss_full = full_model.ssr
    k = X.shape[1]
    chow_stat = ((rss_full - (rss1 + rss2)) / k) / ((rss1 + rss2) / (X.shape[0] - 2 * k))
    
    return chow_stat

In [ ]:
def bootstrapped_chow_test(df, breakpoint, n_bootstraps=1000):
    """
    Perform a bootstrapped Chow test for a structural break at a specified breakpoint.
    """
    X = df.drop(columns=df.columns[-1])
    y = df[df.columns[-1]]
    
    observed_chow_stat = chow_test(X, y, breakpoint)
    bootstrap_stats = []

    for _ in range(n_bootstraps):
        # Resample the data
        df_resampled = resample(df)
        X_resampled = df_resampled.drop(columns=df.columns[-1])
        y_resampled = df_resampled[df.columns[-1]]
        
        # Compute the Chow test statistic for the resampled data
        bootstrap_stat = chow_test(X_resampled, y_resampled, breakpoint)
        bootstrap_stats.append(bootstrap_stat)

    # Calculate the p-value
    bootstrap_stats = np.array(bootstrap_stats)
    p_value = np.mean(bootstrap_stats >= observed_chow_stat)

    return observed_chow_stat, p_value, bootstrap_stats

In [ ]:
np.random.seed(0)
T = 100
n = 4  # Number of variables
dates = pd.date_range('1980-01-01', periods=T, freq='ME')
data = np.cumsum(np.random.randn(T, n), axis=0)  # Random walk to simulate non-stationary data
columns = ['var1', 'var2', 'var3', 'var4']
df = pd.DataFrame(data, index=dates, columns=columns)
print(df)

    # Perform bootstrap Chow test
break_point = '1986-06-30'
observed_stat, p_value, bootstrap_stats = bootstrapped_chow_test(df, breakpoint, n_bootstraps=2000)

print(f"Observed Chow test statistic: {observed_stat}")
print(f"Bootstrapped p-value: {p_value}")

In [ ]:
observed_stat, p_value, bootstrap_stats = bootstrapped_chow_test(df_extended_data_dsdt, T_break, n_bootstraps=2000)
print(f"Observed Chow test statistic: {observed_stat}")
print(f"Bootstrapped p-value: {p_value}")

In [ ]:
observed_stat, p_value, bootstrap_stats = bootstrap_chow_test(dfY, '1986-06-30', n_bootstraps=2000, lags=1, seed=123)
print(f"Observed Chow test statistic: {observed_stat}")
print(f"Bootstrapped p-value: {p_value}")

In [ ]:
za_result = zivot_andrews(df_extended_data_dsdt['CPAC'], trim=0.05, regression='ct', autolag='t-stat')
print("Test Statistic:", za_result[0])
print("P-value:", za_result[1])
if za_result[1] < 0.05:
    print("There is statistical evidence that there is a structural breal at the identified break point.")
print("Critical Values:", za_result[2])
print("Break Point:", za_result[3])

# Comparisons

## Fit VAR Model

In [ ]:
MLS_Phi = MLS(df_extended_data_dsdt)
print(MLS_Phi[:, 1:])

In [ ]:
climate_VAR1 = VAR1(864, MLS_Phi[:, 1:], standard=True, constant=False, lower_bound_Y1=minima_climate, upper_bound_Y1=maxima_climate, seed=None)


In [ ]:
dfclimate_VAR1, dfTclimate_VAR1 = GenerateDF(climate_VAR1, start_year=1950, yearly=False, freq='ME', column_names=['WPAC','CPAC','EPAC','TW'])

In [ ]:
PlotDF(dfclimate_VAR1, model_name = 'Climate Data - Estimated with VAR(1) model')

## Fit CVAR Model

In [ ]:
CVAR_climate_estimates, _ = CVAR_ParEst(df_extended_data_dsdt, 1, 'TW')
climate_CVAR_A = CVAR_climate_estimates['A']
climate_CVAR_Phi = CVAR_climate_estimates['B']

In [ ]:
print(minima_climate)
climate_CVAR1, seed_climate_CVAR1 = CVAR1(864, climate_CVAR_A, climate_CVAR_Phi, standard=True, constant=False, lower_bound_Y1=minima_climate, upper_bound_Y1=maxima_climate, pp_alpha=0.05, seed=1)


In [ ]:
dfclimate_CVAR1, dfTclimate_CVAR1 = GenerateDF(climate_CVAR1, start_year=1950, yearly=False, freq='ME', column_names=['WPAC','CPAC','EPAC','TW'])

In [ ]:
PlotDF(dfclimate_CVAR1, model_name = 'Climate Data - Estimated with CVAR(1) model - WRONG BOUNDS IN DATA')

## Fit PCMCI Tigramite

In [ ]:
four_node_complete_history_reanalysis_results = RunPCMCI(df_extended_data_dsdt, ParCorr(), tau_max=1, tau_min=1, with_mask = True, model = 'reanalysis')
#def RunPCMCI(df, test, tau_max=1, tau_min=1, pc_alpha=0.05, alpha_level=0.01, with_mask = True, model = 'model', show_DAG=True, var_names = ['WPAC', 'CPAC', 'EPAC', 'TW']):


In [ ]:
value_matrix = four_node_complete_history_reanalysis_results['val_matrix']

In [ ]:
def ConstructPhiMatrix(val_matrix):
    "Construct a Phi Matrix for the pcmci option based on an input val_matrix"
    n, _, _ = val_matrix.shape # Or just fill in 4
    Phi = np.zeros((n,n))
    for i in range(0,n): 
        for j in range(0,n): 
            Phi[i,j] = val_matrix[j,i,1]
    return Phi

In [ ]:
climate_pcmci_phi = ConstructPhiMatrix(value_matrix)
identity_4 = np.eye(4)

In [ ]:
climate_pcmci, seed_climate_pcmci = CVAR1(864, identity_4, climate_CVAR_Phi, standard=True, constant=False, lower_bound_Y1=30, upper_bound_Y1=40, pp_alpha=0.05, seed=1)
dfclimate_pcmci, dfTclimate_pcmci = GenerateDF(climate_pcmci, start_year=1950, yearly=False, freq='ME', column_names=['WPAC','CPAC','EPAC','TW'])
PlotDF(dfclimate_pcmci, model_name = 'Climate Data - Estimated with Tigramites PCMCI - WRONG BOUNDS IN DATA')

### Compare outcome matrices

In [ ]:
print("Outcome for VAR")
print("WPAC, CPAC, EPAC, TW")
print(MLS_Phi[:, 1:])
print("")
print("Outcome for CVAR")
print("A")
print(climate_CVAR_A)
print("Phi")
print(climate_CVAR_Phi)
print("Outcome for PCMCI")
print(phi_pcmci)